<a href="https://colab.research.google.com/github/jonbaer/googlecolab/blob/master/Copy_of_mathematical_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copyright 2025 Google DeepMind Technologies Limited

All software is licensed under the Apache License, Version 2.0 (Apache 2.0);
you may not use this file except in compliance with the Apache 2.0 license.
You may obtain a copy of the Apache 2.0 license at:
https://www.apache.org/licenses/LICENSE-2.0

All other materials are licensed under the Creative Commons Attribution 4.0
International License (CC-BY). You may obtain a copy of the CC-BY license at:
https://creativecommons.org/licenses/by/4.0/legalcode

Unless required by applicable law or agreed to in writing, all software and
materials distributed here under the Apache 2.0 or CC-BY licenses are
distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND,
either express or implied. See the licenses for the specific language governing
permissions and limitations under those licenses.

This is not an official Google product.

# > **[Please read me first](https://github.com/google-deepmind/alphaevolve_results/blob/main/README.md)**

# Tensor decompositions (algorithms for matrix multiplication)

This section contains several low-rank tensor decompositions, discovered by AlphaEvolve, and the code to verify their correctness. A tensor with parameters $\langle n,m,p\rangle$ corresponds to multiplying an $n \times m$ and an $m \times p$ matrix, and the rank of the decomposition corresponds to the number of scalar multiplications needed to perform the matrix multiplication.

In [ ]:
#@title Verification function
import numpy as np

def verify_tensor_decomposition(decomposition: tuple[np.ndarray, np.ndarray, np.ndarray], n: int, m: int, p: int, rank: int):
  """Verifies the correctness of the tensor decomposition.

  Args:
    decomposition: a tuple of 3 factor matrices with the same number of columns.
      (The number of columns specifies the rank of the decomposition.) To
      construct a tensor, we take the outer product of the i-th column of the
      three factor matrices, for 1 <= i <= rank, and add up all these outer
      products.
    n: the first parameter of the matrix multiplication tensor.
    m: the second parameter of the matrix multiplication tensor.
    p: the third parameter of the matrix multiplication tensor.
    rank: the expected rank of the decomposition.

  Raises:
    AssertionError: If the decomposition does not have the correct rank, or if
    the decomposition does not construct the 3D tensor which corresponds to
    multiplying an n x m matrix by an m x p matrix.
  """
  # Check that each factor matrix has the correct shape.
  factor_matrix_1, factor_matrix_2, factor_matrix_3 = decomposition
  assert factor_matrix_1.shape == (n * m, rank), f'Expected shape of factor matrix 1 is {(n * m, rank)}. Actual shape is {factor_matrix_1.shape}.'
  assert factor_matrix_2.shape == (m * p, rank), f'Expected shape of factor matrix 1 is {(m * p, rank)}. Actual shape is {factor_matrix_2.shape}.'
  assert factor_matrix_3.shape == (p * n, rank), f'Expected shape of factor matrix 1 is {(p * n, rank)}. Actual shape is {factor_matrix_3.shape}.'

  # Form the matrix multiplication tensor <n, m, p>.
  matmul_tensor = np.zeros((n * m, m * p, p * n), dtype=np.int32)
  for i in range(n):
    for j in range(m):
      for k in range(p):
        matmul_tensor[i * m + j][j * p + k][k * n + i] = 1

  # Check that the tensor is correctly constructed.
  constructed_tensor = np.einsum('il,jl,kl -> ijk', *decomposition)
  assert np.array_equal(constructed_tensor, matmul_tensor), f'Tensor constructed by decomposition does not match the matrix multiplication tensor <{(n,m,p)}>: {constructed_tensor}.'
  print(f'Verified a decomposition of rank {rank} for matrix multiplication tensor <{n},{m},{p}>.')

  # Print the set of values used in the decomposition.
  np.set_printoptions(linewidth=100)
  print('This decomposition uses these factor entries:\n', np.array2string(np.unique(np.vstack((factor_matrix_1, factor_matrix_2, factor_matrix_3))), separator=', '))

## Rank-32 decomposition of <2,4,5> over 0.5*Z

In [ ]:
#@title Data

decomposition_245 = (np.array([[-0. ,  0.5, -0. ,  0.5,  0. ,  0. ,  0.5,  0. , -0.5, -0.5, -0. ,
        -0.5, -0. , -0. ,  0.5,  0. ,  0.5, -1. , -0.5, -0. , -0.5, -0. ,
         0.5, -0. , -0.5,  0.5, -1. ,  0. ,  0.5,  0. ,  0.5, -0. ],
       [ 0.5, -0.5,  0. ,  0.5, -0. ,  0.5, -0.5, -0. ,  0. , -0.5,  0. ,
         0. , -0.5,  0.5,  0. ,  0.5,  0. , -0. , -0. , -0.5, -0. ,  0. ,
         0. ,  0.5, -0.5,  0. , -0. ,  0. ,  0. ,  0. , -0. ,  0.5],
       [ 0. , -0. , -0.5, -0. ,  0.5, -0.5, -0. ,  0. , -0. ,  0. ,  0. ,
         0.5, -0.5, -0. ,  0.5,  0. ,  0.5, -0.5,  0.5,  0. ,  0. , -0.5,
        -0.5,  0. , -0. ,  0.5,  0.5, -0. ,  0. , -0. ,  0. ,  0. ],
       [-0.5,  0. ,  0.5, -1. ,  0.5,  0. ,  0. , -0. , -0.5,  0. , -0. ,
         0. , -0. , -0.5, -0. ,  0.5, -0. , -0.5, -0. , -0.5, -0.5,  0.5,
        -0. ,  0.5, -1. ,  0. ,  0.5, -0. , -0.5, -0. , -0.5, -0.5],
       [ 0. ,  0.5,  0. ,  0.5,  0. ,  0. , -0.5, -0. , -0.5, -0. , -0. ,
         0.5, -0. , -0. ,  0. , -0. , -0. ,  0. ,  0. , -0. , -0. , -0. ,
        -0. , -0. , -0.5, -0.5,  0. ,  0.5,  0.5, -0. ,  0. , -0. ],
       [-0.5, -0.5, -0. , -0.5, -0. , -0.5,  0.5, -0.5, -0. ,  0. ,  0.5,
         0. ,  0.5, -0.5, -0. , -0. ,  0.5, -0. , -0.5,  0.5, -0. , -0. ,
        -0. , -0.5,  0.5, -0. ,  0. ,  0.5,  0. ,  0. ,  0. , -0. ],
       [-0. ,  0. ,  0.5,  0. , -0. ,  0.5, -0. ,  0.5,  0. ,  0. ,  0.5,
        -0.5,  0.5, -0.5, -0. ,  0. , -0.5,  0.5, -0.5,  0. ,  0. , -0.5,
        -0. ,  0.5, -0. , -0.5, -0.5, -0. ,  0. , -0.5, -0. , -0. ],
       [ 0.5, -0. , -0.5, -0. , -0. , -0. ,  0. ,  0. , -0.5,  0. ,  0. ,
        -0. , -0. ,  0. , -0. , -0. , -0. , -0.5, -0. ,  0.5, -0. ,  0.5,
        -0. ,  0. , -0. , -0. ,  0.5,  0. , -0.5, -0.5,  0. , -0. ]],
      dtype=np.float32), np.array([[ 0.5, -0. , -0. ,  0. , -0. , -0.5,  0. ,  0. ,  1. , -0. , -0. ,
        -0. ,  0.5, -0.5,  0.5,  0. , -0. , -0.5, -0. ,  0.5, -0.5,  0. ,
        -0.5, -0.5,  0. , -0. ,  0.5,  0. , -1. ,  0. ,  0.5,  0. ],
       [ 0.5,  0. ,  0. ,  1. ,  0. ,  0.5,  2. ,  0. , -0. , -0. ,  0. ,
         0. , -0.5,  0.5, -0.5,  0. ,  0. ,  0.5, -0. ,  0.5,  0.5,  0. ,
         0.5,  0.5, -1. ,  0. , -0.5, -0. ,  0. , -0. , -0.5,  0. ],
       [ 0.5,  0. , -0. , -0. ,  0. , -0.5, -0. , -1. , -0. ,  0. ,  1. ,
         1. ,  0.5, -0.5, -0.5,  0. , -1. ,  0.5,  1. ,  0.5, -0.5, -0. ,
         0.5, -0.5, -0. ,  1. , -0.5, -2. , -0. , -0. ,  0.5, -0. ],
       [-0.5, -0. , -0. ,  0. ,  0. ,  0.5,  0. ,  0. , -0. ,  0. , -0. ,
        -1. , -0.5,  0.5, -0.5,  0. , -0. , -0.5,  0. , -0.5,  0.5,  0. ,
         0.5,  0.5, -0. , -1. ,  0.5, -0. ,  0. , -0. , -0.5,  0. ],
       [ 0. ,  0. ,  0. ,  1. , -0. , -1. ,  0. , -0. , -0. , -0. , -0. ,
        -1. , -1. , -0. , -0. , -1. , -1. , -0. , -1. , -0. , -1. , -0. ,
        -0. , -0. ,  1. ,  1. , -0. , -0. , -0. , -0. , -1. , -1. ],
       [ 0.5, -2. , -0. ,  1. ,  0. , -0.5,  0. ,  0. ,  1. ,  0. , -0. ,
         0. ,  0.5, -0.5,  0.5,  1. , -0. , -0.5, -0. ,  0.5,  0.5, -0. ,
        -0.5, -0.5, -1. , -0. ,  0.5, -0. , -1. , -0. , -0.5, -1. ],
       [ 0.5, -0. ,  0. ,  0. ,  0. ,  0.5,  0. , -0. , -0. , -0. ,  0. ,
        -0. , -0.5,  0.5, -0.5, -1. ,  0. ,  0.5, -0. ,  0.5, -0.5,  0. ,
         0.5,  0.5, -0. ,  0. , -0.5, -0. , -0. ,  0. ,  0.5,  1. ],
       [-0.5, -0. , -0. , -0. , -0. , -0.5,  0. ,  1. ,  0. ,  0. , -1. ,
         0. ,  0.5,  0.5,  0.5, -0. ,  0. , -0.5, -0. , -0.5,  0.5, -0. ,
        -0.5,  0.5, -0. ,  0. ,  0.5, -0. , -0. , -0. , -0.5,  0. ],
       [ 0.5,  0. , -0. , -0. , -0. ,  0.5,  0. ,  0. ,  0. ,  2. ,  0. ,
         0. , -0.5, -0.5,  0.5, -0. ,  1. ,  0.5, -1. ,  0.5, -0.5, -0. ,
        -0.5, -0.5, -0. ,  0. , -0.5, -0. , -0. ,  0. ,  0.5, -0. ],
       [ 0. , -0. , -0. , -1. ,  0. , -1. ,  0. ,  0. , -0. ,  0. , -0. ,
        -1. , -1. ,  0. , -0. ,  1. , -1. , -0. , -1. , -0. ,  1. , -0. ,
        -0. , -0. , -1. ,  1. , -0. , -0. , -0. , -0. ,  1. ,  1. ],
       [ 0. ,  0. , -0. , -0.5,  0. ,  0.5,  0. , -0. ,  1. ,  0. ,  0. ,
         0.5,  0.5,  0. ,  1. ,  0.5,  0.5, -1. ,  0.5, -0. ,  0.5,  2. ,
         1. , -0. , -0.5, -0.5, -1. ,  0. ,  1. , -0. ,  0.5,  0.5],
       [ 0. ,  0. , -0. ,  0.5,  2. , -0.5,  0. ,  0. , -0. ,  0. , -0. ,
         0.5, -0.5,  1. ,  0. ,  0.5,  0.5, -0. ,  0.5,  0. , -0.5, -0. ,
         0. , -1. ,  0.5, -0.5, -0. , -0. , -0. ,  0. , -0.5,  0.5],
       [-0. , -0. ,  0. , -0.5,  0. ,  0.5,  0. , -1. , -0. , -0. , -1. ,
        -0.5,  0.5, -0. ,  0. ,  0.5, -0.5,  0. , -0.5, -0. ,  0.5,  0. ,
        -0. ,  0. , -0.5,  0.5, -0. , -0. , -0. ,  0. ,  0.5,  0.5],
       [-0. ,  0. ,  0. ,  0.5, -0. , -0.5, -0. , -0. ,  0. ,  0. , -0. ,
         0.5, -0.5,  0. , -1. , -0.5, -0.5,  0. , -0.5, -0. , -0.5,  0. ,
        -1. , -0. ,  0.5, -0.5,  0. ,  0. ,  0. , -0. , -0.5, -0.5],
       [-1. ,  0. , -0. , -0. , -0. ,  1. , -0. , -0. ,  0. , -0. ,  0. ,
         0. , -1. ,  1. ,  1. ,  0. , -0. , -1. , -0. , -1. ,  1. ,  0. ,
        -1. ,  1. ,  0. , -0. ,  1. , -0. , -0. ,  0. , -1. , -0. ],
       [-0. , -0. ,  0. , -0.5, -0. ,  0.5, -0. ,  0. ,  1. ,  0. ,  0. ,
         0.5,  0.5,  0. , -0. ,  0.5,  0.5, -0. ,  0.5,  0. , -0.5, -0. ,
        -0. , -0. , -0.5, -0.5, -0. ,  0. ,  1. ,  0. , -0.5,  0.5],
       [-1. , -0. , -0. , -0.5,  0. , -0.5, -0. ,  0. ,  0. ,  0. , -0. ,
        -0.5, -0.5,  0. ,  0. , -0.5, -0.5, -0. , -0.5,  1. ,  0.5,  0. ,
         0. , -0. , -0.5,  0.5,  0. , -0. ,  0. ,  0. ,  0.5, -0.5],
       [ 1. , -0. , -0. ,  0.5,  0. ,  0.5,  0. ,  1. , -0. , -0. ,  1. ,
         0.5,  0.5, -1. ,  0. , -0.5,  0.5,  0. ,  0.5, -1. , -0.5,  0. ,
        -0. ,  1. ,  0.5, -0.5, -0. ,  0. ,  0. ,  2. , -0.5, -0.5],
       [ 0. ,  0. ,  2. ,  0.5,  0. , -0.5, -0. ,  0. , -0. , -0. , -0. ,
         0.5, -0.5,  0. ,  0. , -0.5, -0.5, -1. , -0.5, -0. ,  0.5, -0. ,
        -0. ,  0. ,  0.5, -0.5, -1. , -0. , -0. ,  0. ,  0.5, -0.5],
       [-1. , -0. , -0. ,  0. ,  0. ,  1. , -0. ,  0. , -0. ,  0. ,  0. ,
        -0. , -1. ,  1. , -1. , -0. ,  0. ,  1. ,  0. , -1. , -1. ,  0. ,
         1. ,  1. , -0. ,  0. , -1. , -0. ,  0. , -0. ,  1. , -0. ]],
      dtype=np.float32), np.array([[ 0. ,  0.5,  0.5,  0.5, -0. , -0. ,  0.5,  0. , -0. , -0. , -0. ,
        -0. , -0. ,  0. ,  0. ,  0. , -0. ,  0.5,  0. , -0. ,  1. , -0.5,
        -0. , -0. ,  0.5,  0. , -0.5, -0. , -0. ,  0. ,  1. ,  0. ],
       [-0. ,  0.5, -0.5, -0.5, -0. , -0. , -0.5,  0. , -1. , -0. , -0. ,
        -0. ,  0. ,  0. ,  0. ,  0. , -0. , -0.5,  0. , -0. , -1. , -0.5,
         0. , -0. , -0.5, -0. ,  0.5,  0. , -1. ,  0. , -1. ,  0. ],
       [ 0. ,  0.5, -0. ,  0.5,  1. , -0. ,  0.5, -0. ,  0. ,  0. , -0. ,
         0. ,  0. , -0. ,  0. , -1. , -0. ,  0. ,  0. , -0. ,  0. ,  0. ,
         0. ,  0. ,  0.5,  0. ,  0. , -0. ,  0. ,  0. , -0. ,  1. ],
       [-1. ,  0.5,  0. ,  0.5, -0. ,  0. , -0.5,  0. ,  0. , -0. ,  0. ,
        -0. , -0. , -1. ,  0. , -1. , -0. ,  0. ,  0. ,  1. ,  0. ,  0. ,
         0. , -1. ,  0.5, -0. ,  0. , -0. , -0. , -1. ,  0. ,  1. ],
       [-0. , -0. ,  0. , -0. ,  1. , -1. ,  0. , -1. , -0. , -1. , -1. ,
        -0. , -1. ,  1. ,  0. ,  0. , -1. , -0. , -1. , -0. ,  0. ,  0. ,
         0. ,  1. , -0. , -0. ,  0. , -0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , -0. , -0. ,  0. ,  0. , -1. , -0. ,  0. , -1. ,
         0. ,  0. , -0. , -0. ,  0. ,  0. ,  0. ,  0. , -0. , -0. ,  0. ,
         0. , -0. ,  0. , -0. ,  0. , -1. ,  0. , -1. ,  0. ,  0. ],
       [-0. , -0. ,  0.5, -0. ,  0. ,  0. ,  0. ,  0. , -0. , -1. , -0. ,
         0. , -0. ,  0. , -1. ,  0. , -0. ,  0.5, -0. , -0. , -0. , -0.5,
         1. , -0. , -0. ,  0. , -0.5,  0. , -0. , -0. ,  0. ,  0. ],
       [ 0. , -0. , -0.5,  0. ,  0. ,  0. , -0. , -0. , -0. ,  0. ,  0. ,
        -1. ,  0. , -0. , -1. ,  0. ,  1. ,  0.5,  1. ,  0. , -0. , -0.5,
         1. ,  0. ,  0. ,  1. , -0.5, -1. , -0. ,  0. ,  0. , -0. ],
       [ 0. ,  0. , -0. , -0. , -0. , -0.5, -0. , -0.5, -0. ,  0. ,  0.5,
        -0. ,  0.5, -0.5,  0.5,  0.5, -0.5,  0. ,  0.5, -0. ,  0.5,  0. ,
         0.5,  0.5, -0. ,  0. ,  0. , -0. ,  0. , -0. , -0.5,  0.5],
       [-0.5, -0. ,  0. ,  0.5,  0. ,  0. ,  0. , -0.5, -0.5, -0. ,  0.5,
        -0.5, -0. , -0.5,  0.5,  0.5, -0.5, -0.5,  0.5, -0.5, -0.5, -0. ,
         0.5,  0.5, -0.5, -0.5, -0.5,  0. ,  0.5,  0. ,  0.5,  0.5]],
      dtype=np.float32))

In [ ]:
#@title Verification
n = 2
m = 4
p = 5
rank = 32

verify_tensor_decomposition(decomposition_245, n, m, p, rank)

## Rank-45 decomposition of <2,4,7> over Z

In [ ]:
#@title Data

decomposition_247 = (np.array([[ 0., -1., -1.,  0.,  1.,  1.,  1., -1., -1.,  0., -0., -0.,  0.,
         0., -0.,  1., -0., -1.,  0.,  0., -0., -0., -0., -1., -0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  1., -1., -0., -0., -0., -0.,  1., -0.,
        -0.,  0., -1., -1., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,  1., -0.,  0., -1.,
        -0., -0., -0., -0., -0.,  1., -0.,  1., -0.,  1.,  1., -1., -1.,
         0., -1., -0.,  0., -0., -1., -0., -0.,  1., -1., -0., -0.,  0.,
         0., -0.,  1.,  0., -0.,  1.],
       [ 0.,  0., -0.,  0.,  1.,  1., -0., -1.,  0., -1.,  0., -0., -0.,
         1., -0.,  1.,  1.,  0., -0.,  0.,  0., -1., -0., -1.,  0.,  0.,
        -0.,  0., -0.,  0., -1.,  0., -1., -0.,  0.,  0.,  1.,  1., -0.,
        -0.,  0., -1., -1., -1., -0.],
       [-0.,  1., -0.,  0., -1., -0., -1.,  0.,  1.,  0.,  0.,  0.,  0.,
        -0., -1., -1.,  0.,  1., -0.,  1.,  0.,  0.,  1.,  1.,  0.,  1.,
        -0., -0., -0.,  0.,  1., -0., -0.,  0., -0., -0., -0., -1.,  0.,
         0.,  0., -0.,  0.,  0., -0.],
       [-1., -1., -1., -0.,  1.,  1.,  0., -1., -0., -0., -0., -0.,  0.,
        -0., -0.,  1.,  1., -1., -0.,  0.,  0., -0., -0., -0., -0., -0.,
         0.,  0.,  1., -1., -0., -0.,  0., -0.,  1., -0., -0., -0.,  0.,
         1.,  0., -1., -0., -0.,  0.],
       [ 1.,  0., -0.,  0., -1., -0., -0.,  0., -1.,  1.,  0.,  1.,  0.,
        -0., -0., -0., -1., -0., -0., -0.,  1., -0.,  1., -0.,  0., -0.,
        -1., -1., -1., -0.,  0., -0.,  0., -0.,  0., -1., -0., -0., -0.,
        -1., -1.,  1., -1.,  0.,  1.],
       [-1.,  0.,  0., -1.,  1., -0.,  0., -1.,  1., -1., -0., -0., -0.,
         0.,  0.,  1.,  1.,  0., -0., -0., -1., -1., -0.,  0.,  0., -0.,
         0., -0.,  1., -0.,  0.,  0.,  0.,  1., -0.,  0., -0., -0., -0.,
         0.,  1., -1.,  0., -0., -0.],
       [ 0., -0., -0.,  0.,  0., -0., -0., -0.,  0., -0.,  1., -1.,  0.,
         1., -0., -1., -0.,  1.,  1.,  1., -0., -0.,  1.,  0.,  0.,  0.,
        -1., -0., -1.,  0.,  0.,  0., -0., -1., -0., -1., -0.,  0., -1.,
         0.,  0.,  0., -1., -0., -0.]], dtype=np.float32), np.array([[-0.,  0.,  0.,  1.,  1.,  1., -0., -1.,  0., -0., -0., -1., -0.,
        -1.,  0.,  0.,  1.,  0.,  1., -0., -0.,  1.,  0., -0.,  0., -1.,
         0., -0.,  0., -0.,  0., -1., -0., -1.,  1., -0.,  0.,  1., -0.,
         1., -0., -0., -0., -1.,  0.],
       [-0.,  1.,  1.,  1., -0.,  1., -0., -0., -1., -0.,  0.,  0.,  0.,
         0., -1.,  0.,  0., -0.,  0.,  0., -1., -0.,  0.,  1.,  1., -0.,
         0., -1., -1., -0., -0., -0.,  0.,  0.,  1.,  1., -1.,  0., -1.,
         1.,  0., -0., -1.,  0., -1.],
       [ 0.,  0.,  1., -0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  1.,
        -0., -0., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,  0.,
        -0.,  0., -0., -0.,  0.,  1.,  0., -0., -1., -0., -0.,  0., -0.,
        -0.,  0., -0., -0.,  0., -0.],
       [-0.,  1.,  1., -0., -1., -0.,  0.,  0., -0.,  0.,  0.,  1., -0.,
         1.,  0., -1., -1., -1., -1., -0.,  0., -1.,  0., -0.,  0.,  1.,
         0.,  0., -0., -1., -1.,  1.,  0.,  0., -1.,  0., -0.,  0., -1.,
        -1., -0., -0.,  0.,  1., -0.],
       [-0., -1., -0., -0.,  0.,  0.,  1.,  0.,  0.,  0., -0.,  0., -0.,
         0.,  1., -0., -0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  0.,
         0.,  0.,  0.,  1., -0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,
        -0.,  0., -0., -0., -0.,  0.],
       [ 0., -0.,  0., -0.,  0.,  1.,  0.,  0.,  0., -0., -0., -0., -0.,
         0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,
        -0., -0., -0., -1., -0., -0.,  1., -0., -0., -0., -1., -0., -0.,
         0.,  0., -0.,  0., -0.,  0.],
       [ 0., -0., -1., -0., -0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,
         0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,
         0., -0., -0., -1., -0., -0., -0., -0., -0., -0.,  0.,  0., -0.,
        -0.,  0., -0.,  0., -0., -0.],
       [ 0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0., -1., -1.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0., -0., -0.,  0., -0., -0., -0.,
         0.,  1., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0.,
         0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0.,  1., -0.,
        -0., -0., -0.,  0.,  0., -0.,  0., -0., -0., -0.,  0.,  0., -0.,
        -0.,  0.,  0., -0.,  0.,  1.],
       [-0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0., -0., -0.,  1.,
         0., -0., -0., -0., -0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,
         0., -1., -0., -0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,
        -0., -0.,  0.,  0., -0., -0.],
       [-0.,  1.,  1.,  0., -1., -0., -0.,  0.,  0.,  0., -0.,  1., -0.,
         1., -0., -1.,  0., -1., -1.,  0., -0., -0.,  0.,  0., -0.,  1.,
         0., -0.,  0., -1., -1., -0.,  0., -0., -1., -0.,  0., -0., -1.,
        -1., -0., -1.,  0.,  1., -0.],
       [ 0., -0.,  0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,  0.,
         0., -0., -0., -0., -0.,  0., -0., -0.,  0.,  0., -0., -1., -0.,
        -1., -0.,  0., -0.,  0.,  0., -0., -0.,  0., -1.,  0., -0.,  1.,
         0., -0., -0., -0., -0., -0.],
       [ 0.,  0.,  0.,  1., -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,
         0., -0., -0., -0., -0.,  0., -0., -1., -0.,  0., -0.,  1.,  0.,
         0.,  0., -0., -0., -0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,
         0.,  1., -0., -0.,  0., -0.],
       [-0.,  0., -0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0.,
        -0.,  0.,  0., -0., -0., -0.,  0., -0.,  0.,  0.,  0., -0., -0.,
        -0.,  1., -0., -1., -0.,  0.,  0.,  0., -1., -0., -0.,  0., -0.,
        -1.,  0.,  0.,  0.,  0., -0.],
       [ 0.,  0.,  0., -0., -0., -0., -0., -0.,  0.,  0., -1., -0., -0.,
         1.,  0., -0., -0.,  0.,  0., -0., -0., -1.,  0., -0., -0., -0.,
         0.,  0.,  0., -0.,  0., -0., -0.,  1., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0.,  0.],
       [-0., -0.,  0., -0.,  0.,  0., -0., -0., -0.,  1., -0., -0., -0.,
        -0., -0., -0.,  0., -0., -0.,  0.,  1.,  0., -0., -0., -0.,  0.,
        -0., -0., -0., -0.,  0., -0.,  0.,  0., -0.,  0.,  1., -0.,  0.,
        -0.,  0.,  0., -0.,  0.,  1.],
       [-1., -0., -1., -1., -0., -1., -0.,  1., -0.,  0.,  0., -0.,  0.,
         0.,  0.,  0., -1.,  0., -0.,  0., -0., -0., -0., -0., -0.,  0.,
        -0.,  0., -0., -0.,  0., -0.,  0., -0., -0.,  0., -0.,  0., -0.,
        -1., -0.,  0.,  0.,  1., -0.],
       [ 0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.,
        -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  1., -0., -0., -0., -0.,
         0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0., -0.,  0., -0.,
        -0., -0.,  0., -0.,  1.,  0.],
       [ 0., -0., -1., -1.,  0., -1., -1.,  1.,  1., -1.,  0., -0., -0.,
         0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -1., -1., -0.,
        -1.,  1., -0., -1., -0.,  0.,  0., -0., -1., -1.,  0.,  0.,  1.,
        -1.,  0.,  0., -0., -0.,  0.],
       [-0.,  0., -0.,  1.,  0.,  0., -0., -0., -0.,  0., -0.,  0., -0.,
         0.,  0., -0., -0., -0., -0.,  0., -0., -0., -0., -0.,  0., -0.,
        -0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  1.,  0.,  0.,
        -0., -0., -0.,  0., -0.,  0.],
       [ 0.,  0.,  1.,  1.,  0.,  1., -0., -1., -0.,  0.,  0., -0.,  0.,
        -0.,  0., -0., -0., -0., -0., -0., -0.,  0.,  0., -0., -0.,  0.,
        -0.,  0., -0.,  0.,  0., -0.,  0., -0., -0.,  0., -0., -0.,  0.,
        -0., -0., -0.,  0., -0.,  0.],
       [ 0.,  0., -0., -0., -0.,  0.,  0., -0., -0.,  0., -1., -0., -0.,
        -0., -0.,  0.,  0.,  0.,  0., -1., -0.,  0., -0., -0., -0.,  0.,
        -0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,
        -0., -0., -0.,  0., -0.,  0.],
       [-0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,
        -0., -1.,  0., -0., -0., -0., -0., -0.,  0., -1., -0.,  0.,  0.,
         0., -0., -0., -0.,  0., -0.,  0.,  0.,  0.,  1., -0., -0.,  0.,
         0., -0., -0.,  0., -0., -1.],
       [ 0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0., -0.,  0.,  1.,  1., -0., -0., -0., -0., -0., -1.,
         0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0.,
        -0., -0.,  0., -0.,  0.,  0.],
       [ 0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0.,
         1.,  0.,  0., -0.,  0., -0.,  1.,  0., -0., -0.,  0., -0.,  0.,
         0., -0.,  0.,  0., -1.,  0., -0.,  0., -0., -0., -0.,  0., -0.,
        -0.,  0., -0.,  0.,  1.,  0.],
       [-0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0.,
         0.,  1.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0.,
        -0., -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0., -1.,
        -0.,  0.,  0.,  0.,  0., -0.],
       [-0., -1., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,
        -0., -0., -0.,  0.,  1.,  0.,  0.,  1., -0., -1., -1., -1.,  0.,
         0.,  1.,  0., -1.,  0., -0.,  1., -0., -1., -0.,  0., -0.,  1.,
        -1., -1.,  0.,  1., -0., -0.],
       [ 0., -1., -1., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,
         0.,  0., -0.,  0.,  1., -0.,  0.,  0.,  0., -0.,  0.,  0., -0.,
         0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0., -0., -0., -0.,  1.,
        -0.,  0., -0.,  0., -0., -0.]], dtype=np.float32), np.array([[ 0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  1., -0.,  0.,
         1., -0., -0.,  0.,  0.,  1., -1.,  0.,  0., -0., -0., -0., -1.,
        -0.,  0.,  0., -0., -1., -0.,  0., -0., -0.,  0., -0.,  1.,  0.,
        -0., -0., -0.,  0.,  0.,  0.],
       [-0.,  0., -0., -0.,  1.,  0., -0.,  0., -0.,  0., -1., -1.,  0.,
         0.,  0., -1.,  0., -0., -0., -0., -0.,  0.,  0.,  0., -0.,  0.,
        -0.,  0.,  0.,  0., -0.,  0., -0.,  1.,  0., -0.,  0., -1.,  0.,
        -0., -0.,  0., -0.,  0., -0.],
       [ 0., -0., -0., -0., -0., -0.,  0.,  0.,  1., -1.,  0., -0., -0.,
        -0., -0., -0.,  0.,  0., -0., -0.,  1.,  0., -1.,  1., -1.,  0.,
        -1.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,  1.,  0., -0.,  0.,
         0., -1.,  0.,  1., -0.,  0.],
       [ 0., -0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,  0., -0.,  0.,
         0., -0.,  0., -0., -0.,  0.,  0., -1.,  0.,  0.,  0.,  1.,  0.,
         1.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0., -1., -0., -0., -0.,
        -0.,  1.,  0., -0., -0.,  1.],
       [-1.,  0., -0.,  0.,  1.,  0.,  0.,  0.,  0.,  0., -0., -0., -1.,
         0., -0.,  0., -1., -0., -0.,  0., -0.,  0.,  0.,  0.,  0., -1.,
        -0., -0.,  0.,  0.,  0.,  1., -0.,  0.,  0.,  0., -0.,  0.,  0.,
         0., -0.,  1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0., -1.,  1.,
        -0., -0., -0.,  0., -0.,  1.,  0.,  0.,  0.,  0., -0., -0.,  0.,
         0.,  1., -0.,  0., -0., -0.,  0.,  0., -1., -0.,  0., -0.,  0.,
         1., -0., -0.,  0., -0., -0.],
       [ 1., -0., -0., -0., -1.,  0., -0., -0.,  0., -0.,  0.,  0., -0.,
         0.,  0., -0.,  1.,  0.,  0., -0., -0., -0., -0., -0., -0., -0.,
        -0.,  0.,  0., -0., -1., -0., -0., -0.,  0., -0.,  0.,  1.,  0.,
        -0.,  0., -1.,  0., -1., -0.],
       [-1.,  0., -0., -0.,  1.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0.,
         1., -0., -1., -1., -0., -0., -0.,  0., -1.,  0.,  0.,  0.,  0.,
        -0.,  0., -0.,  0.,  0., -0., -0.,  1., -0.,  0., -0., -1.,  0.,
        -0., -0.,  0.,  0.,  1., -0.],
       [-0.,  0.,  0., -0.,  0.,  0.,  1.,  0., -0.,  0., -0., -0.,  0.,
         0., -1.,  0., -0., -0.,  0.,  0.,  0.,  0., -1.,  1.,  0.,  0.,
        -1.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,  1., -0., -0.,  0.,
        -0., -0.,  0.,  1.,  0., -0.],
       [ 0.,  1., -0.,  0., -0.,  0., -1., -0.,  1., -0.,  0.,  0., -0.,
        -0., -0., -0.,  0.,  1.,  0., -0.,  0., -0., -0., -0.,  0., -0.,
         1.,  0., -1.,  0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,  1.,
        -0.,  0., -0.,  0.,  0., -0.],
       [ 0., -0., -0.,  0.,  0.,  0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0., -0., -0., -1.,  0., -0., -1., -0., -0.,
         0.,  0., -0., -0., -0.,  0., -1.,  0.,  0., -0.,  1.,  0.,  0.,
         0.,  1., -0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -1.,  0.,  1., -0., -1.,  0., -0.,  0.,  0.,  0.,
         0., -0.,  0., -0., -0., -0., -0., -0., -0.,  0.,  0., -0., -0.,
        -0.,  0.,  1., -0.,  0.,  0.,  1., -0.,  0., -0.,  0.,  0., -0.,
         0., -1., -0., -1., -0.,  0.],
       [-1., -1.,  1.,  0.,  1.,  1.,  1.,  0.,  0., -0.,  0., -0., -0.,
         0.,  0.,  0., -1., -0., -0.,  0.,  0.,  0., -0.,  1., -0., -0.,
        -0., -0.,  0., -1.,  0.,  1.,  1.,  0., -1., -0.,  0., -1., -0.,
         0.,  0.,  1., -0.,  0.,  0.],
       [ 1.,  1., -0., -0., -1., -1., -1.,  1.,  1., -0., -0.,  0.,  0.,
        -0., -0.,  1.,  0.,  1., -0.,  0., -0., -0., -0., -0.,  0., -0.,
         0., -0., -1.,  1.,  0.,  0., -1., -0., -0., -0.,  0.,  1., -0.,
         1., -0., -0.,  1., -0., -0.]], dtype=np.float32))

In [ ]:
#@title Verification
n = 2
m = 4
p = 7
rank = 45

verify_tensor_decomposition(decomposition_247, n, m, p, rank)

## Rank-51 decomposition of <2,4,8> over Z


In [ ]:
#@title Data

decomposition_248 = (np.array([[-1., -0., -0.,  1.,  0.,  0., -0.,  1.,  0., -0.,  0.,  0., -0.,
        -1.,  0.,  0.,  1.,  0.,  0.,  1.,  1., -0., -0., -1., -0., -0.,
         0.,  0.,  0.,  1., -0.,  0.,  1., -0.,  0., -0.,  0., -0., -0.,
        -1.,  1., -0., -0., -0., -0.,  0.,  0.,  1.,  0.,  1., -0.],
       [-0.,  0.,  0., -0.,  0., -0.,  0.,  0., -1., -0., -0.,  0.,  0.,
         1.,  0., -0.,  0., -0.,  0., -1.,  0.,  1., -1.,  1.,  0., -0.,
        -0.,  0.,  0.,  0., -1.,  1.,  1., -0.,  0., -0., -0.,  1.,  1.,
         1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0., -0., -1.],
       [ 0.,  0., -0.,  0., -0.,  0., -0., -1., -1.,  0., -0.,  1., -1.,
         0.,  0., -0., -1.,  0., -0., -0., -1., -0., -1.,  0.,  0., -0.,
        -0., -0., -0., -1.,  1.,  0., -0., -1.,  1., -1., -0., -0., -1.,
         0., -1., -0., -1., -0., -0.,  0.,  0.,  0., -0.,  0.,  0.],
       [-0., -0., -1.,  0.,  0., -0.,  0., -0.,  1., -1.,  0.,  0.,  1.,
        -0., -0.,  0.,  0.,  0.,  1.,  0.,  0., -0., -0.,  0.,  1., -0.,
        -0.,  0.,  1.,  0.,  0., -1.,  0., -0.,  0., -0.,  1.,  0., -1.,
        -0., -0., -1., -0., -1.,  0.,  0.,  1., -1., -0., -1., -0.],
       [-1.,  0., -0.,  1., -0., -1.,  0., -0., -0., -0., -1., -0., -0.,
         0., -1.,  0.,  1., -1., -0., -0., -0., -0., -0.,  0.,  1.,  1.,
         0., -0.,  1., -0., -0.,  0., -0., -0.,  0., -1.,  0.,  0., -0.,
        -1., -0.,  0., -0., -0., -0., -1., -0., -0.,  0., -0.,  1.],
       [-0., -1., -1., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,
        -0., -0.,  1.,  0.,  0., -0., -1., -1.,  0., -1.,  1., -0.,  1.,
         1., -1.,  0.,  0., -1.,  1., -0.,  0.,  0., -0.,  1., -0., -0.,
         1.,  0.,  0., -1., -0.,  1., -0., -0.,  0., -1.,  0., -0.],
       [-0., -1.,  1.,  0., -0.,  0.,  1.,  0.,  0.,  0.,  1., -0., -1.,
         0., -0.,  0., -1., -0., -0.,  1., -1., -1., -1.,  0.,  0., -0.,
         1.,  1.,  0.,  0.,  1.,  0.,  0., -1.,  1.,  0.,  1.,  0.,  0.,
         0., -1.,  0., -0., -0., -0., -0.,  0., -0.,  1.,  0.,  0.],
       [-0.,  1.,  0.,  0., -1.,  1., -1., -0., -0., -1.,  0.,  0.,  1.,
         0.,  1., -1.,  0., -0., -0.,  0., -0.,  1., -0., -1., -0.,  0.,
         0.,  0., -0.,  0.,  0., -1., -0.,  1.,  0., -0., -0., -0.,  0.,
        -0.,  1.,  0.,  1., -1.,  0., -0., -0.,  0.,  1.,  0.,  0.]],
      dtype=np.float32), np.array([[ 0.,  0.,  0.,  1.,  0., -0., -0., -0., -0.,  0.,  0., -0.,  0.,
        -0.,  0., -0.,  0., -1., -0.,  0., -0., -0., -0., -0.,  0., -0.,
         0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0.,
        -0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0.,  0.],
       [-1., -0.,  0.,  0., -0., -0., -0., -1.,  0., -0., -0., -1.,  0.,
        -0.,  0., -0.,  0., -0., -0.,  0.,  0.,  0., -0., -0., -0.,  0.,
         0., -0., -0.,  0., -0., -0.,  0., -0., -0.,  1.,  0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0.,  0., -0.,  0.],
       [-0., -0., -1.,  0., -0., -0., -0., -0.,  0., -0., -0., -0., -1.,
        -0., -1.,  0.,  0.,  0.,  1.,  0.,  0., -0.,  0.,  1.,  0.,  0.,
         0.,  1.,  0., -0., -0., -1.,  0.,  1.,  1.,  0.,  0., -0.,  0.,
         1., -0., -0.,  0., -0., -0.,  0.,  1.,  0., -1.,  0.,  0.],
       [-0., -0.,  0.,  0.,  1.,  1.,  0., -0.,  0., -0., -0., -0.,  0.,
        -0.,  0., -0., -1., -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0.,
         0., -0.,  0.,  0., -0.,  0.,  0.,  1.,  1.,  0.,  0., -0.,  0.,
         0.,  1.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0.],
       [-0., -0., -0., -1., -0., -0., -0., -0.,  0., -0., -0., -0.,  0.,
        -0., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,
         0.,  0., -0.,  0., -0., -0., -1., -0., -0.,  0.,  0.,  1.,  0.,
         0., -0., -0.,  0.,  0., -0.,  0.,  0., -0., -0., -0., -1.],
       [ 0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0., -0., -1., -0., -0.,  0.,  0.,  0., -0., -1.,  0.,
         0., -0.,  0.,  0.,  0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,
        -0., -0.,  1., -0., -0.,  0., -0.,  0.,  0.,  0., -1.,  0.],
       [-1.,  0., -0., -0., -0., -0.,  0.,  0., -0., -0.,  0.,  0., -0.,
        -0., -0.,  0.,  0.,  0.,  1., -0., -0.,  0., -0.,  0.,  0., -0.,
        -0.,  0.,  1., -0.,  0., -0.,  0.,  0.,  0., -0., -0., -0., -0.,
         0., -0., -0., -0., -0.,  0., -0., -0.,  1., -0.,  0.,  0.],
       [-1.,  0., -0.,  0., -0., -0., -0.,  0., -0., -0.,  0.,  0., -0.,
         0., -0., -0.,  0.,  0.,  0., -0., -0., -0., -0.,  0.,  0.,  0.,
         0.,  0., -0., -0., -0., -0.,  0.,  0.,  0., -0.,  0., -0., -0.,
         0., -0., -0., -0., -0., -0.,  1.,  0., -0., -0.,  0.,  0.],
       [ 0., -0.,  0.,  0., -0.,  0., -0., -0.,  0., -0.,  0., -0.,  0.,
         0., -0.,  0.,  0.,  1., -0., -0.,  0.,  0., -0.,  0., -0., -1.,
        -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,
         0.,  0., -0., -0.,  0.,  1., -0., -0., -0.,  0., -0.,  1.],
       [ 0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -1.,  1.,
         0., -0.,  0.,  0., -0., -0., -0., -0.,  0., -0.,  0., -0.,  0.,
        -0.,  0., -0., -0., -0.,  1.,  0., -1., -1.,  0., -0.,  0.,  1.,
         0., -0., -0.,  1.,  0.,  0., -0., -1., -0., -0., -0., -0.],
       [ 0., -0., -1.,  0., -0., -0.,  0.,  0.,  0., -0., -0., -0., -1.,
        -0.,  0., -1.,  0.,  0.,  1.,  0.,  0.,  0.,  0., -0.,  0.,  0.,
         0.,  1.,  0., -0., -0., -1., -0.,  1.,  1., -0.,  0.,  0.,  0.,
        -0., -0., -0.,  0., -0.,  0.,  0.,  1., -0., -1.,  0., -0.],
       [ 0.,  1., -0., -0., -0., -0., -1.,  0.,  0.,  0.,  0., -0.,  0.,
        -0., -0., -0., -0.,  0.,  0., -0.,  0., -1.,  0.,  0.,  0., -0.,
         0.,  0., -0., -0.,  0.,  0., -0., -0., -0., -0.,  0.,  0.,  0.,
         0., -0., -0.,  0.,  0., -1.,  0., -0., -0., -0.,  0., -0.],
       [ 0., -0., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -0., -0.,  0.,
        -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0.,
        -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0., -0., -0., -1.,  0.,
        -0., -0.,  0.,  0.,  0., -1.,  0., -0., -0., -0.,  0.,  0.],
       [ 0., -1.,  0.,  0., -1., -1.,  1.,  0., -0., -0.,  0.,  0., -0.,
         0., -0., -0., -0.,  1.,  0.,  0.,  1.,  1.,  1., -0.,  1.,  0.,
        -0., -0.,  0., -0.,  0., -0., -1., -1., -1., -0., -0.,  0., -0.,
         0., -1., -1., -0., -0.,  1., -0.,  0.,  0.,  0.,  1., -0.],
       [-0.,  0., -0.,  0.,  0.,  0.,  0.,  1., -0.,  0., -1.,  0.,  1.,
         0.,  1., -1., -0., -0.,  1.,  1., -0.,  0., -0., -1., -0., -0.,
        -0., -1.,  1.,  0.,  0.,  1.,  0., -1., -1., -1., -0., -0.,  1.,
         0.,  0.,  0.,  1., -0.,  0., -0., -1.,  1., -0., -0.,  0.],
       [-0.,  0., -0.,  0.,  0.,  0.,  0.,  1., -0.,  0., -1.,  0.,  1.,
        -1., -0.,  0., -0., -0.,  0.,  1., -0., -0., -0.,  0., -0., -0.,
        -0., -1., -0.,  0.,  0.,  1.,  0., -1., -1., -1., -0.,  0.,  1.,
         0.,  0.,  0.,  1.,  0.,  0.,  1., -1., -0., -0., -0.,  0.],
       [ 0., -1.,  0., -0., -1.,  0., -0., -0.,  1., -0.,  0., -0., -0.,
        -0.,  0.,  0., -0., -1.,  0.,  0.,  1.,  1., -0., -0.,  0.,  1.,
        -0., -0.,  0., -1., -0., -0., -1., -1., -1., -0.,  1.,  0.,  0.,
         0.,  0., -1., -0.,  0., -0., -0., -0.,  0.,  0.,  0., -1.],
       [ 0., -0., -0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0., -1., -0.,
        -0., -0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0., -0., -0.,
         0.,  0., -0.,  0.,  0., -0.,  0.,  0., -1.,  0.,  0., -0., -0.,
        -0.,  0., -0., -0.,  0., -0., -0.,  0., -0., -0., -0.,  0.],
       [ 0., -0.,  0., -0., -0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,
         0.,  0., -1., -0., -0., -0., -0., -0.,  0., -0., -0., -0., -0.,
         0., -0., -0., -0.,  0.,  0.,  0., -0.,  1.,  0.,  0., -0., -0.,
        -0.,  0., -0.,  1., -0., -0.,  0., -0., -0., -1., -0.,  0.],
       [ 0., -0.,  0., -0.,  1., -0.,  1.,  0.,  0., -0., -0., -0.,  0.,
        -0.,  0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,
         0., -0.,  0., -0., -0.,  0., -0.,  1.,  1., -0.,  0., -0.,  0.,
        -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0., -0.],
       [-0.,  1., -0.,  0.,  1., -0.,  0., -0., -1.,  0., -0., -0.,  0.,
        -0., -0., -0.,  0., -0., -0., -0.,  0., -1.,  0.,  0.,  0.,  0.,
        -1.,  0., -0., -0.,  0.,  0., -0.,  1.,  1.,  0., -1.,  1.,  0.,
        -0., -0.,  1.,  0.,  0.,  0.,  0.,  0., -0., -0., -0., -0.],
       [-0., -0.,  0.,  0., -0.,  1., -1., -0., -0.,  0., -0.,  0.,  0.,
         0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0., -0., -1., -0.,
         0.,  0.,  0., -1., -0.,  0., -0., -0.,  0., -0., -0., -0.,  0.,
         0.,  1.,  1., -0.,  0.,  0., -0.,  0.,  0.,  0., -1.,  0.],
       [-0.,  0.,  1.,  0.,  0., -0.,  0.,  1., -0.,  0., -1.,  0.,  1.,
         0.,  1., -1., -0.,  0., -0.,  1., -0., -0., -0., -1., -0., -0.,
        -0., -1.,  1.,  0., -1.,  1.,  0., -1., -1., -1., -0., -0., -0.,
         0.,  0.,  0.,  1., -0.,  0.,  0., -1.,  1., -0.,  0.,  0.],
       [ 0.,  0., -0.,  0., -0., -0.,  0., -0., -0., -0., -1.,  0., -0.,
         0., -0., -0., -0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0., -0.,
        -0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -1., -1., -0., -0., -0.,
        -0., -0., -0., -0.,  0., -0.,  1.,  0., -0., -0.,  0.,  0.],
       [-0.,  0.,  0., -0.,  0.,  1.,  0.,  0., -0., -0., -0., -0., -0.,
        -0.,  0., -0.,  0., -1., -0.,  0., -0., -0.,  0., -0., -1., -0.,
        -0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0.,
         0., -0.,  0.,  0., -1.,  0., -0., -0.,  0.,  0.,  0.,  0.],
       [ 0., -0., -0., -0., -0.,  0.,  0.,  0., -0., -0., -0.,  0.,  1.,
        -0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,
         0.,  0.,  0., -0.,  0., -0., -0., -1., -1., -0.,  0., -0., -0.,
        -0.,  0., -0., -0.,  0.,  0., -0., -1.,  0., -0., -0.,  0.],
       [ 0., -0., -1.,  0., -0., -0., -0.,  0.,  0., -0., -0., -0., -1.,
        -0.,  0.,  0., -0.,  0.,  1.,  0.,  0., -0.,  0., -0.,  0.,  0.,
         0.,  1.,  0., -0., -0.,  0., -0.,  1.,  1., -0.,  0., -0.,  0.,
        -0., -0., -0.,  0.,  0., -0.,  0.,  1.,  0., -1.,  0., -0.],
       [-0., -0., -0., -0.,  1., -0.,  0.,  0.,  0.,  0., -0.,  0., -1.,
         0., -0., -0.,  0., -0., -0.,  0.,  0.,  0., -0., -0., -0.,  0.,
         0.,  0., -0., -0., -0.,  0., -0.,  1.,  1., -0.,  0., -0.,  0.,
         0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0., -0.],
       [-0.,  1., -0.,  0.,  1., -0.,  0.,  0.,  0., -0., -0.,  0., -1.,
        -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0.,
        -1.,  0., -0., -0., -0.,  0.,  0.,  1.,  1., -0., -1., -0., -0.,
        -0., -0.,  1.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,  0.],
       [-0., -0.,  0.,  0.,  0.,  1.,  0., -0.,  0.,  0., -0., -0., -0.,
        -0.,  0., -0.,  0., -1., -0.,  0.,  0.,  0.,  0., -0., -1.,  0.,
         0., -0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0., -0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.],
       [ 0., -0., -0.,  0., -0., -0.,  0., -0., -0.,  1.,  0., -0., -0.,
        -0.,  0.,  0., -0.,  0.,  1., -0.,  0.,  0., -0.,  0.,  0.,  0.,
         0.,  0., -0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,
        -0.,  0., -0.,  0.,  0.,  0., -0., -0., -0., -0., -0., -0.],
       [ 0., -0., -0., -0., -0.,  0., -0.,  0., -0.,  1.,  0.,  0., -0.,
         0., -1., -0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,  0., -0.,
         0.,  0., -1., -0., -0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0.,
        -0., -0., -0.,  0., -0., -0.,  1.,  0.,  0., -0., -0., -0.]],
      dtype=np.float32), np.array([[-0., -0.,  0.,  1., -0., -0.,  0., -0., -0.,  0.,  0., -0.,  0.,
         0., -0.,  0.,  0., -1.,  0.,  0., -1., -0.,  1., -0., -1., -0.,
         1.,  0.,  0.,  0., -0., -0., -1.,  0., -0.,  0., -1.,  0.,  0.,
         0.,  0.,  1., -0.,  0.,  0., -0.,  0.,  0., -0., -0., -1.],
       [ 0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,  0.,
        -0.,  0., -0., -0.,  1., -0.,  0., -0., -0.,  0., -0.,  1., -1.,
        -1., -0.,  0., -0., -0., -0., -0., -0., -0., -0.,  1., -0., -0.,
         0., -0., -1., -0.,  1.,  0.,  0., -0.,  0., -0.,  0.,  0.],
       [ 0.,  0.,  1., -0., -0., -0.,  0., -1.,  0., -0.,  0., -1.,  0.,
         0.,  0., -0.,  0.,  0., -0.,  1., -0.,  0.,  0.,  0.,  0., -0.,
         0.,  1., -0., -0.,  1.,  0.,  0.,  0., -0.,  0., -0.,  0.,  1.,
        -0., -0., -0.,  0.,  0., -0.,  0., -1.,  0., -0., -0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  0., -0., -0.,  0., -0.,  1.,  1.,  0.,
        -0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0., -0.,  0.,
        -0., -1.,  0.,  0.,  0.,  0., -0., -1., -1., -1., -0., -0.,  0.,
         0.,  0.,  0., -1., -0.,  0., -0., -0., -0., -1., -0.,  0.],
       [-0., -0.,  1.,  0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0.,
        -1., -0.,  1., -0., -0., -0.,  1.,  0.,  0., -0., -1., -0.,  0.,
        -0.,  1.,  0., -0.,  1., -1.,  0.,  0.,  0., -0.,  0.,  0.,  1.,
        -0., -0.,  0., -1., -0.,  0., -0.,  0.,  0.,  0., -0., -0.],
       [ 0., -0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0., -0.,
         1.,  0., -1.,  0.,  0.,  0., -1.,  0., -0.,  0.,  1.,  0.,  0.,
         0., -1., -0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,
        -1.,  0., -0., -0.,  0.,  0.,  0.,  0., -0., -1.,  0.,  0.],
       [ 0., -0.,  0.,  0., -0.,  0., -1., -0., -1.,  0., -0., -0., -1.,
        -0., -0.,  0., -0., -0., -0.,  0.,  1., -1., -1., -0., -0.,  0.,
        -1.,  0., -0.,  1.,  0., -0., -0., -1.,  0.,  0.,  1., -0.,  0.,
        -0.,  1.,  0., -0., -0., -0.,  0., -1.,  0., -0.,  0., -0.],
       [ 0., -1.,  0., -0., -1., -0.,  1., -0., -0., -0.,  0.,  0., -0.,
         0.,  0., -0., -1.,  0., -0., -0., -1., -0., -0.,  0.,  0., -0.,
         1., -0.,  0., -1., -0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,
        -0., -1., -0., -0.,  0., -0., -0., -0., -0., -0., -0., -0.],
       [ 0.,  0.,  0., -0., -0., -0., -0., -0.,  1.,  0.,  0., -0., -0.,
        -0., -0.,  0.,  0.,  0.,  0.,  0., -1., -0.,  1., -0.,  0., -0.,
         1.,  0., -0.,  0.,  0., -0., -1., -0.,  0.,  0., -1., -1.,  0.,
        -0.,  0., -0., -0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0.],
       [ 0.,  1.,  0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,
         0., -0., -0., -0., -0., -0., -0., -0.,  1.,  0.,  0., -0., -1.,
        -1., -0.,  0., -0.,  0.,  0., -0., -0., -0., -0., -0.,  1., -0.,
        -0., -0.,  0., -0., -0., -1., -0., -0.,  0., -0., -0., -1.],
       [ 0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  0.,
        -0.,  0., -0., -0., -0.,  0., -0.,  1.,  0., -1.,  0., -0.,  0.,
        -1., -0., -0.,  1., -0.,  0., -0.,  0., -0.,  0.,  1.,  0., -0.,
         0., -0., -1.,  0., -0.,  0.,  0.,  0.,  0., -0., -1., -0.],
       [ 0.,  0.,  0., -0.,  0.,  1., -0., -0., -0., -0., -0., -0., -0.,
        -0.,  0.,  0., -1.,  0., -0.,  0., -1., -0., -0., -0., -1., -0.,
         1.,  0.,  0., -1., -0.,  0.,  0., -0.,  0.,  0., -1., -0., -0.,
         0., -1.,  1., -0., -1.,  0., -0.,  0., -0.,  0., -0.,  0.],
       [ 0., -0., -1., -0., -0.,  0.,  0.,  0.,  0., -0., -0., -0., -0.,
         1.,  0.,  0.,  0., -0.,  1., -1., -0., -0.,  0., -0., -0.,  0.,
         0., -1., -0., -0., -1., -0., -0., -0., -0.,  0., -0., -0.,  0.,
         0., -0.,  0., -0., -0.,  0.,  0., -0.,  1., -0.,  0.,  0.],
       [ 0., -0.,  1., -0., -0., -0., -0.,  0.,  0., -1., -0.,  0., -0.,
        -1.,  1., -0.,  0., -0., -1.,  1.,  0.,  0., -0., -1.,  0., -0.,
        -0.,  1.,  1.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,  0., -0.,
         1.,  0., -0., -0.,  0., -0., -0.,  0.,  0., -0., -0., -0.],
       [ 1.,  0.,  0.,  0., -0., -0.,  0.,  1.,  0., -0., -0., -0.,  0.,
        -1., -0., -0., -0., -0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,
         0.,  0., -1., -0.,  0.,  0., -0.,  0., -0.,  1., -0.,  0.,  0.,
        -0., -0.,  0.,  0.,  0., -0.,  1., -0., -1., -0., -0., -0.],
       [ 0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0., -0., -1.,  0.,  0.,
         1., -1.,  0.,  0.,  0., -0., -1.,  0., -0.,  0.,  1.,  0., -0.,
        -0.,  0., -0.,  0.,  0.,  0., -0., -0., -0., -0.,  0., -0., -0.,
        -1.,  0., -0., -0.,  0.,  0., -1., -0., -0., -0.,  0.,  0.]],
      dtype=np.float32))

In [ ]:
#@title Verification
n = 2
m = 4
p = 8
rank = 51

verify_tensor_decomposition(decomposition_248, n, m, p, rank)

##Rank-47 decomposition of <2,5,6> over Z

In [ ]:
#@title Data

decomposition_256 = (np.array([[ 0.,  0.,  1., -0.,  0.,  0., -1., -1.,  0., -0., -1.,  0., -0.,
         0.,  1., -0., -0., -0.,  0.,  0., -1.,  1., -0.,  0.,  1.,  0.,
         0., -0., -0., -1., -0., -1.,  0.,  0.,  0., -0.,  0.,  1., -0.,
        -0., -0., -0.,  0., -0.,  0.,  0., -0.],
       [ 0., -0., -0.,  1., -0., -0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  1., -0., -1.,  0., -0., -1.,  0.,  0.,  0., -0., -1., -1.,
         0., -0.,  0.,  0.,  0.,  0.,  1.,  0., -0., -1.,  0.,  1., -0.,
         0.,  0., -0., -0., -0.,  0., -0.,  0.],
       [-0., -0., -0., -1., -0.,  0., -0., -0.,  0., -1.,  0.,  1.,  0.,
         0.,  0., -1.,  1.,  0., -0., -0., -1.,  0., -1., -1.,  1.,  0.,
        -0.,  0.,  0., -0., -0.,  0., -1., -0., -0.,  1.,  0., -0.,  0.,
         0.,  0., -1., -0., -1.,  1.,  0.,  0.],
       [ 0., -0., -1.,  0., -0., -0., -0., -0.,  0.,  0., -0.,  1.,  0.,
        -0.,  0., -1.,  0.,  0.,  1.,  0.,  0.,  0., -1.,  0., -1., -1.,
        -0., -1., -1., -0., -0., -0., -0., -0.,  1.,  0.,  0.,  0., -0.,
        -0.,  0.,  0.,  1., -1.,  1., -0.,  0.],
       [-1., -1., -0., -0., -1.,  0., -1., -1.,  0., -0., -1.,  0.,  0.,
         0.,  1.,  1.,  0.,  1., -0., -0.,  0.,  1., -0., -0., -0.,  0.,
        -0.,  0., -0., -1., -0.,  0., -0., -0., -0.,  0., -0., -0., -0.,
        -0.,  1.,  1., -0.,  0., -0., -0.,  0.],
       [ 0., -0., -1., -1., -1.,  0., -0.,  1.,  1., -1., -0., -0., -0.,
        -0.,  0., -0., -0., -0.,  1., -0., -0.,  0., -0., -0.,  0., -0.,
         1.,  0.,  0., -0., -1., -0., -0.,  1.,  0., -0.,  0., -1., -1.,
         0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.],
       [-0., -0., -0.,  0., -1., -1., -0., -0.,  1., -1., -0.,  0.,  0.,
         1.,  0., -0.,  1.,  0., -0.,  1.,  0.,  1., -0., -0., -0., -0.,
         1., -1., -0.,  0.,  0.,  0., -1.,  0., -0., -0.,  0., -1.,  0.,
         0., -0.,  0.,  0., -0.,  0., -1., -0.],
       [ 0., -1.,  0.,  0.,  0.,  0., -1.,  0.,  0., -0.,  0., -1.,  1.,
         0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,  0.,  0.,
        -0., -0., -0.,  0., -1.,  0.,  1., -0., -0.,  0.,  1.,  0.,  0.,
         1., -0.,  1.,  1.,  0., -0.,  1.,  0.],
       [-0.,  0.,  1.,  1.,  0., -1.,  0.,  0.,  0.,  0., -0., -1.,  0.,
        -0.,  0., -0.,  0., -0., -1.,  0., -0., -0., -0., -0., -0., -0.,
        -0., -0.,  1., -0., -0., -0.,  0., -1.,  0., -0.,  1.,  0., -0.,
        -0., -0., -0.,  0., -0., -1., -1., -0.],
       [ 1.,  1.,  0.,  0.,  0., -0.,  1.,  1.,  1.,  0.,  1., -0.,  0.,
         0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0., -0.,
        -0., -0., -0., -0.,  0.,  0.,  0., -0., -0.,  0., -1.,  0.,  0.,
        -1., -1., -1., -1.,  1., -0., -0.,  1.]], dtype=np.float32), np.array([[ 0., -1., -0., -0.,  0.,  0.,  1., -1.,  0., -0.,  0.,  0., -1.,
        -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,
         0.,  0., -0.,  0., -1., -0., -0., -0., -0.,  0.,  0.,  0., -0.,
        -0.,  1., -0., -0.,  0., -0., -0., -0.],
       [-0.,  0.,  1., -0., -0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0.,
        -0.,  0.,  0., -0.,  0., -1., -0., -0.,  0., -0.,  0.,  0.,  0.,
        -0., -0.,  1.,  0., -0.,  0., -0., -0., -0., -0.,  0., -0., -1.,
        -0., -0., -0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -0., -0.,  0.,  0., -0., -0.,  0., -0.,  1.,  0.,  0., -0.,
        -0., -0., -0.,  1.,  0.,  0., -1., -1.,  0., -0., -1., -0., -0.,
        -0., -0.,  0., -0., -0., -0.,  0., -0.,  0., -0., -0.,  1., -0.,
         0.,  0.,  0., -0.,  0., -0.,  0., -0.],
       [ 0.,  0.,  1., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,
         0.,  0.,  0., -0., -0., -1., -0., -0.,  0., -0., -0.,  0.,  0.,
        -0., -0.,  1., -0., -0.,  1.,  0., -0.,  0., -0.,  0.,  0.,  0.,
        -0., -0., -0., -0.,  0., -0., -0., -0.],
       [-1., -0., -0.,  0., -1.,  0.,  0., -1., -0., -0., -1.,  0., -0.,
         1.,  0., -0., -0., -1., -0.,  0., -0., -1.,  0.,  0.,  0.,  0.,
         1.,  0., -0.,  1., -0.,  1.,  0.,  0.,  0.,  0.,  0., -0., -0.,
         0.,  0., -0.,  0., -0.,  0., -0., -0.],
       [-1., -0.,  0.,  0.,  0.,  0.,  0., -1.,  0., -0., -1., -0., -0.,
        -0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,
        -0., -0., -0., -0., -0.,  1., -0.,  0., -0.,  0.,  0.,  0., -0.,
         0.,  0., -0.,  0., -0.,  0.,  0., -0.],
       [-1.,  1., -0., -0.,  0., -0., -1., -0., -0., -1., -1.,  0.,  1.,
         0., -1.,  0., -0.,  0.,  0.,  1.,  1., -1.,  0.,  0., -0., -0.,
         1.,  0., -0.,  1.,  1.,  1.,  0.,  0., -0.,  1.,  0.,  0., -0.,
         0., -1., -0.,  0., -0.,  0., -0.,  0.],
       [-0.,  0., -0., -1., -0., -0.,  0.,  0.,  0., -0.,  0.,  1., -0.,
         0., -0.,  0., -0., -0.,  1.,  0.,  0., -0., -0., -0.,  0., -0.,
        -0.,  0., -1.,  0., -1., -0., -1., -1., -0., -0.,  0.,  0.,  1.,
         0., -0.,  0., -0.,  0.,  1.,  1.,  0.],
       [-0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,
         0., -0., -0., -1.,  0.,  0.,  1., -0., -0., -0.,  1., -0.,  0.,
         0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0., -1., -0.,  0., -0.,
         0., -0., -0.,  0.,  0.,  0., -0., -0.],
       [ 0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0.,
         0.,  0., -0.,  0., -0., -0.,  1., -0.,  0., -0.,  0.,  0.,  1.,
        -0., -1.,  0., -0.,  0.,  0.,  0.,  0.,  1., -0., -0., -0., -0.,
        -0.,  0.,  0.,  0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0., -0.,  0., -0., -0.,  0., -0., -0., -0.,
        -1.,  0.,  0.,  0., -0., -0.,  1., -0., -0., -0., -0., -0., -0.,
         0.,  0., -0.,  0., -0.,  0., -0.,  0., -0., -0.,  0., -0.,  0.,
         0.,  0., -0., -0., -0.,  0., -0., -0.],
       [ 0., -0., -0., -0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0., -0.,
         0., -1., -0., -0.,  0.,  0.,  1., -0., -1., -0., -0.,  0., -0.,
        -0.,  0., -0.,  1., -0., -0., -0., -0., -0., -0., -0.,  0.,  0.,
         0.,  0.,  0., -0.,  0.,  0.,  0., -0.],
       [-0.,  1., -0., -0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0.,  1.,
        -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -0.,  0.,  0., -0., -0.,  0., -0.,  0., -0., -0.,  0., -0.,  0.,
         0., -1., -1.,  0., -0.,  0.,  0.,  0.],
       [ 0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  1., -0.,
        -0., -0., -0., -0., -0.,  1.,  0., -0., -0.,  0.,  0.,  0.,  0.,
         0., -0., -1.,  0., -1., -0.,  0.,  0., -0.,  0.,  0., -0.,  1.,
         0.,  0., -0.,  0., -0.,  1., -0.,  0.],
       [ 0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,
        -0., -0.,  0., -1.,  0., -0.,  1., -0.,  0., -0.,  1., -0.,  0.,
        -0., -0.,  0., -0., -0.,  0.,  1., -0.,  0.,  0.,  0.,  0.,  0.,
        -0., -0., -0.,  0., -0.,  0., -0.,  0.],
       [ 0.,  1., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0.,  1.,
        -0.,  0., -0., -0.,  0., -0., -0., -0.,  0., -1., -0., -0., -0.,
        -0.,  0.,  0., -0., -0., -0., -0., -0.,  1., -0.,  0.,  0., -0.,
        -1., -1., -1., -1.,  1., -0.,  0., -1.],
       [-0., -0., -0., -0., -0.,  0.,  0., -0.,  0., -0., -0., -0.,  1.,
         0.,  0., -0., -1., -0., -0.,  1.,  0., -0.,  0., -0., -0., -0.,
         0.,  0., -0.,  0.,  0.,  0.,  1., -0.,  0., -0.,  0., -0., -0.,
        -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.],
       [ 1.,  0.,  0., -0.,  0., -0.,  1.,  0., -0.,  0.,  1.,  0., -0.,
        -0.,  0.,  0.,  0.,  0.,  0., -0., -1.,  0., -0.,  0., -0., -0.,
        -0., -0.,  0., -0.,  0., -1., -0., -0., -0., -0., -0.,  0.,  0.,
        -0., -0., -1., -0., -0., -0.,  0.,  0.],
       [ 0.,  0., -0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0.,
        -0., -0., -0.,  0.,  0.,  0., -0., -0., -0.,  0., -0., -0., -0.,
         0.,  0.,  1.,  0., -0., -0., -0.,  0.,  0.,  0., -1.,  0.,  0.,
        -1., -0., -0., -1., -0.,  0., -0.,  0.],
       [-0.,  0., -0., -0., -0.,  0., -0., -0., -0.,  0., -0.,  0.,  0.,
        -0.,  0.,  0.,  0.,  0., -1., -0.,  0.,  0., -0., -0.,  0., -0.,
        -0.,  0.,  1., -0., -0.,  0., -0.,  1.,  0., -0., -0., -0., -1.,
        -0., -0., -0., -0., -0.,  0.,  0.,  0.],
       [ 0.,  0., -1., -1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0., -0.,
        -0., -0.,  0.,  1.,  0., -0., -1., -1.,  0.,  0., -1., -1.,  0.,
        -0.,  0., -0.,  0., -0., -0., -0.,  0., -0.,  1., -0.,  1.,  0.,
        -0., -0., -0., -0.,  0.,  0., -0.,  0.],
       [ 0.,  0., -0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0., -0.,  0.,
        -0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0.,
        -0., -0.,  1., -0., -0., -0.,  0., -0., -1., -0.,  0., -0.,  0.,
        -0., -0., -0., -0.,  0.,  0., -0., -0.],
       [ 0., -0.,  0.,  0.,  0., -1.,  0.,  0., -0., -0.,  0.,  0.,  0.,
         1., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -0., -1.,  1., -0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0., -0., -0.],
       [-1.,  1.,  0., -0.,  0.,  0., -1., -0., -0.,  0., -1.,  0.,  1.,
        -0.,  0., -0.,  0.,  0., -0., -0.,  1.,  0., -1., -0., -0., -0.,
        -0.,  0.,  1., -0.,  1.,  1., -0., -1.,  0., -0., -1.,  0., -0.,
        -1., -1., -0., -1., -0., -1.,  0.,  0.],
       [-0.,  1.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  1.,
        -0.,  0., -0.,  0., -0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,
        -0., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,  0., -0.,
        -1., -1.,  0.,  0., -0.,  0.,  0.,  0.],
       [ 0.,  1., -0., -0.,  0., -0.,  0., -0., -0.,  0., -1., -0.,  1.,
         0.,  0.,  1., -0.,  0., -0., -0., -0.,  0., -1., -0.,  0., -0.,
        -0., -0.,  0.,  1.,  0.,  1., -0.,  0.,  0., -0.,  0., -0., -0.,
        -1., -1., -0.,  0.,  1., -0.,  0.,  0.],
       [ 1., -1.,  0.,  0.,  1., -0., -0.,  0., -1.,  0.,  1., -0., -1.,
        -0., -0.,  0.,  0., -0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,
        -1., -0.,  0., -1.,  0., -1., -0., -0., -0.,  0., -0.,  0., -0.,
         1., -0.,  0., -0.,  0., -0., -0., -0.],
       [ 0.,  1., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0.,  1.,
         0.,  0.,  1., -0., -0.,  0., -0., -0.,  0., -1.,  0.,  0.,  0.,
        -0., -0., -0., -0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,
        -1., -1.,  0.,  0.,  1.,  0., -0., -1.],
       [ 1.,  0., -0., -0.,  1.,  0., -0.,  0., -1.,  0.,  1.,  0.,  0.,
         0., -0.,  0.,  0.,  1.,  0., -0.,  0., -0., -0.,  0., -0., -0.,
        -1.,  0.,  0., -1.,  0., -1., -0., -0., -0., -0., -0., -0.,  0.,
        -0., -0.,  0., -0., -0., -0., -0.,  0.],
       [ 1.,  0., -0., -0., -0.,  0., -0.,  0.,  0.,  0.,  1., -0.,  0.,
        -0., -0.,  0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0.,
         0., -0.,  0., -1.,  0., -1., -0., -0.,  0., -0., -0., -0., -0.,
        -0., -0.,  0., -0.,  0., -0., -0.,  0.]], dtype=np.float32), np.array([[-0., -1., -0., -0., -1.,  0., -1.,  0., -0., -1.,  0., -0., -0.,
        -0., -0., -1., -0., -1.,  0., -0., -1., -0., -1.,  0., -0., -0.,
         1.,  0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,  0.,  1.,  0.,
         1., -0.,  1., -1.,  0.,  0.,  0.,  0.],
       [-0., -1., -0.,  0., -1., -0.,  0., -0.,  0.,  0.,  0.,  1.,  0.,
        -0.,  0.,  0.,  0., -1., -0.,  0.,  0., -0., -1.,  0.,  0., -0.,
         1.,  0.,  0.,  0.,  1.,  0., -0., -0., -0., -0., -1., -0., -0.,
         1.,  1., -0.,  0., -1., -1., -0., -0.],
       [-1., -0.,  1., -1., -0.,  0., -0., -0., -0., -0.,  1.,  0., -0.,
         0.,  0.,  0., -0.,  0., -1., -0., -0.,  0.,  1., -0., -1., -0.,
         0., -0., -0., -0., -0.,  1., -0., -1., -0., -0., -0.,  0., -0.,
         0., -0., -0.,  0.,  1.,  1.,  0.,  1.],
       [ 0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0., -1., -0.,
        -0., -0., -0., -0.,  0., -0., -0., -0.,  0.,  1., -0.,  0.,  0.,
         0.,  0.,  0., -0.,  0., -0., -0., -1., -0., -0.,  0., -0.,  1.,
         0., -0., -0., -0.,  1.,  1., -1.,  1.],
       [ 0.,  0.,  0.,  0., -1., -0.,  0., -0.,  0., -1., -0.,  0.,  0.,
        -0.,  0.,  0.,  0., -1.,  0., -0.,  0., -0.,  0., -1.,  1.,  0.,
         1.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  1.,  0.,  1.,  0.,
        -0., -0., -0.,  0.,  0.,  0., -0.,  0.],
       [-0., -1.,  0., -1., -1.,  0., -0.,  0., -0., -1., -0., -0., -1.,
        -0., -0., -0., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0.,
         1., -0.,  0., -0., -0., -0.,  1., -0., -0.,  1.,  0., -0.,  0.,
        -0.,  1., -0.,  0.,  0.,  0.,  1., -0.],
       [ 1., -0.,  0., -0., -0.,  0., -0.,  0., -0.,  0., -1., -0., -0.,
         0., -0.,  1.,  0., -0., -0.,  0., -0.,  0.,  0.,  0., -0., -1.,
         0., -0., -0.,  0.,  0., -1., -0., -0., -1.,  0., -0.,  0.,  0.,
         0.,  0.,  0.,  0., -1.,  0.,  0., -1.],
       [-0., -0., -0.,  0., -0., -1., -0., -0.,  0., -0.,  0., -0.,  0.,
        -0., -0., -0.,  0., -0., -1., -0.,  0., -0.,  0.,  0.,  0.,  0.,
        -0.,  1.,  1.,  0., -0.,  0.,  0., -0., -1.,  0.,  1., -0., -1.,
         0.,  0., -0., -1.,  0.,  0., -0., -1.],
       [-0., -0., -0.,  0., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,
        -1.,  1., -0., -1.,  1.,  0., -1.,  0., -1.,  0.,  1.,  0.,  1.,
         0.,  1.,  0.,  0.,  0., -0., -0.,  0., -0.,  0., -0.,  0.,  0.,
         0.,  0.,  0., -0.,  0., -0., -0.,  0.],
       [ 0.,  1., -0., -0.,  1.,  1., -0., -0., -1., -0.,  0., -0.,  1.,
        -1., -0.,  0., -0.,  1., -0.,  0., -0., -0.,  0., -0., -0., -0.,
         0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0., -0., -0.,
         0., -1., -0., -0.,  0.,  0.,  0.,  0.],
       [-1., -0., -0.,  0.,  1.,  0., -0.,  0., -0.,  1.,  1.,  0.,  0.,
        -0., -1., -0., -0., -0.,  0., -0.,  1.,  0.,  1.,  0.,  0., -0.,
        -1.,  0., -0.,  1.,  0., -0., -0.,  0., -0.,  0.,  0., -1.,  0.,
         0., -0.,  0., -0.,  1., -0.,  0.,  1.],
       [ 0., -0.,  0., -0., -0., -0., -1., -1.,  1., -0.,  1., -1.,  0.,
        -0.,  0.,  0., -0., -0., -0.,  0., -0., -1.,  1., -0., -0., -0.,
        -1.,  0.,  0.,  1., -1.,  0., -0.,  0., -0., -0., -0.,  0., -0.,
         0.,  0., -0., -0.,  1.,  1.,  0.,  1.]], dtype=np.float32))

In [ ]:
#@title Verification
n = 2
m = 5
p = 6
rank = 47

verify_tensor_decomposition(decomposition_256, n, m, p, rank)

##Rank-23 decomposition of <3,3,3> over Z

In [ ]:
#@title Data
decomposition_333 = (np.array([[ 1.,  0., -0., -0., -1.,  0.,  0., -0., -0.,  1., -0.,  0.,  0.,
        -0., -0.,  0., -0.,  0., -0., -0.,  0.,  0.,  0.],
       [-1., -0.,  0.,  0., -0.,  0.,  0.,  0., -1.,  0., -0.,  1., -0.,
         0.,  0.,  0., -0., -0., -1., -0.,  0.,  0.,  1.],
       [ 0., -0., -0., -0.,  0.,  0., -1., -0.,  0., -0.,  1.,  1.,  0.,
         0., -0., -0.,  1.,  0.,  0.,  1.,  0., -0., -0.],
       [ 0., -0.,  1.,  1.,  0., -0., -0., -0.,  0., -0.,  1., -0., -1.,
         0.,  0., -0.,  0.,  0., -0., -0., -1., -0., -0.],
       [-0.,  1.,  0., -1.,  0., -1., -0., -1.,  0., -0., -0.,  1., -0.,
        -0., -0., -0.,  0.,  1., -0.,  1., -0., -0.,  0.],
       [ 0.,  1., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  1.,  1., -0.,
        -0.,  0.,  0., -0., -0., -0.,  1.,  0.,  1., -0.],
       [ 1., -0., -1.,  0.,  0.,  0., -0.,  1.,  0., -0.,  0., -0.,  0.,
        -0.,  1., -0.,  0.,  0., -0.,  0., -0.,  0., -1.],
       [-1.,  0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,
        -1., -1., -0.,  0.,  0., -1.,  0., -0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0., -1.,  0.,  0., -0.,  0., -0., -0., -1.,
        -1.,  0.,  1., -0., -0., -1., -0.,  0., -1., -0.]], dtype=np.float32), np.array([[ 0., -0., -1., -1.,  1., -0., -0., -1., -0.,  0., -0., -0.,  0.,
         0.,  0.,  0., -0.,  1., -0., -0.,  1., -0.,  0.],
       [-0.,  0., -1.,  0., -0., -0., -0.,  0., -0.,  1., -0., -0.,  0.,
         0.,  0., -0., -0., -0.,  0., -0.,  1., -0., -0.],
       [ 1.,  0., -1.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,  1.,
         0.,  0.,  1., -0., -0., -1.,  0., -0., -0.,  1.],
       [ 0., -0.,  0.,  0., -0.,  1.,  0., -0., -0.,  0.,  1.,  1., -1.,
        -1.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  0., -0.],
       [ 0.,  0.,  0., -0.,  0.,  1., -0.,  1.,  1., -0.,  1.,  1., -1.,
        -1., -1., -0., -0., -0.,  0.,  1.,  1., -0.,  1.],
       [-0.,  0.,  0., -0., -0.,  1.,  0.,  1., -0.,  0., -0., -0.,  0.,
        -1., -1.,  1., -0., -0., -1., -0., -0., -0.,  1.],
       [ 0., -1., -0., -0.,  0., -1.,  0., -0., -0.,  0., -1.,  0.,  1.,
        -0.,  0., -0.,  0., -1., -0., -1., -1.,  0., -0.],
       [ 0.,  0., -0.,  0.,  0.,  0., -1.,  0.,  0.,  0., -1.,  0.,  1.,
        -0., -0.,  0., -0.,  0.,  0.,  0., -1., -1.,  0.],
       [ 0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,
        -0., -0., -1., -1.,  0., -0.,  0.,  0., -1.,  0.]], dtype=np.float32), np.array([[-0.,  1.,  0., -0., -1.,  0.,  0., -0.,  1.,  0., -0.,  1., -0.,
         0.,  0., -0., -0.,  0., -0., -1., -0., -0., -0.],
       [-0., -1., -0., -1., -0., -0., -0.,  0.,  0.,  0., -0.,  0., -0.,
        -0.,  0., -0., -0.,  1.,  0.,  0., -0., -0., -0.],
       [-0.,  0.,  0.,  0., -0.,  1.,  0., -1.,  0., -0.,  0.,  0., -0.,
         1., -1., -0., -0.,  1., -0., -0., -0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0., -0.,  1.,  0., -1.,  1., -0.,  0., -0.,
         0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.],
       [ 0., -0., -0.,  1., -0., -0., -1., -0., -0.,  0., -1.,  0., -0.,
        -0., -0., -0., -0., -1.,  0.,  1., -1., -0., -0.],
       [-0.,  0.,  1., -0.,  0., -1.,  0.,  1.,  1.,  0.,  0.,  0., -1.,
         0.,  0., -0., -0., -1., -1., -0., -1.,  0.,  1.],
       [ 1.,  0.,  0., -0.,  0., -0.,  0.,  0.,  1.,  0.,  0.,  0., -0.,
         0., -1., -0., -1., -0.,  0., -0., -0., -0.,  1.],
       [ 0., -0., -0.,  0.,  0., -1.,  1., -0.,  0.,  0.,  1.,  0., -1.,
        -0., -0., -1., -0., -0., -0., -1., -0., -1.,  0.],
       [ 0.,  0.,  0.,  0.,  0., -0., -0.,  0., -1.,  0., -0., -0., -0.,
         0.,  1., -1., -0., -0.,  1.,  0., -0.,  0., -1.]], dtype=np.float32))

In [ ]:
#@title Verification
n = 3
m = 3
p = 3
rank = 23

verify_tensor_decomposition(decomposition_333, n, m, p, rank)

## Rank-54 decomposition of <3,4,6> over 0.5*Z


In [ ]:
#@title Data

decomposition_346 = (np.array([[ 0.5,  0. ,  0. ,  0. , -1. ,  1. ,  0. ,  1. , -1. ,  0. ,  0. ,
         0. ,  0.5,  1. , -0.5,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5,  0. ,
         0. ,  0. ,  0. ,  1. , -1. ,  1. ,  0.5, -0.5,  0. ,  0. ,  0.5,
         0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         1. ,  0. ,  0.5, -1. ,  1. , -1. ,  0. ,  1. ,  0. , -1. ],
       [ 0.5,  0. ,  0. ,  0. ,  1. ,  1. ,  0. ,  1. ,  1. ,  0. ,  0. ,
         0. ,  0.5, -1. ,  0.5,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5,  0. ,
         0. ,  0. ,  0. ,  1. ,  1. , -1. ,  0.5, -0.5,  0. ,  0. , -0.5,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        -1. ,  0. ,  0.5,  0. ,  0. , -1. ,  0. ,  1. ,  0. ,  0. ],
       [-0.5,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. , -0.5,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. , -0.5,  0. , -1. ,  0. , -1. ,  0. ,  0. ,  1. ,  0. ,  0. ,
         0. ,  0. ,  0. , -0.5,  0. ,  0.5,  0. ,  0. ,  0. ,  0. ,  0.5,
        -1. ,  0. , -0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  1. ,  0. ,  0. , -1. ,  1. ,  0. , -1. ,  1. ,  0. ,  0. ,
         0. ,  0.5, -1. , -0.5,  0. ,  0. ,  0. ,  0. ,  0. , -0.5,  0. ,
         0. ,  0.5,  0. ,  0. ,  1. ,  0. , -0.5, -0.5, -1. ,  0. , -0.5,
         0. ,  0. ,  0. ,  0.5,  0. , -0.5,  0. ,  0. ,  0. ,  0. ,  0.5,
         0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  1. ,  0. ,  0. ],
       [-0.5,  0. , -1. , -0.5,  0. ,  0. , -1. ,  0. ,  0. , -1. , -1. ,
        -1. ,  0. ,  0. ,  0.5, -1. , -0.5,  0. ,  0. , -0.5,  0.5,  0. ,
         0. , -0.5,  1. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        -0.5,  0. ,  1. ,  0. , -0.5,  0.5,  0. , -1. , -0.5,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0.5,  0. , -1. ,  0. ],
       [-0.5,  0. ,  0. , -0.5,  0. ,  0. ,  1. ,  0. ,  0. ,  1. , -1. ,
         0. ,  0. ,  0. , -0.5,  1. ,  0.5,  0. ,  0. , -0.5, -0.5,  0. ,
         0. ,  0.5,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0.5,  0. ,  1. ,  0. , -0.5, -0.5,  0. ,  1. ,  0.5,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0.5,  0. ,  0. ,  0. ],
       [ 0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  1. ,  0.5,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0.5,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  1. ,  0. ,
         0. ,  0. , -1. ,  0.5,  0. ,  0. ,  1. ,  0. ,  0. ,  0. , -0.5,
         0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , -0.5,  0. ,  0. ,  1. ,  1. ,  0. ,  1. , -1. ,
         0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0. , -0.5,  0.5,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,
         0.5,  0. ,  0. , -0.5,  0.5,  0.5,  1. , -1. ,  0.5,  0. , -0.5,
         0. ,  0. ,  0.5,  0. ,  0. , -1. ,  0.5,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  1. ,  0.5, -1. ,  1. ,  0. ,  0. , -1. ,  0. ,  0. ,
         0.5,  0.5,  0. ,  0. ,  0. ,  0. , -0.5, -1. ,  0.5,  0. ,  1. ,
         0. ,  0. , -0.5,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0.5,  0. , -1. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5,  1. ,  0. ,
         1. ,  0. ,  0. , -1. ,  0.5,  0. ,  0. ,  0. ,  0. , -0.5],
       [ 0. ,  0. ,  0. ,  0.5,  1. , -1. ,  0. ,  0. ,  1. ,  0. ,  0. ,
         0.5,  0.5,  0. ,  0. ,  0. ,  0. , -0.5, -1. ,  0.5,  0. , -1. ,
         0. ,  0. , -0.5,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        -0.5,  0. , -1. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5, -1. ,  0. ,
        -1. ,  0. ,  0. ,  0. , -0.5,  0. ,  0. ,  0. ,  0. ,  0.5],
       [ 0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. , -1. ,
        -0.5,  0. , -1. ,  0. ,  0. ,  0. ,  0.5,  0. , -0.5,  0. , -1. ,
         0.5,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,
         0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        -1. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0.5],
       [ 0. ,  1. ,  0. ,  0.5, -1. ,  1. ,  0. ,  0. ,  1. , -1. ,  1. ,
         0. ,  0.5, -1. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,
        -0.5,  0. ,  0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,
        -0.5,  0. ,  0. ,  0. , -0.5,  0. ,  0. ,  0. , -0.5,  1. ,  0. ,
         0. ,  1. ,  0. ,  0. ,  0.5,  0. ,  0. ,  1. ,  0. ,  0. ]],
      dtype=np.float32), np.array([[ 0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5, -1. ,
         0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  1. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         1. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. , -1. ],
       [ 0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,
        -1. ,  1. , -0.5,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ],
       [ 0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. , -1. ,  0. ],
       [-1. ,  0. , -1. ,  0. ,  0. , -1. ,  0.5,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  1. , -1. ,  0. ,  0. ,  1. ,  0. , -1. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. , -1. ,  0.5, -1. ,
         0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. , -0.5,  1. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ],
       [ 0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,
         0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,
         0. ,  0. ,  0. ,  1. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [-1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. , -0.5,  0. ,
         0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  1. ,  1. , -1. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,
        -1. ,  1. , -0.5,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. , -1. ,
         0. ,  0. ,  1. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  1. ],
       [ 0. , -0.5,  1. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  1. ,  0.5,  0. ,  0. ,  0. ,  1. ,  0. ,  1. ,  0. ,  0. ,
         0. ,  0. ,  1. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  1. , -1. ,  0. , -0.5,  0. ,  0. ,  0. ,
        -0.5,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ],
       [ 0. ,  0. , -1. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        -1. ,  0. , -0.5,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  1. ,  0. ,
         0. , -1. , -1. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. , -1. ,
         0. ,  0. ,  0. ,  1. , -1. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,
         0. , -1. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. , -1. ,  0. ],
       [-1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        -1. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. , -1. , -1. ,  0.5,  1. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  1. ,
         1. ,  0. ,  0. ,  0. ,  1. ,  0. , -1. ,  0. ,  0. , -0.5,  0. ,
         1. ,  0. ,  0.5,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  1. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. , -1. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        -1. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 1. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. , -1. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,
         0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. , -1. ,
         0. ,  0. , -1. ,  0. ,  0. ,  1. ,  1. ,  0. ,  0. ,  0. ],
       [ 0. ,  0.5,  0. ,  1. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,
        -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,
         0. ,  0. ,  1. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,
         0. ,  0. , -1. ,  0. ,  1. , -1. ,  0. , -0.5,  0. ,  0. ,  0. ,
        -0.5,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         1. ,  0. , -0.5,  1. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,
         0. , -1. , -1. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. , -1. ,
         0. ,  0. ,  1. , -1. , -1. ,  2. ,  0. ,  0.5,  0. ,  0. ,  0. ,
         0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. , -1. ,  0. ,  0. , -1. ,  0.5,  0. ,  0. , -1. ,  0. ,
         0. ,  0. ,  0. ,  0. , -1. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  1. , -1. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  1. ,  0. ,
        -1. ,  0. ,  0. , -1. , -1. ,  0. ,  0. ,  0. ,  0. ,  0.5, -1. ,
         0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  1. ,  1. ,  0. ,  0. ,  1. ,  1. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,
        -1. ,  0. ,  0. ,  0. , -1. ,  0. ,  1. ,  0. ,  0. , -0.5,  0. ,
         0. ,  0. , -0.5,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,
         1. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  1. ],
       [ 0. ,  1. , -1. ,  0. , -1. , -1. ,  0. ,  0. ,  0. , -1. ,  0. ,
         0. , -1. ,  0. ,  0. , -1. ,  2. ,  0. , -1. ,  0. ,  0. ,  0. ,
         0. ,  0. , -1. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,
        -1. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  1. ,  0.5,  0. ,
         0. ,  0. ,  0. ,  1. , -1. ,  0. ,  0. , -1. ,  0. ,  0. ],
       [-1. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  1. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -0.5,  0. ,
         0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. , -1. ,
         0. ,  0. ,  1. ,  0. ,  0. , -1. , -1. ,  0. ,  0. ,  0. ],
       [ 0. ,  0.5,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,
         1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,
         0. ,  0. , -1. ,  0. , -1. ,  0. ,  0. ,  0. , -0.5,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. , -1. ,  1. ,  0. ,  0.5,  0. ,  0. ,  0. ,
         0.5,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        -1. ,  0. , -0.5,  1. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,
         0. , -1. ,  1. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. , -1. ,
         0. ,  0. ,  0. ,  1. ,  1. ,  0. ,  0. , -0.5,  0. ,  0. ,  0. ,
         0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. , -1. ,  0. ,  0. , -1. , -0.5,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. , -1. , -1. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,
        -1. ,  0. ,  0. ,  1. , -1. ,  0. ,  0. ,  0. ,  0. ,  0.5, -1. ,
         0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  1. ,  1. ,  0. ,  0. ,  1. , -1. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,
         1. ,  0. ,  0. ,  0. ,  1. ,  0. ,  1. ,  0. ,  0. ,  0.5,  0. ,
         0. ,  0. , -0.5,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  1. ],
       [ 0. ,  0. , -1. ,  0. ,  1. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,
         0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        -1. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. , -1. ,  0.5,  0. ,
         0. ,  0. ,  0. , -1. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ]],
      dtype=np.float32), np.array([[ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. , -1. ,  0. ,  0. ,  0. ,
         0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. , -1. ,
         0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0.5, -1. ,  0. ,  1. ,  0. ],
       [ 0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -0.5,  1. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0.5,  0. , -1. ,  1. ,  0. , -1. ,  0. ,  0. ,  1. ],
       [ 0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  1. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. , -0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,
         1. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. , -1. ,  0. ,  0. ,
         0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,
         0. ,  0. ,  1. , -1. ,  0. ,  1. ,  0.5,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,
         0. ,  0. ,  1. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,
        -1. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  1. ,
         0. ,  1. ,  0. ,  0. ,  0. ,  1. ,  0. ,  1. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,
         0. ,  0. ,  0. , -1. ,  0. ,  1. ,  0. ,  1. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ],
       [ 0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,
        -1. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        -1. ,  0. ,  1. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. , -1. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. , -1. ,  0. , -1. ,  0. ,  0.5,  0. ,  1. ,  0. ,  0. , -1. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 1. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. , -1. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. , -0.5, -1. ,  0. ,  0. , -1. ,  0. ,
         0. , -1. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. , -0.5,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,
         1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,
         0. ,  0. ,  0. ,  0. ,  0.5,  0. , -1. ,  0. ,  1. ,  0. ,  0. ,
         0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  1. , -0.5,
         0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,
        -1. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. , -1. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  1. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -0.5,  0. ,  0. , -0.5,
        -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. , -1. ,  1. ,  0. ,  0. ,  0. ,  0. ,  1. ],
       [ 0. ,  0. ,  0. ,  0. , -0.5, -0.5,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  1. , -1. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,
         0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [-1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0.5,  0. ,  0. ,  0. ,
         0. ,  0. , -1. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  1. , -1. ,  0. ,  0. ,
         0. ,  0. , -1. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0.5,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ]],
      dtype=np.float32))

In [ ]:
#@title Verification
n = 3
m = 4
p = 6
rank = 54

verify_tensor_decomposition(decomposition_346, n, m, p, rank)

##Rank-63 decomposition of <3,4,7> over 0.5*C

In [ ]:
#@title Data

decomposition_347 = (np.array([[ 0. -0.j , -1. +0.j ,  0. +0.j ,  0. +1.j ,  0. +0.j , -0. +0.j ,
        -0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. -0.j , -0. +0.j ,  1. +0.j ,  0. +0.j ,  0. -1.j ,  0. +0.j ,
         1. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  1. +0.j , -0. +1.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j , -1. -0.j ,  0. +0.j ,  0. -0.j ,  0. -1.j , -0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. -1.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  1. +0.j ,
         0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,
        -0. +1.j ,  0. -0.j , -0. +0.j , -0. +0.j ,  1. +0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.j ,  0. +0.j ],
       [ 0. +0.j , -0.5-0.j , -0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
         0. -0.j ,  0. -0.j ,  0. +0.j ,  0.5+0.j , -0. +0.j ,  0. -0.j ,
        -0. +0.j ,  0.5+0.j , -0. +0.j ,  0. +0.j ,  0. +0.5j, -0. +0.5j,
        -0.5-0.j , -0. +0.j ,  0. +0.j ,  0.5+0.j , -0.5+0.j ,  0. -0.5j,
         0. +0.j , -0. +0.5j,  0. +0.j , -0. +0.j ,  0. -0.j ,  0.5+0.j ,
        -0.5+0.j ,  0.5+0.j , -0. +0.j , -0. +0.j ,  0. +0.5j, -0.5+0.j ,
         0. -0.5j, -0.5+0.j ,  0. +0.j , -1. +0.j , -0. +0.5j,  0. -0.5j,
        -0.5-0.j ,  0.5+0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j , -0.5+0.j ,
         0.5+0.j , -0.5-0.j ,  0. +0.j , -0. +0.j ,  0. -0.5j, -0. +0.5j,
         0. -0.5j, -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0.5+0.j ,
         0. +0.j ,  0.5+0.j ,  0. +0.j ],
       [ 0. -0.j , -0. +0.j ,  0. -0.j , -1. -0.j ,  0. -0.j , -0. +0.j ,
        -0. +0.j , -0. +0.j ,  0. -0.j ,  0. -1.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. -1.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  1. +0.j ,
         0. +0.j ,  0. -0.j ,  0. -1.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  1. +0.j ,  0. -0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,
         0. -1.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +1.j ,
         1. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,
        -0. +1.j , -0. +0.j , -1. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -1. +0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -1.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ],
       [-0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,  1. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +1.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j ,  0. -1.j ,  0. -1.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,  0. -1.j ,
         0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,
        -0. +0.j , -0. +1.j , -0. +0.j ,  0. -0.j , -0. +0.j ,  1. +0.j ,
         0. -0.j ,  0. -1.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,
         0. -1.j , -0. +1.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,  1. +0.j ,
         0. -0.j , -0. +1.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +1.j ,  0. +0.j ],
       [-1. -0.j ,  0. +1.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. -1.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. -1.j , -0. +0.j , -1. -0.j ,  0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. -1.j ,  0. +0.j ,  0. -0.j ,  0. -1.j ,  0. -0.j ,
        -1. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  0. -1.j ,
        -0. +0.j , -0. +1.j ,  0. +0.j , -0.5-0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
        -0. +0.j ,  0. -0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -1.j , -0. +0.j ,  0. +0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.5j,  0. -0.5j, -0. +0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.5j, -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
        -0. +0.j ,  0. -0.5j, -0. +0.j ,  0.5+0.j ,  0. +0.j , -0. +0.j ,
        -0. +0.5j,  0. +0.5j,  0. +0.j ,  0. +0.j , -0. +0.5j,  0. +0.j ,
         0.5+0.j ,  0.5+0.j , -0. +0.j ,  0.5+0.j , -0. +0.j ,  0. -0.5j,
         0. +0.j , -0. +0.5j,  0.5+0.j , -0. +0.j , -0. +0.j ,  0. -0.5j,
        -0.5+0.j ,  0. -0.5j, -0.5+0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j , -0.5+0.j ,  0. +0.j ,
         0. -0.5j,  0. +0.j , -0. +0.5j,  0. -0.5j, -0.5+0.j ,  0. -0.j ,
        -0.5-0.j ,  0. +0.5j,  0. +0.j ,  0. +0.5j, -0. +0.j , -0. +0.j ,
         0. -0.j ,  0. -0.5j,  1. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j , -1. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
        -1. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -1.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j , -0. +1.j ,  0. -0.5j,  0. +0.j ,  1. +0.j ,
         0. -1.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +1.j ,  0. +0.j ,
         1. +0.j ,  0. +0.j ,  1. +0.j ,  0. +0.j ,  0. +1.j ,  0. -0.j ,
        -0. +0.j ,  0. +0.j , -0.5+0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.j ,  0. +0.j ],
       [ 0. -1.j ,  0. +0.j , -1. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j , -0. +1.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j , -0. +1.j , -0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,
        -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j ,  1. +0.j , -0. +1.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j ,  0. -1.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j , -0. +0.j ,  1. +0.j , -0. +0.j ,  0. +0.j ,
         0. -1.j ,  1. +0.j , -0.5-0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  1. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  1. +0.j ,
         1. +0.j ,  1. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. -0.j ,  0. -1.j ,  0. -1.j ,  0. +0.j ,
         0. -0.j ,  1. +0.j ,  0. -0.j , -0. +0.j , -0. +0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j , -0. +1.j ,  0. +0.j , -0. +0.j ,
         0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. +1.j ,  0. -0.j ,  1. +0.j ,
         0. -0.j , -1. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  1. +0.j ,  1. +0.j ,
         1. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j , -0. +0.j ,  0.5+0.j ,  0. -0.j ,  0. +0.5j,  0. -0.j ,
         0.5+0.j ,  0.5+0.j ,  0. +0.5j,  0.5+0.j ,  0.5+0.j ,  0.5+0.j ,
         0. -0.5j, -0. +0.j , -0. +0.j ,  0. -0.5j,  0. -0.5j,  0. +0.5j,
        -0. +0.j , -0.5+0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. -0.5j,
         0. +0.5j,  0. -0.j , -0. +1.j , -0. +0.5j,  0. +0.j ,  0. +0.j ,
        -0.5-0.j , -0. +0.j ,  0. +0.5j,  0. -0.j , -0. +0.5j,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. -0.j ,  0. -0.5j,
         0. +0.j ,  0. -0.j , -0. +0.5j,  0. +0.5j,  0. +0.5j, -0.5-0.j ,
         0. +0.j , -0.5-0.j , -0.5+0.j ,  0.5+0.j ,  0. +0.j ,  0. -0.5j,
         0. +0.j ,  0. +0.j ,  0. +0.j , -0.5-0.j ,  0.5+0.j ,  0.5+0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +1.j ,  0. -1.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,  1. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +1.j , -0. +0.j , -0. +0.j ,
        -1. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  1. +0.j ,  0. +0.j ,
         0. -1.j ,  0. +0.j , -1. -0.j , -0. +0.j , -1. +0.j , -0. +0.j ,
         0. -0.j ,  0. -0.j ,  1. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j , -1. -0.j ,  0. +0.j , -1. -0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,
        -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. +1.j ,  0. -0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0. -1.j ,  0. -0.j ,  1. +0.j , -0. +1.j ,
         0. +0.j ,  0. +0.j , -1. -0.j ,  0. +1.j ,  0. -0.j ,  0. +0.j ,
         1. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  1. +0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  1. +0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  1. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. -1.j , -0. +1.j , -0. +0.j ,  1. +0.j ,
        -0. +0.j ,  0. +0.j , -0. +0.j ,  0. -1.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ]]), np.array([[ 0. +0.j , -0. +0.j ,  0. -0.5j,  0. +0.j ,  0.5+0.j , -0. +1.j ,
         0. +0.j ,  0. +0.j , -0. +0.5j,  0. +0.j , -0. +0.j ,  0. -0.j ,
         0. +1.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. -0.j , -1. -0.j , -0. +0.j ,  0. -0.j , -0. +0.j , -1. -0.j ,
        -0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j , -0. +0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.5j,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -1. +0.j ,
         0. +0.j ,  0. -0.j ,  0. -0.j , -0.5-0.j ,  0. -0.j ,  0. +0.5j,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. -1.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ],
       [-0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. -0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.5j,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. -0.j ,  0. -0.j , -0. +0.5j,  0. +0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j , -0. +0.j ,
         0. +0.j ,  0. -0.5j,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j ,  0.5+0.j ,  1. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  0.5+0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j , -0.5-0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. -0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  1. +0.j , -0.5+0.j ,
         0. -0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j , -0.5-0.j ,
         0. -0.j ,  0. -1.j , -0. +0.j ,  0. -0.j , -0. +0.j , -0. +0.j ,
         0. -1.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,
         0. -0.5j, -0. +0.j ,  0.5+0.j ,  0. -0.j , -1. -0.j , -0. +0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. -0.5j,  0. -1.j ,
         0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,  0. -0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,  0. -0.j ,
         0. -1.j , -0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  1. +0.j ,  0. +0.j , -0. +0.j ,  0.5+0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j , -0. +0.j ,  0. -0.j , -0. +0.j , -0. +0.j ,
         0. -0.j , -0. +0.j , -0. +1.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  1. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,
        -0. +0.j , -0.5+0.j ,  0. +0.j ,  0.5+0.j ,  0. -0.j ,  0. -0.5j,
         0. -0.j ,  0. -1.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  1. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.5j,
         0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j , -1. +0.j , -0. +0.j ,
         0. +0.j , -0.5-0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. -1.j , -0. +0.j , -1. +0.j ,  0. -0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
        -0. +0.j ,  0. -1.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0.5+0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j , -0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,
        -0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.5j,
         0. +0.5j, -0. +0.j ,  0. +0.j ,  0. -0.5j, -0. +0.j , -0. +0.j ,
         0. -1.j ,  0. +0.j ,  0. -0.5j,  0. +0.j , -0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j , -0. +0.j , -0.5-0.j ,  0. -0.j ,
         0. -0.j , -0. +0.j ,  0. -0.j ,  0. +0.j , -0. +1.j ,  0. -1.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j , -0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,
         1. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j ,  0. -0.5j,  0. -0.5j, -0. +0.j ,
        -0. +0.j ,  0.5+0.j ,  0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j , -0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0.5+0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j ,  0. -0.j , -0. +0.j , -0.5+0.j ,  0. +0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j ],
       [ 0. +1.j ,  0. -1.j ,  0. +0.j , -0.5-0.j ,  0. +0.j ,  0.5+0.j ,
         0. +0.j ,  1. +0.j ,  0. -0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.5j,  0.5+0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,
        -0.5-0.j , -0. +1.j , -0.5+0.j ,  0. +0.5j, -1. -0.j ,  0. +0.5j,
        -0. +0.5j, -0. +0.5j,  0. +0.j ,  0.5+0.j , -0.5-0.j ,  0. -0.5j,
         0. +0.j ,  0. +0.j , -0. +0.j ,  0. -1.j ,  0.5+0.j ,  0. -0.j ,
         0. +0.j ,  0. -0.j , -0.5-0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +1.j ,
         0. +0.5j,  0. -0.5j,  0. +0.5j, -0. +0.j , -0. +0.j , -0. +0.j ,
        -0. +0.5j, -0. +0.j ,  0. +1.j , -0.5-0.j , -0. +1.j ,  0. -0.5j,
        -0. +0.5j, -0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0. -1.j ,  0. +0.j ,  1. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j , -0. +1.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. -0.j ,  0. -1.j , -0. +0.j ,  0. -0.j ,  0. -0.j ,
         0. -0.j ,  0. -0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,  0. -1.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j , -1. +0.j ,  0. -0.j , -0. +1.j ,
         0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  1. +0.j ,  0. -0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +1.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. -1.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. -1.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j ,  1. +0.j , -0. +0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j , -1. -0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j , -1. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j , -1. -0.j ,
         0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j , -0. +0.j , -1. -0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  1. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -1.j ,  0. +0.j ,  1. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,
         0. -0.j ,  0. -0.j ,  0. +1.j ,  0. -0.j ,  0. -1.j ,  0. +0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ],
       [-0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -1. +0.j , -0. +0.j ,
         0. -0.j ,  0. -0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  1. +0.j , -0. +0.j , -1. -0.j ,  0. +0.j ,  0. +1.j ,
         0. +0.j ,  0. -0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,  0. -1.j ,
         0. -0.j ,  0. +1.j ,  0. +0.j ,  0. -0.j , -0. +0.j , -0. +0.j ,
         0. +0.j ,  1. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,
        -0. +0.j , -0. +0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j , -0. +0.j , -0. +0.j , -0. +0.j , -1. -0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -1.j ,  0. -0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j , -0. +1.j ,
         0. -1.j , -0. +0.j ,  0. +0.j ,  0. +1.j ,  0. -0.j , -0. +0.j ,
         0. -0.j , -0. +0.j ,  0. +1.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  1. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j , -0. +1.j ,  0. -0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j , -0. +1.j ,  0. +1.j ,  0. +0.j ,
         0. +0.j , -1. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -1.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -1. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j , -1. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,
        -0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,  0. -0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j , -1. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j ,  0. +1.j ,  0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,
        -0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  1. +0.j ],
       [ 0. +0.j ,  0. +0.j , -0.5+0.j ,  0. -0.j ,  0. -0.5j,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +1.j ,  0. +0.j ,  1. +0.j ,
         1. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  1. +0.j ,
         0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
        -0. +0.j , -0. +0.j , -0.5+0.j ,  0. +0.j ,  1. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j , -0.5-0.j ,
         0. +0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. -1.j , -0. +0.5j,  0. +0.j ,  0.5+0.j ,
        -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ],
       [ 0. -0.j ,  0. +0.5j,  0. +0.j ,  1. +0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j , -0.5+0.j ,  0. +0.j ,
         1. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,
         0. -1.j ,  0.5+0.j ,  0. -0.j ,  0. -0.j , -1. -0.j ,  0. -0.j ,
         0. -1.j ,  0. +0.j ,  0. -0.j , -0. +0.5j, -0. +1.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
        -0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ],
       [-0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. +1.j ,  0. -0.5j,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
        -0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.5j,  0. -0.j ,  0. +0.j ,  0. +0.j ,
        -0.5-0.j , -0. +0.j ,  0. -0.5j, -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j , -0.5-0.j ,  0. -0.j ,  0.5+0.j ,  0. -0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
        -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j , -0. +0.j ,  1. +0.j ,  0. -1.j , -0. +0.j , -0. +0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j , -0. +0.j ,
         0. +1.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,
        -0. +1.j ,  0. +0.5j, -0. +0.j ,  0. -0.5j,  0. -0.j , -0.5+0.j ,
        -0. +0.j , -1. -0.j ,  0. +0.j , -0. +0.j , -0. +0.j , -0. +0.j ,
         0. -1.j , -0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,  0.5+0.j ,
         0. +0.j , -0.5-0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
         0. -0.j , -0. +0.5j,  0. +0.j ],
       [ 0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.5j,
         0. -0.j ,  0. -0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
        -0.5+0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,  0.5+0.j ,
        -0.5+0.j ,  0. +0.j , -0. +0.j ,  0.5+0.j ,  0. -0.j ,  0. +0.j ,
         1. +0.j ,  0. +0.j ,  0.5+0.j , -0. +0.j , -0. +0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.5j,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ],
       [-0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. +1.j ,  0.5+0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  1. +0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j ,  0.5+0.j ,  0.5+0.j ,  1. +0.j ,
         0. +0.j , -0. +0.5j, -0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j , -0. +0.j , -0.5-0.j ,  1. +0.j ,  0. -0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
        -0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.5j,
         0. -0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.5j,  0. -1.j ,
        -0. +1.j , -0. +0.j ,  0. +0.j ],
       [-1. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.5j, -0. +0.j ,  0. +0.5j,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. -0.j ,  0. -1.j ,
         0. +0.j , -0.5+0.j ,  0. +0.5j,  0. +0.j , -0. +0.j ,  0. -1.j ,
        -0. +0.5j,  0. +0.j ,  0. -0.5j, -0.5-0.j ,  0. +0.j , -0.5-0.j ,
         0.5+0.j , -0.5-0.j ,  0. -0.j ,  0. -0.5j,  0. -0.5j,  0.5+0.j ,
         1. +0.j ,  0. +0.j ,  0. +1.j , -1. +0.j ,  0. -0.5j,  0. +0.j ,
         1. +0.j , -0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j , -0.5-0.j ,  0. -0.j ,  0. -0.j ,
        -0.5-0.j ,  0.5+0.j , -0.5+0.j ,  0. +0.j ,  0. +1.j ,  0. +0.j ,
        -0.5-0.j ,  0. +0.j , -1. -0.j ,  0. -0.5j,  0. -0.j , -0.5+0.j ,
         0.5+0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0.5+0.j , -0. +0.j ,  0. +0.5j,  0. +0.j ,
         0. +0.j , -0. +0.j , -0.5+0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,
        -1. -0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. -0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j , -0.5+0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.5j,  0. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,  0. -0.j ,
         0. +0.j , -0. +0.j , -1. -0.j , -0. +0.j , -0.5-0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. -1.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j , -0. +0.j ,  0. -0.j ,
         0. -0.j ,  0.5+0.j ,  0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.5j,  0. +1.j ,  1. +0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.5j,
        -0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -1.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.5j,  0. +0.j ,
         0. +0.j ,  0. -1.j ,  0. +0.j ],
       [-0. +0.j ,  0. +0.j ,  0. -1.j ,  0. +0.j ,  1. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -1.j , -0. +0.5j,
         0. -0.j ,  0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.5j,
         0. -0.j ,  0. +0.j ,  0. -0.j ,  1. +0.j ,  0. -0.j , -0. +0.j ,
         0. +0.j , -0. +0.j ,  0. -0.5j,  0. +0.j , -0. +1.j ,  0. +0.j ,
        -0.5+0.j , -0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +1.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j , -0.5-0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
        -0. +0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j , -0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. -0.j ,
         0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. -0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.5j,  0. -0.j , -0. +0.5j,  0. +0.j ,  0.5+0.j ,
        -0. +0.j ,  1. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0.5-0.j ,
        -0. +0.j , -0.5-0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. -0.5j,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j ,  1. +0.j , -0. +0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.5j,
        -0. +0.j ,  0. +0.j ,  0. -1.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j , -1. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j , -0. +0.j ,
         0.5+0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -0.5-0.j ,
         0.5+0.j ,  0. -0.j ,  0. +0.j , -0.5-0.j ,  0. +0.j , -0. +1.j ,
        -1. -0.j ,  0. +0.j , -0.5+0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. +0.5j, -0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  1. +0.j ,  0. +0.j ],
       [ 0. +0.j , -0. +0.j , -1. +0.j ,  0. +0.j ,  0. -1.j ,  1. +0.j ,
        -0. +1.j ,  0.5+0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
        -0. +0.j ,  0. -0.j ,  0. -0.j ,  0.5+0.j ,  0.5+0.j ,  0. +0.j ,
        -0. +0.j , -0. +0.5j,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. -0.j , -0.5-0.j ,  0. -0.j , -0. +0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +1.j ,  0. -0.j , -0. +0.5j,
         0. +0.j ,  0. -1.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ],
       [ 1. +0.j , -0. +0.j ,  0. +1.j ,  0. -0.5j, -1. -0.j ,  0. -0.5j,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. -0.j ,
        -0. +0.j ,  0.5+0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. -0.5j,  0. +0.j ,  0. -0.5j, -0.5-0.j , -0. +0.j ,  0.5+0.j ,
        -0.5-0.j , -0.5-0.j ,  0. +0.j , -0. +0.5j,  0. -0.5j, -0.5+0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j ,  1. +0.j , -0. +0.5j,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j , -0. +1.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j , -0. +0.j ,
        -0.5+0.j , -0.5-0.j , -0.5-0.j ,  1. +0.j , -0. +0.j , -0. +0.j ,
         0.5+0.j ,  0. +1.j , -1. -0.j ,  0. +0.5j, -0. +0.j ,  0.5+0.j ,
        -0.5+0.j ,  1. +0.j ,  0. +0.j ]]), np.array([[ 0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. -0.j ,  1. +0.j ,  0. -0.j ,  0. +0.j ,
         1. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j , -0. +1.j ,
         0. -0.j ,  0. -1.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,  0. -0.j , -0. +0.j ,
         0. -0.j , -1. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j , -1. +0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
        -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j , -0. +0.j , -1. +0.j ,
         0. -1.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  1. +0.j ,  0. +0.j ],
       [-0.5+0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,
        -0. +0.j , -0. +0.j , -0. +1.j ,  0. +0.j , -0. +0.j , -0. +0.j ,
        -0. +1.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. -0.j , -0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +1.j ,  0. -1.j ,  0. +0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j , -0. +1.j , -1. -0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ],
       [-0. +0.j , -0. +0.j , -0. +1.j ,  0. +0.j ,  0. +0.j ,  0. -1.j ,
         0. +0.j , -0. +0.j , -1. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
        -1. +0.j ,  0. -0.j , -0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  1. +0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. -0.j , -0. +0.j , -1. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j ,  0. -1.j ,  0. -0.j , -0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j , -1. +0.j , -0. +0.j , -1. +0.j ,  0. -0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j , -1. -0.j , -0. +1.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j , -0. +0.j , -1. +0.j ,
         0. -0.j ,  0. +1.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +1.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0.5+0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
        -1. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,  1. +0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
        -0. +0.j ,  1. +0.j ,  0. -0.j , -1. +0.j ,  0. -0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  1. +0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,  0. -0.j ,
        -1. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. -0.j , -0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.j , -0. +0.j , -1. -0.j ,  1. +0.j , -0. +0.j ,
         0. +0.j , -1. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j , -0. +1.j ,
        -0. +1.j , -0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  1. +0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. -1.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  1. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. -1.j , -0. +0.j , -0. +0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j ],
       [ 0. -0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  1. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
        -1. -0.j ,  0. -0.j ,  0. -0.j , -0. +1.j , -0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,
         0. -1.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -1.j ,  0. -1.j ,
         1. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. -0.j ,  1. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,
         0. -1.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
        -0. +0.j ,  0. -0.j ,  0. +0.j ],
       [ 0. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +1.j ,  0. -0.j ,
         0. +0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
         0. -1.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j , -0. +0.j ,  1. +0.j ,  0. -1.j , -0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j , -1. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. -1.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  1. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. -0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j , -1. +0.j , -1. -0.j ,
         0. +0.j ,  0. -0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. -1.j ,  0. -1.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  1. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. -1.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,
         0. -0.j ,  0. -0.j ,  0. -0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,
         0. +1.j , -0. +0.j ,  0. +0.j ],
       [-0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,
        -0. +0.j , -0. +0.j ,  0. -1.j ,  0. -0.j , -0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j , -0. +1.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. -0.j ,  0. -1.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,
         0. +0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,
         0. -0.j , -1. -0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. +1.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j , -0. +0.j ,  0. -0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,
         0. -1.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j , -0. +0.j ,
         0. +0.j ,  1. +0.j ,  0. +0.j ],
       [-0.5-0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
        -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. +1.j ,
         0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j , -0. +1.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
         0. -0.j ,  1. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. +1.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j , -1. +0.j , -0. +1.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0. +1.j ,  0. -0.j , -0. +1.j ,  0. +0.j ,
         0. -0.j , -0. +0.j , -1. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. -0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  1. +0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. -0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j , -0. +0.j , -1. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. -1.j ,  0. +0.j ,  0. -1.j ,  0. -0.j , -0. +0.j ,
         0. +0.j , -1. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -1. -0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j , -0. +0.j , -0. +1.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         1. +0.j ,  0. +0.j , -1. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. -0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,  0. -0.j ,
         0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +1.j ,
        -1. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -1.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,
         0. +1.j , -0. +0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. -0.j ,  0. +0.j ],
       [-0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j , -0. +0.j , -0. +0.j ,
        -0. +0.j , -1. -0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +1.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j , -0. +1.j ,  0. +0.j ,  1. +0.j ,
         0. -0.j , -0. +0.j ,  0. -0.j , -0. +0.j ,  0. -0.j ,  0. -0.j ,
        -1. -0.j ,  0. -0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -1. +0.j ,  0. -0.j ,
         0. -0.j , -0. +0.j , -1. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. -0.j ,  0. -1.j , -0. +0.j ,  1. +0.j ,
         0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +1.j ,
         0. +0.j ,  0. -0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j , -0. +1.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
        -0. +0.j ,  0. -0.j ,  0. -0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,
        -0. +1.j ,  0. +0.j , -1. -0.j ,  0. +0.j , -1. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +1.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j , -0. +1.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. -0.j ,  1. +0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,
        -1. -0.j ,  0. +0.j ,  0. -0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,
         0. -0.j ,  1. +0.j , -0. +0.j ,  0. +0.j , -1. -0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  1. +0.j ,
        -0. +0.j ,  0. -1.j , -0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
        -0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  1. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,
        -0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j , -1. +0.j , -1. +0.j ,
        -0. +0.j ,  0. -0.j ,  0. +0.j ],
       [-0.5+0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -1.j ,  1. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -1.j ,  0. +0.j ,  0. -0.j ,
         0. -0.j , -0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +1.j ,  0. -0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j , -0. +0.j ,  1. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j ,  1. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
        -0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. -1.j ,
         1. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j , -0. +0.j ,
        -0. +0.j ,  0. +0.j , -0. +0.j ,  1. +0.j ,  0. -0.j ,  0. +0.j ,
        -0. +0.j ,  1. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,
         0. -1.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
        -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +1.j ,  0. -0.j , -0. +0.j ,
        -1. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. -1.j ,  0. +0.j ,  0. -1.j ,  0. -0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,
        -0. +0.j ,  0. -1.j , -1. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
        -1. -0.j ,  0. -0.j ,  1. +0.j ,  0. -0.j ,  0. -0.j ,  0. -0.j ,
         0. -0.j , -1. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -1.j ,
        -0. +0.j , -1. -0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,  0. -1.j ,
         1. +0.j ,  0. +1.j ,  0. +0.j , -0. +1.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. -0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. -1.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,
        -1. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -1.j ,  0. +0.j ],
       [ 0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,
        -0. +0.j , -0. +0.j , -0. +0.j ,  0. -1.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,
         0. -0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  1. +0.j ,  0. -0.j , -0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j ,  1. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  1. +0.j ,  0. -0.j ,  0. -0.j , -1. -0.j ,
         0. +0.j ,  0. +0.j , -0. +1.j ,  0. -0.j ,  0. +0.j , -1. -0.j ,
         0. -0.j ,  0. +0.j ,  0. -0.j ,  0. -1.j ,  0. +0.j , -0. +0.j ,
        -0. +0.j ,  0. -1.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
        -1. -0.j , -0. +0.j ,  0. +1.j ,  0. -1.j ,  0. -1.j ,  0. +0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j , -0. +1.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j , -1. +0.j ,  1. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. -1.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j ,  0. -0.j ,  1. +0.j ,  0. +0.j , -0. +0.j ,
         0. +1.j ,  0. +0.j ,  0. +0.j ]]))

In [ ]:
#@title Verification
n = 3
m = 4
p = 7
rank = 63

verify_tensor_decomposition(decomposition_347, n, m, p, rank)

##Rank-74 decomposition of <3,4,8> over Z


In [ ]:
#@title Data

decomposition_348 = (np.array([[ 1., -0.,  0., -0.,  0.,  1.,  0., -1.,  0., -1., -0., -0.,  1.,
         1.,  0., -1.,  0., -1., -0., -0.,  0.,  1.,  0.,  1.,  0.,  1.,
        -1., -1., -1.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,
         1.,  1., -1.,  1.,  1.,  0., -0., -0.,  0., -0., -0., -0.,  0.,
        -0., -1., -0., -1.,  1., -1.,  0.,  0., -1., -1., -0.,  0., -0.,
         1., -1., -1.,  1., -0.,  0., -0.,  0., -1.],
       [-0., -0., -0.,  1., -0.,  1., -0.,  0., -1.,  0.,  0., -0.,  1.,
         1., -0., -1., -0.,  0., -0.,  0., -0., -0., -0.,  1., -0., -0.,
         0.,  0.,  0., -1., -0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,
        -0., -0.,  0.,  1., -0., -0.,  0.,  0., -0., -0., -0.,  0., -0.,
         0., -1., -0.,  0., -0., -1.,  0., -0.,  0.,  0., -0.,  0., -0.,
         1.,  0.,  0., -0.,  0., -0., -0., -0., -1.],
       [-0.,  0.,  0., -0., -0.,  1.,  0., -1.,  0.,  0., -0.,  1., -0.,
        -0., -0.,  0.,  0., -1.,  0., -0.,  0., -0., -0.,  1.,  0., -0.,
         0.,  0., -1.,  0.,  0., -1.,  0., -0.,  0.,  0.,  0., -0.,  1.,
         1., -0., -1.,  1.,  1., -0.,  0., -0.,  0., -0.,  0., -0.,  0.,
         0.,  0.,  0., -1.,  1., -1., -0.,  0.,  0.,  0.,  0., -0.,  1.,
        -0.,  0., -1.,  1., -0.,  0.,  1.,  0., -1.],
       [-1.,  0.,  0.,  1., -0.,  0., -0.,  1., -1., -0., -0.,  0.,  0.,
         0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,  1., -1.,
        -0.,  1.,  1.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0., -1., -0.,  0., -1.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,
        -0., -0.,  0.,  1.,  0., -0.,  0., -0., -0.,  1., -0., -0.,  0.,
         0., -0., -0., -1., -0.,  0.,  0.,  0., -0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0., -0.,  0., -0.,  0.,
         0., -0., -0.,  1., -0., -1., -0.,  0.,  0., -0., -1.,  0.,  0.,
        -0., -0., -0.,  0., -1.,  1., -0.,  0.,  1., -0.,  0.,  0.,  0.,
        -1.,  0.,  1., -1.,  0., -0., -0.,  0.,  1., -1.,  0., -0.,  0.,
        -1., -0., -1.,  1.,  0., -0.,  0.,  0., -0., -0., -1.,  0.,  0.,
         0., -0.,  1., -1., -0., -0., -0., -0.,  1.],
       [-0., -0., -1.,  0., -0.,  0., -1., -0., -0.,  0., -0., -0.,  0.,
         0.,  0., -0., -0., -0., -1.,  1., -1., -0.,  0.,  0., -0., -0.,
         0.,  0., -0., -0., -1., -0.,  1., -0.,  1., -0., -0., -1.,  0.,
         0., -0., -0.,  0., -0.,  0., -0., -0., -0., -1., -1.,  1., -1.,
         0., -0., -1., -0., -0., -0., -0., -1.,  0.,  0., -1.,  0.,  0.,
         0.,  0., -0.,  0.,  1.,  0., -0.,  0.,  1.],
       [-0., -0.,  0., -0., -0.,  0.,  0.,  1.,  0.,  0.,  0.,  0., -0.,
        -0.,  0.,  0.,  1.,  1.,  0., -0.,  0., -0.,  0., -1., -0., -0.,
         0.,  0.,  1.,  0.,  0.,  1., -0., -0.,  1., -0., -0.,  0., -1.,
        -1., -0.,  1., -1.,  0.,  0.,  0.,  0.,  1.,  0.,  0., -0.,  0.,
         0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0., -1., -1.,
        -0.,  0.,  1., -1., -0., -0.,  0., -0.,  1.],
       [-0., -0.,  0.,  0., -1.,  0., -1., -1., -0.,  0.,  0.,  0.,  0.,
        -0., -0.,  0.,  0.,  0.,  0.,  1., -1.,  0.,  0., -0.,  0., -0.,
        -0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0., -0., -0.,  0., -0.,
        -0., -0., -1.,  1., -0., -0., -0., -0., -1.,  0., -1., -0.,  0.,
        -0., -0., -0., -1.,  0.,  0., -0., -1., -0.,  0.,  0., -0., -0.,
         0., -0.,  0., -0., -0., -0.,  0., -0.,  0.],
       [ 0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0., -1.,  0., -0.,  1.,
         0., -1., -1.,  0.,  0.,  1.,  0., -0.,  1., -1., -0., -0.,  1.,
        -0., -1.,  0., -0., -0.,  0., -1.,  0., -1., -1.,  0.,  1.,  0.,
        -0., -0.,  0., -0., -0., -1.,  0.,  1.,  0.,  1., -0.,  0., -0.,
        -0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0., -1., -0.,  0., -0.,
         1., -1.,  0., -0.,  0., -1., -0., -0.,  0.],
       [ 0.,  0., -0., -0.,  0., -0.,  1., -0.,  0.,  0., -1.,  0., -0.,
        -0.,  0.,  0., -0., -0.,  1., -1., -0., -0.,  0., -0.,  0., -0.,
        -0.,  0., -0.,  0.,  1., -0., -1., -0., -1., -1.,  0.,  1.,  0.,
         0., -0., -0., -0.,  0.,  0., -1., -0., -0.,  1.,  1., -1.,  1.,
         0.,  0.,  1., -0.,  0.,  0., -1.,  1., -0.,  0., -0.,  0.,  0.,
         1.,  0., -0.,  0., -1.,  0., -0.,  0.,  0.],
       [-1., -1.,  0., -0., -0., -0., -0.,  0., -0.,  0.,  0., -0., -0.,
        -1.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0., -0., -0.,
         1.,  0.,  0., -0., -0.,  0., -1.,  0., -1., -1., -1.,  1., -0.,
        -0.,  0.,  0., -0., -0.,  0.,  0.,  1.,  0.,  1., -0.,  0., -0.,
         0., -0., -0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0.,
        -0.,  0.,  0., -0.,  0., -1.,  1., -0.,  0.],
       [-0., -0.,  0., -0., -0., -0.,  1., -0.,  0.,  1.,  0., -0.,  0.,
        -0., -0.,  1.,  0.,  0., -0.,  0., -0.,  0.,  1., -0., -0., -1.,
         0.,  1., -0., -0.,  1.,  0.,  0.,  0., -0., -0., -0.,  0., -0.,
        -0.,  0., -0.,  0.,  0., -0., -1., -1., -0.,  0.,  1., -1., -0.,
        -0., -0., -0., -0., -0.,  0., -1.,  1.,  0., -0., -0., -0., -0.,
         0., -0.,  0.,  0.,  0., -0.,  0., -1.,  0.]], dtype=np.float32), np.array([[ 0., -0., -0., -0., -0., -0., -0., -0.,  0., -0., -0.,  0.,  0.,
         0., -0., -0., -0., -1., -0., -0.,  0., -0., -0., -0., -0., -0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  1.,
        -1.,  0.,  0., -0., -0., -0.,  0.,  0.,  0., -0.,  0., -0., -0.,
         0.,  0., -0., -0., -0., -0.,  0., -0., -1., -0., -0., -0., -0.,
        -0.,  1., -0., -0., -0.,  0.,  0.,  0.,  0.],
       [-0., -0., -0.,  0., -0.,  0.,  0., -0., -0.,  0., -0.,  0., -0.,
        -0., -0., -0., -0.,  0.,  1., -0., -0., -1., -0.,  0., -0., -0.,
        -0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0., -0.,  0., -0.,  0.,
         0.,  0.,  0.,  0., -0., -1.,  0., -0.,  0.,  0.,  0.,  0.,  1.,
        -0., -0., -1.,  0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0.,  0.,
         0., -0., -0., -0., -0., -0., -0.,  0., -0.],
       [ 0.,  0.,  0., -0., -0.,  0.,  0., -0.,  0., -1.,  0., -0.,  0.,
        -0., -0.,  0.,  0., -1.,  1.,  1.,  0.,  0.,  1.,  0., -0.,  0.,
        -0.,  0.,  0., -0., -1.,  0., -0.,  1., -0., -0., -0.,  0.,  1.,
        -1.,  0.,  0., -0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0.,  1.,
         0., -0.,  0.,  0.,  0., -0.,  0., -1., -1., -0.,  0., -0.,  0.,
         0., -0.,  0., -0., -0., -0., -0.,  0.,  0.],
       [-0.,  1.,  0., -0.,  0., -0., -0.,  0.,  0., -0., -0.,  0.,  0.,
        -0., -0.,  0., -0., -0., -1.,  0.,  0., -0.,  0., -0.,  0.,  0.,
         1., -0., -0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,  1., -0.,
         0., -0., -0., -0.,  0., -0., -0., -0., -0.,  1., -0., -0., -0.,
        -0.,  0., -0., -0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,
        -0.,  1., -0.,  0.,  1., -1.,  0.,  0.,  0.],
       [-0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,
        -0., -0.,  0.,  0., -1.,  1., -0., -0.,  0., -0.,  0.,  0.,  0.,
         0., -0.,  0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,  0.,  1.,
        -1., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0., -0.,  1.,
        -1.,  0., -1., -0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0.,
        -0.,  0., -0.,  0., -0., -0., -0., -0.,  0.],
       [ 0.,  0.,  0., -0., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0.,
        -0.,  1.,  0., -0.,  0.,  1.,  0., -0., -0., -0., -0.,  0.,  0.,
         0.,  0., -0.,  0.,  0., -1.,  0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0.,  0.,  0.,  0., -0., -0., -0., -0., -0., -0.,  1.,
         0.,  0., -1., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0., -1.,
        -0.,  0.,  1.,  0., -0., -0., -0., -0.,  0.],
       [-0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0., -0.,  0., -0.,  1.,
         0.,  0., -0., -0., -0.,  0.,  0.,  0., -1.,  0., -1., -0.,  0.,
         0., -0.,  0.,  1., -0., -0., -0.,  0., -0., -0., -0.,  0., -1.,
        -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,
        -0.,  1., -0., -0., -0., -1., -0., -0., -0., -0., -0.,  0.,  0.,
         0.,  0., -1., -0.,  0., -0., -0., -0., -0.],
       [ 0.,  0., -0., -0.,  0.,  0., -0., -0., -0., -0., -0., -0., -0.,
        -0.,  0.,  0., -0., -0.,  0.,  0.,  0., -1.,  0.,  0.,  1., -0.,
         0.,  0., -1., -0., -0., -0., -0.,  0., -0., -0., -0., -0., -1.,
        -0., -1.,  0., -0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,  0.,
        -0., -0., -0.,  0., -0.,  0.,  0., -0., -0.,  1., -0.,  0., -0.,
        -0.,  0., -1.,  1.,  0., -0., -0., -0., -0.],
       [ 0., -0., -0., -0., -0., -0., -0.,  0., -0., -1., -0., -0., -0.,
         0., -0., -1.,  0., -0., -0.,  0.,  1.,  0., -0.,  0., -0.,  0.,
         0.,  0., -0., -0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0.,
         1., -0.,  1., -1.,  0., -0.,  1.,  0., -0., -0.,  0., -0.,  0.,
        -0., -0., -0.,  0., -0.,  0.,  0., -1., -0., -0., -0., -0., -0.,
        -1., -1.,  0.,  0.,  0., -0.,  0.,  0.,  1.],
       [ 0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0., -0., -1.,  0., -1.,
        -0., -0., -0., -0., -0.,  0.,  0., -0.,  1., -0.,  0.,  0.,  0.,
        -0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0., -0., -0.,
        -0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0., -1.,
        -0., -0., -0., -0.,  0., -0., -0., -0., -0.,  0.,  0.,  0.,  0.,
         1., -0.,  0.,  0., -0.,  0., -0., -0., -0.],
       [ 0.,  0., -0., -0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,
        -0.,  0.,  0., -0., -0.,  0., -1.,  1., -0.,  0., -0., -0., -0.,
        -0.,  0.,  0.,  0., -0.,  0., -0., -1., -0., -0., -0., -0.,  0.,
         1.,  0.,  1., -1.,  0., -0., -0.,  0., -0.,  0.,  0., -0., -1.,
        -0.,  0., -0., -0.,  0., -0.,  0., -0., -0., -0.,  0., -0.,  0.,
         0.,  0., -0., -0.,  0.,  0., -0.,  0.,  1.],
       [ 0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,  0., -0.,  0., -0.,
        -1.,  0.,  0., -0.,  0., -0., -0., -0.,  0.,  0., -0., -0., -0.,
        -1.,  0., -0., -0.,  0.,  0., -0., -0.,  0., -1.,  0., -1.,  0.,
         0., -0., -0.,  0.,  0., -0., -0.,  0., -0.,  0., -0., -0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0., -0., -0.,
        -1., -1.,  0., -0., -1.,  1.,  0., -0.,  0.],
       [ 0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,  0.,  0.,  0.,
        -0.,  0.,  0., -0.,  1.,  0.,  0., -0., -0., -0.,  0., -0., -0.,
        -0.,  0., -0., -0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,  0.,
         1., -0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0., -1.,
         1.,  0.,  1.,  0., -0.,  1.,  0.,  0.,  0., -0., -1.,  0.,  0.,
        -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1., -0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0.,  0.,
        -0.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,  0., -1., -0., -0.,
        -0.,  0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,
         0., -0., -0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0.,  0.,
        -0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0., -0.,
        -0., -0., -1., -0.,  1.,  0., -0.,  0.,  1.],
       [ 0.,  0.,  0.,  0., -0., -0., -0., -0., -0., -0.,  0., -0., -1.,
         0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  1., -0.,  1.,  0., -0.,
        -0., -0., -0., -1.,  0., -0., -0., -0.,  0., -0., -0., -0., -0.,
         0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0.,
         0.,  0.,  0.,  0., -0., -0., -0., -0., -0.,  0., -0.,  0., -0.,
         1., -0.,  1., -0.,  0.,  0., -0.,  0., -1.],
       [ 0.,  0., -1., -1., -0., -0., -0.,  0., -0., -0., -0., -0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1., -1., -0.,
        -0.,  1., -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0., -0., -0.,
        -0., -0.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,  0., -1., -0.,
         0., -0.,  0., -0.,  0.,  0.,  1., -0., -0., -1., -0.,  0.,  0.,
        -0.,  0.,  1.,  0.,  0.,  0., -0., -0., -1.],
       [-0.,  0., -0., -0., -0., -0., -0., -0.,  0., -0.,  0.,  0.,  0.,
         0., -0., -0., -0.,  1.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,
         1.,  0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,  0., -0., -1.,
        -0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,
        -0., -0.,  0., -0., -1.,  0.,  0., -0.,  1., -0., -0., -0.,  0.,
        -0.,  0.,  0., -0., -0.,  0.,  1.,  0.,  0.],
       [-0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0., -0.,  1., -0.,
        -0., -1.,  0., -1., -0., -1., -0., -0., -0.,  0.,  0.,  0., -0.,
        -0.,  0., -0.,  0., -0.,  1.,  0.,  0., -1.,  0.,  1., -1., -0.,
        -0., -0.,  0.,  0., -0., -0., -0., -0.,  0.,  0.,  0.,  0., -1.,
        -0.,  0.,  1.,  0.,  0., -0.,  0., -0., -0., -0.,  0.,  0., -0.,
         0.,  0., -0., -0., -1., -0., -0.,  0., -0.],
       [ 0., -0., -0.,  0.,  0., -0.,  0., -0., -0.,  1.,  0., -0.,  0.,
        -0.,  0.,  0.,  0.,  1., -1.,  0.,  0., -0., -1.,  0.,  0.,  0.,
        -0., -0., -0.,  0.,  1., -0., -1.,  0., -1.,  0., -0.,  0., -1.,
        -0., -0., -1., -0.,  0.,  0.,  0., -0., -1., -0., -0., -0.,  0.,
         0.,  0., -0., -0.,  0., -0.,  0.,  1.,  1.,  0.,  0., -0., -0.,
         0., -0., -0.,  0.,  0., -0., -0., -0.,  0.],
       [-0., -1.,  0., -0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0.,
        -0.,  0., -0., -0., -0.,  1., -0.,  0., -0., -0., -0.,  0.,  0.,
         0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., -0.,  0.,  0.,  0.,
        -0.,  0., -0., -0.,  0., -0.,  0., -0., -0., -1., -0., -0., -0.,
        -0., -0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,
        -0.,  0., -0.,  0.,  0., -0.,  1., -0., -0.],
       [ 0., -0., -0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0.,
         0.,  0.,  0.,  0., -0., -1., -0., -0., -0.,  0.,  0., -0.,  0.,
        -0.,  0.,  0.,  0., -0., -0.,  0., -0., -1., -0., -0.,  0., -1.,
        -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  1., -0.,  0.,  0.,
        -0., -0., -0., -0.,  0.,  0.,  0., -0., -0., -0., -0.,  1.,  0.,
        -0., -0., -0., -0.,  0.,  0., -0.,  0.,  0.],
       [ 0., -0.,  0., -0., -0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0.,
         0., -1.,  0., -1., -0., -1.,  0., -0., -0.,  0., -0.,  0.,  0.,
        -0.,  0.,  0.,  0., -0.,  1.,  0.,  0., -1.,  0., -0., -1.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0., -0., -1.,
        -0., -0.,  1., -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0.,  1.,
        -0.,  0.,  0.,  0., -1., -0., -0., -0.,  0.],
       [-0., -0., -0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0., -0.,  0.,
         1.,  0., -0., -0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,  0.,
         0., -0., -0.,  0.,  0., -0., -0.,  0., -0., -0., -0., -0.,  1.,
        -0., -0., -0., -0., -0., -0.,  0.,  0., -0.,  0., -0., -0.,  0.,
        -0., -1., -0., -0., -0.,  1., -0.,  0.,  0.,  0.,  0.,  0., -0.,
         0.,  0., -0., -0., -0., -0., -1., -0., -0.],
       [ 1., -0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0.,
        -0., -0.,  0., -0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0., -0.,
         0.,  0.,  1.,  0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,  1.,
         0.,  1., -0., -0.,  1.,  0.,  0., -0., -0.,  0., -0., -0., -0.,
        -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,
        -0.,  0., -0.,  0., -0.,  0., -1., -0., -0.],
       [-0., -0., -0., -0.,  0.,  0.,  0.,  1., -0.,  1., -0.,  0.,  0.,
         0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,  1.,
        -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,
        -1.,  0., -1., -0., -0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,
         0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,
        -0.,  1., -0., -0., -0.,  0., -0., -0.,  0.],
       [ 0., -0., -0., -0.,  0., -0., -1.,  0.,  0.,  0., -0., -0., -0.,
         0.,  0., -0.,  0.,  0., -0.,  1., -0., -1.,  0.,  0., -0.,  0.,
         0., -1.,  0., -0., -0., -0., -0., -0.,  0., -0., -0.,  0.,  0.,
         0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0., -0., -0., -0.,  1.,
         0.,  0.,  0., -0., -0., -0., -0., -0.,  0.,  1.,  0., -0., -0.,
         0.,  0., -0.,  0.,  0., -0.,  0., -1., -0.],
       [ 0., -0.,  0., -0.,  0.,  0., -1.,  1., -0., -0.,  0., -0.,  0.,
        -0., -0.,  0.,  0.,  0., -0.,  1.,  0.,  0., -0.,  0.,  0., -0.,
         0.,  0.,  0., -0.,  0., -0.,  0.,  1.,  0., -0.,  0.,  0., -0.,
        -1.,  0., -1., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  1.,
         0., -0.,  0., -0., -0., -0.,  0., -0.,  0., -0.,  0., -0., -0.,
         0., -0.,  0.,  0., -0., -0.,  0.,  0.,  0.],
       [ 1., -0.,  0.,  0., -0., -0.,  1., -0., -0., -0., -0., -0.,  0.,
        -0., -0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0.,
         1.,  1., -0.,  0.,  0.,  0., -1.,  0., -0., -0., -0.,  1.,  0.,
         0.,  0.,  0.,  0., -0.,  0.,  0.,  1.,  0., -0.,  0.,  0., -0.,
        -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,
        -0.,  1., -0., -0.,  1., -1., -0., -0.,  0.],
       [ 0., -0., -0., -0., -1.,  0., -1., -0., -0., -0.,  0., -0.,  0.,
        -0., -0.,  0.,  0., -1., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,
         0.,  0., -1.,  0.,  1., -0., -0., -0.,  0., -0.,  0.,  0., -0.,
        -1.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0.,  1.,
        -1., -0., -1.,  1., -0.,  0., -0.,  1., -0., -0., -0., -0.,  0.,
        -0.,  0., -0.,  0., -0., -0.,  0.,  0.,  0.],
       [ 0., -0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,
        -0., -0.,  0.,  0.,  0., -0., -0., -0., -0., -0., -0.,  0., -0.,
         0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0., -0.,  0.,  0., -0.,
        -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0.,  1.,  1., -0.,
         0.,  0.,  0.,  1., -0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,
        -0.,  0.,  1., -1., -1., -0., -0.,  0.,  0.],
       [ 0., -0.,  0.,  0., -0.,  0., -0., -1., -1.,  0.,  0.,  0.,  1.,
        -0., -0.,  1., -0., -0., -0., -0.,  0., -1.,  0., -1., -0., -1.,
         0.,  0., -0.,  1.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0.,
        -0., -0., -0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -0., -0., -0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,
         0.,  0., -1., -0., -0., -0.,  0.,  0., -0.],
       [-0., -0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.,
        -0., -0., -0., -0., -0., -0., -0.,  0., -1., -0.,  0.,  1.,  0.,
         0., -1.,  0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0.,
        -0.,  0., -0., -0., -0., -0.,  0.,  0.,  0., -0.,  0., -0., -0.,
        -0.,  0., -0., -1.,  0.,  0.,  0.,  0.,  0.,  1., -0., -0.,  0.,
        -0.,  0., -1.,  1., -0., -0.,  0.,  0., -0.]], dtype=np.float32), np.array([[-0., -0., -0., -0., -0.,  0.,  0.,  0.,  1., -1.,  0.,  0.,  0.,
        -0.,  0.,  1., -0., -0.,  0.,  0.,  0.,  0., -1., -0.,  0., -1.,
         0., -0., -0., -0.,  0., -0.,  0.,  0., -0., -0., -0., -0., -0.,
        -0., -0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,
         0.,  0.,  0., -0., -1., -0., -0., -0.,  1., -0.,  0., -0., -0.,
        -0., -0., -0.,  0., -0., -0.,  0., -0., -0.],
       [-0., -0.,  0.,  0.,  1.,  0., -0., -0., -0.,  0., -0.,  0.,  0.,
        -0.,  0., -0., -0.,  1.,  0.,  0., -0.,  0.,  0., -0., -0.,  0.,
        -0., -0., -0., -0., -1.,  0., -0., -1.,  0.,  0.,  0.,  0.,  0.,
         1.,  0.,  1., -0., -0.,  0., -1.,  0., -0., -0., -0.,  0., -0.,
        -1.,  0., -0., -0., -1., -0., -0.,  1., -0.,  0., -0., -0.,  0.,
         0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,  1.,  0., -0., -0.,
         0.,  0., -0.,  0., -0.,  0.,  0., -0., -0.,  1.,  0.,  0., -0.,
         1.,  0.,  0., -0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,  0.,
        -0., -0., -0., -0.,  0., -0., -1., -0.,  0.,  0., -0., -0.,  0.,
         0., -0.,  0.,  0.,  0.,  0., -0., -0., -1.,  0.,  0.,  0., -0.,
        -0., -1.,  0., -0., -0., -1.,  0., -0., -0.],
       [ 0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0., -0.,  1., -1.,
         0., -0., -0., -0.,  0.,  0., -0.,  0., -1.,  0., -0., -1.,  0.,
         0., -0., -0., -1., -0., -0., -0.,  0.,  0.,  0., -0.,  0., -0.,
         0., -0.,  0., -0., -0., -1., -0., -0., -0., -0.,  0.,  0., -0.,
         0., -0., -0., -0., -0., -0.,  0.,  0., -0.,  1., -0., -0., -0.,
         0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0.],
       [-0., -0., -0.,  0.,  0., -0.,  0., -0., -0.,  0.,  1.,  1.,  0.,
        -0.,  0.,  0., -0., -0., -0.,  1., -0.,  0., -0.,  0.,  0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  1.,  0., -1., -0., -0.,  0., -0.,  0.,
        -0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  1.,
        -1.,  0.,  1.,  0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0.,  1.,
         0.,  0.,  0., -0., -0., -0.,  0., -0., -0.],
       [-0.,  0., -0., -0., -0.,  0., -0.,  0.,  0., -0.,  1., -0., -0.,
        -0., -0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,  0.,
        -0., -0., -0., -0.,  0., -0.,  0.,  0.,  0., -0., -1., -0., -0.,
         0.,  0.,  0., -0., -0.,  1.,  0., -0., -0.,  0.,  0., -0., -0.,
         0.,  0., -0., -0., -0., -0., -0., -0.,  0.,  0.,  0., -0., -0.,
         0.,  0., -0.,  0., -0.,  0., -0.,  1.,  0.],
       [ 0.,  0., -0.,  0.,  0.,  0., -0.,  1.,  0.,  1., -0.,  0., -0.,
         0.,  0., -1., -0.,  0.,  0.,  0.,  0., -0.,  1., -0., -0.,  1.,
         0.,  0., -0., -0., -0., -0.,  0., -0., -0., -0., -0., -0., -0.,
        -0., -0.,  1., -1., -0., -0., -0., -0., -1.,  0., -0.,  0., -0.,
         0., -0., -0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,
         0., -0., -0.,  0.,  0., -0., -0., -0., -0.],
       [-0., -0.,  0., -0., -1.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,
        -0., -0.,  0.,  0.,  0., -0., -0., -1.,  0., -0., -0.,  0., -0.,
        -0., -0., -0.,  0.,  1.,  0., -0.,  1., -0., -0.,  0., -0.,  0.,
         0.,  0.,  0.,  0., -0.,  0.,  1.,  0., -1.,  0.,  0., -0.,  0.,
        -0.,  0.,  0., -0., -0., -0.,  0., -1.,  0., -0.,  0., -0.,  0.,
         0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0., -0., -0.,
         0.,  0., -0., -0., -0.,  0.,  1., -1., -0., -1., -0.,  0.,  0.,
         0.,  0.,  0., -0., -0., -0.,  1., -0.,  0., -0., -0., -0., -0.,
        -0., -0.,  0., -0.,  0., -0.,  1., -1.,  0.,  0., -0.,  0., -0.,
         0., -0.,  0., -0.,  0., -0.,  0., -1., -0.,  0.,  0., -0., -0.,
        -0., -0.,  0.,  0., -0., -0., -0., -1., -0.],
       [-1., -1.,  0.,  0., -0., -1.,  0., -0.,  0., -0.,  0.,  0.,  0.,
        -1.,  0.,  0.,  0., -0., -0., -0.,  0.,  0., -0.,  0., -0., -0.,
        -1.,  0.,  0., -0.,  0.,  0.,  0., -0., -0., -0., -0., -0.,  0.,
         0.,  0., -0.,  0.,  1.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,
        -0.,  0.,  0., -0.,  1.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,
        -0., -0.,  0., -0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0., -0.,  0.,  0., -0., -0., -0., -0., -0., -0.,  0.,
         0.,  0., -0., -1.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,
         0., -0., -0.,  0., -0., -0., -1., -0.,  1.,  1.,  0.,  1., -0.,
         0., -0.,  0., -0., -0., -0., -0.,  0., -1., -1.,  0.,  0., -0.,
        -0., -0., -0., -0., -0., -0.,  0.,  0.,  0., -0., -0., -1., -0.,
        -0.,  0., -0.,  0., -0., -0., -0., -0.,  0.],
       [-0.,  1.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,  0.,  0., -0.,
        -0.,  0., -0.,  0., -0., -0., -0.,  0., -0., -0.,  0., -0.,  0.,
        -0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0.,  1.,  0., -0.,  0.,
        -0., -0., -0.,  0.,  0.,  0., -0., -1.,  0.,  0., -0.,  0., -0.,
         0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,
        -0., -0.,  0., -0.,  0.,  1.,  0.,  0., -0.],
       [ 0.,  0., -0.,  0., -0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,
         0.,  0., -0.,  0.,  1.,  0.,  0., -0., -0.,  0.,  0., -0.,  0.,
         0., -0., -1., -0.,  0.,  0.,  0.,  0., -0., -0., -0., -0., -1.,
        -0., -1.,  0., -0., -0., -0., -0., -0., -0.,  0., -0., -0., -0.,
        -0., -1.,  0.,  0., -0., -1., -0.,  0., -1.,  0., -0., -1., -0.,
         0.,  0.,  0., -0., -0.,  0., -0., -0., -0.],
       [-0., -0., -0., -0.,  1.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,
        -0., -0.,  0.,  0., -0., -0.,  0., -0.,  0., -0., -0.,  0., -0.,
        -0., -0.,  0.,  0., -0., -0.,  0., -0., -0., -0.,  0., -0.,  0.,
         0.,  0.,  0., -0., -0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,
         1.,  0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,  1., -1., -0.,
         0., -0., -0.,  0., -0.,  0.,  0., -0.,  0.],
       [ 0.,  1., -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,
         0.,  1., -0., -0.,  0.,  1.,  0., -0., -0.,  1., -0., -0., -0.,
         0.,  0., -0., -0.,  1.,  0.,  0.,  0.,  0., -0., -0., -0., -0.,
         0., -0., -0., -0.,  0., -1.,  0., -0., -0.,  1., -0., -0., -0.,
        -0., -0.,  1.,  0., -0., -0.,  0., -0., -0.,  0.,  1.,  0.,  0.,
        -0., -0., -0., -0., -0., -0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -1.,  0.,  0.,  0., -0., -0.,  0., -0., -1., -0.,
         0., -0., -0., -1.,  0.,  0., -0., -0., -0.,  0., -1.,  1.,  0.,
         0.,  0., -0.,  1., -0., -1., -0.,  0., -0., -0.,  0., -0., -0.,
        -0.,  0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,
         0., -0., -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,
         0., -0., -1.,  1., -0.,  0., -0.,  0., -0.],
       [-0., -0., -1.,  0., -0.,  0., -0.,  0.,  0., -0.,  0., -1.,  0.,
        -0.,  0.,  0., -1.,  0., -0.,  0.,  0.,  0., -0.,  0., -0., -0.,
        -0., -0., -0., -0.,  0., -1.,  0., -0., -0., -0., -0.,  0., -0.,
         0.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,  0., -1., -1.,  0.,
        -0., -0.,  0., -0., -0., -0., -1., -0.,  0., -0., -0.,  0., -1.,
         0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.],
       [ 0.,  0., -1., -0.,  0., -0.,  0., -0., -0.,  0.,  0.,  0., -0.,
         0., -1., -0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  1., -1.,  0.,
        -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0., -1.,  0.,
         0., -0.,  0.,  0., -0.,  0., -1.,  0., -0.,  0.,  0., -0.,  0.,
        -0., -0., -0.,  0., -1.,  1.,  0.,  0., -0.],
       [ 0.,  0.,  0.,  0.,  0., -1., -0., -0.,  1.,  0., -0.,  0.,  0.,
        -0., -0.,  0.,  0., -0., -0., -0.,  0., -0.,  0., -0., -0.,  0.,
         0.,  0.,  0.,  1., -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,
         0., -0., -0., -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,
         0., -1., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0., -0.,
         0., -0.,  0., -0., -0., -0., -0.,  0.,  0.],
       [ 0., -0., -1., -0., -0., -1.,  0.,  0., -0.,  0.,  0.,  0., -0.,
         0., -0., -0.,  0., -0., -0.,  0., -1.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,
        -0., -0., -0.,  1.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  1., -0., -0., -0., -0.,  1.,  0.,  0.,
        -0., -0., -0., -0.,  0., -0., -0.,  0., -1.],
       [-0., -0.,  0.,  0., -0.,  0., -0.,  0.,  0., -0., -1.,  0.,  1.,
        -1.,  0.,  1.,  0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,  0.,
        -0., -0., -0.,  0., -0., -0.,  0.,  0., -0.,  1., -0.,  0., -0.,
        -0.,  0.,  0., -0., -0.,  0., -1., -0., -0., -0.,  0.,  0.,  0.,
        -0.,  1., -0., -0., -0., -0., -0., -0.,  0., -0., -0., -0., -0.,
         1.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0.],
       [-0.,  0.,  0., -1., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,
         0.,  0., -0.,  0., -0., -0., -0.,  0., -0.,  0.,  0.,  1.,  0.,
         0., -0., -0., -0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,
        -0., -1., -0.,  0.,  1., -0., -0., -0.,  0., -0.,  0.,  0., -0.,
         0., -0., -0., -0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0.,
         0., -0.,  0., -0.,  0., -0., -0., -0.,  0.],
       [ 0.,  0.,  0.,  0.,  1., -0., -0., -0., -0., -0.,  0.,  0., -0.,
         0., -0., -0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,
         0.,  0.,  1.,  0.,  0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,
         0.,  0.,  0.,  0.,  1., -0., -0.,  0.,  0.,  0., -1., -1.,  0.,
        -0., -0.,  0.,  1., -0.,  0., -1., -0., -0.,  0.,  0.,  0.,  0.,
        -0., -0., -0., -1., -0.,  0., -0.,  0., -0.],
       [-1., -0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0.,
        -0.,  0.,  0., -0.,  0., -0., -0.,  0., -0., -0.,  0.,  0., -0.,
         0., -1., -0., -0., -0., -0., -0.,  0., -0.,  0., -0.,  0., -0.,
         0.,  1., -0., -0., -0.,  0., -0., -1., -0., -0.,  0.,  0., -0.,
         0.,  0., -0., -0., -0., -0., -1.,  0., -0., -1., -0., -0., -0.,
         0., -0.,  0.,  0.,  0., -0., -0., -1.,  0.]], dtype=np.float32))

In [ ]:
#@title Verification
n = 3
m = 4
p = 8
rank = 74

verify_tensor_decomposition(decomposition_348, n, m, p, rank)

##Rank-68 decomposition of <3,5,6> over Z


In [ ]:
#@title Data

decomposition_356 = (np.array([[-0.,  0., -0., -0.,  1.,  0., -0., -0., -0., -0.,  0.,  0.,  0.,
        -0.,  0.,  0., -0.,  0.,  1.,  1.,  0.,  0.,  0., -1.,  0.,  0.,
        -0.,  0., -0.,  0.,  0.,  0., -1., -0., -0., -1., -0., -0., -0.,
        -0.,  0.,  1.,  0., -1., -1., -0., -1.,  0.,  1.,  1., -0.,  0.,
        -0., -0.,  0., -0., -0.,  0., -1., -1.,  0.,  0., -0., -0.,  0.,
         0.,  0.,  0.],
       [-0.,  1.,  0.,  0.,  1.,  1., -0.,  0., -0.,  0.,  0.,  0., -0.,
        -0.,  0., -1., -0.,  0.,  1., -0., -0.,  0., -0., -0., -0.,  0.,
        -0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0., -0., -0., -0., -0.,
         0., -0.,  1.,  0., -0., -1.,  1.,  0.,  0.,  1.,  1.,  1.,  0.,
         0., -0., -0.,  1., -0.,  0., -1., -0., -0., -0., -0.,  1.,  1.,
        -0., -0., -0.],
       [-0.,  0.,  0.,  0., -0., -0., -0., -0., -0., -0.,  0.,  0.,  0.,
         0., -1., -0.,  0., -0.,  0., -1., -0.,  0.,  0.,  1.,  0.,  1.,
        -0., -1., -0.,  0.,  1.,  0.,  1.,  0.,  0.,  1., -0., -0., -0.,
         0., -0.,  0.,  0., -0., -0.,  0.,  1.,  0., -0.,  0., -0.,  0.,
        -0., -0.,  0.,  0.,  1., -0.,  0.,  1., -1.,  0.,  0., -0.,  1.,
         0.,  1., -0.],
       [-0.,  0., -0., -0., -0.,  0., -0.,  0.,  0.,  1., -0.,  1.,  1.,
         0., -0., -0.,  0., -0.,  0., -0.,  0., -1., -1.,  0.,  0., -0.,
         0., -0.,  0., -0., -0.,  0., -0.,  0.,  0., -0., -0., -0., -1.,
        -0., -0., -0.,  0.,  1., -0., -0.,  0.,  1.,  0., -0., -0.,  0.,
        -1.,  1., -1., -0., -0., -0., -0., -0., -0.,  0.,  0.,  0.,  1.,
        -0.,  0., -0.],
       [ 0., -0.,  0.,  0., -1.,  0., -0.,  0.,  0.,  1.,  0.,  1.,  0.,
        -0., -0.,  1.,  0.,  0., -1.,  0., -0., -0.,  0.,  1.,  0.,  1.,
         0., -1., -0., -0.,  1.,  0.,  0.,  0., -0., -0., -0., -0.,  0.,
        -0.,  0., -1.,  0., -0.,  1.,  0.,  0., -0., -0., -1.,  0.,  0.,
        -0., -0.,  0., -0.,  1.,  0.,  1.,  0., -1., -0., -0.,  0., -0.,
        -0.,  1., -0.],
       [-1.,  0.,  0.,  0., -0., -0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0., -0.,  0., -0., -0., -0., -0.,  0., -0., -0., -0., -1.,
        -0.,  0.,  0., -0.,  0.,  0.,  1., -0.,  1., -0.,  1.,  0., -0.,
         0., -0., -1.,  0.,  1.,  0.,  0.,  0., -0.,  0., -1.,  1.,  1.,
        -0., -0., -0.,  0., -0., -1., -0.,  1., -0., -0.,  0., -0., -0.,
         0., -0.,  0.],
       [-0., -1., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -1., -0.,  0.,
         0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,
        -0., -0.,  0., -0., -1., -0.,  0., -0.,  0., -1., -0.,  0., -0.,
        -1.,  0., -1.,  1., -0.,  1., -1., -0., -0., -0.,  0., -0.,  0.,
         0., -0.,  0.,  0., -0., -1., -0.,  0., -0., -0.,  0.,  0., -1.,
         0., -0.,  1.],
       [ 0., -1., -0.,  0., -0., -0.,  1., -0.,  0.,  0.,  0.,  0.,  0.,
        -0.,  0.,  0., -0., -0., -0., -0.,  0., -0., -0.,  0.,  0., -0.,
         0.,  1.,  0.,  0., -1., -0., -1., -0., -0., -1., -0.,  0., -0.,
        -1.,  0.,  0.,  1., -0.,  1., -0., -0.,  0.,  0., -0.,  0.,  0.,
        -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0.,  1., -0.,  0., -1.,
         0.,  0.,  1.],
       [ 0.,  0., -0.,  1., -0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,
        -1.,  0.,  0.,  1., -0., -0., -0., -1., -0., -0., -0.,  1.,  0.,
        -0., -1.,  0., -1., -0.,  0.,  1., -1., -0.,  0., -0.,  0., -0.,
        -0.,  0., -1.,  0.,  0., -0., -1., -0., -0.,  0., -0.,  0., -0.,
        -0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,  0., -1.,  0.,  0.,
         0., -0., -0.],
       [ 0., -0., -1.,  0., -0., -0.,  1.,  0., -0., -1.,  1., -1., -0.,
        -0.,  0.,  0., -0.,  0.,  1., -1., -0., -0., -0., -0.,  0.,  0.,
         0.,  1., -1.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,
         0.,  0.,  1.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0., -0.,  0., -1.,  0.,
        -0., -1., -0.],
       [-0.,  0., -0., -0.,  0., -0., -1., -0., -1., -0.,  0.,  0., -1.,
         1.,  0., -0., -0.,  1.,  0., -0., -0., -0.,  1.,  0.,  0., -1.,
        -0.,  0., -0.,  0., -0., -0., -0., -1.,  0.,  0.,  1., -0., -0.,
         0.,  0., -1.,  0., -0.,  0.,  0., -0.,  0., -0., -1., -0.,  1.,
         0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0., -1.,
        -1., -0., -1.],
       [ 1., -0.,  0.,  0.,  0.,  0., -0., -0., -1.,  0., -1.,  0.,  0.,
         0.,  0.,  0.,  0.,  1.,  0., -0., -0.,  0.,  1.,  0., -1.,  0.,
         0.,  0.,  0.,  0., -1., -0., -0., -1.,  0.,  0., -0.,  0.,  0.,
         0., -1., -1.,  1.,  0.,  1., -0.,  0., -1., -0.,  0., -0.,  0.,
         0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0., -0., -0.,  0., -1.,
        -0., -0.,  0.],
       [ 0., -0.,  0.,  0., -0.,  0.,  1., -1., -0., -0., -0.,  0.,  1.,
        -1.,  0.,  0.,  0., -0., -0., -0.,  0., -1.,  0., -0.,  0.,  0.,
        -1.,  1.,  0.,  1., -1.,  0., -0., -0., -0.,  0., -1.,  0., -0.,
         0.,  0., -0.,  0.,  1.,  1.,  0., -0.,  0., -0.,  0.,  0., -0.,
        -0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0.,
         1.,  0.,  1.],
       [-0., -0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0., -1.,
         0.,  0., -0., -0.,  1.,  0.,  0., -1.,  1.,  1.,  0., -0., -0.,
         1., -1.,  0., -1.,  0.,  0.,  0., -1., -0.,  0., -0., -0.,  0.,
        -0.,  0., -1., -0., -1.,  0.,  0., -0., -1., -0.,  0.,  0., -0.,
        -0.,  0., -0., -0.,  0., -0., -0.,  0.,  0., -0., -1.,  0., -1.,
         0.,  0., -0.],
       [-0., -0.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,
         0., -0., -1.,  0.,  0.,  0.,  0., -1., -0., -0., -1., -0., -1.,
        -0.,  0., -0., -0., -1., -1.,  0., -0., -0.,  0., -0., -1., -1.,
        -0.,  0.,  0.,  0.,  0., -1.,  0.,  0., -0.,  0.,  1.,  0.,  0.,
        -0., -0., -1., -0., -1., -0., -1.,  0.,  0.,  0., -1.,  0., -0.,
         0., -0., -0.]], dtype=np.float32), np.array([[-0.,  0., -0., -0., -0., -0.,  0.,  0., -0.,  0., -0., -0., -1.,
        -0.,  1.,  0., -0., -0.,  0.,  0.,  0.,  1.,  0.,  0., -0.,  0.,
         0.,  0., -0., -0.,  0.,  0., -0., -0., -0., -0.,  0., -0., -0.,
        -0., -0.,  0., -0., -1.,  0., -0., -1.,  0.,  0.,  0.,  0.,  0.,
         0., -0.,  0., -0.,  0.,  0., -0.,  1.,  0., -0., -0., -0., -0.,
        -0., -0., -0.],
       [ 1., -0., -0., -0.,  0., -0., -0.,  0., -1.,  0.,  0.,  0., -0.,
        -0.,  0., -0.,  0.,  0., -0.,  0., -0., -0., -0.,  0.,  0., -0.,
        -0., -0.,  0.,  0., -0., -0., -0., -0.,  1., -0., -0.,  0., -0.,
        -0.,  1., -0.,  0.,  0., -0., -0.,  0., -0., -1., -0.,  1., -0.,
        -0., -0., -0.,  1.,  0.,  0.,  0., -0., -0.,  0.,  0., -0.,  0.,
        -0., -0.,  0.],
       [-0., -0., -1.,  1., -0.,  0., -0.,  0., -0.,  0.,  1.,  1.,  0.,
         0.,  0., -0., -0., -0., -1.,  0., -0.,  0., -0., -0., -1.,  0.,
        -0., -0., -0.,  0., -0., -1.,  0., -1.,  0.,  0.,  0.,  0.,  0.,
        -0., -0., -1.,  0.,  0., -0., -1., -0., -0., -0., -0., -0., -0.,
         0.,  1., -1.,  0.,  0., -1., -0., -0.,  0.,  0.,  1.,  1.,  0.,
        -0., -0., -0.],
       [ 0.,  0., -0.,  0.,  0.,  0., -0., -1., -0.,  0., -0., -0.,  0.,
         0., -1.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0.,
        -0., -0., -0.,  0., -0.,  0.,  0.,  0., -1.,  0.,  1.,  0., -0.,
        -0.,  0.,  0., -0., -0.,  0.,  0.,  1.,  0.,  0.,  0.,  0., -0.,
        -0., -0., -0., -0.,  0., -0.,  0., -1.,  0.,  0.,  0., -0., -0.,
         1.,  0.,  0.],
       [-0., -0.,  0., -0.,  0., -0., -0., -1., -0., -0.,  0.,  0., -0.,
        -0.,  0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  1., -0., -1.,
         0.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,  0.,  1.,  0., -0.,
         0., -0.,  0., -0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  1.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,
         1.,  0., -0.],
       [ 1., -0.,  0.,  0.,  0., -0., -0., -0., -1., -0.,  0.,  0., -0.,
        -0., -0.,  1., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,
         0.,  0., -0., -0., -0., -0., -0.,  0., -0., -0.,  0.,  0., -0.,
         0.,  1.,  0.,  0.,  0., -0., -0., -0.,  0.,  0., -1.,  0., -1.,
         0., -0., -0., -0.,  0.,  0., -1.,  0., -0., -0.,  0., -0., -0.,
        -0., -0., -0.],
       [-0.,  0., -0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  2.,
         1., -1., -0.,  1.,  0.,  0., -0., -0., -1.,  1., -0., -0., -0.,
         0., -0.,  0., -0., -0.,  0.,  1., -0., -0., -1.,  0.,  0., -0.,
         0.,  0.,  0.,  0.,  1., -0.,  0.,  1.,  0.,  1., -0.,  0., -0.,
        -0., -0., -0.,  0., -0.,  0., -0., -1., -0.,  0., -0., -0.,  1.,
        -1.,  0., -0.],
       [-1., -0., -0., -0., -0., -0., -0., -0., -0.,  0., -0., -0., -0.,
         0., -0., -0., -0., -0., -0.,  0.,  0., -0.,  0., -0., -0., -0.,
        -0., -0., -0.,  0., -0., -0.,  0., -0., -1., -0.,  0., -0., -0.,
        -0., -1.,  0., -0.,  0., -0., -0.,  0.,  0.,  0., -0., -1.,  0.,
        -0., -0.,  0., -1., -0., -1., -0.,  0., -0.,  0., -0., -0., -0.,
        -0.,  0., -0.],
       [-0.,  0.,  1., -1.,  0., -0., -0.,  0., -0.,  0., -1.,  0.,  0.,
         0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  1., -0.,
        -0.,  0., -0., -0.,  0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0.,  0., -0., -0.,  1.,  0., -0.,  0., -0.,  0., -0.,
        -0.,  0.,  0., -0., -0., -0., -0., -0.,  0.,  0., -0., -1., -0.,
        -0.,  0., -0.],
       [-0., -1., -0., -0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,
         0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0., -0., -0.,  0., -0.,
         0.,  0., -0., -0., -0.,  0.,  0., -0., -0.,  0., -0., -0., -0.,
         1., -1., -0., -1., -0.,  0., -0., -0.,  0., -0.,  0., -0.,  0.,
        -0.,  0.,  0., -1., -0.,  0., -0.,  0., -0.,  1.,  0., -0.,  0.,
        -0.,  0., -0.],
       [ 0., -0., -0.,  0., -0.,  0.,  1.,  1., -1.,  0., -0.,  0., -0.,
        -0., -0., -0., -0., -0.,  0., -1., -0.,  0.,  0., -2.,  0.,  1.,
         0.,  0.,  1., -0.,  0.,  0., -0.,  0., -0.,  0., -1.,  0., -0.,
        -0., -0., -0.,  0., -0., -1., -0.,  1., -0.,  0., -0., -0., -1.,
         0., -0., -0.,  0., -1., -0., -1., -0., -0., -0., -0.,  0.,  0.,
        -1.,  0.,  1.],
       [-0., -0.,  1.,  0.,  0.,  1.,  0.,  0., -0.,  0., -1., -0.,  0.,
        -0., -0.,  0.,  0., -0., -0., -0.,  0., -0., -0., -0.,  0., -0.,
        -0., -0., -0.,  0.,  0., -0., -0.,  0.,  0., -0.,  0., -0.,  0.,
        -0., -1.,  0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,
         0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0., -1., -0.,
        -0.,  0., -0.],
       [ 0.,  0., -0., -0., -0.,  0., -0.,  0.,  0., -0., -0.,  0., -1.,
        -1.,  1.,  0., -1., -0.,  0., -0., -0.,  1., -0.,  0.,  0.,  0.,
        -0.,  0.,  0., -0., -0.,  0., -1., -0.,  0.,  0.,  0.,  0.,  0.,
        -0., -0., -0.,  0., -1.,  0.,  0., -0., -0.,  0.,  0.,  0., -0.,
        -0.,  0.,  0., -0., -0.,  0., -0.,  1.,  0.,  0., -0., -0.,  0.,
         1., -0., -0.],
       [ 1., -0.,  0.,  0.,  0., -0., -0.,  0., -1., -0., -0., -0.,  0.,
        -0., -0., -0.,  0., -0.,  0., -0., -0., -0.,  0., -0., -0.,  0.,
         0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  1.,  1.,  0.,  0., -0.,
        -1.,  1., -0., -0., -0., -0.,  0., -1.,  0., -1., -0.,  1.,  0.,
        -0., -0., -0.,  1., -0.,  1.,  0., -0.,  0.,  0.,  0.,  0., -0.,
        -1.,  0.,  1.],
       [ 0.,  0., -0., -0., -0., -0.,  0.,  0.,  0., -0.,  0., -1.,  0.,
        -0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0.,
         0.,  1.,  0.,  1.,  0.,  1., -0., -0.,  0., -0.,  0., -0.,  0.,
         0., -0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0., -0.,  0., -0.,
        -0., -1.,  1., -0., -0.,  0.,  0., -0., -0., -1., -1., -0., -0.,
        -0.,  1., -0.],
       [-0.,  0., -0., -0., -0., -0., -0., -1.,  0.,  0., -0., -0., -0.,
        -0., -1.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0., -0.,
         0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0.,
         0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0.,
        -0., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -1.,  0.,  0., -0.,
        -0., -0.,  0.],
       [-0.,  0.,  0.,  0., -0., -0., -1., -1., -0., -0.,  0., -0.,  0.,
         0., -0., -0., -0.,  0., -0.,  1.,  0., -0., -0.,  1., -0., -1.,
        -0.,  0., -1.,  0., -0.,  0., -0.,  0.,  0., -0.,  1., -0., -0.,
         0.,  0., -0.,  0., -0.,  0.,  0., -1.,  0., -0., -0., -0.,  1.,
         0.,  0.,  0.,  0.,  1.,  0., -0., -0., -0., -0.,  0.,  0., -0.,
         0., -0.,  0.],
       [ 0.,  0., -1., -0.,  0., -1., -0., -1.,  0., -0.,  1.,  0.,  0.,
        -0., -0., -1., -0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,
         0.,  0., -0., -0., -1., -0., -0., -0.,  0., -0.,  0., -0.,  0.,
         0., -0.,  0., -1., -0., -0., -0., -0.,  0.,  0.,  0., -0., -0.,
         0.,  0.,  0., -0.,  1., -0., -0., -0., -0.,  0., -0.,  1., -0.,
        -0., -0., -0.],
       [ 0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0., -1.,
        -1.,  0.,  0., -1.,  0.,  0.,  0., -0., -0., -0.,  0.,  0., -0.,
        -0., -0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,
        -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,
         1., -0., -0., -0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0., -0.,
         1., -0.,  0.],
       [ 0.,  0., -0.,  1.,  0., -0.,  0.,  0.,  1.,  0., -0., -0., -0.,
        -0., -0.,  0., -0., -1., -0., -0.,  0.,  0., -1.,  0.,  0.,  0.,
         0.,  0., -0., -0.,  0.,  0.,  0., -1.,  0.,  0.,  0., -0.,  0.,
         0.,  0.,  0., -0., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,
        -1., -0., -0., -0., -0., -0.,  0., -0., -0.,  0., -0.,  0.,  0.,
        -0., -0.,  0.],
       [-0.,  0., -0.,  1., -0.,  0., -0., -0.,  0., -0., -0., -0.,  0.,
         0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0., -0., -0., -0.,
         0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,  0.,
         0., -1.,  1.,  0., -0., -0., -0., -0.,  0.,  0., -1., -0., -0.,
         0.,  0.,  0.],
       [ 0.,  0., -0.,  1., -0.,  0., -0., -1., -0.,  0., -0.,  0.,  0.,
        -0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  1., -0.,  0.,  0., -0.,
        -1.,  0., -0., -1.,  0., -0., -0.,  0., -0.,  0., -0., -0., -0.,
         0., -0.,  0., -0., -0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0.,
        -1.,  0.,  0., -0.,  0.,  0., -0.,  0., -0., -0., -0., -0., -0.,
         0., -0., -0.],
       [-0., -0., -0., -0.,  0.,  0.,  0., -0.,  0., -0., -0., -0.,  1.,
         1.,  0., -0., -0.,  0., -0.,  0.,  1.,  0.,  0., -0., -0., -0.,
        -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0., -0.,  0.,  1.,  1.,
         0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  0.,
         0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0.,
        -1.,  0., -0.],
       [-0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0., -0., -0., -0.,  0.,
         0., -0.,  0., -0.,  0., -0.,  0., -0., -0.,  0., -0.,  1., -0.,
        -0.,  0.,  0.,  0., -0.,  1.,  0., -0., -0., -0., -0., -0., -0.,
         0.,  1., -0., -0.,  0., -0., -0.,  0., -1., -0.,  0.,  0.,  0.,
         0.,  0.,  1., -0.,  0.,  0., -0.,  0.,  0., -0., -1.,  0.,  0.,
         0., -0.,  0.],
       [ 0., -0., -0.,  0., -0., -0.,  0., -0., -0.,  1.,  0., -0., -0.,
        -0., -0., -0., -0.,  0.,  0.,  1.,  0., -0., -0.,  1.,  0.,  0.,
        -0.,  0., -0.,  0.,  0., -0., -0., -0., -0., -0.,  0., -0., -1.,
        -0.,  0., -0., -0., -0.,  0.,  0., -1., -0., -0.,  0.,  0., -0.,
        -1.,  0., -0., -0.,  0.,  0., -0., -0., -0., -0., -0., -0.,  0.,
         0., -0.,  0.],
       [ 0.,  0.,  1.,  0., -1., -0.,  0.,  0.,  0., -0., -0., -0.,  0.,
        -0., -0.,  0.,  0., -0.,  1., -0.,  0., -0.,  0.,  0., -0.,  0.,
         0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0., -0.,  0., -1.,  0.,
         0., -0.,  0., -0., -0.,  0., -0., -0., -0., -1.,  0.,  0., -0.,
         0., -0., -0., -0.,  0., -0.,  1., -0.,  0.,  0.,  0.,  0., -0.,
        -0.,  0.,  0.],
       [-0.,  0.,  1., -0., -0., -0.,  0., -0.,  0.,  0., -0.,  1., -0.,
        -0.,  0., -0., -0.,  0., -0., -0.,  0.,  0., -0.,  0., -0., -0.,
         0., -0.,  0.,  0.,  0., -1., -0., -0.,  0.,  0.,  0.,  0.,  0.,
         0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0.,
         0.,  1., -1.,  0., -0., -0., -0., -0., -0.,  0., -0., -0.,  0.,
        -0.,  0.,  0.],
       [-0.,  0.,  1., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0., -0.,
        -0.,  1.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,  0.,  0.,  0.,
        -0.,  0., -0., -0.,  0., -1.,  0.,  0., -0.,  0., -0., -0.,  0.,
         0., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0.,
        -0.,  0.,  0., -0.,  1.,  0., -0.,  0., -1., -0.,  0.,  0., -0.,
        -0.,  1.,  0.],
       [ 0.,  0., -0., -0.,  0., -0., -0., -0.,  0., -0., -0., -0.,  0.,
         0.,  0., -0., -0.,  0., -0., -1., -0.,  0., -0., -1.,  0., -0.,
         0.,  0.,  1., -0.,  0.,  0., -0., -0.,  0., -0., -0., -1.,  0.,
         0., -0.,  0., -0., -0.,  0.,  0.,  1., -0., -0., -0., -0., -0.,
         0.,  0., -0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,
         0., -0.,  0.],
       [-0., -0.,  1.,  0.,  0.,  1.,  0.,  0., -0., -0., -0.,  0., -0.,
         0., -0.,  1., -0., -0.,  0., -0., -0.,  0.,  0., -0.,  0., -0.,
         0., -0., -0., -0., -0., -1.,  0., -0.,  0.,  0., -0.,  0.,  0.,
         0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0.,
        -0., -0.,  0.,  0., -0.,  0., -0., -0.,  0., -0.,  0., -1., -0.,
        -0., -0., -0.]], dtype=np.float32), np.array([[ 0.,  1., -0., -0.,  0.,  0., -0.,  0., -0.,  1.,  0.,  0.,  0.,
        -0., -0.,  0.,  0., -1., -0., -1., -0., -1.,  1., -0.,  0.,  0.,
        -1.,  0., -1.,  0., -0.,  0., -0., -0., -1.,  1., -0., -0.,  0.,
         1., -0., -0., -0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,
        -1., -0.,  0., -1., -0.,  0., -0.,  1., -0.,  0.,  0., -0.,  1.,
         0., -0.,  0.],
       [ 0.,  1.,  0., -0., -0., -0.,  0., -0.,  0.,  0., -0.,  0.,  0.,
        -0., -0., -0., -1., -0.,  0., -1.,  0., -0., -0.,  0., -0., -0.,
        -0., -0., -1., -0.,  0., -0.,  1.,  0., -1.,  1.,  0.,  0.,  0.,
         1., -0.,  0.,  0.,  0., -0.,  0.,  1.,  0., -0., -0.,  1.,  0.,
         0., -0., -0., -1.,  0.,  0., -0.,  1., -0., -0., -0.,  0., -0.,
         0., -0., -0.],
       [ 0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  1.,
         1.,  0., -0.,  1., -1.,  0., -0., -1., -1.,  1.,  0., -0., -0.,
        -1., -0.,  0., -0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  1.,
         0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,
        -1.,  0., -0., -0.,  0., -0., -0., -0., -0., -0.,  0.,  0., -0.,
        -0.,  0., -0.],
       [-0.,  1.,  0., -0.,  1.,  0.,  0., -0.,  0.,  0., -0., -0.,  0.,
        -0., -0., -0., -0., -1., -0., -0.,  0.,  0.,  1.,  0.,  0., -0.,
         0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,  1.,  0., -0.,  0.,
         1.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0., -1., -0.,  0., -0.,
         0.,  0., -0., -1., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  1.,
        -0.,  0., -0.],
       [ 0.,  1., -0.,  0.,  1., -0., -0.,  0., -0.,  0.,  0.,  0., -0.,
         0., -0., -0.,  0.,  1.,  1.,  0., -0.,  0.,  0., -0.,  0., -0.,
         0., -0.,  0., -0., -0., -0.,  0.,  1., -0., -0., -0., -0.,  0.,
         1.,  0., -1.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,  1., -0.,
         0., -0., -0., -1.,  0.,  1.,  0.,  0., -0., -0.,  0., -0.,  0.,
         0., -0.,  0.],
       [-1., -1., -0., -0.,  1.,  0., -0., -0.,  1.,  0., -0.,  0., -0.,
         0., -0., -0.,  0., -1.,  0., -0., -0., -0.,  0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,  0., -0.,  0.,
        -1.,  0., -0., -0., -0.,  1., -0.,  0.,  0.,  0., -1., -1., -0.,
        -0., -0., -0.,  1., -0.,  0., -1.,  0.,  0.,  0.,  0., -0., -0.,
         0.,  0.,  1.],
       [ 0., -0., -1., -0., -1., -1.,  0., -0.,  0., -0.,  0.,  1.,  0.,
        -0., -0.,  0.,  0., -0., -1.,  0.,  0.,  0., -0., -0., -0.,  0.,
        -0., -0., -0., -0., -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,
         0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,
         0., -1.,  0.,  0., -0.,  0., -0.,  0., -1.,  0., -0., -1., -0.,
         0.,  1.,  0.],
       [ 0., -0., -1.,  1., -1., -1.,  0., -0.,  0., -0.,  0.,  0.,  0.,
        -0., -0., -0., -0., -1., -1.,  0.,  0., -0., -0., -0., -0.,  0.,
        -1.,  1.,  0., -1., -0.,  0.,  0., -1.,  0., -0.,  0., -0., -0.,
        -0., -0.,  1., -0., -0., -0., -1.,  0., -0.,  0., -0., -0.,  0.,
         0., -0.,  0., -0., -0., -0., -0., -0., -1.,  0., -0., -1.,  0.,
        -0.,  1.,  0.],
       [ 0., -0., -0., -1.,  0.,  0., -0.,  0., -0.,  0., -0.,  0., -0.,
         0., -0.,  0., -0.,  1., -0.,  0., -0., -0.,  0.,  0., -1., -0.,
         1.,  0.,  0.,  1., -0.,  0.,  0.,  1., -0.,  0.,  0.,  0.,  0.,
         0., -0., -0.,  0., -0.,  0., -0., -0., -1.,  0.,  0., -0.,  0.,
        -0., -1.,  1., -0.,  0.,  0., -0., -0.,  0., -0.,  1., -0., -0.,
        -0.,  0.,  0.],
       [ 0.,  0.,  0., -0., -0.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,
        -0.,  1., -0., -0., -0., -0.,  0., -0., -1.,  0., -0., -0.,  0.,
        -1.,  0., -0.,  0.,  0., -0.,  0., -0., -1.,  0., -0.,  0.,  0.,
         0.,  0., -0., -0.,  1.,  0., -0., -0., -0.,  0., -0.,  1.,  0.,
        -0.,  0., -0., -1., -0.,  0.,  0.,  1.,  1.,  0.,  0.,  0., -0.,
         0.,  0.,  0.],
       [ 0.,  1., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,
         0., -0., -0., -0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0., -0.,
         1., -1.,  0.,  1.,  0., -0., -0., -0., -1., -0., -0., -0.,  0.,
         0.,  0., -0., -0.,  0., -0., -0.,  0., -0.,  0., -0.,  1., -0.,
        -0., -0., -0., -1.,  0., -0.,  0.,  0.,  1., -1., -0., -0.,  0.,
        -0., -1.,  0.],
       [-0., -1.,  0.,  0., -0., -0., -0.,  1., -0., -0., -0., -0., -0.,
         0.,  0., -0., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0., -1.,
        -1., -0., -0., -0., -1., -0.,  0., -0.,  1., -0.,  1.,  0.,  0.,
        -0., -0.,  0., -1.,  0.,  0.,  0.,  0., -0., -0.,  0., -1.,  0.,
         0.,  0., -0.,  1., -1.,  0., -0.,  0.,  1.,  0., -0.,  0., -0.,
         0., -0., -0.],
       [-0.,  0.,  0.,  0., -1.,  0., -0.,  0., -0.,  1.,  0., -0.,  0.,
        -0., -0.,  0., -0.,  0.,  0., -1., -0.,  0., -0., -1., -0., -1.,
         0.,  0., -1.,  0., -0.,  0.,  0., -0.,  0.,  0., -0.,  1., -1.,
         0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0.,  1., -0.,  1.,
        -0., -0.,  0., -0.,  0., -0.,  1., -0.,  0.,  0., -0., -0.,  0.,
         0.,  0.,  0.],
       [ 0., -1., -0.,  0., -0.,  0., -1., -0.,  0., -0., -0., -0., -0.,
        -1.,  0.,  0., -1.,  0.,  0.,  0., -0.,  0.,  0., -0., -0.,  0.,
         0., -0., -1., -0.,  0., -0., -0.,  0.,  1.,  0.,  1.,  0., -0.,
        -1., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -1.,  0.,
        -0., -0., -0.,  1., -0., -0.,  0., -0.,  0.,  0.,  0., -0., -0.,
         1.,  0.,  1.],
       [ 0.,  1.,  0.,  0., -1.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,
         1., -0., -0.,  1.,  0.,  0., -0., -1.,  0., -0.,  0., -0., -0.,
         0., -0., -0., -0.,  0.,  0.,  0., -0., -1., -0., -1.,  1., -0.,
         1., -0., -0., -0.,  0., -1.,  0., -0., -0.,  0.,  1.,  1.,  1.,
        -0., -0., -0., -1.,  0.,  0.,  1., -0., -0., -0., -0., -0., -0.,
        -1., -0., -1.],
       [ 0., -0.,  0., -0.,  0.,  1.,  0.,  0.,  0.,  0.,  0., -0., -0.,
         0., -0.,  1.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,  0.,  1.,
        -0.,  0., -0., -0., -0.,  1., -0.,  0., -0.,  0.,  0., -0., -0.,
        -0., -0., -0.,  0., -0., -0.,  0., -0., -0.,  0., -1.,  0., -1.,
        -0.,  1., -1.,  0.,  1.,  0.,  0.,  0., -1.,  0., -0.,  0.,  0.,
        -0.,  0.,  0.],
       [-1., -1.,  0., -0., -0.,  1., -0.,  0.,  0., -0.,  1.,  0.,  0.,
        -0., -0., -0., -0., -0.,  0., -0.,  0.,  0., -0.,  0.,  1., -0.,
        -0., -0.,  0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0.,
         0., -1., -0., -1.,  0., -0.,  1.,  0.,  0.,  0.,  0., -1., -0.,
        -0., -0., -0.,  1.,  0., -0., -0., -0., -0., -0., -0.,  1.,  0.,
         0.,  0., -0.],
       [ 1.,  1.,  0., -0.,  0., -0.,  0., -0.,  0.,  0., -0., -0.,  0.,
        -0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0., -0.,  1.,
        -0.,  0., -0., -0.,  1.,  1., -0.,  0.,  0., -0.,  0.,  0., -0.,
         0.,  1.,  0.,  1.,  0., -0.,  0.,  0.,  1.,  0.,  0.,  1., -1.,
         0.,  1., -1., -1.,  1., -0.,  0.,  0., -1.,  0.,  0., -0., -0.,
        -0.,  0.,  0.]], dtype=np.float32))

In [ ]:
#@title Verification
n = 3
m = 5
p = 6
rank = 68

verify_tensor_decomposition(decomposition_356, n, m, p, rank)

## Rank-80 decomposition of <3,5,7> over Z

In [ ]:
#@title Data

decomposition_357 = (np.array([[ 0., -0., -0., -0., -0., -0., -0.,  0., -0.,  0., -0.,  0., -1.,
         0., -1., -0.,  0., -0., -0.,  1.,  0., -0., -0., -0., -0., -0.,
         0., -0.,  0.,  0.,  0., -1., -0.,  0.,  0.,  0.,  0.,  1., -0.,
        -0.,  0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  1., -0.,
        -0.,  0., -1.,  0.,  0., -0.,  0.,  0., -0.,  1., -0.,  0.,  1.,
         0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0., -1.,  1., -0.,
         0.,  1.],
       [-1., -1., -0.,  0., -0.,  0., -0.,  0., -1., -0., -0., -1.,  1.,
         1., -0., -0.,  0., -1., -0., -0., -0., -0.,  0.,  1.,  0.,  0.,
         0., -1.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0.,
         0., -0., -1.,  0., -0.,  0., -1.,  0., -0., -0.,  0.,  0.,  0.,
        -0., -0., -0.,  0., -0., -0., -1., -0., -0.,  0., -0.,  0., -0.,
         0.,  0., -1.,  0., -0.,  1.,  1., -0., -0., -1., -0., -0., -0.,
        -0., -1.],
       [ 0.,  0.,  1.,  1.,  0.,  0.,  0.,  0., -0., -0.,  0., -0.,  1.,
         0.,  0.,  1., -0.,  0.,  0.,  1., -0., -0., -1., -0.,  0.,  0.,
         0., -0.,  0., -0.,  1., -0.,  0.,  0.,  1., -0., -0.,  0.,  1.,
         0., -0.,  0., -0., -1.,  0.,  0., -0.,  1., -0.,  0., -0., -1.,
        -0., -0., -0., -0., -0.,  0.,  0., -0., -1., -0., -0.,  0.,  0.,
        -0., -0., -0., -0., -0., -0.,  0., -0., -0.,  0., -0., -0.,  0.,
        -1., -0.],
       [-1., -0., -0., -0.,  0.,  0., -0., -0., -1.,  0., -0., -0.,  0.,
         1., -1., -0.,  0., -0.,  0., -0., -0.,  0.,  0.,  1.,  0., -0.,
         0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0., -0., -0.,  1., -0.,
         0., -0., -1.,  1.,  1.,  0.,  0.,  0.,  0., -0., -0.,  1., -0.,
         0., -0., -1., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  1.,
        -0.,  1., -0., -0., -0.,  0.,  0.,  0.,  0., -1., -1.,  1., -0.,
         0., -0.],
       [ 1., -0., -1., -0., -0., -0.,  0., -0.,  1.,  0.,  0., -0.,  0.,
        -1.,  1., -1.,  0., -1., -0., -0., -0., -0., -0., -0.,  0.,  0.,
         0., -1., -0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,
        -1.,  0.,  1., -0., -0., -0., -1.,  1.,  0., -0.,  0.,  0.,  1.,
         0., -0.,  0.,  0., -0., -0.,  0.,  0.,  1., -0.,  0., -0., -1.,
         0., -1.,  0.,  0., -0.,  1., -0.,  0., -0.,  1., -0., -1.,  0.,
        -0.,  0.],
       [-0.,  0., -0., -0.,  0.,  1., -0., -0.,  0.,  0., -0., -0., -0.,
         0.,  1.,  0., -0., -0.,  1., -0.,  0.,  0.,  0.,  0., -1., -0.,
         0., -0., -1., -0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,
         0.,  0., -0.,  0.,  0.,  1.,  0., -0.,  0.,  0., -1.,  0.,  0.,
         0., -0., -0., -1.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0.,
        -0., -0., -0.,  0., -0., -0., -0.,  0., -0.,  0., -0., -0.,  0.,
        -0., -0.],
       [ 0.,  0., -0.,  0., -0., -1., -0., -1., -0., -0., -0.,  0., -0.,
         0.,  0., -0.,  0.,  0.,  0.,  0., -0., -1., -0., -0.,  0., -0.,
        -1.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., -1.,  0.,  0.,
         0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -0., -0.,  0.,  1.,  0., -0., -0., -0., -0., -0., -0.,  0.,  0.,
         1., -0.,  0., -1.,  1., -1.,  0., -0., -0., -0.,  0., -0., -1.,
         0.,  0.],
       [-0., -0., -0., -0., -1.,  0., -1., -1.,  0.,  0., -0.,  0., -0.,
         0., -0., -1., -0., -0.,  1.,  0., -0., -1.,  0.,  0., -0., -1.,
         0.,  0.,  0., -1.,  0.,  0.,  1.,  0.,  0.,  0., -1., -0., -0.,
         0.,  0., -0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.,
        -0., -0.,  0.,  1.,  0., -0., -0.,  0.,  0., -0.,  1.,  1.,  0.,
         1.,  0.,  0., -1., -0.,  0.,  0., -0., -1.,  0.,  0., -0., -1.,
         0., -0.],
       [ 0., -0., -0., -0.,  0., -0., -0.,  0., -0.,  0., -1.,  0., -0.,
        -1.,  1., -0., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -1., -0.,
        -1.,  0., -0.,  0., -0., -0.,  0.,  0.,  0.,  1.,  0., -1., -0.,
         0., -0.,  0., -0., -0.,  1., -1.,  0., -0.,  0., -0.,  0.,  1.,
        -0.,  0., -0., -0.,  0., -0.,  0., -0.,  0.,  0., -0., -1., -0.,
        -0., -1., -0., -0.,  1.,  0.,  0., -0., -0., -0., -0., -0.,  0.,
         0.,  0.],
       [-1., -0.,  1.,  0., -0.,  0.,  0., -0.,  0., -0.,  1.,  0.,  0.,
         1., -1.,  1.,  0.,  0.,  0.,  0.,  1.,  0., -0.,  0.,  0., -0.,
         0.,  1.,  0., -0.,  0.,  0., -1.,  0.,  0.,  0., -0.,  1., -0.,
        -0., -0., -0.,  0.,  0., -1.,  1., -1.,  0., -0.,  0., -0., -1.,
         0., -0.,  0., -0.,  0., -0., -0.,  0.,  0., -0., -0., -0., -0.,
        -0.,  1.,  0., -0., -0., -1., -0., -0., -0., -0., -0.,  1.,  0.,
        -0.,  0.],
       [ 0., -0., -0., -1., -0.,  0.,  1.,  0., -0.,  0.,  0.,  0., -0.,
         0.,  0.,  0., -1., -0.,  1., -1., -0., -0., -0., -0., -0., -1.,
        -0.,  0., -0., -0.,  0., -0., -0., -0., -0., -0., -0., -0., -1.,
         0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,  0., -0., -1.,  0.,
         0., -1.,  0., -1., -1., -0., -0.,  1.,  0., -0.,  0.,  0.,  0.,
         0.,  0., -1., -1.,  0., -0., -0.,  1.,  0., -0., -0., -0.,  1.,
        -0., -1.],
       [-0.,  0.,  0.,  1., -0.,  0., -1., -0., -0.,  0., -0., -0., -0.,
         0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,  0.,  0., -0., -0.,
         0., -0., -0.,  0., -0., -0.,  0.,  1.,  0., -0., -1.,  0.,  0.,
        -0., -0., -0., -0., -0., -0., -0., -0.,  0.,  1.,  0.,  0.,  0.,
         0.,  1., -0.,  1.,  1., -0.,  0., -0.,  0.,  0.,  0., -0., -0.,
        -0.,  0.,  1., -0.,  0.,  0., -1.,  0., -0.,  1.,  0.,  0., -1.,
        -0.,  1.],
       [ 0., -0.,  0.,  0.,  0.,  0., -1., -0.,  0.,  1.,  0., -0.,  0.,
        -0.,  0.,  0.,  0., -0.,  1., -1.,  0., -1.,  0., -0.,  0., -1.,
        -0., -0., -0., -1.,  0.,  0.,  1., -0., -1., -0., -1.,  0., -1.,
         0.,  1., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0.,
         0.,  1.,  0.,  1.,  0.,  0., -0.,  1., -0., -0.,  1.,  1.,  0.,
         1., -0., -0., -1., -0., -0., -0., -0., -1.,  0., -0., -0., -1.,
        -0., -0.],
       [ 0., -1.,  0.,  0., -0., -0.,  0., -0.,  0., -1.,  0., -0., -0.,
        -0., -0.,  0., -1.,  0., -0., -0., -0.,  1.,  0.,  0.,  0.,  0.,
        -0.,  0., -0.,  1., -1.,  0., -0.,  1.,  0.,  0.,  0.,  0., -0.,
        -0.,  0., -0., -1.,  0.,  0.,  0., -0.,  0.,  0.,  0., -1., -0.,
        -0., -0., -0., -0.,  0., -1.,  0.,  0., -0.,  0., -0., -1., -0.,
         0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,  1.,  1.,  0.,  0.,
         0.,  0.],
       [-0.,  0., -0.,  0.,  0.,  0., -0.,  0., -1.,  0.,  0.,  0., -0.,
         0.,  0., -0.,  1., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0.,
         0.,  0., -0., -0.,  0., -0., -1., -0.,  0., -0.,  0.,  0., -0.,
         1., -1.,  0., -0., -0.,  0., -0.,  0., -1.,  1., -0.,  0., -0.,
        -1., -0.,  1., -0.,  0., -0., -1., -0.,  0., -0., -1., -0., -0.,
        -1.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0., -1.,  0., -0.,  0.,
         0., -0.]], dtype=np.float32), np.array([[-0.,  0., -0., -0.,  0.,  0., -0.,  0., -1., -0.,  0., -0.,  0.,
         0., -0., -0., -1., -0.,  0.,  0.,  0.,  1., -0., -0., -0.,  0.,
        -1.,  0.,  0., -0., -0., -0.,  1.,  1., -0., -0.,  0., -0.,  0.,
        -1., -0., -0., -0.,  0.,  1., -0.,  0., -0.,  1.,  0., -1., -0.,
         0., -0., -1.,  0.,  0., -0., -1., -0., -0.,  0.,  0.,  1.,  0.,
        -1.,  0., -0., -0.,  0.,  0.,  1.,  0., -0.,  1.,  0., -0., -0.,
        -0., -0.],
       [-0., -0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0.,
        -0.,  1., -0., -0., -0., -0.,  0.,  0., -0., -0., -0.,  0., -0.,
        -0.,  0., -1.,  0., -0.,  0., -0.,  0.,  0., -0.,  1., -1., -0.,
        -0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0., -0., -1., -0., -0.,
         0., -0.,  0.,  1.,  0., -0.,  0.,  0.,  0.,  0., -0., -0., -0.,
        -0.,  1.,  0., -0.,  0., -0., -0., -0., -0.,  0.,  0., -0., -1.,
        -0.,  0.],
       [-1.,  0.,  1.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,
         1., -1.,  0.,  0., -0.,  1., -0.,  0.,  0.,  0., -0., -1., -1.,
         0., -1.,  1.,  0., -0.,  0., -0., -0., -0.,  0., -0., -0.,  0.,
         0.,  0., -0.,  0.,  0., -0.,  1., -0.,  0.,  0., -0., -0., -1.,
        -0., -0., -0.,  0.,  0.,  0., -0., -0., -0.,  0., -0., -0., -0.,
         0., -1., -0.,  0.,  0., -0., -0.,  0., -1., -0., -0., -1.,  0.,
         1., -0.],
       [-0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,
        -0.,  1., -0.,  0., -0.,  1., -0.,  0.,  0.,  0., -0.,  0., -1.,
        -0., -0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -1., -0.,
        -0., -0., -0.,  1.,  0., -0., -0., -0.,  0., -0., -0., -1.,  0.,
        -0., -0.,  0., -0.,  0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,
        -0.,  1.,  0., -0., -0.,  0., -0., -1., -1., -0.,  1., -0.,  0.,
        -0., -0.],
       [-0., -0.,  0., -0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0., -0.,
         0., -1.,  0., -0.,  0.,  0., -1., -0.,  0.,  0.,  0., -0., -0.,
        -0., -0.,  1.,  0., -0., -0., -0., -0., -1., -0., -0.,  1.,  1.,
        -0.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,  0.,  0., -0.,
         0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  1.,  0.,  0.,  0.,
         0., -1., -0., -0.,  0., -0., -0.,  0., -0.,  0., -0., -0., -0.,
        -0., -0.],
       [ 0., -0., -0., -0., -0., -0., -0.,  0.,  0., -0., -0.,  0.,  0.,
         0., -0.,  0., -0.,  0., -1., -0., -0.,  0., -0., -0.,  0.,  1.,
         0.,  0.,  0.,  0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,
         0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0., -0., -0., -0., -0.,  1.,  1., -0.,  0., -0.,  0.,
        -0., -0.],
       [ 1., -0., -1.,  0.,  0.,  0., -0., -0., -0., -0.,  0.,  0., -0.,
        -1.,  1., -0., -0., -0.,  0., -0.,  0.,  0., -0.,  0.,  1.,  0.,
        -0.,  1., -1., -0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,  0.,
        -0.,  0.,  0.,  0., -0.,  0., -1.,  0., -0., -0.,  1., -0.,  1.,
         0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0., -1.,  0.,  0., -0.,
         0.,  1., -1.,  0., -0.,  0.,  1.,  0., -0., -0., -0.,  1., -0.,
        -1., -1.],
       [-0.,  0., -0., -0., -0.,  0.,  0.,  0., -0., -0., -0., -0.,  0.,
        -0.,  0., -0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,
        -0.,  0., -0.,  0., -0., -0., -0.,  0.,  0., -0.,  1.,  0.,  0.,
        -0., -0.,  0.,  0., -0., -0., -0., -0., -0.,  1., -0., -0., -0.,
         1., -0., -0., -0., -0., -0., -1., -0.,  0.,  0., -1.,  0.,  0.,
        -1.,  0., -0., -0., -0.,  0.,  1., -0.,  1.,  0., -0.,  0.,  0.,
         0.,  0.],
       [-0.,  0.,  1., -0., -0.,  1.,  0.,  0., -0., -0.,  0., -0.,  0.,
         0., -0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0., -0., -0.,  0.,
        -0., -0.,  0.,  0., -0., -0.,  0., -0., -0., -0.,  2., -0., -0.,
         0.,  0.,  0.,  0.,  0., -0.,  1.,  1., -0., -0., -1.,  0., -1.,
         0.,  0., -0.,  1., -0.,  0., -0.,  0., -0., -0.,  0., -0.,  0.,
        -0.,  0.,  0.,  1., -1., -1., -0.,  0.,  0., -0., -0., -0., -1.,
         1., -0.],
       [ 0.,  0., -0., -0., -0., -1., -1., -0.,  0., -0.,  0., -0., -0.,
         0., -0., -0., -0., -0.,  1.,  0., -0., -0., -0., -0., -0., -1.,
         0., -1.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,
        -0.,  0.,  0.,  0.,  0., -0., -0., -1.,  0., -0.,  1., -0., -0.,
        -0., -0., -0., -1.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0.,
        -0.,  0., -0.,  0.,  0.,  1., -0.,  0., -2.,  0.,  0.,  0., -0.,
         0.,  0.],
       [ 0.,  1.,  1., -0., -0.,  0., -0.,  0., -0., -0.,  0.,  0., -0.,
         0.,  0., -0., -0., -0., -0., -0., -0.,  1., -0., -0.,  0., -0.,
        -0.,  0.,  0.,  1.,  0.,  0.,  0.,  1., -0.,  0., -1., -0.,  0.,
        -0., -0., -0.,  0., -0., -0.,  1.,  1., -0., -0., -0., -0., -1.,
        -0.,  0.,  0.,  0.,  0., -1., -0., -0.,  0., -0.,  0.,  0., -0.,
         0.,  0.,  0.,  0., -1., -1.,  1., -0., -1.,  0., -0.,  0.,  0.,
         1., -0.],
       [ 0., -0., -1., -1.,  0., -0., -0., -0., -0.,  0., -0.,  0.,  1.,
        -0.,  0., -0.,  0.,  0., -0., -1.,  0., -0., -1., -0.,  0.,  0.,
        -0.,  0., -0., -0., -0., -0.,  0.,  0., -1., -0., -0.,  0.,  1.,
         0., -0.,  0., -0., -0.,  0., -1., -1., -0., -0.,  0.,  0.,  1.,
         0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,  1.,  0.,  0.,  0.,
         0., -0., -0.,  0.,  1.,  1.,  0., -0.,  0., -0.,  0.,  0.,  0.,
        -0., -1.],
       [-0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -1.,  0.,
         0., -0.,  0.,  0., -0., -0., -0., -0., -0., -0.,  0., -0.,  1.,
        -0., -0., -0., -0.,  0., -0., -0.,  0., -0., -0., -1., -0., -0.,
         0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0.,
        -0., -0.,  0.,  0., -1., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,
        -0., -0., -1., -1., -0., -0., -0.,  1.,  1., -0.,  0., -0.,  0.,
        -0.,  0.],
       [-0., -0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  1., -0.,
         0.,  0., -0., -0., -0., -0.,  0., -0.,  0., -0.,  0.,  0., -0.,
        -0.,  1., -0., -0.,  0., -0., -0.,  0.,  0.,  0., -0., -0., -0.,
        -0., -0., -0.,  0., -0.,  0.,  0.,  1.,  0., -0., -0.,  0.,  0.,
         0.,  0., -0.,  0.,  0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,
        -0., -0., -0.,  0., -0., -1.,  1., -0.,  0., -0., -0., -0.,  0.,
         0.,  0.],
       [ 0.,  0., -0., -0., -0., -0.,  0., -0., -0., -0.,  0.,  0., -0.,
         0.,  0., -0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0.,
         0., -0.,  0., -0.,  0., -0., -0., -0., -1., -0., -0., -0.,  0.,
         0.,  1., -0.,  0., -0., -0.,  0.,  0.,  1., -0.,  0.,  0., -0.,
        -1.,  0.,  0., -0., -0.,  0., -0., -0., -0., -0.,  1.,  0.,  0.,
        -0., -0.,  0., -0.,  0., -0., -0., -0., -1.,  0., -0., -0.,  0.,
        -0., -0.],
       [ 0., -0., -1., -0., -0., -1.,  0.,  1.,  0., -0.,  0., -0.,  0.,
         0., -0., -1., -0.,  0.,  1., -0., -0.,  0.,  0., -0., -0.,  0.,
        -0.,  0., -0., -0., -0., -0.,  0., -0.,  0., -0., -1.,  0.,  0.,
         0., -0., -0.,  0., -0., -0., -0., -1., -0., -0.,  1., -0., -0.,
        -0., -0., -0., -1.,  0., -0., -0., -0.,  0., -0., -0., -0.,  0.,
        -0., -0., -0., -1., -0., -0., -0., -0., -0., -0.,  0.,  0.,  1.,
        -1., -0.],
       [ 0.,  0.,  1.,  0., -1.,  0.,  0., -0., -0., -0.,  0., -0., -0.,
        -0.,  0.,  1., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0.,
         0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,
         0., -0., -0., -0.,  0., -0., -0.,  1.,  0., -0.,  0., -0., -0.,
        -0., -0.,  0.,  0., -0., -0., -0., -0., -0.,  0., -0.,  0.,  0.,
        -0., -0., -0.,  0.,  0., -0., -0.,  0.,  1.,  0., -0.,  0., -0.,
         0.,  0.],
       [ 0., -0., -1., -0.,  0.,  0., -0., -0., -0., -1.,  0., -0.,  0.,
         0., -0., -1.,  0., -0.,  0., -0., -0.,  0., -0.,  0., -0., -0.,
         0., -0.,  0., -1.,  1.,  0.,  0., -0., -1., -0.,  0., -0.,  0.,
        -0.,  0., -0., -0.,  0., -0., -0., -1., -0.,  0., -0.,  0.,  0.,
        -0.,  0., -0.,  0., -0.,  1.,  0.,  0.,  0., -0.,  0.,  0.,  0.,
        -0., -0.,  0.,  0., -0., -0.,  0., -0.,  1.,  0.,  0.,  0.,  0.,
        -1., -0.],
       [-0., -0.,  1., -0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0.,
        -0.,  0.,  1.,  0.,  0.,  0.,  0., -0.,  0.,  1., -0.,  0., -0.,
         0.,  0.,  0., -0.,  0., -0., -0.,  0.,  1.,  0.,  0., -0., -0.,
        -0.,  0., -0.,  0.,  0.,  0., -0.,  1.,  0., -0.,  0., -0.,  0.,
         0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0.,
         0., -0., -0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0., -0.,
        -0., -0.],
       [ 0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,
        -0.,  0.,  0., -0., -0., -0., -0.,  0., -0.,  1., -0.,  0., -1.,
        -0.,  0.,  0., -0., -0., -0., -0., -0.,  0.,  0., -0.,  0.,  1.,
        -0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0., -0., -0., -0.,  0.,
        -0., -0.,  0.,  0., -0., -0.,  0., -1., -0.,  0.,  0., -0.,  0.,
        -0.,  0.,  0., -0., -0., -0.,  0., -1., -1., -0.,  0.,  0., -0.,
        -1.,  0.],
       [ 0., -0., -1.,  1.,  1.,  1.,  1., -1., -0., -0.,  0., -0., -0.,
        -0., -0., -1., -0.,  0., -1.,  0.,  0., -0., -1.,  0., -0.,  1.,
        -0.,  0., -0., -0., -0., -0.,  0.,  0., -1.,  0.,  1., -0., -1.,
        -0., -0.,  0.,  0., -0.,  0.,  0., -1.,  0.,  0., -1., -0.,  0.,
         0., -1.,  0.,  1.,  1.,  0., -0.,  1., -0.,  0.,  0.,  0.,  0.,
         0., -0., -0.,  1., -0., -0.,  0.,  1.,  1., -0.,  0.,  0., -1.,
         1., -0.],
       [ 0., -0.,  0.,  0., -0., -0., -0., -0.,  1.,  0., -0., -0., -0.,
        -0., -0.,  0.,  0., -0., -0., -0.,  0., -1., -0.,  0., -0.,  0.,
         1.,  0.,  0., -0., -0.,  0., -0., -1., -0.,  0.,  0.,  0., -0.,
         1.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -1., -0.,  0., -0.,
        -1.,  0.,  0., -0., -0., -0.,  1.,  0.,  0., -0.,  1., -1., -0.,
         1., -0., -0.,  0.,  0., -0., -1.,  0., -1., -1.,  0.,  0., -0.,
        -0., -0.],
       [-0.,  0., -1.,  0., -0., -1.,  0., -0., -0., -0., -0.,  0., -0.,
         0., -0., -0.,  0., -0.,  1., -0., -0., -1., -0.,  0., -0., -0.,
         1.,  0., -0., -0., -0.,  0.,  0.,  0.,  0., -1., -1., -0., -0.,
        -0., -0.,  0.,  0.,  0., -0.,  0., -1., -0., -0.,  1.,  0.,  1.,
        -0.,  0.,  0., -1.,  0., -0.,  0.,  0., -0.,  0.,  0., -1., -0.,
        -0., -1.,  0., -1.,  0.,  0.,  0., -0., -0.,  0.,  0., -0.,  1.,
        -1., -0.],
       [ 1.,  0., -1.,  0.,  0., -0.,  0., -0.,  0., -0.,  0., -0.,  0.,
        -1., -0., -0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,
        -0.,  1.,  0., -1., -0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.,
         0.,  0.,  0., -0.,  0.,  0., -1.,  1., -0., -0.,  0.,  0.,  1.,
        -0., -0., -0., -0.,  0., -0.,  0., -0., -0.,  0.,  0.,  1.,  0.,
        -0.,  1., -0., -0.,  0.,  0., -0., -0.,  1., -0., -0., -0.,  0.,
        -1., -0.],
       [-0., -0., -1.,  0.,  0., -0., -0.,  0., -0., -0., -0., -0., -0.,
         0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0., -0.,
         0.,  0.,  0., -1.,  0., -0., -0.,  0.,  0., -0., -0., -0.,  0.,
        -0., -0.,  0., -1., -0.,  0.,  0., -1.,  0.,  0., -0., -0.,  1.,
        -0., -0., -0., -0.,  0.,  1., -0., -0., -0., -0., -0.,  1., -0.,
         0., -1., -0.,  0., -0.,  0.,  0.,  0.,  1.,  0., -0.,  0.,  0.,
        -1., -0.],
       [-0., -0.,  1.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,
        -0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  1.,  0., -0., -0.,
         0.,  0.,  0.,  0.,  1., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,
         0., -0., -0.,  1., -1., -0.,  0.,  1.,  0., -0.,  0.,  0., -1.,
        -0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,
         0.,  1., -0.,  0., -0., -0., -0.,  0.,  0., -0., -0., -0., -0.,
         0., -0.],
       [-0.,  0.,  1.,  0., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,
        -0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0.,
        -0., -0.,  0.,  1.,  0.,  1., -0.,  0., -0.,  0.,  0.,  1., -0.,
         0.,  0., -0., -0.,  0.,  0.,  0., -0., -0., -0.,  0., -0., -1.,
        -0., -0., -0., -0., -0., -1., -0.,  0., -0., -1.,  0., -1.,  0.,
        -0., -0.,  0., -0., -0.,  0.,  0., -0., -1., -0., -1.,  1.,  0.,
         1., -0.],
       [ 0.,  1., -0.,  0.,  0., -0.,  0., -0., -0., -0.,  0., -1., -0.,
         0.,  0.,  0., -0.,  0.,  0., -0., -0., -0., -0., -1., -0., -0.,
        -0., -1.,  0.,  0., -0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,
        -0., -0.,  0., -1., -0., -0.,  1., -1., -0., -0., -0.,  0., -0.,
         0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  0.,
         0., -1., -0., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0.,
         0.,  0.],
       [-0.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0., -0.,  0.,
         0.,  0., -0., -0., -0., -0., -0.,  0.,  0., -0., -0.,  0., -0.,
        -0.,  0., -0.,  0., -0.,  0.,  1., -0., -0., -0., -0., -0., -0.,
         1., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0., -0., -0., -0.,
        -1.,  0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0.,  1.,  0., -0.,
        -0.,  0.,  0., -0.,  0.,  0., -0., -0., -1.,  0.,  0., -0., -0.,
        -0.,  0.],
       [-0., -0., -1.,  0.,  0., -1., -0., -0.,  0.,  0.,  1., -0.,  0.,
         0.,  0., -0.,  0., -0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -0.,  0., -0., -0., -0., -0.,  1., -0., -0., -1., -1.,  0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0., -0., -1.,  0.,  0.,  1., -0.,  1.,
         0., -0.,  0., -1.,  0., -0.,  0., -0., -0., -0., -0.,  0.,  0.,
        -1., -0., -0., -1., -0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  1.,
        -1.,  0.],
       [-0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0., -0.,  0.,
         0.,  0.,  0.,  0., -0., -0.,  0.,  1.,  0.,  0.,  0., -0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0., -0., -0., -0., -0.,
        -0., -0.,  0.,  0.,  0., -0., -0.,  1., -0., -0., -0.,  0.,  0.,
         0.,  0.,  0.,  0., -0., -0., -0.,  0., -0.,  0., -1.,  0., -0.,
         0., -0.,  0., -0., -0., -0., -0., -0.,  1.,  0., -0., -0., -0.,
        -0.,  0.],
       [-1., -0.,  0.,  0., -0.,  0., -0.,  0.,  1.,  0.,  0., -1.,  0.,
        -0.,  0., -0.,  0.,  1., -0., -0.,  1.,  0.,  0.,  1., -0., -0.,
        -0., -1.,  0.,  0.,  0.,  0., -1., -0.,  0.,  0., -0.,  0., -0.,
         1., -0., -1., -0.,  0.,  0., -0., -1., -0., -0., -0., -0., -0.,
         1.,  0.,  0., -0., -0.,  0., -1.,  0., -0.,  0., -1.,  0., -0.,
         0., -0., -0.,  0., -0.,  0., -0., -0.,  1.,  0., -0.,  0.,  0.,
        -0.,  0.],
       [ 0., -0.,  1., -0., -0., -0., -0., -0., -0.,  0., -0.,  0.,  0.,
        -0., -0.,  0., -0., -0.,  0., -0.,  0., -0.,  1., -0., -0.,  0.,
         0., -0.,  0., -0., -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0.,
        -1.,  0.,  0.,  0., -0., -0.,  0.,  1.,  1., -0., -0., -0., -0.,
         0., -0., -0., -0., -0., -0.,  0., -0., -1.,  0., -0.,  0., -0.,
        -0.,  0., -0., -0., -0., -0.,  0., -0., -0.,  0., -0.,  0., -0.,
        -0., -0.],
       [ 1.,  0., -0.,  0., -0.,  0., -0., -0., -1.,  0.,  0.,  1., -0.,
         0.,  0., -0.,  0., -1., -0., -0., -1., -0.,  0., -1.,  0.,  0.,
         0.,  1.,  0.,  0., -0.,  1.,  1., -0.,  0.,  0., -0.,  0., -0.,
        -0., -0.,  1.,  0., -0., -0.,  0., -0., -0.,  0.,  0., -0., -0.,
        -1.,  0.,  1.,  0., -0.,  0.,  1.,  0.,  0., -0.,  1., -0., -1.,
         0.,  0., -0., -0., -0., -0.,  0., -0., -1., -0., -0.,  1.,  0.,
        -0.,  0.],
       [-0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,  0.,  0., -1., -0.,
        -0., -0., -0., -0.,  1., -0.,  0., -0.,  0., -0.,  0., -0.,  0.,
         0., -1., -0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,  0., -0.,
         1., -0., -0., -0.,  0.,  0., -0., -1., -0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0., -0.,  0.,  0., -1.,  0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,
        -0., -0.]], dtype=np.float32), np.array([[ 0., -0., -0.,  0., -0.,  0.,  0., -0., -1., -0., -0., -0., -0.,
         0.,  0.,  0., -0.,  1., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,
         0., -0., -0.,  0.,  0., -0., -0.,  0., -0.,  0., -0.,  0.,  0.,
        -1.,  0., -1.,  0., -0.,  0.,  0.,  0.,  1., -0., -0., -0., -0.,
         1.,  0.,  1., -0.,  0., -0.,  1.,  0., -1.,  0.,  0., -0., -1.,
        -0.,  0., -0.,  0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,  0.,
        -0., -0.],
       [-0.,  0.,  0.,  0., -0., -0., -0., -0.,  0., -0., -1.,  0., -0.,
        -0., -0., -0., -0., -0.,  0., -0.,  1.,  0.,  0., -0., -0., -0.,
        -1., -0., -0., -0., -0., -0., -1., -0.,  0.,  1., -0.,  0.,  0.,
        -0., -1., -0., -0., -0.,  1.,  0.,  0.,  0., -0.,  0.,  0., -0.,
         0.,  0., -0.,  0., -0.,  0.,  0., -0., -0., -0.,  1., -0.,  0.,
        -1., -0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0., -0.,  0., -0.,
         0.,  0.],
       [ 0.,  0., -0.,  0., -0., -0.,  0., -0., -1.,  0., -0., -0., -0.,
         0.,  0.,  0.,  1., -0., -0., -0.,  0.,  0., -0., -0.,  0., -0.,
        -0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  0.,  0., -0., -0.,
         0.,  1., -1.,  0., -0.,  0.,  0., -0.,  0.,  1., -0., -0., -0.,
         1., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,
         0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0., -1.,  0.,  0., -0.,
         0., -0.],
       [ 0.,  1., -0., -0., -0.,  0., -0., -0., -0., -0.,  1., -0.,  1.,
        -1.,  0.,  0.,  0., -0.,  0., -1.,  0.,  0., -0.,  0.,  0., -0.,
         0., -0., -0., -0., -1.,  0.,  0.,  0.,  0.,  0.,  0., -1., -1.,
        -0.,  0.,  0.,  1., -0.,  0., -1., -0., -0.,  0.,  0., -0.,  1.,
        -0.,  0., -0.,  0.,  0.,  1., -0.,  1.,  0.,  0., -0., -0., -0.,
         0., -1.,  0., -0.,  0.,  0.,  0., -0.,  0., -0.,  1.,  0.,  0.,
         1.,  0.],
       [-0., -0.,  0.,  0.,  0., -1., -1., -1., -0., -0.,  1., -0., -0.,
         0., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0.,
        -0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0., -1.,  0.,  0.,  0.,
         0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,
        -0.,  1., -0., -1.,  0., -0.,  0.,  0.,  0., -0., -0., -0., -0.,
         0.,  0., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,  1.,
         0.,  0.],
       [-0., -0.,  0.,  0.,  0., -0.,  0.,  1., -0.,  0., -0.,  0., -0.,
         0.,  0., -0.,  0.,  0.,  0., -0., -0., -1., -0.,  0.,  0.,  0.,
        -0.,  0., -0., -0.,  0., -0.,  0.,  1., -0., -0., -1., -0., -0.,
         0., -0., -0., -0.,  0., -0., -0.,  0., -0., -1.,  0., -0., -0.,
        -0., -1.,  0., -0., -1., -0., -0.,  0.,  0.,  0., -0., -0., -0.,
         1.,  0.,  0.,  1.,  0.,  0.,  0.,  0., -0., -0., -0., -0., -1.,
         0., -0.],
       [-1., -0.,  1.,  1.,  0.,  0.,  0., -0., -0.,  0.,  0., -1.,  1.,
         0., -0.,  0., -0.,  0., -0.,  0.,  1.,  0.,  1., -0., -0.,  0.,
        -0.,  1., -0., -0.,  0., -1., -0., -0., -0.,  0.,  0.,  0., -0.,
         0.,  0.,  1., -0.,  0.,  0.,  0.,  1., -0., -0., -0.,  0., -0.,
        -0.,  0.,  0., -0., -1., -0., -0.,  0.,  0., -0., -0., -0., -0.,
        -0., -0.,  1.,  0.,  0., -0., -0.,  0., -0., -0., -0., -1.,  0.,
         1., -1.],
       [-1.,  0.,  0.,  0.,  1., -0., -1., -0., -0., -0., -0.,  0., -0.,
         1.,  0., -0., -0.,  0.,  0., -0.,  1.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0.,
        -0.,  0.,  1.,  0., -0.,  0., -0.,  0.,  0., -0., -1., -0., -0.,
        -0.,  1., -0., -1.,  0., -0.,  0., -0., -0.,  0., -0.,  0., -0.,
         0., -0.,  0., -0., -0., -0., -0., -0., -0., -0.,  0.,  0.,  1.,
        -0.,  0.],
       [-0.,  0.,  0.,  0., -1.,  0.,  1.,  0., -0.,  0., -0., -0., -0.,
        -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  1.,
        -0., -0.,  0., -1., -0., -0., -0., -0.,  0.,  0., -0.,  0.,  0.,
        -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.,
         1., -1.,  0., -0., -0.,  1.,  0., -0., -0., -0.,  1.,  0.,  0.,
        -0., -0.,  0., -0., -0.,  0., -0.,  1., -1.,  0.,  0.,  0., -0.,
        -0., -0.],
       [ 0., -1.,  0.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,
         0.,  0., -0., -0., -0.,  0.,  0., -0.,  0.,  0.,  1.,  0., -0.,
         0.,  0.,  0.,  0.,  1., -1., -0.,  0., -0., -0.,  0., -0., -0.,
         0.,  0., -1., -1., -1., -0.,  0., -0., -0.,  0.,  0., -0., -0.,
         0.,  0.,  0.,  0., -0., -1., -0., -0.,  0., -0.,  0.,  0.,  1.,
         0.,  0., -0., -0.,  0., -0., -0.,  0.,  0., -0., -1., -0., -0.,
        -0.,  0.],
       [ 1.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  1., -0., -0., -0.,
        -1.,  1.,  0.,  0., -0., -0., -0.,  0., -1., -0.,  0.,  0., -0.,
        -1.,  0., -1.,  1., -0.,  0., -0., -0., -0.,  0.,  0., -1., -0.,
         0.,  0., -1.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,
         0., -0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0., -1.,  1.,
        -0., -0.,  0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,  1.,  0.,
        -0.,  0.],
       [-0.,  0., -0.,  0., -0.,  0., -0.,  0., -1., -1.,  0., -0., -0.,
         0.,  0., -0., -1., -0.,  0., -0., -0., -0., -0.,  0.,  0.,  0.,
         0., -0., -0.,  0., -0., -1.,  0.,  1., -0.,  0., -0.,  0., -0.,
        -0.,  0., -1.,  0., -0.,  0., -0., -0.,  0.,  0., -0.,  1.,  0.,
         0.,  0.,  1., -0.,  0., -1., -0.,  0., -0., -0.,  0., -0., -0.,
        -0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0., -1., -1., -0., -0.,
         0., -0.],
       [-0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0., -0.,  1.,
         0.,  0., -0.,  0., -0., -0., -1., -0.,  0., -1.,  0.,  0., -0.,
         0.,  0., -0., -0.,  0.,  0.,  0.,  0., -0., -0., -0., -0., -1.,
         0., -0., -0.,  0., -1., -0., -0.,  0., -0., -0., -0.,  0.,  0.,
        -0., -0.,  0.,  0., -0., -0.,  0.,  1., -1., -0., -0., -0.,  0.,
         0., -0., -0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0., -0.,  0.,
         0.,  0.],
       [-0., -0.,  1.,  0., -1., -1., -1., -1., -0.,  1., -0., -0., -0.,
        -0.,  0., -1., -0., -0., -0.,  0., -0., -1., -0., -0., -1., -0.,
        -1., -0., -1.,  1., -0., -0.,  0., -0., -0., -1., -0.,  0.,  0.,
         0., -0., -0., -0.,  0., -0.,  0.,  0.,  0., -0., -0., -0., -1.,
        -0.,  1.,  0., -1.,  0., -0., -0., -0., -1.,  0., -0., -1.,  0.,
        -0.,  0., -0., -0.,  1.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  1.,
        -0.,  0.],
       [ 0., -0.,  0., -1.,  0., -0., -0., -0., -0., -1.,  0.,  0., -0.,
        -0.,  0., -0., -0.,  0., -0.,  0., -0., -0., -1.,  0.,  0., -0.,
         0., -0.,  0.,  0., -1., -0.,  0.,  0., -1.,  0.,  0., -0., -1.,
        -0.,  1., -0.,  0., -0.,  0., -0., -0., -1., -0.,  0., -0., -0.,
        -0., -1.,  0.,  0., -0.,  0.,  0.,  1., -0., -0., -0., -0.,  0.,
         0., -0.,  0.,  0., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,
         0.,  0.],
       [-0.,  0.,  0.,  1.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  1.,
        -0., -0.,  0., -0.,  0.,  0.,  1.,  0., -0., -0.,  0., -0.,  0.,
        -0.,  0.,  0., -0., -0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        -0., -0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,
        -0.,  0., -0.,  0., -1.,  0.,  0., -1., -0.,  1., -0., -0.,  1.,
         0., -0.,  1.,  0., -0., -0., -0.,  0., -0.,  0., -0., -0., -0.,
        -0., -1.],
       [ 0., -0.,  0.,  0., -0., -1., -1., -0., -0.,  0.,  0.,  0.,  0.,
         0.,  1., -0.,  0.,  0., -1., -0., -0., -0.,  0.,  0.,  1.,  1.,
        -0., -0., -1.,  0., -0., -0.,  0., -0.,  0.,  0., -0., -1., -0.,
        -0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0.,  0., -1., -0., -0.,
         0.,  1.,  0., -1., -0.,  0., -0.,  1., -0.,  0., -0., -0.,  1.,
        -0.,  0.,  0.,  1., -0., -0.,  0.,  0.,  0., -0.,  0.,  1.,  1.,
        -0., -0.],
       [-0., -0.,  0., -0., -0., -0., -0., -0.,  0., -0.,  0., -0., -0.,
        -0., -0., -0., -1., -0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,
         0., -0.,  0., -0.,  0., -1., -0., -0.,  0., -0., -0.,  0.,  0.,
         0., -0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0., -1.,  1.,  0., -1.,  0., -0., -1.,  0.,  0.,  0.,  0.,  0.,
         0., -0.,  0.,  0., -0.,  0.,  0.,  1., -0.,  0., -1., -0., -0.,
        -0.,  0.],
       [-0., -0.,  0.,  1.,  0., -0.,  0.,  0., -0.,  0.,  0., -1.,  1.,
        -0.,  0.,  0.,  0., -1., -0.,  0., -0.,  0.,  0., -1., -0., -0.,
         0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,
        -0., -0.,  1.,  0., -0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0.,
         0.,  0., -0.,  0., -1.,  0., -0.,  0., -0.,  0.,  0., -0.,  0.,
         0.,  0.,  1., -0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0.,
         0., -1.],
       [-1.,  0., -0., -0.,  0.,  0., -1.,  0.,  0.,  0., -0., -0., -0.,
         1.,  0.,  0., -0., -1., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -0., -1.,  0., -0.,  0.,  0., -0.,  0.,  0., -0., -0.,  0., -0.,
         0.,  0.,  1.,  0., -0.,  0., -1.,  0., -0.,  0., -1., -0.,  0.,
         0.,  1., -0., -1., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,
        -0.,  0., -0.,  0.,  1.,  1., -0.,  0.,  0.,  0., -0., -0.,  1.,
         0.,  0.],
       [-0., -1., -0.,  0., -0.,  0., -0.,  0.,  1.,  0., -0., -1.,  0.,
         0., -0., -0., -0., -0.,  0.,  0.,  0.,  0., -0., -0., -0., -0.,
         0.,  0., -0.,  0.,  0., -0.,  0., -1.,  0.,  0., -0.,  0., -0.,
        -0.,  0.,  1., -0., -0.,  0.,  0.,  0., -0., -1., -0.,  0.,  0.,
        -0., -1., -0.,  0., -1., -0.,  1.,  0., -0., -0., -0.,  0.,  0.,
         0.,  0.,  1., -0.,  0.,  0., -1., -0.,  0.,  1., -0., -0., -0.,
        -0., -0.]], dtype=np.float32))

In [ ]:
#@title Verification
n = 3
m = 5
p = 7
rank = 80

verify_tensor_decomposition(decomposition_357, n, m, p, rank)

##Rank-48 decomposition of <4,4,4> over 0.5*C


In [ ]:
#@title Data
decomposition_444 = (np.array([[ 0.5+0.5j,  0.5+0.5j,  0. +0.j ,  0. -0.5j,  0.5+0.5j,  0. +0.j ,
         0. +0.5j,  0.5+0.5j,  0. -0.5j, -0.5+0.5j, -0.5+0.5j,  0.5+0.j ,
         0. +0.j , -0. +0.j ,  0. +0.5j,  0. +0.j , -0.5+0.j ,  0.5+0.5j,
         0. +0.5j,  0. -0.j ,  0. +0.j ,  0. +0.5j, -0.5-0.5j, -0. +0.j ,
        -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j, -0.5+0.5j,  0.5+0.5j,
        -0. +0.j ,  0.5+0.j , -0. +0.j ,  0.5+0.j ,  0. -0.5j,  0. +0.j ,
        -0. +0.j ,  0.5+0.j ,  0. +0.j , -0.5+0.j , -0.5-0.5j,  0.5-0.5j,
         0.5+0.5j,  0. +0.5j,  0. -0.j , -0.5+0.5j,  0.5-0.5j,  0.5+0.j ],
       [ 0.5+0.5j,  0. +0.j , -0.5+0.j , -0.5+0.j , -0.5-0.5j,  0. +0.j ,
         0. +0.j , -0.5-0.5j,  0. -0.5j,  0. -0.j ,  0.5-0.5j,  0.5+0.j ,
         0.5+0.5j,  0.5-0.5j, -0.5+0.j ,  0. +0.j , -0. +0.5j,  0.5+0.5j,
        -0. +0.5j,  0. +0.j ,  0.5+0.5j,  0. -0.5j,  0. +0.j ,  0. +0.j ,
         0.5+0.j , -0. +0.j ,  0.5+0.5j,  0. -0.5j, -0.5+0.5j, -0. +0.j ,
         0.5+0.5j, -0.5+0.j ,  0. +0.j , -0. +0.5j,  0. +0.5j, -0. +0.j ,
        -0.5-0.5j,  0. -0.5j,  0.5-0.5j,  0. -0.5j, -0.5-0.5j,  0. -0.j ,
         0. -0.j ,  0.5+0.j ,  0. +0.j ,  0.5-0.5j,  0. +0.j , -0. +0.5j],
       [ 0. +0.j ,  0. +0.j ,  0.5+0.j ,  0.5+0.j ,  0. +0.j ,  0.5-0.5j,
         0. +0.j ,  0. +0.j , -0.5-0.j ,  0. +0.j ,  0. -0.j ,  0. -0.5j,
        -0.5-0.5j, -0.5+0.5j,  0.5+0.j , -0.5+0.5j,  0. +0.5j,  0. +0.j ,
        -0.5+0.j ,  0.5-0.5j, -0.5-0.5j, -0.5-0.j ,  0. -0.j , -0.5-0.5j,
         0. -0.5j,  0.5-0.5j,  0.5+0.5j,  0.5+0.j ,  0. -0.j ,  0. +0.j ,
         0.5+0.5j,  0. -0.5j,  0.5+0.5j,  0. -0.5j, -0.5+0.j ,  0.5-0.5j,
        -0.5-0.5j,  0. -0.5j,  0.5-0.5j,  0. -0.5j,  0. +0.j ,  0. +0.j ,
        -0. +0.j , -0.5-0.j ,  0.5-0.5j,  0. -0.j ,  0. +0.j , -0. +0.5j],
       [ 0. +0.j , -0.5+0.5j,  0. +0.j , -0.5-0.j ,  0. -0.j , -0.5-0.5j,
         0.5+0.j ,  0. +0.j ,  0. -0.5j, -0.5-0.5j,  0. +0.j , -0.5+0.j ,
         0. +0.j , -0. +0.j , -0.5+0.j ,  0.5+0.5j,  0. -0.5j,  0. +0.j ,
        -0. +0.5j,  0.5+0.5j,  0. +0.j ,  0. -0.5j, -0.5+0.5j,  0.5-0.5j,
        -0.5-0.j ,  0.5+0.5j,  0. +0.j , -0. +0.5j,  0. +0.j ,  0.5-0.5j,
         0. -0.j ,  0.5+0.j ,  0.5-0.5j,  0. -0.5j,  0. +0.5j,  0.5+0.5j,
         0. +0.j ,  0. -0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j , -0.5-0.5j,
        -0.5+0.5j, -0.5-0.j , -0.5-0.5j,  0. +0.j ,  0.5+0.5j,  0. -0.5j],
       [ 0.5-0.5j,  0.5+0.5j, -0. +0.j , -0. +0.5j, -0.5+0.5j,  0. +0.j ,
        -0.5+0.j , -0.5-0.5j,  0.5+0.j ,  0.5+0.5j, -0.5+0.5j, -0.5-0.j ,
        -0. +0.j ,  0. -0.j ,  0.5+0.j ,  0. -0.j , -0.5-0.j ,  0.5+0.5j,
        -0. +0.5j,  0. -0.j ,  0. +0.j ,  0. -0.5j,  0.5-0.5j,  0. -0.j ,
         0. -0.5j,  0. +0.j ,  0. +0.j , -0.5+0.j , -0.5-0.5j, -0.5-0.5j,
        -0. +0.j ,  0.5+0.j ,  0. -0.j , -0.5+0.j , -0.5-0.j ,  0. -0.j ,
        -0. +0.j , -0. +0.5j, -0. +0.j , -0.5+0.j ,  0.5+0.5j, -0.5+0.5j,
         0.5-0.5j,  0.5+0.j , -0. +0.j ,  0.5+0.5j,  0.5-0.5j, -0. +0.5j],
       [ 0.5-0.5j, -0. +0.j ,  0. -0.5j, -0.5+0.j ,  0.5-0.5j,  0. +0.j ,
         0. -0.j ,  0.5+0.5j,  0.5+0.j ,  0. +0.j ,  0.5-0.5j, -0.5-0.j ,
        -0.5+0.5j,  0.5-0.5j,  0. -0.5j,  0. -0.j ,  0. -0.5j,  0.5+0.5j,
        -0. +0.5j,  0. +0.j ,  0.5+0.5j,  0. +0.5j,  0. +0.j ,  0. -0.j ,
        -0. +0.5j,  0. +0.j , -0.5-0.5j, -0.5+0.j , -0.5-0.5j, -0. +0.j ,
        -0.5-0.5j, -0.5+0.j ,  0. +0.j , -0. +0.5j,  0.5+0.j ,  0. +0.j ,
        -0.5+0.5j, -0.5+0.j , -0.5-0.5j,  0. +0.5j,  0.5+0.5j,  0. -0.j ,
         0. +0.j ,  0. +0.5j,  0. +0.j , -0.5-0.5j,  0. -0.j ,  0.5+0.j ],
       [ 0. -0.j , -0. +0.j , -0. +0.5j,  0.5+0.j ,  0. +0.j ,  0.5-0.5j,
         0. -0.j , -0. +0.j ,  0. -0.5j,  0. -0.j ,  0. -0.j ,  0. +0.5j,
         0.5-0.5j, -0.5+0.5j, -0. +0.5j,  0.5-0.5j,  0. -0.5j,  0. +0.j ,
        -0.5+0.j ,  0.5-0.5j, -0.5-0.5j,  0.5+0.j ,  0. +0.j ,  0.5-0.5j,
         0.5+0.j , -0.5-0.5j, -0.5-0.5j,  0. -0.5j,  0. -0.j ,  0. +0.j ,
        -0.5-0.5j,  0. -0.5j, -0.5+0.5j,  0. -0.5j, -0. +0.5j, -0.5+0.5j,
        -0.5+0.5j, -0.5+0.j , -0.5-0.5j, -0. +0.5j,  0. -0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.5j, -0.5-0.5j, -0. +0.j , -0. +0.j ,  0.5+0.j ],
       [ 0. +0.j , -0.5+0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j , -0.5-0.5j,
         0. +0.5j,  0. -0.j ,  0.5+0.j , -0.5+0.5j,  0. +0.j ,  0.5+0.j ,
         0. +0.j ,  0. -0.j , -0. +0.5j, -0.5-0.5j,  0. -0.5j,  0. -0.j ,
         0. +0.5j,  0.5+0.5j,  0. -0.j ,  0. +0.5j, -0.5-0.5j,  0.5+0.5j,
         0. -0.5j,  0.5-0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j , -0.5+0.5j,
         0. +0.j ,  0.5+0.j ,  0.5+0.5j,  0. +0.5j,  0.5+0.j , -0.5-0.5j,
        -0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0.5+0.5j,
         0.5+0.5j, -0. +0.5j, -0.5+0.5j,  0. +0.j ,  0.5+0.5j,  0.5+0.j ],
       [ 0.5-0.5j, -0.5-0.5j,  0. -0.j ,  0. -0.5j, -0.5+0.5j,  0. +0.j ,
         0.5+0.j , -0.5-0.5j, -0.5-0.j , -0.5-0.5j,  0.5-0.5j,  0.5+0.j ,
        -0. +0.j , -0. +0.j ,  0.5+0.j , -0. +0.j , -0.5-0.j ,  0.5+0.5j,
        -0. +0.5j,  0. +0.j ,  0. -0.j ,  0. -0.5j,  0.5-0.5j,  0. +0.j ,
         0. -0.5j,  0. +0.j ,  0. +0.j , -0.5-0.j ,  0.5+0.5j,  0.5+0.5j,
         0. +0.j , -0.5+0.j ,  0. -0.j , -0.5-0.j ,  0.5+0.j ,  0. -0.j ,
         0. +0.j , -0. +0.5j,  0. +0.j ,  0.5+0.j , -0.5-0.5j, -0.5+0.5j,
         0.5-0.5j, -0.5-0.j ,  0. -0.j , -0.5-0.5j,  0.5-0.5j,  0. -0.5j],
       [ 0.5-0.5j, -0. +0.j ,  0. +0.5j, -0.5+0.j ,  0.5-0.5j,  0. +0.j ,
         0. +0.j ,  0.5+0.5j, -0.5-0.j ,  0. +0.j , -0.5+0.5j,  0.5+0.j ,
        -0.5+0.5j,  0.5-0.5j, -0. +0.5j,  0. +0.j , -0. +0.5j,  0.5+0.5j,
        -0. +0.5j,  0. -0.j , -0.5-0.5j,  0. +0.5j,  0. +0.j ,  0. +0.j ,
         0. +0.5j,  0. +0.j ,  0.5+0.5j, -0.5-0.j ,  0.5+0.5j, -0. +0.j ,
        -0.5-0.5j,  0.5+0.j ,  0. +0.j ,  0. -0.5j, -0.5-0.j ,  0. +0.j ,
         0.5-0.5j,  0.5+0.j , -0.5-0.5j, -0. +0.5j, -0.5-0.5j,  0. +0.j ,
         0. -0.j , -0. +0.5j, -0. +0.j ,  0.5+0.5j,  0. +0.j ,  0.5+0.j ],
       [ 0. +0.j , -0. +0.j ,  0. -0.5j,  0.5+0.j ,  0. -0.j , -0.5+0.5j,
        -0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. -0.j ,  0. +0.5j,
         0.5-0.5j, -0.5+0.5j,  0. -0.5j,  0.5-0.5j, -0. +0.5j, -0. +0.j ,
         0.5+0.j ,  0.5-0.5j,  0.5+0.5j, -0.5+0.j ,  0. +0.j ,  0.5-0.5j,
        -0.5+0.j ,  0.5+0.5j,  0.5+0.5j, -0. +0.5j,  0. +0.j ,  0. +0.j ,
        -0.5-0.5j,  0. -0.5j,  0.5-0.5j,  0. +0.5j,  0. +0.5j,  0.5-0.5j,
         0.5-0.5j,  0.5+0.j , -0.5-0.5j, -0. +0.5j,  0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. -0.5j, -0.5-0.5j, -0. +0.j , -0. +0.j ,  0.5+0.j ],
       [-0. +0.j ,  0.5-0.5j,  0. +0.j , -0.5-0.j , -0. +0.j ,  0.5+0.5j,
         0. -0.5j,  0. +0.j ,  0.5+0.j ,  0.5-0.5j,  0. +0.j ,  0.5+0.j ,
        -0. +0.j , -0. +0.j , -0. +0.5j, -0.5-0.5j,  0. -0.5j,  0. +0.j ,
         0. -0.5j,  0.5+0.5j,  0. -0.j ,  0. -0.5j, -0.5-0.5j,  0.5+0.5j,
         0. +0.5j, -0.5+0.5j,  0. +0.j , -0.5-0.j ,  0. +0.j ,  0.5-0.5j,
         0. +0.j ,  0.5+0.j , -0.5-0.5j,  0. +0.5j,  0.5+0.j ,  0.5+0.5j,
        -0. +0.j ,  0.5+0.j ,  0. -0.j ,  0. +0.5j,  0. +0.j ,  0.5+0.5j,
         0.5+0.5j,  0. -0.5j, -0.5+0.5j,  0. +0.j ,  0.5+0.5j, -0.5+0.j ],
       [ 0.5-0.5j,  0.5-0.5j, -0. +0.j , -0.5-0.j ,  0.5-0.5j, -0. +0.j ,
        -0.5+0.j , -0.5+0.5j,  0.5+0.j , -0.5-0.5j,  0.5+0.5j,  0. -0.5j,
         0. +0.j , -0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. -0.5j, -0.5+0.5j,
        -0.5+0.j , -0. +0.j ,  0. -0.j , -0.5-0.j , -0.5+0.5j,  0. -0.j ,
        -0. +0.5j,  0. +0.j ,  0. +0.j , -0.5+0.j , -0.5-0.5j,  0.5-0.5j,
         0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.5j,  0.5+0.j , -0. +0.j ,
         0. +0.j ,  0. -0.5j, -0. +0.j , -0. +0.5j, -0.5+0.5j,  0.5+0.5j,
         0.5-0.5j, -0.5+0.j ,  0. +0.j , -0.5-0.5j,  0.5+0.5j,  0. +0.5j],
       [ 0.5-0.5j, -0. +0.j ,  0. -0.5j,  0. -0.5j, -0.5+0.5j,  0. -0.j ,
         0. -0.j ,  0.5-0.5j,  0.5+0.j ,  0. -0.j , -0.5-0.5j,  0. -0.5j,
         0.5-0.5j,  0.5+0.5j,  0. -0.5j,  0. +0.j ,  0.5+0.j , -0.5+0.5j,
        -0.5+0.j ,  0. +0.j ,  0.5-0.5j,  0.5+0.j , -0. +0.j , -0. +0.j ,
         0. -0.5j,  0. +0.j ,  0.5-0.5j, -0.5+0.j , -0.5-0.5j, -0. +0.j ,
        -0.5+0.5j, -0. +0.5j,  0. -0.j ,  0.5+0.j , -0.5+0.j ,  0. +0.j ,
         0.5-0.5j,  0.5+0.j , -0.5-0.5j,  0.5+0.j , -0.5+0.5j,  0. +0.j ,
         0. +0.j ,  0. -0.5j, -0. +0.j ,  0.5+0.5j,  0. +0.j ,  0.5+0.j ],
       [ 0. +0.j ,  0. +0.j , -0. +0.5j, -0. +0.5j,  0. +0.j , -0.5-0.5j,
         0. -0.j ,  0. -0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
        -0.5+0.5j, -0.5-0.5j, -0. +0.5j, -0.5-0.5j,  0.5+0.j ,  0. -0.j ,
        -0. +0.5j,  0.5+0.5j, -0.5+0.5j, -0. +0.5j,  0. +0.j , -0.5+0.5j,
         0.5+0.j ,  0.5+0.5j,  0.5-0.5j,  0. +0.5j, -0. +0.j ,  0. +0.j ,
        -0.5+0.5j,  0.5+0.j , -0.5+0.5j, -0.5+0.j ,  0. +0.5j, -0.5-0.5j,
         0.5-0.5j,  0.5+0.j , -0.5-0.5j,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j , -0. +0.5j, -0.5-0.5j, -0. +0.j ,  0. +0.j ,  0.5+0.j ],
       [-0. +0.j ,  0.5+0.5j,  0. -0.j , -0. +0.5j,  0. +0.j , -0.5+0.5j,
         0. +0.5j,  0. +0.j , -0.5+0.j ,  0.5-0.5j,  0. +0.j ,  0. -0.5j,
        -0. +0.j ,  0. +0.j , -0. +0.5j, -0.5+0.5j,  0.5+0.j ,  0. +0.j ,
         0.5+0.j , -0.5+0.5j, -0. +0.j , -0.5+0.j ,  0.5+0.5j, -0.5-0.5j,
         0. -0.5j, -0.5+0.5j,  0. -0.j , -0.5-0.j ,  0. +0.j , -0.5-0.5j,
        -0. +0.j , -0. +0.5j,  0.5+0.5j,  0.5+0.j ,  0.5+0.j ,  0.5-0.5j,
         0. -0.j , -0.5-0.j ,  0. +0.j , -0.5-0.j ,  0. -0.j ,  0.5-0.5j,
         0.5+0.5j,  0. -0.5j, -0.5+0.5j,  0. +0.j , -0.5+0.5j,  0.5+0.j ]],
      dtype=np.complex64), np.array([[-0.5-0.j , -0. +0.j ,  0. -0.j ,  0. -0.5j, -0.5-0.j ,  0. +0.j ,
         0. +0.j , -0.5-0.j ,  0.5+0.j , -0. +0.j ,  0. -0.j ,  0. -0.5j,
        -0.5+0.j , -0. +0.5j,  0. +0.j , -0. +0.5j,  0. +0.j ,  0. -0.5j,
         0. -0.j ,  0. -0.5j,  0. +0.j , -0. +0.j ,  0. -0.5j, -0.5-0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,
        -0.5+0.j ,  0.5+0.j ,  0. +0.j , -0. +0.j ,  0. +0.5j,  0. -0.j ,
         0. -0.j , -0. +0.j , -0. +0.5j,  0. -0.5j,  0. +0.j ,  0.5+0.j ,
        -0. +0.5j,  0.5+0.j ,  0. -0.5j,  0. +0.j , -0.5+0.j ,  0.5+0.j ],
       [ 0. +0.j , -0. +0.5j,  0.5+0.5j,  0. -0.j , -0. +0.j ,  0.5+0.j ,
        -0.5-0.5j,  0. -0.j ,  0. +0.j ,  0. +0.5j, -0. +0.5j,  0. -0.j ,
         0. -0.j ,  0. +0.j , -0.5+0.j ,  0. -0.j ,  0.5+0.j ,  0. -0.j ,
         0. -0.5j,  0. +0.j ,  0. -0.5j, -0.5+0.j ,  0. -0.j , -0. +0.j ,
         0. +0.5j,  0. +0.5j,  0.5+0.j , -0. +0.5j,  0.5+0.j ,  0. +0.5j,
         0. -0.j ,  0. -0.j , -0. +0.5j, -0.5-0.j ,  0. +0.j ,  0. -0.5j,
        -0.5-0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. -0.j , -0. +0.j ],
       [ 0. -0.j ,  0. -0.j ,  0. -0.j ,  0. +0.5j,  0.5+0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j , -0.5+0.j ,  0. +0.5j,  0. +0.j ,  0. -0.j ,
         0.5+0.j ,  0. -0.5j,  0. +0.j , -0. +0.j ,  0.5+0.j , -0. +0.5j,
        -0. +0.j ,  0. +0.5j,  0. +0.j , -0.5+0.j , -0. +0.5j,  0.5+0.j ,
         0. +0.j ,  0. +0.5j,  0.5+0.j , -0. +0.5j,  0. -0.j ,  0. +0.5j,
         0. -0.j , -0.5-0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
        -0.5+0.j , -0. +0.5j, -0. +0.j ,  0. -0.j ,  0. +0.5j,  0. +0.j ,
        -0. +0.j , -0.5-0.j ,  0. -0.j ,  0. -0.5j,  0.5+0.j ,  0. +0.j ],
       [ 0. -0.j , -0. +0.5j,  0. -0.j ,  0. +0.j ,  0.5+0.j ,  0.5+0.j ,
         0. +0.j ,  0.5+0.j , -0.5+0.j ,  0. +0.5j,  0. +0.5j, -0. +0.5j,
         0.5+0.j , -0. +0.j ,  0. +0.j ,  0. -0.5j,  0. -0.j ,  0. -0.j ,
         0. -0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.5j,  0.5+0.j ,
         0. +0.j ,  0. +0.5j,  0. -0.j ,  0. +0.5j,  0. -0.j ,  0. -0.j ,
         0.5+0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,
        -0.5+0.j ,  0. +0.5j,  0. +0.j ,  0. +0.5j,  0. -0.j , -0.5-0.j ,
         0. +0.j , -0.5-0.j ,  0. -0.j ,  0. -0.5j, -0. +0.j ,  0. +0.j ],
       [-0.5-0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
         0. -0.j ,  0.5+0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
        -0.5+0.j ,  0. -0.5j, -0.5+0.j , -0. +0.5j,  0.5+0.j ,  0. -0.5j,
         0. +0.j , -0. +0.5j,  0. +0.j ,  0. +0.j ,  0. -0.5j, -0.5-0.j ,
         0. +0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,
        -0.5+0.j , -0.5-0.j , -0. +0.j ,  0.5+0.j ,  0. -0.5j, -0. +0.j ,
        -0. +0.j ,  0. -0.5j,  0. -0.5j,  0. +0.j ,  0. -0.j ,  0.5+0.j ,
         0. -0.5j, -0. +0.j , -0. +0.5j,  0. +0.j ,  0.5+0.j ,  0. -0.j ],
       [ 0. +0.j , -0. +0.5j, -0.5-0.5j,  0. -0.5j,  0. +0.j ,  0.5+0.j ,
         0.5+0.5j,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. -0.5j, -0. +0.j ,
        -0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,
         0. -0.5j,  0. +0.j ,  0. -0.5j,  0.5+0.j ,  0. +0.j , -0. +0.j ,
         0. -0.5j, -0. +0.5j, -0.5-0.j ,  0. +0.5j,  0.5+0.j ,  0. -0.5j,
         0. -0.j ,  0. -0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.5j,
        -0.5+0.j ,  0. +0.j ,  0. +0.j , -0. +0.5j,  0. +0.5j,  0. +0.j ,
         0. +0.j , -0.5-0.j ,  0. -0.j , -0. +0.5j, -0. +0.j ,  0.5+0.j ],
       [-0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j, -0.5+0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j , -0.5+0.j , -0. +0.5j,  0. +0.j ,  0. -0.j ,
         0.5+0.j , -0. +0.5j,  0. -0.j , -0. +0.j , -0.5-0.j , -0. +0.5j,
         0. +0.j ,  0. -0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.5j,  0.5+0.j ,
         0. +0.j ,  0. +0.5j, -0.5-0.j , -0. +0.5j,  0. -0.j ,  0. -0.5j,
        -0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.5j,
        -0.5+0.j , -0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
         0. -0.j , -0.5-0.j , -0. +0.j ,  0. +0.5j, -0.5+0.j , -0. +0.j ],
       [ 0. -0.j ,  0. +0.5j,  0. +0.j , -0. +0.j , -0.5+0.j ,  0.5+0.j ,
         0. -0.j , -0.5-0.j , -0.5+0.j ,  0. +0.5j,  0. -0.5j, -0. +0.5j,
         0.5+0.j ,  0. -0.j , -0. +0.j ,  0. -0.5j,  0. +0.j ,  0. -0.j ,
         0. -0.5j, -0. +0.j ,  0. -0.5j,  0. -0.j ,  0. +0.5j,  0.5+0.j ,
         0. +0.j , -0. +0.5j,  0. +0.j ,  0. +0.5j,  0. -0.j , -0. +0.j ,
         0.5+0.j ,  0. -0.j , -0. +0.j , -0.5+0.j ,  0. +0.j ,  0. -0.j ,
        -0.5+0.j ,  0. +0.5j,  0. +0.j ,  0. +0.5j,  0. -0.j , -0.5+0.j ,
         0. -0.j , -0.5-0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j , -0. +0.j ],
       [ 0.5+0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,  0.5+0.j ,  0. +0.j ,
         0. -0.j , -0.5-0.j , -0. +0.j ,  0. -0.j ,  0. -0.j , -0. +0.j ,
         0.5+0.j , -0. +0.5j,  0.5+0.j ,  0. -0.5j,  0.5+0.j ,  0. -0.5j,
         0. -0.5j,  0. +0.5j, -0. +0.j , -0.5-0.j ,  0. -0.5j, -0.5+0.j ,
         0. -0.5j,  0. +0.j ,  0. -0.j ,  0. -0.5j,  0. -0.j , -0. +0.j ,
        -0.5+0.j , -0. +0.j , -0. +0.j , -0.5-0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. -0.5j,  0. -0.5j,  0. +0.j ,  0. -0.j , -0.5-0.j ,
        -0. +0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0.5+0.j ,  0. -0.j ],
       [ 0. -0.j , -0. +0.5j,  0.5+0.5j,  0. +0.5j,  0. +0.j ,  0.5+0.j ,
         0.5+0.5j, -0. +0.j ,  0.5+0.j ,  0. -0.5j,  0. -0.5j, -0. +0.5j,
         0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j , -0. +0.5j,  0. +0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j ,  0. -0.5j, -0.5+0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.5j,
        -0. +0.j , -0.5-0.j ,  0. -0.5j,  0. -0.j , -0. +0.5j,  0. -0.5j,
        -0.5-0.j ,  0. -0.j ,  0. +0.j ,  0. +0.5j,  0. -0.5j,  0. -0.j ,
         0. +0.j ,  0.5+0.j , -0. +0.j ,  0. -0.5j,  0. +0.j ,  0.5+0.j ],
       [ 0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.5j, -0.5+0.j , -0. +0.j ,
         0. -0.j ,  0. -0.j ,  0. -0.j ,  0. -0.5j, -0. +0.j ,  0. +0.5j,
        -0.5-0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j , -0.5+0.j , -0. +0.5j,
         0. +0.5j,  0. -0.5j,  0. -0.j ,  0. -0.j ,  0. +0.5j,  0.5+0.j ,
        -0. +0.5j,  0. -0.5j, -0.5-0.j ,  0. -0.j , -0. +0.j ,  0. +0.5j,
        -0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j , -0. +0.5j,  0. -0.5j,
        -0.5-0.j , -0. +0.5j,  0. +0.j ,  0. -0.j ,  0. -0.5j, -0. +0.j ,
        -0. +0.j ,  0.5+0.j ,  0. -0.j ,  0. -0.5j, -0.5-0.j , -0. +0.j ],
       [ 0. +0.j , -0. +0.5j,  0. -0.j ,  0. +0.j , -0.5-0.j ,  0.5+0.j ,
         0. -0.j ,  0.5+0.j ,  0. -0.j ,  0. -0.5j,  0. -0.5j,  0. +0.j ,
        -0.5+0.j ,  0. +0.j ,  0. +0.j , -0. +0.5j,  0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j , -0. +0.5j,  0.5+0.j , -0. +0.5j,  0.5+0.j ,
         0. +0.5j,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0.5+0.j , -0.5-0.j , -0. +0.j ,  0.5+0.j ,  0. +0.5j,  0. +0.j ,
        -0.5-0.j , -0. +0.5j,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0.5+0.j ,
        -0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. -0.j ],
       [ 0. -0.5j,  0. +0.j , -0. +0.j ,  0.5+0.j , -0. +0.5j,  0. -0.j ,
        -0. +0.j , -0. +0.5j, -0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,
         0. +0.5j, -0.5-0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0.5+0.j ,
         0.5+0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.5j,  0.5+0.j ,  0. +0.5j,
         0.5+0.j , -0. +0.j ,  0. +0.j , -0.5+0.j , -0. +0.j , -0. +0.j ,
        -0. +0.5j, -0. +0.j , -0. +0.j ,  0. -0.j ,  0. -0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j , -0.5-0.j , -0.5-0.j ,  0. +0.j , -0. +0.5j,
         0.5+0.j ,  0. -0.5j,  0.5+0.j ,  0. +0.j , -0. +0.5j,  0. +0.5j],
       [ 0. +0.j ,  0.5+0.j ,  0.5-0.5j,  0. +0.j ,  0. +0.j , -0. +0.5j,
         0.5-0.5j, -0. +0.j ,  0. -0.5j,  0.5+0.j , -0.5+0.j , -0.5+0.j ,
        -0. +0.j , -0. +0.j , -0. +0.5j,  0. -0.j ,  0. -0.5j,  0. +0.j ,
         0. -0.j ,  0. -0.j ,  0.5+0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j , -0.5+0.j ,  0. -0.5j,  0. +0.j ,  0. -0.5j, -0.5-0.j ,
         0. +0.j ,  0. +0.5j,  0.5+0.j ,  0. -0.5j, -0.5-0.j , -0.5+0.j ,
         0. -0.5j, -0.5+0.j ,  0. +0.j ,  0. -0.j ,  0.5+0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.j , -0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. -0.j ],
       [ 0. -0.j ,  0. -0.j , -0. +0.j , -0.5-0.j ,  0. -0.5j,  0. +0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j , -0.5-0.j ,
         0. -0.5j,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j, -0.5+0.j ,
        -0.5-0.j , -0.5-0.j ,  0. -0.j , -0. +0.j , -0.5-0.j ,  0. -0.5j,
        -0.5-0.j , -0.5-0.j ,  0. -0.5j,  0. -0.j , -0. +0.j , -0.5+0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j , -0.5+0.j , -0.5+0.j ,
         0. -0.5j, -0.5-0.j ,  0. -0.j , -0. +0.j ,  0.5+0.j , -0. +0.j ,
         0. +0.j , -0. +0.5j,  0. +0.j ,  0.5+0.j ,  0. -0.5j, -0. +0.j ],
       [ 0. +0.j ,  0.5+0.j , -0. +0.j ,  0. +0.j ,  0. -0.5j, -0. +0.5j,
        -0. +0.j ,  0. -0.5j, -0. +0.j ,  0.5+0.j , -0.5+0.j ,  0. +0.j ,
         0. -0.5j, -0. +0.j ,  0. +0.j , -0.5-0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. -0.5j, -0.5+0.j ,  0. -0.5j,
        -0.5+0.j , -0.5+0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,
         0. -0.5j,  0. +0.5j,  0. -0.j ,  0. -0.5j, -0.5-0.j ,  0. +0.j ,
         0. -0.5j, -0.5-0.j , -0. +0.j ,  0.5+0.j , -0. +0.j ,  0. -0.5j,
         0. +0.j ,  0. +0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ]],
      dtype=np.complex64), np.array([[-0. +0.5j,  0. -0.5j,  0. +0.j ,  0. -0.j , -0. +0.j , -0.5+0.j ,
         0. +0.j , -0. +0.j ,  0.5+0.j , -0. +0.5j,  0. +0.j , -0.5+0.5j,
        -0. +0.j , -0. +0.j ,  0.5+0.j ,  0. -0.5j, -0.5-0.5j,  0. +0.5j,
        -0.5-0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. -0.5j,  0. +0.j , -0. +0.5j, -0. +0.5j,  0.5+0.j ,  0. +0.j ,
        -0. +0.5j,  0. +0.j ,  0. -0.5j,  0. -0.j ,  0.5+0.j ,  0. -0.j ,
         0.5+0.j ,  0. -0.5j, -0.5-0.j ,  0.5-0.5j,  0. -0.5j,  0. +0.j ,
        -0.5-0.j , -0.5-0.j , -0.5+0.j ,  0. +0.j ,  0. -0.5j,  0.5+0.j ],
       [-0.5+0.j , -0.5-0.j , -0. +0.j ,  0. +0.j ,  0. +0.j , -0.5-0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.5j,  0. -0.5j,  0. +0.j ,  0.5-0.5j,
         0. +0.j , -0. +0.j ,  0. -0.5j, -0. +0.5j, -0.5+0.5j, -0. +0.5j,
        -0.5-0.5j,  0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,
        -0.5+0.j , -0. +0.j ,  0.5+0.j , -0.5-0.j ,  0. -0.5j,  0. +0.j ,
         0.5+0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
         0.5+0.j , -0.5-0.j , -0.5-0.j , -0.5-0.5j,  0. +0.5j, -0. +0.j ,
         0.5+0.j ,  0. +0.5j,  0. -0.5j,  0. -0.j , -0.5-0.j ,  0. -0.5j],
       [ 0. -0.5j,  0. +0.5j,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.5j,
        -0. +0.j ,  0. +0.j ,  0. -0.5j,  0.5+0.j , -0. +0.j ,  0.5+0.5j,
        -0. +0.j ,  0. +0.j , -0. +0.5j, -0.5-0.j , -0.5-0.5j,  0.5+0.j ,
        -0.5+0.5j,  0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,
        -0.5-0.j ,  0. -0.j , -0. +0.5j,  0.5+0.j , -0.5+0.j ,  0. +0.j ,
         0. -0.5j, -0. +0.j ,  0. -0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j ,
         0. -0.5j, -0.5+0.j ,  0. -0.5j, -0.5+0.5j, -0.5-0.j ,  0. -0.j ,
         0. -0.5j, -0. +0.5j, -0.5-0.j ,  0. +0.j ,  0. -0.5j,  0. +0.5j],
       [ 0. -0.5j,  0. -0.5j, -0. +0.j ,  0. -0.j ,  0. +0.j ,  0.5+0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.5j, -0. +0.5j,  0. +0.j , -0.5+0.5j,
         0. -0.j , -0. +0.j ,  0. -0.5j,  0. -0.5j,  0.5+0.5j,  0. -0.5j,
         0.5+0.5j,  0. -0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0.5+0.j , -0. +0.j ,  0. -0.5j,  0.5+0.j ,  0.5+0.j ,  0. -0.j ,
         0. +0.5j,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. -0.5j, -0. +0.j ,
        -0.5+0.j ,  0.5+0.j , -0.5-0.j ,  0.5-0.5j,  0. -0.5j,  0. +0.j ,
         0.5+0.j ,  0. -0.5j, -0.5-0.j ,  0. +0.j , -0. +0.5j,  0. -0.5j],
       [ 0. -0.5j,  0. -0.j ,  0.5+0.j , -0.5-0.5j,  0. -0.j ,  0.5+0.j ,
         0.5+0.j ,  0. +0.j , -0.5-0.j , -0. +0.j , -0. +0.j ,  0.5-0.5j,
        -0.5+0.j , -0. +0.5j,  0. +0.5j, -0. +0.5j,  0. -0.j ,  0. -0.5j,
         0.5+0.5j, -0. +0.j ,  0.5+0.j , -0. +0.j , -0.5+0.j ,  0. -0.j ,
         0. +0.5j, -0. +0.j , -0. +0.j ,  0. -0.5j, -0.5-0.j ,  0. -0.5j,
        -0. +0.j ,  0. -0.j ,  0. +0.5j, -0.5-0.5j, -0.5+0.j ,  0. +0.j ,
         0. +0.j , -0.5-0.j ,  0. -0.j ,  0. +0.j ,  0. +0.5j, -0. +0.5j,
        -0. +0.j ,  0. -0.5j,  0.5+0.j ,  0. +0.j ,  0. -0.j ,  0. -0.5j],
       [ 0.5+0.j ,  0. -0.j , -0.5-0.j ,  0.5-0.5j,  0. -0.j ,  0.5+0.j ,
         0.5+0.j ,  0. -0.j , -0. +0.5j,  0. +0.j ,  0. -0.j , -0.5+0.5j,
         0.5+0.j , -0.5+0.j , -0.5-0.j ,  0. -0.5j, -0. +0.j ,  0. -0.5j,
         0.5+0.5j,  0. +0.j ,  0. +0.5j,  0. +0.j , -0.5+0.j ,  0. +0.j ,
         0.5+0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j , -0. +0.5j,  0.5+0.j ,
         0. +0.j ,  0. -0.j ,  0.5+0.j ,  0.5-0.5j,  0. -0.5j,  0. +0.j ,
         0. +0.j ,  0. -0.5j,  0. -0.j ,  0. -0.j ,  0. -0.5j, -0.5-0.j ,
         0. +0.j ,  0.5+0.j ,  0. +0.5j,  0. +0.j ,  0. -0.j ,  0.5+0.j ],
       [ 0. +0.5j,  0. +0.j ,  0. +0.5j, -0.5-0.5j,  0. +0.j ,  0. -0.5j,
        -0. +0.5j,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j , -0.5-0.5j,
         0. +0.5j, -0. +0.5j, -0.5+0.j ,  0.5+0.j ,  0. -0.j , -0.5-0.j ,
         0.5-0.5j, -0. +0.j , -0.5+0.j ,  0. -0.j , -0. +0.5j,  0. +0.j ,
         0.5+0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0.5+0.j ,  0. -0.5j,
         0. +0.j ,  0. +0.j , -0. +0.5j,  0.5+0.5j, -0. +0.5j,  0. -0.j ,
         0. -0.j , -0. +0.5j,  0. +0.j , -0. +0.j ,  0.5+0.j ,  0. -0.5j,
         0. +0.j , -0.5+0.j ,  0.5+0.j ,  0. -0.j ,  0. -0.j ,  0.5+0.j ],
       [-0. +0.5j,  0. +0.j , -0.5-0.j , -0.5-0.5j, -0. +0.j , -0.5+0.j ,
         0.5+0.j ,  0. -0.j ,  0. -0.5j,  0. +0.j ,  0. -0.j ,  0.5-0.5j,
        -0.5+0.j , -0. +0.5j, -0.5-0.j , -0. +0.5j,  0. +0.j , -0. +0.5j,
        -0.5-0.5j,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0.5+0.j , -0. +0.j ,
        -0.5-0.j ,  0. +0.j ,  0. -0.j , -0.5+0.j , -0.5+0.j ,  0. -0.5j,
        -0. +0.j ,  0. -0.j ,  0. -0.5j,  0.5+0.5j, -0. +0.5j,  0. -0.j ,
         0. -0.j ,  0. +0.5j, -0. +0.j ,  0. +0.j , -0. +0.5j,  0. -0.5j,
         0. +0.j , -0.5-0.j ,  0.5+0.j ,  0. -0.j , -0. +0.j ,  0.5+0.j ],
       [ 0. -0.j ,  0. +0.j , -0.5-0.j ,  0.5+0.j ,  0. +0.j , -0.5-0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.5j,  0. -0.j ,  0. -0.j ,  0. +0.5j,
         0.5+0.j ,  0. -0.5j,  0. -0.5j,  0. -0.5j, -0.5+0.j ,  0. -0.j ,
        -0.5+0.j , -0. +0.5j, -0.5-0.j , -0. +0.5j,  0. +0.j , -0.5+0.j ,
         0. -0.5j, -0.5-0.j , -0. +0.5j,  0.5+0.j , -0. +0.j ,  0. -0.j ,
        -0. +0.5j, -0.5-0.j ,  0. -0.5j,  0.5+0.j ,  0.5+0.j ,  0. +0.5j,
         0.5+0.j ,  0. -0.5j, -0.5-0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.5j, -0.5-0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ],
       [ 0. +0.j ,  0. +0.j ,  0.5+0.j , -0.5-0.j ,  0. -0.j , -0.5-0.j ,
         0. +0.j ,  0. -0.j , -0.5+0.j ,  0. +0.j ,  0. -0.j ,  0. -0.5j,
        -0.5+0.j ,  0.5+0.j ,  0.5+0.j , -0. +0.5j, -0.5-0.j ,  0. -0.j ,
        -0.5+0.j , -0. +0.5j,  0. -0.5j,  0. -0.5j, -0. +0.j ,  0. +0.5j,
        -0.5-0.j ,  0. -0.5j,  0.5+0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
         0.5+0.j , -0.5-0.j , -0.5-0.j , -0.5-0.j , -0. +0.5j,  0. -0.5j,
         0.5+0.j , -0.5+0.j , -0.5-0.j ,  0. -0.5j,  0. -0.j ,  0. -0.j ,
         0. -0.j , -0.5-0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. -0.5j],
       [-0. +0.j ,  0. -0.j ,  0. -0.5j,  0.5+0.j ,  0. -0.j ,  0. +0.5j,
         0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. -0.j ,  0. +0.j , -0. +0.5j,
         0. -0.5j,  0. -0.5j,  0.5+0.j , -0.5-0.j , -0.5+0.j ,  0. -0.j ,
        -0.5-0.j , -0.5+0.j ,  0.5+0.j ,  0. -0.5j,  0. +0.j ,  0.5+0.j ,
        -0.5+0.j ,  0.5+0.j , -0. +0.5j, -0. +0.5j,  0. +0.j ,  0. -0.j ,
         0. -0.5j,  0.5+0.j ,  0. -0.5j, -0.5+0.j ,  0. -0.5j, -0.5+0.j ,
         0. -0.5j, -0.5-0.j ,  0. -0.5j,  0. +0.5j,  0. +0.j ,  0. -0.j ,
         0. +0.j ,  0.5+0.j , -0.5+0.j ,  0. -0.j ,  0. +0.j , -0. +0.5j],
       [-0. +0.j , -0. +0.j ,  0.5+0.j , -0. +0.5j,  0. +0.j ,  0.5+0.j ,
         0. +0.j ,  0. -0.j , -0.5-0.j ,  0. +0.j , -0. +0.j , -0.5-0.j ,
         0.5+0.j ,  0. -0.5j,  0.5+0.j ,  0. -0.5j, -0. +0.5j, -0. +0.j ,
        -0. +0.5j,  0. +0.5j,  0.5+0.j ,  0.5+0.j , -0. +0.j , -0.5-0.j ,
         0.5+0.j ,  0.5+0.j ,  0. -0.5j, -0. +0.5j,  0. -0.j ,  0. +0.j ,
        -0. +0.5j,  0. -0.5j,  0. +0.5j,  0. -0.5j,  0. -0.5j,  0. -0.5j,
        -0.5-0.j ,  0.5+0.j , -0.5-0.j ,  0.5+0.j , -0. +0.j ,  0. +0.j ,
         0. -0.j ,  0.5+0.j , -0.5+0.j ,  0. -0.j ,  0. +0.j ,  0. -0.5j],
       [-0. +0.5j,  0. -0.5j,  0. +0.j , -0. +0.5j,  0. -0.5j,  0. +0.j ,
        -0.5-0.j ,  0.5+0.j ,  0.5+0.j , -0. +0.5j, -0.5+0.j , -0.5-0.j ,
         0. +0.j , -0. +0.j ,  0.5+0.j , -0. +0.j ,  0. -0.5j,  0. +0.5j,
         0. -0.5j,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0.5+0.j , -0. +0.j ,
         0.5+0.j , -0. +0.j ,  0. +0.j ,  0. +0.5j,  0.5+0.j ,  0. +0.5j,
         0. +0.j ,  0. -0.5j, -0. +0.j , -0. +0.5j, -0. +0.5j,  0. +0.j ,
         0. +0.j ,  0.5+0.j ,  0. +0.j ,  0.5+0.j ,  0. -0.5j,  0. -0.5j,
        -0.5-0.j , -0.5+0.j ,  0. +0.j ,  0. -0.5j,  0. -0.5j,  0. +0.5j],
       [-0.5+0.j , -0.5+0.j , -0. +0.j ,  0. +0.5j, -0.5-0.j ,  0. +0.j ,
        -0.5+0.j , -0.5-0.j ,  0. -0.5j,  0. -0.5j, -0.5+0.j ,  0.5+0.j ,
         0. +0.j , -0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.5j,  0. +0.5j,
         0. -0.5j,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0.5+0.j ,  0. +0.j ,
         0. -0.5j,  0. -0.j ,  0. -0.j , -0.5+0.j ,  0. -0.5j, -0.5-0.j ,
        -0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.5j, -0.5-0.j ,  0. +0.j ,
         0. +0.j , -0. +0.5j,  0. +0.j , -0.5+0.j ,  0. +0.5j,  0.5+0.j ,
         0.5+0.j , -0. +0.5j, -0. +0.j ,  0.5+0.j , -0.5-0.j , -0.5-0.j ],
       [ 0. -0.5j,  0. +0.5j, -0. +0.j , -0. +0.5j,  0. -0.5j,  0. -0.j ,
         0. -0.5j, -0. +0.5j,  0. -0.5j,  0.5+0.j ,  0. -0.5j,  0.5+0.j ,
        -0. +0.j , -0. +0.j , -0. +0.5j,  0. +0.j ,  0. -0.5j,  0.5+0.j ,
         0. +0.5j,  0. -0.j , -0. +0.j , -0.5-0.j ,  0. -0.5j,  0. +0.j ,
         0. +0.5j,  0. +0.j ,  0. +0.j ,  0.5+0.j , -0.5-0.j ,  0. +0.5j,
        -0. +0.j ,  0. -0.5j,  0. +0.j ,  0. -0.5j, -0.5+0.j , -0. +0.j ,
         0. -0.j ,  0. -0.5j, -0. +0.j , -0.5-0.j , -0.5-0.j ,  0. +0.5j,
         0. -0.5j,  0. +0.5j,  0. +0.j ,  0. -0.5j,  0. -0.5j, -0.5-0.j ],
       [ 0. -0.5j,  0. -0.5j,  0. -0.j ,  0.5+0.j ,  0. +0.5j, -0. +0.j ,
        -0.5-0.j , -0.5-0.j , -0. +0.5j,  0. +0.5j, -0.5-0.j ,  0. +0.5j,
         0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0.5+0.j ,  0. -0.5j,
         0.5+0.j , -0. +0.j , -0. +0.j ,  0. -0.5j, -0.5+0.j ,  0. -0.j ,
         0. -0.5j,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0.5+0.j , -0. +0.5j,
         0. -0.j , -0.5-0.j ,  0. -0.j , -0.5+0.j , -0.5+0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.5j,  0. -0.j ,  0. -0.5j,  0. -0.5j, -0. +0.5j,
         0.5+0.j ,  0. -0.5j,  0. +0.j ,  0. -0.5j, -0. +0.5j, -0.5-0.j ]],
      dtype=np.complex64))

In [ ]:
#@title Verification
n = 4
m = 4
p = 4
rank = 48

verify_tensor_decomposition(decomposition_444, m, n, p, rank)

##Rank-96 decomposition of <4,4,8> over 0.5*C


This decomposition can be obtained by doubling the rank-48 decomposition of <4,4,4> provided above.

## Rank-61 decomposition of <4,4,5> over Z

In [ ]:
#@title Data

decomposition_445 = (np.array([[-0.,  0., -0.,  0.,  0., -1.,  0.,  1., -0.,  0.,  0.,  0., -0.,
        -0., -0., -0., -1., -0., -0.,  0., -0., -0., -0.,  0.,  0., -0.,
        -0., -0., -0., -0., -0., -0., -0.,  0.,  0., -0.,  1., -0., -1.,
         1., -0., -1., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,
         0., -0.,  1.,  0., -1., -1., -0.,  0., -1.],
       [ 0.,  1.,  0.,  0., -0.,  1., -1., -0.,  0.,  0., -1.,  0., -0.,
        -0.,  1., -0.,  0., -0., -1., -1., -0., -1., -1.,  0.,  0., -0.,
         0.,  0.,  1., -0., -0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,
        -0.,  1., -1.,  0., -0.,  0.,  0., -1., -0.,  0., -0.,  0., -0.,
         0.,  0., -0., -0.,  0.,  0., -1.,  0., -0.],
       [-0., -0., -0.,  0., -0.,  0., -0.,  1.,  0.,  0.,  0., -1.,  1.,
         0., -0., -0., -1.,  1.,  0.,  0., -0.,  0.,  0., -1.,  0.,  1.,
        -0.,  0., -0., -0., -0., -0., -0.,  0.,  0., -0.,  1., -0., -1.,
         1., -0.,  0., -0.,  0., -1.,  1.,  0.,  0.,  0.,  0., -1.,  1.,
        -0., -0.,  1., -0., -1., -1.,  0.,  0., -1.],
       [-0., -0.,  0.,  0., -0., -0., -1., -0., -0.,  0., -1., -0.,  0.,
         0., -0., -0., -1., -0.,  0., -1.,  0.,  0.,  0., -0.,  0., -0.,
         0., -0.,  1.,  0., -0.,  0., -0., -0., -0., -0., -0., -0.,  0.,
        -0.,  1., -1.,  0.,  0., -0., -1., -1.,  0.,  0., -0., -0.,  0.,
         0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.],
       [ 0., -0., -0.,  0., -1.,  0., -0.,  0., -0., -0.,  0.,  0.,  0.,
         0., -0., -0.,  0., -0., -0., -0.,  0., -0., -0., -0., -0.,  0.,
         0., -0.,  1., -0., -0.,  0., -0., -0., -0., -0., -0., -0.,  1.,
        -0.,  0., -1., -0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  1.,  0.,
        -1., -0.,  0., -0., -1.,  0., -0.,  0.,  0.],
       [-0., -0., -0.,  0.,  1., -0.,  1., -0., -1., -1.,  0., -1., -0.,
         1.,  0.,  0.,  0.,  1., -0., -0.,  1., -0., -0.,  0., -1., -0.,
        -0.,  0., -0.,  1., -0.,  1., -0., -0.,  0.,  0.,  1.,  0.,  0.,
        -0.,  0.,  0., -0.,  1., -1., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0., -1.,  0., -0., -0.,  0.],
       [-0., -0.,  0., -0., -0., -0.,  1., -0.,  0.,  0., -0., -1., -0.,
        -0.,  0., -0.,  0.,  1.,  0., -0., -0.,  0., -0.,  0.,  0., -0.,
         1., -0., -0.,  1.,  0., -0.,  0., -0.,  0.,  0.,  1.,  0.,  0.,
        -0., -0.,  0.,  0., -0., -1., -0., -0.,  0., -1., -0.,  0., -0.,
         0., -0., -0.,  0., -1.,  0.,  0.,  0.,  0.],
       [-0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0., -1.,  0.,  0.,
        -0., -0., -0.,  0., -0., -0.,  0., -0.,  0., -0., -0.,  0.,  0.,
        -1., -0.,  1., -0., -0., -0., -0., -0.,  1., -0., -0., -0.,  1.,
         0., -0., -1.,  0., -0.,  0.,  0., -1.,  0., -1., -0.,  1., -1.,
        -1.,  0., -1.,  0.,  0.,  0.,  0.,  0., -0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0., -0., -1.,
         0.,  1.,  1.,  0., -0., -0.,  0.,  0.,  0., -0., -0., -0.,  0.,
         0.,  1., -0.,  0., -0.,  0.,  0., -1., -0.,  1.,  0., -0., -0.,
         0., -0., -1.,  0., -1.,  1.,  0.,  0.,  0., -1., -0.,  1.,  0.,
         0.,  0.,  0., -0.,  0., -0.,  0., -1., -0.],
       [-0.,  1., -0., -1., -0., -0.,  0., -0.,  0., -1.,  0.,  0., -0.,
        -0.,  0., -0.,  0.,  1., -0.,  0., -0.,  0., -0., -0., -1., -0.,
        -0., -1., -0., -0.,  1., -0.,  1.,  0.,  0., -0., -0., -0.,  0.,
        -0., -0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,
        -0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0., -0., -0.,  0., -0., -0.,  0., -0.,  0.,  0.,
        -0., -0.,  1., -0.,  1.,  0.,  0.,  0., -0.,  0.,  0., -0., -0.,
         0., -0.,  0.,  0., -0.,  0.,  0., -1., -0.,  1., -0.,  0.,  0.,
        -0., -0.,  0.,  0., -1., -0., -0.,  0., -0., -1.,  0., -0., -0.,
        -0., -0., -0.,  1.,  0.,  0.,  0., -1.,  0.],
       [-0., -0., -0., -1., -0.,  0.,  0., -0., -0., -1.,  0.,  0.,  0.,
        -0., -0., -0., -1.,  1.,  1.,  0., -0., -0.,  1.,  0., -1.,  1.,
        -0.,  0., -0., -0.,  1., -0.,  1.,  0.,  0., -0., -0., -0., -0.,
        -0., -0.,  0., -0.,  0., -0.,  0.,  0., -1.,  0.,  0., -0., -0.,
         0., -0.,  0., -1.,  0.,  0., -0.,  0.,  0.],
       [-0., -0.,  0., -1.,  0.,  0., -0., -0., -0., -1.,  1., -0.,  0.,
        -0., -0., -0., -1., -0.,  1.,  0.,  0.,  1.,  0.,  0., -0., -0.,
        -0., -0.,  0., -0., -0.,  1.,  0.,  1., -0., -0., -0.,  0.,  0.,
         1., -1.,  0.,  0.,  0., -0., -0.,  0., -0.,  0., -1., -0., -0.,
         0.,  1.,  1.,  0., -1., -1.,  0.,  0.,  0.],
       [ 0.,  1., -1.,  0., -0., -0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,
        -0., -0.,  1.,  0., -0., -0., -1.,  0.,  0., -1., -0., -0.,  0.,
         0.,  0., -0., -0., -0., -0., -1., -0., -0., -0., -0., -0., -0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0., -1., -0.,  0., -0., -0.,  0.,
         0., -1.,  0., -0., -0.,  0., -1.,  0.,  1.],
       [-1.,  1., -1.,  0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,  0.,
         0., -0.,  1.,  0., -0., -0., -1., -1., -0., -1., -0.,  1.,  0.,
         0., -0., -0.,  0.,  0., -0., -1.,  0.,  1., -0., -0.,  0., -0.,
        -0.,  0., -0., -1., -1.,  0.,  0., -1.,  1., -1.,  0., -0.,  0.,
         0.,  0., -0., -0.,  0.,  0., -1., -1.,  1.],
       [ 1., -0.,  0., -1., -0.,  0.,  0., -0.,  0., -1., -0.,  0., -0.,
         0.,  0., -0., -1., -0.,  1.,  0., -0., -0., -0.,  0.,  0., -0.,
         0.,  0., -0., -0., -0., -0.,  0.,  1., -0.,  0.,  0.,  1., -0.,
        -0., -1.,  0., -0., -0.,  0.,  0., -1.,  0.,  0., -0., -0.,  0.,
         0., -0., -0.,  0.,  0., -1.,  0., -0., -0.]], dtype=np.float32),
 np.array([[ 0., -0.,  0., -0., -0.,  1., -0., -0., -0.,  0., -1., -0.,  0.,
         0.,  1.,  0.,  0.,  0., -0.,  0.,  0.,  1., -0., -0., -0., -0.,
        -0.,  0.,  1.,  0., -0., -0., -0.,  0.,  0., -0., -0., -0.,  0.,
        -0., -0.,  1., -0., -0., -0.,  0., -0., -0.,  0.,  1.,  0.,  0.,
        -0.,  0.,  0., -0.,  0., -0., -0.,  0., -0.],
       [-0.,  0., -0., -0.,  1., -0., -0.,  0.,  1.,  0., -1., -0.,  0.,
        -1.,  0., -0.,  0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,  0.,
        -0., -0.,  1., -0.,  0., -1., -0., -0., -0., -0.,  0.,  0., -0.,
        -0.,  0.,  1.,  0., -0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,
         0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.],
       [ 1.,  0., -0.,  0., -0., -0.,  0., -0., -0., -0.,  0.,  0., -1.,
         0., -0., -0., -0., -0.,  0., -0., -0., -0., -0.,  0., -0., -1.,
        -0.,  0., -0.,  0., -0.,  0., -0.,  1.,  1.,  1., -0., -1., -0.,
         1.,  0., -0., -1.,  0., -0., -0.,  0., -1.,  0.,  0., -1., -0.,
         1., -0.,  1., -1., -0., -1.,  0., -0.,  0.],
       [ 0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  1., -1.,  0.,
        -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0., -0.,  0., -0.,
        -0.,  0., -1., -0.,  0., -0.,  0.,  0.,  0.,  0.,  1., -0., -1.,
        -0., -0., -1., -0., -0.,  1., -0., -0., -0.,  0., -0., -1., -1.,
         0.,  0.,  1., -0.,  1., -0., -0.,  0., -0.],
       [ 1.,  0.,  0.,  0., -0.,  0., -0.,  1., -0., -0.,  0., -0.,  0.,
         0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0., -1., -0., -1.,
         0.,  0., -0.,  0., -0., -0., -0.,  1.,  1.,  1.,  0., -1.,  1.,
        -0., -0., -0., -1., -0.,  0.,  0., -0., -1.,  0., -1.,  0.,  1.,
         1.,  0., -0., -1.,  0., -1., -0.,  0., -0.],
       [-0.,  0.,  0.,  1.,  0., -0.,  0., -0., -0., -1.,  0.,  0., -0.,
         0., -0.,  0.,  0.,  0.,  1.,  0., -0.,  1., -0., -0., -0., -0.,
        -0., -0., -0.,  0., -0.,  1.,  0., -0.,  0., -0., -0., -0., -0.,
         0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,  1.,  0.,  0.,
        -0., -1.,  0.,  0.,  0., -0.,  0., -0., -0.],
       [ 0.,  0., -0.,  1., -0.,  0., -0.,  0.,  1., -1.,  0.,  0., -0.,
        -1., -1., -0.,  0.,  0.,  1., -0., -0., -0., -0.,  0., -0.,  0.,
         0.,  1., -0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0., -0.,  0.,
        -0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,
         0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.],
       [-0., -1.,  0.,  1., -0.,  0., -0.,  0., -0., -1.,  0., -0.,  0.,
         0.,  0.,  1., -0., -0.,  1., -0.,  0., -0.,  1.,  0., -1.,  0.,
         0., -0., -0.,  0., -0., -0.,  1., -0., -0., -0.,  0.,  0., -0.,
        -0.,  0.,  0.,  0., -1.,  0.,  0., -0., -1.,  0., -0.,  0., -0.,
        -0.,  0., -0.,  0.,  0.,  0., -0.,  1.,  0.],
       [-0.,  0., -0., -0., -0.,  0., -1.,  0.,  0., -0.,  0.,  0., -0.,
        -0., -0., -0.,  0., -0.,  0., -1., -1.,  0.,  1.,  1., -1.,  1.,
         1.,  0., -0.,  1., -1.,  0.,  0.,  0., -1.,  0., -0.,  0.,  0.,
         0., -0.,  0., -0.,  0., -0.,  1., -0., -0., -0.,  0.,  0., -1.,
         0., -0.,  0.,  0., -0., -0.,  1., -0., -0.],
       [-0.,  0., -1., -0.,  0.,  0.,  1., -0., -0.,  0., -0.,  0., -0.,
         0.,  0., -0.,  0.,  0., -0.,  1., -0.,  0., -0., -1., -0., -1.,
        -1., -0.,  0., -1.,  1.,  0.,  1., -0.,  1., -0., -0., -0., -0.,
         0.,  0.,  0., -1.,  0., -0., -1., -0., -1.,  0.,  0.,  0.,  1.,
        -0., -0.,  0.,  0., -0., -0.,  0., -0., -0.],
       [ 0.,  0.,  1.,  0., -0., -1.,  0., -1.,  0., -0., -0.,  0., -0.,
        -0., -1.,  1., -0.,  1.,  0.,  0.,  0., -1., -0.,  0., -1., -0.,
        -0., -0.,  0.,  1., -1., -1., -0.,  0.,  0., -1., -1.,  0., -0.,
         0.,  0.,  0.,  0., -1., -1., -0., -0.,  0.,  1., -1.,  1., -0.,
        -1.,  1.,  0., -0.,  0., -0., -0.,  0., -1.],
       [ 0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -1., -0., -0.,  0., -0.,
         1., -0., -0.,  0.,  1.,  0.,  0.,  0.,  0., -0.,  0., -1., -0.,
        -0.,  0.,  0., -0., -1., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,
        -0., -0.,  0.,  0., -1., -1.,  0.,  0., -0.,  1.,  0.,  1.,  0.,
        -1., -0.,  0., -0.,  0.,  0., -0.,  0.,  0.],
       [ 0., -0., -0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  1.,
        -0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0., -0., -0.,  1.,
        -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0., -0., -0.,
         0., -0., -0., -0.,  0.,  0.,  0., -0.,  1.,  1.,  0.,  1.,  0.,
        -1., -0.,  0.,  1., -0., -0., -0., -1.,  0.],
       [ 0.,  0.,  0.,  0.,  0., -0., -0., -0., -0., -0.,  0.,  1., -0.,
         0., -0.,  0., -0., -1.,  0., -0.,  1., -0.,  0.,  0.,  1.,  0.,
        -1.,  0., -0., -1.,  1., -0., -0.,  0.,  1.,  0.,  0.,  0., -0.,
        -0., -0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0., -0., -0.,  1.,
        -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.],
       [ 0.,  0., -0., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0.,
        -0., -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0.,  1.,  0.,  1.,
         1., -0., -0., -0.,  0., -0.,  0.,  0., -1.,  0., -0., -0.,  0.,
        -0.,  0., -0.,  1., -0., -0., -0.,  0.,  1., -0., -0., -0., -1.,
         0.,  0.,  0.,  1.,  0., -0.,  0.,  0., -0.],
       [-0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  1., -0.,  0.,
        -0.,  0.,  0.,  0.,  0., -1.,  0., -0., -1., -0.,  0.,  0.,  0.,
        -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0., -0.,  0.,  1.,  0.,
         0., -1.,  0., -0.,  0., -0.,  0., -0.,  0., -0.,  0., -0., -0.,
        -0., -0.,  0., -0.,  0., -0., -0., -0.,  0.],
       [ 0., -0.,  0., -1., -1., -0., -0., -0., -1., -0.,  1.,  0.,  0.,
         1.,  1.,  0.,  0.,  0., -1.,  0., -0.,  0., -0., -0., -0., -0.,
         0., -1., -1.,  0.,  1.,  1., -0.,  0.,  0.,  0.,  0., -0., -0.,
         0.,  0.,  0., -0., -0., -0., -0., -0., -0.,  0., -0.,  0.,  0.,
         1.,  0.,  0.,  0., -0., -0., -0.,  0., -0.],
       [-1., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0., -0.,
        -0., -0.,  0., -1., -0.,  0., -0., -0.,  0.,  0., -0., -0., -0.,
         0.,  0.,  0., -0., -0., -0.,  0.,  0., -1.,  0., -0.,  1., -0.,
        -1.,  0., -0.,  1., -0., -0., -0.,  0.,  1.,  0., -0., -0.,  0.,
         0.,  0., -1., -0., -0.,  1.,  0.,  0.,  0.],
       [-0.,  0., -0., -0., -0.,  0., -0.,  0., -0., -0.,  0., -0.,  0.,
         0.,  0., -0.,  0., -0., -0.,  1., -0., -0., -1., -1., -0., -1.,
         0., -0., -0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0.,
         0., -0., -0.,  0.,  0.,  0., -1., -1.,  0., -0.,  0., -0.,  1.,
         0., -0.,  0., -0.,  0., -0., -1., -0.,  0.],
       [-1.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,
        -0., -0., -0.,  0.,  0., -0.,  0., -0., -0., -0.,  1., -0.,  1.,
        -0.,  0.,  0., -0.,  0., -0.,  0.,  0., -1.,  0.,  0., -0.,  0.,
         0., -0.,  0.,  1.,  0.,  0.,  1., -0.,  1.,  0., -0.,  0., -1.,
         0., -0.,  0., -0.,  0., -0., -0., -0., -0.]], dtype=np.float32),
 np.array([[ 0.,  0.,  0., -0.,  0., -1., -0., -1., -0.,  0., -0.,  0., -0.,
         0., -0.,  0.,  0.,  0., -0.,  0.,  0., -1., -0.,  0., -0.,  0.,
         0.,  0., -0., -0., -0.,  0., -0., -0., -0., -0., -0.,  1.,  0.,
        -1., -1., -0.,  0., -0.,  0.,  0., -0.,  0.,  0., -1., -0., -0.,
         0.,  0., -0., -0., -0.,  1.,  0., -0., -0.],
       [ 0.,  0., -0., -0.,  1., -0.,  0.,  1., -0.,  0., -1.,  0.,  0.,
        -0.,  0.,  0.,  0., -0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,
        -0.,  0.,  1.,  0., -0.,  1., -0., -0.,  0., -0., -1., -1., -0.,
         1.,  1., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,  1.,  0.,  0.,
        -0.,  0.,  0., -0., -1., -1.,  0., -0., -0.],
       [ 0.,  1., -0., -1.,  0., -0., -0.,  1.,  0.,  0., -0., -0.,  0.,
        -0.,  1.,  1.,  0.,  0., -1.,  0.,  0.,  1.,  0., -0.,  0., -0.,
        -0.,  1.,  0., -0.,  0.,  0., -0., -0., -0., -0.,  0., -1., -0.,
         1., -0., -0., -0.,  0., -0.,  0., -0., -0., -0.,  1., -0., -0.,
         0.,  0.,  0., -0.,  0., -1., -0.,  0.,  1.],
       [ 0.,  0., -0.,  0.,  0., -0.,  0., -1.,  0.,  0., -0., -0.,  0.,
         0., -0.,  0., -0.,  0., -0., -0., -0., -0.,  0.,  0., -0., -0.,
        -0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0.,  0., -0.,  1.,  0.,
        -1.,  0.,  0.,  0.,  0., -0., -0., -0.,  0., -0., -1., -0., -0.,
        -0.,  1., -0., -0., -0.,  1., -0.,  0., -1.],
       [ 0.,  0.,  0.,  0., -0.,  1., -0.,  1., -1.,  0., -0.,  0., -1.,
         1., -1., -0.,  0.,  0., -0.,  0., -0., -0., -0.,  0., -0.,  0.,
        -0., -0., -1., -0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,  1.,
        -0., -0., -1.,  0.,  0.,  0., -0., -0.,  0.,  0., -0., -1., -0.,
         1., -0.,  0.,  0., -0.,  0.,  0., -0., -0.],
       [ 0., -0.,  0.,  0., -1.,  0.,  0., -1., -0.,  0., -0.,  0.,  1.,
        -1., -0., -0., -0.,  0.,  0., -0., -0., -0.,  0., -0., -0., -0.,
         0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0., -0.,  1., -0., -1.,
         0.,  0., -0., -0., -0.,  1., -0., -0.,  0., -0.,  0.,  1.,  0.,
        -1., -0.,  0.,  0., -0., -0.,  0., -0.,  0.],
       [-0.,  0., -1., -0., -0.,  0.,  0.,  0.,  1.,  0.,  0., -0., -0.,
        -1.,  0., -1.,  0.,  0., -0.,  0.,  1.,  0.,  0., -0.,  1., -0.,
        -0., -1., -0.,  0.,  1., -0., -1., -0., -0.,  0.,  0., -0.,  0.,
         0., -0., -0., -0.,  1., -0.,  0.,  0., -0., -0.,  0., -0., -0.,
         0., -0., -0.,  0.,  0., -0., -0.,  0.,  0.],
       [-0.,  0.,  1.,  1., -0.,  0.,  0., -0.,  0.,  1., -0., -0.,  0.,
         1., -0., -0.,  0., -0., -0., -0., -1.,  0.,  0.,  0., -1., -0.,
         0.,  0., -0.,  0., -1., -1.,  1.,  0., -0.,  0.,  0.,  0.,  0.,
        -0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0., -0., -0., -0., -0.,
         0., -1., -0., -0.,  0.,  0., -0.,  0., -0.],
       [-0., -0., -0.,  0., -0., -0.,  0., -1., -0.,  0., -0., -0.,  0.,
         0., -0., -1.,  1., -0., -0., -0., -0., -0., -1.,  1.,  0.,  1.,
         0.,  0., -0.,  0., -0., -0.,  0., -1.,  0., -1.,  0.,  0.,  0.,
         0., -0., -0.,  1., -0., -0., -0.,  0.,  1., -0., -0., -0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  1., -1., -1.],
       [-0., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0., -1.,
        -0.,  0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,  0.,  0.,  0.,
         0., -0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0.,
        -1.,  0., -0.,  0., -1., -1., -0., -0.,  0., -1.,  0., -1., -0.,
        -0., -0.,  1., -0.,  1., -0.,  0., -1.,  0.],
       [-0., -1.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  1.,
        -0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,  1., -1.,  0., -1.,
         0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,
         0., -0., -0., -1.,  0.,  0., -0.,  0., -1., -0.,  0., -0., -0.,
        -0., -0.,  0.,  0.,  0.,  0., -1.,  1.,  0.],
       [-0., -0.,  0.,  0., -0., -0., -0.,  1.,  0.,  0.,  0.,  0., -0.,
         0., -0.,  1.,  0., -0., -0.,  0., -0., -0., -0., -0., -0.,  0.,
         0., -0., -0., -0., -0., -0.,  0.,  1.,  0.,  1., -0.,  0.,  0.,
         1.,  0., -0., -0., -0., -0., -0.,  0.,  0., -0., -0., -0.,  0.,
        -0.,  0.,  0.,  0.,  0., -1.,  0.,  1.,  1.],
       [ 0.,  0., -1., -0., -0.,  0.,  1., -0.,  0., -0., -0., -1.,  0.,
        -0.,  0., -0.,  0., -0.,  0., -1.,  0.,  0., -0., -0.,  0.,  0.,
        -0.,  0.,  0.,  1., -0.,  0.,  0., -0.,  0., -0.,  1., -0., -0.,
        -0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,
        -0., -0.,  0., -0., -0.,  0., -1., -0.,  1.],
       [ 0.,  0.,  0., -0., -0., -0., -0., -0.,  0.,  0.,  0., -1., -0.,
         0., -0.,  0.,  0., -0.,  0.,  0., -1.,  0., -0.,  1.,  0., -0.,
        -0., -0., -0.,  0.,  0., -0.,  0., -0., -1.,  0.,  0.,  0., -0.,
         1., -0.,  0.,  1., -0., -0.,  0.,  0., -0., -0., -0., -0., -1.,
         0., -0., -1., -0., -1.,  0., -0., -0., -0.],
       [-0.,  1., -0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0.,  1.,  0.,
         0., -0., -0.,  0., -1.,  0., -0.,  1., -0., -1., -0.,  1., -0.,
        -0., -0., -0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0.,
        -0.,  0.,  0., -0.,  1.,  1.,  0.,  0., -0., -0., -0., -0.,  0.,
         0., -0.,  0., -0., -0.,  0.,  1.,  0., -0.],
       [ 0., -0.,  1., -0., -0.,  0., -1.,  0., -0., -0.,  0.,  1.,  0.,
        -0., -0.,  0.,  0., -0.,  0.,  1., -0.,  0., -0., -1.,  0.,  0.,
        -0., -0., -0., -1.,  0.,  0., -0.,  0.,  1.,  0., -1., -0., -0.,
        -1., -0., -0., -1.,  0.,  0.,  0.,  1.,  0., -0.,  0., -0.,  1.,
        -0.,  0.,  1., -0., -0., -0., -0.,  0., -1.],
       [-0.,  0., -1.,  0., -0.,  0., -0.,  1.,  0.,  0.,  0.,  0., -0.,
        -0.,  0.,  0.,  0.,  0.,  0., -1., -0.,  0., -0., -1.,  0.,  0.,
         0., -0.,  0., -0., -0., -0.,  0., -0., -0.,  0., -0., -0.,  0.,
        -0.,  0.,  0., -0., -0., -0., -1., -0.,  0., -0.,  0.,  0., -0.,
        -0.,  0., -0.,  0., -0.,  0., -1., -0.,  1.],
       [-0.,  0.,  0.,  0., -0., -0.,  0.,  1.,  0., -0.,  0.,  0., -0.,
        -0.,  0.,  0.,  0., -0.,  0., -0., -1.,  0., -0., -0.,  0.,  0.,
         1.,  0., -0., -1.,  0., -0.,  0.,  0., -1.,  0., -1.,  0.,  1.,
         1., -0.,  0.,  1., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,
        -0., -0., -1.,  0., -1.,  0., -0., -0., -0.],
       [-0.,  1.,  1.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0., -1.,
        -0., -0.,  1.,  0.,  0.,  0., -0., -0., -0., -1.,  1., -0.,  1.,
         0., -0.,  0., -0.,  0.,  0.,  1., -0., -0.,  1., -0.,  0.,  0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,  0.,
         0., -0., -0.,  1., -0.,  0.,  1.,  0., -0.],
       [-1.,  0.,  1., -0.,  0., -0.,  0., -1.,  0., -0.,  0., -0., -0.,
        -0., -0.,  0., -0., -0., -0., -0., -0.,  0., -0.,  0.,  0.,  0.,
         0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,  0.,
        -1.,  0., -0., -1., -0., -0., -0., -0., -0., -0.,  0., -0., -0.,
         0., -0., -0.,  0., -0.,  1., -0.,  0., -1.]], dtype=np.float32))

In [ ]:
#@title Verification
n = 4
m = 4
p = 5
rank = 61

verify_tensor_decomposition(decomposition_445, n, m, p, rank)

## Rank-85 decomposition of <4,4,7> over Z


In [ ]:
#@title Data

decomposition_447 = (np.array([[ 0., -0.,  0., -0., -0.,  0., -0., -0.,  0., -0.,  1., -0.,  0.,
         0., -0.,  0., -0., -0.,  0.,  0.,  0., -1., -0.,  0.,  0., -0.,
        -0.,  1.,  0., -0.,  0.,  0., -1., -0., -1., -0.,  0.,  0., -0.,
        -0.,  0., -0.,  0., -0., -0.,  0., -0.,  1.,  0.,  0., -0.,  1.,
         0.,  1.,  1., -0., -0., -1., -1.,  0., -0.,  0.,  0., -0., -0.,
         0.,  0.,  1., -0.,  0., -0.,  0., -1., -0., -0.,  0., -1., -0.,
         0.,  0.,  0., -0., -0., -0.,  1.],
       [ 0.,  0.,  0., -0.,  0., -1.,  0.,  0.,  0.,  0.,  1.,  0., -1.,
        -0.,  1.,  1., -0.,  0., -0., -0., -0., -0.,  0.,  0., -0.,  0.,
        -0., -0., -0., -0., -0., -0.,  0., -0., -0.,  0., -0., -0., -0.,
         0., -1.,  1., -1.,  0.,  1., -1.,  0., -0., -0., -0., -0., -0.,
         1., -0.,  0.,  0.,  0., -0., -1., -1., -0.,  0., -0.,  1.,  0.,
         0., -0.,  1.,  0.,  0.,  0., -1., -0., -0., -0.,  0., -0.,  1.,
         0., -0., -1.,  1.,  0.,  1.,  0.],
       [ 0., -0., -0., -0.,  0.,  1.,  0.,  0., -0., -0.,  0., -0.,  0.,
         0.,  0.,  0.,  0., -0., -0., -0., -0., -0.,  0., -0., -0.,  0.,
         0., -0.,  0., -1.,  0., -0.,  0.,  0., -0., -0., -0., -0.,  0.,
        -0.,  1.,  0., -0., -0., -1.,  1.,  0.,  0.,  0.,  0., -0., -0.,
        -1., -0.,  0., -0., -0., -0., -0.,  0.,  0., -0.,  0., -1.,  0.,
         1., -1.,  0.,  0.,  1.,  0.,  0., -0.,  0., -0., -0., -0., -1.,
         1., -0., -0.,  0.,  1., -1., -0.],
       [-0., -0., -0., -0., -0.,  0.,  0., -0., -0., -0., -0.,  0., -0.,
        -0., -1.,  0., -0., -0.,  0., -0.,  0., -1., -0.,  0., -0., -0.,
         0., -0., -0.,  1., -0., -1.,  0.,  0., -1.,  0.,  0.,  0.,  0.,
        -0.,  0., -1., -0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0., -0.,
        -0., -0.,  1., -0.,  0., -1., -0.,  1.,  0., -0.,  0.,  0.,  0.,
        -1., -0., -0., -0., -1.,  0.,  1., -1.,  0., -0., -0., -1., -0.,
        -1.,  0.,  1., -1.,  0., -0., -0.],
       [ 0.,  0., -0., -0., -1., -0., -0., -0.,  1.,  1.,  0.,  0.,  0.,
         0., -0.,  0., -0.,  0.,  1.,  0.,  0., -0., -0., -0.,  0., -0.,
         0.,  0.,  0.,  0.,  0., -0., -0., -0.,  1., -0., -0.,  0., -0.,
        -0.,  0., -0., -0.,  0.,  0., -0., -0.,  0., -0., -0.,  1.,  0.,
        -0.,  0.,  0., -1., -1., -0., -0., -0.,  0.,  0., -0., -0.,  0.,
        -0., -0.,  0., -1.,  0., -0., -0., -0., -1., -1.,  0., -0., -0.,
        -0.,  0., -0.,  0.,  0., -0.,  0.],
       [-0.,  0., -1., -0.,  0., -0.,  0.,  0.,  1.,  0.,  0., -1.,  0.,
        -0.,  0., -0., -1., -0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,
        -0.,  0., -0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,
         0.,  0.,  0., -0., -0., -1., -0., -0.,  0., -1., -1., -0.,  0.,
        -1.,  0., -0.,  0., -0., -1.,  1.,  1.,  1.,  1.,  0.,  0., -1.,
         0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,
        -1., -0., -0., -1., -0.,  0.,  0.],
       [-0., -0., -0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  1.,  0.,
        -0.,  0., -0.,  0.,  1.,  0., -0., -0.,  0., -1., -1., -0., -0.,
        -0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0.,
        -1., -0.,  0., -0.,  1., -0.,  0.,  0.,  0.,  1.,  0., -0., -0.,
         1., -0., -0.,  0., -1.,  0.,  0., -1., -0., -0.,  0., -0., -0.,
         0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -1., -0.,
        -0., -0.,  0., -0.,  0., -0., -0.],
       [-0.,  0.,  1., -0.,  0., -0., -0., -0., -0.,  1.,  0.,  0., -0.,
        -0., -0.,  0.,  1., -1.,  1.,  0.,  0., -0.,  1.,  1., -0., -0.,
        -1.,  0., -1., -1.,  0., -0., -0.,  0.,  1., -0.,  1., -1., -0.,
         1.,  0., -0., -0., -1., -0.,  0.,  0.,  0., -0.,  1., -0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0., -0., -1., -1., -0.,  0., -0.,
         1.,  0.,  0.,  0.,  0.,  1., -0.,  1.,  0.,  0., -0.,  1.,  0.,
         1.,  0., -0., -0., -0.,  0.,  0.],
       [ 0.,  0.,  0., -1., -0.,  0., -0., -0.,  0., -1., -1.,  0., -0.,
         0.,  0.,  0., -1., -0.,  0.,  1.,  1.,  1.,  0.,  0.,  0., -1.,
        -0.,  0.,  1., -0.,  0., -0., -0., -0., -0., -1.,  0.,  1., -0.,
        -0.,  0., -0., -0., -0., -0., -0., -0.,  0.,  0.,  0., -0.,  0.,
        -0.,  0.,  0., -0., -0., -0., -0.,  0., -0.,  0., -0.,  0., -0.,
         0., -0.,  0., -0., -0., -0.,  0., -0., -0., -0., -1., -0., -0.,
         0.,  0.,  0., -0.,  0., -0.,  0.],
       [-0.,  1.,  1., -0.,  0.,  1.,  0., -0.,  0., -0., -1.,  1.,  1.,
        -0., -1.,  0., -0.,  0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,
         0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0.,
         0.,  1.,  0.,  1.,  0.,  0., -0., -1., -0.,  0., -0.,  0., -0.,
         0., -0., -0.,  0.,  0., -0., -0.,  0., -1.,  0., -1., -1., -0.,
        -0.,  0., -1., -0., -0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0.,
         1., -0.,  1.,  0., -0.,  0.,  0.],
       [-0., -1., -1.,  0.,  0., -1.,  0.,  0., -0., -0., -0., -1.,  0.,
        -0., -0., -0., -0., -1., -0., -0.,  0.,  0.,  1., -0., -0.,  0.,
         0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0., -1.,  0.,  0.,
        -0., -1.,  0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0., -0.,  0.,
         0.,  0., -0., -0., -0.,  0., -0.,  1., -0., -0.,  1., -0.,  0.,
        -1.,  1., -0.,  0., -1.,  0.,  1.,  0.,  0.,  0.,  0., -0., -0.,
        -1., -0., -0.,  0.,  0., -0., -0.],
       [-0., -0., -0.,  0., -0., -0.,  0., -0., -0., -1.,  0., -0.,  0.,
        -0., -0., -0., -1.,  1.,  0., -0.,  1.,  1., -1.,  0., -0.,  0.,
         1.,  0.,  1., -0.,  0., -0., -0.,  0., -0.,  0., -0.,  1.,  0.,
         0., -0., -0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0., -0.,
        -0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  1., -0.,  0., -0.,  0.,
        -0.,  0., -0., -0.,  1.,  0., -1.,  0.,  0.,  0.,  0.,  0., -0.,
        -0.,  0., -1., -0., -0., -0.,  0.],
       [-0., -0., -0.,  0.,  0.,  0.,  1.,  1., -0.,  0., -0., -0., -0.,
        -0.,  0., -0.,  0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,
        -0.,  1., -0.,  0., -1.,  0.,  0.,  0., -0.,  0., -0., -0.,  1.,
         1.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  1., -0.,
        -0.,  1., -0.,  0.,  0.,  0., -1.,  0., -0., -1., -0.,  0.,  0.,
         0., -0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0., -1.,  0.,  0.,
         0., -1.,  0.,  0., -0., -0., -0.],
       [ 0.,  1., -0.,  1., -0.,  0., -0.,  0., -0., -0., -0.,  0., -0.,
         1., -0.,  1.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  1.,  0.,
        -0., -0.,  0., -0., -0., -0.,  0., -0., -0.,  0., -0.,  0.,  1.,
         0., -0., -0.,  0.,  0., -0.,  0., -1., -0.,  0.,  0., -0.,  0.,
         0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0., -1.,
        -0.,  0., -0.,  0., -0.,  0.,  0., -0., -0.,  1.,  0., -0., -0.,
        -0., -0.,  0., -0.,  0.,  1., -1.],
       [ 0., -1., -1.,  0., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0.,
         0., -0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,
        -0., -0., -0., -0., -1., -0.,  0., -1., -0.,  1., -1.,  0., -0.,
         0.,  0., -0., -0.,  1., -1., -0., -0., -1., -0.,  0., -0.,  0.,
         0., -0.,  0.,  1.,  0., -0.,  0.,  0., -0., -0., -0., -1., -0.,
         0., -0., -0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0., -1.,
         0., -0., -0., -0.,  1., -1., -0.],
       [ 1., -0.,  1.,  0., -0., -0., -0.,  1.,  0., -0.,  0., -0., -0.,
         0., -1.,  0., -0.,  0., -0.,  0., -0., -0.,  0., -0.,  0.,  1.,
         0., -0.,  0.,  0.,  0., -1.,  1., -0., -0.,  0.,  1.,  0., -0.,
         1.,  0., -1., -0., -1., -0., -0., -0., -0., -0., -0., -0.,  0.,
         0., -0., -0.,  0., -0., -0.,  0., -0.,  0., -1., -0., -0., -0.,
        -0.,  0., -0.,  1.,  0.,  1., -0., -0., -0.,  0.,  0., -0., -0.,
        -0., -1., -0., -1.,  0., -0., -0.]], dtype=np.float32), np.array([[ 0., -0., -0., -1., -0.,  0.,  0., -0., -0.,  1., -0.,  0., -0.,
         0.,  0., -0., -0., -0.,  0.,  1., -0.,  0.,  0.,  0., -1., -0.,
        -1.,  0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,  0., -1.,  1.,
         0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0., -0.,  0.,
         0., -1.,  0.,  0., -0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,
        -0.,  0., -0., -0.,  0.,  0.,  0., -0.,  1.,  1.,  0.,  0., -0.,
         0.,  0., -0.,  0.,  0., -0., -1.],
       [-0.,  0., -0., -0.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,  0.,
         0., -0.,  0., -0.,  0., -0.,  0., -0.,  1., -0., -0., -0.,  0.,
        -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  1.,  0., -0., -0.,  0.,
         0., -0., -0.,  0.,  0., -0., -0.,  0.,  0., -0.,  0., -0.,  1.,
         0.,  1., -0., -0.,  0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,
        -1.,  1., -0.,  0.,  1., -0., -0., -1.,  0.,  0.,  0., -0.,  0.,
         0.,  0.,  0., -0., -0.,  0., -0.],
       [-0., -0., -0., -0., -1.,  0.,  1.,  0., -0., -1.,  0., -0., -0.,
        -0., -0.,  0., -0.,  0., -0.,  0.,  0.,  1., -0., -0., -0.,  0.,
         1., -0.,  0.,  0.,  0., -0.,  0., -0.,  1., -0.,  0.,  1., -0.,
        -0., -0., -0., -0., -0.,  0.,  0., -0., -0., -0.,  0.,  0., -0.,
        -0., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,
        -1.,  1.,  0.,  0.,  1.,  0., -0., -1.,  0.,  0., -0., -0., -0.,
         0., -0., -0., -0., -0., -0., -0.],
       [ 0.,  0.,  0., -0., -0., -0., -0., -0.,  0., -0.,  1., -0.,  0.,
         0., -0.,  0.,  0., -0., -0., -0., -0., -0., -0.,  0., -0., -0.,
         0.,  1.,  0., -0., -0.,  0., -0., -0., -0., -0., -0.,  0.,  0.,
        -0.,  0.,  0., -1., -0.,  0.,  0.,  0., -0.,  0.,  0., -1., -0.,
        -0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0.,
         0., -0.,  1., -0., -0., -0.,  0.,  0., -1.,  0., -0.,  0., -0.,
        -0.,  0., -0., -0.,  0.,  0., -0.],
       [ 0., -0., -0., -0.,  0., -0.,  0., -0., -0., -1., -1.,  0., -0.,
        -0., -0.,  0.,  0., -0.,  0., -1., -0., -0.,  0.,  0., -0.,  0.,
         1.,  0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,  1.,  0.,
        -0., -0.,  0.,  1.,  0.,  0., -0., -0., -0., -0.,  0., -0., -0.,
         0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  0., -0.,
         0., -0., -1., -0.,  0., -0.,  0.,  0.,  0.,  0.,  1.,  0., -0.,
         0.,  0.,  0.,  0., -0., -0., -0.],
       [-0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  1.,  0.,  0.,
        -0.,  0., -0., -0., -0., -0., -0.,  0., -0., -0.,  0.,  0., -0.,
         0.,  0., -0., -0., -1., -0., -0., -1., -0., -1.,  0.,  0., -0.,
        -0.,  0., -0., -1., -0.,  0.,  0.,  0., -1.,  0.,  0., -0., -0.,
         0.,  1., -0.,  1., -0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,
         0., -0.,  1.,  0., -0.,  0., -0., -0.,  1., -0., -1.,  0., -0.,
         0.,  0., -0., -0., -0., -0.,  0.],
       [-1., -0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0., -0., -0.,  0.,
        -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  1.,
        -0.,  1., -0., -0.,  0.,  0.,  1.,  0.,  0.,  0., -0., -0., -0.,
        -0., -0., -0.,  0.,  0., -0.,  0., -0.,  0., -0., -0., -1.,  0.,
         0., -1., -0., -0., -0., -0.,  0., -0., -0.,  0., -0., -0., -0.,
         0., -0., -0.,  1., -0.,  0., -0.,  0.,  0., -0.,  1.,  0.,  0.,
         0., -0.,  0., -0.,  0., -0.,  0.],
       [-0., -0., -0.,  1., -0., -0., -0., -0., -1.,  0.,  0., -0., -0.,
        -0.,  0., -1., -1., -0., -0., -1.,  0., -0., -0., -0.,  1., -0.,
         1., -0., -0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0.,  1.,  0.,
        -0., -0.,  0., -0., -0.,  0.,  0., -0., -0.,  0., -0.,  0., -0.,
        -0., -0., -0.,  0.,  0., -0.,  0., -0.,  1.,  0.,  0., -0.,  0.,
         0., -0., -0.,  0., -0.,  0., -0.,  0., -1., -1., -0., -0., -0.,
        -0., -0., -0.,  0., -0., -0., -0.],
       [-0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0.,
         0., -0.,  1., -0., -0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,
        -0., -1., -0.,  0., -0., -0., -0.,  0.,  0.,  0., -0., -0., -1.,
        -0.,  0.,  0., -0., -0.,  1., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
         1., -1.,  0.,  0.,  0., -0., -0.,  1., -0., -0., -0.,  0.,  0.,
         1., -0.,  0., -0.,  0.,  0.,  1.],
       [-0., -1., -1.,  0., -0., -0., -1.,  0., -0., -0.,  0.,  0., -1.,
        -0., -0., -0., -0., -0., -0., -0., -0.,  0., -0.,  0.,  0.,  0.,
         0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0., -0.,  1.,  0., -1.,
         0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,
         0.,  0., -0., -0., -0., -0., -0., -0.,  0., -1.,  0., -0., -0.,
        -0.,  0.,  0.,  0.,  0.,  1.,  0., -0., -0., -0., -0., -0., -0.,
        -0.,  0.,  0., -0., -0., -0., -0.],
       [-0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  1., -0.,  0.,  0.,  0.,
         0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,
        -0., -1.,  0., -0., -0., -0.,  0., -0., -0., -0., -0., -0., -0.,
         0.,  0.,  0.,  1.,  0., -0.,  0., -0., -0., -0.,  0., -0.,  0.,
         0.,  0.,  0., -0., -0.,  0.,  1.,  0., -0.,  0., -0., -0.,  1.,
        -0.,  0., -1., -0.,  0., -0., -0., -0.,  1.,  1., -0., -0., -0.,
         0.,  0.,  0.,  0., -0., -0.,  0.],
       [-0., -0., -0., -1., -0.,  0., -0.,  0., -0., -0., -0.,  0., -0.,
        -0.,  0., -0.,  1.,  0., -0.,  1.,  0., -0., -0.,  0.,  0., -0.,
        -1., -0.,  0., -0., -0., -0.,  0.,  0., -0., -0.,  0., -1., -0.,
         0.,  0.,  0., -1.,  0.,  0.,  0., -1.,  0., -0.,  0., -0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -1., -0., -0., -0., -0.,
        -0.,  0., -0.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,  0., -0.,
         0.,  0., -0.,  0., -0., -0.,  0.],
       [ 0., -0., -0.,  0., -0., -0., -0., -0., -1.,  0.,  0.,  0., -0.,
        -1., -0., -0., -0., -0.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,
        -0., -0.,  0., -0., -0.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,
         0.,  0., -0.,  1.,  0.,  1.,  0., -0., -0., -0., -0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0., -1., -1.,
        -0.,  0.,  0.,  0., -0.,  0.,  0., -0., -1., -1., -0., -0., -1.,
        -0., -0.,  0., -0., -1., -1., -0.],
       [-0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,
        -1., -1.,  0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.,
         0., -1., -0., -0.,  0.,  1.,  0., -0., -0.,  0., -0.,  0.,  0.,
        -0., -0., -1., -0., -0., -0., -0.,  0.,  0.,  0., -0., -0., -0.,
         0., -0., -0., -0., -0.,  0.,  1.,  0.,  0.,  0.,  0.,  0., -0.,
         0., -0., -1.,  0., -0., -0.,  0.,  0., -0., -0.,  0.,  0., -0.,
         0.,  0., -0., -1.,  0.,  0.,  0.],
       [-0.,  0., -1.,  1.,  0., -0., -0.,  0., -1.,  0.,  0., -0.,  0.,
        -0.,  0.,  0., -1., -0., -0., -1., -0., -0., -0., -0.,  1.,  0.,
         1., -0., -0., -1., -1., -0., -0.,  0., -0.,  0.,  0.,  1., -1.,
         1.,  0.,  0., -0.,  1.,  0., -0., -0.,  1., -1., -0.,  0., -0.,
        -0.,  0., -0., -0.,  0.,  1.,  0.,  0.,  1., -1., -1.,  0.,  0.,
         0., -0., -0.,  0., -0.,  0.,  0.,  0., -1., -1., -0.,  1., -0.,
         1., -0., -0.,  0.,  1.,  0.,  1.],
       [ 0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0., -0.,
         0., -0., -0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,
         0., -0.,  0.,  1., -0.,  0.,  0., -0.,  0., -0., -0.,  0., -0.,
         0.,  0., -0.,  0.,  0., -0.,  0., -0., -1., -0., -0.,  0., -1.,
         0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  0.,
         1., -1., -0., -0.,  0., -0., -0.,  1., -0., -0.,  0., -1.,  0.,
        -0.,  0.,  0., -0., -1.,  0., -0.],
       [ 0.,  0.,  0.,  0., -0.,  0., -1., -0., -0., -0., -0.,  0.,  0.,
         0., -0.,  0., -0.,  0., -0., -0., -0.,  0., -0., -0.,  0.,  0.,
         0.,  0.,  0., -0.,  1.,  0., -0.,  0.,  0., -0.,  1.,  0.,  0.,
        -1., -0., -0.,  0., -1., -0.,  0., -0.,  0., -0., -0.,  0., -0.,
         0., -0., -0., -0., -0.,  0., -0., -0.,  0.,  0., -0., -0.,  0.,
        -0., -1.,  0., -0.,  0.,  1., -0., -0., -0., -0.,  0.,  0.,  0.,
         0.,  0.,  0., -0.,  0.,  0.,  0.],
       [-0., -0., -0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0., -1.,  0.,
         0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,
        -0., -1., -0., -0., -0., -0.,  0., -0.,  0.,  0., -0., -0., -0.,
         0., -0.,  0.,  1.,  0.,  1., -0., -0.,  0., -0., -0., -0.,  0.,
        -1., -0., -0.,  1.,  1.,  0.,  1.,  0.,  0., -0., -0.,  0., -0.,
        -0.,  0., -1., -0.,  0., -0.,  0., -0.,  1., -0.,  0.,  0., -0.,
         0.,  0., -0.,  0., -0.,  0.,  0.],
       [-0.,  0., -0.,  0.,  0., -1., -0.,  0., -0., -0., -0., -0.,  0.,
        -0.,  0.,  0., -0., -1., -0.,  1.,  0.,  0.,  0., -0.,  0., -0.,
        -1., -0., -1., -0., -0., -0.,  0.,  0., -0.,  1.,  0., -1., -0.,
         0., -0.,  0., -1.,  0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0.,
        -0.,  0.,  0., -0., -0.,  0.,  0.,  0.,  0., -0., -0.,  1.,  0.,
        -0.,  0., -0.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,  0., -0.,
         0., -0., -0.,  0., -0.,  0.,  0.],
       [ 0.,  0., -0.,  0., -0.,  1., -0., -0.,  0., -0.,  0., -0.,  0.,
         0., -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,
        -0., -0.,  0., -0., -0.,  0.,  0.,  1., -0., -0.,  0., -0., -0.,
         0.,  0., -0.,  1., -0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,
        -0.,  0.,  0., -1., -1.,  0.,  0., -0.,  0., -0., -0., -1.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0., -0., -0., -1.,  0., -0., -0.,  0.,
         0., -0.,  0.,  0., -1.,  0.,  0.],
       [-0.,  0.,  0., -0., -0., -0., -0.,  0., -0., -0.,  0., -1.,  0.,
         0., -1.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.,
         0., -1.,  0.,  0.,  0.,  1.,  0.,  1., -0., -0., -0.,  0.,  0.,
        -0.,  0., -1., -0., -0., -0.,  1.,  0., -0.,  0., -0., -0., -0.,
         0., -0., -0., -0., -0., -0.,  1., -1., -0.,  0.,  0.,  0., -0.,
         0., -0., -1.,  0.,  0.,  0., -1.,  0., -0., -0.,  0.,  0.,  1.,
        -0.,  0.,  1., -1.,  0., -0., -0.],
       [-0.,  0.,  0.,  1.,  0.,  0., -0.,  0., -1.,  0.,  0., -0.,  0.,
        -0., -0.,  0., -1., -0., -0., -1., -0., -0., -0.,  0.,  1.,  0.,
         1., -0., -0.,  0.,  0.,  1.,  1.,  0., -0.,  0., -0.,  1., -1.,
        -0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,  0.,  1.,  0., -0.,
        -0.,  0.,  1., -0., -0.,  1.,  0., -0., -0., -1.,  0.,  0.,  0.,
         0., -0., -0.,  0., -0.,  0.,  0.,  0., -1., -1., -0., -0.,  0.,
        -0.,  1.,  0., -0., -0.,  0.,  1.],
       [ 0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0.,  0., -0., -0., -0.,
        -0., -0., -0., -0., -0.,  0., -0.,  0.,  0., -0., -0.,  0., -0.,
         0., -0.,  0., -0., -0., -1., -1., -0.,  0., -0.,  0.,  0., -0.,
         0.,  0., -0.,  0., -0.,  0., -0., -0.,  0.,  0.,  0., -0., -1.,
        -0.,  0., -1.,  0.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,
         1., -1., -0., -0., -1., -0., -0.,  1., -0., -0.,  0.,  0., -0.,
        -0.,  0.,  0., -0.,  0., -0., -0.],
       [ 0.,  0.,  0.,  0., -0., -0., -1., -0., -0., -0.,  0.,  0.,  0.,
         0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,
        -1.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,
         0., -0., -0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,
        -0., -0.,  0., -0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,
         1., -1.,  0., -0., -1.,  1., -0., -0., -0., -0., -0.,  0.,  0.,
        -0., -1., -0.,  0., -0.,  0.,  0.],
       [-0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0., -1.,  0.,
        -0.,  0., -0., -0.,  0., -1.,  0., -0.,  0., -1.,  0.,  0.,  0.,
        -0., -1., -0.,  0.,  0., -0., -0.,  0.,  0.,  0.,  0., -0., -0.,
        -0., -0.,  0.,  1.,  0., -0., -0., -0.,  0.,  0.,  0., -0.,  0.,
         0.,  0., -0., -0., -0.,  0.,  1., -1.,  0., -0., -0., -0., -0.,
        -0.,  0., -1.,  1.,  0., -0.,  0., -0.,  1., -0., -0.,  0., -0.,
         0.,  0., -0., -1., -0., -0.,  0.],
       [-0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,
         0., -1.,  0., -0.,  0., -0.,  1., -1., -0., -0.,  0.,  0.,  1.,
        -1.,  0., -0.,  0.,  0., -0., -0.,  0., -0.,  0., -0., -1., -0.,
        -0.,  0.,  0., -1., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -0.,  0., -0., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,
         0., -0., -0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,  0.,  0.,
        -0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -0.,  0., -0., -0., -0.,  0.,  0., -0.,  0., -0., -0.,
         0.,  1.,  0., -0.,  0., -0.,  0.,  1.,  0., -0.,  1.,  0., -1.,
        -0., -0., -1.,  0.,  1., -1., -1.,  1., -0.,  1., -0., -0.,  0.,
        -1., -0., -0.,  1.,  0., -0., -0.,  0.,  1., -0.,  0.,  0.,  0.,
        -0.,  0., -1., -1., -1.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.,
         0., -0., -0.,  0.,  0.,  0., -0., -0., -1.,  0.,  0.,  1.,  0.,
        -0.,  1., -1., -0.,  0.,  0.,  0.],
       [ 1.,  0.,  0., -0.,  0., -0., -0.,  0.,  0., -0.,  0., -1.,  0.,
         0., -1.,  0.,  0., -0.,  0., -0.,  0.,  0., -1.,  0., -0.,  0.,
         0., -1., -0., -0.,  0.,  1.,  0., -0., -0.,  0., -0.,  0.,  0.,
        -0., -0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0., -0.,
        -0., -0., -0.,  0.,  0., -0.,  1., -1., -0.,  0., -0.,  0.,  0.,
         0., -0., -1., -0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0.,
         0., -0.,  1., -1., -0., -0., -0.]], dtype=np.float32), np.array([[-0., -0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0., -0.,
        -0., -0., -1., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -1.,  0.,
        -0., -0.,  0.,  0.,  0., -0., -0., -0., -0.,  0.,  0.,  0.,  0.,
        -0.,  1., -0., -0., -0.,  0.,  0., -0.,  0., -0.,  1., -0., -0.,
         0., -0.,  0.,  0.,  0., -1.,  0., -0.,  0.,  0., -1.,  0.,  0.,
         0., -0.,  0.,  0.,  0., -0., -0., -0., -0.,  0., -0., -0.,  0.,
         1.,  0., -0., -0.,  0.,  0., -1.],
       [-0., -0.,  0., -0.,  0.,  0.,  0., -0., -1.,  0., -0.,  0.,  0.,
        -0., -0.,  0., -0., -0.,  0., -0.,  0.,  0., -0., -0., -1., -0.,
         0., -0.,  0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,
        -0., -0., -0.,  0.,  0.,  0.,  0., -0., -0., -1.,  1., -0., -0.,
         0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -1.,
        -0.,  0.,  0., -0., -0., -0., -0., -0., -0., -1.,  0., -0.,  0.,
        -0.,  0.,  0.,  0., -0., -0.,  0.],
       [-0., -0.,  0.,  1., -0.,  0., -0., -0.,  0., -0., -0.,  0., -0.,
        -0., -0.,  0.,  1., -0.,  0.,  0.,  0., -0., -0., -0., -1.,  0.,
        -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -0., -0., -0., -0., -0., -0., -0.,  1.,  0.,  0.,  1.,  0., -0.,
        -0.,  0.,  0., -0., -0.,  0., -0.,  0., -1.,  0., -1.,  0., -0.,
         0.,  0.,  0., -0.,  0., -0., -0., -0.,  0.,  0., -0., -0., -0.,
        -0.,  0., -0.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  1.,  0.,  0.,  0., -0., -0., -0.,  0., -0.,  0., -0.,
        -0., -0., -0.,  0.,  0., -0., -0., -0., -0., -0.,  0.,  1., -0.,
        -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,  1.,
         0.,  0., -0., -0.,  0., -0., -0.,  0., -0.,  0., -1.,  0.,  0.,
        -0., -0.,  0., -0., -0., -0.,  0.,  0., -0.,  1.,  1.,  0., -0.,
        -0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,
         0., -0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0., -0.,  0.,
        -0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0., -1.,  0., -0.,
        -0.,  0.,  0., -1., -0.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,
         0.,  1.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,  1.,  0.,  1.,
         0.,  0., -1., -0., -0., -1., -0., -0., -0.,  0., -1., -0., -0.,
         0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  1., -0.,
         1., -0., -0.,  0., -0., -0.,  0.],
       [ 0., -0.,  0.,  0., -1.,  0.,  0.,  0.,  0., -0.,  0., -0., -0.,
        -0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,  0., -1.,  0., -0.,
        -0.,  0., -0.,  0., -0.,  0., -0.,  0.,  1., -0.,  0.,  0.,  0.,
         0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0., -1.,  1.,  0., -0.,
        -0.,  0., -1., -0.,  0., -1., -0., -0., -0.,  0.,  0., -0.,  0.,
         0., -0., -0.,  0.,  0., -0.,  0.,  1.,  0.,  0.,  0.,  1., -0.,
        -0.,  0., -0., -0., -0., -0., -0.],
       [ 0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0.,  0., -0., -0.,
        -0.,  0.,  0., -0.,  0., -0., -1.,  0.,  1.,  0., -1., -0.,  0.,
         1.,  0., -0., -1., -0.,  0., -0.,  0., -0.,  0.,  0., -1.,  0.,
         0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  1., -0., -0.,
         0.,  0., -1.,  0., -0., -1.,  0., -0.,  0.,  0., -1.,  0.,  0.,
        -1., -0., -0.,  0., -1., -0.,  0.,  1., -0.,  0.,  0.,  1., -0.,
         1.,  0.,  0., -0., -0., -0., -0.],
       [-0., -1.,  1., -0., -0.,  0.,  0.,  1.,  0., -0.,  0., -0., -0.,
         0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0.,  1., -0.,  0.,
         0.,  0., -0., -0., -1.,  0., -1., -0.,  0.,  0.,  0., -0.,  1.,
         1.,  0.,  0.,  0.,  1., -0.,  0.,  0.,  1.,  0., -1., -0., -1.,
        -0.,  1., -0., -0.,  0., -0., -0.,  0., -0.,  1.,  1., -0., -0.,
        -0., -0., -0., -0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,
         0.,  0., -0.,  0., -0., -0., -1.],
       [-0., -0., -1., -0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  1.,
         0., -0.,  0., -0., -0., -0., -0., -0., -0., -0.,  1.,  0.,  0.,
         0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0., -1., -0., -0.,
         1., -0., -0., -0.,  1., -0.,  0., -0.,  0., -0.,  1.,  0., -1.,
         0., -0., -0., -0., -0.,  0., -0.,  0.,  0., -1., -1.,  0., -0.,
        -1.,  1.,  0., -0.,  0.,  1., -0.,  1.,  0., -0., -0.,  0.,  0.,
        -0., -1., -0.,  0.,  0.,  0.,  0.],
       [-0., -0.,  0.,  0.,  1.,  0.,  0.,  0.,  0., -0., -0.,  0., -0.,
         0., -0., -0.,  0., -0., -0., -0.,  0., -0., -0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,
         1.,  0., -0., -0., -0.,  0.,  0., -0., -0., -1.,  1.,  0.,  0.,
         0.,  0., -0., -0., -0., -0., -0.,  0.,  0., -1.,  0., -0.,  0.,
         0., -0.,  0., -0.,  0.,  1., -0., -0.,  0.,  0., -0.,  0.,  0.,
         0., -1., -0.,  0.,  0., -0., -0.],
       [-0., -0., -1., -0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0.,
         0., -0., -0., -0., -0.,  0.,  1., -0.,  0., -0.,  1.,  0.,  0.,
        -1.,  0.,  0., -0.,  0.,  0.,  0., -0., -0., -0., -1.,  1.,  0.,
         1.,  0.,  0., -0.,  1., -0.,  0., -0., -0.,  0.,  1.,  0.,  0.,
        -0., -0., -0., -0., -0., -0., -0.,  0.,  0., -1., -1., -0., -0.,
         0., -0.,  0., -0., -0.,  1.,  0., -0.,  0., -0., -0.,  0.,  0.,
         0., -1.,  0.,  0., -0.,  0., -0.],
       [-0., -1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0., -0.,  0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0., -1., -0., -0.,
        -0.,  0., -0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  0., -0.,
        -1.,  0.,  0.,  0., -1., -0., -0., -0.,  0.,  0., -1., -0., -0.,
        -0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  1.,  1., -0.,  0.,
        -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0.,
        -0.,  1., -0.,  0., -0.,  0., -0.],
       [-0.,  0., -0., -0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0., -1., -0.,
         0.,  1., -0., -0., -0.,  1.,  1.,  1., -0.,  0., -0.,  0.,  0.,
         0., -0., -0., -0.,  0., -1., -1., -0.,  0., -0., -0.,  1.,  0.,
         0., -0.,  1., -1., -0.,  0., -1., -0., -0., -0.,  0.,  0., -1.,
        -0.,  0., -0., -1., -0.,  0.,  0.,  0.,  1., -1.,  0.,  0., -1.,
        -0., -0., -0.,  1.,  0., -0.,  0.],
       [-0.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0., -0., -0.,
        -0.,  0., -0.,  0.,  0., -1., -0.,  0., -0., -0., -0., -1., -0.,
        -0., -0.,  0.,  0., -0., -0., -0.,  1.,  0., -0.,  0., -0., -0.,
         0.,  0., -0., -0.,  0., -1., -1., -0., -0.,  0., -0., -0., -0.,
        -1.,  0., -0., -1., -0., -0.,  0., -0., -0., -0., -0.,  0., -1.,
         0., -0., -0.,  0.,  0.,  0., -0.,  0.,  1., -1.,  0.,  0., -1.,
        -0., -0., -0., -0., -0., -0., -0.],
       [-0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -1.,  1.,  0.,
        -0., -1., -0., -0., -0.,  0.,  0., -1.,  0.,  0., -0., -1.,  1.,
         0., -0.,  0.,  0.,  0., -0., -0.,  1.,  0., -0., -0.,  0., -0.,
         0.,  0., -0.,  0.,  0., -1., -1., -0., -0.,  0., -0.,  1., -0.,
        -1.,  0., -0., -1., -0., -0., -1.,  1.,  0., -0.,  0., -0., -1.,
        -0., -0.,  1., -1., -0.,  0., -0.,  0.,  1., -1., -1., -0., -1.,
         0., -0.,  1.,  1., -0., -0., -0.],
       [ 0.,  0., -0.,  0.,  0.,  0.,  0., -1., -0.,  0., -0.,  0., -0.,
        -0., -0.,  0., -0., -0., -0., -0., -0.,  0., -0.,  0.,  1.,  0.,
         0.,  0.,  0., -0., -0.,  0.,  0., -1.,  0., -0.,  0., -0., -0.,
         0., -0., -0., -0.,  0., -0.,  1., -0.,  0., -0.,  0., -1., -0.,
        -0.,  0.,  0.,  1.,  0., -0., -0.,  0.,  0., -0., -0., -0., -0.,
         0., -0.,  0.,  1., -0., -0., -0.,  0., -1.,  1., -0.,  0.,  1.,
        -0.,  0.,  0.,  0.,  0., -1., -0.],
       [ 0., -0., -0.,  1.,  0., -0., -0., -0.,  0.,  0., -0.,  0.,  0.,
        -0.,  1.,  0.,  0., -0., -0., -1.,  0., -0., -0.,  1., -1., -1.,
        -0.,  1.,  0.,  1.,  0.,  1.,  1., -0.,  0., -1., -0., -0.,  0.,
         0., -0., -0.,  1.,  0.,  0.,  0.,  1., -1., -0., -0., -0.,  0.,
        -0., -0.,  1.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -1.,  0.,
        -0.,  0., -1.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  1., -1.,  0.,
        -0., -0.,  0., -0., -1.,  0.,  0.],
       [-0.,  0.,  0., -0., -1.,  0., -0., -0., -1., -1.,  0., -0., -0.,
        -0., -0., -0., -1., -0.,  0.,  0., -1.,  0., -0.,  1., -1.,  0.,
         0., -0., -1., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  1., -0.,
         0.,  0., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0., -0.,  0.,
         0., -0., -0.,  0., -0., -0.,  0.,  0., -0., -0., -0.,  0., -1.,
        -0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0., -1., -0.,  0.,  0.,
         0., -0., -0., -0.,  0., -0., -0.],
       [-0.,  0., -0.,  1.,  0.,  0., -0., -0.,  0.,  0.,  0., -0., -0.,
        -0.,  0., -0., -0.,  1.,  0., -1., -1.,  0., -0.,  1., -1.,  0.,
        -0., -0., -1., -0.,  0., -0., -0., -0.,  0.,  0., -0., -0., -0.,
         0.,  0., -0., -0.,  0.,  0.,  0.,  1.,  0., -0., -0., -0.,  0.,
         0., -0., -0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,
        -0., -0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,
         0., -0., -0., -0.,  0.,  0., -0.],
       [ 0.,  0., -0., -1., -0.,  0.,  0., -1., -0., -0.,  0., -0., -0.,
        -0.,  0.,  0., -0.,  0., -0.,  1., -0.,  0.,  0., -1.,  1.,  1.,
         0.,  0., -0.,  0.,  1.,  0., -0.,  0., -0.,  1.,  0.,  0., -0.,
        -1., -0., -0.,  0., -1.,  0., -0.,  0.,  0., -0.,  0.,  0., -0.,
         0.,  0.,  0., -0., -0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0.,  0.,  0.,  0., -0., -1., -0., -0.,
        -0.,  0., -0., -0., -0.,  0., -0.],
       [ 0., -0.,  0., -0., -0.,  0., -0.,  0., -0., -0.,  0., -0., -0.,
         0., -0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0.,  1.,  0.,  0.,
         0., -0.,  0.,  1.,  0., -0., -0.,  1.,  0., -0.,  0.,  0.,  0.,
        -0.,  0., -0., -0., -0.,  0., -1., -0., -1., -0.,  0., -0.,  0.,
         0.,  0., -0.,  0., -0., -0.,  0., -0., -0.,  0.,  0., -0.,  0.,
         0., -0.,  0.,  0., -0., -0.,  0., -0., -0.,  0., -0., -1., -1.,
         0.,  0., -0., -0., -1., -0.,  0.],
       [-0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0.,
         0.,  0., -0.,  0., -0.,  0., -0.,  0., -0., -0.,  1.,  0., -0.,
        -0., -0.,  0., -0.,  0., -0., -0.,  1.,  0., -0.,  0., -0., -0.,
        -0., -0., -0.,  0., -0., -1., -1.,  0.,  0.,  0., -0., -0.,  0.,
        -1., -0., -0., -1.,  1., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,
         0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0., -1.,
        -0.,  0., -0., -0.,  0., -0., -0.],
       [-0.,  0., -0., -0.,  0., -1.,  0.,  0., -0.,  0.,  0., -0.,  0.,
         0.,  0., -0., -0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0., -0.,
         0.,  0., -1., -0.,  0.,  0.,  0.,  1., -0.,  1.,  0., -0., -0.,
        -0.,  0., -0.,  0., -0.,  0., -1., -0.,  0., -0., -0., -0., -0.,
         0., -0., -0.,  0., -0., -0.,  0., -0.,  0., -0.,  0.,  1.,  0.,
        -0., -0., -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0., -1.,
         0.,  0., -0., -0.,  0., -0., -0.],
       [-0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0.,
        -0., -0., -0., -0.,  0., -0., -0.,  0.,  0.,  0., -1., -0., -0.,
        -0., -0., -0.,  0.,  1.,  0., -0., -1.,  0., -0., -0., -0.,  0.,
        -1.,  0., -0., -0., -1., -0.,  1., -0., -0., -0.,  0.,  0., -0.,
        -0.,  0., -0., -0.,  0., -0., -0.,  0.,  0., -0., -0.,  0.,  0.,
        -0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  1.,
         0., -0.,  0.,  0., -0., -1., -0.],
       [-1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,  0., -0.,  0.,
        -0.,  0.,  0.,  0.,  0., -1., -1.,  0., -0., -0.,  0., -0., -0.,
        -0.,  0., -1.,  0., -0., -0.,  1.,  0., -0.,  0.,  0.,  0., -0.,
         0., -0., -1., -0., -0., -0., -0., -0.,  0., -0., -0., -0., -0.,
         0., -0., -0., -0.,  0., -0.,  0., -0.,  0., -0.,  0., -0., -0.,
         0.,  0.,  0.,  0., -0.,  0., -0.],
       [-1., -0.,  0., -0., -0.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,
        -0., -0.,  0., -0., -0.,  1.,  0., -0.,  0., -1.,  0., -0.,  0.,
         0., -0., -0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0.,
        -0.,  0., -1.,  0., -0., -0., -0., -0.,  0.,  0.,  0., -0.,  0.,
        -0.,  0.,  0., -0., -0.,  0.,  0.,  1.,  0.,  0.,  0., -0., -0.,
        -0.,  0., -0., -1., -0., -0., -1., -0.,  0., -0.,  0., -0., -0.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [-1., -0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0.,
        -0.,  1.,  0.,  0.,  0., -0., -0.,  1., -0.,  0.,  0., -0., -1.,
        -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,
        -0.,  0., -1., -0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -0.,  0.,  0., -0., -0.,  0., -0.,  0., -0.,  0.,  0., -0., -0.,
         0.,  0.,  0., -0.,  0., -0., -1., -0.,  0., -0., -0., -0., -0.,
        -0.,  0., -1.,  0.,  0.,  0., -0.],
       [ 1.,  0., -0., -0., -0.,  0., -0.,  1., -0., -0., -0.,  0.,  0.,
        -1.,  0., -0.,  0., -0.,  0.,  0.,  0., -0., -0., -0.,  0., -0.,
        -0.,  0.,  0.,  0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,  0.,
        -0., -0., -0.,  0., -0., -0., -1.,  0., -0., -0.,  0., -0.,  0.,
         0., -0., -0.,  0., -0.,  0., -0.,  0., -0., -0., -0., -0., -0.,
         0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0., -0., -0., -1.,
        -0.,  0., -0.,  0., -0.,  1.,  0.]], dtype=np.float32))

In [ ]:
#@title Verification
n = 4
m = 4
p = 7
rank = 85

verify_tensor_decomposition(decomposition_447, n, m, p, rank)

##Rank-90 decomposition of <4,5,6> over Z

In [ ]:
#@title Data

decomposition_456 = (np.array([[ 0., -0.,  0.,  0.,  0.,  0., -1., -0.,  1.,  0.,  0., -0.,  0.,
        -0.,  0., -0., -0.,  0., -0., -1., -1.,  0., -0.,  0., -0., -1.,
        -0.,  0.,  1., -1., -0.,  0., -0.,  1., -0.,  1., -0.,  0.,  0.,
        -0.,  0.,  0., -1.,  1.,  0., -1., -0., -0.,  0.,  0., -0., -0.,
        -1.,  0., -0.,  1.,  0., -0.,  0., -0.,  0.,  1., -0., -0.,  1.,
        -0., -0.,  0., -0.,  0., -1.,  1., -0.,  0.,  1., -0., -0., -0.,
         0., -0.,  0.,  0.,  1.,  0.,  1., -0.,  0.,  0., -0.,  0.],
       [-0.,  0., -0., -0.,  0.,  1., -0., -0.,  0., -1.,  1.,  0., -1.,
         0.,  0.,  0.,  1., -1.,  1., -0.,  0., -1.,  0., -0., -0.,  0.,
         0., -0., -0., -0., -0., -0.,  0., -0., -0., -0., -1.,  0.,  0.,
         1.,  0.,  0., -1., -0.,  0.,  0.,  0., -0., -0.,  1.,  1.,  0.,
         0.,  0., -0., -0., -1., -0.,  0.,  0., -0.,  0.,  1.,  1.,  0.,
         0.,  1.,  0., -0., -1., -0., -0.,  0.,  0., -0.,  1.,  0., -1.,
         1., -0., -0.,  1., -0., -1., -0., -0.,  0., -0.,  1.,  1.],
       [ 0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0., -0.,  0.,  1.,
         0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0., -1., -0.,  0.,
        -0.,  1., -0., -0., -0., -0., -0., -0., -0.,  0.,  0.,  0.,  1.,
         0., -0., -1.,  0., -0., -0.,  0., -0.,  1.,  0., -0., -1., -0.,
        -0., -0., -0., -0., -0., -1.,  0.,  0., -1., -0.,  0., -0.,  0.,
        -0., -0.,  0., -0., -0., -0., -0., -0.,  0., -0., -0., -0., -0.,
         0., -0., -0.,  0., -0., -0., -0.,  0., -0., -0., -1., -0.],
       [ 0.,  0., -0.,  0., -0.,  0., -0., -0.,  1.,  0., -1., -0.,  0.,
        -1., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,
         0., -0.,  0., -1.,  0.,  0., -0.,  1.,  0.,  0.,  0., -1.,  0.,
         0.,  0.,  1., -1.,  1., -0., -1., -0.,  0.,  0., -0.,  0., -0.,
        -1., -0.,  0., -0.,  0.,  0., -0.,  0.,  1., -0., -0., -0.,  1.,
         0., -1., -0.,  0.,  0., -1., -0., -1.,  0., -0., -0., -0.,  0.,
         0., -0.,  0., -0.,  0., -0.,  1., -0., -0., -1., -0.,  0.],
       [-0.,  0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,
         1.,  0.,  0., -1.,  0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,
        -0., -1., -0.,  1.,  1.,  0.,  0.,  0., -0., -0., -0.,  1., -0.,
        -0.,  1., -0., -0., -1., -0.,  1., -0., -1.,  0.,  0.,  0., -0.,
         1., -1., -0.,  0.,  0., -0., -0., -0., -0.,  0., -0., -0., -1.,
         0.,  0., -0., -0., -0.,  1.,  0., -0.,  0.,  0.,  0., -0., -0.,
        -1.,  0.,  0.,  0., -0.,  1., -1.,  0., -0., -0.,  0., -1.],
       [ 0., -0.,  1.,  0.,  0., -0.,  0., -0., -0.,  0.,  0., -1.,  0.,
         0.,  0., -0.,  0., -0., -0., -0.,  0., -0., -1.,  0., -0., -0.,
         0., -0., -0.,  0., -0., -1.,  1., -0., -0., -0., -0.,  0., -0.,
        -0.,  0., -0.,  1.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  1.,
        -0., -0., -1., -0., -0., -0., -1., -0.,  0.,  0., -0.,  0.,  0.,
        -0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0.,
         0., -1.,  0.,  0.,  0., -0., -0.,  0., -1., -0.,  0., -0.],
       [-0.,  0.,  0., -0., -0., -0., -0., -0., -0.,  1., -0.,  0., -0.,
         0.,  1.,  0., -0.,  1.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,
         0., -0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,
        -0., -0., -0.,  1., -0.,  0.,  0., -0.,  0., -0., -1.,  0.,  0.,
         1.,  0.,  0.,  0.,  0., -0.,  1., -0.,  0.,  0., -1.,  0.,  0.,
         0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0.,
        -1., -0., -0., -1., -0., -0., -1.,  0., -0.,  0., -1., -1.],
       [-0.,  0., -0., -0., -0.,  0., -0., -0., -0., -0.,  0.,  0., -0.,
         0., -1., -0., -1., -0., -0.,  0., -0., -0., -0., -0.,  0., -0.,
        -1., -1.,  0., -0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,  0.,
         0., -0., -0.,  0., -1.,  0.,  0.,  0., -1., -0.,  0., -0.,  0.,
        -0.,  0., -0.,  0., -0., -0.,  0., -0.,  0., -0.,  0., -0.,  0.,
        -1., -0., -0., -0., -0.,  1., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -0.,  0., -0.,  0., -0.,  1.,  0., -0.,  1.,  0.,  1.,  0.],
       [ 0., -0., -0.,  1., -0.,  0., -0.,  0., -1.,  0.,  0., -0., -0.,
        -0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0.,
        -0., -0., -0., -0.,  0.,  0.,  1.,  0.,  0., -0.,  1., -0.,  0.,
        -0., -0., -0.,  1.,  0.,  0.,  0.,  0., -0., -0., -0.,  0.,  1.,
        -0.,  1.,  0.,  0.,  0.,  1., -1.,  0.,  0.,  0., -0.,  0., -0.,
        -0., -0.,  0., -0.,  0.,  0., -0.,  1.,  0., -0.,  0.,  0.,  0.,
         0., -0.,  0., -0.,  0., -0.,  0., -0., -1.,  0.,  0.,  0.],
       [-0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0., -0.,
        -0., -0., -0.,  1.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,
         1.,  1.,  0., -1.,  0.,  0., -0., -0., -0.,  0., -0.,  0.,  0.,
        -0., -0.,  0., -0.,  1., -0., -0.,  0.,  1.,  0.,  0.,  0., -1.,
        -1.,  0.,  0., -0.,  0.,  0., -0., -0., -0., -0., -0., -0., -0.,
         1., -0., -0., -0., -0., -1., -0.,  0.,  0., -0.,  0., -0.,  0.,
         1.,  0.,  0.,  0.,  0., -1.,  1., -0., -0., -0., -0.,  1.],
       [-0.,  0., -1., -0.,  1., -0.,  0.,  1., -0.,  0.,  0.,  1., -0.,
         0., -0., -0.,  0.,  0., -0.,  1., -0.,  0.,  1.,  0., -0.,  1.,
        -0., -0.,  0.,  1., -0.,  1., -1., -0.,  0., -1., -0., -0.,  0.,
         0., -0.,  0., -0., -1., -0.,  1.,  0., -0., -0., -0.,  0., -1.,
         1.,  0., -0., -1.,  0.,  0.,  1.,  0., -0., -0.,  0., -0., -1.,
         0.,  0., -0., -0., -0.,  1., -1.,  0.,  1., -1., -0., -0.,  0.,
         0.,  1., -0.,  0., -1.,  0., -1.,  0.,  1., -0.,  0., -0.],
       [-0.,  0.,  1.,  0., -0.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,
        -0., -1.,  0., -1., -0., -1., -1., -0.,  1., -0., -0.,  0., -0.,
        -0.,  0., -0.,  0., -0.,  0., -0.,  0., -0., -0.,  0.,  0., -0.,
        -0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,  0.,  0., -1., -0.,
        -1.,  0.,  0., -0., -0., -0., -1., -0.,  0.,  0., -0.,  0., -0.,
         0., -0., -0., -0.,  1., -0.,  0.,  0., -0.,  1., -1., -0.,  0.,
         0., -0.,  1., -0.,  0.,  0.,  1., -0.,  0., -0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -0., -0., -0., -1.,  0.,  0., -0., -0., -0.,
        -0.,  1., -0.,  1., -0.,  0.,  0., -0., -0., -1.,  0.,  0., -1.,
         1.,  0., -0., -0.,  0.,  0., -0.,  0., -0.,  1., -0.,  0.,  0.,
        -0., -0.,  0., -0.,  1., -0., -0., -1., -0., -0.,  0.,  1.,  0.,
         0., -0.,  0., -0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,
        -0., -0.,  0.,  0.,  0., -1., -0.,  0., -0., -0.,  0.,  0.,  0.,
         0., -0.,  0., -0.,  0., -0., -0., -1., -1., -0.,  0., -0.],
       [-0., -0., -1., -0.,  0.,  0., -0.,  1.,  0., -0., -0.,  1.,  0.,
         0.,  0., -1., -0., -0., -0.,  1.,  0.,  0.,  1.,  1., -1.,  1.,
         0.,  0., -0.,  1., -1.,  1., -1., -1.,  0., -1., -0., -0.,  0.,
        -1.,  0., -0.,  0., -1.,  0.,  1., -0.,  0.,  0.,  0.,  0., -1.,
         1., -0., -0., -1., -0., -0.,  1., -0.,  0.,  0.,  0., -0., -1.,
        -0.,  0., -1., -0., -0.,  1., -1.,  0.,  1., -1.,  0., -0.,  0.,
         0., -0., -1., -0., -1.,  0., -1.,  1.,  1.,  0.,  0.,  0.],
       [ 0., -0., -0., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0.,
        -0.,  0.,  1., -0.,  0., -0., -0.,  0., -0.,  0., -0., -0.,  0.,
        -1.,  0.,  0.,  0., -0., -1.,  0.,  0.,  0., -0.,  0.,  0.,  0.,
        -0., -0.,  0., -0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,  0.,  1.,
         0., -0.,  0.,  1., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0., -0.,  0.,  0.,  0.,  0.,  0., -0., -1., -0.,  0.,  0.,  0.,
         0., -0.,  0., -0.,  1., -0.,  0.,  0., -0., -0., -0.,  0.],
       [-1.,  0., -0.,  0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,  0.,
        -0., -0., -0.,  0.,  0., -0., -1., -0.,  0., -0., -0.,  0.,  0.,
         0.,  0.,  1., -1.,  0.,  0., -0.,  0., -0.,  1., -0.,  0., -0.,
         0., -0., -0., -1.,  1., -0., -1.,  0., -0.,  0., -0.,  0.,  0.,
        -1.,  0., -0.,  1., -0., -0., -0., -0.,  0.,  0., -0., -0., -0.,
         0., -0., -0.,  0., -1., -0.,  1., -0.,  0., -0.,  0.,  0.,  0.,
         0., -0.,  0.,  1.,  0., -0.,  0.,  0., -0., -0., -0.,  0.],
       [-0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0., -1.,  0.,  0., -1.,
         0.,  0., -0.,  1., -1.,  1., -0.,  0.,  0.,  0., -0., -0., -0.,
        -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  1., -0.,  0.,  0.,  0.,
         0.,  0., -0., -1., -0.,  0., -0.,  0.,  0., -0., -0.,  1., -0.,
        -0.,  0., -0.,  0., -1.,  0., -0., -1.,  0.,  0.,  1.,  0., -0.,
         0., -0., -0.,  0., -1.,  0., -0.,  0., -0., -0.,  1., -0., -1.,
         1.,  0.,  0.,  1., -0., -1., -0.,  0.,  0., -0.,  1.,  1.],
       [ 0.,  0., -0., -0., -0., -0., -0., -0.,  0., -0.,  0.,  0.,  1.,
        -0., -0.,  0.,  0.,  1., -1., -0.,  0.,  0.,  0., -0.,  0., -0.,
         0., -0.,  0., -0., -0.,  0.,  0.,  0., -1., -0.,  0., -0.,  0.,
        -0.,  0.,  0., -0.,  0., -1.,  0.,  0.,  1.,  0., -0., -1.,  0.,
         0.,  0.,  0.,  0.,  1.,  0., -0.,  0.,  0.,  0., -0., -0.,  0.,
         0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0., -0., -0.,  1.,  0.,
        -0., -0., -0.,  0., -0.,  0.,  0., -0.,  0.,  0., -1.,  0.],
       [-1.,  1.,  0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,
        -0.,  0., -0.,  0., -0., -0., -0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0., -1.,  0., -0., -0.,  0.,  0.,  0.,  0., -1.,  0.,
        -0.,  0.,  1., -1.,  1.,  0., -1.,  0.,  0.,  0.,  0., -0.,  0.,
        -1., -0.,  0., -0.,  0., -0.,  0., -0., -0.,  0., -0.,  0., -0.,
         0., -1., -0.,  1., -1., -0., -0., -0.,  0., -0., -0.,  0.,  0.,
        -0., -0., -0.,  1.,  0.,  0.,  0.,  0.,  0., -1., -0., -0.],
       [-0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,  1.,  0.,  0.,  0.,
         0., -0.,  0., -1., -0.,  0.,  0., -0.,  0., -0.,  0.,  0., -0.,
         0., -0., -0.,  1.,  0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0.,
         0., -0., -0., -0., -1.,  1.,  1.,  0., -1.,  1.,  0., -0.,  0.,
         1.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0., -1.,  0.,  1.,
        -1.,  0., -0., -0., -0.,  1., -0., -0.,  0.,  0.,  0., -1.]],
      dtype=np.float32), np.array([[-1., -1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
        -0., -0.,  0., -0.,  0., -0., -0.,  1., -0.,  1.,  0., -0., -1.,
        -0., -0., -1.,  0., -0., -0., -0., -0., -0.,  1.,  0., -0., -0.,
        -0., -0., -0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,  0., -0.,
        -0., -0.,  0., -0.,  0., -0., -0.,  0.,  0., -1., -0., -0.,  0.,
         0., -0., -0., -0., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0.,
         0., -1.,  0.,  0.,  0., -0., -0.,  1., -0.,  0., -0., -0.],
       [-1., -1.,  0.,  0.,  0.,  0.,  0., -0., -1., -0., -0., -0., -0.,
         0., -0.,  1., -0.,  0., -0.,  0.,  1., -0.,  0.,  0., -0., -0.,
        -0., -0., -1., -0., -0.,  1., -0.,  0.,  0., -0., -0., -0., -0.,
         0., -0.,  0.,  1.,  0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,
         0.,  0.,  1., -1.,  0., -0., -0.,  0., -0., -1., -1., -0., -0.,
        -0., -0.,  0.,  1.,  0.,  0.,  0.,  1.,  1., -0.,  0.,  0.,  0.,
         0., -0.,  0., -1., -1., -0.,  0.,  0.,  0.,  0.,  0., -0.],
       [-0., -0., -0.,  0., -0.,  0., -0.,  0., -1.,  0.,  0., -0.,  0.,
        -0.,  0.,  0.,  0., -0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,
        -0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,
         0., -0., -0.,  1., -0.,  0., -0.,  0., -0., -0., -0.,  0., -0.,
         0., -0.,  1., -0., -0.,  0., -0.,  0.,  0.,  0., -1.,  0.,  0.,
         0.,  0.,  0.,  1.,  0., -0.,  0.,  1.,  0., -0., -0.,  0.,  0.,
         0.,  1., -0., -1.,  0., -0., -0.,  0.,  0., -0.,  0., -0.],
       [-1., -1.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0., -1., -0.,
        -0., -0., -0.,  0.,  0., -0.,  0.,  1., -0.,  0.,  0., -1., -0.,
        -0., -0., -1., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,
         0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,
         0., -0., -0.,  0., -0.,  0., -0., -0.,  0.,  0., -0., -0., -0.,
         0., -0.,  0., -0., -0.,  0., -1., -0., -0., -0., -0.,  0., -0.,
        -0., -1.,  0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0.],
       [-1., -1., -1., -0., -0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,
         0., -0.,  0.,  0., -0., -0., -1.,  1., -0.,  0., -0.,  0.,  0.,
        -0.,  0., -1., -0.,  0.,  0., -0., -0.,  0.,  0.,  0.,  0., -0.,
        -0.,  0., -0.,  0., -0., -0., -0.,  0., -0., -0.,  0., -0., -0.,
        -0.,  0., -0., -0.,  0., -0., -0., -0.,  0., -1., -0., -0.,  0.,
         0.,  0., -0., -0., -0., -0., -0., -0.,  0., -1., -0., -0.,  0.,
         0., -1.,  1.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.],
       [ 1.,  1., -0., -0., -1., -0., -1.,  0., -1.,  0.,  0.,  0.,  0.,
        -0.,  0.,  0., -0.,  0., -0., -0., -1., -0.,  0., -0.,  0.,  0.,
         0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.,
        -0.,  0., -0.,  1., -0.,  0., -0., -0., -0., -0., -0., -0.,  0.,
        -0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0., -0., -1.,  0.,  0.,
        -0.,  0.,  0.,  1.,  0., -0., -0.,  1.,  0.,  0., -0., -0., -0.,
         0.,  0., -0., -1.,  0.,  0., -0., -0., -0.,  0., -0., -0.],
       [-0., -0., -1., -1., -0.,  1.,  0., -0., -0.,  0.,  0.,  1., -0.,
        -0., -0., -0., -0., -1.,  1., -0.,  0.,  1.,  0., -0., -0.,  0.,
        -0.,  0.,  0.,  0., -0., -0.,  1., -0., -1.,  0., -1., -0.,  0.,
        -1., -0.,  0., -0., -0., -0., -0.,  0., -0.,  0., -1., -0., -0.,
        -0., -0.,  0., -0.,  1.,  0., -1., -0., -0., -0., -0., -0.,  0.,
         0.,  0., -1.,  0.,  0., -0., -0., -0.,  0.,  0.,  0., -1., -0.,
         0., -0.,  1., -0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.],
       [-0., -0.,  0., -0.,  0.,  1.,  0., -0., -0.,  1.,  0., -0.,  0.,
         0., -0.,  0.,  0.,  0., -0.,  0., -0., -1., -0., -0.,  0., -0.,
         0.,  0., -0., -0., -0., -0.,  0.,  0.,  0., -0., -0.,  0., -0.,
         1.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,  1., -0.,  0., -0.,
         0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  1.,  0., -0.,
         0.,  0.,  1.,  0., -0.,  0.,  0., -0., -0., -0., -1.,  0.,  1.,
        -0.,  0., -1., -0., -0., -0.,  0.,  0.,  0.,  0., -0., -0.],
       [-0., -0.,  1.,  1.,  0., -0., -0., -0.,  0., -0., -0., -1., -0.,
        -0., -0., -0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0.,
        -0., -0.,  0., -0., -0.,  0., -1., -0., -0., -0.,  1., -0., -0.,
         0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  1., -0.,  0.,
         0., -0.,  0.,  0., -0.,  0.,  1., -0., -0., -0.,  1., -0., -0.,
        -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0., -0.,
         0., -0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,  0., -0.],
       [ 0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,
        -0.,  0.,  0.,  0.,  0.,  0.,  1., -0.,  0., -0., -0.,  0., -0.,
        -0., -0., -0., -1.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,
        -0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,
        -1., -0.,  0.,  0.,  0.,  0.,  0., -1.,  0., -0., -0., -0., -0.,
        -0., -1.,  0.,  0.,  0.,  0., -1.,  0., -0., -0.,  0.,  0.,  0.,
        -0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,  1.,  0., -1.],
       [ 0.,  0., -1., -1., -0., -0., -0., -0.,  0.,  0., -0.,  1.,  0.,
        -0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0.,
         0., -0., -0.,  0., -0.,  0.,  1., -0., -0.,  0., -1., -0.,  0.,
        -1.,  0., -0., -0., -0.,  0., -0., -0.,  0., -0., -0., -0.,  0.,
        -0.,  0., -0., -0.,  0.,  0., -1., -1., -0., -0., -0., -1.,  0.,
        -0., -0., -1.,  0.,  0., -0., -0., -0., -0.,  0., -0., -0.,  0.,
         0., -0.,  1., -0., -0.,  0., -0.,  0.,  0., -0., -0., -0.],
       [-1.,  0., -0., -0., -0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,
         0., -0.,  0.,  0.,  0., -0., -0.,  0.,  1., -0., -0., -0.,  0.,
         0., -0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,
        -1., -0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -0., -0.,  0., -0., -0., -0., -0., -0.,  0., -0.,  1.,  0.,  0.,
        -0.,  0., -1.,  0., -1., -0., -0.,  0.,  0.,  0., -0., -0., -0.,
        -0., -0.,  1.,  1.,  0., -0., -0., -0., -0.,  0.,  0.,  0.],
       [ 0.,  0., -0., -1.,  0., -0.,  0.,  0., -0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  1., -1., -0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0., -0.,  0., -1.,
         0.,  0., -0., -0., -0.,  0., -0., -0.,  0.,  0., -0.,  0., -0.,
        -0., -0.,  0., -0.,  0., -1.,  0., -0.,  0., -0.,  0., -0.,  0.,
         0.,  0., -1., -0., -0., -0., -0., -0.,  0.,  0., -0., -1.,  0.,
         0., -0.,  0.,  0.,  0.,  0., -0.,  1., -1.,  0.,  0., -0.],
       [ 0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0.,  1.,  0., -0., -1.,
         0.,  0., -0., -1., -0.,  1.,  0.,  0.,  0., -0.,  1.,  0.,  0.,
        -0., -0., -0., -0., -0., -0.,  0.,  0., -0., -0., -0., -0., -1.,
         0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  1., -0.,  1., -0.,
         0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  1.,  0., -0.,
        -0.,  0.,  1.,  0.,  0.,  0., -0.,  0., -0., -0., -1.,  0.,  1.,
         1., -0., -0., -0., -0., -1.,  0., -1., -0.,  0., -0., -0.],
       [ 0.,  0.,  0.,  1.,  0., -0.,  0., -0.,  0.,  0.,  0., -1.,  0.,
         0.,  0., -0., -0., -1., -0.,  0.,  0., -0., -1., -0., -0., -0.,
        -0., -0., -0.,  0., -0., -0., -1.,  0.,  0., -0.,  0., -0.,  0.,
         0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,
         0., -0.,  0.,  0., -0.,  1., -0.,  0.,  0.,  0.,  1.,  0., -0.,
        -0., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0., -0.,  0., -0.,
         0.,  0., -0., -0., -0.,  0.,  0., -0.,  1., -0.,  1.,  0.],
       [ 0., -0., -0., -0.,  0.,  0., -0., -0., -0., -0., -0.,  0., -0.,
        -0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,
        -0.,  0., -0., -1., -0.,  0., -0.,  0.,  0., -1.,  0.,  0., -0.,
        -0.,  0.,  1.,  0., -1.,  0., -0.,  0.,  1., -0., -0.,  0., -0.,
         0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0., -0.,  0., -0.,
        -0.,  0., -0.,  0.,  0.,  0., -1.,  0., -0., -0., -0., -1., -0.,
        -0., -0., -0., -0., -0., -0.,  0., -0.,  0.,  1.,  0.,  0.],
       [ 0., -0., -1., -1.,  0., -0.,  0.,  0.,  0.,  0., -1.,  1., -0.,
         0., -0., -0.,  0., -0., -0., -1., -0.,  0., -0., -0., -0.,  1.,
        -0.,  1., -0.,  0., -0., -0.,  1.,  0.,  0., -1., -1.,  0.,  0.,
        -1.,  0.,  1.,  0., -1., -1., -0., -0.,  0.,  0.,  0., -0., -0.,
         1.,  0.,  0., -0.,  0.,  0., -1.,  0.,  1., -0., -0., -1.,  0.,
        -0.,  1., -1., -0., -0.,  1., -0.,  0.,  0., -1., -0., -1., -0.,
         0., -0.,  1.,  0.,  0., -0., -1., -0., -0., -0., -0., -0.],
       [-1., -0., -0.,  0., -0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,
         0., -1.,  0.,  0., -0., -1., -0.,  0.,  0., -0., -1., -0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0.,
         0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -1.,  0.,
        -0., -0.,  0., -0., -0.,  0.,  0., -0.,  0., -0.,  1.,  0.,  0.,
         0.,  0., -1.,  0., -1., -0.,  0., -0.,  0.,  0., -0., -0.,  0.,
         0., -0.,  0.,  1.,  0.,  0., -0.,  1.,  0.,  0., -0.,  0.],
       [-0.,  1., -0., -1., -0.,  0., -0., -0.,  0.,  0.,  0.,  1., -0.,
         0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0., -0., -1., -0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0.,  1.,  0., -0.,  0.,  0.,  0., -1.,
        -0., -0., -1., -0., -0.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,
        -0.,  0.,  0., -0.,  0., -1., -0.,  0., -1., -0.,  0.,  0.,  0.,
        -0.,  0., -1.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,
         0.,  1.,  0.,  0.,  0., -0., -0., -0., -0.,  0., -0.,  0.],
       [-0.,  1.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,
        -1., -0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.,
        -0., -0., -0., -0., -1.,  0., -0.,  0., -0., -0.,  0.,  1., -0.,
        -0., -1.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,
         0.,  1.,  0.,  0.,  0.,  0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0.,  1., -1.,  0.,  0.,  0., -1., -0., -0., -0.,  0., -0.,
        -0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.],
       [ 0., -0.,  0.,  1.,  0., -0., -0., -0., -0.,  0.,  0., -1., -0.,
        -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0., -0., -0.,
         0., -0., -0., -0., -0.,  0., -1., -0., -0., -0., -0., -0., -0.,
        -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,
         0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0., -0.,
        -0.,  0.,  0., -1.,  0.,  0., -0., -1., -0., -0.,  0., -0.,  0.,
         0., -1., -0., -0., -0., -0.,  0., -0.,  0., -0.,  0.,  0.],
       [-0.,  1., -0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0.,  1., -0.,
        -0.,  0.,  0., -0.,  0., -0., -0.,  0., -0., -0., -0.,  1.,  0.,
         0.,  0.,  0.,  0., -0., -0.,  1., -0., -0.,  0.,  0.,  0., -0.,
        -0., -0., -0., -0., -0., -0., -0.,  0., -0.,  0., -0.,  0.,  0.,
        -0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,
         0., -0.,  0.,  0., -0., -0., -0., -0.,  0.,  0., -0.,  0.,  0.,
        -0.,  1.,  0., -0.,  0., -0., -0.,  0.,  0.,  1., -0.,  0.],
       [ 0.,  1.,  0., -1.,  0.,  0.,  0.,  0., -0.,  0., -1.,  1.,  0.,
         0.,  0.,  0.,  0., -0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,
        -0., -0., -0.,  0., -0., -0.,  1., -0.,  0., -0., -1., -0., -0.,
        -1.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -0., -0.,  0., -0.,  0., -0., -0., -0., -0.,  0., -0., -1.,  0.,
         0.,  1., -1.,  0.,  0., -0., -0., -0.,  0.,  0., -0., -0.,  0.,
         0.,  1.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0., -0.],
       [-0., -1., -0.,  0.,  0.,  0.,  1.,  0.,  1.,  0., -0.,  0., -0.,
         0., -0., -0., -0., -0.,  0., -0.,  1., -0., -0.,  0.,  0.,  0.,
        -0.,  0., -0.,  0., -0.,  0., -0.,  1., -0., -0., -0.,  0., -0.,
         0., -0.,  0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,
        -0.,  0., -0., -0.,  0., -0.,  0., -0., -0.,  0., -0.,  0.,  0.,
         0., -0., -1., -1., -0., -0., -0., -1.,  0.,  0.,  0.,  0., -0.,
         0.,  0.,  0., -0.,  0., -0., -0., -0.,  0., -0., -0.,  0.],
       [ 0.,  0.,  0., -1.,  0., -0.,  0.,  0., -0.,  0.,  0.,  1., -0.,
         1., -0.,  0., -0., -0., -0., -0., -0.,  0.,  1., -1.,  0., -1.,
         0.,  0.,  0.,  0.,  0., -0.,  1., -0., -0.,  1.,  0., -1., -1.,
         0., -0., -1.,  0.,  1.,  0., -1.,  1., -0.,  1., -0.,  0., -0.,
        -0.,  0., -0.,  1., -0., -1., -0., -0., -1., -0.,  0.,  0., -1.,
        -1., -0., -1., -0., -0., -1., -0.,  0., -0.,  0.,  0.,  0.,  0.,
         0., -0.,  0.,  0.,  1., -0., -0.,  1., -1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0.,  1.,  0., -0., -0.,
        -0., -0.,  1.,  0., -0., -0.,  0., -0.,  0.,  0., -0., -0., -0.,
        -0.,  0., -0.,  0., -1.,  0.,  0.,  0., -0., -0., -0., -0., -0.,
        -0., -1., -0., -0.,  0., -0.,  0.,  0., -0.,  1., -0.,  0., -0.,
         0.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,  0.,  1.,  0., -0.,
        -0., -0.,  1., -0.,  0.,  0., -0.,  0., -0., -0.,  0., -0.,  0.,
         1.,  0., -0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0.],
       [ 0., -0., -0.,  1., -0., -0.,  0., -0.,  0.,  1., -0., -1.,  0.,
        -0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0.,
         0., -0., -0., -0.,  0., -1., -1., -0., -0., -0., -0., -0., -0.,
         0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0.,  1.,
         0., -1., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  1., -0., -0.,
        -0.,  0., -0., -0., -0.,  0.,  0.,  0., -0., -0., -0., -0.,  0.,
         1., -0.,  0., -0., -0., -0.,  0.,  0.,  0., -0.,  0., -0.],
       [ 0.,  0., -0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,
         0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,  0., -0., -0.,  0.,
        -0., -0.,  0., -1., -0., -0., -0., -0.,  0.,  0., -0., -1., -0.,
         0.,  0., -0., -0., -0., -0., -1., -0., -0.,  1., -0., -0., -0.,
        -0., -0., -0.,  1.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.,
         0.,  0., -0.,  0.,  0., -0., -1., -0.,  0.,  0., -0., -0., -0.,
         0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,  1., -0.,  0.],
       [ 0.,  0., -1., -1., -0.,  0.,  0., -0.,  0., -0., -1.,  1., -0.,
         1., -0.,  0., -0., -0.,  0., -1.,  0., -0., -0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0., -0., -0.,  1.,  0.,  0.,  0., -1., -1., -0.,
        -1., -0., -0., -0., -0., -0., -1.,  0., -0.,  1., -0.,  0.,  0.,
         1.,  0., -0.,  1.,  0.,  0., -1., -0.,  0.,  0.,  0., -1., -1.,
         0.,  1., -1.,  0.,  0., -0., -0., -0.,  0., -1., -0., -0.,  0.,
         0.,  0.,  1.,  0.,  1.,  0., -1.,  0., -0., -0., -0.,  0.],
       [-1., -0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,
        -0., -1., -1., -1.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,
        -1.,  0.,  0.,  0.,  1.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,
         0., -0.,  0.,  0., -0.,  0., -0., -0.,  0., -0., -0.,  0.,  0.,
        -0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,  1.,  0.,  0.,
        -0., -0., -1., -0., -1., -0., -0., -0.,  0.,  0., -1., -0., -0.,
         0.,  0.,  0.,  1.,  0.,  0., -0., -0., -0., -0.,  0.,  0.]],
      dtype=np.float32), np.array([[ 0.,  0., -0.,  0.,  0., -0., -0.,  1.,  0.,  0., -0.,  0.,  1.,
        -0., -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0., -0., -0.,  1.,
         0., -1., -0., -0.,  0.,  0., -0., -0.,  1., -0., -0., -0., -1.,
        -0.,  0.,  0.,  0., -0., -0., -0., -0.,  0.,  0., -0., -0.,  0.,
        -0., -0., -0., -0., -1., -0.,  0.,  0., -1., -0., -0., -0.,  0.,
        -1., -0., -0.,  0.,  0., -1.,  0., -0., -0.,  0., -0.,  0.,  0.,
        -0., -0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,  0., -0.],
       [-0., -0.,  0.,  0.,  0., -0., -0.,  1.,  0.,  0., -0.,  0.,  1.,
        -0., -0., -0., -0., -1.,  0., -0., -0., -0., -1.,  0., -0., -0.,
        -0., -0.,  0.,  0., -0., -0., -0., -0., -0., -0., -0.,  0., -1.,
         0.,  0., -0., -0., -0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,
         0.,  0., -0., -0., -1., -1., -0., -0.,  0., -0., -0.,  0., -0.,
        -1.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.,
         0.,  0., -0., -0.,  0.,  0., -0., -0., -1., -0.,  1.,  0.],
       [ 0.,  0.,  0.,  0., -0., -0., -0.,  1., -0.,  0., -0.,  0.,  1.,
         0., -0., -0.,  0., -0., -1., -0.,  0.,  0., -0., -1.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0., -1.,
         0., -0.,  0., -0., -0., -0., -0.,  1., -0.,  0.,  0.,  1.,  0.,
         0., -0.,  0., -0., -1., -0.,  0., -0., -0., -0., -0., -0.,  0.,
         0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,
        -0., -0., -0., -0.,  0., -0.,  0., -1., -0., -0.,  0.,  0.],
       [ 0., -0., -0., -0.,  0., -0., -0., -0., -0.,  0., -0., -0.,  0.,
        -0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0., -1.,
         0., -0., -0., -0.,  0., -0.,  0.,  0., -1.,  1., -0.,  0.,  0.,
         0., -0., -1., -0., -1., -1.,  0., -0., -1.,  0., -0., -0.,  0.,
         0.,  0., -0.,  0., -0.,  0.,  0., -0.,  1.,  0.,  0., -0.,  0.,
        -0.,  0., -0., -0.,  0.,  1.,  0.,  0., -0.,  0.,  0., -1., -0.,
        -0., -0., -0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0., -0.],
       [ 0.,  0.,  0.,  0., -0.,  1., -0., -0., -0.,  0.,  0.,  0., -1.,
         1.,  0., -0.,  0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,
        -0.,  0., -0.,  0.,  0.,  0., -0.,  0., -1., -0.,  0., -0.,  0.,
         0., -1.,  0., -0., -0., -0., -0.,  0.,  0., -0., -0.,  0.,  0.,
         0.,  0., -0., -0.,  1.,  0., -0.,  0.,  0.,  0.,  0., -0., -1.,
         0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  1.,  0.,  0., -0.,  0.,
        -0., -0., -0., -0., -1., -0.,  0.,  0., -0., -0., -0., -0.],
       [ 0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  1.,  0.,  0., -0.,
         0., -0., -0.,  0., -0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,
        -0.,  0.,  0.,  0., -0., -1., -0.,  0.,  0., -0.,  0.,  0., -0.,
         0., -1., -0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  1.,
         0.,  1.,  0., -0., -0., -0., -0.,  0.,  0., -0., -0.,  0., -0.,
        -0., -0.,  0., -0.,  0., -0.,  0.,  0.,  1., -0., -0., -0., -1.,
         1., -0.,  0.,  0.,  0., -1., -0., -0., -0.,  0.,  0., -0.],
       [ 0., -0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0.,
         0., -0.,  1., -1.,  0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,
         1., -0., -0., -0.,  1., -0.,  0.,  0., -0., -0.,  0., -0.,  0.,
        -0., -1.,  0., -0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0.,
        -0., -0., -0., -0.,  0., -0.,  0.,  0., -0., -0.,  0., -0., -0.,
        -0.,  0., -0., -0., -0., -0.,  0., -0.,  1., -0.,  1., -0., -1.,
         0., -0., -0., -0.,  0., -1., -0.,  0.,  0.,  0., -0., -0.],
       [-0.,  0., -0., -0., -0., -1.,  0.,  0.,  0., -0., -0., -0.,  0.,
        -1.,  0., -0., -0., -0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0.,
         0.,  0., -0.,  0.,  0., -0.,  0., -0.,  1.,  0., -0., -1.,  0.,
         0.,  0., -0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0., -0.,
        -0.,  0., -0., -1., -1., -0.,  0., -0.,  0., -0.,  0., -0.,  1.,
        -0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -1.,
        -0., -0., -0., -0.,  1.,  0., -0., -0.,  0., -0., -0., -0.],
       [-0., -0., -0.,  1.,  0., -0., -1., -0., -1., -0.,  0., -0.,  1.,
        -1.,  0., -0., -0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,
        -0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  1.,  0., -1.,  0., -1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,
        -0.,  1.,  0.,  0., -1., -1.,  0., -0., -0.,  0.,  0., -1.,  1.,
        -0.,  0., -0.,  0.,  0.,  0., -0.,  1., -1., -0., -0., -0.,  0.,
        -0.,  0., -0.,  0.,  1., -0.,  0., -0., -0.,  0.,  0.,  0.],
       [-0.,  0.,  0.,  1.,  0., -0.,  0.,  0., -0., -0.,  0.,  0.,  1.,
        -0., -0., -0., -0., -1.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,
         0.,  0., -0.,  0.,  0.,  1., -0., -0., -0.,  0., -1.,  0., -1.,
        -0., -0.,  0., -0., -0., -0.,  0.,  0., -0., -0., -1.,  0., -1.,
         0.,  0., -1.,  0., -1., -1., -0., -0.,  0.,  0.,  0., -1., -0.,
         0., -0.,  0., -0., -0., -0., -0.,  0., -1.,  0., -0., -0.,  0.,
         0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  1.,  0.],
       [ 0., -0.,  1., -0.,  0.,  0., -0.,  0., -0., -0., -0., -1.,  1.,
        -0., -0.,  0.,  0., -0., -1.,  0., -0., -1.,  1., -1., -1., -0.,
        -0.,  0., -0.,  0., -0.,  1.,  0., -0.,  0.,  0., -0., -0., -1.,
         1., -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  1.,  0.,
        -0., -0., -1.,  0., -1., -0.,  0.,  0.,  0.,  0.,  0., -1., -0.,
        -0.,  0., -0., -0.,  0.,  0.,  0.,  0., -1., -0., -0.,  0.,  0.,
         0.,  1.,  1.,  0.,  0.,  0., -0., -1.,  0., -0., -0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  1.,  0., -0.,  1., -0.,  0.,  0.,
         1.,  0.,  0., -0., -1., -0., -0., -1., -0.,  0., -0., -0.,  0.,
         0.,  0.,  1., -0., -0.,  0., -0., -0., -1.,  0.,  0.,  1., -0.,
        -0.,  0., -0.,  0.,  0., -0., -1.,  0., -0., -1., -1.,  0.,  0.,
         0., -0., -0.,  1.,  0., -0., -0.,  0.,  0.,  0.,  1.,  0., -1.,
         0.,  0.,  0., -1., -0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,
        -0., -0., -0., -1., -1., -0.,  0.,  0.,  0.,  0., -0.,  0.],
       [ 0., -1.,  0.,  0., -0., -0.,  1., -0.,  0., -0., -1., -0., -0.,
        -1.,  0.,  0.,  0.,  0.,  0., -0., -1., -0., -0.,  0., -0.,  0.,
        -0., -0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,  0., -1.,  0.,
         0., -0., -1.,  0.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,
         0.,  0., -0.,  0., -0., -0., -0., -0.,  1.,  1., -0., -0., -0.,
        -0., -1.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,
         0.,  0., -0.,  0.,  0., -0., -0., -0.,  0., -1.,  0.,  0.],
       [-0.,  0.,  0.,  1., -0., -0.,  0.,  0., -0., -0., -0.,  1.,  0.,
        -0.,  0.,  0.,  0., -0., -0., -1.,  0., -0.,  0., -0.,  0.,  1.,
         0.,  0., -1.,  1.,  0., -0.,  1.,  0., -0., -1.,  0., -0., -0.,
         0.,  0.,  0.,  0.,  1.,  0., -1.,  0.,  0.,  0.,  0.,  0., -0.,
        -1., -0., -0.,  1., -0.,  0., -0.,  0., -0.,  1.,  0., -0., -1.,
         0.,  0.,  0., -0., -0., -1.,  1., -0.,  0., -1., -0.,  0., -0.,
         0.,  0., -0., -0., -1.,  0.,  1.,  0., -0., -0.,  0., -0.],
       [-0., -0.,  0., -0., -0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0.,
        -0., -0.,  0.,  0., -0.,  0., -1.,  0., -0.,  0., -0., -1.,  1.,
        -0.,  0., -1.,  0., -0.,  0., -0.,  0., -0., -1., -0.,  0.,  0.,
        -0., -0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,
         0., -0.,  0.,  1.,  0., -0.,  0., -0.,  0.,  1., -0.,  0.,  0.,
         0., -0., -0.,  0., -0., -0.,  1.,  0., -0., -1.,  0.,  0., -0.,
        -0.,  0.,  0., -0., -1., -0.,  0., -0., -0.,  0.,  0., -0.],
       [-0.,  1.,  0.,  0.,  0.,  0., -1.,  0., -0., -0.,  1.,  0.,  0.,
         1.,  0., -0., -0., -0.,  0., -1.,  1., -0., -0., -0., -0.,  1.,
         0., -0., -1., -0.,  0.,  0., -0.,  0., -0., -1.,  0.,  1.,  0.,
         0.,  0.,  1., -0.,  1., -0., -1.,  0.,  1.,  0.,  0.,  0.,  0.,
        -1., -0.,  0.,  1.,  0.,  0., -0., -0., -1., -0., -0.,  0., -1.,
         0.,  1.,  0., -0., -0., -1., -0., -0., -0., -1., -0., -0., -0.,
         0.,  0., -0., -0., -1., -0.,  1.,  0., -0.,  0.,  0., -1.],
       [ 0., -0., -0.,  0., -0., -0., -0., -1., -0., -0.,  1., -0., -0.,
        -0., -0., -0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0., -1.,
         0.,  1., -0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0.,
        -0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0., -0.,
        -0., -0.,  0.,  0., -0., -0.,  0.,  0.,  0., -1.,  0., -1.,  1.,
         1., -0., -0., -0., -0.,  1., -0., -0., -1.,  0.,  0., -0., -0.,
        -0., -0.,  0.,  0.,  1., -0., -0., -0.,  0.,  0.,  0.,  0.],
       [-0., -0., -1., -0.,  0.,  0., -0., -1.,  0.,  0., -0., -0., -0.,
         0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0., -1.,
        -0., -0., -0.,  0., -0.,  0.,  0.,  0., -0., -0., -1., -0., -0.,
         0., -0., -0.,  0.,  0., -0., -0., -0.,  0., -0., -1., -0., -0.,
        -0.,  0., -0.,  0., -0.,  0., -1.,  0.,  0., -1.,  0., -1.,  1.,
         1.,  0., -0.,  0., -0.,  1.,  0.,  0., -1.,  1., -0., -0., -0.,
        -0., -0.,  0.,  0.,  1., -0., -1., -0.,  0.,  0.,  0.,  0.],
       [-0.,  0., -0., -0., -0.,  0., -0., -1.,  0.,  0.,  0., -0.,  0.,
        -0.,  0.,  0.,  0., -0., -0.,  0.,  0., -1., -0.,  0., -0., -1.,
         0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0.,
         1.,  0., -0.,  0.,  0., -0.,  0., -1., -0., -0., -0.,  0., -0.,
        -0.,  0., -0.,  0., -0., -0.,  0., -0., -0., -1.,  0., -1., -0.,
        -0., -0., -0., -0., -0.,  0.,  0.,  0., -1.,  1., -0., -0.,  0.,
        -0.,  0.,  1.,  0.,  1.,  0., -0., -0.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -0., -0.,  0., -0., -0., -0.,  0.,  0., -1., -0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0.,  1., -0.,  0., -0.,  0.,  0.,  0.,
        -0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0.,
         0.,  0., -0.,  0.,  0.,  1.,  0., -0., -0.,  0., -0.,  0., -0.,
         1., -0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0., -0., -0.,
        -0., -1., -0., -0.,  0.,  0.,  0., -0., -0.,  1., -0.,  0.,  0.,
        -0., -0.,  0.,  0.,  0., -0., -1., -0.,  0.,  0.,  0.,  1.],
       [ 0., -0.,  0.,  0.,  0.,  1.,  1.,  0.,  0., -0., -0., -0., -2.,
         0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0.,  1.,  0., -0.,
        -0., -0.,  0.,  0.,  1.,  0., -0.,  1., -2., -0., -0., -0.,  1.,
        -1., -1., -0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0., -0., -0.,  2., -0., -0.,  0., -0.,  0.,  0.,  1.,  0.,
        -0.,  0.,  1.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,  0., -0.,
        -0., -0.,  0., -0.,  0., -0., -0., -0., -0., -0.,  0., -0.],
       [ 0., -0., -0.,  0.,  0.,  0., -0., -0., -1., -0., -0.,  0., -1.,
         0.,  1., -0., -1.,  0.,  1.,  0., -0.,  1., -0.,  1.,  0., -0.,
        -0., -0.,  0.,  0.,  1., -0., -0.,  1., -0., -0.,  0., -0.,  1.,
        -1., -1.,  0.,  1.,  0.,  0., -0.,  0.,  0., -0., -0., -1., -0.,
         0.,  0.,  1., -0.,  1.,  0.,  0.,  0., -0.,  0., -0.,  1.,  0.,
        -0.,  0.,  1., -0.,  1., -0., -0., -0., -0., -0.,  1., -0., -1.,
         0., -0.,  0., -1., -0., -1., -0., -0., -0., -0.,  0.,  0.],
       [-0., -0., -0., -0., -1.,  0.,  0., -0., -0.,  0.,  0.,  0., -1.,
         0., -0., -0., -1., -0.,  1.,  0.,  0.,  1.,  0.,  1.,  0., -0.,
         1.,  0., -0.,  0.,  1.,  0.,  0., -0., -0.,  0.,  0., -0.,  1.,
        -1., -1.,  0., -0., -0., -0.,  0.,  0.,  0., -0., -0., -1.,  0.,
        -0.,  0., -0., -0.,  1., -0., -0., -0.,  0., -0., -0.,  1., -0.,
        -0.,  0.,  1.,  0., -0., -0.,  0., -0.,  0.,  0.,  1., -0., -1.,
        -0., -0., -0.,  0.,  0., -1.,  0., -0.,  0.,  0.,  0.,  0.],
       [-1., -0., -0., -0., -0., -1., -1.,  0., -0.,  0., -0., -0., -0.,
         0.,  0.,  0., -0., -0.,  1., -0.,  1.,  1., -0., -0.,  0.,  0.,
        -0.,  0., -1., -0., -0.,  0.,  0., -0.,  2.,  0., -0., -0., -0.,
         0.,  0., -0., -0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0., -0.,
        -0.,  0.,  0.,  0., -1., -0., -0., -0., -0., -0.,  0.,  0., -0.,
        -0.,  0., -0., -0.,  1.,  0., -0., -0., -0., -0.,  1., -0., -1.,
        -0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]],
      dtype=np.float32))

In [ ]:
#@title Verification
n = 4
m = 5
p = 6
rank = 90

verify_tensor_decomposition(decomposition_456, n, m, p, rank)

## Rank-93 decomposition of <5,5,5> over Z



In [ ]:
#@title Data

decomposition_555 = (np.array([[ 0.,  0.,  0., -1., -0., -0.,  0.,  0., -0., -0., -0.,  1.,  0.,
         0.,  0.,  0., -0., -0.,  1.,  0., -0., -1., -0.,  0.,  0., -0.,
         0.,  0.,  0., -1., -1.,  1., -1.,  0., -0., -0.,  0., -0., -0.,
        -0.,  0., -0.,  0., -0.,  0., -0.,  0., -0.,  0., -0.,  0., -0.,
        -1., -0., -0.,  0.,  0., -0.,  0., -0., -0.,  0., -0., -0.,  1.,
        -0., -0.,  0.,  0., -0.,  0.,  0., -1.,  0., -0., -0.,  0., -0.,
        -0., -1.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0.,
         0., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0.,
         0., -0.,  0.,  0.,  0., -1., -0., -1.,  1., -0., -0., -1., -1.,
        -1.,  0.,  1.,  1.,  1., -1.,  1., -0.,  0.,  0., -0., -0., -0.,
        -0.,  0., -0.,  0.,  0.,  0.,  0.,  1., -0.,  0., -1.,  1., -0.,
         1.,  0., -0.,  0., -0.,  1.,  0., -0., -0.,  0., -0., -0., -1.,
         1.,  0.,  0.,  0.,  1.,  0.,  0.,  1., -0.,  1., -0., -0.,  1.,
        -1.,  0., -1.,  0.,  0., -0., -0.,  0., -0., -0., -1., -0., -1.,
         0.,  0.],
       [-0.,  0., -0.,  0.,  1.,  0.,  0., -0.,  0.,  1., -1., -0.,  0.,
         1.,  0., -1.,  0., -0.,  0., -0., -0.,  0., -0., -0., -0., -1.,
        -0., -0.,  0., -0., -0., -1.,  1., -1.,  0., -1., -0., -1.,  0.,
        -0., -1., -0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0.,
         1.,  0., -0.,  0., -1., -0.,  0.,  0., -0.,  0., -0., -1.,  0.,
        -0., -0., -0., -0.,  0.,  0.,  0.,  1.,  0.,  0., -0., -0.,  0.,
        -0.,  0.,  0., -0.,  1., -0.,  0.,  0., -0., -1., -0., -0.,  0.,
         1.,  0.],
       [-0., -0.,  1., -1.,  0., -0.,  0., -0.,  0.,  0., -0.,  1., -0.,
        -0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0., -0., -0., -0., -1.,
        -1.,  1.,  0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0., -0.,
         0., -0.,  0.,  0., -0.,  0., -1., -0.,  1., -0., -0.,  1., -0.,
        -0., -0.,  0.,  0.,  0.,  1., -0.,  0., -0.,  0.,  0.,  0., -0.,
         0., -1.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,
        -0., -1.,  0.,  0.,  0.,  1.,  1., -0.,  0.,  0., -0., -0., -1.,
        -0., -1.],
       [ 0.,  0.,  0., -0., -1., -0.,  0., -0.,  0., -1.,  0., -0.,  0.,
        -1.,  0.,  1.,  0.,  0.,  0.,  0., -0.,  1., -0., -0.,  0.,  0.,
        -0., -0., -0.,  1.,  1., -0.,  0.,  1., -0.,  1., -0.,  1.,  0.,
        -0.,  1., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -1.,  1., -0.,
         0.,  0., -0., -0.,  1.,  1.,  0.,  0.,  0.,  0., -0.,  1.,  0.,
        -0.,  0., -0., -0.,  0.,  1., -0.,  0., -0., -0.,  0.,  0.,  0.,
         0., -0., -0.,  0.,  0., -0., -0.,  0.,  0.,  1., -0., -0.,  0.,
        -1., -0.],
       [ 0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,
        -0.,  1., -0.,  0.,  0., -0., -0., -0.,  0., -0., -1., -0.,  0.,
        -0.,  0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,  0., -0.,  0.,
        -0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0., -0.,
         0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  1., -0., -0., -0.,  0.,
        -1.,  0., -1., -0., -1.,  0., -0.,  1., -0.,  0.,  1., -0., -0.,
         0.,  1., -0., -1.,  0.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,
         0.,  1.],
       [-0.,  0., -0.,  0., -0.,  1., -0.,  0.,  0.,  0., -0., -1.,  0.,
        -0., -1.,  0.,  0., -0., -0.,  0.,  0.,  0.,  1.,  1.,  1., -0.,
         0.,  0., -1.,  0.,  0.,  0., -0., -0., -1., -0., -0.,  0.,  1.,
        -0.,  0., -0.,  0., -0., -0.,  0.,  0., -0., -1.,  0., -0., -0.,
        -0.,  0.,  1., -0., -0.,  0., -0.,  0., -1.,  0., -0.,  0.,  0.,
         0.,  0.,  0., -0.,  0.,  0.,  0., -1., -0., -0., -0.,  0., -1.,
        -0., -0.,  0., -0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,  0.,
         0., -0.],
       [-0.,  0., -0.,  0., -1.,  0., -0.,  0., -0.,  0.,  0., -1., -0.,
         0., -0., -0., -0.,  0., -0.,  0., -0., -0.,  0.,  1.,  1., -0.,
         0.,  0., -0., -0.,  0., -0.,  0.,  0., -0., -0.,  0.,  1.,  0.,
        -0.,  1., -0., -0., -0.,  0., -0.,  0.,  0., -1.,  0., -0.,  1.,
         0., -0.,  1., -0.,  1.,  0.,  0., -0.,  0.,  0., -0., -0.,  0.,
        -0.,  0.,  0., -0., -0.,  0.,  0., -1.,  0.,  0.,  0.,  0., -1.,
        -0.,  0.,  0., -0., -0.,  0.,  0.,  0.,  0., -0., -0., -1.,  0.,
        -0.,  0.],
       [ 0.,  1., -1., -0., -0., -0., -0., -0.,  1., -0.,  0., -1., -1.,
        -0., -0., -0., -0.,  0., -0., -1., -0.,  0.,  0.,  0., -0.,  0.,
        -0., -1., -0.,  0., -0.,  0.,  0., -0., -1., -0.,  0.,  0.,  1.,
        -0.,  0.,  1.,  0., -0.,  0.,  1., -0., -1.,  0.,  0., -0., -0.,
         0., -0.,  1.,  0., -0.,  0., -0.,  0.,  0., -0.,  1., -0.,  0.,
         0.,  1., -1., -0.,  0., -0., -1., -0., -0.,  0.,  0.,  0.,  0.,
        -1.,  1.,  0., -0., -0., -1., -1., -1., -0., -0., -1.,  0., -0.,
         0.,  1.],
       [-0.,  0.,  0., -0.,  1.,  1.,  0.,  1., -0.,  1., -0., -0.,  0.,
        -0.,  0., -1.,  1.,  0.,  0., -0.,  0., -1.,  0., -0., -0., -0.,
        -0., -0., -1., -0., -1.,  0., -0., -0., -1.,  0., -0., -1.,  0.,
         0., -1., -0.,  0., -0., -0.,  0.,  0., -0., -0.,  0., -1.,  0.,
         0., -0.,  0., -1., -1., -0., -0., -0.,  0.,  0., -0.,  0.,  0.,
         0., -0.,  0.,  0.,  0., -0., -0., -0., -0.,  0.,  0., -0.,  0.,
        -0.,  0., -0.,  0.,  0.,  0., -0., -0., -1.,  0., -0.,  0., -0.,
         0.,  0.],
       [ 0.,  0., -0.,  1., -0., -0., -0., -0.,  0., -0., -0., -1., -0.,
         1.,  0.,  0.,  0., -0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,
        -0.,  0., -0., -0.,  0., -1.,  1., -0.,  0.,  0., -0., -1.,  0.,
        -1., -0., -0., -0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0., -0.,
         1., -0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,
         0.,  0.,  0., -0., -0.,  0., -0.,  1., -0., -0.,  0., -1.,  0.,
         0.,  1., -0., -0., -0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0., -0., -0., -0., -0.,  0.,  0.,  0., -0., -0., -0., -0.,
        -1.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0.,  1.,  1.,
        -0., -0., -1., -0., -0.,  1., -1., -0., -0.,  0.,  1.,  1.,  0.,
         1., -0.,  0., -1., -0., -0.,  0., -1.,  0.,  0.,  0.,  0., -0.,
        -1., -0., -0.,  0.,  1.,  0., -0., -0.,  0.,  0.,  0.,  1., -0.,
        -1., -0., -0.,  0., -0.,  0.,  0., -1., -0.,  0., -0.,  1.,  0.,
         1.,  0., -0., -0.,  0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,
         0.,  0.],
       [ 0., -0., -0., -0.,  0., -0.,  0., -0., -0., -0., -0., -0.,  0.,
        -1., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  1.,
        -0., -0.,  0., -0.,  0.,  1., -1., -0., -0.,  1., -0.,  1.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,
        -1., -0.,  0., -0.,  1.,  0.,  0.,  0.,  0., -0.,  0.,  1.,  0.,
         0.,  0., -0.,  1., -0.,  0.,  0., -1.,  0.,  0.,  0.,  1.,  0.,
         0., -0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0., -0.,  0.,  0., -0.,  0., -1.,  0.,
         0., -0., -0., -0., -0., -0., -0., -0.,  0.,  0., -0., -0.,  1.,
         0., -0., -0.,  0.,  0.,  0., -0.,  0., -0., -0.,  1.,  0., -0.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.,  0., -0., -0., -0.,  0., -0.,
        -0., -0., -0., -0.,  1.,  0.,  0.,  0.,  0., -0., -0.,  1., -0.,
         0.,  1., -0.,  0., -0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0.,
        -0.,  1., -0.,  0.,  0., -1., -1.,  0., -0., -0.,  0., -0., -0.,
         0., -0.],
       [-1., -0.,  0., -0., -0.,  0., -0., -0., -0.,  0., -0., -0., -0.,
        -0.,  0., -0., -0.,  0., -1., -0., -0.,  0., -0.,  0.,  0.,  0.,
        -1., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -1.,  1.,  0.,  0.,
         1., -0., -0., -1.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,
        -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.,
        -0., -0.,  0., -1., -0., -1., -0., -0., -0., -0.,  0.,  0.,  0.,
        -0., -0., -1.,  0., -1., -0.,  0.,  0.,  0., -0.,  0.,  0., -0.,
         0.,  0.],
       [-0.,  0., -0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,
         0.,  0., -0.,  0., -1., -0., -0., -0., -0., -0.,  1.,  0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,
         0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  1., -0., -0., -0.,  0.,
         0., -1.,  0.,  0., -0., -0., -0.,  0., -0., -0., -0., -0., -0.,
         0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0., -0.,  0.,  0.,
         0., -1., -0., -0.,  0.,  1., -0.,  1.,  0., -0.,  0., -0., -0.,
         0., -1.],
       [-0.,  0.,  1.,  0.,  0.,  0., -0.,  0., -1., -0., -0.,  1.,  0.,
        -0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0., -1.,  0., -0.,
         0., -0.,  0., -0.,  0., -0., -0., -0.,  1., -0.,  0.,  0.,  1.,
        -0., -0., -0.,  0.,  0.,  1., -1.,  0., -0.,  0., -0.,  0., -1.,
         0.,  0., -1.,  0., -0.,  0.,  1., -1., -1.,  1.,  0., -0., -0.,
         0., -0.,  0., -0., -0.,  0.,  1., -1.,  0.,  0.,  0., -0., -0.,
        -0.,  0., -0.,  0.,  0.,  0.,  1.,  0.,  1., -0., -0., -0., -0.,
         0., -0.],
       [ 0.,  0.,  1., -0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  1.,  0.,
         0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0., -1., -0., -0.,
         0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  1.,  0.,
         0.,  1.,  0., -0.,  0.,  1., -1., -0., -0., -0., -0.,  0.,  0.,
        -0., -0., -1.,  0.,  1.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,
         0.,  0.,  0.,  0.,  0., -0.,  1., -1.,  0., -0., -0.,  0.,  0.,
         0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,
         0.,  0.],
       [ 0., -0.,  1., -0., -0.,  0., -0., -0., -1., -0.,  0.,  1., -0.,
         0., -0.,  0., -0., -1., -0., -0.,  0., -0.,  0., -0., -0.,  0.,
         0., -0.,  0., -0., -0.,  0.,  0.,  0.,  1.,  0.,  0., -0.,  0.,
        -0.,  0.,  0., -0.,  1., -0., -1., -0.,  1., -0.,  0.,  0.,  0.,
        -0.,  0., -1., -0., -0.,  0., -0.,  0., -0.,  0., -1., -0., -0.,
         0.,  0.,  1.,  0.,  0., -0.,  1., -0., -0.,  0.,  0., -0.,  0.,
         0., -1., -0., -0.,  0.,  1.,  1.,  1.,  0., -0.,  0.,  0., -0.,
         0., -1.],
       [-0.,  0.,  0., -0.,  0., -0.,  0.,  1., -1.,  1., -0., -0., -0.,
         0., -0., -0., -0.,  0.,  0., -0., -0., -1.,  0., -0., -0., -0.,
         0., -0.,  0.,  0.,  0.,  0., -0., -0.,  1., -0., -0., -1.,  0.,
        -0., -1., -0., -0., -0.,  0., -0., -0.,  0.,  0.,  0., -1.,  0.,
        -0., -0., -0.,  0., -1., -0.,  1.,  0., -0., -0.,  0.,  0., -0.,
         0., -0., -0., -0., -0.,  0., -0., -0., -0.,  0., -0., -0., -0.,
         0.,  0., -0., -0., -0.,  0.,  1.,  0.,  0., -0.,  0., -0., -0.,
        -0., -0.],
       [-0., -0., -0., -1., -0.,  0., -0.,  0.,  0., -0., -0.,  1., -0.,
         0., -0., -0., -0., -0.,  0.,  0., -0.,  0., -0., -1., -0., -0.,
        -0.,  0., -0., -0., -0.,  1., -1.,  0., -0.,  0.,  0.,  0.,  0.,
         0.,  0., -0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,
        -1., -0., -0.,  0., -0.,  0., -0., -0., -0., -0., -0.,  0., -0.,
         0.,  0., -1.,  0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0., -0., -0., -0., -0., -0.,  0., -0., -0.,  0.,
        -0.,  0.],
       [ 0., -1.,  0.,  0.,  0.,  1.,  0.,  0.,  0., -0., -0., -1., -0.,
        -0., -0.,  0., -0.,  0., -0., -0., -0., -0.,  1.,  1., -0., -1.,
        -0., -0., -0., -0., -0., -1.,  1.,  0., -1., -0., -0.,  0.,  0.,
         0., -0.,  0., -0., -0., -0.,  0.,  0., -0., -1.,  0.,  0.,  0.,
         1.,  0.,  1.,  0.,  0., -0., -0., -0., -0.,  0., -0.,  0., -0.,
        -0.,  0.,  0.,  0.,  0., -0., -0., -0.,  1.,  0.,  1.,  0.,  0.,
         0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  0., -0.,  0., -0.,
         0.,  0.],
       [-0.,  0., -0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0., -1.,  0.,
         1., -0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0.,  1.,  0., -1.,
        -0.,  0., -0., -0.,  0., -1.,  1.,  0., -0., -1., -0., -0., -0.,
        -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,
         1., -0.,  1., -0., -0.,  0., -0., -0., -0., -0.,  0., -1., -0.,
        -0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0., -0., -0.,
         0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0.,
         1., -0.],
       [-0., -0.,  0., -1., -0.,  0.,  0., -0.,  1.,  0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0.,  0., -0., -0., -0.,  0., -1.,
         0., -0., -0., -0., -0., -0.,  0.,  0., -1., -0.,  0.,  0., -0.,
        -0.,  0.,  1.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0., -0.,
        -0.,  0.,  1.,  0., -0., -0.,  0.,  0.,  0., -0.,  1., -0.,  0.,
         0., -0., -1.,  0.,  0., -0., -1., -0.,  0., -0.,  0.,  0., -0.,
        -0.,  0.,  0., -0., -0.,  0., -0., -1., -0., -0., -0., -0.,  0.,
        -0., -0.],
       [ 0.,  0., -0., -0., -0.,  0., -1., -0., -0., -0.,  0.,  0.,  0.,
        -1.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0.,
        -0., -0., -0.,  1., -0., -0., -0.,  1., -1.,  1.,  0., -0., -0.,
         0., -0.,  0.,  0., -0.,  0., -0., -0., -0., -0., -1., -0.,  0.,
         0., -0.,  0., -0., -0.,  1.,  0., -0., -0.,  0.,  0.,  1., -0.,
        -0.,  0., -0.,  0., -0.,  0., -0., -0., -0., -0., -0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,
        -1.,  0.]], dtype=np.float32), np.array([[ 0.,  1.,  0.,  0., -0.,  0., -0.,  0.,  0., -0., -0., -0.,  1.,
        -0.,  0.,  0., -0.,  0.,  0.,  0.,  1., -1.,  0.,  0.,  0.,  0.,
         1., -0.,  0.,  0.,  0., -0.,  0., -0.,  0., -1., -1.,  1., -1.,
         0., -0., -0., -0.,  0., -0.,  0.,  1.,  0., -0.,  1., -1., -0.,
        -1., -1.,  0.,  0.,  1., -1.,  0., -0.,  1., -0., -0., -1., -0.,
        -0.,  0.,  0.,  0., -0.,  0., -0., -1., -1., -0., -0.,  0., -0.,
         1., -0., -1.,  0.,  0., -0., -0., -0.,  0., -0.,  1.,  0., -1.,
         0.,  0.],
       [-0., -0.,  1.,  1.,  0., -0.,  0.,  0., -1., -0.,  0.,  1., -0.,
         0.,  0., -0.,  0.,  0.,  0.,  0., -1., -0.,  0.,  1., -0.,  0.,
         0.,  0., -0.,  0., -0., -0., -0., -0., -0., -0.,  0.,  0., -0.,
        -0., -0.,  0., -0., -0.,  1., -1.,  0.,  1., -0., -0.,  0., -0.,
        -0.,  0., -1.,  0., -0., -0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0., -1., -0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0., -0.,  0., -0.,  0.,  1., -0.,  0.,  0., -0., -0., -0.,
        -0., -1.],
       [ 0., -0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,
         1., -0., -0., -0.,  0., -0., -0., -0., -0., -0.,  0.,  0.,  0.,
        -0.,  0., -0.,  1.,  0., -1.,  0.,  0.,  0., -1., -0., -0.,  0.,
        -0., -0., -0., -1., -0.,  0.,  0., -0., -0., -0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  1.,  0., -0.,
         0.,  0., -1., -0., -0.,  0., -0.,  0., -1.,  0.,  1.,  0., -0.,
        -0., -0.,  0.,  0.,  0., -0.,  0.,  1.,  0., -0., -0., -0., -0.,
        -0.,  0.],
       [-0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,
        -0.,  0., -0., -0., -0., -1., -0., -1., -0., -0.,  0., -0., -0.,
         0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0., -0., -0.,
         1., -0.,  0., -1.,  0., -0., -0., -1., -0.,  0., -0.,  0.,  0.,
        -0.,  1.,  0., -0., -0., -0., -0.,  0., -0.,  0.,  1., -0.,  1.,
         1., -0., -1.,  0., -1.,  0., -0., -0.,  0., -0.,  0.,  0., -0.,
        -0., -0.,  1., -1., -0., -0.,  0.,  1., -0., -0.,  0.,  0.,  0.,
         0.,  0.],
       [-0.,  0., -0., -1.,  0., -0.,  0.,  0.,  1.,  0.,  0., -0.,  0.,
        -0., -0., -0.,  0.,  0., -1., -0.,  0., -0., -0.,  0.,  0., -0.,
        -0., -0., -0., -0., -0., -0., -0.,  0., -0.,  0.,  0., -0.,  0.,
         1.,  0., -0., -1., -0.,  0., -0., -1.,  0., -0.,  0., -0.,  0.,
         0.,  0., -0.,  0., -0.,  0., -1., -0.,  0.,  0.,  1.,  0.,  0.,
         1., -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0.,
        -0., -1.,  1., -0., -0.,  1., -1.,  0.,  0., -0.,  0., -0.,  0.,
        -0.,  0.],
       [ 0.,  1., -0., -0., -0., -0.,  0.,  0., -0., -0.,  0.,  0.,  1.,
         0., -0., -0., -0., -0.,  0.,  0.,  1., -0., -0., -0.,  0.,  0.,
         0.,  0., -0.,  0., -0.,  0., -0.,  0., -0., -0.,  0.,  0., -1.,
        -0., -0., -0.,  0., -0.,  0., -0.,  1.,  0.,  0.,  0., -0., -0.,
        -0., -0.,  0.,  0.,  0., -0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,
        -0., -0., -0., -0.,  0., -0.,  0., -0., -1., -1., -0.,  0., -0.,
         1., -0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  1.,  0.,  0.,
        -0.,  0.],
       [-0.,  0., -0.,  0., -0.,  1.,  0., -0., -0., -0.,  0., -0.,  0.,
        -0., -0.,  0., -1.,  0., -0., -0., -1.,  0., -0.,  0., -1.,  0.,
         0.,  0., -1.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,
        -0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  1.,  0., -0., -1.,
         0., -0., -0.,  0.,  0., -0., -0., -0.,  0., -0.,  0.,  0., -0.,
         0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0., -1.,
         0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  1., -0., -0.,  1., -0.,
         0., -0.],
       [-1., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,
        -0., -0., -0.,  0., -0.,  0., -0., -0.,  0., -1., -0., -0., -0.,
         0., -0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0.,
         0., -0., -0., -1.,  0., -0.,  0.,  0., -0., -0., -0., -0., -0.,
        -0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,  1.,  0., -0., -0.,
         0.,  0., -0.,  0.,  0., -0., -0.,  0., -1., -1., -0., -0.,  0.,
         0.,  0.,  1., -0., -0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,
        -0., -0.],
       [-1., -0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0., -0., -0., -0.,
         0.,  1., -0.,  0., -0.,  0.,  0., -1., -0., -1.,  0., -0.,  0.,
         0., -0.,  0., -0., -0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,
        -0., -0.,  0., -1., -0., -0., -0., -1.,  0., -0., -0.,  0., -0.,
        -0.,  0., -0., -0., -0., -0.,  0., -1.,  1.,  0.,  0.,  0.,  0.,
         1.,  0., -0., -0., -1., -0.,  0.,  0., -0.,  0., -1.,  0.,  0.,
         0., -0.,  1., -1.,  0., -0.,  0., -0.,  0.,  0., -0., -0., -0.,
         0.,  0.],
       [-1.,  0.,  0., -0.,  0., -1., -0., -0., -0.,  0.,  0.,  0., -0.,
         0.,  0.,  0.,  1., -0., -0., -0., -0., -0., -1., -0.,  0., -0.,
        -0.,  0.,  1., -0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0.,
        -0.,  0.,  0., -1., -0., -0.,  0., -1., -0.,  0.,  0.,  0., -0.,
         0., -0., -0.,  0., -0.,  0., -0., -1.,  0., -0., -0.,  0.,  0.,
        -0., -0.,  0., -0., -0.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,
         0.,  0.,  1.,  0.,  0.,  0., -0., -0., -1., -0., -0., -0., -0.,
         0., -0.],
       [-0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,  1.,  0.,  0.,  0.,
        -0., -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0.,
         1., -0., -0.,  0.,  0.,  0.,  0., -0.,  0., -1., -1.,  0.,  0.,
        -0.,  1., -0.,  0., -0., -1., -0., -0., -0., -0.,  1., -1., -1.,
        -0., -0.,  0., -0.,  1., -1.,  0., -1.,  0., -1., -0., -1.,  0.,
         0., -0.,  0., -0.,  0.,  0., -0., -0., -0.,  1., -0., -0., -0.,
        -0.,  0., -1.,  0.,  0., -0., -0.,  0.,  0., -0.,  0., -0., -1.,
         0., -0.],
       [-0., -0., -0.,  1., -0., -1.,  0., -0., -1.,  0., -1.,  1., -0.,
         0., -0., -0.,  1.,  0., -0., -0.,  0., -0.,  0., -0.,  1.,  0.,
         0.,  0.,  1., -0.,  0., -0.,  1., -0.,  0., -0., -0., -0.,  0.,
         0., -0., -0.,  0.,  0.,  1., -1.,  0.,  0., -1.,  0.,  0.,  1.,
        -0.,  0., -1.,  0., -0.,  0.,  1.,  0.,  0., -0.,  0.,  0., -0.,
        -0.,  0.,  0., -0.,  0., -0., -1., -0., -0.,  0., -0.,  0., -0.,
         0.,  0., -0.,  0.,  0.,  0.,  1., -0., -1.,  0.,  0., -1., -0.,
        -0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0., -0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  1.,  0., -0., -0.,
         0.,  0., -0.,  0.,  0.,  0.,  0.,  1., -0., -1.,  0., -0., -0.,
         0., -0., -0., -0., -0., -0.,  0.,  0.,  0., -1.,  1.,  0., -0.,
         0.,  0., -1.,  0., -0.,  0.,  0., -0.,  0., -1.,  1.,  0.,  0.,
         0., -0., -0., -0., -0.,  0., -1.,  0.,  0.,  1.,  0.,  0.,  0.,
         0., -0., -1., -0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0.,
        -1.,  0.],
       [ 1., -0.,  0.,  0.,  1.,  0.,  0., -0., -0., -1.,  0., -0., -0.,
        -1.,  0., -1.,  0., -0., -1., -0.,  0., -1.,  1., -0.,  0.,  0.,
        -0., -0., -0., -1., -1., -0.,  0.,  1.,  0.,  0., -0.,  1.,  0.,
         1., -1., -0., -0., -0.,  1., -0., -0.,  0., -1.,  0.,  0.,  1.,
        -0., -0., -1., -0.,  0., -0.,  0.,  1.,  0., -0.,  1.,  0.,  1.,
         0., -0., -0.,  0.,  0., -0., -1.,  0.,  0.,  0., -0.,  1.,  0.,
         0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0., -1.,  0.,
        -1., -0.],
       [ 1., -0., -0., -1., -0.,  1.,  0., -0.,  1., -0.,  0., -1., -0.,
        -0., -0.,  0., -1.,  0.,  0., -0., -0.,  0.,  1., -0., -1., -0.,
        -0.,  0., -1.,  0., -0., -0., -1.,  0.,  0.,  0.,  0.,  0., -0.,
         0.,  0., -0., -0., -0., -0.,  1., -0.,  0., -0., -0., -0.,  0.,
        -0., -0.,  0.,  0.,  0.,  0., -1.,  1.,  0.,  0.,  1., -0., -0.,
         0., -0.,  0., -1., -0., -0., -0., -0.,  0., -0., -0.,  0., -0.,
         0., -0.,  0., -0., -1., -0., -1.,  0.,  1.,  0.,  0., -0., -0.,
         0.,  0.],
       [-0., -1., -0.,  0.,  0., -0.,  0., -0., -0., -0.,  0., -0., -1.,
         0.,  0., -0.,  0.,  0.,  0., -0., -1., -0., -0.,  0., -0., -1.,
        -1.,  0., -0., -0.,  0.,  0.,  0., -0., -0.,  1.,  1.,  0.,  0.,
         0., -0., -0., -0.,  1.,  0.,  0., -1., -0., -0., -1., -0.,  0.,
        -0.,  0.,  0., -0., -0.,  1.,  0., -0.,  0.,  0.,  0.,  1., -0.,
         0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  1.,  0., -0., -0.,  0.,
        -1.,  0.,  1.,  0., -0.,  0., -0., -0.,  0.,  0., -1.,  0.,  1.,
        -0.,  0.],
       [ 0.,  0., -1., -0.,  0., -0.,  0.,  0.,  1., -0., -0., -0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0., -0.,  0., -0.,  0.,
         0.,  1., -0.,  0.,  0., -0.,  0.,  0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -1.,  1., -0.,  0., -0., -0., -0., -0.,
        -0., -0.,  0., -0.,  0., -0., -1., -0., -0.,  0., -0.,  0., -0.,
         0., -0.,  0., -0., -0.,  0.,  1., -0.,  0., -0.,  0.,  0., -0.,
         0., -0.,  0.,  0.,  0., -0., -1., -0.,  0.,  0.,  1., -0., -1.,
        -0., -0.],
       [-0., -1., -0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0., -0.,
         0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0., -0., -0.,  0., -1.,
         0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,  1.,  0., -0.,  0.,
        -0., -0.,  1.,  1.,  0.,  0., -0.,  0., -0.,  0., -1., -0.,  0.,
        -0.,  0., -0.,  0.,  0.,  1.,  0., -0., -0., -0., -1.,  1., -0.,
         0., -0.,  0., -0.,  0., -0., -0., -0.,  1., -0., -0., -0., -0.,
        -0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,
        -0.,  0.],
       [-0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0., -0., -0.,  0.,
        -0.,  0.,  0., -0., -1., -0., -1.,  1., -0.,  0., -0., -0.,  0.,
         1.,  1.,  0., -0.,  0.,  0.,  0., -0.,  0., -0., -1., -0.,  0.,
         0., -0., -0.,  1.,  0., -0.,  0.,  1.,  1., -0., -0., -0., -0.,
        -0., -1.,  0.,  0., -0., -0.,  0., -0., -0.,  0., -1.,  0.,  0.,
         0., -1., -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0.,  0., -0.,
         1.,  0., -1., -0., -0.,  1., -0., -1.,  0.,  0.,  1., -0., -1.,
        -0., -0.],
       [-0.,  0.,  0., -0.,  0., -0., -0.,  0., -1.,  0.,  0.,  0., -0.,
        -0., -0.,  0., -0.,  0.,  0.,  0., -0., -0., -0., -0., -0., -0.,
         1.,  0.,  0., -0.,  0., -0., -0., -0.,  0., -0., -1.,  0.,  0.,
         0.,  0., -0.,  1.,  0., -0.,  0.,  1.,  0., -0., -0.,  0.,  0.,
        -0.,  0.,  0.,  0.,  0., -0.,  1.,  0.,  0.,  0., -1., -0., -0.,
        -0., -1., -0.,  0., -0., -0., -0., -0., -0.,  0.,  0., -0.,  0.,
         1.,  0., -1.,  0.,  0.,  0.,  1., -0.,  0., -0.,  0.,  0., -0.,
         0.,  0.],
       [-0.,  0.,  0., -0.,  0., -0., -0., -1., -0.,  0.,  0.,  0., -0.,
        -0., -0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,
         1., -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0., -0.,  0., -0.,
         0., -0., -0., -0., -0., -0., -0.,  0., -0., -0.,  1., -1., -0.,
         0., -0., -0., -0.,  0., -1.,  1., -1., -0., -1., -0., -0., -0.,
         0.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,  1., -0., -0.,  0.,
        -0.,  0., -1., -0.,  0., -0.,  0., -0., -1.,  0.,  0.,  0., -1.,
        -0.,  0.],
       [ 0., -0., -0.,  0., -0.,  0.,  1.,  1.,  0.,  1., -1., -0.,  0.,
         0.,  0.,  1.,  1., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0.,
         0., -0.,  0.,  0.,  0., -0., -0., -1., -0., -0.,  0.,  0., -0.,
        -0., -0., -0., -0.,  0.,  0., -0., -0., -0.,  0., -0., -0., -0.,
        -0., -0.,  0.,  1.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0.,
        -0.,  0., -0.,  0., -0., -1.,  0.,  0.,  0., -0., -0.,  0., -0.,
        -0., -0.,  0.,  0., -1., -0.,  0., -0.,  0., -1., -0., -0., -0.,
        -0.,  0.],
       [ 1.,  0.,  0., -0., -0.,  1.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,
         0., -0.,  0.,  0., -0., -0., -0., -0., -0.,  1., -0., -0., -0.,
        -0.,  0.,  0., -0., -0., -0., -0.,  0., -1.,  0., -0.,  0., -0.,
        -0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,  1., -0., -0.,
         0., -0., -0., -0., -0.,  0.,  0.,  0.,  0., -1.,  1.,  0., -0.,
         0., -0., -0., -0., -0., -0., -0., -0.,  0.,  1., -0.,  0.,  0.,
        -0.,  0., -1.,  0., -0.,  0.,  0., -0., -0., -0.,  0., -0.,  0.,
         0., -0.],
       [ 1.,  0.,  0., -0.,  0.,  1.,  1.,  1.,  1.,  0., -0., -0.,  0.,
        -0.,  0.,  0., -0., -0., -1.,  0.,  0., -1.,  1., -0.,  0.,  0.,
         0., -0., -0., -1., -1., -0.,  0., -0., -1., -0.,  0.,  0., -0.,
         0.,  0., -0.,  0., -0., -0., -0., -0., -0., -0., -0.,  0.,  0.,
        -0., -0., -0.,  1., -0., -0., -1.,  1., -0.,  0.,  1., -0.,  1.,
        -0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,
         0., -0., -0.,  0., -0., -0.,  0.,  0.,  1.,  0., -0., -0., -0.,
         0.,  0.],
       [ 1., -0., -0.,  0., -0.,  1.,  0., -0.,  1.,  0., -0., -0., -0.,
         0.,  0.,  0., -1., -0.,  0., -0., -0., -0.,  1.,  0., -0.,  0.,
         0., -0.,  0.,  0., -0.,  0.,  0., -0., -1., -0., -0., -0., -0.,
         0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,  0.,  0., -0.,  0.,
        -0., -0., -0.,  0.,  0., -0., -1.,  1., -0., -0.,  1.,  0., -0.,
        -0., -0.,  0.,  0., -0.,  1., -0., -0.,  0., -0., -0.,  0.,  0.,
        -0., -0.,  0.,  0.,  0., -0.,  0., -0.,  1., -0., -0., -0., -0.,
         0., -0.]], dtype=np.float32), np.array([[-0., -0., -0.,  0.,  0., -0.,  0., -1.,  0.,  1., -0.,  0.,  1.,
        -0., -0.,  1., -0., -0., -0.,  0.,  0.,  1., -0., -0., -0., -0.,
        -0.,  1., -0.,  0., -1., -0., -0., -0., -0., -0.,  0., -0.,  0.,
        -0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0., -1., -0.,
         0., -0.,  0., -1., -0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0.,
        -0.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,
        -0.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,  0., -1.,  0., -1.,
         0., -0.],
       [ 0., -0.,  0., -0.,  0., -0., -0., -0., -0., -0.,  0., -0.,  1.,
        -0., -1.,  0., -1., -0., -0., -0., -0., -0., -0., -0.,  0., -0.,
         0., -0., -0.,  0., -0., -0.,  0., -0., -0., -0.,  0.,  0., -1.,
         0.,  0., -0.,  0., -0., -0., -0., -0., -0.,  0., -0.,  0., -1.,
         0., -0., -0., -1., -0.,  0., -0.,  1.,  1.,  0., -0.,  0., -0.,
        -0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0., -0.,
         0., -0.,  0., -0.,  0.,  0., -0., -0.,  1.,  0., -0.,  1.,  0.,
        -0.,  0.],
       [-0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,
        -0., -0., -0., -0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,
        -1.,  1.,  0., -0., -0., -0., -0.,  0.,  0.,  0.,  1., -1.,  0.,
        -0., -1.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,
         0.,  0.,  0., -0.,  1., -0., -0., -0., -0., -0., -0., -0.,  0.,
         0., -1.,  0., -0.,  0., -0., -0., -0.,  0., -0., -0.,  1.,  0.,
         1., -0.,  0., -0.,  0.,  0., -0., -0., -0.,  0., -1., -0., -1.,
        -0.,  0.],
       [-0.,  0.,  0.,  0.,  0., -0.,  0., -1.,  0.,  1., -0.,  0.,  0.,
         0.,  1.,  1.,  1., -0.,  0.,  0., -0.,  1., -0., -0., -0.,  0.,
        -0.,  0.,  0.,  0., -1.,  0.,  0., -0., -0.,  0.,  0.,  1.,  0.,
        -0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0., -0., -0., -0.,  1.,
         0., -0., -0., -0.,  0.,  0.,  0., -1., -1.,  0.,  0.,  0.,  0.,
         0., -0.,  0.,  0.,  0., -0., -0., -1.,  0.,  0.,  0., -1.,  0.,
        -0., -0.,  0., -0., -0., -0.,  0., -0., -1., -0.,  0., -1.,  0.,
         0.,  0.],
       [ 0., -1., -0.,  0.,  0., -0., -0.,  1., -0., -1., -0., -0., -1.,
         0.,  0., -1., -0.,  0., -0., -0.,  0., -1.,  0., -0., -0.,  1.,
         0.,  0.,  0.,  0.,  1., -0., -0., -0., -0.,  0., -0., -1., -0.,
         0., -1., -1.,  0., -0., -0.,  0., -0.,  0., -0., -0.,  1.,  0.,
         1., -0.,  0.,  1.,  1., -1.,  0.,  0., -0., -0.,  0.,  1.,  0.,
        -0., -0., -0.,  0., -0., -0., -0.,  0., -0., -0., -0.,  1.,  0.,
         0., -0., -0., -0., -0.,  0., -0., -0.,  0., -0.,  0., -0., -0.,
        -0.,  0.],
       [ 0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,  1.,  0.,  1.,
        -0.,  0.,  0.,  0.,  1.,  0., -1.,  1., -0., -0.,  0.,  0., -0.,
        -0.,  1., -0., -0., -0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,
        -0., -0.,  0.,  0.,  0.,  0., -0., -0., -1., -0., -0., -0.,  0.,
        -0., -0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0., -0.,
         0.,  0.,  0.,  0.,  1.,  0.,  0., -0.,  0., -0., -0.,  0.,  0.,
        -0.,  0.,  0., -1., -0., -0.,  0.,  0.,  0., -1., -1., -0.,  0.,
         0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0., -0.,  0., -0., -0.,  1.,
         0., -0., -1.,  0.,  0., -0., -1.,  1., -0., -0.,  0., -0.,  0.,
        -0.,  0., -0., -0., -0., -0.,  0.,  0., -0., -0., -0., -0.,  0.,
         0.,  0., -0., -0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,  0.,
        -0.,  0.,  0., -0., -0.,  0., -0.,  0., -0., -0., -0., -0., -0.,
         0., -0., -0., -0.,  1.,  0., -0., -0.,  0.,  0., -0.,  0.,  1.,
         0., -0., -0., -1., -0.,  0., -0., -0.,  0., -1., -1.,  1.,  0.,
         0.,  0.],
       [ 0., -0., -1.,  0., -0., -0., -0., -0.,  0.,  0.,  1., -1.,  0.,
        -0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0., -1., -1.,  0.,
         0., -0., -0.,  0., -0.,  0., -1.,  0., -0., -0., -0., -0., -0.,
         0., -0., -0.,  0.,  0., -0.,  1.,  0., -0.,  0., -0.,  0.,  0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,  0.,  0., -0.,
        -0., -0.,  0., -1., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -1.,
        -0., -0., -0.,  0.,  1.,  0., -0., -0.,  0.,  0.,  0.,  0., -0.,
         0.,  1.],
       [ 0., -0., -1., -0., -1.,  0.,  0., -0., -0.,  1., -0., -0.,  0.,
        -0.,  0.,  1.,  0., -1., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -0., -1.,  0., -0., -0., -0.,  0., -0., -0., -0.,  0., -0.,  0.,
         0.,  1., -0.,  0.,  0.,  1., -0.,  0.,  1., -0., -0., -0.,  1.,
        -0.,  0., -0.,  0.,  0., -0.,  0., -0., -0., -0.,  0., -0.,  0.,
         0., -0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0.,
         0., -0., -0., -0., -0., -0., -0.,  0., -0.,  0.,  0., -1., -0.,
        -0., -0.],
       [ 0.,  0., -1., -0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0., -1.,
        -0., -0., -0., -0., -1., -0.,  1., -1.,  0., -0., -1., -0., -0.,
         0., -1., -0.,  0.,  0., -0.,  0., -1.,  0.,  0.,  0.,  0., -0.,
        -0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  1., -1., -0.,  0., -0.,
         0., -0., -1.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,  0.,
        -0., -0., -0.,  0., -1.,  0., -1., -0.,  0., -0.,  0., -0., -1.,
        -0., -0., -0.,  1., -0., -0., -0.,  0.,  0.,  1.,  1.,  0., -0.,
         1., -0.],
       [ 0.,  0., -0.,  0.,  0.,  0.,  1.,  0., -0., -0.,  0.,  0., -1.,
         0., -0., -1., -0., -0., -0., -0.,  0., -0., -0.,  0., -0., -0.,
        -0., -1., -0., -1.,  1., -0.,  0., -1., -0., -0., -0.,  0.,  0.,
        -0.,  0., -0., -0., -0., -0., -0., -0.,  0., -0., -1., -0.,  0.,
        -0., -0., -0.,  1.,  0.,  1.,  0., -0., -0.,  0.,  0., -0., -0.,
         0., -0.,  0., -0., -0.,  0., -0., -0., -0., -1.,  0., -0., -0.,
         0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0.,  1., -0.,  1.,
         0.,  0.],
       [ 0., -1.,  0., -0.,  0.,  1.,  0., -0., -0.,  0.,  0.,  0., -1.,
        -0.,  1.,  0.,  1.,  0., -0.,  0.,  0.,  0., -1., -0., -0.,  0.,
        -0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0., -0.,
         0., -0.,  0.,  0., -0.,  0., -0., -0., -0.,  1.,  0.,  0., -0.,
        -0.,  0.,  0.,  1.,  0.,  0., -0., -0., -0.,  0.,  0., -0., -0.,
         0.,  0.,  0., -0.,  0., -0., -0.,  0.,  1., -0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,
         0., -0.],
       [ 0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0.,
         1.,  0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,
         1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1., -1., -0.,  0.,
        -0.,  0.,  0.,  1., -0., -0., -0., -1.,  0., -0., -0., -0., -0.,
        -0.,  0.,  0., -0., -0.,  0., -0.,  0., -0., -0.,  0.,  1.,  0.,
        -0.,  1.,  0.,  0.,  0.,  0.,  0., -0.,  0., -1.,  0., -1.,  0.,
        -1.,  0.,  1.,  0., -0., -0., -0.,  0.,  0.,  0.,  1.,  0.,  1.,
        -1.,  0.],
       [-0.,  0., -0.,  0., -0., -0.,  0.,  1., -1., -1., -0., -0.,  0.,
        -0., -1., -1., -1.,  0., -0.,  0.,  0., -1., -0., -0.,  0., -0.,
         0., -0., -0.,  0.,  1.,  0.,  0., -0., -0., -0.,  0., -1.,  0.,
        -0., -1.,  1., -0., -0., -1., -0., -0.,  0.,  0., -0.,  0., -1.,
        -0.,  1., -0.,  0., -0.,  0.,  1.,  1.,  1.,  1.,  1., -0., -0.,
         0.,  0.,  0., -0., -0., -0., -1.,  1.,  0., -0., -0.,  1., -0.,
         0.,  0.,  0., -0.,  0., -0., -0.,  1.,  1.,  0.,  0.,  1., -0.,
        -0., -0.],
       [ 0.,  1., -0.,  0.,  0.,  0., -1.,  0., -0., -0., -0., -0.,  1.,
         1., -0.,  1.,  0., -0., -0.,  0., -0.,  0., -0., -0.,  0., -0.,
         0., -0., -0.,  1., -1., -1., -0.,  1., -0., -0., -0.,  0.,  0.,
         0., -0.,  1., -0., -0., -0., -0.,  0.,  0., -0., -0.,  0.,  0.,
         0., -0., -0., -1.,  0., -0.,  0., -0.,  0., -0., -0.,  0.,  0.,
         0., -0.,  0.,  0., -0.,  0.,  0.,  0., -1.,  0.,  0., -1.,  0.,
        -0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,
        -1.,  0.],
       [ 0.,  0., -0.,  0., -0., -0.,  0., -0.,  0.,  0., -0., -0., -0.,
         0., -0.,  1., -0., -1., -0.,  1.,  0., -0.,  0.,  0., -0., -0.,
        -0.,  0., -0., -0., -1.,  0., -0.,  0., -0.,  0., -0.,  0., -0.,
        -0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  1.,  0., -0., -0.,  0.,
        -0.,  0.,  0., -1., -0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  1.,
        -0., -0.,  0.,  0., -1., -0., -0., -0., -0., -0., -0., -0.,  0.,
        -0.,  0., -0.,  1.,  0.,  0., -0.,  0., -0.,  1., -0.,  0.,  0.,
         0., -1.],
       [-0., -0., -0., -0., -1.,  0.,  0., -0., -0., -0.,  0., -0., -0.,
         0., -1.,  1.,  0., -0.,  0.,  1., -0.,  0.,  0.,  0.,  0., -0.,
        -0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0., -0., -0., -0.,
         0.,  0., -0., -0.,  0., -0.,  0., -0.,  0.,  0., -0., -0., -0.,
        -0.,  0.,  0., -1.,  0., -0., -0., -0., -0., -0.,  0.,  0.,  0.,
         0., -0.,  0.,  0., -0., -0., -0., -0.,  0., -0., -0.,  0.,  0.,
         0., -0., -0.,  1.,  0., -0., -0.,  0.,  0.,  1., -0., -0.,  0.,
        -0., -0.],
       [-0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0., -0.,  0.,
         0., -0.,  0., -0.,  0.,  1.,  0.,  0., -0., -0., -0., -0., -0.,
         0.,  0., -0., -0., -0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,
        -1., -0., -0.,  0.,  0.,  0., -0., -0.,  1., -0., -0.,  0., -0.,
         0., -0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0.,  1.,
        -1., -0.,  0., -0., -1., -0., -0., -0., -0.,  0.,  0.,  1.,  0.,
        -0.,  1.,  0.,  0.,  0., -1., -0., -0., -0., -0., -0., -0.,  0.,
         0., -1.],
       [-0.,  0., -0., -0.,  1., -0., -0., -0.,  0.,  0., -0.,  0., -0.,
         0.,  1., -0.,  0.,  1., -0.,  0.,  0.,  1., -0., -0.,  0.,  0.,
         0., -0.,  0.,  0., -1., -0., -0.,  0., -0., -0.,  0.,  1., -0.,
         0.,  0.,  0., -0.,  0., -0., -0.,  0., -0., -0., -0., -0., -0.,
        -0., -1.,  0.,  0.,  0.,  0., -0.,  0., -1., -0.,  0.,  0.,  0.,
        -0., -0.,  0.,  0.,  0., -0., -0., -1., -0., -0.,  0., -1., -0.,
         0., -0., -0., -0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0., -0.,
         0., -0.],
       [-0., -0.,  0., -0., -0., -0., -0.,  0., -0.,  0., -0.,  0.,  0.,
        -1.,  0., -1.,  0.,  1.,  0., -1.,  0.,  0., -0.,  0.,  0., -0.,
        -0.,  0.,  0., -1.,  1.,  1., -0.,  0.,  0., -0., -0.,  0.,  0.,
         0., -0.,  0.,  0., -0.,  0., -0., -0., -0., -0., -0., -0.,  0.,
        -0., -0.,  0.,  1.,  0.,  0., -0., -0., -0.,  0.,  0., -0.,  0.,
        -0., -0.,  1.,  0., -0., -0., -0.,  0., -0., -0., -1.,  1.,  0.,
         0., -0., -0., -1.,  0., -0.,  0.,  1., -0., -1., -0., -0., -0.,
         0., -0.],
       [-1., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        -0.,  0.,  0.,  0.,  1., -1., -1., -0., -0., -0., -0.,  0., -0.,
        -1.,  1.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0.,
        -0.,  0.,  0., -0., -0., -0., -0.,  0., -1., -0., -0., -0., -0.,
        -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0., -0.,  0., -1.,
         0.,  0., -0., -0.,  1.,  1., -0., -0.,  0.,  1.,  0., -0., -0.,
        -0.,  0., -1., -1., -1.,  0., -0.,  0.,  0., -1., -1.,  0., -1.,
        -0.,  1.],
       [ 0., -0., -0.,  0.,  1.,  0.,  0.,  0.,  0., -0., -0., -0.,  1.,
        -0.,  0., -1., -1.,  0.,  0., -1., -0., -0.,  0., -0., -1.,  0.,
         0., -0., -1., -0.,  0., -0.,  0., -0.,  0., -0., -0.,  0., -0.,
         0.,  0.,  0.,  0., -0.,  0., -0., -1., -0.,  0.,  0.,  0.,  0.,
         0.,  0., -0.,  0.,  0., -0., -0., -0.,  0., -0., -0., -0., -0.,
        -1.,  0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  0.,  0., -0.,
        -1., -0., -0., -1., -0.,  0., -0.,  0.,  0., -1.,  0.,  1., -0.,
         0., -0.],
       [-1., -0.,  0.,  0., -0., -0., -0.,  0., -0.,  0., -0., -0.,  0.,
        -0.,  0.,  0.,  0., -0., -1.,  0.,  0., -0., -0., -0.,  0.,  0.,
        -1.,  1., -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0., -0.,  1., -1.,  0.,  0., -0., -0.,
         0., -0.,  0., -0.,  0., -0., -0.,  0.,  0.,  0., -0., -0., -1.,
         1., -1.,  0., -1.,  1.,  0., -0.,  0.,  0.,  1., -0., -0.,  0.,
         1., -1., -1.,  0., -0.,  1., -0., -0., -0.,  0., -1.,  0., -1.,
         0.,  1.],
       [-0.,  0., -0., -0., -1., -0.,  0., -1.,  0.,  1., -0.,  0.,  0.,
        -0.,  0.,  1.,  1., -1., -0., -0.,  0., -0., -0.,  0., -0.,  0.,
        -0.,  0., -0., -0., -0., -0., -0., -0.,  0., -0.,  0.,  0., -0.,
        -0.,  1.,  0.,  0.,  0.,  1., -1., -0., -0., -0., -0.,  0.,  1.,
        -0., -0.,  0.,  0.,  0., -0., -1.,  0., -0., -0.,  0.,  0.,  0.,
         0., -1.,  0., -0.,  0.,  0., -0.,  0., -0., -0., -0.,  0., -0.,
        -0., -0.,  0., -0.,  0.,  1.,  1.,  0., -1., -0.,  0., -1.,  0.,
        -0.,  0.],
       [-0.,  0.,  0.,  1., -0., -1.,  1.,  0., -1.,  0.,  0.,  1., -1.,
        -0., -0.,  0., -0., -1., -0.,  1., -0., -0.,  0.,  0.,  1.,  0.,
        -0., -0.,  1.,  0.,  0.,  0., -0., -1.,  1.,  0., -0., -0.,  0.,
         0.,  0.,  0.,  0., -0., -0., -1.,  1., -0., -1., -0., -0., -0.,
         0., -0., -1.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
         1., -1., -0., -0.,  0., -0., -1., -0., -0., -0., -0.,  0., -0.,
         1., -1.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,  1.,  0., -0., -0.,
         1., -0.]], dtype=np.float32))

In [ ]:
#@title Verification
n = 5
m = 5
p = 5
rank = 93

verify_tensor_decomposition(decomposition_555, n, m, p, rank)

# B.1. First autocorrelation inequality
For any function $f:\mathbb{R} \rightarrow \mathbb{R}$, define the *autoconvolution* of $f$, written $f*f$, as
$$f*f (t) := \int_\mathbb{R} f(t-x) f(x)\ dx.$$

Let $C_1$ denote the largest constant for which one has
\begin{equation}
 \max_{-1/2 \leq t \leq 1/2} f*f(t) \geq C_1 \left(\int_{-1/4}^{1/4} f(x)\ dx\right)^2
\end{equation}
for all non-negative $f: \mathbb{R} \rightarrow \mathbb{R}$.  This problem arises in additive combinatorics, relating to the size of Sidon sets.  It is currently known that
$$ 1.28 \leq C_1 \leq 1.5098$$
with the lower bound proven by [Cloninger and Steinerberger (2017)](https://www.ams.org/journals/proc/2017-145-08/S0002-9939-2017-13690-9/S0002-9939-2017-13690-9.pdf) and the upper bound achieved by [Matolcsi and Vinuesa (2010)](https://www.sciencedirect.com/science/article/pii/S0022247X10006001) via a step function construction. AlphaEvolve found a step function with 600 equally-spaced intervals on $[-1/4,1/4]$ that gives a better upper bound of $C_1 \leq 1.5053$.

In [ ]:
#@title Data

step_function_heights_1 = [9.00854017233134, 4.581788824134047, 5.954983797866223, 3.7314786029786324, 4.25817159660483, 3.544987049547799, 0.08876194700959494, 0.0491697316439318, 1.7439263266999894, 3.8548164683263795, 3.621038728073569, 4.04587668218835, 4.68211946529981, 5.5904236142896675, 4.737832747546433, 3.1594093823451055, 1.902874129984629, 2.7870307391136304, 3.277574995692391, 1.8981329099596054, 1.526040859367755, 2.305128838504833, 5.17673786436095, 4.583218228762042, 3.9910761392791887, 2.784600928752006, 5.450687602543662, 6.170368723277989, 7.045569321986071e-16, 7.149948549556939e-15, 0.0, 0.0, 0.0, 0.0, 1.2580295353835013e-15, 0.0, 0.0, 0.0, 0.0, 3.873037303627252e-15, 0.0, 0.0, 2.020385656008168e-06, 0.000293922119342568, 0.0, 4.9514916125368726e-15, 7.282654612521097e-16, 1.906059354629418e-14, 0.0, 3.3528418595404916e-15, 1.5099558045700925e-15, 4.901439953827422e-15, 0.0, 8.851999542886555e-15, 0.0, 0.0, 0.0005211322699854395, 0.3757576289315001, 0.25176470069965495, 4.1179587840945515e-06, 0.0, 2.946431316197597e-15, 0.0, 1.0333089131925899e-16, 2.591940622467849e-15, 0.0, 6.852171628124262e-15, 0.0, 0.0, 1.3885601200927435e-14, 2.5015636739088256e-15, 1.4382184696274247e-14, 1.235388698636516e-15, 9.328196456283097e-15, 6.938490364750181e-15, 5.581796597296351e-17, 0.0, 0.0, 5.1220388613389905e-15, 0.0, 6.085199919293191e-15, 0.0, 0.0, 1.0633201915504476e-14, 6.240893078396387e-16, 0.0, 9.242385301100576e-15, 2.1818685641605435e-15, 0.0, 3.841626602268906e-15, 0.0013592097228050644, 8.120066974555713e-15, 8.479388423870961e-16, 2.5924005380166956e-15, 0.0, 2.6610672065525727e-15, 0.0, 1.233819156251431e-14, 8.819083406210366e-15, 0.0, 4.492323424835768e-16, 0.0, 3.0916450306058138e-15, 0.0, 0.0, 3.404186949211756e-15, 4.54126650881379e-15, 1.462631558763152e-14, 0.0, 0.0, 0.0, 1.4460597710909072e-15, 9.521734973996671e-15, 0.0, 4.559858799705722e-15, 7.864867909828807e-16, 0.0, 1.7856864350178655e-16, 0.00021045010164189585, 0.26541232693216404, 0.8094426381528257, 0.5750041584597478, 0.23313281323505236, 3.6007277514467585e-05, 0.0, 0.7828826491881691, 0.43382874037802, 1.3263698571911402, 0.5441713262465393, 0.9864380574571914, 0.6776516652004773, 0.5910950602641856, 0.507419190418916, 0.5231329501406576, 0.9391246115133585, 0.4508771959372286, 0.28283039994676146, 1.2889986480406397, 0.9649046182943108, 1.4104382244415803, 1.3916682358533747, 0.8743196646011149, 0.7627485335443527, 0.2103862254578538, 0.14545209168646947, 0.019762475547189184, 1.2279396984729254, 0.012006361768949678, 1.7677675926679783, 0.9303739918691369, 1.0966313889580412, 0.40142701455261154, 0.1477985748190306, 0.1310850821272394, 0.0027642064206369592, 0.6718883532064702, 0.287789791442545, 1.1886491680958895, 0.6459736548490735, 0.88966666001013, 0.36931312374260505, 0.6840914190936884, 0.38692129734520775, 0.8050006872194091, 0.26610729268169875, 0.002941709304056364, 0.5150673486621109, 0.4049854152265144, 1.1607178193685956, 1.7547854228356075, 0.0, 0.8531817250969695, 2.3845552035650363e-05, 0.035208188035124974, 0.06799207369201249, 0.14050016250524128, 0.4862562534194792, 1.508781726996261, 0.46943710673489225, 0.22962993226722195, 1.589825945710927e-11, 3.51517770993058e-15, 2.4398590319680178e-15, 1.1666504235544564e-06, 0.0021946672216711, 0.34171503722540436, 0.4703022197366691, 0.1313974666218601, 0.11754826815054241, 0.0, 2.2387234387833643e-16, 0.0, 7.192783695625604e-05, 0.4486935802226264, 1.234691190028419, 2.8985055264499153, 1.0234017394012231, 2.7375379465420373, 0.5899927642043619, 1.4461499611411766, 0.7033498408537826, 1.6505029216035125, 0.9593634797752735, 0.009302210703764222, 0.0004181359389419785, 0.0, 0.0, 0.0023430720926212976, 0.42801036705183393, 0.6031743194865573, 1.8862845950884395, 1.0944504439060767, 1.3978223736063145, 0.13603422891356853, 0.8568768273359568, 0.5287328963079988, 0.04201038853661816, 0.5746932650501643, 0.7698787794362285, 2.2478052766496255, 1.3267115762262056, 1.3819155415467284, 1.210307904386098, 1.2050374056121944, 0.973960636675429, 0.13506178694552, 0.0017211602091930576, 1.2080793667302383, 0.9431703684918005, 0.004927152124127672, 0.26457949335968395, 0.219096730428291, 0.8972094379125464, 1.009247390062118, 2.5396761105116816, 2.0567929964131704, 2.5384945885180765, 2.051772820060434, 2.841483226472209, 2.5484575236736253, 2.900405077014117, 2.7293223781158513, 2.8016507480694623, 2.5235338506952227, 2.842495616436774, 3.6113040879253218, 2.4409992918997654, 2.8613737519007785, 2.0376653653073236, 2.873716631081072, 2.7431139992026585, 2.3176851657187343, 2.963845077577065, 2.1297112056154828, 3.1281786712157276, 1.559962066888169, 1.5175735153572592, 1.8986372289826554, 2.422172211485286, 1.4024751115172904, 1.6645681102200025, 1.0890488631004245, 0.9551468779062758, 0.4210663124027455, 0.7844656815643463, 1.3849725648239561, 1.1400002207678432, 1.2589535564861496, 0.00010847583255872839, 0.33022246693439483, 0.009991411612394792, 3.897603693807049e-14, 0.0, 0.0, 4.615098985648224e-16, 0.0, 0.0, 0.00019552451607645426, 7.535959259635103e-15, 0.0, 0.0, 0.0, 3.391322478636254e-15, 0.0, 5.23262076392476e-16, 0.0, 0.0013138779575907105, 3.8754445450268335e-05, 0.6732744675805906, 0.1403000505612687, 0.1066587972153134, 3.248799681911591e-05, 0.0002955183459383166, 0.003362359949825627, 1.258993392593319, 0.6721840514813638, 1.4023077519116312, 0.26971646568749497, 1.5317811712427716e-06, 0.0, 0.0004643255998670316, 0.21977013225378167, 0.4192480303816599, 1.5193826730071023, 1.140986767032434e-14, 0.00023172541086911792, 0.0, 2.8126101180648514e-16, 0.3250641832764737, 0.2631639184319745, 9.581064115586714e-06, 6.911241239536039e-08, 2.1977413193942923e-15, 0.0, 0.009284626840990289, 0.38181362260215157, 0.9229241174303233, 1.2389265199825776, 4.003701357117134e-15, 0.0018347819223694948, 0.09741977675536181, 0.7177732728524777, 0.893918190086622, 0.7592094697415344, 0.33941833076021116, 0.9895237780462212, 1.2103201657651075, 0.1562357060123791, 0.05779942219135546, 0.046035988201598356, 0.20605598834789451, 0.7087789468745198, 0.9835248618606901, 0.15343068950958472, 0.7458575667928422, 1.1650897270704061, 0.8603305066335794, 0.21035575411632462, 0.1359607526732069, 0.27083984880209855, 0.6401471037748572, 3.021195033008772e-11, 0.2302514662496976, 0.6133656015904811, 0.0014526177283533382, 0.2833008515927792, 0.6173666814379335, 0.35174441102605253, 0.42994053621927014, 0.17019592622467353, 0.18730260272562482, 0.20732692125660646, 0.0006488239072035948, 0.0008689690193708769, 0.0060827028748053225, 0.42803050608386095, 0.5802023431307022, 0.07735146147149399, 0.2677857202504946, 0.0009184606747345728, 0.43037124020293993, 0.3617534843366182, 0.3772422594534307, 0.2584947755294139, 0.001169636807952639, 1.9643151751270573e-08, 0.0, 0.0, 0.0, 2.0182994696828576e-15, 0.06977916421885891, 0.2579938169821856, 0.37671532887212333, 0.024565028057913885, 0.12617825469188657, 0.6340448467030686, 0.5754643941945634, 0.13048109721771595, 1.4341543078480776, 1.3216214397106167, 0.4415648326891368, 0.8216927039920507, 0.9032727935119587, 1.7656705299211204, 1.627512140396068, 1.5918752695978569, 2.051239519025683, 1.9657798505455604, 2.8356702084496526, 2.5590427999121284, 2.982526516926714, 2.6272105137933197, 2.960027241401188, 3.3362990605756897, 3.131536821850587, 3.159111523746814, 2.7752077904104517, 1.0508812833436538, 0.21463498939778325, 0.6073490769165661, 0.267278502966685, 0.3440305423365897, 1.0124744653679076, 1.342256457731422, 0.2717826360960419, 1.6632114845610049, 1.7120322558818795, 1.575579186781418, 2.202279058543633, 2.005531691208538, 1.891973046186819, 2.2982791457282916, 2.4252951011748562, 1.3467523990450576, 1.394123734140854, 1.306121994024973, 1.3563331357893305, 1.908587215779497, 2.2410142807813824, 2.1400334226110425, 2.0238641943935187, 2.4448713165408282, 0.001517101822563153, 9.482987339023494e-15, 0.0, 1.5827373963789938e-15, 1.580023867552888, 2.5839838573569534, 1.5871346370389885, 4.1337617125689725, 0.0016876790012300346, 2.337637442823331, 1.9268402331708496, 2.509223443618991, 2.8573979857307554, 2.7429627532040692, 2.3184117402885605, 2.2519888495692886, 1.441733890843454, 2.3283267069330638, 2.090507069768287, 1.616388780668859, 0.30852077577914405, 1.2418308849676503, 0.749579822648432, 2.0216862557918627, 1.8471265276557536, 1.9409844374088654, 2.029630658555306, 1.7488835640200255, 1.4429217698293368, 0.09853693097516952, 1.5685094105495399, 0.060035092997817674, 1.1562109869575399, 0.9883011451997243, 1.257630809337659, 1.6997562951967606, 0.4508041502784602, 0.3164090446061367, 1.4182969827012353, 1.3595162629204571, 1.475081471520821, 3.021289456736385, 3.0956206508951407, 2.481681959913101, 2.1308362149915583, 2.9008847410243757, 2.909122424144527, 2.7204695218309363, 2.087469496170989, 1.3538856364999683, 0.2008306196121235, 1.600964614957816, 1.4250387287265247, 1.6911311698607534, 1.1526705582269934, 0.7292975452608408, 0.4173852602091413, 0.2662677349829448, 1.5122097133171966, 1.0836674077065491, 1.031414686946613, 0.8173974802808452, 0.7095506482976092, 0.5949976898998941, 0.29670773819783247, 0.5083829941296425, 0.6440505445429058, 0.053833868680306, 7.794970013943188e-16, 0.7045208512938965, 1.6120699276991068, 1.5318231382310976, 1.7340273755678486, 2.4381603392169247, 2.6170276892155027, 2.5906147953962844, 3.407886846404895, 3.6212633825479905, 2.329416094716585, 2.958660792491976, 2.6669305434977773, 2.1590860132417564, 2.4937418622856145, 2.5589089762124786, 1.3338118137548678, 1.1942787402156965, 1.7418035300505763, 1.4081188229888932, 1.2487225960375548, 1.4492676314261193, 1.2654783371285574, 1.1685912162797853, 1.0148303874944786, 1.1962440020710763, 1.305708313372589, 0.6602148155530632, 0.337166044389861, 0.8396055147211476, 0.8562349502018103, 0.588778548048956, 0.7049070769530035, 1.2538977263308646, 1.4831897704424597, 1.4593441911500031, 2.1621717599542505, 2.4273857543891015, 2.426355640271325, 2.83832034285733, 2.7641303296460444, 2.2050969080359004, 2.6355562577584215, 3.1005626046243817, 2.4089187966341488, 1.8919645338161346, 1.8840157076492403, 1.344761629829863, 1.404294123950026, 1.8721961393692923, 1.3226408636613955, 0.4215497636181964, 0.5726863357586803, 1.0258923965461795, 1.1819610363504558, 0.8368490648663582, 0.6515561348082733, 0.6685731745760881, 0.5334870649826413, 0.8710519187832059, 0.6669646197224997, 0.5260752114304805, 0.3876797985565807, 0.03621327582895155, 0.46897871650384915, 0.8718533580569904, 0.7009452451531725, 1.4931853849244896, 1.8652719440498333, 1.6631794982365034, 1.494779190512575, 2.508688004725345, 3.0433643835464537, 3.2533878501144433, 3.579790260747532, 2.164640103097207, 0.6698924809914789, 2.1342050222506663, 2.5814605344559984, 1.6583152357630657, 1.3111552900920307, 1.20851491437197, 0.3334479204279151, 0.0027238985981172218, 0.7485037657977041, 0.23706880539492062, 0.3990097623354095, 4.751136081369487e-05, 1.5362095500430528, 0.46926869783190056, 0.0007246232360620678, 0.0, 5.239717734593537e-16, 9.938359637204445e-16, 0.0, 1.7385067095755083e-16, 4.106727240038999e-15, 0.10511094949367368, 0.026846967487429776, 0.0796163088839284, 0.8797518497354565, 2.616397453997683, 3.9912371044774604, 3.6233174077890604, 1.5672138389164023, 1.8304904251881515, 2.748948532497653, 3.287747311072218, 4.0926517829783675, 6.029811160308903]


In [ ]:
#@title Verification
import numpy as np

convolution_1 = np.convolve(step_function_heights_1, step_function_heights_1)
C_upper_bound = 2 * len(step_function_heights_1) * np.max(convolution_1) / np.sum(step_function_heights_1)**2
print(f"The discovered step function shows C1 <= {C_upper_bound}.")

In [ ]:
#@title Visualization
import numpy as np
import matplotlib.pyplot as plt


def plot_step_function(step_heights_input: list[float], title=""):
  """Plots a step function with equally-spaced intervals on [-1/4,1/4]."""
  num_steps = len(step_heights_input)

  # Generate x values for plotting (need to plot steps properly).
  step_edges_plot = np.linspace(-0.25, 0.25, num_steps + 1)
  x_plot = np.array([])
  y_plot = np.array([])

  for i in range(num_steps):
    x_start = step_edges_plot[i]
    x_end = step_edges_plot[i+1]
    x_step_vals = np.linspace(x_start, x_end, 100)  # Points within each step.
    y_step_vals = np.full_like(x_step_vals, step_heights_input[i])
    x_plot = np.concatenate((x_plot, x_step_vals))
    y_plot = np.concatenate((y_plot, y_step_vals))

  # Plot the step function.
  plt.figure(figsize=(8, 5))
  plt.plot(x_plot, y_plot)
  plt.xlabel("x")
  plt.ylabel("f(x)")
  plt.title(title)
  plt.xlim([-0.3, 0.3])  # Adjust x-axis limits if needed.
  plt.ylim([-1, max(step_heights_input) * 1.2])  # Adjust y-axis limits.
  plt.grid(True)
  plt.step(step_edges_plot[:-1], step_heights_input, where='post', color='green', linewidth=2)  # Overlay with plt.step for clarity.
  plt.show()

plot_step_function(step_function_heights_1, title="AlphaEvolve's discovered step function")
plot_step_function(convolution_1, title="Autoconvolution of the discovered step function")

# B.2. Second autocorrelation inequality
Let $C_2$ be the smallest constant for which one has
$$ \|f*f\|_2^2 \leq C_2 \|f*f\|_1 \|f*f\|_\infty$$
for all non-negative $f:\mathbb{R} \rightarrow \mathbb{R}$.
It is known that
$$ 0.88922 \leq C_2 \leq 1,$$
with the lower bound coming from a step function construction by [Matolcsi and Vinuesa (2010)](https://www.sciencedirect.com/science/article/pii/S0022247X10006001). AlphaEvolve found a step function with 50 equally-spaced intervals on $[-1/4,1/4]$ that gives a slightly better lower bound $0.8962 \leq C_2$.

In [ ]:
#@title Data
import numpy as np

heights_sequence_2 = [0.0022217502753395443, 0.798058737836952, 0.4369294656327977, 1.1704958412868685, 1.3413665690827143, 1.5342366222696133, 1.7690742844401723, 1.9329450122360183, 2.2225113878900893, 1.9363966992163675, 2.0382191032475467, 2.2010898310433933, 2.0229605588392388, 2.029541518023742, 2.2636974412575626, 1.9622346498507677, 2.0781053776466134, 2.9856571697702514, 3.4418422600649374, 3.3477129878607825, 3.253250196453988, 3.420135507780267, 3.2509579118114464, 3.2308578066681575, 3.4707132763246245, 2.6462657430572087, 0.9614362498214617, 0, 0.0008733532713782356, 0.00041056186458359313, 0.00029587319086208687, 5.039012949497012e-06, 0, 0.5858888998745988, 6.741440691998236, 7.934548956206666e-06, 0.00013382382526231794, 4.551621108101551e-06, 0.0008898629473865954, 1.083008496291632e-05, 0.0006121618352774956, 0.0011493704284828532, 7.157034681754761, 9.111886252846807, 3.3127569806426527, 8.556232703271356e-06, 0.00017950056213609822, 2.7122354902710758e-06, 1.4036462843158317e-05, 1.1451768709981007e-05]
convolution_2 = np.convolve(heights_sequence_2, heights_sequence_2)

In [ ]:
#@title Verification

# Calculate the 2-norm squared: ||f*f||_2^2
num_points = len(convolution_2)
x_points = np.linspace(-0.5, 0.5, num_points + 2)
x_intervals = np.diff(x_points) # Width of each interval
y_points = np.concatenate(([0], convolution_2, [0]))
l2_norm_squared = 0.0
for i in range(len(convolution_2) + 1):  # Iterate through intervals
  y1 = y_points[i]
  y2 = y_points[i+1]
  h = x_intervals[i]
  # Integral of (mx + c)^2 = h/3 * (y1^2 + y1*y2 + y2^2) where m = (y2-y1)/h, c = y1 - m*x1, interval is [x1, x2], y1 = mx1+c, y2=mx2+c
  interval_l2_squared = (h / 3) * (y1**2 + y1 * y2 + y2**2)
  l2_norm_squared += interval_l2_squared

# Calculate the 1-norm: ||f*f||_1
norm_1 = np.sum(np.abs(convolution_2)) / (len(convolution_2) + 1)

# Calculate the infinity-norm: ||f*f||_inf
norm_inf = np.max(np.abs(convolution_2))
C_lower_bound = l2_norm_squared / (norm_1 * norm_inf)

print(f"This step function shows that C2 >= {C_lower_bound}")

In [ ]:
#@title Visualization
import numpy as np
import matplotlib.pyplot as plt


def plot_step_function(step_heights_input: list[float], title=""):
  """Plots a step function with equally-spaced intervals on [-1/4,1/4]."""
  num_steps = len(step_heights_input)

  # Generate x values for plotting (need to plot steps properly).
  step_edges_plot = np.linspace(-0.25, 0.25, num_steps + 1)
  x_plot = np.array([])
  y_plot = np.array([])

  for i in range(num_steps):
    x_start = step_edges_plot[i]
    x_end = step_edges_plot[i+1]
    x_step_vals = np.linspace(x_start, x_end, 100)  # Points within each step.
    y_step_vals = np.full_like(x_step_vals, step_heights_input[i])
    x_plot = np.concatenate((x_plot, x_step_vals))
    y_plot = np.concatenate((y_plot, y_step_vals))

  # Plot the step function.
  plt.figure(figsize=(8, 5))
  plt.plot(x_plot, y_plot)
  plt.xlabel("x")
  plt.ylabel("f(x)")
  plt.title(title)
  plt.xlim([-0.3, 0.3])  # Adjust x-axis limits if needed.
  plt.ylim([-1, max(step_heights_input) * 1.2])  # Adjust y-axis limits.
  plt.grid(True)
  plt.step(step_edges_plot[:-1], step_heights_input, where='post', color='green', linewidth=2)  # Overlay with plt.step for clarity.
  plt.show()

plot_step_function(heights_sequence_2, title="AlphaEvolve's discovered step function")
plot_step_function(convolution_2, title="Autoconvolution of the discovered step function")

# B.3. Third autocorrelation inequality

Let $C_3$ be the largest constant satisfying
$$ \max_{-1/2 \leq t \leq 1/2} \left|f*f(t)\right| \geq C_3 \left(\int_{-1/4}^{1/4} f(x)\ dx\right)^2$$
for any function $f: \mathbb{R} \rightarrow \mathbb{R}$. Then there is a step function that gives the upper bound $C_3 \leq 1.4581$, see page 75 of [Vinuesa's thesis (2009)](https://www.icmat.es/Thesis/CVinuesa.pdf). AlphaEvolve discovered a step function with 400 equally-spaced intervals on $[-1/4,1/4]$ that gives a slightly better upper bound $C_3 \leq 1.4557$.

In [ ]:
#@title Data
import numpy as np

height_sequence_3 = np.array([1.2065002259, 0.4579577248, 0.5163353850, 0.4072619373, 0.3357781229, 0.2857531610, 0.2820916949, 0.2605397021, -0.0945455083, 0.0263675774, 0.0066053345, 0.4231327261, 0.1460589430, 0.2693078743, 0.0894390737, 0.1956637306, 0.2479890347, -0.0252634907, 0.1662117356, -0.2218493784, 0.0739312998, 0.2415490072, 0.1698246070, 0.3250002134, 0.0020647116, -0.0688859195, 0.0585905178, 0.0939453029, -0.3060000075, -0.1837206775, -0.2352337979, 0.0835844954, -0.0058309576, -0.0832913333, -0.0166100706, -0.0593295469, -0.0358427923, -0.0137145337, -0.2200581708, -0.0020420409, -0.0154295569, -0.1668919220, 0.1254567765, 0.0104704418, -0.1281078357, -0.0961891358, -0.0180100824, 0.0285695118, -0.1876421178, -0.0503548117, 0.0705665536, -0.0849054288, -0.1712106082, 0.1330779045, -0.0395224921, -0.1763118729, 0.1943874403, 0.3136366436, 0.2907763337, 0.0972482020, 0.2859490417, 0.0544729197, 0.2859115117, 0.1089164067, 0.0171270233, -0.0434679259, -0.1024364047, -0.1167575690, -0.0003774918, -0.0344795844, -0.0972136207, -0.1226383339, -0.0252432553, 0.0400353923, 0.0455096903, 0.0122078271, -0.0826149005, 0.0068314042, -0.1352404062, 0.1294605758, -0.0250363591, 0.0356465906, 0.0085605877, -0.0734465832, 0.0239223522, 0.0560616111, 0.1051222588, 0.1186521152, 0.2114356563, 0.0580833852, 0.1532644654, 0.0340677496, 0.0584859676, 0.0676309739, 0.0501669055, 0.0098826564, -0.0699961802, -0.0655442615, -0.2431379908, 0.0191721224, -0.0101123101, 0.0395025740, 0.0369236983, -0.0405410938, 0.0043783628, -0.0022377932, 0.1882810103, -0.0392647889, 0.1110883981, -0.0056550805, -0.0165795658, 0.0484482116, 0.1040965030, 0.0315331695, 0.0290282853, 0.1700089425, 0.0788158652, 0.3053423957, 0.1407591357, 0.1288565749, 0.0731018735, 0.1212327910, 0.0265656763, -0.0026874534, 0.1448773580, -0.1569172936, -0.0332279397, -0.0961953570, -0.1057990671, -0.0208089846, 0.0622545893, 0.0086508578, 0.1134542356, 0.0796647542, -0.0118068228, 0.1183252889, 0.0322959545, 0.1390562294, 0.1334327477, 0.0790933677, -0.0121592574, 0.0355197021, 0.0680023155, 0.1367771272, 0.2764105186, 0.0917362039, 0.1789370907, 0.0801451602, 0.1507336094, 0.1499095182, 0.1338559365, 0.0883362440, 0.0130685381, -0.0010501160, 0.0434483442, -0.0452555246, -0.0160911253, 0.0001037672, -0.0215276925, 0.0922451592, 0.0914967270, 0.1247376137, 0.0674269368, 0.1233065439, 0.0662811693, 0.1056929066, 0.1236410207, 0.1092652789, 0.1253119938, 0.1350819190, 0.0760669548, 0.0975781877, 0.3030153656, -0.0405615005, 0.0736039858, 0.1516256451, 0.2310415709, 0.0903196002, 0.1368532961, 0.1558915065, 0.1395537845, 0.1134530692, 0.1672360224, 0.1779020155, 0.1110537453, 0.3179860968, 0.2418612890, 0.1879389889, 0.1759242853, 0.1751315796, 0.0707347421, 0.1498901200, 0.0009217008, 0.1217537983, 0.0774209961, 0.1585992473, 0.0621134399, 0.1515100010, 0.0780855072, 0.0997527327, 0.1158696157, 0.0293743767, 0.0551188205, 0.0597680188, -0.2775720739, 0.0759704568, 0.1250316677, 0.2132197633, -0.0252846511, -0.2298263857, -0.1916394076, -0.0828138996, -0.4176293110, -0.7396661838, -0.5511700738, -0.5448611785, -0.1727176925, -0.1101347125, 0.2043220162, -0.1082288708, -0.1748688117, 0.0862407071, -0.0531903867, -0.0616308223, 0.0700887776, 0.2081066310, 0.0954712433, -0.2241273540, 0.0753693148, 0.0747931780, -0.0124247160, -0.0919875318, 0.0869789932, 0.0228948007, 0.0330558155, 0.0365086857, 0.0425990861, 0.0946517743, 0.0461941250, 0.0314389948, 0.1547977504, 0.0995246754, 0.0972214450, 0.0642031653, 0.1125917823, 0.0823779771, 0.1138470746, 0.0561091801, 0.0318706784, 0.0083585533, 0.0141864593, 0.0550841887, 0.0552072046, 0.0056539829, 0.0160881555, 0.0233437573, 0.0715285236, 0.0809609974, 0.0733507409, 0.0740536003, 0.0214532872, 0.0210531815, 0.0613127802, 0.1098196117, 0.0679995089, 0.0841437462, 0.0422816392, 0.0795029912, 0.1219842632, 0.0629392926, 0.0442479973, 0.1754482798, 0.1892151967, 0.0506690589, 0.1127489948, 0.0489164780, 0.1195561384, 0.0803870907, 0.1088302988, 0.0086773701, 0.0350070450, -0.0039451117, -0.0491785508, 0.0777041816, 0.1256031497, 0.0758115755, 0.0338649690, 0.0774479672, 0.1600267712, 0.1116514343, 0.0393425971, 0.0880571065, 0.1752844511, 0.0567110352, 0.0848220320, 0.1023379468, 0.1731927552, 0.1015859496, 0.0456034593, 0.1484852609, 0.2335600810, 0.2680922857, 0.0964039102, 0.1370762748, 0.1194107677, 0.0705488401, 0.0492937128, 0.0010280509, 0.0748767880, 0.0413276428, -0.0120807694, -0.1011133614, 0.0378549584, 0.0742557513, 0.0834599468, 0.0828277891, 0.1937349434, 0.0722637117, 0.0539880627, 0.1092967592, 0.1314066075, 0.2276951671, 0.0993954217, 0.0506623717, 0.0665635040, 0.1475022076, 0.1348793959, 0.1120191289, 0.2268202779, 0.1968416912, 0.1785939824, 0.0653614828, 0.1308139129, 0.1845238365, 0.0400508726, -0.0030416083, 0.0612302085, 0.0446805476, 0.0413828618, 0.0087997710, -0.0288662567, 0.0106587067, 0.0459708143, 0.0777709776, 0.1302908220, 0.1269325239, 0.1144772737, 0.1448279485, 0.0854580145, 0.0780628793, 0.1185972450, 0.1083017344, 0.1382659716, 0.1161843882, 0.1031074870, 0.1081799914, 0.0968144848, 0.1369405176, 0.1597198239, 0.1076990897, 0.0998522537, 0.0980860040, 0.0856864888, 0.1147330385, 0.1090380435, 0.0478343797, 0.1104568134, 0.0535653627, 0.0395945072, 0.0615590595, 0.0846094168, 0.1366808400, 0.0979556009, 0.1150407810, 0.1093077855, 0.0701841815, 0.1536579376, 0.0281163265, -0.0188640466, 0.1095483493, 0.2435488950, 0.2224846591, 0.0349037817, 0.0471722551, 0.1366470882, 0.1619649438, -0.0201219126, 0.0776987862, 0.1770040649, 0.1642189494, 0.1510757616, 0.2256426202, 0.1989086542, 0.1778289411, 0.1869462368, 0.2779359655, 0.2578238178, 0.3850396218, 0.4772987312, 0.5429612326])
convolution_3 = np.convolve(height_sequence_3, height_sequence_3)

In [ ]:
#@title Verification
C_upper_bound = abs(2 * len(height_sequence_3) * np.max(convolution_3) / (np.sum(height_sequence_3)**2))
print(f"The discovered step function shows C3 <= {C_upper_bound}")

In [ ]:
#@title Visualization
import numpy as np
import matplotlib.pyplot as plt


def plot_step_function(step_heights_input: list[float], title=""):
  """Plots a step function with equally-spaced intervals on [-1/4,1/4]."""
  num_steps = len(step_heights_input)

  # Generate x values for plotting (need to plot steps properly).
  step_edges_plot = np.linspace(-0.25, 0.25, num_steps + 1)
  x_plot = np.array([])
  y_plot = np.array([])

  for i in range(num_steps):
    x_start = step_edges_plot[i]
    x_end = step_edges_plot[i+1]
    x_step_vals = np.linspace(x_start, x_end, 100)  # Points within each step.
    y_step_vals = np.full_like(x_step_vals, step_heights_input[i])
    x_plot = np.concatenate((x_plot, x_step_vals))
    y_plot = np.concatenate((y_plot, y_step_vals))

  # Plot the step function.
  plt.figure(figsize=(8, 5))
  plt.plot(x_plot, y_plot)
  plt.xlabel("x")
  plt.ylabel("f(x)")
  plt.title(title)
  plt.xlim([-0.3, 0.3])  # Adjust x-axis limits if needed.
  plt.ylim([-1, max(step_heights_input) * 1.2])  # Adjust y-axis limits.
  plt.grid(True)
  plt.step(step_edges_plot[:-1], step_heights_input, where='post', color='green', linewidth=2)  # Overlay with plt.step for clarity.
  plt.show()

plot_step_function(height_sequence_3, title="AlphaEvolve's discovered step function")
plot_step_function(convolution_3, title="Autoconvolution of the discovered step function")

# B.4. An uncertainty inequality

Given a function $f: \mathbb{R}\rightarrow \mathbb{R}$, define the Fourier
transform $\widehat f(\xi) := \int_\mathbb{R} f(x) e^{-2\pi i x\xi}\ dx$ and
$$ A(f) := \inf \{r > 0: f(x) \geq 0 \hbox{ for all } |x| \geq r \}.$$
Let $C_4$ be the largest constant satisfying
$$ A(f) A(\widehat f) \geq C_4$$
for all even $f$ with $\max ( f(0), \widehat f(0)) < 0$.  It is known that
$$ 0.2025 \leq C_4 \leq 0.3523;$$
see Theorem 2 of
[Gonçalves et al. (2017)](https://www.sciencedirect.com/science/article/pii/S0022247X17301804)
(the upper bound is stated as $0.353$ in the paper, but rounding their solution to the fourth digit gives $0.3523$).
We improved the upper bound to $C_4 \leq 0.3521$ with a similar linear combination as in [Gonçalves et al. (2017)](https://www.sciencedirect.com/science/article/pii/S0022247X17301804)
 but with refined constants that were found by AlphaEvolve.


We address the problem using two different formulations described in section B.4. These two formulations lead to known upper bounds $C_4 \leq 0.3523$ and $C_4 \leq 0.3284$, respectively. Using AlphaEvolve, we improve these bounds to $C_4 \leq 0.3521$ and $C_4 \leq 0.3216$, respectively.


First, following the method in [Gonçalves et al. (2017)](https://www.sciencedirect.com/science/article/pii/S0022247X17301804), we consider test functions of the form $f(x) = P(x)e^{-\pi x^2}$, where $P(x)$ is an even polynomial. For this form, the Fourier transform $\hat f(\xi)$ has the structure $P(\xi)e^{-\pi \xi^2}$, which implies $A(f) = A(\hat f)$ is the largest positive root of $P(x)$ (for functions that are positive for large $|x|$). The inequality then becomes $C_4 \le (A(f))^2$.

The polynomial $P(x)$ is chosen as a linear combination of Hermite polynomials of even order $H_{4k}(x)$, specifically $P(x) = \sum_{k=0}^m c_k H_{4k}(x)$. The goal is to find coefficients $c_k$ that minimize the largest positive root of $P(x)$ while satisfying certain conditions on $P(x)$ (related to the $f(0)<0$ condition and $P(x)>0$ for large $|x|$). In the implementation below, the polynomial $P(x)$ is constructed with the technical constraint $P(0)=0$, meaning $P(x)$ has a factor of $x^2$. The largest positive root of $P(x)$ is then the largest positive root of $P(x)/x^2$. The upper bound on $C_4$ is given by this largest root squared, divided by $2\pi$.

AlphaEvolve found the coefficients ($c_0$, $c_1$, $c_2$) defining a polynomial $P(x) = c_0 H_0(x) + c_1 H_4(x) + c_2 H_8(x)$ that provides an improved upper bound for $C_4$.

In [ ]:
#@title Data & Utils

import numpy as np
import sympy

coefficients_sota = np.array([
    -113 / 100,
    1 / 25,
    1 / 3240,
])

coefficients_alphaevolve = np.array([
    0.3292519302257546,
    -0.01158510802599293,
    -8.921606035407065e-05
])


def verify_hermite_combination(inputs: sympy.Expr):
  x = sympy.symbols('x')
  # Check the value at 0 is 0.
  value_at_0 = inputs.subs(x, 0)
  assert value_at_0 == 0, f'The value at 0 is {value_at_0} != 0.'
  assert sympy.limit(inputs, x, sympy.oo) > 0, 'Limit at infty is not positive.'


def find_hermite_combination(coeffs: np.ndarray) -> sympy.Expr:
  """Computes the Hermite combination for given coefficients."""
  m = len(coeffs)
  rational_coeffs = [sympy.Rational(c) for c in coeffs]
  degrees = np.arange(0, 4 * m + 4, 4)
  x = sympy.symbols('x')
  hps = [
      sympy.polys.orthopolys.hermite_poly(n=i, x=x, polys=False)
      for i in degrees
  ]

  # All but the last coefficient.
  partial_fn: sympy.Expr = sum(
      rational_coeffs[i] * hps[i] for i in range(len(rational_coeffs))
  )

  # Impose the condition that the root at 0 should be 0.
  a = hps[-1].subs(x, 0)
  b = -partial_fn.subs(x, 0)
  last_coeff = b / a
  rational_coeffs.append(sympy.Rational(last_coeff))

  res_fn = sum(rational_coeffs[i] * hps[i] for i in range(len(rational_coeffs)))

  if sympy.limit(res_fn, x, sympy.oo) < 0:
    res_fn = -res_fn

  return res_fn


def get_upper_bound(coeffs: np.ndarray) -> float:
  """Computes the upper bound for the given Hermite combination."""
  g_exp = find_hermite_combination(coeffs)
  x = sympy.symbols('x')
  gq_fn = sympy.exquo(g_exp, x**2)
  rroots = sympy.real_roots(gq_fn, x)
  approx_roots = list()
  largest_sign_change = 0

  for root in rroots:
    approx_root = root.eval_rational(n=200)
    approx_root_p = approx_root + sympy.Rational(1e-198)
    approx_root_m = approx_root - sympy.Rational(1e-198)
    approx_roots.append(approx_root)
    is_sign_change = (
        (gq_fn.subs(x, approx_root_p) > 0 and gq_fn.subs(x, approx_root_m) < 0)
        or (
            gq_fn.subs(x, approx_root_p) < 0
            and gq_fn.subs(x, approx_root_m) > 0
        )
    )
    if is_sign_change:
      largest_sign_change = max(largest_sign_change, approx_root)

  return float(largest_sign_change**2) / (2 * np.pi)

In [ ]:
#@title Verifier

sota_exp = find_hermite_combination(coefficients_sota)
alphaevolve_exp = find_hermite_combination(coefficients_alphaevolve)

verify_hermite_combination(sota_exp)
verify_hermite_combination(alphaevolve_exp)

print(
    'The construction of Gonçalves et al. (2017) is correct and gives the '
    'upper bound: C4 <=',
    get_upper_bound(coefficients_sota)
)
print(
    'AlphaEvolve construction is correct and gives the upper bound: C4 <=',
    get_upper_bound(coefficients_alphaevolve)
)

In [ ]:
#@title Plotting

import matplotlib.pyplot as plt

x = sympy.symbols('x')
sota_fn = sympy.lambdify(x, sota_exp, modules=["numpy"])
sota_total_fn = lambda x: np.exp(-np.pi * x**2) * sota_fn(
    np.sqrt(2 * np.pi) * x
)

alphaevolve_fn = sympy.lambdify(x, alphaevolve_exp, modules=["numpy"])
alphaevolve_total_fn = lambda x: np.exp(-np.pi * x**2) * alphaevolve_fn(
    np.sqrt(2 * np.pi) * x
)

x_vals = np.linspace(-5, 5, 1000)

plt.figure(figsize=(8, 5))

plt.plot(
    x_vals,
    sota_total_fn(x_vals),
    label="SOTA",
    color="red"
)

plt.plot(
    x_vals,
    alphaevolve_total_fn(x_vals),
    label="AlphaEvolve",
    color="green",
)

plt.axhline(0, color="black", linestyle="--", linewidth=0.5)  # x-axis
plt.xlabel("x")
plt.ylabel("f(x)")
plt.title(
    "SOTA and AlphaEvolve's discovered function for the uncertainty inequality"
)
plt.legend()
plt.grid(True)
plt.show()

Next, we follow the basis construction outlined by [H. Cohn and F. Gonçalves (2019)](https://link.springer.com/article/10.1007/s00222-019-00875-4).

For comparison we use the data provided in Table 2, p. 821, which outlines the A+(d) constants. In particular, we are interested in A+(1) (reported as about $0.572990$) - its square also gives an upper bound on the uncertainty principle coefficient (i.e. $C_4\leq 0.3284$).

Instead of Hermite polynomials, here we use Laguerre polynomials (following H. Cohn and F. Gonçalves above). To this end, below we adjust our verifier accordingly.

Below we report a configuration AlphaEvolve found with 12 roots and A+(1) of about $0.56708661892$. This result implies the new upper bound $C_4 \leq 0.3216$.


In [ ]:
# @title Utils

import sys
from typing import Sequence
import matplotlib.pyplot as plt
import numpy as np
import scipy.special
import sympy

sys.set_int_max_str_digits(40000)


def find_laguerre_combination(m: int, n: int, z: Sequence[float]) -> sympy.Expr:

  alpha = sympy.Rational(n, 2) - 1
  degrees = np.arange(0, 4 * m + 4, 2)
  x = sympy.symbols("x")
  lps = [
      sympy.polys.orthopolys.laguerre_poly(n=i, x=x, alpha=alpha, polys=False)
      for i in degrees
  ]

  num_lps = len(lps)
  num_conditions = 2 * m + 2  # Root at 0, double roots at z_i

  if num_lps < num_conditions:
    raise ValueError(
        "Not enough Laguerre polynomials to satisfy all conditions."
    )

  # Create a system of linear equations to solve for alpha_i
  A = sympy.Matrix(num_conditions, num_lps, lambda i, j: 0)
  b = sympy.Matrix(num_conditions, 1, lambda i, j: 0)

  b[1] = 1  # Make sure the derivative of the function at zero is 1

  # Condition 1: Root at 0
  for j in range(num_lps):
    A[0, j] = lps[j].subs(x, 0)
    A[1, j] = lps[j].diff(x).subs(x, 0)

  # Conditions 2 to 2m+2: Double roots at z_i (g(z_i) = 0 and g'(z_i) = 0, where
  # g is the function we are constructing)
  for i in range(0, m):
    zi = sympy.Rational(z[i])  # Ensure z_i is rational

    # g(z_i) = 0
    for j in range(num_lps):
      A[2 * i + 2, j] = lps[j].subs(x, zi)

    # g'(z_i) = 0 (derivative with respect to x)
    for j in range(num_lps):
      deriv_lp_j = lps[j].diff(x)  # Symbolic differentiation
      A[2 * i + 3, j] = deriv_lp_j.subs(x, zi)

  # Solve the linear system Ax = b for x (coefficients alpha_i)
  x = A.LUsolve(b)  # Use LU decomposition for efficient solving
  alpha_coeffs = [x[i] for i in range(num_lps)]
  g_fn = sum(alpha_coeffs[i] * lps[i] for i in range(len(alpha_coeffs)))
  return g_fn


In [ ]:
# @title Data

# Config with twelve roots

m = 12
n = 1

zs = [
    3.6331003,
    5.6714292,
    33.09981679,
    38.849942887744,
    41.1543366,
    43.18733472973041,
    50.98385922,
    58.63890192096693,
    96.0237184433404,
    111.2160645819441,
    118.90258667688155,
    141.44196227075895,
]

In [ ]:
# @title Visualization and Root Checking

zs = np.sort(zs)

g_fn = find_laguerre_combination(m, n, zs)

x = sympy.symbols("x")

dg_fn = sympy.diff(g_fn, x)
gn_fn = sympy.lambdify(x, g_fn, modules=["numpy"])
dgn_fn = sympy.lambdify(x, dg_fn, modules=["numpy"])

div = sympy.prod([(x - sympy.Rational(z)) ** 2 for z in zs]) * x
gq_fn = sympy.exquo(g_fn, div)

g_fn.subs(x, sympy.Rational(0))
dg_fn.subs(x, sympy.Rational(1)) == 0

for z in zs:
  assert (
      g_fn.subs(x, sympy.Rational(z)) == 0
  ), f"Checking condition g({sympy.Rational(z)}) = {g_fn.subs(x, sympy.Rational(z))} failed."
  assert (
      dg_fn.subs(x, sympy.Rational(z)) == 0
  ), f"Checking condition g'({sympy.Rational(z)}) = {dg_fn.subs(x, sympy.Rational(z))} failed."

# Plotting
x_vals = np.linspace(-0.1, 4, 200)

plt.figure(figsize=(8, 5))
plt.plot(
    x_vals,
    gn_fn(x_vals),
)

plt.axhline(0, color="black", linestyle="--", linewidth=0.5)  # x-axis
plt.xlabel("x")
plt.ylabel("g(x)")
plt.title("Linear combination of Laguerre polynomials found by AlphaEvolve")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# @title Verification


rroots = sympy.real_roots(gq_fn, x)
approx_roots = list()
largest_sign_change = 0
all_sign_changes = list()

for root in rroots:
  approx_root = root.eval_rational(n=200)
  approx_root_p = approx_root + sympy.Rational(1e-198)
  approx_root_m = approx_root - sympy.Rational(1e-198)
  approx_roots.append(approx_root)
  is_sign_change = (
      gq_fn.subs(x, approx_root_p) > 0 and gq_fn.subs(x, approx_root_m) < 0
  ) or (gq_fn.subs(x, approx_root_p) < 0 and gq_fn.subs(x, approx_root_m) > 0)
  if is_sign_change:
    all_sign_changes.append(approx_root)
    largest_sign_change = max(largest_sign_change, approx_root)

lsc = float(largest_sign_change)
upper_bound = lsc / 2 / np.pi
ap_estimate = np.sqrt(upper_bound)
print(f"The construction of AlphaEvolve is correct and gives the A+(1) estimate: {ap_estimate}")
print(f"This implies the upper bound: C4 <= {upper_bound}")


# B.5. Erdős' minimum overlap problem

Let $C_5$ be the largest constant satisfying
$$ \sup_{x \in [-2,2]} \int_{-1}^1 f(t) g(x+t)\ dt\geq C_5$$
for all non-negative $f,g: [-1,1] \to [0,1]$ with $f+g=1$ on $[-1,1]$ and
$\int f(x)\ dx = 1$, where we extend $f,g$ by zero outside of $[-1,1]$. This
constant controls the asymptotics of the Minimum Overlap Problem described by
[Erdős (1955)](https://link.springer.com/article/10.1007/BF02760020). The bounds
$$0.379005 \leq C_5 \leq 0.380927$$
are known; the lower bound was obtained by
[White (2023)](https://www.impan.pl/en/publishing-house/journals-and-series/acta-arithmetica/online/115217/a-new-bound-for-erdos-minimum-overlap-problem)
via convex programming methods.

It is known (see [Haugland (2016)](https://arxiv.org/abs/1609.08000)) that
this constant is equal to the infimum, over all step
functions $h$ on $[0, 2]$ with values in $[0, 1]$ and satisfying
$
\int_0^2 h(x)dx = 1
$
of
$$
\max_k \int h(x)(1 - h(x+k))dx
.$$ The upper bound to the Erdős minimum overlap problem was then obtained by using this result, in
[Haugland (2016)](https://arxiv.org/abs/1609.08000) by a step function
construction.

AlphaEvolve discovered an alternative step function construction that improves
the upper bound to $C_5 \leq 0.380924$.

In [ ]:
#@title Verification and plotting code

import numpy as np
import matplotlib.pyplot as plt


def plot_step_function(step_heights_input: list[float]):
  num_steps = len(step_heights_input)

  # Generate x values for plotting (need to plot steps properly).
  step_edges_plot = np.linspace(-0.25, 0.25, num_steps + 1)
  x_plot = np.array([])
  y_plot = np.array([])

  for i in range(num_steps):
    x_start = step_edges_plot[i]
    x_end = step_edges_plot[i + 1]
    x_step_vals = np.linspace(x_start, x_end, 100)  # Points within each step.
    y_step_vals = np.full_like(x_step_vals, step_heights_input[i])
    x_plot = np.concatenate((x_plot, x_step_vals))
    y_plot = np.concatenate((y_plot, y_step_vals))

  # Plot the step function.
  plt.figure(figsize=(8, 5))
  plt.plot(x_plot, y_plot)
  plt.xlabel("x")
  plt.ylabel("f(x)")
  plt.title(
      "Step function found by AlphaEvolve for Erdős' minimum overlap problem"
  )
  plt.xlim([-0.3, 0.3])
  plt.ylim([-1, max(step_heights_input) * 1.2])
  plt.grid(True)
  plt.step(
      step_edges_plot[:-1],
      step_heights_input,
      where='post',
      color='green',
      linewidth=2,
  )  # Overlay with plt.step for clarity.
  plt.show()


def verify_sequence(sequence: list[float]):
  """Raises an error if the sequence does not satisfy the constraints."""
  # Check that all values are between 0 and 1.
  if not all(0 <= val <= 1 for val in sequence):
    raise AssertionError("All values in the sequence must be between 0 and 1.")
  # Check that the sum of values in the sequence is exactly n / 2.0.
  if not np.sum(sequence) == len(sequence) / 2.0:
    raise AssertionError(
        "The sum of values in the sequence must be exactly n / 2.0. The sum is "
        f"{np.sum(sequence)} but it should be {len(sequence) / 2.0}.")
  print(
      "The sequence generates a valid step function for Erdős' minimum "
      "overlap problem."
  )


def compute_upper_bound(sequence: list[float]) -> float:
  """Returns the upper bound for a sequence of coefficients."""
  convolution_values = np.correlate(
      np.array(sequence), 1 - np.array(sequence), mode='full'
  )
  return np.max(convolution_values) / len(sequence) * 2

In [ ]:
#@title Data
best_sequence = np.array([
    0.0, 0.0, 0.0, 0.0, 0.0,
    0.0, 3.60911302e-10, 3.62124044e-10, 4.02849974e-12,
    4.47352578e-12, 4.76914172e-12, 0.506074303,
    0.632046692, 0.679332798, 0.888193865,
    0.889214704, 0.678231235, 2.976636922840846e-07,
    0.0947643739, 0.0143926342, 0.423931858,
    0.598073612, 0.803909612, 0.683098916,
    0.314749384, 0.404059484, 0.858443734,
    0.796503042, 0.590433152, 0.41056218,
    0.270932695, 0.613384276, 0.709501647,
    0.580573615, 0.803538112, 0.715263878,
    0.822611331, 0.808433879, 0.683533985,
    0.645719012, 0.889417725, 0.943389845,
    0.841536959, 0.794505216, 0.941943428,
    0.962223227, 0.961270753, 0.992409079,
])
reversed_sequence = best_sequence[::-1]
best_sequence = np.concatenate((best_sequence[:-1], reversed_sequence))

In [ ]:
#@title Verification and plotting

verify_sequence(best_sequence)
plot_step_function(best_sequence)
new_upper_bound = compute_upper_bound(best_sequence)
print(
    f"The sequence provides the following upper bound on the Erdős minimum overlap problem: C5 <= {new_upper_bound}."
)

# B.6. Sums and differences of finite sets

Let $C_6$ be the largest constant for which the following statement holds: there exist arbitrarily large finite sets of integers $A,B$ with $|A+B| \ll |A|$ and $|A-B| \gg |A+B|^{C_6}$.
(Here $A+B = \{a+b : a \in A, b \in B\}$ and $A-B = \{a-b : a \in A, b \in B\}$ denote the sumset and difference set, respectively. The notation $X \ll Y$ means that $X \le C Y$ for some constant $C$ independent of the sets $A,B$ (for sufficiently large sets $A,B$). The notation $X \gg Y$ means that $X \ge C' Y$ for some positive constant $C'$ independent of the sets $A,B$ (for sufficiently large sets $A,B$).)

It is known that
$$ 1.14465 \leq C_6 \leq \frac{4}{3}, $$
where the upper and lower bounds can be respectively found in Corollary 3 and
Theorem 1 of
[Gyarmati et al. (2007)](https://projecteuclid.org/journals/functiones-et-approximatio-commentarii-mathematici/volume-37/issue-1/Sums-and-differences-of-finite-sets/10.7169/facm/1229618749.full).
To build the lower bound, Gyarmati et al. use the inequality
$$C_6 \geq 1 + \frac{\log \frac{|U-U|}{|U+U|}}{\log (2 \max (U) + 1)}, $$
which holds for any finite set $U$ of non-negative integers containing zero satisfying $|U-U| \leq 2 \max (U) + 1$.

AlphaEvolve found two constructions:
- A set $U_1$ of 2003 integers that improves the lower bound to $1.1479 \leq C_6$.
- A set $U_2$ of 54265 integers that further improves the lower bound to $1.1584 \leq C_6$.

For completeness, we provide both constructions below.

In [ ]:
#@title Data

solution_1 = [0, 110, 116, 122, 128, 134, 140, 2509, 2515, 2521, 2527, 2533, 4908, 4914, 4920, 4926, 6036, 6042, 6048, 6054, 6060, 7307, 7313, 7319, 8435, 8441, 8447, 8453, 9706, 9712, 10834, 10840, 10846, 11962, 11968, 11974, 11980, 12105, 13233, 13239, 14361, 14367, 14373, 14552, 14558, 14564, 14570, 14576, 15632, 16760, 16766, 16951, 16957, 16963, 16969, 17888, 17894, 17900, 19159, 19350, 19356, 19362, 20287, 20293, 20478, 20484, 20490, 20496, 21749, 21755, 22686, 22877, 22883, 22889, 23814, 23820, 24148, 25276, 25282, 26213, 26404, 26410, 26416, 27675, 28803, 28809, 28994, 29000, 29006, 29012, 29740, 31202, 31393, 31399, 31405, 32330, 32336, 33792, 33798, 34729, 34920, 34926, 34932, 35262, 35268, 35274, 35280, 35286, 36191, 37319, 37325, 37661, 37667, 37673, 37679, 38256, 39718, 40060, 40066, 40072, 40846, 40852, 41188, 41194, 41200, 41206, 42459, 42465, 43245, 43436, 43442, 43448, 43587, 43593, 43599, 44858, 45835, 45841, 45986, 45992, 46772, 47114, 47120, 47126, 48234, 48385, 49362, 49368, 49513, 49519, 49704, 49710, 49716, 49722, 51761, 51912, 52103, 52109, 52115, 53040, 53046, 54502, 54508, 55288, 55439, 55630, 55636, 55642, 56901, 57878, 57884, 58029, 58035, 58966, 60277, 60428, 61556, 61562, 63804, 63955, 64146, 64152, 64158, 66545, 66551, 67482, 68944, 70072, 70078, 70414, 70420, 70426, 70432, 72320, 72471, 72813, 72819, 72825, 75212, 75218, 75998, 76340, 76346, 76352, 77611, 78588, 78594, 78739, 78745, 80987, 81138, 82266, 82272, 84514, 84665, 84856, 84862, 84868, 85627, 85633, 85639, 85645, 85651, 87255, 87261, 88026, 88032, 88038, 88044, 88192, 89654, 90425, 90431, 90437, 90782, 90788, 91553, 91559, 91565, 91571, 92824, 92830, 93030, 93181, 93952, 93958, 93964, 95223, 96351, 96357, 96708, 97479, 97485, 97491, 98750, 99298, 99304, 99878, 99884, 100069, 100075, 100081, 100087, 101697, 102277, 102468, 102474, 102480, 103405, 103411, 104867, 104873, 105224, 105566, 105572, 105578, 105804, 105995, 106001, 106007, 107266, 107965, 107971, 108394, 108400, 109331, 110364, 110793, 111492, 111498, 111921, 111927, 113740, 113891, 114320, 114511, 114517, 114523, 116910, 116916, 117418, 117847, 119309, 120008, 120014, 120437, 120443, 120779, 120785, 120791, 120797, 122407, 122836, 123178, 123184, 123190, 125577, 125583, 125934, 126363, 126705, 126711, 126717, 127976, 128953, 128959, 129104, 129110, 131352, 131503, 132631, 132637, 134450, 134879, 135030, 135221, 135227, 135233, 137620, 137626, 138557, 140019, 140718, 140724, 141147, 141153, 143117, 143395, 143546, 146644, 147073, 149663, 149669, 152062, 155160, 155589, 155931, 155937, 155943, 158330, 158336, 160729, 161857, 161863, 164105, 164256, 167783, 170373, 170379, 171144, 171150, 171156, 171162, 172772, 173543, 173549, 173555, 175870, 175942, 175948, 176299, 177070, 177076, 177082, 178341, 179469, 179475, 181868, 182996, 183002, 184815, 185395, 185586, 185592, 185598, 187985, 187991, 188922, 190384, 191083, 191089, 191512, 191518, 193482, 193911, 197009, 197438, 200028, 200034, 202427, 205525, 205954, 206296, 206302, 206308, 208085, 208091, 208097, 208103, 208109, 208118, 208124, 208128, 208130, 208134, 208136, 208140, 208142, 208146, 208152, 208695, 208701, 210484, 210490, 210496, 210502, 210517, 210523, 210527, 210529, 210533, 210535, 210539, 210545, 211094, 212222, 212228, 212883, 212889, 212895, 212916, 212922, 212926, 212928, 212932, 212938, 214011, 214017, 214023, 214029, 214044, 214050, 214054, 214056, 214060, 214062, 214066, 214072, 214470, 214621, 215282, 215288, 215315, 215321, 215325, 215331, 216410, 216416, 216422, 216443, 216449, 216453, 216455, 216459, 216465, 217681, 217714, 217724, 218148, 218809, 218815, 218842, 218848, 218852, 218858, 219937, 219943, 219949, 219970, 219976, 219980, 219982, 219986, 219992, 220738, 220744, 221208, 221241, 221251, 222336, 222342, 222369, 222375, 222379, 222385, 222527, 222533, 222539, 222545, 222560, 222566, 222570, 222572, 222576, 222578, 222582, 222588, 223137, 224735, 224768, 224778, 224926, 224932, 224938, 224959, 224965, 224969, 224971, 224975, 224981, 225863, 225869, 225896, 225902, 225906, 225912, 226235, 226664, 227325, 227331, 227358, 227364, 227368, 227374, 228262, 228295, 228305, 228453, 228459, 228465, 228486, 228492, 228496, 228498, 228502, 228508, 229724, 229757, 229767, 230852, 230858, 230885, 230891, 230895, 230901, 231789, 231822, 231832, 233251, 233284, 233294, 234379, 234385, 234412, 234418, 234422, 234428, 235180, 236778, 236811, 236821, 236969, 236975, 236981, 237002, 237008, 237012, 237014, 237018, 237024, 239368, 239374, 239401, 239407, 239411, 239417, 240305, 240338, 240348, 241448, 241454, 241767, 241800, 241810, 242895, 242901, 242928, 242934, 242938, 242944, 243237, 243243, 243249, 243255, 243270, 243276, 243280, 243282, 243286, 243288, 243292, 243298, 243847, 245294, 245327, 245337, 245636, 245642, 245648, 245669, 245675, 245679, 245681, 245685, 245691, 247374, 248035, 248041, 248068, 248074, 248078, 248084, 248821, 248854, 248864, 249163, 249169, 249175, 249196, 249202, 249206, 249208, 249212, 249218, 250434, 250467, 250477, 251411, 251417, 251444, 251450, 251454, 251460, 251562, 251568, 251595, 251601, 251605, 251611, 253810, 253843, 253853, 253961, 253994, 254004, 255089, 255095, 255122, 255128, 255132, 255138, 255890, 256661, 256667, 256673, 257337, 257370, 257380, 257488, 257521, 257531, 257679, 257685, 257691, 257712, 257718, 257722, 257724, 257728, 257734, 259060, 259066, 260078, 260084, 260111, 260117, 260121, 260127, 261015, 261048, 261058, 261459, 262477, 262510, 262520, 262587, 262593, 263605, 263611, 263638, 263644, 263648, 263654, 264986, 265853, 265886, 265896, 266004, 266037, 266047, 268513, 269531, 269564, 269574, 271103, 271109, 272121, 272127, 272154, 272160, 272164, 272170, 273502, 274520, 274553, 274563, 276600, 277029, 278047, 278080, 278090, 278389, 278395, 278401, 278422, 278428, 278432, 278434, 278438, 278444, 280788, 280794, 280821, 280827, 280831, 280837, 283187, 283220, 283230, 284315, 284321, 284348, 284354, 284358, 284364, 285545, 286563, 286596, 286606, 286714, 286747, 286757, 290241, 290274, 290284, 291813, 291819, 292831, 292837, 292864, 292870, 292874, 292880, 293602, 293608, 293614, 293620, 293635, 293641, 293645, 293647, 293651, 293653, 293657, 293663, 294212, 295230, 295263, 295273, 296001, 296007, 296013, 296034, 296040, 296044, 296046, 296050, 296056, 297739, 298400, 298406, 298433, 298439, 298443, 298449, 298757, 298790, 298800, 299528, 299534, 299540, 299561, 299567, 299571, 299573, 299577, 299583, 300799, 300832, 300842, 301927, 301933, 301960, 301966, 301970, 301976, 304326, 304359, 304369, 305454, 305460, 305487, 305493, 305497, 305503, 306255, 307273, 307306, 307316, 307853, 307886, 307896, 308044, 308050, 308056, 308077, 308083, 308087, 308089, 308093, 308099, 310443, 310449, 310476, 310482, 310486, 310492, 311380, 311413, 311423, 312842, 312875, 312885, 313541, 313547, 313574, 313580, 313584, 313590, 313970, 313976, 314003, 314009, 314013, 314019, 315940, 315973, 315983, 316369, 316402, 316412, 319467, 319500, 319510, 319896, 319929, 319939, 322486, 322492, 322519, 322525, 322529, 322535, 324885, 324918, 324928, 326965, 327983, 328016, 328026, 328412, 328445, 328455, 328754, 328760, 328766, 328787, 328793, 328797, 328799, 328803, 328809, 331153, 331159, 331186, 331192, 331196, 331202, 333552, 333585, 333595, 334680, 334686, 334713, 334719, 334723, 334729, 336928, 336961, 336971, 337079, 337112, 337122, 340606, 340639, 340649, 342178, 342184, 343196, 343202, 343229, 343235, 343239, 343245, 344577, 345595, 345628, 345638, 348104, 348693, 348726, 348736, 349122, 349155, 349165, 356620, 357638, 357671, 357681, 363906, 363912, 363939, 363945, 363949, 363955, 366305, 366338, 366348, 369832, 369865, 369875, 377330, 378348, 378381, 378391, 379119, 379125, 379131, 379152, 379158, 379162, 379164, 379168, 379174, 381518, 381524, 381551, 381557, 381561, 381567, 383917, 383950, 383960, 385045, 385051, 385078, 385084, 385088, 385094, 387444, 387477, 387487, 390971, 391004, 391014, 393561, 393567, 393594, 393600, 393604, 393610, 395960, 395993, 396003, 399058, 399091, 399101, 399487, 399520, 399530, 408003, 408036, 408046, 414271, 414277, 414304, 414310, 414314, 414320, 416060, 416066, 416072, 416078, 416093, 416099, 416103, 416105, 416109, 416111, 416115, 416121, 416126, 416132, 416136, 416138, 416142, 416144, 416146, 416148, 416152, 416154, 416158, 416164, 416670, 416703, 416713, 418459, 418465, 418471, 418492, 418498, 418502, 418504, 418508, 418514, 418525, 418531, 418535, 418537, 418541, 418545, 418547, 418551, 418557, 420197, 420230, 420240, 420858, 420864, 420891, 420897, 420901, 420907, 420924, 420930, 420934, 420940, 420944, 420950, 421986, 421992, 421998, 422019, 422025, 422029, 422031, 422035, 422041, 422052, 422058, 422062, 422064, 422068, 422072, 422074, 422078, 422084, 423257, 423290, 423300, 423323, 423333, 423343, 424385, 424391, 424418, 424424, 424428, 424434, 424451, 424457, 424461, 424467, 424471, 424477, 426784, 426817, 426827, 426850, 426860, 426870, 427695, 427912, 427918, 427945, 427951, 427955, 427961, 427978, 427984, 427988, 427994, 427998, 428004, 428713, 428746, 428756, 430311, 430344, 430354, 430377, 430387, 430397, 430502, 430508, 430514, 430535, 430541, 430545, 430547, 430551, 430557, 430568, 430574, 430578, 430580, 430584, 430588, 430590, 430594, 430600, 432901, 432907, 432934, 432940, 432944, 432950, 432967, 432973, 432977, 432983, 432987, 432993, 433838, 433871, 433881, 433904, 433914, 433924, 435300, 435333, 435343, 435366, 435376, 435386, 436428, 436434, 436461, 436467, 436471, 436477, 436494, 436500, 436504, 436510, 436514, 436520, 438827, 438860, 438870, 438893, 438903, 438913, 442354, 442387, 442397, 442420, 442430, 442440, 444944, 444950, 444977, 444983, 444987, 444993, 445010, 445016, 445020, 445026, 445030, 445036, 447343, 447376, 447386, 447409, 447419, 447429, 449423, 449456, 449466, 450870, 450903, 450913, 450936, 450946, 450956, 451212, 451218, 451224, 451245, 451251, 451255, 451257, 451261, 451267, 451278, 451284, 451288, 451290, 451294, 451298, 451300, 451304, 451310, 453611, 453617, 453644, 453650, 453654, 453660, 453677, 453683, 453687, 453693, 453697, 453703, 456010, 456043, 456053, 456076, 456086, 456096, 457138, 457144, 457171, 457177, 457181, 457187, 457204, 457210, 457214, 457220, 457224, 457230, 459386, 459419, 459429, 459452, 459462, 459472, 459537, 459570, 459580, 459603, 459613, 459623, 463064, 463097, 463107, 463130, 463140, 463150, 464636, 464642, 464669, 464675, 464679, 464685, 465654, 465660, 465687, 465693, 465697, 465703, 465720, 465726, 465730, 465736, 465740, 465746, 467035, 467068, 467078, 468053, 468086, 468096, 468119, 468129, 468139, 470562, 470595, 470605, 471580, 471613, 471623, 471646, 471656, 471666, 479078, 479111, 479121, 480096, 480129, 480139, 480162, 480172, 480182, 486364, 486370, 486397, 486403, 486407, 486413, 486430, 486436, 486440, 486446, 486450, 486456, 488763, 488796, 488806, 488829, 488839, 488849, 492290, 492323, 492333, 492356, 492366, 492376, 499788, 499821, 499831, 500806, 500839, 500849, 500872, 500882, 500892, 501577, 501583, 501589, 501610, 501616, 501620, 501622, 501626, 501632, 501643, 501649, 501653, 501655, 501659, 501663, 501665, 501669, 501675, 503976, 503982, 504009, 504015, 504019, 504025, 504042, 504048, 504052, 504058, 504062, 504068, 506375, 506408, 506418, 506441, 506451, 506461, 507503, 507509, 507536, 507542, 507546, 507552, 507569, 507575, 507579, 507585, 507589, 507595, 509902, 509935, 509945, 509968, 509978, 509988, 513429, 513462, 513472, 513495, 513505, 513515, 516019, 516025, 516052, 516058, 516062, 516068, 516085, 516091, 516095, 516101, 516105, 516111, 518418, 518451, 518461, 518484, 518494, 518504, 521516, 521549, 521559, 521582, 521592, 521602, 521945, 521978, 521988, 522011, 522021, 522031, 530461, 530494, 530504, 530527, 530537, 530547, 536729, 536735, 536762, 536768, 536772, 536778, 536795, 536801, 536805, 536811, 536815, 536821, 539128, 539161, 539171, 539194, 539204, 539214, 542655, 542688, 542698, 542721, 542731, 542741, 550153, 550186, 550196, 551171, 551204, 551214, 551237, 551247, 551257, 571881, 571914, 571924, 571947, 571957, 571967, 587094, 587100, 587127, 587133, 587137, 587143, 587160, 587166, 587170, 587176, 587180, 587186, 589493, 589526, 589536, 589559, 589569, 589579, 593020, 593053, 593063, 593086, 593096, 593106, 601536, 601569, 601579, 601602, 601612, 601622, 622246, 622279, 622289, 622312, 622322, 622332, 624035, 624041, 624047, 624068, 624074, 624078, 624080, 624084, 624090, 624101, 624107, 624111, 624113, 624117, 624121, 624123, 624127, 624133, 624134, 624140, 624144, 624146, 624150, 624154, 624156, 624160, 624164, 624166, 624170, 624176, 626434, 626440, 626467, 626473, 626477, 626483, 626500, 626506, 626510, 626516, 626520, 626526, 626533, 626539, 626543, 626549, 626553, 626559, 626563, 626569, 628833, 628866, 628876, 628899, 628909, 628919, 628932, 628942, 628952, 628962, 629961, 629967, 629994, 630000, 630004, 630010, 630027, 630033, 630037, 630043, 630047, 630053, 630060, 630066, 630070, 630076, 630080, 630086, 630090, 630096, 632360, 632393, 632403, 632426, 632436, 632446, 632459, 632469, 632479, 632489, 635887, 635920, 635930, 635953, 635963, 635973, 635986, 635996, 636006, 636016, 638477, 638483, 638510, 638516, 638520, 638526, 638543, 638549, 638553, 638559, 638563, 638569, 638576, 638582, 638586, 638592, 638596, 638602, 638606, 638612, 640876, 640909, 640919, 640942, 640952, 640962, 640975, 640985, 640995, 641005, 644403, 644436, 644446, 644469, 644479, 644489, 644502, 644512, 644522, 644532, 652919, 652952, 652962, 652985, 652995, 653005, 653018, 653028, 653038, 653048, 659187, 659193, 659220, 659226, 659230, 659236, 659253, 659259, 659263, 659269, 659273, 659279, 659286, 659292, 659296, 659302, 659306, 659312, 659316, 659322, 661586, 661619, 661629, 661652, 661662, 661672, 661685, 661695, 661705, 661715, 665113, 665146, 665156, 665179, 665189, 665199, 665212, 665222, 665232, 665242, 672611, 672644, 672654, 672677, 672687, 672697, 673629, 673662, 673672, 673695, 673705, 673715, 673728, 673738, 673748, 673758, 694339, 694372, 694382, 694405, 694415, 694425, 694438, 694448, 694458, 694468, 709552, 709558, 709585, 709591, 709595, 709601, 709618, 709624, 709628, 709634, 709638, 709644, 709651, 709657, 709661, 709667, 709671, 709677, 709681, 709687, 711951, 711984, 711994, 712017, 712027, 712037, 712050, 712060, 712070, 712080, 715478, 715511, 715521, 715544, 715554, 715564, 715577, 715587, 715597, 715607, 723994, 724027, 724037, 724060, 724070, 724080, 724093, 724103, 724113, 724123, 744704, 744737, 744747, 744770, 744780, 744790, 744803, 744813, 744823, 744833, 795069, 795102, 795112, 795135, 795145, 795155, 795168, 795178, 795188, 795198, 832010, 832016, 832043, 832049, 832053, 832059, 832076, 832082, 832086, 832092, 832096, 832102, 832109, 832115, 832119, 832125, 832129, 832135, 832139, 832142, 832145, 832148, 832152, 832158, 832162, 832168, 832172, 832178, 832182, 832188, 834409, 834442, 834452, 834475, 834485, 834495, 834508, 834518, 834528, 834538, 834541, 834551, 834561, 834571, 834581, 837936, 837969, 837979, 838002, 838012, 838022, 838035, 838045, 838055, 838065, 838068, 838078, 838088, 838098, 838108, 846452, 846485, 846495, 846518, 846528, 846538, 846551, 846561, 846571, 846581, 846584, 846594, 846604, 846614, 846624, 867162, 867195, 867205, 867228, 867238, 867248, 867261, 867271, 867281, 867291, 867294, 867304, 867314, 867324, 867334, 917527, 917560, 917570, 917593, 917603, 917613, 917626, 917636, 917646, 917656, 917659, 917669, 917679, 917689, 917699, 1039985, 1040018, 1040028, 1040051, 1040061, 1040071, 1040084, 1040094, 1040104, 1040114, 1040117, 1040127, 1040137, 1040147, 1040150, 1040157, 1040160, 1040170, 1040180, 1040190, 1040200]

solution_2 = [0, 6, 43, 80, 117, 154, 191, 228, 2084, 2121, 2158, 2195, 2232, 2269, 4162, 4199, 4236, 4273, 4310, 6240, 6277, 6314, 6351, 6796, 6833, 6870, 6907, 6944, 6981, 8318, 8355, 8392, 8874, 8911, 8948, 8985, 9022, 10396, 10433, 10952, 10989, 11026, 11063, 12474, 13030, 13067, 13104, 13586, 13623, 13660, 13697, 13734, 15108, 15145, 15664, 15701, 15738, 15775, 17186, 17742, 17779, 17816, 19820, 19857, 20376, 20413, 20450, 20487, 21898, 22454, 22491, 22528, 24532, 24569, 26610, 27166, 27203, 27240, 29244, 29281, 31322, 33956, 33993, 36034, 40746, 309562, 309599, 309636, 309673, 309710, 309747, 311640, 311677, 311714, 311751, 311788, 313718, 313755, 313792, 313829, 315796, 315833, 315870, 316352, 316389, 316426, 316463, 316500, 317874, 317911, 318430, 318467, 318504, 318541, 319952, 320508, 320545, 320582, 322586, 322623, 323142, 323179, 323216, 323253, 324664, 325220, 325257, 325294, 327298, 327335, 329376, 329932, 329969, 330006, 332010, 332047, 334088, 336722, 336759, 338800, 343512, 619118, 619155, 619192, 619229, 619266, 621196, 621233, 621270, 621307, 623274, 623311, 623348, 625352, 625389, 625908, 625945, 625982, 626019, 627430, 627986, 628023, 628060, 630064, 630101, 632142, 632698, 632735, 632772, 634776, 634813, 636854, 639488, 639525, 641566, 646278, 928674, 928711, 928748, 928785, 930752, 930789, 930826, 932830, 932867, 934908, 935464, 935501, 935538, 937542, 937579, 939620, 942254, 942291, 944332, 949044, 1090636, 1090673, 1090710, 1090747, 1090784, 1090821, 1092714, 1092751, 1092788, 1092825, 1092862, 1094792, 1094829, 1094866, 1094903, 1096870, 1096907, 1096944, 1097426, 1097463, 1097500, 1097537, 1097574, 1098948, 1098985, 1099504, 1099541, 1099578, 1099615, 1101026, 1101582, 1101619, 1101656, 1103660, 1103697, 1104216, 1104253, 1104290, 1104327, 1105738, 1106294, 1106331, 1106368, 1108372, 1108409, 1110450, 1111006, 1111043, 1111080, 1113084, 1113121, 1115162, 1117796, 1117833, 1119874, 1124586, 1238230, 1238267, 1238304, 1240308, 1240345, 1242386, 1245020, 1245057, 1247098, 1251810, 1400192, 1400229, 1400266, 1400303, 1400340, 1402270, 1402307, 1402344, 1402381, 1404348, 1404385, 1404422, 1406426, 1406463, 1406982, 1407019, 1407056, 1407093, 1408504, 1409060, 1409097, 1409134, 1411138, 1411175, 1413216, 1413772, 1413809, 1413846, 1415850, 1415887, 1417928, 1420562, 1420599, 1422640, 1427352, 1547786, 1547823, 1549864, 1554576, 1709748, 1709785, 1709822, 1709859, 1711826, 1711863, 1711900, 1713904, 1713941, 1715982, 1716538, 1716575, 1716612, 1718616, 1718653, 1720694, 1723328, 1723365, 1725406, 1730118, 1857342, 2019304, 2019341, 2019378, 2021382, 2021419, 2023460, 2026094, 2026131, 2028172, 2032884, 2181266, 2181303, 2181340, 2181377, 2181414, 2183344, 2183381, 2183418, 2183455, 2185422, 2185459, 2185496, 2187500, 2187537, 2188056, 2188093, 2188130, 2188167, 2189578, 2190134, 2190171, 2190208, 2192212, 2192249, 2194290, 2194846, 2194883, 2194920, 2196924, 2196961, 2199002, 2201636, 2201673, 2203714, 2208426, 2328860, 2328897, 2330938, 2335650, 2490822, 2490859, 2490896, 2490933, 2492900, 2492937, 2492974, 2494978, 2495015, 2497056, 2497612, 2497649, 2497686, 2499690, 2499727, 2501768, 2504402, 2504439, 2506480, 2511192, 2523896, 2523933, 2523970, 2524007, 2524044, 2524081, 2525974, 2526011, 2526048, 2526085, 2526122, 2528052, 2528089, 2528126, 2528163, 2530130, 2530167, 2530204, 2530686, 2530723, 2530760, 2530797, 2530834, 2532208, 2532245, 2532764, 2532801, 2532838, 2532875, 2534286, 2534842, 2534879, 2534916, 2536920, 2536957, 2537476, 2537513, 2537550, 2537587, 2538998, 2539554, 2539591, 2539628, 2541632, 2541669, 2543710, 2544266, 2544303, 2544340, 2546344, 2546381, 2548422, 2551056, 2551093, 2553134, 2557846, 2638416, 2800378, 2800415, 2800452, 2802456, 2802493, 2804534, 2807168, 2807205, 2809246, 2813958, 2833452, 2833489, 2833526, 2833563, 2833600, 2835530, 2835567, 2835604, 2835641, 2837608, 2837645, 2837682, 2839686, 2839723, 2840242, 2840279, 2840316, 2840353, 2841764, 2842320, 2842357, 2842394, 2844398, 2844435, 2846476, 2847032, 2847069, 2847106, 2849110, 2849147, 2851188, 2853822, 2853859, 2855900, 2860612, 3109934, 3109971, 3112012, 3116724, 3143008, 3143045, 3143082, 3143119, 3145086, 3145123, 3145160, 3147164, 3147201, 3149242, 3149798, 3149835, 3149872, 3151876, 3151913, 3153954, 3156588, 3156625, 3158666, 3163378, 3271896, 3271933, 3271970, 3272007, 3273974, 3274011, 3274048, 3276052, 3276089, 3278130, 3278686, 3278723, 3278760, 3280764, 3280801, 3282842, 3285476, 3285513, 3287554, 3292266, 3419490, 3452564, 3452601, 3452638, 3454642, 3454679, 3456720, 3459354, 3459391, 3461432, 3466144, 3581452, 3581489, 3581526, 3583530, 3583567, 3585608, 3588242, 3588279, 3590320, 3595032, 3599084, 3599121, 3599158, 3599195, 3599232, 3599269, 3601162, 3601199, 3601236, 3601273, 3601310, 3603240, 3603277, 3603314, 3603351, 3605318, 3605355, 3605392, 3605874, 3605911, 3605948, 3605985, 3606022, 3607396, 3607433, 3607952, 3607989, 3608026, 3608063, 3609474, 3610030, 3610067, 3610104, 3612108, 3612145, 3612664, 3612701, 3612738, 3612775, 3614186, 3614526, 3614563, 3614600, 3614637, 3614674, 3614742, 3614779, 3614816, 3616604, 3616641, 3616678, 3616715, 3616820, 3616857, 3618682, 3618719, 3618756, 3618898, 3619454, 3619491, 3619528, 3620760, 3620797, 3621316, 3621353, 3621390, 3621427, 3621532, 3621569, 3622838, 3623394, 3623431, 3623468, 3623610, 3625472, 3625509, 3626244, 3626281, 3627550, 3628106, 3628143, 3628180, 3628322, 3630184, 3630221, 3632262, 3633034, 3634896, 3634933, 3636974, 3641686, 3762120, 3762157, 3764198, 3768910, 3891008, 3891045, 3893086, 3897798, 3908640, 3908677, 3908714, 3908751, 3908788, 3910718, 3910755, 3910792, 3910829, 3912796, 3912833, 3912870, 3914874, 3914911, 3915430, 3915467, 3915504, 3915541, 3916952, 3917508, 3917545, 3917582, 3919586, 3919623, 3921664, 3922220, 3922257, 3922294, 3924082, 3924119, 3924156, 3924193, 3924298, 3924335, 3926160, 3926197, 3926234, 3926376, 3928238, 3928275, 3929010, 3929047, 3930316, 3930872, 3930909, 3930946, 3931088, 3932950, 3932987, 3935028, 3935800, 3937662, 3937699, 3939740, 3944452, 4071676, 4200564, 4218196, 4218233, 4218270, 4218307, 4220274, 4220311, 4220348, 4222352, 4222389, 4224430, 4224986, 4225023, 4225060, 4227064, 4227101, 4229142, 4231776, 4231813, 4233638, 4233675, 4233712, 4233854, 4235716, 4235753, 4237794, 4238566, 4240428, 4240465, 4242506, 4247218, 4362526, 4362563, 4362600, 4364604, 4364641, 4366682, 4369316, 4369353, 4371394, 4376106, 4527752, 4527789, 4527826, 4529830, 4529867, 4531908, 4534542, 4534579, 4536620, 4541332, 4543194, 4543231, 4545272, 4549984, 4672082, 4672119, 4674160, 4678872, 4689714, 4689751, 4689788, 4689825, 4689862, 4691792, 4691829, 4691866, 4691903, 4693870, 4693907, 4693944, 4695948, 4695985, 4696504, 4696541, 4696578, 4696615, 4698026, 4698582, 4698619, 4698656, 4700660, 4700697, 4702738, 4703294, 4703331, 4703368, 4705156, 4705193, 4705230, 4705267, 4705372, 4705409, 4707234, 4707271, 4707308, 4707450, 4709312, 4709349, 4710084, 4710121, 4711390, 4711946, 4711983, 4712020, 4712162, 4714024, 4714061, 4716102, 4716874, 4718736, 4718773, 4720814, 4725526, 4837308, 4837345, 4839386, 4844098, 4852750, 4981638, 4999270, 4999307, 4999344, 4999381, 5001348, 5001385, 5001422, 5003426, 5003463, 5005504, 5006060, 5006097, 5006134, 5008138, 5008175, 5010216, 5012850, 5012887, 5014712, 5014749, 5014786, 5014928, 5016790, 5016827, 5018868, 5019640, 5021502, 5021539, 5023580, 5028292, 5047786, 5047823, 5047860, 5047897, 5047934, 5049864, 5049901, 5049938, 5049975, 5051942, 5051979, 5052016, 5054020, 5054057, 5054576, 5054613, 5054650, 5054687, 5056098, 5056654, 5056691, 5056728, 5058732, 5058769, 5060810, 5061366, 5061403, 5061440, 5063444, 5063481, 5065522, 5068156, 5068193, 5070234, 5074946, 5146864, 5308826, 5308863, 5308900, 5310904, 5310941, 5312982, 5315616, 5315653, 5317694, 5322406, 5324268, 5324305, 5326346, 5331058, 5357342, 5357379, 5357416, 5357453, 5359420, 5359457, 5359494, 5361498, 5361535, 5363576, 5364132, 5364169, 5364206, 5366210, 5366247, 5368288, 5370922, 5370959, 5373000, 5377712, 5453156, 5453193, 5455234, 5459946, 5618382, 5618419, 5620460, 5625172, 5633824, 5666898, 5666935, 5666972, 5668976, 5669013, 5671054, 5673688, 5673725, 5675766, 5680478, 5762712, 5780344, 5780381, 5780418, 5780455, 5782422, 5782459, 5782496, 5784500, 5784537, 5786578, 5787134, 5787171, 5787208, 5789212, 5789249, 5791290, 5793924, 5793961, 5795786, 5795823, 5795860, 5796002, 5797864, 5797901, 5799942, 5800714, 5802576, 5802613, 5804654, 5809366, 5927938, 5976454, 5976491, 5978532, 5983244, 6036640, 6036677, 6036714, 6036751, 6036788, 6036825, 6038718, 6038755, 6038792, 6038829, 6038866, 6040796, 6040833, 6040870, 6040907, 6042874, 6042911, 6042948, 6043430, 6043467, 6043504, 6043541, 6043578, 6044952, 6044989, 6045508, 6045545, 6045582, 6045619, 6047030, 6047586, 6047623, 6047660, 6049664, 6049701, 6050220, 6050257, 6050294, 6050331, 6051742, 6052298, 6052335, 6052372, 6054376, 6054413, 6056454, 6057010, 6057047, 6057084, 6059088, 6059125, 6061166, 6063800, 6063837, 6065878, 6070590, 6089900, 6089937, 6089974, 6091978, 6092015, 6094056, 6096690, 6096727, 6098768, 6103480, 6105342, 6105379, 6107420, 6112132, 6122974, 6123011, 6123048, 6123085, 6123122, 6125052, 6125089, 6125126, 6125163, 6127130, 6127167, 6127204, 6129208, 6129245, 6129764, 6129801, 6129838, 6129875, 6131286, 6131842, 6131879, 6131916, 6133920, 6133957, 6135998, 6136554, 6136591, 6136628, 6138416, 6138453, 6138490, 6138527, 6138632, 6138669, 6140494, 6140531, 6140568, 6140710, 6142572, 6142609, 6143344, 6143381, 6144650, 6145206, 6145243, 6145280, 6145422, 6147284, 6147321, 6149362, 6150134, 6151996, 6152033, 6154074, 6158786, 6286010, 6346196, 6346233, 6346270, 6346307, 6346344, 6348274, 6348311, 6348348, 6348385, 6350352, 6350389, 6350426, 6352430, 6352467, 6352986, 6353023, 6353060, 6353097, 6354508, 6355064, 6355101, 6355138, 6357142, 6357179, 6359220, 6359776, 6359813, 6359850, 6361854, 6361891, 6363932, 6366566, 6366603, 6368644, 6373356, 6399456, 6399493, 6401534, 6406246, 6414898, 6432530, 6432567, 6432604, 6432641, 6434608, 6434645, 6434682, 6436686, 6436723, 6438764, 6439320, 6439357, 6439394, 6441398, 6441435, 6443476, 6446110, 6446147, 6447972, 6448009, 6448046, 6448188, 6450050, 6450087, 6452128, 6452900, 6454762, 6454799, 6456840, 6461552, 6543786, 6655752, 6655789, 6655826, 6655863, 6657830, 6657867, 6657904, 6659908, 6659945, 6661986, 6662542, 6662579, 6662616, 6664620, 6664657, 6666698, 6669332, 6669369, 6671410, 6676122, 6709012, 6742086, 6742123, 6742160, 6744164, 6744201, 6746242, 6748876, 6748913, 6750954, 6755666, 6757528, 6757565, 6759606, 6764318, 6870974, 6871011, 6871048, 6873052, 6873089, 6875130, 6877764, 6877801, 6879842, 6884554, 6886416, 6886453, 6888494, 6893206, 6965308, 6965345, 6965382, 6967386, 6967423, 6969464, 6972098, 6972135, 6974176, 6978888, 7051642, 7051679, 7053720, 7058432, 7067084, 7127270, 7127307, 7127344, 7127381, 7127418, 7129348, 7129385, 7129422, 7129459, 7131426, 7131463, 7131500, 7133504, 7133541, 7134060, 7134097, 7134134, 7134171, 7135582, 7136138, 7136175, 7136212, 7138216, 7138253, 7140294, 7140850, 7140887, 7140924, 7142928, 7142965, 7145006, 7147640, 7147677, 7149718, 7154430, 7180530, 7180567, 7182608, 7187320, 7195972, 7198162, 7198199, 7198236, 7198273, 7198310, 7200240, 7200277, 7200314, 7200351, 7202318, 7202355, 7202392, 7204396, 7204433, 7204952, 7204989, 7205026, 7205063, 7206474, 7207030, 7207067, 7207104, 7209108, 7209145, 7211186, 7211742, 7211779, 7211816, 7213604, 7213641, 7213678, 7213715, 7213820, 7213857, 7215682, 7215719, 7215756, 7215898, 7217760, 7217797, 7218532, 7218569, 7219838, 7220394, 7220431, 7220468, 7220610, 7222472, 7222509, 7224550, 7225322, 7227184, 7227221, 7229046, 7229083, 7229120, 7229262, 7231124, 7231161, 7233202, 7233974, 7235836, 7235873, 7237914, 7242626, 7274864, 7274901, 7276942, 7281654, 7361198, 7436826, 7436863, 7436900, 7436937, 7438904, 7438941, 7438978, 7440982, 7441019, 7443060, 7443616, 7443653, 7443690, 7445694, 7445731, 7447772, 7450406, 7450443, 7452484, 7457196, 7490086, 7507718, 7507755, 7507792, 7507829, 7509796, 7509833, 7509870, 7511874, 7511911, 7513952, 7514508, 7514545, 7514582, 7516586, 7516623, 7518664, 7521298, 7521335, 7523160, 7523197, 7523234, 7523376, 7525238, 7525275, 7527316, 7528088, 7529950, 7529987, 7532028, 7536740, 7538602, 7538639, 7540680, 7545392, 7571676, 7571713, 7571750, 7571787, 7573754, 7573791, 7573828, 7575832, 7575869, 7577910, 7578466, 7578503, 7578540, 7580544, 7580581, 7582622, 7584420, 7585256, 7585293, 7587334, 7592046, 7746382, 7746419, 7746456, 7748460, 7748497, 7750538, 7753172, 7753209, 7755250, 7759962, 7817274, 7817311, 7817348, 7819352, 7819389, 7821430, 7824064, 7824101, 7826142, 7830854, 7832716, 7832753, 7834794, 7839506, 7848158, 7881232, 7881269, 7881306, 7883310, 7883347, 7885388, 7888022, 7888059, 7890100, 7894812, 7961604, 7961641, 7963682, 7968394, 7977046, 8055938, 8055975, 8058016, 8062728, 8126830, 8126867, 8128908, 8133620, 8142272, 8190788, 8190825, 8192866, 8197578, 8217900, 8217937, 8217974, 8218011, 8219978, 8220015, 8220052, 8222056, 8222093, 8224134, 8224690, 8224727, 8224764, 8226768, 8226805, 8228846, 8231480, 8231517, 8233558, 8238270, 8271160, 8288792, 8288829, 8288866, 8288903, 8290870, 8290907, 8290944, 8292948, 8292985, 8295026, 8295582, 8295619, 8295656, 8297660, 8297697, 8299738, 8302372, 8302409, 8304234, 8304271, 8304308, 8304450, 8306312, 8306349, 8308390, 8309162, 8311024, 8311061, 8313102, 8317814, 8319676, 8319713, 8321754, 8326466, 8365494, 8436386, 8500344, 8527456, 8527493, 8527530, 8529534, 8529571, 8531612, 8534246, 8534283, 8536324, 8541036, 8560530, 8560567, 8560604, 8560641, 8560678, 8562608, 8562645, 8562682, 8562719, 8564686, 8564723, 8564760, 8566764, 8566801, 8567320, 8567357, 8567394, 8567431, 8568842, 8569398, 8569435, 8569472, 8571476, 8571513, 8573554, 8574110, 8574147, 8574184, 8576188, 8576225, 8578266, 8580900, 8580937, 8582978, 8587690, 8598348, 8598385, 8598422, 8600426, 8600463, 8602504, 8605138, 8605175, 8607216, 8611928, 8613790, 8613827, 8615868, 8620580, 8629232, 8646864, 8646901, 8646938, 8646975, 8648942, 8648979, 8649016, 8651020, 8651057, 8653098, 8653654, 8653691, 8653728, 8655732, 8655769, 8657810, 8660444, 8660481, 8662306, 8662343, 8662380, 8662522, 8664384, 8664421, 8666462, 8667234, 8669096, 8669133, 8671174, 8675886, 8837012, 8837049, 8839090, 8843802, 8870086, 8870123, 8870160, 8870197, 8872164, 8872201, 8872238, 8874242, 8874279, 8876320, 8876876, 8876913, 8876950, 8878954, 8878991, 8881032, 8883666, 8883703, 8885744, 8890456, 8907904, 8907941, 8909982, 8914694, 8923346, 8956420, 8956457, 8956494, 8958498, 8958535, 8960576, 8963210, 8963247, 8965288, 8970000, 8971862, 8971899, 8973940, 8978652, 9052234, 9146568, 9179642, 9179679, 9179716, 9181720, 9181757, 9183798, 9186432, 9186469, 9188510, 9193222, 9217460, 9265976, 9266013, 9268054, 9272766, 9281418, 9308530, 9308567, 9308604, 9310608, 9310645, 9312686, 9315320, 9315357, 9317398, 9322110, 9379422, 9379459, 9379496, 9381500, 9381537, 9383578, 9386212, 9386249, 9388290, 9393002, 9394864, 9394901, 9396942, 9401654, 9410306, 9489198, 9489235, 9491276, 9495988, 9575532, 9618086, 9618123, 9620164, 9624876, 9635718, 9635755, 9635792, 9635829, 9635866, 9637796, 9637833, 9637870, 9637907, 9639874, 9639911, 9639948, 9641952, 9641989, 9642508, 9642545, 9642582, 9642619, 9644030, 9644586, 9644623, 9644660, 9646664, 9646701, 9648742, 9649298, 9649335, 9649372, 9651160, 9651197, 9651234, 9651271, 9651376, 9651413, 9653238, 9653275, 9653312, 9653454, 9655316, 9655353, 9656088, 9656125, 9657394, 9657950, 9657987, 9658024, 9658166, 9660028, 9660065, 9662106, 9662878, 9664740, 9664777, 9666818, 9671530, 9688978, 9689015, 9691056, 9695768, 9704420, 9722052, 9722089, 9722126, 9722163, 9724130, 9724167, 9724204, 9726208, 9726245, 9728286, 9728842, 9728879, 9728916, 9730920, 9730957, 9732998, 9735632, 9735669, 9737494, 9737531, 9737568, 9737710, 9739572, 9739609, 9741650, 9742422, 9744284, 9744321, 9746362, 9751074, 9752936, 9752973, 9755014, 9759726, 9798754, 9927642, 9945274, 9945311, 9945348, 9945385, 9947352, 9947389, 9947426, 9949430, 9949467, 9951508, 9952064, 9952101, 9952138, 9954142, 9954179, 9956220, 9958854, 9958891, 9960716, 9960753, 9960790, 9960932, 9962794, 9962831, 9964872, 9965644, 9967506, 9967543, 9969584, 9974296, 9998534, 10031608, 10031645, 10031682, 10033686, 10033723, 10035764, 10038398, 10038435, 10040476, 10045188, 10047050, 10047087, 10049128, 10053840, 10062492, 10095566, 10095603, 10095640, 10097644, 10097681, 10099722, 10102356, 10102393, 10104434, 10109146, 10254830, 10254867, 10254904, 10256908, 10256945, 10258986, 10261620, 10261657, 10263698, 10268410, 10270272, 10270309, 10272350, 10277062, 10341164, 10341201, 10343242, 10347954, 10356606, 10399160, 10399197, 10401238, 10405122, 10405159, 10405950, 10407200, 10411912, 10470052, 10470089, 10472130, 10476842, 10485494, 10564386, 10564423, 10566464, 10571176, 10579828, 10650720, 10708716, 10714678, 10726348, 10726385, 10726422, 10726459, 10728426, 10728463, 10728500, 10730504, 10730541, 10732582, 10733138, 10733175, 10733212, 10735216, 10735253, 10737294, 10739928, 10739965, 10741790, 10741827, 10741864, 10742006, 10743868, 10743905, 10745946, 10746718, 10748580, 10748617, 10750658, 10755370, 10779608, 10797240, 10797277, 10797314, 10797351, 10799318, 10799355, 10799392, 10801396, 10801433, 10803474, 10804030, 10804067, 10804104, 10806108, 10806145, 10808186, 10810820, 10810857, 10812682, 10812719, 10812756, 10812898, 10814760, 10814797, 10816838, 10817610, 10819472, 10819509, 10821550, 10826262, 10828124, 10828161, 10830202, 10834914, 10843566, 10873942, 11035904, 11035941, 11035978, 11037982, 11038019, 11040060, 11042694, 11042731, 11044772, 11049484, 11051346, 11051383, 11053424, 11058136, 11084420, 11084457, 11084494, 11084531, 11086498, 11086535, 11086572, 11088576, 11088613, 11090654, 11091210, 11091247, 11091284, 11093288, 11093325, 11095366, 11098000, 11098037, 11100078, 11104790, 11106796, 11106833, 11106870, 11108874, 11108911, 11110952, 11113586, 11113623, 11115664, 11120376, 11122238, 11122275, 11124316, 11129028, 11137680, 11170754, 11170791, 11170828, 11172832, 11172869, 11174910, 11177544, 11177581, 11179622, 11184334, 11186196, 11186233, 11188274, 11192986, 11345460, 11345497, 11347538, 11352250, 11360902, 11393976, 11394013, 11394050, 11396054, 11396091, 11398132, 11400766, 11400803, 11402844, 11407556, 11416352, 11416389, 11418430, 11423142, 11431794, 11480310, 11480347, 11482388, 11487100, 11489790, 11495752, 11560682, 11655016, 11703532, 11703569, 11705610, 11710322, 11725908, 11789866, 11816978, 11817015, 11817052, 11819056, 11819093, 11821134, 11823768, 11823805, 11825846, 11830558, 11832420, 11832457, 11834498, 11839210, 11887870, 11887907, 11887944, 11889948, 11889985, 11892026, 11894660, 11894697, 11896738, 11901450, 11903312, 11903349, 11905390, 11910102, 11918754, 12013088, 12073274, 12073311, 12073348, 12073385, 12073422, 12075352, 12075389, 12075426, 12075463, 12077430, 12077467, 12077504, 12079508, 12079545, 12080064, 12080101, 12080138, 12080175, 12081586, 12082142, 12082179, 12082216, 12084220, 12084257, 12086298, 12086854, 12086891, 12086928, 12088932, 12088969, 12091010, 12093644, 12093681, 12095722, 12100434, 12126534, 12126571, 12128612, 12133324, 12141976, 12159608, 12159645, 12159682, 12159719, 12161686, 12161723, 12161760, 12163764, 12163801, 12165842, 12166398, 12166435, 12166472, 12168476, 12168513, 12170554, 12173188, 12173225, 12175050, 12175087, 12175124, 12175266, 12177128, 12177165, 12179206, 12179978, 12181840, 12181877, 12183918, 12188630, 12197426, 12197463, 12199504, 12204216, 12212868, 12245942, 12245979, 12246016, 12248020, 12248057, 12250098, 12252732, 12252769, 12254810, 12259522, 12261384, 12261421, 12263462, 12268174, 12276826, 12382830, 12382867, 12382904, 12382941, 12384908, 12384945, 12384982, 12386986, 12387023, 12389064, 12389620, 12389657, 12389694, 12391698, 12391735, 12393776, 12396410, 12396447, 12398488, 12403200, 12436090, 12469164, 12469201, 12469238, 12471242, 12471279, 12473320, 12475954, 12475991, 12478032, 12482744, 12484606, 12484643, 12486684, 12491396, 12506982, 12555498, 12555535, 12557576, 12562288, 12570940, 12619456, 12619493, 12621534, 12626246, 12692386, 12692423, 12692460, 12694464, 12694501, 12696542, 12699176, 12699213, 12701254, 12705966, 12778720, 12778757, 12780798, 12785510, 12794162, 12865054, 12907608, 12907645, 12909686, 12914398, 12923050, 12929012, 12978500, 12978537, 12980578, 12985290, 12993942, 13001942, 13001979, 13004020, 13008732, 13088276, 13163904, 13163941, 13163978, 13164015, 13165982, 13166019, 13166056, 13168060, 13168097, 13170138, 13170694, 13170731, 13170768, 13172772, 13172809, 13174850, 13177484, 13177521, 13179562, 13184274, 13217164, 13234796, 13234833, 13234870, 13234907, 13236874, 13236911, 13236948, 13238952, 13238989, 13241030, 13241586, 13241623, 13241660, 13243664, 13243701, 13245742, 13248376, 13248413, 13250238, 13250275, 13250312, 13250454, 13252316, 13252353, 13254394, 13255166, 13257028, 13257065, 13259106, 13263818, 13265680, 13265717, 13267758, 13272470, 13288056, 13311498, 13321130, 13321167, 13321204, 13323208, 13323245, 13325286, 13327920, 13327957, 13329998, 13334710, 13336572, 13336609, 13338650, 13343362, 13352014, 13473460, 13473497, 13473534, 13475538, 13475575, 13477616, 13480250, 13480287, 13482328, 13487040, 13544352, 13544389, 13544426, 13546430, 13546467, 13548508, 13551142, 13551179, 13553220, 13557932, 13559794, 13559831, 13561872, 13566584, 13575236, 13608310, 13608347, 13608384, 13610388, 13610425, 13612466, 13615100, 13615137, 13617178, 13621890, 13630686, 13630723, 13632764, 13637476, 13646128, 13694644, 13694681, 13696722, 13701434, 13710086, 13783016, 13783053, 13785094, 13789806, 13853908, 13853945, 13855986, 13860698, 13869350, 13917866, 13917903, 13919944, 13924656, 13940242, 13998238, 14004200, 14069130, 14092572, 14163464, 14227422, 14254534, 14254571, 14254608, 14256612, 14256649, 14258690, 14261324, 14261361, 14263402, 14268114, 14325426, 14325463, 14325500, 14327504, 14327541, 14329582, 14332216, 14332253, 14334294, 14339006, 14340868, 14340905, 14342946, 14347658, 14356310, 14396318, 14396355, 14396392, 14398396, 14398433, 14400474, 14403108, 14403145, 14405186, 14409898, 14411760, 14411797, 14413838, 14418550, 14427202, 14564090, 14564127, 14566168, 14570880, 14597164, 14597201, 14597238, 14597275, 14599242, 14599279, 14599316, 14601320, 14601357, 14603398, 14603954, 14603991, 14604028, 14606032, 14606069, 14608110, 14610744, 14610781, 14612822, 14617534, 14634982, 14635019, 14637060, 14641772, 14650424, 14683498, 14683535, 14683572, 14685576, 14685613, 14687654, 14690288, 14690325, 14692366, 14697078, 14698940, 14698977, 14701018, 14705730, 14705874, 14705911, 14707952, 14712664, 14721316, 14769832, 14769869, 14771910, 14776622, 14785274, 14873646, 14906720, 14906757, 14906794, 14908798, 14908835, 14910876, 14913510, 14913547, 14915588, 14920300, 14944538, 14993054, 14993091, 14995132, 14999844, 15008496, 15015430, 15079388, 15143346, 15216276, 15216313, 15218354, 15223066, 15302610, 15345164, 15345201, 15347242, 15351954, 15416056, 15416093, 15418134, 15422846, 15431498, 15486948, 15486985, 15489026, 15493738, 15502390, 15525832, 15654720, 15672352, 15672389, 15672426, 15672463, 15674430, 15674467, 15674504, 15676508, 15676545, 15678586, 15679142, 15679179, 15679216, 15681220, 15681257, 15683298, 15685932, 15685969, 15687794, 15687831, 15687868, 15688010, 15689872, 15689909, 15691950, 15692722, 15694584, 15694621, 15696662, 15701374, 15725612, 15758686, 15758723, 15758760, 15760764, 15760801, 15762842, 15765476, 15765513, 15767554, 15772266, 15774128, 15774165, 15776206, 15780918, 15789570, 15796504, 15845020, 15845057, 15847098, 15851810, 15860462, 15981908, 15981945, 15981982, 15983986, 15984023, 15986064, 15988698, 15988735, 15990776, 15995488, 15997350, 15997387, 15999428, 16004140, 16068242, 16068279, 16070320, 16075032, 16083684, 16132200, 16132237, 16134278, 16138990, 16154576, 16218534, 16291464, 16291501, 16293542, 16298254, 16306906, 16377798, 16435794, 16441756, 16506686, 16577578, 16601020, 16762982, 16763019, 16763056, 16765060, 16765097, 16767138, 16767492, 16767529, 16767566, 16767603, 16767640, 16767677, 16769570, 16769607, 16769644, 16769681, 16769718, 16769772, 16769809, 16771648, 16771685, 16771722, 16771759, 16771850, 16773726, 16773763, 16773800, 16774282, 16774319, 16774356, 16774393, 16774430, 16775804, 16775841, 16776360, 16776397, 16776434, 16776471, 16776562, 16777882, 16778424, 16778438, 16778461, 16778475, 16778512, 16780502, 16780516, 16780553, 16781072, 16781109, 16781146, 16781183, 16782594, 16783150, 16783187, 16783224, 16785214, 16785228, 16785265, 16787306, 16787862, 16787899, 16787936, 16789940, 16789977, 16792018, 16794652, 16794689, 16796730, 16801442, 16833874, 16833911, 16833948, 16835952, 16835989, 16838030, 16840664, 16840701, 16842742, 16847454, 16849316, 16849353, 16851394, 16856106, 16864758, 16920208, 16920245, 16922286, 16926998, 16935650, 17072538, 17072575, 17074616, 17077048, 17077085, 17077122, 17077159, 17077196, 17079126, 17079163, 17079200, 17079237, 17079328, 17081204, 17081241, 17081278, 17083282, 17083319, 17083838, 17083875, 17083912, 17083949, 17085360, 17085916, 17085953, 17085990, 17087980, 17087994, 17088031, 17090072, 17090628, 17090665, 17090702, 17092706, 17092743, 17094784, 17097418, 17097455, 17099496, 17104208, 17121054, 17121091, 17121128, 17123132, 17123169, 17125210, 17127844, 17127881, 17129922, 17134634, 17143430, 17143467, 17145508, 17150220, 17158872, 17207388, 17207425, 17209466, 17214178, 17222830, 17229764, 17293722, 17382094, 17386604, 17386641, 17386678, 17386715, 17388682, 17388719, 17388756, 17390760, 17390797, 17392838, 17393394, 17393431, 17393468, 17395472, 17395509, 17397550, 17400184, 17400221, 17402262, 17406974, 17430610, 17430647, 17432688, 17437400, 17452986, 17516944, 17696160, 17696197, 17696234, 17698238, 17698275, 17700316, 17702950, 17702987, 17705028, 17709740, 17740166, 17853612, 17853649, 17855690, 17858122, 17858159, 17858196, 17858233, 17858270, 17860200, 17860237, 17860274, 17860311, 17860402, 17862278, 17862315, 17862352, 17864356, 17864393, 17864912, 17864949, 17864986, 17865023, 17866434, 17866990, 17867027, 17867064, 17869054, 17869068, 17869105, 17871146, 17871702, 17871739, 17871776, 17873780, 17873817, 17875858, 17878492, 17878529, 17880570, 17885282, 17924504, 17924541, 17926582, 17931294, 17939946, 17995396, 17995433, 17997474, 18002186, 18005716, 18005753, 18007794, 18010838, 18012506, 18109908, 18109945, 18109982, 18110019, 18111986, 18112023, 18112060, 18114064, 18114101, 18116142, 18116698, 18116735, 18116772, 18118776, 18118813, 18120854, 18123488, 18123525, 18125566, 18130278, 18163168, 18167678, 18167715, 18167752, 18167789, 18169756, 18169793, 18169830, 18171834, 18171871, 18173912, 18174468, 18174505, 18174542, 18176546, 18176583, 18178624, 18181258, 18181295, 18183336, 18188048, 18196242, 18196279, 18196316, 18198320, 18198357, 18200398, 18203032, 18203069, 18205110, 18209822, 18211684, 18211721, 18213762, 18218474, 18234060, 18282576, 18282613, 18284654, 18289366, 18298018, 18304952, 18315272, 18368910, 18419464, 18419501, 18419538, 18421542, 18421579, 18423620, 18426254, 18426291, 18428332, 18433044, 18477234, 18477271, 18477308, 18479312, 18479349, 18481390, 18484024, 18484061, 18486102, 18490814, 18505798, 18505835, 18507876, 18512588, 18521240, 18592132, 18656090, 18729020, 18729057, 18731098, 18735810, 18786790, 18786827, 18788868, 18793580, 18815354, 18944242, 18948752, 18948789, 18948826, 18948863, 18950830, 18950867, 18950904, 18952908, 18952945, 18954986, 18955542, 18955579, 18955616, 18957620, 18957657, 18959698, 18962332, 18962369, 18964410, 18969122, 19015134, 19038576, 19086026, 19096346, 19200538, 19200575, 19200612, 19202616, 19202653, 19204694, 19207328, 19207365, 19209406, 19214118, 19258308, 19258345, 19258382, 19260386, 19260423, 19262464, 19265098, 19265135, 19267176, 19271430, 19271467, 19271504, 19271888, 19273508, 19273545, 19275586, 19278220, 19278257, 19280298, 19285010, 19286872, 19286909, 19288950, 19291382, 19291419, 19291456, 19291493, 19291530, 19293460, 19293497, 19293534, 19293571, 19293662, 19295538, 19295575, 19295612, 19297616, 19297653, 19298172, 19298209, 19298246, 19298283, 19299694, 19300250, 19300287, 19300324, 19302314, 19302328, 19302365, 19304406, 19304962, 19304999, 19305036, 19307040, 19307077, 19309118, 19311752, 19311789, 19313830, 19318542, 19357764, 19357801, 19359842, 19364554, 19373206, 19444098, 19510094, 19510131, 19512172, 19516884, 19567864, 19567901, 19569942, 19574654, 19580986, 19581023, 19583064, 19587776, 19596428, 19600938, 19600975, 19601012, 19601049, 19603016, 19603053, 19603090, 19605094, 19605131, 19607172, 19607728, 19607765, 19607802, 19609806, 19609843, 19611884, 19614518, 19614555, 19616596, 19621308, 19644944, 19644981, 19647022, 19651734, 19667320, 19731278, 19819650, 19877420, 19890542, 19910494, 19910531, 19910568, 19912572, 19912609, 19914650, 19917284, 19917321, 19919362, 19924074, 19954500, 20039382, 20039419, 20039456, 20041460, 20041497, 20043538, 20046172, 20046209, 20048250, 20052962, 20220050, 20220087, 20222128, 20226840, 20291168, 20291205, 20293246, 20297958, 20348938, 20348975, 20351016, 20355728, 20362060, 20362097, 20364138, 20366570, 20366607, 20366644, 20366681, 20366718, 20368648, 20368685, 20368722, 20368759, 20368850, 20370726, 20370763, 20370800, 20372804, 20372841, 20373360, 20373397, 20373434, 20373471, 20374882, 20375438, 20375475, 20375512, 20377502, 20377516, 20377553, 20379594, 20380150, 20380187, 20380224, 20382012, 20382049, 20382086, 20382123, 20382228, 20382265, 20384090, 20384127, 20384164, 20384306, 20386168, 20386205, 20386940, 20386977, 20388246, 20388802, 20388839, 20388876, 20389018, 20390880, 20390917, 20392958, 20393730, 20395592, 20395629, 20397670, 20402382, 20432952, 20432989, 20435030, 20439742, 20448394, 20519286, 20529606, 20600724, 20633798, 20633835, 20633872, 20635876, 20635913, 20637954, 20640588, 20640625, 20642666, 20647378, 20658494, 20671616, 20676126, 20676163, 20676200, 20676237, 20678204, 20678241, 20678278, 20680282, 20680319, 20682360, 20682916, 20682953, 20682990, 20684994, 20685031, 20687072, 20689706, 20689743, 20691568, 20691605, 20691642, 20691784, 20693646, 20693683, 20695724, 20696496, 20698358, 20698395, 20700436, 20705148, 20720132, 20720169, 20722210, 20726922, 20735574, 20742508, 20806466, 20943354, 20943391, 20945432, 20950144, 20985682, 20985719, 20985756, 20987760, 20987797, 20989838, 20992472, 20992509, 20994550, 20999262, 21001124, 21001161, 21003202, 21007914, 21029688, 21130012, 21130049, 21132090, 21136802, 21252910, 21295238, 21295275, 21297316, 21302028, 21310680, 21381798, 21439568, 21452690, 21457200, 21457237, 21457274, 21457311, 21459278, 21459315, 21459352, 21461356, 21461393, 21463434, 21463990, 21464027, 21464064, 21466068, 21466105, 21468146, 21470780, 21470817, 21472642, 21472679, 21472716, 21472858, 21474720, 21474757, 21476798, 21477570, 21479432, 21479469, 21481510, 21486222, 21523582, 21594474, 21604794, 21708986, 21709023, 21709060, 21711064, 21711101, 21713142, 21715776, 21715813, 21717854, 21722566, 21724428, 21724465, 21726506, 21731218, 21766756, 21766793, 21766830, 21768834, 21768871, 21770912, 21773546, 21773583, 21775624, 21780336, 21782198, 21782235, 21784276, 21788988, 21795320, 21795357, 21797398, 21802110, 21810762, 21815272, 21815309, 21815346, 21815383, 21817350, 21817387, 21817424, 21819428, 21819465, 21821506, 21822062, 21822099, 21822136, 21824140, 21824177, 21826218, 21828852, 21828889, 21830930, 21835642, 21881654, 22018542, 22018579, 22020620, 22025332, 22033984, 22076312, 22076349, 22078390, 22083102, 22091754, 22104876, 22124828, 22124865, 22124902, 22126906, 22126943, 22128984, 22131618, 22131655, 22133696, 22138408, 22168834, 22220642, 22328098, 22385868, 22434384, 22434421, 22436462, 22441174, 22547830, 22547867, 22547904, 22549908, 22549945, 22551986, 22554620, 22554657, 22556698, 22561410, 22563272, 22563309, 22565350, 22570062, 22743940, 22799616, 22799653, 22801694, 22804126, 22804163, 22804200, 22804237, 22804274, 22806204, 22806241, 22806278, 22806315, 22806406, 22808282, 22808319, 22808356, 22810360, 22810397, 22810916, 22810953, 22810990, 22811027, 22812438, 22812994, 22813031, 22813068, 22815058, 22815072, 22815109, 22817150, 22817706, 22817743, 22817780, 22819784, 22819821, 22821862, 22824496, 22824533, 22826574, 22831286, 22857386, 22857423, 22859464, 22864176, 22870508, 22870545, 22872586, 22872828, 22877298, 22885950, 22890460, 22890497, 22890534, 22890571, 22892538, 22892575, 22892612, 22894616, 22894653, 22896694, 22897250, 22897287, 22897324, 22899328, 22899365, 22901406, 22904040, 22904077, 22905902, 22905939, 22905976, 22906118, 22907980, 22908017, 22910058, 22910830, 22912692, 22912729, 22914770, 22919482, 22956842, 23109172, 23113682, 23113719, 23113756, 23113793, 23115760, 23115797, 23115834, 23117838, 23117875, 23119916, 23120472, 23120509, 23120546, 23122550, 23122587, 23124628, 23127262, 23127299, 23129340, 23134052, 23157688, 23157725, 23159766, 23164478, 23166942, 23180064, 23200016, 23200053, 23200090, 23202094, 23202131, 23204172, 23206806, 23206843, 23208884, 23213596, 23215458, 23215495, 23217536, 23222248, 23244022, 23423238, 23423275, 23423312, 23425316, 23425353, 23427394, 23430028, 23430065, 23432106, 23436818, 23467244, 23509572, 23509609, 23511650, 23516362, 23525014, 23638460, 23638497, 23640538, 23645250, 23653902, 23732794, 23732831, 23734872, 23739584, 23819128, 23890246, 23894756, 23894793, 23894830, 23894867, 23896834, 23896871, 23896908, 23898912, 23898949, 23900990, 23901546, 23901583, 23901620, 23903624, 23903661, 23905702, 23908336, 23908373, 23910414, 23915126, 23948016, 23961138, 23965648, 23965685, 23965722, 23965759, 23967726, 23967763, 23967800, 23969804, 23969841, 23971882, 23972438, 23972475, 23972512, 23974516, 23974553, 23976594, 23979228, 23979265, 23981090, 23981127, 23981164, 23981306, 23983168, 23983205, 23985246, 23986018, 23987880, 23987917, 23989958, 23994670, 23996532, 23996569, 23998610, 24003322, 24032030, 24042350, 24146542, 24146579, 24146616, 24148620, 24148657, 24150698, 24153332, 24153369, 24155410, 24160122, 24204312, 24204349, 24204386, 24206390, 24206427, 24208468, 24211102, 24211139, 24213180, 24217892, 24232876, 24232913, 24234954, 24239666, 24248318, 24275204, 24275241, 24275278, 24277282, 24277319, 24279360, 24281994, 24282031, 24284072, 24288784, 24290646, 24290683, 24292724, 24297436, 24306088, 24319210, 24339162, 24339199, 24339236, 24341240, 24341277, 24343318, 24345952, 24345989, 24348030, 24352742, 24456098, 24456135, 24458176, 24462888, 24513868, 24513905, 24515946, 24520658, 24542432, 24584760, 24584797, 24586838, 24591550, 24600202, 24648718, 24648755, 24650796, 24655508, 24729090, 24765654, 24823424, 24894316, 24958274, 24985386, 24985423, 24985460, 24987464, 24987501, 24989542, 24992176, 24992213, 24994254, 24998966, 25056278, 25056315, 25056352, 25058356, 25058393, 25060434, 25063068, 25063105, 25065146, 25069858, 25071720, 25071757, 25073798, 25078510, 25087162, 25237172, 25237209, 25239250, 25243962, 25294942, 25294979, 25297020, 25301732, 25308064, 25308101, 25310142, 25314854, 25323506, 25328016, 25328053, 25328090, 25328127, 25330094, 25330131, 25330168, 25332172, 25332209, 25334250, 25334806, 25334843, 25334880, 25336884, 25336921, 25338962, 25341596, 25341633, 25343674, 25348386, 25365834, 25365871, 25367912, 25372624, 25381276, 25394398, 25414350, 25414387, 25414424, 25416428, 25416465, 25418506, 25421140, 25421177, 25423218, 25427930, 25429792, 25429829, 25431870, 25436582, 25546728, 25604498, 25617620, 25637572, 25637609, 25637646, 25639650, 25639687, 25641728, 25644362, 25644399, 25646440, 25651152, 25675390, 25681578, 25723906, 25723943, 25725984, 25730696, 25739348, 25947128, 25947165, 25949206, 25953918, 26033462, 26076016, 26076053, 26078094, 26082806, 26146908, 26146945, 26148986, 26153698, 26162350, 26256684, 26327802, 26385572, 26398694, 26403204, 26403241, 26403278, 26403315, 26405282, 26405319, 26405356, 26407360, 26407397, 26409438, 26409994, 26410031, 26410068, 26412072, 26412109, 26414150, 26416784, 26416821, 26418646, 26418683, 26418720, 26418862, 26420724, 26420761, 26422802, 26423574, 26425436, 26425473, 26427514, 26432226, 26456464, 26469586, 26489538, 26489575, 26489612, 26491616, 26491653, 26493694, 26496328, 26496365, 26498406, 26503118, 26504980, 26505017, 26507058, 26511770, 26520422, 26670432, 26670469, 26672510, 26677222, 26712760, 26712797, 26712834, 26714838, 26714875, 26716916, 26719550, 26719587, 26721628, 26726340, 26728202, 26728239, 26730280, 26734992, 26756766, 26799094, 26799131, 26801172, 26805884, 26814536, 26863052, 26863089, 26865130, 26869842, 26979988, 27022316, 27022353, 27024394, 27029106, 27037758, 27108650, 27166646, 27172608, 27237538, 27331872, 27493834, 27493871, 27493908, 27495912, 27495949, 27497990, 27500624, 27500661, 27502702, 27507414, 27509276, 27509313, 27511354, 27516066, 27564726, 27564763, 27564800, 27566804, 27566841, 27568882, 27571516, 27571553, 27573594, 27578306, 27580168, 27580205, 27582246, 27586958, 27595610, 27745620, 27745657, 27747698, 27752410, 27761062, 27803390, 27803427, 27805468, 27810180, 27818832, 27831954, 27851906, 27851943, 27851980, 27853984, 27854021, 27856062, 27858696, 27858733, 27860774, 27865486, 27874282, 27874319, 27876360, 27881072, 27889724, 27938240, 27938277, 27940318, 27945030, 27953682, 28055176, 28112946, 28161462, 28161499, 28163540, 28168252, 28183838, 28247796, 28471018, 28584464, 28584501, 28586542, 28591254, 28599906, 28655356, 28655393, 28657434, 28662146, 28670798, 28836250, 28840760, 28840797, 28840834, 28840871, 28842838, 28842875, 28842912, 28844916, 28844953, 28846994, 28847550, 28847587, 28847624, 28849628, 28849665, 28851706, 28854340, 28854377, 28856418, 28861130, 28894020, 28907142, 28927094, 28927131, 28927168, 28929172, 28929209, 28931250, 28933884, 28933921, 28935962, 28940674, 28942536, 28942573, 28944614, 28949326, 28964912, 29013428, 29013465, 29015506, 29020218, 29028870, 29150316, 29150353, 29150390, 29152394, 29152431, 29154472, 29157106, 29157143, 29159184, 29163896, 29194322, 29236650, 29236687, 29238728, 29243440, 29252092, 29322984, 29386942, 29459872, 29459909, 29461950, 29466662, 29546206, 29675094, 29745986, 29769428, 29931390, 29931427, 29931464, 29933468, 29933505, 29935546, 29938180, 29938217, 29940258, 29944970, 30002282, 30002319, 30002356, 30004360, 30004397, 30006438, 30009072, 30009109, 30011150, 30015862, 30017724, 30017761, 30019802, 30024514, 30033166, 30088616, 30088653, 30090694, 30095406, 30104058, 30183176, 30183213, 30185254, 30189966, 30240946, 30240983, 30243024, 30247736, 30269510, 30311838, 30311875, 30313916, 30318628, 30327280, 30375796, 30375833, 30377874, 30382586, 30398172, 30462130, 30492732, 30550502, 30621394, 30685352, 31022020, 31022057, 31024098, 31028810, 31092912, 31092949, 31094990, 31099702, 31108354, 31163804, 31163841, 31165882, 31170594, 31179246, 31273806, 31331576, 31344698, 31364650, 31364687, 31364724, 31366728, 31366765, 31368806, 31371440, 31371477, 31373518, 31378230, 31402468, 31450984, 31451021, 31453062, 31457774, 31466426, 31473360, 31537318, 31674206, 31674243, 31676284, 31680996, 31760540, 31983762, 32112650, 32183542, 32254434, 32439838, 32439875, 32439912, 32441916, 32441953, 32443994, 32446628, 32446665, 32448706, 32453418, 32455280, 32455317, 32457358, 32462070, 32526172, 32526209, 32528250, 32532962, 32541614, 32612506, 32707066, 32749394, 32749431, 32751472, 32756184, 32764836, 32835728, 32899686, 33058950, 33530468, 33530505, 33532546, 33534978, 33535015, 33535052, 33535089, 33535126, 33537056, 33537093, 33537130, 33537167, 33537258, 33539134, 33539171, 33539208, 33541212, 33541249, 33541768, 33541805, 33541842, 33541879, 33543290, 33543846, 33543883, 33543920, 33545910, 33545924, 33545961, 33548002, 33548558, 33548595, 33548632, 33550636, 33550673, 33552714, 33555348, 33555385, 33557426, 33562138, 33601360, 33601397, 33603438, 33608150, 33616802, 33687694, 33782254, 33840024, 33844534, 33844571, 33844608, 33844645, 33846612, 33846649, 33846686, 33848690, 33848727, 33850768, 33851324, 33851361, 33851398, 33853402, 33853439, 33855480, 33858114, 33858151, 33860192, 33864904, 33888540, 33888577, 33890618, 33895330, 33910916, 33974874, 34154090, 34154127, 34154164, 34156168, 34156205, 34158246, 34160880, 34160917, 34162958, 34167670, 34198096, 34463646, 34463683, 34465724, 34470436, 34621098, 34625608, 34625645, 34625682, 34625719, 34627686, 34627723, 34627760, 34629764, 34629801, 34631842, 34632398, 34632435, 34632472, 34634476, 34634513, 34636554, 34639188, 34639225, 34641266, 34645978, 34691990, 34762882, 34773202, 34877394, 34877431, 34877468, 34879472, 34879509, 34881550, 34884184, 34884221, 34886262, 34890974, 34935164, 34935201, 34935238, 34937242, 34937279, 34939320, 34941954, 34941991, 34944032, 34948744, 34963728, 34963765, 34965806, 34970518, 34979170, 35050062, 35186950, 35186987, 35189028, 35193740, 35244720, 35244757, 35246798, 35251510, 35273284, 35496506, 35554276, 35716238, 35716275, 35716312, 35718316, 35718353, 35720394, 35723028, 35723065, 35725106, 35729818, 35968024, 35968061, 35970102, 35974814, 36025794, 36025831, 36027872, 36032584, 36038916, 36038953, 36040994, 36045706, 36054358, 36058868, 36058905, 36058942, 36058979, 36060946, 36060983, 36061020, 36063024, 36063061, 36065102, 36065658, 36065695, 36065732, 36067736, 36067773, 36069814, 36072448, 36072485, 36074526, 36079238, 36125250, 36219810, 36277580, 36335350, 36348472, 36368424, 36368461, 36368498, 36370502, 36370539, 36372580, 36375214, 36375251, 36377292, 36382004, 36412430, 36677980, 36678017, 36680058, 36684770, 36806868, 36806905, 36808946, 36813658, 36987536, 37058654, 37116424, 37129546, 37134056, 37134093, 37134130, 37134167, 37136134, 37136171, 37136208, 37138212, 37138249, 37140290, 37140846, 37140883, 37140920, 37142924, 37142961, 37145002, 37147636, 37147673, 37149498, 37149535, 37149572, 37149714, 37151576, 37151613, 37153654, 37154426, 37156288, 37156325, 37158366, 37163078, 37200438, 37401284, 37401321, 37403362, 37408074, 37443612, 37443649, 37443686, 37445690, 37445727, 37447768, 37450402, 37450439, 37452480, 37457192, 37459054, 37459091, 37461132, 37465844, 37487618, 37710840, 37753168, 37753205, 37755246, 37759958, 37768610, 37897498, 38062724, 38224686, 38224723, 38224760, 38226764, 38226801, 38228842, 38231476, 38231513, 38233554, 38238266, 38240128, 38240165, 38242206, 38246918, 38476472, 38476509, 38478550, 38483262, 38491914, 38534242, 38534279, 38536320, 38541032, 38549684, 38562806, 38582758, 38582795, 38582832, 38584836, 38584873, 38586914, 38589548, 38589585, 38591626, 38596338, 38786028, 38843798, 38892314, 38892351, 38894392, 38899104, 39201870, 39315316, 39315353, 39317394, 39322106, 39330758, 39567102, 39571612, 39571649, 39571686, 39571723, 39573690, 39573727, 39573764, 39575768, 39575805, 39577846, 39578402, 39578439, 39578476, 39580480, 39580517, 39582558, 39585192, 39585229, 39587270, 39591982, 39624872, 39637994, 39657946, 39657983, 39658020, 39660024, 39660061, 39662102, 39664736, 39664773, 39666814, 39671526, 39673388, 39673425, 39675466, 39680178, 39881168, 39881205, 39881242, 39883246, 39883283, 39885324, 39887958, 39887995, 39890036, 39894748, 39925174, 39967502, 39967539, 39969580, 39974292, 39982944, 40190724, 40190761, 40192802, 40197514, 40277058, 40405946, 40500280, 40662242, 40662279, 40662316, 40664320, 40664357, 40666398, 40669032, 40669069, 40671110, 40675822, 40733134, 40733171, 40733208, 40735212, 40735249, 40737290, 40739924, 40739961, 40742002, 40746714, 40748576, 40748613, 40750654, 40755366, 40764018, 40914028, 40914065, 40916106, 40920818, 40971798, 40971835, 40973876, 40978588, 41000362, 41042690, 41042727, 41044768, 41049480, 41058132, 41106648, 41106685, 41108726, 41113438, 41223584, 41281354, 41352246, 41416204, 41752872, 41752909, 41754950, 41759662, 41823764, 41823801, 41825842, 41830554, 41839206, 42004658, 42062428, 42075550, 42095502, 42095539, 42095576, 42097580, 42097617, 42099658, 42102292, 42102329, 42104370, 42109082, 42133320, 42181836, 42181873, 42183914, 42188626, 42197278, 42405058, 42405095, 42407136, 42411848, 42491392, 42714614, 42843502, 42914394, 43170690, 43170727, 43170764, 43172768, 43172805, 43174846, 43177480, 43177517, 43179558, 43184270, 43186132, 43186169, 43188210, 43192922, 43257024, 43257061, 43259102, 43263814, 43272466, 43437918, 43480246, 43480283, 43482324, 43487036, 43495688, 43566580, 43630538, 43789802, 44261320, 44261357, 44263398, 44268110, 44276762, 44332212, 44332249, 44334290, 44339002, 44347654, 44513106, 44570876, 44619392, 44619429, 44621470, 44626182, 44641768, 44705726, 44928948, 45351950, 45422842, 45608246, 45608283, 45608320, 45610324, 45610361, 45612402, 45615036, 45615073, 45617114, 45621826, 45694580, 45694617, 45696658, 45701370, 45710022, 45780914, 45917802, 45917839, 45919880, 45924592, 46004136, 46227358, 46698876, 46698913, 46700954, 46705666, 46769768, 46769805, 46771846, 46776558, 46785210, 46856102, 46950662, 47008432, 47079324, 47143282, 47789506, 47860398, 47931290, 48132136, 48132173, 48134214, 48138926, 48218470, 48441692, 49207324, 49207361, 49209402, 49214114, 49222766, 49293658, 49516880, 50297954, 50302464, 50302501, 50302538, 50302575, 50304542, 50304579, 50304616, 50306620, 50306657, 50308698, 50309254, 50309291, 50309328, 50311332, 50311369, 50313410, 50316044, 50316081, 50318122, 50322834, 50368846, 50612020, 50612057, 50612094, 50614098, 50614135, 50616176, 50618810, 50618847, 50620888, 50625600, 50656026, 50921576, 50921613, 50923654, 50928366, 51231132, 51393094, 51393131, 51393168, 51395172, 51395209, 51397250, 51399884, 51399921, 51401962, 51406674, 51644880, 51644917, 51646958, 51651670, 51702650, 51702687, 51704728, 51709440, 51731214, 51954436, 52012206, 52483724, 52483761, 52485802, 52490514, 52735510, 52793280, 52806402, 52826354, 52826391, 52826428, 52828432, 52828469, 52830510, 52833144, 52833181, 52835222, 52839934, 53135910, 53135947, 53137988, 53142700, 53445466, 53574354, 53901542, 53901579, 53901616, 53903620, 53903657, 53905698, 53908332, 53908369, 53910410, 53915122, 53916984, 53917021, 53919062, 53923774, 54168770, 54211098, 54211135, 54213176, 54217888, 54226540, 54520654, 54992172, 54992209, 54994250, 54998962, 55007614, 55243958, 55301728, 55350244, 55350281, 55352322, 55357034, 55659800, 56082802, 56339098, 56339135, 56339172, 56341176, 56341213, 56343254, 56345888, 56345925, 56347966, 56352678, 56425432, 56425469, 56427510, 56432222, 56440874, 56648654, 56648691, 56650732, 56655444, 56734988, 56958210, 57429728, 57429765, 57431806, 57436518, 57500620, 57500657, 57502698, 57507410, 57516062, 57681514, 57739284, 57810176, 57874134, 58520358, 58591250, 58862988, 58863025, 58865066, 58869778, 58949322, 59172544, 59938176, 59938213, 59940254, 59944966, 59953618, 60024510, 60247732, 61028806, 61099698, 61386878, 62375732, 62375769, 62377810, 62382522, 62462066, 62685288, 63466362, 63537254, 64899622, 65974810, 67069950, 67069987, 67070024, 67072028, 67072065, 67074106, 67076740, 67076777, 67078818, 67083530, 67379506, 67379543, 67381584, 67386296, 67689062, 68160580, 68160617, 68162658, 68167370, 68412366, 68470136, 69251210, 69593840, 69593877, 69595918, 69600630, 69903396, 70669028, 70669065, 70671106, 70675818, 70684470, 70978584, 71759658, 72117730, 73106584, 73106621, 73108662, 73113374, 73192918, 73416140, 74197214, 74268106, 75630474, 76705662, 79143218, 79790198, 79790235, 79790272, 79790309, 79790346, 79790383, 79792276, 79792313, 79792350, 79792387, 79792424, 79794354, 79794391, 79794428, 79794465, 79796432, 79796469, 79796506, 79796988, 79797025, 79797062, 79797099, 79797136, 79798510, 79798547, 79799066, 79799103, 79799140, 79799177, 79800588, 79801144, 79801181, 79801218, 79803222, 79803259, 79803778, 79803815, 79803852, 79803889, 79805300, 79805856, 79805893, 79805930, 79807934, 79807971, 79810012, 79810568, 79810605, 79810642, 79812646, 79812683, 79814724, 79817358, 79817395, 79819436, 79824148, 80099754, 80099791, 80099828, 80099865, 80099902, 80101832, 80101869, 80101906, 80101943, 80103910, 80103947, 80103984, 80105988, 80106025, 80106544, 80106581, 80106618, 80106655, 80108066, 80108622, 80108659, 80108696, 80110700, 80110737, 80112778, 80113334, 80113371, 80113408, 80115412, 80115449, 80117490, 80120124, 80120161, 80122202, 80126914, 80409310, 80409347, 80409384, 80409421, 80411388, 80411425, 80411462, 80413466, 80413503, 80415544, 80416100, 80416137, 80416174, 80418178, 80418215, 80420256, 80422890, 80422927, 80424968, 80429680, 80718866, 80718903, 80718940, 80720944, 80720981, 80723022, 80725656, 80725693, 80727734, 80732446, 80880828, 80880865, 80880902, 80880939, 80880976, 80882906, 80882943, 80882980, 80883017, 80884984, 80885021, 80885058, 80887062, 80887099, 80887618, 80887655, 80887692, 80887729, 80889140, 80889696, 80889733, 80889770, 80891774, 80891811, 80893852, 80894408, 80894445, 80894482, 80896486, 80896523, 80898564, 80901198, 80901235, 80903276, 80907988, 81028422, 81028459, 81030500, 81035212, 81190384, 81190421, 81190458, 81190495, 81192462, 81192499, 81192536, 81194540, 81194577, 81196618, 81197174, 81197211, 81197248, 81199252, 81199289, 81201330, 81203964, 81204001, 81206042, 81210754, 81337978, 81499940, 81499977, 81500014, 81502018, 81502055, 81504096, 81506730, 81506767, 81508808, 81513520, 81809496, 81809533, 81811574, 81816286, 81971458, 81971495, 81971532, 81971569, 81973536, 81973573, 81973610, 81975614, 81975651, 81977692, 81978248, 81978285, 81978322, 81980326, 81980363, 81982404, 81985038, 81985075, 81987116, 81991828, 82119052, 82281014, 82281051, 82281088, 82283092, 82283129, 82285170, 82287804, 82287841, 82289882, 82294594, 82314088, 82314125, 82314162, 82314199, 82314236, 82316166, 82316203, 82316240, 82316277, 82318244, 82318281, 82318318, 82320322, 82320359, 82320878, 82320915, 82320952, 82320989, 82322400, 82322956, 82322993, 82323030, 82325034, 82325071, 82327112, 82327668, 82327705, 82327742, 82329746, 82329783, 82331824, 82334458, 82334495, 82336536, 82341248, 82590570, 82590607, 82592648, 82597360, 82623644, 82623681, 82623718, 82623755, 82625722, 82625759, 82625796, 82627800, 82627837, 82629878, 82630434, 82630471, 82630508, 82632512, 82632549, 82634590, 82637224, 82637261, 82639302, 82644014, 82900126, 82933200, 82933237, 82933274, 82935278, 82935315, 82937356, 82939990, 82940027, 82942068, 82946780, 83062088, 83062125, 83062162, 83064166, 83064203, 83066244, 83068878, 83068915, 83070956, 83075668, 83242756, 83242793, 83244834, 83249546, 83371644, 83371681, 83373722, 83378434, 83389276, 83389313, 83389350, 83389387, 83389424, 83391354, 83391391, 83391428, 83391465, 83393432, 83393469, 83393506, 83395510, 83395547, 83396066, 83396103, 83396140, 83396177, 83397588, 83398144, 83398181, 83398218, 83400222, 83400259, 83402300, 83402856, 83402893, 83402930, 83404718, 83404755, 83404792, 83404829, 83404934, 83404971, 83406796, 83406833, 83406870, 83407012, 83408874, 83408911, 83409646, 83409683, 83410952, 83411508, 83411545, 83411582, 83411724, 83413586, 83413623, 83415664, 83416436, 83418298, 83418335, 83420376, 83425088, 83552312, 83681200, 83698832, 83698869, 83698906, 83698943, 83700910, 83700947, 83700984, 83702988, 83703025, 83705066, 83705622, 83705659, 83705696, 83707700, 83707737, 83709778, 83712412, 83712449, 83714274, 83714311, 83714348, 83714490, 83716352, 83716389, 83718430, 83719202, 83721064, 83721101, 83723142, 83727854, 83837436, 83837473, 83839514, 83844226, 84008388, 84008425, 84008462, 84010466, 84010503, 84012544, 84015178, 84015215, 84017256, 84021968, 84023830, 84023867, 84025908, 84030620, 84146992, 84152718, 84152755, 84154796, 84159508, 84317944, 84317981, 84320022, 84324734, 84333386, 84462274, 84479906, 84479943, 84479980, 84480017, 84481984, 84482021, 84482058, 84484062, 84484099, 84486140, 84486696, 84486733, 84486770, 84488774, 84488811, 84490852, 84493486, 84493523, 84495348, 84495385, 84495422, 84495564, 84497426, 84497463, 84499504, 84500276, 84502138, 84502175, 84504216, 84508928, 84627500, 84789462, 84789499, 84789536, 84791540, 84791577, 84793618, 84796252, 84796289, 84798330, 84803042, 84804904, 84804941, 84806982, 84811694, 84837978, 84838015, 84838052, 84838089, 84840056, 84840093, 84840130, 84842134, 84842171, 84844212, 84844768, 84844805, 84844842, 84846846, 84846883, 84848924, 84851558, 84851595, 84853636, 84858348, 84928066, 85099018, 85099055, 85101096, 85105808, 85114460, 85147534, 85147571, 85147608, 85149612, 85149649, 85151690, 85154324, 85154361, 85156402, 85161114, 85243348, 85408574, 85457090, 85457127, 85459168, 85463880, 85570536, 85570573, 85570610, 85572614, 85572651, 85574692, 85577326, 85577363, 85579404, 85584116, 85585978, 85586015, 85588056, 85592768, 85766646, 85826832, 85826869, 85826906, 85826943, 85826980, 85828910, 85828947, 85828984, 85829021, 85830988, 85831025, 85831062, 85833066, 85833103, 85833622, 85833659, 85833696, 85833733, 85835144, 85835700, 85835737, 85835774, 85837778, 85837815, 85839856, 85840412, 85840449, 85840486, 85842490, 85842527, 85844568, 85847202, 85847239, 85849280, 85853992, 85880092, 85880129, 85882170, 85886882, 85895534, 85913166, 85913203, 85913240, 85913277, 85915244, 85915281, 85915318, 85917322, 85917359, 85919400, 85919956, 85919993, 85920030, 85922034, 85922071, 85924112, 85926746, 85926783, 85928608, 85928645, 85928682, 85928824, 85930686, 85930723, 85932764, 85933536, 85935398, 85935435, 85937476, 85942188, 86136388, 86136425, 86136462, 86136499, 86138466, 86138503, 86138540, 86140544, 86140581, 86142622, 86143178, 86143215, 86143252, 86145256, 86145293, 86147334, 86149968, 86150005, 86152046, 86156758, 86189648, 86222722, 86222759, 86222796, 86224800, 86224837, 86226878, 86229512, 86229549, 86231590, 86236302, 86238164, 86238201, 86240242, 86244954, 86361326, 86445944, 86445981, 86446018, 86448022, 86448059, 86450100, 86452734, 86452771, 86454812, 86459524, 86532278, 86532315, 86534356, 86539068, 86547720, 86661166, 86661203, 86663244, 86667956, 86676608, 86755500, 86755537, 86757578, 86762290, 86841834, 86917462, 86917499, 86917536, 86917573, 86919540, 86919577, 86919614, 86921618, 86921655, 86923696, 86924252, 86924289, 86924326, 86926330, 86926367, 86928408, 86931042, 86931079, 86933120, 86937832, 86970722, 86988354, 86988391, 86988428, 86988465, 86990432, 86990469, 86990506, 86992510, 86992547, 86994588, 86995144, 86995181, 86995218, 86997222, 86997259, 86999300, 87001934, 87001971, 87003796, 87003833, 87003870, 87004012, 87005874, 87005911, 87007952, 87008724, 87010586, 87010623, 87012664, 87017376, 87019238, 87019275, 87021316, 87026028, 87065056, 87227018, 87227055, 87227092, 87229096, 87229133, 87231174, 87233808, 87233845, 87235886, 87240598, 87297910, 87297947, 87297984, 87299988, 87300025, 87302066, 87304700, 87304737, 87306778, 87311490, 87313352, 87313389, 87315430, 87320142, 87328794, 87361868, 87361905, 87361942, 87363946, 87363983, 87366024, 87368658, 87368695, 87370736, 87375448, 87436514, 87536574, 87536611, 87538652, 87543364, 87607466, 87607503, 87609544, 87614256, 87622908, 87671424, 87671461, 87673502, 87678214, 87751796, 87846130, 87917022, 87980980, 88008092, 88008129, 88008166, 88010170, 88010207, 88012248, 88014882, 88014919, 88016960, 88021672, 88078984, 88079021, 88079058, 88081062, 88081099, 88083140, 88085774, 88085811, 88087852, 88092564, 88094426, 88094463, 88096504, 88101216, 88109868, 88317648, 88317685, 88319726, 88324438, 88350722, 88350759, 88350796, 88350833, 88352800, 88352837, 88352874, 88354878, 88354915, 88356956, 88357512, 88357549, 88357586, 88359590, 88359627, 88361668, 88364302, 88364339, 88366380, 88371092, 88388540, 88388577, 88390618, 88395330, 88403982, 88437056, 88437093, 88437130, 88439134, 88439171, 88441212, 88443846, 88443883, 88445924, 88450636, 88452498, 88452535, 88454576, 88459288, 88627204, 88660278, 88660315, 88660352, 88662356, 88662393, 88664434, 88667068, 88667105, 88669146, 88673858, 88698096, 88746612, 88746649, 88748690, 88753402, 88762054, 88969834, 88969871, 88971912, 88976624, 89056168, 89098722, 89098759, 89100800, 89105512, 89169614, 89169651, 89171692, 89176404, 89185056, 89279390, 89408278, 89425910, 89425947, 89425984, 89426021, 89427988, 89428025, 89428062, 89430066, 89430103, 89432144, 89432700, 89432737, 89432774, 89434778, 89434815, 89436856, 89439490, 89439527, 89441352, 89441389, 89441426, 89441568, 89443430, 89443467, 89445508, 89446280, 89448142, 89448179, 89450220, 89454932, 89479170, 89512244, 89512281, 89512318, 89514322, 89514359, 89516400, 89519034, 89519071, 89521112, 89525824, 89527686, 89527723, 89529764, 89534476, 89543128, 89735466, 89735503, 89735540, 89737544, 89737581, 89739622, 89742256, 89742293, 89744334, 89749046, 89750908, 89750945, 89752986, 89757698, 89821800, 89821837, 89823878, 89828590, 89837242, 89874070, 89885758, 89885795, 89887836, 89892548, 90045022, 90045059, 90047100, 90051812, 90060464, 90131356, 90189352, 90195314, 90260244, 90354578, 90516540, 90516577, 90516614, 90518618, 90518655, 90520696, 90523330, 90523367, 90525408, 90530120, 90531982, 90532019, 90534060, 90538772, 90587432, 90587469, 90587506, 90589510, 90589547, 90591588, 90594222, 90594259, 90596300, 90601012, 90602874, 90602911, 90604952, 90609664, 90618316, 90826096, 90826133, 90828174, 90832886, 90841538, 90874612, 90874649, 90874686, 90876690, 90876727, 90878768, 90881402, 90881439, 90883480, 90888192, 90896988, 90897025, 90899066, 90903778, 90912430, 90960946, 90960983, 90963024, 90967736, 90976388, 91135652, 91184168, 91184205, 91186246, 91190958, 91206544, 91270502, 91493724, 91607170, 91607207, 91609248, 91613960, 91622612, 91678062, 91678099, 91680140, 91684852, 91693504, 91863466, 91863503, 91863540, 91863577, 91865544, 91865581, 91865618, 91867622, 91867659, 91869700, 91870256, 91870293, 91870330, 91872334, 91872371, 91874412, 91877046, 91877083, 91879124, 91883836, 91916726, 91949800, 91949837, 91949874, 91951878, 91951915, 91953956, 91956590, 91956627, 91958668, 91963380, 91965242, 91965279, 91967320, 91972032, 91987618, 92036134, 92036171, 92038212, 92042924, 92051576, 92173022, 92173059, 92173096, 92175100, 92175137, 92177178, 92179812, 92179849, 92181890, 92186602, 92259356, 92259393, 92261434, 92266146, 92274798, 92345690, 92409648, 92482578, 92482615, 92484656, 92489368, 92568912, 92697800, 92768692, 92792134, 92954096, 92954133, 92954170, 92956174, 92956211, 92958252, 92960886, 92960923, 92962964, 92967676, 93024988, 93025025, 93025062, 93027066, 93027103, 93029144, 93031778, 93031815, 93033856, 93038568, 93040430, 93040467, 93042508, 93047220, 93055872, 93111322, 93111359, 93113400, 93118112, 93126764, 93263652, 93263689, 93265730, 93270442, 93334544, 93334581, 93336622, 93341334, 93349986, 93398502, 93398539, 93400580, 93405292, 93420878, 93484836, 93573208, 93644100, 93708058, 94044726, 94044763, 94046804, 94051516, 94115618, 94115655, 94117696, 94122408, 94131060, 94186510, 94186547, 94188588, 94193300, 94201952, 94354282, 94387356, 94387393, 94387430, 94389434, 94389471, 94391512, 94394146, 94394183, 94396224, 94400936, 94425174, 94473690, 94473727, 94475768, 94480480, 94489132, 94496066, 94560024, 94696912, 94696949, 94698990, 94703702, 94783246, 95006468, 95135356, 95206248, 95277140, 95462544, 95462581, 95462618, 95464622, 95464659, 95466700, 95469334, 95469371, 95471412, 95476124, 95477986, 95478023, 95480064, 95484776, 95548878, 95548915, 95550956, 95555668, 95564320, 95635212, 95772100, 95772137, 95774178, 95778890, 95787542, 95858434, 95922392, 96081656, 96553174, 96553211, 96555252, 96557684, 96557721, 96557758, 96557795, 96557832, 96559762, 96559799, 96559836, 96559873, 96559964, 96561840, 96561877, 96561914, 96563918, 96563955, 96564474, 96564511, 96564548, 96564585, 96565996, 96566552, 96566589, 96566626, 96568616, 96568630, 96568667, 96570708, 96571264, 96571301, 96571338, 96573342, 96573379, 96575420, 96578054, 96578091, 96580132, 96584844, 96624066, 96624103, 96626144, 96630856, 96639508, 96710400, 96862730, 96867240, 96867277, 96867314, 96867351, 96869318, 96869355, 96869392, 96871396, 96871433, 96873474, 96874030, 96874067, 96874104, 96876108, 96876145, 96878186, 96880820, 96880857, 96882898, 96887610, 96911246, 96911283, 96913324, 96918036, 96933622, 96997580, 97176796, 97176833, 97176870, 97178874, 97178911, 97180952, 97183586, 97183623, 97185664, 97190376, 97220802, 97486352, 97486389, 97488430, 97493142, 97643804, 97648314, 97648351, 97648388, 97648425, 97650392, 97650429, 97650466, 97652470, 97652507, 97654548, 97655104, 97655141, 97655178, 97657182, 97657219, 97659260, 97661894, 97661931, 97663972, 97668684, 97714696, 97785588, 97795908, 97900100, 97900137, 97900174, 97902178, 97902215, 97904256, 97906890, 97906927, 97908968, 97913680, 97957870, 97957907, 97957944, 97959948, 97959985, 97962026, 97964660, 97964697, 97966738, 97971450, 97986434, 97986471, 97988512, 97993224, 98001876, 98072768, 98209656, 98209693, 98211734, 98216446, 98267426, 98267463, 98269504, 98274216, 98295990, 98519212, 98576982, 98738944, 98738981, 98739018, 98741022, 98741059, 98743100, 98745734, 98745771, 98747812, 98752524, 98990730, 98990767, 98992808, 98997520, 99048500, 99048537, 99050578, 99055290, 99061622, 99061659, 99063700, 99068412, 99077064, 99081574, 99081611, 99081648, 99081685, 99083652, 99083689, 99083726, 99085730, 99085767, 99087808, 99088364, 99088401, 99088438, 99090442, 99090479, 99092520, 99095154, 99095191, 99097232, 99101944, 99147956, 99300286, 99358056, 99371178, 99391130, 99391167, 99391204, 99393208, 99393245, 99395286, 99397920, 99397957, 99399998, 99404710, 99435136, 99700686, 99700723, 99702764, 99707476, 99829574, 99829611, 99831652, 99836364, 100010242, 100081360, 100139130, 100152252, 100156762, 100156799, 100156836, 100156873, 100158840, 100158877, 100158914, 100160918, 100160955, 100162996, 100163552, 100163589, 100163626, 100165630, 100165667, 100167708, 100170342, 100170379, 100172204, 100172241, 100172278, 100172420, 100174282, 100174319, 100176360, 100177132, 100178994, 100179031, 100181072, 100185784, 100223144, 100423990, 100424027, 100426068, 100430780, 100466318, 100466355, 100466392, 100468396, 100468433, 100470474, 100473108, 100473145, 100475186, 100479898, 100481760, 100481797, 100483838, 100488550, 100510324, 100604922, 100733546, 100775874, 100775911, 100777952, 100782664, 100791316, 100920204, 101085430, 101247392, 101247429, 101247466, 101249470, 101249507, 101251548, 101254182, 101254219, 101256260, 101260972, 101262834, 101262871, 101264912, 101269624, 101499178, 101499215, 101501256, 101505968, 101514620, 101556948, 101556985, 101559026, 101563738, 101572390, 101585512, 101605464, 101605501, 101605538, 101607542, 101607579, 101609620, 101612254, 101612291, 101614332, 101619044, 101808734, 101866504, 101915020, 101915057, 101917098, 101921810, 102224576, 102338022, 102338059, 102340100, 102344812, 102353464, 102589808, 102594318, 102594355, 102594392, 102594429, 102596396, 102596433, 102596470, 102598474, 102598511, 102600552, 102601108, 102601145, 102601182, 102603186, 102603223, 102605264, 102607898, 102607935, 102609976, 102614688, 102647578, 102660700, 102680652, 102680689, 102680726, 102682730, 102682767, 102684808, 102687442, 102687479, 102689520, 102694232, 102696094, 102696131, 102698172, 102702884, 102903874, 102903911, 102903948, 102905952, 102905989, 102908030, 102910664, 102910701, 102912742, 102917454, 102947880, 102990208, 102990245, 102992286, 102996998, 103005650, 103213430, 103213467, 103215508, 103220220, 103299764, 103428652, 103522986, 103684948, 103684985, 103685022, 103687026, 103687063, 103689104, 103691738, 103691775, 103693816, 103698528, 103755840, 103755877, 103755914, 103757918, 103757955, 103759996, 103762630, 103762667, 103764708, 103769420, 103771282, 103771319, 103773360, 103778072, 103786724, 103936734, 103936771, 103938812, 103943524, 103994504, 103994541, 103996582, 104001294, 104023068, 104065396, 104065433, 104067474, 104072186, 104080838, 104129354, 104129391, 104131432, 104136144, 104246290, 104304060, 104374952, 104438910, 104775578, 104775615, 104777656, 104782368, 104846470, 104846507, 104848548, 104853260, 104861912, 105027364, 105085134, 105098256, 105118208, 105118245, 105118282, 105120286, 105120323, 105122364, 105124998, 105125035, 105127076, 105131788, 105156026, 105204542, 105204579, 105206620, 105211332, 105219984, 105427764, 105427801, 105429842, 105434554, 105514098, 105737320, 105866208, 105937100, 106193396, 106193433, 106193470, 106195474, 106195511, 106197552, 106200186, 106200223, 106202264, 106206976, 106208838, 106208875, 106210916, 106215628, 106279730, 106279767, 106281808, 106286520, 106295172, 106460624, 106502952, 106502989, 106505030, 106509742, 106518394, 106589286, 106653244, 106812508, 107284026, 107284063, 107286104, 107290816, 107299468, 107354918, 107354955, 107356996, 107361708, 107370360, 107535812, 107593582, 107642098, 107642135, 107644176, 107648888, 107664474, 107728432, 107951654, 108374656, 108445548, 108630952, 108630989, 108631026, 108633030, 108633067, 108635108, 108637742, 108637779, 108639820, 108644532, 108717286, 108717323, 108719364, 108724076, 108732728, 108803620, 108940508, 108940545, 108942586, 108947298, 109026842, 109250064, 109721582, 109721619, 109723660, 109728372, 109792474, 109792511, 109794552, 109799264, 109807916, 109878808, 109973368, 110031138, 110102030, 110165988, 110812212, 110883104, 110953996, 111154842, 111154879, 111156920, 111161632, 111241176, 111464398, 112230030, 112230067, 112232108, 112236820, 112245472, 112316364, 112539586, 113320660, 113325170, 113325207, 113325244, 113325281, 113327248, 113327285, 113327322, 113329326, 113329363, 113331404, 113331960, 113331997, 113332034, 113334038, 113334075, 113336116, 113338750, 113338787, 113340828, 113345540, 113391552, 113634726, 113634763, 113634800, 113636804, 113636841, 113638882, 113641516, 113641553, 113643594, 113648306, 113678732, 113944282, 113944319, 113946360, 113951072, 114253838, 114415800, 114415837, 114415874, 114417878, 114417915, 114419956, 114422590, 114422627, 114424668, 114429380, 114667586, 114667623, 114669664, 114674376, 114725356, 114725393, 114727434, 114732146, 114753920, 114977142, 115034912, 115338569, 115338606, 115338643, 115338680, 115338717, 115338754, 115340647, 115340684, 115340721, 115340758, 115340795, 115342725, 115342762, 115342799, 115342836, 115344803, 115344840, 115344877, 115345359, 115345396, 115345433, 115345470, 115345507, 115346881, 115346918, 115347437, 115347474, 115347511, 115347548, 115348959, 115349515, 115349552, 115349589, 115351593, 115351630, 115352149, 115352186, 115352223, 115352260, 115353671, 115354227, 115354264, 115354301, 115356305, 115356342, 115358383, 115358939, 115358976, 115359013, 115361017, 115361054, 115363095, 115365729, 115365766, 115367807, 115372519, 115506430, 115506467, 115508508, 115513220, 115648125, 115648162, 115648199, 115648236, 115648273, 115650203, 115650240, 115650277, 115650314, 115652281, 115652318, 115652355, 115654359, 115654396, 115654915, 115654952, 115654989, 115655026, 115656437, 115656993, 115657030, 115657067, 115659071, 115659108, 115661149, 115661705, 115661742, 115661779, 115663783, 115663820, 115665861, 115668495, 115668532, 115670573, 115675285, 115758216, 115815986, 115829108, 115849060, 115849097, 115849134, 115851138, 115851175, 115853216, 115855850, 115855887, 115857928, 115862640, 115957681, 115957718, 115957755, 115957792, 115959759, 115959796, 115959833, 115961837, 115961874, 115963915, 115964471, 115964508, 115964545, 115966549, 115966586, 115968627, 115971261, 115971298, 115973339, 115978051, 116158616, 116158653, 116160694, 116165406, 116267237, 116267274, 116267311, 116269315, 116269352, 116271393, 116274027, 116274064, 116276105, 116280817, 116429199, 116429236, 116429273, 116429310, 116429347, 116431277, 116431314, 116431351, 116431388, 116433355, 116433392, 116433429, 116435433, 116435470, 116435989, 116436026, 116436063, 116436100, 116437511, 116438067, 116438104, 116438141, 116440145, 116440182, 116442223, 116442779, 116442816, 116442853, 116444857, 116444894, 116446935, 116449569, 116449606, 116451647, 116456359, 116468172, 116576793, 116576830, 116578871, 116583583, 116597060, 116738755, 116738792, 116738829, 116738866, 116740833, 116740870, 116740907, 116742911, 116742948, 116744989, 116745545, 116745582, 116745619, 116747623, 116747660, 116749701, 116752335, 116752372, 116754413, 116759125, 116886349, 116924248, 116924285, 116924322, 116926326, 116926363, 116928404, 116931038, 116931075, 116933116, 116937828, 116939690, 116939727, 116941768, 116946480, 117048311, 117048348, 117048385, 117050389, 117050426, 117052467, 117055101, 117055138, 117057179, 117061891, 117191476, 117233804, 117233841, 117235882, 117240594, 117249246, 117357867, 117357904, 117359945, 117364657, 117519829, 117519866, 117519903, 117519940, 117521907, 117521944, 117521981, 117523985, 117524022, 117526063, 117526619, 117526656, 117526693, 117528697, 117528734, 117530775, 117533409, 117533446, 117535487, 117540199, 117543360, 117667423, 117829385, 117829422, 117829459, 117831463, 117831500, 117833541, 117836175, 117836212, 117838253, 117842965, 117862459, 117862496, 117862533, 117862570, 117862607, 117864537, 117864574, 117864611, 117864648, 117866615, 117866652, 117866689, 117868693, 117868730, 117869249, 117869286, 117869323, 117869360, 117870771, 117871327, 117871364, 117871401, 117873405, 117873442, 117875483, 117876039, 117876076, 117876113, 117878117, 117878154, 117880195, 117882829, 117882866, 117884907, 117889619, 118014878, 118014915, 118016956, 118021668, 118030320, 118138941, 118138978, 118141019, 118145731, 118172015, 118172052, 118172089, 118172126, 118174093, 118174130, 118174167, 118176171, 118176208, 118178249, 118178805, 118178842, 118178879, 118180883, 118180920, 118182961, 118185595, 118185632, 118187673, 118192385, 118266664, 118324434, 118372950, 118372987, 118375028, 118379740, 118448497, 118481571, 118481608, 118481645, 118483649, 118483686, 118485727, 118488361, 118488398, 118490439, 118495151, 118610459, 118610496, 118610533, 118612537, 118612574, 118614615, 118617249, 118617286, 118619327, 118624039, 118682506, 118791127, 118791164, 118793205, 118797917, 118920015, 118920052, 118922093, 118926805, 118937647, 118937684, 118937721, 118937758, 118937795, 118939725, 118939762, 118939799, 118939836, 118941803, 118941840, 118941877, 118943881, 118943918, 118944437, 118944474, 118944511, 118944548, 118945959, 118946515, 118946552, 118946589, 118948593, 118948630, 118950671, 118951227, 118951264, 118951301, 118953089, 118953126, 118953163, 118953200, 118953305, 118953342, 118955167, 118955204, 118955241, 118955383, 118957245, 118957282, 118958017, 118958054, 118959323, 118959879, 118959916, 118959953, 118960095, 118961957, 118961994, 118964035, 118964807, 118966669, 118966706, 118968747, 118973459, 119100683, 119105508, 119229571, 119247203, 119247240, 119247277, 119247314, 119249281, 119249318, 119249355, 119251359, 119251396, 119253437, 119253993, 119254030, 119254067, 119256071, 119256108, 119258149, 119260783, 119260820, 119262645, 119262682, 119262719, 119262861, 119264723, 119264760, 119266801, 119267573, 119269435, 119269472, 119271513, 119276225, 119361804, 119361841, 119361878, 119363882, 119363919, 119365960, 119368594, 119368631, 119370672, 119375384, 119448138, 119448175, 119450216, 119454928, 119463580, 119556759, 119556796, 119556833, 119558837, 119558874, 119560915, 119563549, 119563586, 119565627, 119570339, 119572201, 119572238, 119574279, 119578991, 119671360, 119671397, 119673438, 119678150, 119701089, 119701126, 119703167, 119707879, 119757694, 119866315, 119866352, 119868393, 119873105, 119881757, 119980916, 120010645, 120028277, 120028314, 120028351, 120028388, 120030355, 120030392, 120030429, 120032433, 120032470, 120034511, 120035067, 120035104, 120035141, 120037145, 120037182, 120039223, 120041857, 120041894, 120043719, 120043756, 120043793, 120043935, 120045797, 120045834, 120047875, 120048647, 120050509, 120050546, 120052587, 120057299, 120175871, 120337833, 120337870, 120337907, 120339911, 120339948, 120341989, 120344623, 120344660, 120346701, 120351413, 120353275, 120353312, 120355353, 120360065, 120386349, 120386386, 120386423, 120386460, 120388427, 120388464, 120388501, 120390505, 120390542, 120392583, 120393139, 120393176, 120393213, 120395217, 120395254, 120397295, 120399929, 120399966, 120402007, 120406719, 120452434, 120452471, 120454512, 120459224, 120523326, 120523363, 120525404, 120530116, 120538768, 120647389, 120647426, 120649467, 120654179, 120662831, 120695905, 120695942, 120695979, 120697983, 120698020, 120700061, 120702695, 120702732, 120704220, 120704773, 120709485, 120761990, 120791719, 120832882, 120896840, 120956945, 121005461, 121005498, 121007539, 121012251, 121118907, 121118944, 121118981, 121120985, 121121022, 121123063, 121125697, 121125734, 121127775, 121132487, 121134349, 121134386, 121136427, 121141139, 121315017, 121375203, 121375240, 121375277, 121375314, 121375351, 121377281, 121377318, 121377355, 121377392, 121379359, 121379396, 121379433, 121381437, 121381474, 121381993, 121382030, 121382067, 121382104, 121383515, 121384071, 121384108, 121384145, 121386149, 121386186, 121388227, 121388783, 121388820, 121388857, 121390861, 121390898, 121392939, 121395573, 121395610, 121397651, 121402363, 121428463, 121428500, 121430541, 121435253, 121443905, 121461537, 121461574, 121461611, 121461648, 121463615, 121463652, 121463689, 121465693, 121465730, 121467771, 121468327, 121468364, 121468401, 121470405, 121470442, 121472483, 121475117, 121475154, 121476979, 121477016, 121477053, 121477195, 121479057, 121479094, 121481135, 121481907, 121483769, 121483806, 121485847, 121490559, 121543064, 121613956, 121684759, 121684796, 121684833, 121684870, 121686837, 121686874, 121686911, 121688915, 121688952, 121690993, 121691549, 121691586, 121691623, 121693627, 121693664, 121695705, 121698339, 121698376, 121700417, 121705129, 121738019, 121771093, 121771130, 121771167, 121773171, 121773208, 121775249, 121777883, 121777920, 121779961, 121784673, 121786535, 121786572, 121788613, 121793325, 121885694, 121885731, 121887772, 121892484, 121972028, 121994315, 121994352, 121994389, 121996393, 121996430, 121998471, 122001105, 122001142, 122003183, 122007895, 122080649, 122080686, 122082727, 122087439, 122096091, 122195250, 122209537, 122209574, 122211615, 122216327, 122224979, 122303871, 122303908, 122305949, 122310661, 122390205, 122465833, 122465870, 122465907, 122465944, 122467911, 122467948, 122467985, 122469989, 122470026, 122472067, 122472623, 122472660, 122472697, 122474701, 122474738, 122476779, 122479413, 122479450, 122481491, 122486203, 122519093, 122536725, 122536762, 122536799, 122536836, 122538803, 122538840, 122538877, 122540881, 122540918, 122542959, 122543515, 122543552, 122543589, 122545593, 122545630, 122547671, 122550305, 122550342, 122552167, 122552204, 122552241, 122552383, 122554245, 122554282, 122556323, 122557095, 122558957, 122558994, 122561035, 122565747, 122567609, 122567646, 122569687, 122574399, 122613427, 122775389, 122775426, 122775463, 122777467, 122777504, 122779545, 122782179, 122782216, 122784257, 122788969, 122846281, 122846318, 122846355, 122848359, 122848396, 122850437, 122853071, 122853108, 122855149, 122859861, 122861723, 122861760, 122863801, 122868513, 122877165, 122910239, 122910276, 122910313, 122912317, 122912354, 122914395, 122917029, 122917066, 122919107, 122923819, 122960882, 122960919, 122962960, 122967672, 122976324, 123047216, 123084945, 123084982, 123087023, 123091735, 123155837, 123155874, 123157915, 123162627, 123171279, 123219795, 123219832, 123221873, 123226585, 123270438, 123300167, 123394501, 123465393, 123529351, 123556463, 123556500, 123556537, 123558541, 123558578, 123560619, 123563253, 123563290, 123565331, 123570043, 123627355, 123627392, 123627429, 123629433, 123629470, 123631511, 123634145, 123634182, 123636223, 123640935, 123642797, 123642834, 123644875, 123649587, 123658239, 123866019, 123866056, 123868097, 123872809, 123899093, 123899130, 123899167, 123899204, 123901171, 123901208, 123901245, 123903249, 123903286, 123905327, 123905883, 123905920, 123905957, 123907961, 123907998, 123910039, 123912673, 123912710, 123914751, 123919463, 123936911, 123936948, 123938989, 123943701, 123952353, 123985427, 123985464, 123985501, 123987505, 123987542, 123989583, 123992217, 123992254, 123994295, 123999007, 124000869, 124000906, 124002947, 124007659, 124051512, 124122404, 124175575, 124208649, 124208686, 124208723, 124210727, 124210764, 124212805, 124215439, 124215476, 124217517, 124222229, 124246467, 124294983, 124295020, 124297061, 124301773, 124310425, 124409584, 124518205, 124518242, 124520283, 124524995, 124604539, 124647093, 124647130, 124649171, 124653883, 124717985, 124718022, 124720063, 124724775, 124733427, 124827761, 124956649, 124974281, 124974318, 124974355, 124974392, 124976359, 124976396, 124976433, 124978437, 124978474, 124980515, 124981071, 124981108, 124981145, 124983149, 124983186, 124985227, 124987861, 124987898, 124989723, 124989760, 124989797, 124989939, 124991801, 124991838, 124993879, 124994651, 124996513, 124996550, 124998591, 125003303, 125027541, 125060615, 125060652, 125060689, 125062693, 125062730, 125064771, 125067405, 125067442, 125069483, 125074195, 125076057, 125076094, 125078135, 125082847, 125091499, 125283837, 125283874, 125283911, 125285915, 125285952, 125287993, 125290627, 125290664, 125292705, 125297417, 125299279, 125299316, 125301357, 125306069, 125370171, 125370208, 125372249, 125376961, 125385613, 125398438, 125398475, 125400516, 125405228, 125434129, 125434166, 125436207, 125440919, 125484772, 125498603, 125498640, 125498677, 125498714, 125498751, 125498788, 125500681, 125500718, 125500755, 125500792, 125500829, 125502759, 125502796, 125502833, 125502870, 125504837, 125504874, 125504911, 125505393, 125505430, 125505467, 125505504, 125505541, 125506915, 125506952, 125507471, 125507508, 125507545, 125507582, 125508993, 125509549, 125509586, 125509623, 125511627, 125511664, 125512183, 125512220, 125512257, 125512294, 125513705, 125514261, 125514298, 125514335, 125516339, 125516376, 125518417, 125518973, 125519010, 125519047, 125521051, 125521088, 125523129, 125525763, 125525800, 125527841, 125532553, 125593393, 125593430, 125595471, 125600183, 125608835, 125679727, 125707994, 125737723, 125743685, 125808159, 125808196, 125808233, 125808270, 125808307, 125808615, 125810237, 125810274, 125810311, 125810348, 125812315, 125812352, 125812389, 125814393, 125814430, 125814949, 125814986, 125815023, 125815060, 125816471, 125817027, 125817064, 125817101, 125819105, 125819142, 125821183, 125821739, 125821776, 125821813, 125823817, 125823854, 125825895, 125828529, 125828566, 125830607, 125835319, 125902949, 126064911, 126064948, 126064985, 126066989, 126067026, 126069067, 126071701, 126071738, 126073779, 126078491, 126080353, 126080390, 126082431, 126087143, 126117715, 126117752, 126117789, 126117826, 126119793, 126119830, 126119867, 126121871, 126121908, 126123949, 126124505, 126124542, 126124579, 126126583, 126126620, 126128661, 126131295, 126131332, 126133373, 126135803, 126135840, 126135877, 126137881, 126137918, 126138085, 126139959, 126142593, 126142630, 126144671, 126149383, 126151245, 126151282, 126153323, 126158035, 126166687, 126374467, 126374504, 126376545, 126381257, 126389909, 126422983, 126423020, 126423057, 126425061, 126425098, 126427139, 126427271, 126427308, 126427345, 126429349, 126429386, 126429773, 126429810, 126431427, 126431851, 126434061, 126434098, 126436139, 126436563, 126440851, 126445359, 126445396, 126447437, 126452149, 126460801, 126489068, 126509317, 126509354, 126511395, 126516107, 126524759, 126559960, 126589233, 126589270, 126589307, 126589344, 126589381, 126591311, 126591348, 126591385, 126591422, 126593389, 126593426, 126593463, 126595467, 126595504, 126596023, 126596060, 126596097, 126596134, 126597545, 126598101, 126598138, 126598175, 126600179, 126600216, 126602257, 126602813, 126602850, 126602887, 126604891, 126604928, 126606969, 126609603, 126609640, 126611681, 126616393, 126684023, 126732539, 126732576, 126734617, 126736827, 126736864, 126738905, 126739329, 126743617, 126754915, 126818873, 126898789, 126898826, 126898863, 126898900, 126900867, 126900904, 126900941, 126902945, 126902982, 126905023, 126905579, 126905616, 126905653, 126907657, 126907694, 126909735, 126912369, 126912406, 126914447, 126919159, 127042095, 127046383, 127155541, 127155578, 127157619, 127162331, 127170983, 127208345, 127208382, 127208419, 127210423, 127210460, 127212501, 127215135, 127215172, 127217213, 127221925, 127226433, 127226470, 127228511, 127233223, 127241875, 127411837, 127411874, 127411911, 127411948, 127413915, 127413952, 127413989, 127415993, 127416030, 127418071, 127418627, 127418664, 127418701, 127420705, 127420742, 127422783, 127425417, 127425454, 127427495, 127432207, 127465097, 127498171, 127498208, 127498245, 127500249, 127500286, 127502327, 127504961, 127504998, 127507039, 127511751, 127513613, 127513650, 127515691, 127517901, 127517938, 127519979, 127520403, 127524691, 127535989, 127584505, 127584542, 127586583, 127591295, 127599947, 127679863, 127679900, 127679937, 127679974, 127681941, 127681978, 127682015, 127684019, 127684056, 127686097, 127686653, 127686690, 127686727, 127688731, 127688768, 127690809, 127693443, 127693480, 127695521, 127700233, 127721393, 127721430, 127721467, 127723471, 127723508, 127725549, 127728183, 127728220, 127730261, 127734973, 127807727, 127807764, 127809805, 127814517, 127823169, 127827457, 127894061, 127922328, 127958019, 127989419, 127989456, 127989493, 127991497, 127991534, 127993575, 127996209, 127996246, 127998287, 128002999, 128022493, 128022530, 128022567, 128022604, 128022641, 128024571, 128024608, 128024645, 128024682, 128026649, 128026686, 128026723, 128028727, 128028764, 128029283, 128029320, 128029357, 128029394, 128030805, 128030949, 128030986, 128031361, 128031398, 128031435, 128033027, 128033439, 128033476, 128035517, 128036073, 128036110, 128036147, 128037739, 128038151, 128038188, 128040229, 128042863, 128042900, 128044941, 128049653, 128117283, 128246171, 128298975, 128299012, 128301053, 128305765, 128317063, 128332049, 128332086, 128332123, 128332160, 128334127, 128334164, 128334201, 128336205, 128336242, 128338283, 128338839, 128338876, 128338913, 128340505, 128340917, 128340954, 128342995, 128345629, 128345666, 128347707, 128352419, 128502467, 128502504, 128502541, 128504545, 128504582, 128506623, 128509257, 128509294, 128511335, 128516047, 128573359, 128573396, 128573433, 128575437, 128575474, 128577515, 128580149, 128580186, 128582227, 128586939, 128588801, 128588838, 128590879, 128595591, 128604243, 128608531, 128641605, 128641642, 128641679, 128643683, 128643720, 128645761, 128648395, 128648432, 128650473, 128655185, 128659693, 128659730, 128661771, 128666483, 128675135, 128770493, 128770530, 128770567, 128772571, 128772608, 128774649, 128777283, 128777320, 128779361, 128784073, 128812023, 128812060, 128814101, 128818813, 128882915, 128882952, 128884993, 128889705, 128898357, 128946873, 128946910, 128948951, 128951161, 128951198, 128953239, 128953663, 128957951, 128969249, 128997516, 129033207, 129080049, 129080086, 129082127, 129086839, 129097681, 129097718, 129097755, 129097792, 129097829, 129099759, 129099796, 129099833, 129099870, 129101837, 129101874, 129101911, 129103915, 129103952, 129104471, 129104508, 129104545, 129104582, 129105993, 129106549, 129106586, 129106623, 129108627, 129108664, 129110705, 129111261, 129111298, 129111335, 129113123, 129113160, 129113197, 129113234, 129113339, 129113376, 129115201, 129115238, 129115275, 129115417, 129117279, 129117316, 129118051, 129118088, 129119357, 129119913, 129119950, 129119987, 129120129, 129121579, 129121991, 129122028, 129124069, 129124841, 129126703, 129126740, 129128781, 129133493, 129192471, 129256429, 129260717, 129389605, 129407237, 129407274, 129407311, 129407348, 129409315, 129409352, 129409389, 129411393, 129411430, 129413471, 129414027, 129414064, 129414101, 129416105, 129416142, 129418183, 129420817, 129420854, 129422679, 129422716, 129422753, 129422895, 129424757, 129424794, 129426835, 129427607, 129429469, 129429506, 129431547, 129436259, 129593097, 129593134, 129595175, 129599887, 129663989, 129664026, 129666067, 129670779, 129679431, 129716793, 129716830, 129716867, 129718871, 129718908, 129720949, 129723583, 129723620, 129725661, 129730373, 129732235, 129732272, 129734313, 129734881, 129734918, 129736959, 129739025, 129741671, 129750323, 129861123, 129861160, 129863201, 129867913, 129902653, 129935727, 129935764, 129935801, 129937805, 129937842, 129939883, 129942517, 129942554, 129944595, 129949307, 129973545, 130022061, 130022098, 130024139, 130026349, 130026386, 130028427, 130028851, 130033139, 130037503, 130041791, 130044437, 130092656, 130092693, 130092730, 130094734, 130094771, 130096812, 130099446, 130099483, 130101524, 130106236, 130108395, 130170679, 130188311, 130188348, 130188385, 130188422, 130190389, 130190426, 130190463, 130192467, 130192504, 130194545, 130195101, 130195138, 130195175, 130197179, 130197216, 130199257, 130201891, 130201928, 130203753, 130203790, 130203827, 130203969, 130205831, 130205868, 130207909, 130208681, 130210543, 130210580, 130212621, 130217333, 130245283, 130245320, 130247361, 130252073, 130331617, 130335905, 130402212, 130402249, 130404290, 130409002, 130497867, 130497904, 130497941, 130499945, 130499982, 130502023, 130504657, 130504694, 130506735, 130511447, 130513309, 130513346, 130515387, 130520099, 130546383, 130546420, 130546457, 130546494, 130548461, 130548498, 130548535, 130550539, 130550576, 130552617, 130553173, 130553210, 130553247, 130554839, 130555251, 130555288, 130557329, 130559963, 130560000, 130562041, 130566753, 130683727, 130711768, 130754619, 130807423, 130807460, 130809501, 130814213, 130822865, 130825511, 130855939, 130855976, 130856013, 130858017, 130858054, 130860095, 130862729, 130862766, 130864807, 130869519, 130951753, 131010915, 131010952, 131010989, 131012993, 131013030, 131015071, 131017705, 131017742, 131019783, 131024495, 131026357, 131026394, 131028435, 131033147, 131097249, 131097286, 131099327, 131104039, 131112691, 131116979, 131165495, 131165532, 131167573, 131172285, 131183286, 131183323, 131183583, 131185364, 131190076, 131278941, 131278978, 131279015, 131281019, 131281056, 131283097, 131285731, 131285768, 131287809, 131292521, 131294383, 131294420, 131296461, 131301173, 131320471, 131320508, 131322549, 131327261, 131335913, 131406805, 131435072, 131470763, 131475051, 131492842, 131535237, 131535274, 131535311, 131535348, 131535385, 131537315, 131537352, 131537389, 131537426, 131539393, 131539430, 131539467, 131541471, 131541508, 131542027, 131542064, 131542101, 131542138, 131543549, 131544105, 131544142, 131544179, 131546183, 131546220, 131548261, 131548817, 131548854, 131548891, 131550895, 131550932, 131552973, 131555607, 131555644, 131557685, 131562397, 131588497, 131588534, 131590575, 131595287, 131603939, 131621571, 131621608, 131621645, 131621682, 131623649, 131623686, 131623723, 131625727, 131625764, 131627805, 131628361, 131628398, 131628435, 131630027, 131630439, 131630476, 131632517, 131635151, 131635188, 131637013, 131637050, 131637087, 131637229, 131639091, 131639128, 131641169, 131641941, 131643803, 131643840, 131645881, 131650593, 131844793, 131844830, 131844867, 131844904, 131846871, 131846908, 131846945, 131848949, 131848986, 131851027, 131851583, 131851620, 131851657, 131853661, 131853698, 131855739, 131858373, 131858410, 131860451, 131865163, 131898053, 131931127, 131931164, 131931201, 131933205, 131933242, 131935283, 131937917, 131937954, 131939995, 131944707, 131946569, 131946606, 131948647, 131953359, 132101545, 132101582, 132103623, 132106055, 132106092, 132106129, 132106166, 132106203, 132108133, 132108170, 132108207, 132108244, 132108335, 132110211, 132110248, 132110285, 132112289, 132112326, 132112845, 132112882, 132112919, 132112956, 132114367, 132114923, 132114960, 132114997, 132116987, 132117001, 132117038, 132119079, 132119635, 132119672, 132119709, 132121713, 132121750, 132123791, 132126425, 132126462, 132128503, 132133215, 132154349, 132154386, 132154423, 132156427, 132156464, 132158505, 132161139, 132161176, 132163217, 132167929, 132172437, 132172474, 132174515, 132179227, 132187879, 132240683, 132240720, 132242761, 132247473, 132256125, 132258771, 132273916, 132369571, 132369608, 132371649, 132376361, 132385013, 132411101, 132415611, 132415648, 132415685, 132415722, 132417689, 132417726, 132417763, 132419767, 132419804, 132421845, 132422401, 132422438, 132422475, 132424479, 132424516, 132426557, 132429191, 132429228, 132431269, 132435981, 132459617, 132459654, 132461695, 132463905, 132463942, 132465983, 132466407, 132470695, 132481993, 132545951, 132550239, 132616546, 132616583, 132618624, 132623336, 132625867, 132625904, 132625941, 132625978, 132627945, 132627982, 132628019, 132630023, 132630060, 132632101, 132632657, 132632694, 132632731, 132634735, 132634772, 132636813, 132639447, 132639484, 132641525, 132646237, 132679127, 132696759, 132696796, 132696833, 132696870, 132698837, 132698874, 132698911, 132700915, 132700952, 132702993, 132703549, 132703586, 132703623, 132705627, 132705664, 132707705, 132710339, 132710376, 132712201, 132712238, 132712275, 132712417, 132714279, 132714316, 132716357, 132717129, 132718991, 132719028, 132721069, 132725167, 132725204, 132725241, 132725781, 132727245, 132727282, 132727643, 132727680, 132729323, 132729721, 132731957, 132731994, 132734035, 132734433, 132738747, 132769173, 132773461, 132926102, 132935423, 132935460, 132935497, 132937501, 132937538, 132939579, 132942213, 132942250, 132944291, 132949003, 133006315, 133006352, 133006389, 133008393, 133008430, 133010471, 133013105, 133013142, 133015183, 133019895, 133021757, 133021794, 133023835, 133028547, 133034723, 133034760, 133036801, 133037199, 133041513, 133070273, 133070310, 133070347, 133072351, 133072388, 133074429, 133077063, 133077100, 133079141, 133083853, 133192175, 133196685, 133196722, 133196759, 133196796, 133198763, 133198800, 133198837, 133200841, 133200878, 133202919, 133203475, 133203512, 133203549, 133205553, 133205590, 133207631, 133210265, 133210302, 133212343, 133217055, 133244979, 133245016, 133247057, 133251769, 133263067, 133315871, 133315908, 133317949, 133322661, 133331313, 133333959, 133344279, 133379829, 133379866, 133381907, 133386619, 133448471, 133448508, 133448545, 133450549, 133450586, 133452627, 133455261, 133455298, 133457339, 133460201, 133462051, 133506241, 133506278, 133506315, 133508319, 133508356, 133510397, 133513031, 133513068, 133515109, 133519821, 133534805, 133534842, 133536883, 133541595, 133550247, 133554535, 133621139, 133625427, 133689385, 133691734, 133691771, 133693812, 133698524, 133707176, 133716497, 133716534, 133716571, 133718575, 133718612, 133720653, 133723287, 133723324, 133725365, 133730077, 133758027, 133758064, 133760105, 133764817, 133787389, 133787426, 133787463, 133789467, 133789504, 133791545, 133794179, 133794216, 133796257, 133800969, 133802831, 133802868, 133804909, 133809621, 133815797, 133815834, 133817875, 133818273, 133822587, 133844361, 134001290, 134026053, 134026090, 134028131, 134032843, 134059127, 134059164, 134059201, 134059238, 134061205, 134061242, 134061279, 134063283, 134063320, 134065361, 134065917, 134065954, 134065991, 134067583, 134067995, 134068032, 134070073, 134072707, 134072744, 134074785, 134079497, 134096945, 134096982, 134099023, 134103735, 134112387, 134125353, 134145461, 134145498, 134145535, 134147539, 134147576, 134149617, 134152251, 134152288, 134154329, 134159041, 134160903, 134160940, 134162981, 134167693, 134287315, 134287352, 134287389, 134289393, 134289430, 134291471, 134294105, 134294142, 134296183, 134300895, 134335609, 134368683, 134368720, 134368757, 134370761, 134370798, 134372839, 134375473, 134375510, 134377551, 134382263, 134406501, 134455017, 134455054, 134457095, 134461807, 134470459, 134539101, 134539138, 134541179, 134545891, 134596871, 134596908, 134598949, 134603661, 134609993, 134610030, 134612071, 134616783, 134625435, 134629945, 134629982, 134630019, 134630056, 134632023, 134632060, 134632097, 134634101, 134634138, 134636179, 134636735, 134636772, 134636809, 134638813, 134638850, 134640891, 134643525, 134643562, 134645603, 134650315, 134678239, 134678276, 134680317, 134685029, 134696327, 134764573, 134782364, 134807127, 134807164, 134809205, 134813917, 134848657, 134878019, 134878056, 134880097, 134884809, 134893461, 134906427, 134919549, 134939501, 134939538, 134939575, 134941579, 134941616, 134943657, 134946291, 134946328, 134948369, 134953081, 134983507, 134987795, 135116683, 135134315, 135134352, 135134389, 135134426, 135136393, 135136430, 135136467, 135138471, 135138508, 135140436, 135140549, 135141105, 135141142, 135141179, 135143183, 135143220, 135145261, 135147895, 135147932, 135149757, 135149794, 135149831, 135149973, 135151835, 135151872, 135153913, 135154685, 135156547, 135156584, 135158625, 135163337, 135187575, 135220649, 135220686, 135220723, 135222727, 135222764, 135224805, 135227439, 135227476, 135229517, 135234229, 135236091, 135236128, 135238169, 135242881, 135249057, 135249094, 135251135, 135251533, 135255847, 135377945, 135377982, 135380023, 135384735, 135443871, 135443908, 135443945, 135445949, 135445986, 135448027, 135450661, 135450698, 135452739, 135457451, 135459313, 135459350, 135461391, 135466103, 135530205, 135530242, 135532283, 135536995, 135545647, 135558613, 135594163, 135594200, 135596241, 135600953, 135629731, 135687501, 135700623, 135701655, 135701663, 135701692, 135701694, 135701700, 135701729, 135701731, 135701737, 135701766, 135701768, 135701774, 135701803, 135701805, 135701811, 135701840, 135701842, 135701848, 135701879, 135703733, 135703741, 135703770, 135703772, 135703778, 135703807, 135703809, 135703815, 135703844, 135703846, 135703852, 135703881, 135703883, 135703889, 135703920, 135705133, 135705170, 135705207, 135705244, 135705811, 135705819, 135705848, 135705850, 135705856, 135705885, 135705887, 135705893, 135705922, 135705924, 135705930, 135705961, 135707211, 135707248, 135707285, 135707889, 135707897, 135707926, 135707928, 135707934, 135707963, 135707965, 135707971, 135708002, 135708445, 135708453, 135708482, 135708484, 135708490, 135708519, 135708521, 135708527, 135708556, 135708558, 135708564, 135708593, 135708595, 135708601, 135708632, 135709289, 135709326, 135709967, 135709975, 135710004, 135710006, 135710012, 135710043, 135710523, 135710531, 135710560, 135710562, 135710568, 135710597, 135710599, 135710605, 135710634, 135710636, 135710642, 135710673, 135711367, 135711923, 135711960, 135711997, 135712045, 135712053, 135712084, 135712601, 135712609, 135712638, 135712640, 135712646, 135712675, 135712677, 135712683, 135712714, 135714001, 135714038, 135714679, 135714687, 135714716, 135714718, 135714724, 135714755, 135715235, 135715243, 135715272, 135715274, 135715280, 135715309, 135715311, 135715317, 135715346, 135715348, 135715354, 135715385, 135716079, 135716757, 135716765, 135716796, 135717313, 135717321, 135717350, 135717352, 135717358, 135717387, 135717389, 135717395, 135717426, 135718713, 135718750, 135719391, 135719399, 135719428, 135719430, 135719436, 135719467, 135720575, 135720612, 135720649, 135720791, 135721469, 135721477, 135721508, 135722025, 135722033, 135722062, 135722064, 135722070, 135722099, 135722101, 135722107, 135722138, 135722653, 135722690, 135724103, 135724111, 135724140, 135724142, 135724148, 135724179, 135724731, 135725503, 135726181, 135726189, 135726220, 135727365, 135727402, 135728815, 135728823, 135728852, 135728854, 135728860, 135728891, 135729443, 135730893, 135730901, 135730932, 135734155, 135735605, 135735613, 135735644, 135753427, 135753464, 135755505, 135760217, 135768869, 135771515, 135839761, 135897757, 135903719, 135968649, 135972361, 135972398, 135974439, 135979151, 136011211, 136011219, 136011248, 136011250, 136011256, 136011285, 136011287, 136011293, 136011322, 136011324, 136011330, 136011359, 136011361, 136011367, 136011398, 136013289, 136013297, 136013326, 136013328, 136013334, 136013363, 136013365, 136013371, 136013400, 136013402, 136013408, 136013439, 136014689, 136014726, 136014763, 136015367, 136015375, 136015404, 136015406, 136015412, 136015441, 136015443, 136015449, 136015480, 136016767, 136016804, 136017445, 136017453, 136017482, 136017484, 136017490, 136017521, 136018001, 136018009, 136018038, 136018040, 136018046, 136018075, 136018077, 136018083, 136018112, 136018114, 136018120, 136018151, 136018845, 136019523, 136019531, 136019562, 136020079, 136020087, 136020116, 136020118, 136020124, 136020153, 136020155, 136020161, 136020192, 136021479, 136021516, 136022157, 136022165, 136022194, 136022196, 136022202, 136022233, 136023557, 136024235, 136024243, 136024274, 136024791, 136024799, 136024828, 136024830, 136024836, 136024865, 136024867, 136024873, 136024904, 136026869, 136026877, 136026906, 136026908, 136026914, 136026945, 136028269, 136028947, 136028955, 136028986, 136030131, 136030168, 136031581, 136031589, 136031618, 136031620, 136031626, 136031657, 136032209, 136033659, 136033667, 136033698, 136036921, 136038371, 136038379, 136038410, 136058695, 136062983, 136129290, 136129327, 136131368, 136136080, 136215624, 136224945, 136224982, 136225019, 136227023, 136227060, 136229101, 136231735, 136231772, 136233813, 136238525, 136240387, 136240424, 136242465, 136247177, 136281917, 136295837, 136295874, 136295911, 136297915, 136297952, 136299993, 136302627, 136302664, 136304705, 136309417, 136311279, 136311316, 136313357, 136318069, 136320767, 136320775, 136320804, 136320806, 136320812, 136320841, 136320843, 136320849, 136320878, 136320880, 136320886, 136320917, 136322845, 136322853, 136322882, 136322884, 136322890, 136322919, 136322921, 136322927, 136322958, 136324245, 136324282, 136324923, 136324931, 136324960, 136324962, 136324968, 136324999, 136326323, 136326721, 136327001, 136327009, 136327040, 136327557, 136327565, 136327594, 136327596, 136327602, 136327631, 136327633, 136327639, 136327670, 136329635, 136329643, 136329672, 136329674, 136329680, 136329711, 136331035, 136331713, 136331721, 136331752, 136334347, 136334355, 136334384, 136334386, 136334392, 136334423, 136336425, 136336433, 136336464, 136339687, 136341137, 136341145, 136341176, 136438846, 136468575, 136534501, 136534538, 136536579, 136541291, 136549943, 136583017, 136583054, 136583091, 136585095, 136585132, 136587173, 136589807, 136589844, 136591885, 136596597, 136605393, 136605430, 136607471, 136612183, 136620835, 136630323, 136630331, 136630360, 136630362, 136630368, 136630397, 136630399, 136630405, 136630436, 136632401, 136632409, 136632438, 136632440, 136632446, 136632477, 136633801, 136634479, 136634487, 136634518, 136637113, 136637121, 136637150, 136637152, 136637158, 136637189, 136639191, 136639199, 136639230, 136643903, 136643911, 136643942, 136669351, 136669388, 136671429, 136676141, 136684793, 136792285, 136792293, 136792322, 136792324, 136792330, 136792359, 136792361, 136792367, 136792396, 136792398, 136792404, 136792433, 136792435, 136792441, 136792472, 136794363, 136794371, 136794400, 136794402, 136794408, 136794437, 136794439, 136794445, 136794474, 136794476, 136794482, 136794513, 136795763, 136795800, 136795837, 136796441, 136796449, 136796478, 136796480, 136796486, 136796515, 136796517, 136796523, 136796554, 136797841, 136797878, 136798519, 136798527, 136798556, 136798558, 136798564, 136798595, 136799075, 136799083, 136799112, 136799114, 136799120, 136799149, 136799151, 136799157, 136799186, 136799188, 136799194, 136799225, 136799919, 136800597, 136800605, 136800636, 136801153, 136801161, 136801190, 136801192, 136801198, 136801227, 136801229, 136801235, 136801266, 136802553, 136802590, 136803231, 136803239, 136803268, 136803270, 136803276, 136803307, 136804631, 136805309, 136805317, 136805348, 136805865, 136805873, 136805902, 136805904, 136805910, 136805939, 136805941, 136805947, 136805978, 136807943, 136807951, 136807980, 136807982, 136807988, 136808019, 136809343, 136810021, 136810029, 136810060, 136811205, 136811242, 136812655, 136812663, 136812692, 136812694, 136812700, 136812731, 136813283, 136814733, 136814741, 136814772, 136817995, 136819445, 136819453, 136819484, 136844057, 136892573, 136892610, 136894651, 136899363, 136914949, 136939879, 136939887, 136939916, 136939918, 136939924, 136939955, 136941957, 136941965, 136941996, 136946669, 136946677, 136946708, 136978907, 137047549, 137047586, 137049627, 137054339, 137062991, 137101841, 137101849, 137101878, 137101880, 137101886, 137101915, 137101917, 137101923, 137101952, 137101954, 137101960, 137101991, 137103919, 137103927, 137103956, 137103958, 137103964, 137103993, 137103995, 137104001, 137104032, 137105319, 137105356, 137105997, 137106005, 137106034, 137106036, 137106042, 137106073, 137107397, 137108075, 137108083, 137108114, 137108631, 137108639, 137108668, 137108670, 137108676, 137108705, 137108707, 137108713, 137108744, 137110709, 137110717, 137110746, 137110748, 137110754, 137110785, 137112109, 137112787, 137112795, 137112826, 137115421, 137115429, 137115458, 137115460, 137115466, 137115497, 137117499, 137117507, 137117538, 137120761, 137122211, 137122219, 137122250, 137133883, 137153835, 137153872, 137153909, 137155913, 137155950, 137157991, 137160625, 137160662, 137162703, 137167415, 137202129, 137219920, 137249435, 137249443, 137249474, 137290812, 137315575, 137315612, 137317653, 137322365, 137331017, 137357105, 137386467, 137386504, 137388545, 137393257, 137401909, 137411397, 137411405, 137411434, 137411436, 137411442, 137411471, 137411473, 137411479, 137411510, 137413475, 137413483, 137413512, 137413514, 137413520, 137413551, 137414875, 137415553, 137415561, 137415592, 137418187, 137418195, 137418224, 137418226, 137418232, 137418263, 137420265, 137420273, 137420304, 137424977, 137424985, 137425016, 137463391, 137463428, 137465469, 137470181, 137571871, 137571908, 137571945, 137571982, 137573949, 137573986, 137574023, 137576027, 137576064, 137578105, 137578661, 137578698, 137578735, 137580739, 137580776, 137582817, 137585451, 137585488, 137587529, 137592241, 137625131, 137658205, 137658242, 137658279, 137660283, 137660320, 137662361, 137664995, 137665032, 137667073, 137671785, 137673647, 137673684, 137675725, 137680437, 137696023, 137720953, 137720961, 137720990, 137720992, 137720998, 137721029, 137723031, 137723039, 137723070, 137727743, 137727751, 137727782, 137744539, 137744576, 137746617, 137751329, 137759981, 137772947, 137881427, 137881464, 137881501, 137882915, 137882923, 137882952, 137882954, 137882960, 137882989, 137882991, 137882997, 137883026, 137883028, 137883034, 137883065, 137883505, 137883542, 137884993, 137885001, 137885030, 137885032, 137885038, 137885067, 137885069, 137885075, 137885106, 137885583, 137886393, 137886430, 137887071, 137887079, 137887108, 137887110, 137887116, 137887147, 137888217, 137888254, 137888471, 137889149, 137889157, 137889188, 137889705, 137889713, 137889742, 137889744, 137889750, 137889779, 137889781, 137889787, 137889818, 137890295, 137891783, 137891791, 137891820, 137891822, 137891828, 137891859, 137893183, 137893861, 137893869, 137893900, 137895007, 137896495, 137896503, 137896532, 137896534, 137896540, 137896571, 137898573, 137898581, 137898612, 137901835, 137903285, 137903293, 137903324, 137967761, 137967798, 137969839, 137974551, 137983203, 138030509, 138030517, 138030548, 138054095, 138118053, 138138179, 138142689, 138142726, 138142763, 138142800, 138144767, 138144804, 138144841, 138146845, 138146882, 138148923, 138149479, 138149516, 138149553, 138151557, 138151594, 138153635, 138156269, 138156306, 138158347, 138163059, 138190983, 138191020, 138192471, 138192479, 138192508, 138192510, 138192516, 138192545, 138192547, 138192553, 138192584, 138193061, 138194549, 138194557, 138194586, 138194588, 138194594, 138194625, 138195949, 138196627, 138196635, 138196666, 138197773, 138199261, 138199269, 138199298, 138199300, 138199306, 138199337, 138201339, 138201347, 138201378, 138206051, 138206059, 138206090, 138209071, 138225545, 138225553, 138225582, 138225584, 138225590, 138225619, 138225621, 138225627, 138225656, 138225658, 138225664, 138225693, 138225695, 138225701, 138225732, 138227623, 138227631, 138227660, 138227662, 138227668, 138227697, 138227699, 138227705, 138227734, 138227736, 138227742, 138227773, 138229023, 138229060, 138229097, 138229701, 138229709, 138229738, 138229740, 138229746, 138229775, 138229777, 138229783, 138229814, 138231101, 138231138, 138231779, 138231787, 138231816, 138231818, 138231824, 138231855, 138232335, 138232343, 138232372, 138232374, 138232380, 138232409, 138232411, 138232417, 138232446, 138232448, 138232454, 138232485, 138233179, 138233857, 138233865, 138233896, 138234413, 138234421, 138234450, 138234452, 138234458, 138234487, 138234489, 138234495, 138234526, 138235813, 138235850, 138236491, 138236499, 138236528, 138236530, 138236536, 138236567, 138237891, 138238569, 138238577, 138238608, 138239125, 138239133, 138239162, 138239164, 138239170, 138239199, 138239201, 138239207, 138239238, 138241203, 138241211, 138241240, 138241242, 138241248, 138241279, 138242603, 138243281, 138243289, 138243320, 138244465, 138244502, 138245915, 138245923, 138245952, 138245954, 138245960, 138245991, 138246543, 138247993, 138248001, 138248032, 138251255, 138252705, 138252713, 138252744, 138277317, 138406205, 138452245, 138452282, 138452319, 138454323, 138454360, 138456401, 138459035, 138459072, 138461113, 138465825, 138477097, 138496251, 138500539, 138502027, 138502035, 138502064, 138502066, 138502072, 138502103, 138504105, 138504113, 138504144, 138508817, 138508825, 138508856, 138535101, 138535109, 138535138, 138535140, 138535146, 138535175, 138535177, 138535183, 138535212, 138535214, 138535220, 138535251, 138537179, 138537187, 138537216, 138537218, 138537224, 138537253, 138537255, 138537261, 138537292, 138538579, 138538616, 138539257, 138539265, 138539294, 138539296, 138539302, 138539333, 138540657, 138541335, 138541343, 138541374, 138541891, 138541899, 138541928, 138541930, 138541936, 138541965, 138541967, 138541973, 138542004, 138543969, 138543977, 138544006, 138544008, 138544014, 138544045, 138545369, 138546047, 138546055, 138546086, 138548681, 138548689, 138548718, 138548720, 138548726, 138548757, 138550759, 138550767, 138550798, 138554021, 138555471, 138555479, 138555510, 138653180, 138662501, 138662538, 138662575, 138664579, 138664616, 138666657, 138669291, 138669328, 138671369, 138676081, 138733393, 138733430, 138733467, 138735471, 138735508, 138737549, 138740183, 138740220, 138742261, 138746973, 138748835, 138748872, 138750913, 138755625, 138761801, 138761838, 138763879, 138764277, 138768591, 138811583, 138811591, 138811622, 138819727, 138819764, 138821805, 138826517, 138835169, 138844657, 138844665, 138844694, 138844696, 138844702, 138844731, 138844733, 138844739, 138844770, 138846735, 138846743, 138846772, 138846774, 138846780, 138846811, 138848135, 138848813, 138848821, 138848852, 138851447, 138851455, 138851484, 138851486, 138851492, 138851523, 138853525, 138853533, 138853564, 138858237, 138858245, 138858276, 138972057, 138972094, 138973545, 138973553, 138973582, 138973584, 138973590, 138973619, 138973621, 138973627, 138973658, 138974135, 138975623, 138975631, 138975660, 138975662, 138975668, 138975699, 138977023, 138977701, 138977709, 138977740, 138978847, 138980335, 138980343, 138980372, 138980374, 138980380, 138980411, 138982413, 138982421, 138982452, 138987125, 138987133, 138987164, 139042949, 139042986, 139045027, 139049739, 139058391, 139071357, 139106907, 139106944, 139108985, 139113697, 139129283, 139154213, 139154221, 139154250, 139154252, 139154258, 139154289, 139156291, 139156299, 139156330, 139161003, 139161011, 139161042, 139193241, 139233319, 139233356, 139233393, 139235397, 139235434, 139237475, 139240109, 139240146, 139242187, 139246899, 139281613, 139283101, 139283109, 139283138, 139283140, 139283146, 139283177, 139285179, 139285187, 139285218, 139289891, 139289899, 139289930, 139300733, 139300741, 139300770, 139300772, 139300778, 139300807, 139300809, 139300815, 139300844, 139300846, 139300852, 139300881, 139300883, 139300889, 139300920, 139302811, 139302819, 139302848, 139302850, 139302856, 139302885, 139302887, 139302893, 139302922, 139302924, 139302930, 139302961, 139304211, 139304248, 139304285, 139304889, 139304897, 139304926, 139304928, 139304934, 139304963, 139304965, 139304971, 139305002, 139306289, 139306326, 139306967, 139306975, 139307004, 139307006, 139307012, 139307043, 139307523, 139307531, 139307560, 139307562, 139307568, 139307597, 139307599, 139307605, 139307634, 139307636, 139307642, 139307673, 139308367, 139309045, 139309053, 139309084, 139309601, 139309609, 139309638, 139309640, 139309646, 139309675, 139309677, 139309683, 139309714, 139311001, 139311038, 139311679, 139311687, 139311716, 139311718, 139311724, 139311755, 139313079, 139313757, 139313765, 139313796, 139314313, 139314321, 139314350, 139314352, 139314358, 139314387, 139314389, 139314395, 139314426, 139316175, 139316183, 139316212, 139316214, 139316220, 139316249, 139316251, 139316257, 139316286, 139316288, 139316294, 139316325, 139316391, 139316399, 139316428, 139316430, 139316436, 139316467, 139317791, 139318253, 139318261, 139318290, 139318292, 139318298, 139318327, 139318329, 139318335, 139318366, 139318469, 139318477, 139318508, 139319653, 139319690, 139320331, 139320339, 139320368, 139320370, 139320376, 139320407, 139321103, 139321111, 139321140, 139321142, 139321148, 139321179, 139321731, 139322409, 139322417, 139322448, 139322965, 139322973, 139323002, 139323004, 139323010, 139323039, 139323041, 139323047, 139323078, 139323181, 139323189, 139323220, 139325043, 139325051, 139325080, 139325082, 139325088, 139325119, 139326443, 139327121, 139327129, 139327160, 139327893, 139327901, 139327932, 139329755, 139329763, 139329792, 139329794, 139329800, 139329831, 139331833, 139331841, 139331872, 139335095, 139336545, 139336553, 139336584, 139352505, 139416463, 139463769, 139463777, 139463808, 139485105, 139485142, 139487183, 139491895, 139542875, 139542912, 139544953, 139549665, 139571439, 139592657, 139592665, 139592696, 139610289, 139610297, 139610326, 139610328, 139610334, 139610363, 139610365, 139610371, 139610400, 139610402, 139610408, 139610439, 139612367, 139612375, 139612404, 139612406, 139612412, 139612441, 139612443, 139612449, 139612480, 139613767, 139613804, 139614445, 139614453, 139614482, 139614484, 139614490, 139614521, 139615845, 139616523, 139616531, 139616562, 139617079, 139617087, 139617116, 139617118, 139617124, 139617153, 139617155, 139617161, 139617192, 139619157, 139619165, 139619194, 139619196, 139619202, 139619233, 139620557, 139621235, 139621243, 139621274, 139623869, 139623877, 139623906, 139623908, 139623914, 139623945, 139625731, 139625739, 139625768, 139625770, 139625776, 139625805, 139625807, 139625813, 139625844, 139625947, 139625955, 139625986, 139627809, 139627817, 139627846, 139627848, 139627854, 139627885, 139629209, 139629887, 139629895, 139629926, 139630659, 139630667, 139630698, 139632521, 139632529, 139632558, 139632560, 139632566, 139632597, 139634599, 139634607, 139634638, 139639311, 139639319, 139639350, 139677725, 139677762, 139679803, 139684515, 139728368, 139753131, 139753168, 139755209, 139759921, 139794661, 139824023, 139824060, 139826101, 139830813, 139839465, 139852431, 139894915, 139894952, 139896993, 139901705, 139910357, 139919845, 139919853, 139919882, 139919884, 139919890, 139919919, 139919921, 139919927, 139919958, 139921923, 139921931, 139921960, 139921962, 139921968, 139921999, 139923323, 139924001, 139924009, 139924040, 139926635, 139926643, 139926672, 139926674, 139926680, 139926711, 139928713, 139928721, 139928752, 139933425, 139933433, 139933464, 139935287, 139935295, 139935324, 139935326, 139935332, 139935363, 139937365, 139937373, 139937404, 139942077, 139942085, 139942116, 139987281, 140062687, 140064175, 140064183, 140064212, 140064214, 140064220, 140064251, 140066253, 140066261, 140066292, 140070965, 140070973, 140071004, 140095761, 140095798, 140095835, 140097839, 140097876, 140099917, 140102551, 140102588, 140104629, 140109341, 140133579, 140182095, 140182132, 140184173, 140188885, 140197537, 140204471, 140229401, 140229409, 140229438, 140229440, 140229446, 140229477, 140231479, 140231487, 140231518, 140236191, 140236199, 140236230, 140244843, 140244851, 140244882, 140268429, 140323949, 140323986, 140326027, 140330739, 140373731, 140373739, 140373770, 140391363, 140391371, 140391400, 140391402, 140391408, 140391437, 140391439, 140391445, 140391474, 140391476, 140391482, 140391513, 140393441, 140393449, 140393478, 140393480, 140393486, 140393515, 140393517, 140393523, 140393554, 140394841, 140394878, 140395519, 140395527, 140395556, 140395558, 140395564, 140395595, 140396919, 140397597, 140397605, 140397636, 140398153, 140398161, 140398190, 140398192, 140398198, 140398227, 140398229, 140398235, 140398266, 140400231, 140400239, 140400268, 140400270, 140400276, 140400307, 140401631, 140402309, 140402317, 140402348, 140404943, 140404951, 140404980, 140404982, 140404988, 140405019, 140405317, 140405354, 140406805, 140406813, 140406842, 140406844, 140406850, 140406879, 140406881, 140406887, 140406918, 140407021, 140407029, 140407060, 140407395, 140408883, 140408891, 140408920, 140408922, 140408928, 140408959, 140410283, 140410961, 140410969, 140411000, 140411733, 140411741, 140411772, 140412107, 140413595, 140413603, 140413632, 140413634, 140413640, 140413671, 140415673, 140415681, 140415712, 140420385, 140420393, 140420424, 140491651, 140538957, 140538965, 140538996, 140575735, 140633505, 140646627, 140666579, 140666616, 140666653, 140668657, 140668694, 140670735, 140673369, 140673406, 140675447, 140680159, 140700919, 140700927, 140700956, 140700958, 140700964, 140700993, 140700995, 140701001, 140701032, 140702997, 140703005, 140703034, 140703036, 140703042, 140703073, 140704397, 140705075, 140705083, 140705114, 140707709, 140707717, 140707746, 140707748, 140707754, 140707785, 140709787, 140709795, 140709826, 140714499, 140714507, 140714538, 140714873, 140716361, 140716369, 140716398, 140716400, 140716406, 140716437, 140718439, 140718447, 140718478, 140723151, 140723159, 140723190, 140749435, 140749443, 140749472, 140749474, 140749480, 140749509, 140749511, 140749517, 140749546, 140749548, 140749554, 140749585, 140751513, 140751521, 140751550, 140751552, 140751558, 140751587, 140751589, 140751595, 140751626, 140752913, 140752950, 140753591, 140753599, 140753628, 140753630, 140753636, 140753667, 140754991, 140755669, 140755677, 140755708, 140756225, 140756233, 140756262, 140756264, 140756270, 140756299, 140756301, 140756307, 140756338, 140758303, 140758311, 140758340, 140758342, 140758348, 140758379, 140759703, 140760381, 140760389, 140760420, 140763015, 140763023, 140763052, 140763054, 140763060, 140763091, 140765093, 140765101, 140765132, 140768355, 140769805, 140769813, 140769844, 140843761, 140914653, 140976135, 140976172, 140978213, 140982925, 140985545, 141010475, 141010483, 141010512, 141010514, 141010520, 141010551, 141012553, 141012561, 141012592, 141017265, 141017273, 141017304, 141025917, 141025925, 141025956, 141058991, 141058999, 141059028, 141059030, 141059036, 141059065, 141059067, 141059073, 141059104, 141061069, 141061077, 141061106, 141061108, 141061114, 141061145, 141062469, 141063147, 141063155, 141063186, 141065781, 141065789, 141065818, 141065820, 141065826, 141065857, 141067859, 141067867, 141067898, 141072571, 141072579, 141072610, 141154805, 141154813, 141154844, 141170949, 141170986, 141171023, 141173027, 141173064, 141175105, 141177739, 141177776, 141179817, 141184529, 141186391, 141186428, 141188469, 141193181, 141257283, 141257320, 141259361, 141264073, 141272725, 141285691, 141320031, 141320039, 141320070, 141343617, 141368547, 141368555, 141368584, 141368586, 141368592, 141368623, 141370625, 141370633, 141370664, 141375337, 141375345, 141375376, 141414579, 141480505, 141480542, 141481993, 141482001, 141482030, 141482032, 141482038, 141482067, 141482069, 141482075, 141482106, 141482583, 141484071, 141484079, 141484108, 141484110, 141484116, 141484147, 141485471, 141486149, 141486157, 141486188, 141487295, 141488783, 141488791, 141488820, 141488822, 141488828, 141488859, 141490861, 141490869, 141490900, 141495573, 141495581, 141495612, 141495947, 141497435, 141497443, 141497472, 141497474, 141497480, 141497511, 141499513, 141499521, 141499552, 141504225, 141504233, 141504264, 141566839, 141630797, 141678103, 141678111, 141678142, 141738289, 141738297, 141738326, 141738328, 141738334, 141738363, 141738365, 141738371, 141738400, 141738402, 141738408, 141738437, 141738439, 141738445, 141738476, 141740367, 141740375, 141740404, 141740406, 141740412, 141740441, 141740443, 141740449, 141740478, 141740480, 141740486, 141740517, 141741767, 141741804, 141741841, 141742445, 141742453, 141742482, 141742484, 141742490, 141742519, 141742521, 141742527, 141742558, 141743845, 141743882, 141744523, 141744531, 141744560, 141744562, 141744568, 141744599, 141745079, 141745087, 141745116, 141745118, 141745124, 141745153, 141745155, 141745161, 141745190, 141745192, 141745198, 141745229, 141745923, 141746601, 141746609, 141746640, 141747157, 141747165, 141747194, 141747196, 141747202, 141747231, 141747233, 141747239, 141747270, 141748557, 141748594, 141749235, 141749243, 141749272, 141749274, 141749280, 141749311, 141750635, 141751313, 141751321, 141751352, 141751869, 141751877, 141751906, 141751908, 141751914, 141751943, 141751945, 141751951, 141751982, 141753947, 141753955, 141753984, 141753986, 141753992, 141754023, 141755347, 141756025, 141756033, 141756064, 141757209, 141757246, 141758659, 141758667, 141758696, 141758698, 141758704, 141758735, 141759287, 141760737, 141760745, 141760776, 141763999, 141765449, 141765457, 141765488, 141790061, 141791549, 141791557, 141791586, 141791588, 141791594, 141791625, 141793627, 141793635, 141793666, 141798339, 141798347, 141798378, 141806991, 141806999, 141807030, 141824623, 141824631, 141824660, 141824662, 141824668, 141824697, 141824699, 141824705, 141824734, 141824736, 141824742, 141824773, 141826701, 141826709, 141826738, 141826740, 141826746, 141826775, 141826777, 141826783, 141826814, 141828101, 141828138, 141828779, 141828787, 141828816, 141828818, 141828824, 141828855, 141830179, 141830857, 141830865, 141830896, 141831413, 141831421, 141831450, 141831452, 141831458, 141831487, 141831489, 141831495, 141831526, 141833491, 141833499, 141833528, 141833530, 141833536, 141833567, 141834891, 141835569, 141835577, 141835608, 141838203, 141838211, 141838240, 141838242, 141838248, 141838279, 141840065, 141840073, 141840102, 141840104, 141840110, 141840139, 141840141, 141840147, 141840178, 141840281, 141840289, 141840320, 141842143, 141842151, 141842180, 141842182, 141842188, 141842219, 141843543, 141844221, 141844229, 141844260, 141844993, 141845001, 141845032, 141846855, 141846863, 141846892, 141846894, 141846900, 141846931, 141848933, 141848941, 141848972, 141853645, 141853653, 141853684, 142008995, 142047845, 142047853, 142047882, 142047884, 142047890, 142047919, 142047921, 142047927, 142047956, 142047958, 142047964, 142047995, 142049923, 142049931, 142049960, 142049962, 142049968, 142049997, 142049999, 142050005, 142050036, 142051323, 142051360, 142052001, 142052009, 142052038, 142052040, 142052046, 142052077, 142053401, 142054079, 142054087, 142054118, 142054635, 142054643, 142054672, 142054674, 142054680, 142054709, 142054711, 142054717, 142054748, 142056713, 142056721, 142056750, 142056752, 142056758, 142056789, 142058113, 142058791, 142058799, 142058830, 142061425, 142061433, 142061462, 142061464, 142061470, 142061501, 142063503, 142063511, 142063542, 142066765, 142068215, 142068223, 142068254, 142101105, 142101113, 142101144, 142134179, 142134187, 142134216, 142134218, 142134224, 142134253, 142134255, 142134261, 142134292, 142136257, 142136265, 142136294, 142136296, 142136302, 142136333, 142137657, 142138335, 142138343, 142138374, 142140969, 142140977, 142141006, 142141008, 142141014, 142141045, 142143047, 142143055, 142143086, 142147759, 142147767, 142147798, 142149621, 142149629, 142149658, 142149660, 142149666, 142149697, 142151699, 142151707, 142151738, 142156411, 142156419, 142156450, 142165924, 142201615, 142261579, 142261616, 142263657, 142266089, 142266126, 142266163, 142266200, 142266237, 142268167, 142268204, 142268241, 142268278, 142268369, 142270245, 142270282, 142270319, 142272323, 142272360, 142272879, 142272916, 142272953, 142272990, 142274401, 142274957, 142274994, 142275031, 142277021, 142277035, 142277072, 142279113, 142279669, 142279706, 142279743, 142281747, 142281784, 142283825, 142286459, 142286496, 142288537, 142293249, 142332471, 142332508, 142334549, 142339261, 142347913, 142357401, 142357409, 142357438, 142357440, 142357446, 142357475, 142357477, 142357483, 142357514, 142359479, 142359487, 142359516, 142359518, 142359524, 142359555, 142360879, 142361557, 142361565, 142361596, 142364191, 142364199, 142364228, 142364230, 142364236, 142364267, 142366269, 142366277, 142366308, 142370981, 142370989, 142371020, 142418805, 142443735, 142443743, 142443772, 142443774, 142443780, 142443811, 142445813, 142445821, 142445852, 142450525, 142450533, 142450564, 142459177, 142459185, 142459216, 142571135, 142572623, 142572631, 142572660, 142572662, 142572668, 142572699, 142574701, 142574709, 142574740, 142575645, 142575682, 142575719, 142575756, 142577723, 142577760, 142577797, 142579413, 142579421, 142579452, 142579801, 142579838, 142581879, 142582435, 142582472, 142582509, 142584513, 142584550, 142586591, 142588065, 142588073, 142588104, 142589225, 142589262, 142591303, 142596015, 142619651, 142619688, 142621729, 142626441, 142642027, 142666957, 142666965, 142666994, 142666996, 142667002, 142667033, 142669035, 142669043, 142669074, 142673747, 142673755, 142673786, 142705985, 142753291, 142753299, 142753330, 142828919, 142828927, 142828956, 142828958, 142828964, 142828993, 142828995, 142829001, 142829030, 142829032, 142829038, 142829069, 142830997, 142831005, 142831034, 142831036, 142831042, 142831071, 142831073, 142831079, 142831110, 142832397, 142832434, 142833075, 142833083, 142833112, 142833114, 142833120, 142833151, 142834475, 142835153, 142835161, 142835192, 142835709, 142835717, 142835746, 142835748, 142835754, 142835783, 142835785, 142835791, 142835822, 142837787, 142837795, 142837824, 142837826, 142837832, 142837863, 142839187, 142839865, 142839873, 142839904, 142842499, 142842507, 142842536, 142842538, 142842544, 142842575, 142844577, 142844585, 142844616, 142847839, 142849289, 142849297, 142849328, 142882179, 142882187, 142882218, 142885201, 142885238, 142885275, 142887279, 142887316, 142889357, 142891991, 142892028, 142894069, 142898781, 142899811, 142899819, 142899848, 142899850, 142899856, 142899885, 142899887, 142899893, 142899922, 142899924, 142899930, 142899961, 142901889, 142901897, 142901926, 142901928, 142901934, 142901963, 142901965, 142901971, 142902002, 142903289, 142903326, 142903967, 142903975, 142904004, 142904006, 142904012, 142904043, 142905367, 142906045, 142906053, 142906084, 142906601, 142906609, 142906638, 142906640, 142906646, 142906675, 142906677, 142906683, 142906714, 142908679, 142908687, 142908716, 142908718, 142908724, 142908755, 142910079, 142910757, 142910765, 142910796, 142913391, 142913399, 142913428, 142913430, 142913436, 142913467, 142915253, 142915261, 142915290, 142915292, 142915298, 142915327, 142915329, 142915335, 142915366, 142915469, 142915477, 142915508, 142917331, 142917339, 142917368, 142917370, 142917376, 142917407, 142918731, 142919409, 142919417, 142919448, 142920181, 142920189, 142920220, 142922043, 142922051, 142922080, 142922082, 142922088, 142922119, 142924121, 142924129, 142924160, 142928833, 142928841, 142928872, 142929207, 142930695, 142930703, 142930732, 142930734, 142930740, 142930771, 142932773, 142932781, 142932812, 142937485, 142937493, 142937524, 142976513, 142976521, 142976552, 143084183, 143138475, 143138483, 143138512, 143138514, 143138520, 143138549, 143138551, 143138557, 143138588, 143140553, 143140561, 143140590, 143140592, 143140598, 143140629, 143141953, 143142631, 143142639, 143142670, 143145265, 143145273, 143145302, 143145304, 143145310, 143145341, 143147343, 143147351, 143147382, 143152055, 143152063, 143152094, 143190469, 143190506, 143192547, 143194757, 143194794, 143196835, 143197259, 143201547, 143209367, 143209375, 143209404, 143209406, 143209412, 143209441, 143209443, 143209449, 143209480, 143211445, 143211453, 143211482, 143211484, 143211490, 143211521, 143212845, 143213523, 143213531, 143213562, 143216157, 143216165, 143216194, 143216196, 143216202, 143216233, 143218235, 143218243, 143218274, 143222947, 143222955, 143222986, 143224809, 143224817, 143224846, 143224848, 143224854, 143224885, 143226887, 143226895, 143226926, 143231599, 143231607, 143231638, 143240251, 143240259, 143240290, 143273325, 143273333, 143273362, 143273364, 143273370, 143273399, 143273401, 143273407, 143273438, 143275403, 143275411, 143275440, 143275442, 143275448, 143275479, 143276803, 143277481, 143277489, 143277520, 143280115, 143280123, 143280152, 143280154, 143280160, 143280191, 143282193, 143282201, 143282232, 143286905, 143286913, 143286944, 143352209, 143356719, 143356756, 143356793, 143356830, 143358797, 143358834, 143358871, 143360875, 143360912, 143362953, 143363509, 143363546, 143363583, 143365587, 143365624, 143367665, 143370299, 143370336, 143372377, 143377089, 143423101, 143448031, 143448039, 143448068, 143448070, 143448076, 143448107, 143450109, 143450117, 143450148, 143454821, 143454829, 143454860, 143493993, 143500025, 143504313, 143518923, 143518931, 143518960, 143518962, 143518968, 143518999, 143521001, 143521009, 143521040, 143525713, 143525721, 143525752, 143534365, 143534373, 143534404, 143582881, 143582889, 143582918, 143582920, 143582926, 143582957, 143584959, 143584967, 143584998, 143589671, 143589679, 143589710, 143608505, 143608542, 143608579, 143610583, 143610620, 143612661, 143615295, 143615332, 143617373, 143622085, 143663253, 143663261, 143663292, 143666275, 143666312, 143666349, 143668353, 143668390, 143670431, 143673065, 143673102, 143675143, 143679855, 143694839, 143694876, 143696917, 143701629, 143710281, 143757587, 143757595, 143757626, 143781173, 143828479, 143828487, 143828518, 143892437, 143892445, 143892476, 143918061, 143918098, 143919549, 143919557, 143919586, 143919588, 143919594, 143919623, 143919625, 143919631, 143919662, 143920139, 143921627, 143921635, 143921664, 143921666, 143921672, 143921703, 143923027, 143923705, 143923713, 143923744, 143924851, 143926339, 143926347, 143926376, 143926378, 143926384, 143926415, 143928417, 143928425, 143928456, 143933129, 143933137, 143933168, 143975831, 143975868, 143977909, 143982621, 143990441, 143990449, 143990478, 143990480, 143990486, 143990515, 143990517, 143990523, 143990554, 143992519, 143992527, 143992556, 143992558, 143992564, 143992595, 143993919, 143994597, 143994605, 143994636, 143997231, 143997239, 143997268, 143997270, 143997276, 143997307, 143999309, 143999317, 143999348, 144004021, 144004029, 144004060, 144004395, 144005883, 144005891, 144005920, 144005922, 144005928, 144005959, 144007961, 144007969, 144008000, 144012673, 144012681, 144012712, 144021325, 144021333, 144021364, 144179323, 144179360, 144179397, 144181401, 144181438, 144183479, 144186113, 144186150, 144188191, 144192903, 144227617, 144229105, 144229113, 144229142, 144229144, 144229150, 144229181, 144231183, 144231191, 144231222, 144235895, 144235903, 144235934, 144262179, 144262187, 144262216, 144262218, 144262224, 144262253, 144262255, 144262261, 144262290, 144262292, 144262298, 144262329, 144264257, 144264265, 144264294, 144264296, 144264302, 144264331, 144264333, 144264339, 144264370, 144265657, 144265694, 144266335, 144266343, 144266372, 144266374, 144266380, 144266411, 144267735, 144268413, 144268421, 144268452, 144268969, 144268977, 144269006, 144269008, 144269014, 144269043, 144269045, 144269051, 144269082, 144271047, 144271055, 144271084, 144271086, 144271092, 144271123, 144272447, 144273125, 144273133, 144273164, 144275759, 144275767, 144275796, 144275798, 144275804, 144275835, 144277837, 144277845, 144277876, 144281099, 144282549, 144282557, 144282588, 144285387, 144299997, 144300005, 144300034, 144300036, 144300042, 144300073, 144302075, 144302083, 144302114, 144306787, 144306795, 144306826, 144315439, 144315447, 144315478, 144348513, 144348521, 144348550, 144348552, 144348558, 144348587, 144348589, 144348595, 144348626, 144350591, 144350599, 144350628, 144350630, 144350636, 144350667, 144351991, 144352669, 144352677, 144352708, 144355303, 144355311, 144355340, 144355342, 144355348, 144355379, 144357381, 144357389, 144357420, 144362093, 144362101, 144362132, 144363955, 144363963, 144363992, 144363994, 144364000, 144364031, 144366033, 144366041, 144366072, 144370745, 144370753, 144370784, 144447349, 144447386, 144447423, 144449427, 144449464, 144451505, 144454139, 144454176, 144456217, 144460929, 144488879, 144488916, 144490957, 144495669, 144538661, 144538669, 144538700, 144571735, 144571743, 144571772, 144571774, 144571780, 144571809, 144571811, 144571817, 144571848, 144573813, 144573821, 144573850, 144573852, 144573858, 144573889, 144575213, 144575891, 144575899, 144575930, 144578525, 144578533, 144578562, 144578564, 144578570, 144578601, 144580603, 144580611, 144580642, 144585315, 144585323, 144585354, 144609553, 144609561, 144609592, 144658069, 144658077, 144658106, 144658108, 144658114, 144658145, 144660147, 144660155, 144660186, 144664859, 144664867, 144664898, 144673511, 144673519, 144673550, 144699135, 144699172, 144701213, 144705925, 144756905, 144756942, 144758983, 144763695, 144770027, 144770064, 144772105, 144776817, 144785469, 144789979, 144790016, 144790053, 144790090, 144792057, 144792094, 144792131, 144794135, 144794172, 144796213, 144796769, 144796806, 144796843, 144798435, 144798847, 144798884, 144800925, 144803559, 144803596, 144805637, 144810349, 144856361, 144881291, 144881299, 144881328, 144881330, 144881336, 144881367, 144883369, 144883377, 144883408, 144888081, 144888089, 144888120, 144967625, 144967633, 144967664, 145008691, 145010179, 145010187, 145010216, 145010218, 145010224, 145010255, 145012257, 145012265, 145012296, 145016969, 145016977, 145017008, 145066461, 145079583, 145081071, 145081079, 145081108, 145081110, 145081116, 145081147, 145083149, 145083157, 145083188, 145087861, 145087869, 145087900, 145096513, 145096521, 145096552, 145099535, 145099572, 145099609, 145101613, 145101650, 145103691, 145106325, 145106362, 145108403, 145113115, 145143541, 145190847, 145190855, 145190886, 145269953, 145269990, 145272031, 145276743, 145319735, 145319743, 145319774, 145337367, 145337375, 145337404, 145337406, 145337412, 145337441, 145337443, 145337449, 145337478, 145337480, 145337486, 145337517, 145339445, 145339453, 145339482, 145339484, 145339490, 145339519, 145339521, 145339527, 145339558, 145340845, 145340882, 145341523, 145341531, 145341560, 145341562, 145341568, 145341599, 145342923, 145343601, 145343609, 145343640, 145344157, 145344165, 145344194, 145344196, 145344202, 145344231, 145344233, 145344239, 145344270, 145346235, 145346243, 145346272, 145346274, 145346280, 145346311, 145347635, 145348313, 145348321, 145348352, 145350947, 145350955, 145350984, 145350986, 145350992, 145351023, 145352809, 145352817, 145352846, 145352848, 145352854, 145352883, 145352885, 145352891, 145352922, 145353025, 145353033, 145353064, 145354887, 145354895, 145354924, 145354926, 145354932, 145354963, 145356287, 145356965, 145356973, 145357004, 145357737, 145357745, 145357776, 145359599, 145359607, 145359636, 145359638, 145359644, 145359675, 145361677, 145361685, 145361716, 145366389, 145366397, 145366428, 145390627, 145390635, 145390666, 145409091, 145409128, 145411169, 145415881, 145423701, 145423709, 145423738, 145423740, 145423746, 145423775, 145423777, 145423783, 145423814, 145425779, 145425787, 145425816, 145425818, 145425824, 145425855, 145427179, 145427857, 145427865, 145427896, 145430491, 145430499, 145430528, 145430530, 145430536, 145430567, 145432569, 145432577, 145432608, 145437281, 145437289, 145437320, 145439143, 145439151, 145439180, 145439182, 145439188, 145439219, 145441221, 145441229, 145441260, 145445933, 145445941, 145445972, 145454585, 145454593, 145454624, 145521739, 145537979, 145538016, 145540057, 145544769, 145579509, 145646923, 145646931, 145646960, 145646962, 145646968, 145646997, 145646999, 145647005, 145647036, 145649001, 145649009, 145649038, 145649040, 145649046, 145649077, 145650401, 145651079, 145651087, 145651118, 145653713, 145653721, 145653750, 145653752, 145653758, 145653789, 145655791, 145655799, 145655830, 145660503, 145660511, 145660542, 145662365, 145662373, 145662402, 145662404, 145662410, 145662441, 145664443, 145664451, 145664482, 145669155, 145669163, 145669194, 145714359, 145718647, 145733257, 145733265, 145733294, 145733296, 145733302, 145733333, 145735335, 145735343, 145735374, 145740047, 145740055, 145740086, 145748699, 145748707, 145748738, 145789765, 145797215, 145797223, 145797252, 145797254, 145797260, 145797291, 145799293, 145799301, 145799332, 145804005, 145804013, 145804044, 145847535, 145860657, 145865167, 145865204, 145865241, 145865278, 145867245, 145867282, 145867319, 145869323, 145869360, 145871401, 145871957, 145871994, 145872031, 145874035, 145874072, 145876113, 145878747, 145878784, 145880609, 145880646, 145880683, 145880825, 145882687, 145882724, 145884765, 145885537, 145887399, 145887436, 145889477, 145894189, 145931549, 145956479, 145956487, 145956516, 145956518, 145956524, 145956555, 145958557, 145958565, 145958596, 145963269, 145963277, 145963308, 145971921, 145971929, 145971960, 146042813, 146042821, 146042852, 146100809, 146100817, 146100848, 146106771, 146106779, 146106810, 146132395, 146132432, 146134473, 146139185, 146171701, 146171709, 146171740, 146174723, 146174760, 146174797, 146176801, 146176838, 146178879, 146181513, 146181550, 146183591, 146188303, 146190165, 146190202, 146192243, 146196955, 146218729, 146266035, 146266043, 146266074, 146360583, 146427997, 146428005, 146428034, 146428036, 146428042, 146428071, 146428073, 146428079, 146428110, 146430075, 146430083, 146430112, 146430114, 146430120, 146430151, 146431475, 146432153, 146432161, 146432192, 146434787, 146434795, 146434824, 146434826, 146434832, 146434863, 146436865, 146436873, 146436904, 146441577, 146441585, 146441616, 146441951, 146443439, 146443447, 146443476, 146443478, 146443484, 146443515, 146445517, 146445525, 146445556, 146450229, 146450237, 146450268, 146484279, 146484316, 146486357, 146491069, 146498889, 146498897, 146498926, 146498928, 146498934, 146498963, 146498965, 146498971, 146499002, 146499721, 146500967, 146500975, 146501004, 146501006, 146501012, 146501043, 146502367, 146503045, 146503053, 146503084, 146505679, 146505687, 146505716, 146505718, 146505724, 146505755, 146507757, 146507765, 146507796, 146512469, 146512477, 146512508, 146514331, 146514339, 146514368, 146514370, 146514376, 146514407, 146516409, 146516417, 146516448, 146521121, 146521129, 146521160, 146529773, 146529781, 146529812, 146628609, 146703213, 146703250, 146705291, 146710003, 146737553, 146737561, 146737590, 146737592, 146737598, 146737629, 146739631, 146739639, 146739670, 146744343, 146744351, 146744382, 146752995, 146753003, 146753034, 146786069, 146786077, 146786106, 146786108, 146786114, 146786143, 146786145, 146786151, 146786182, 146788147, 146788155, 146788184, 146788186, 146788192, 146788223, 146789547, 146790225, 146790233, 146790264, 146792859, 146792867, 146792896, 146792898, 146792904, 146792935, 146793835, 146794937, 146794945, 146794976, 146799649, 146799657, 146799688, 146808445, 146808453, 146808482, 146808484, 146808490, 146808521, 146810523, 146810531, 146810562, 146815235, 146815243, 146815274, 146823887, 146823895, 146823926, 146860142, 146860179, 146862220, 146866932, 146872403, 146872411, 146872440, 146872442, 146872448, 146872479, 146874481, 146874489, 146874520, 146879193, 146879201, 146879232, 146887845, 146887853, 146887884, 146955797, 146955834, 146955871, 146957875, 146957912, 146959953, 146962587, 146962624, 146964665, 146969377, 146971239, 146971276, 146973317, 146978029, 147012769, 147047109, 147047117, 147047148, 147095625, 147095633, 147095662, 147095664, 147095670, 147095701, 147097703, 147097711, 147097742, 147102415, 147102423, 147102454, 147118001, 147118009, 147118040, 147169698, 147181959, 147181967, 147181998, 147207583, 147207620, 147209661, 147214373, 147223025, 147265353, 147265390, 147267431, 147272143, 147280795, 147293917, 147313869, 147313906, 147313943, 147315947, 147315984, 147318025, 147320659, 147320696, 147322737, 147327449, 147405181, 147405189, 147405220, 147517139, 147518627, 147518635, 147518664, 147518666, 147518672, 147518703, 147520705, 147520713, 147520744, 147525417, 147525425, 147525456, 147534069, 147534077, 147534108, 147574909, 147589519, 147589527, 147589556, 147589558, 147589564, 147589595, 147591597, 147591605, 147591636, 147596309, 147596317, 147596348, 147604961, 147604969, 147605000, 147623425, 147623462, 147625503, 147630215, 147774923, 147774931, 147774960, 147774962, 147774968, 147774997, 147774999, 147775005, 147775034, 147775036, 147775042, 147775073, 147777001, 147777009, 147777038, 147777040, 147777046, 147777075, 147777077, 147777083, 147777114, 147778401, 147778438, 147779079, 147779087, 147779116, 147779118, 147779124, 147779155, 147780479, 147781157, 147781165, 147781196, 147781713, 147781721, 147781750, 147781752, 147781758, 147781787, 147781789, 147781795, 147781826, 147783791, 147783799, 147783828, 147783830, 147783836, 147783867, 147785191, 147785869, 147785877, 147785908, 147788503, 147788511, 147788540, 147788542, 147788548, 147788579, 147790581, 147790589, 147790620, 147793843, 147795293, 147795301, 147795332, 147828183, 147828191, 147828222, 147861257, 147861265, 147861294, 147861296, 147861302, 147861331, 147861333, 147861339, 147861370, 147863335, 147863343, 147863372, 147863374, 147863380, 147863411, 147864735, 147865413, 147865421, 147865452, 147868047, 147868055, 147868084, 147868086, 147868092, 147868123, 147870125, 147870133, 147870164, 147874837, 147874845, 147874876, 147876699, 147876707, 147876736, 147876738, 147876744, 147876775, 147878777, 147878785, 147878816, 147883489, 147883497, 147883528, 147899075, 147899083, 147899114, 147932981, 147947591, 147947599, 147947628, 147947630, 147947636, 147947667, 147949669, 147949677, 147949708, 147950772, 147954381, 147954389, 147954420, 147963033, 147963041, 147963072, 148046427, 148046464, 148048505, 148053217, 148061869, 148084479, 148084487, 148084516, 148084518, 148084524, 148084553, 148084555, 148084561, 148084592, 148086557, 148086565, 148086594, 148086596, 148086602, 148086633, 148087957, 148088635, 148088643, 148088674, 148091269, 148091277, 148091306, 148091308, 148091314, 148091345, 148093347, 148093355, 148093386, 148098059, 148098067, 148098098, 148170813, 148170821, 148170850, 148170852, 148170858, 148170889, 148172891, 148172899, 148172930, 148177603, 148177611, 148177642, 148186255, 148186263, 148186294, 148257147, 148257155, 148257186, 148298213, 148302723, 148302760, 148302797, 148302834, 148304801, 148304838, 148304875, 148306879, 148306916, 148308957, 148309513, 148309550, 148309587, 148311591, 148311628, 148313669, 148316303, 148316340, 148318381, 148321105, 148321113, 148321144, 148323093, 148355983, 148369105, 148389057, 148389094, 148389131, 148391135, 148391172, 148393213, 148394035, 148394043, 148394072, 148394074, 148394080, 148394111, 148395847, 148395884, 148396113, 148396121, 148396152, 148397925, 148400825, 148400833, 148400864, 148402637, 148404499, 148404536, 148406577, 148411289, 148480369, 148480377, 148480408, 148609257, 148609265, 148609296, 148612279, 148612316, 148612353, 148614357, 148614394, 148616435, 148619069, 148619106, 148621147, 148625859, 148656285, 148680149, 148680157, 148680188, 148698613, 148698650, 148700691, 148703591, 148703599, 148703630, 148705403, 148714055, 148865553, 148865561, 148865590, 148865592, 148865598, 148865627, 148865629, 148865635, 148865666, 148867631, 148867639, 148867668, 148867670, 148867676, 148867707, 148869031, 148869709, 148869717, 148869748, 148872343, 148872351, 148872380, 148872382, 148872388, 148872419, 148873541, 148873578, 148873615, 148873652, 148874421, 148874429, 148874460, 148875619, 148875656, 148875693, 148877697, 148877734, 148879133, 148879141, 148879172, 148879775, 148880331, 148880368, 148880405, 148882409, 148882446, 148884487, 148887121, 148887158, 148889199, 148893911, 148921835, 148921872, 148923913, 148928625, 148936445, 148936453, 148936482, 148936484, 148936490, 148936519, 148936521, 148936527, 148936558, 148938523, 148938531, 148938560, 148938562, 148938568, 148938599, 148939923, 148940601, 148940609, 148940640, 148943235, 148943243, 148943272, 148943274, 148943280, 148943311, 148945313, 148945321, 148945352, 148950025, 148950033, 148950064, 148951887, 148951895, 148951924, 148951926, 148951932, 148951963, 148953965, 148953973, 148954004, 148958677, 148958685, 148958716, 148967329, 148967337, 148967368, 149008169, 149022779, 149022787, 149022816, 149022818, 149022824, 149022855, 149024857, 149024865, 149024896, 149029569, 149029577, 149029608, 149038221, 149038229, 149038260, 149137057, 149175109, 149175117, 149175146, 149175148, 149175154, 149175185, 149177187, 149177195, 149177226, 149181899, 149181907, 149181938, 149183097, 149183134, 149183171, 149185175, 149185212, 149187253, 149189887, 149189924, 149191965, 149196677, 149227103, 149231391, 149246001, 149246009, 149246038, 149246040, 149246046, 149246077, 149248079, 149248087, 149248118, 149252791, 149252799, 149252830, 149261443, 149261451, 149261482, 149309959, 149309967, 149309996, 149309998, 149310004, 149310035, 149312037, 149312045, 149312076, 149316749, 149316757, 149316788, 149332335, 149332343, 149332374, 149384032, 149393353, 149393390, 149393427, 149395431, 149395468, 149396293, 149396301, 149396332, 149397509, 149400143, 149400180, 149402221, 149406933, 149464245, 149464282, 149464319, 149466323, 149466360, 149468401, 149471035, 149471072, 149473113, 149477825, 149479687, 149479724, 149481765, 149484665, 149484673, 149484704, 149486477, 149492653, 149492690, 149494731, 149495129, 149499443, 149555557, 149555565, 149555596, 149619515, 149619523, 149619554, 149645139, 149645176, 149647217, 149651929, 149702909, 149702946, 149704987, 149709699, 149731473, 149773801, 149773838, 149775879, 149780591, 149789243, 149802209, 149837759, 149837796, 149839837, 149844549, 149954695, 149956183, 149956191, 149956220, 149956222, 149956228, 149956259, 149958261, 149958269, 149958300, 149962973, 149962981, 149963012, 149964171, 149964208, 149964245, 149966249, 149966286, 149968327, 149970961, 149970998, 149973039, 149977751, 150012465, 150027075, 150027083, 150027112, 150027114, 150027120, 150027151, 150029153, 150029161, 150029192, 150033865, 150033873, 150033904, 150042517, 150042525, 150042556, 150083357, 150097967, 150097975, 150098004, 150098006, 150098012, 150098043, 150100045, 150100053, 150100084, 150104757, 150104765, 150104796, 150113409, 150113417, 150113448, 150147315, 150215957, 150215994, 150218035, 150222747, 150265739, 150265747, 150265778, 150273727, 150273764, 150275805, 150280517, 150298813, 150298821, 150298850, 150298852, 150298858, 150298887, 150298889, 150298895, 150298926, 150300891, 150300899, 150300928, 150300930, 150300936, 150300967, 150302291, 150302969, 150302977, 150303008, 150305603, 150305611, 150305640, 150305642, 150305648, 150305679, 150307681, 150307689, 150307720, 150312393, 150312401, 150312432, 150336631, 150336639, 150336670, 150385147, 150385155, 150385184, 150385186, 150385192, 150385223, 150387225, 150387233, 150387264, 150391937, 150391945, 150391976, 150400589, 150400597, 150400628, 150407523, 150407531, 150407562, 150459220, 150471481, 150471489, 150471520, 150483983, 150484020, 150486061, 150490773, 150525513, 150554875, 150554912, 150556953, 150561665, 150570317, 150583283, 150608369, 150608377, 150608406, 150608408, 150608414, 150608445, 150610447, 150610455, 150610486, 150615159, 150615167, 150615198, 150694703, 150694711, 150694742, 150735769, 150793539, 150806661, 150826613, 150826650, 150826687, 150828691, 150828728, 150830769, 150833403, 150833440, 150835481, 150840193, 150864431, 150912947, 150912984, 150915025, 150917925, 150917933, 150917964, 150919737, 150928389, 151046813, 151046821, 151046852, 151054801, 151054838, 151056879, 151061591, 151117705, 151117713, 151117744, 151136169, 151136206, 151138247, 151142959, 151188597, 151188605, 151188636, 151222503, 151306587, 151364357, 151374001, 151374009, 151374038, 151374040, 151374046, 151374075, 151374077, 151374083, 151374114, 151376079, 151376087, 151376116, 151376118, 151376124, 151376155, 151377479, 151378157, 151378165, 151378196, 151380791, 151380799, 151380828, 151380830, 151380836, 151380867, 151382869, 151382877, 151382908, 151387581, 151387589, 151387620, 151389443, 151389451, 151389480, 151389482, 151389488, 151389519, 151391521, 151391529, 151391560, 151396233, 151396241, 151396272, 151397431, 151397468, 151397505, 151399509, 151399546, 151401587, 151404221, 151404258, 151406299, 151411011, 151445725, 151460335, 151460343, 151460372, 151460374, 151460380, 151460411, 151462413, 151462421, 151462452, 151467125, 151467133, 151467164, 151475777, 151475785, 151475816, 151546669, 151546677, 151546708, 151574613, 151645505, 151683557, 151683565, 151683594, 151683596, 151683602, 151683633, 151685635, 151685643, 151685674, 151690347, 151690355, 151690386, 151698999, 151699007, 151699038, 151706987, 151707024, 151709065, 151713777, 151769891, 151769899, 151769930, 151833849, 151833857, 151833888, 151901801, 151901838, 151901875, 151903879, 151903916, 151905957, 151908591, 151908628, 151910669, 151915381, 151917243, 151917280, 151919321, 151924033, 151988135, 151988172, 151990213, 151993113, 151993121, 151993152, 151994925, 152003577, 152016543, 152145431, 152169029, 152211357, 152211394, 152213435, 152218147, 152226799, 152297691, 152361649, 152464631, 152464639, 152464668, 152464670, 152464676, 152464707, 152466709, 152466717, 152466748, 152469141, 152469149, 152469178, 152469180, 152469186, 152469215, 152469217, 152469223, 152469252, 152469254, 152469260, 152469289, 152469291, 152469297, 152469328, 152471219, 152471227, 152471256, 152471258, 152471264, 152471293, 152471295, 152471301, 152471330, 152471332, 152471338, 152471369, 152471421, 152471429, 152471460, 152472619, 152472656, 152472693, 152473297, 152473305, 152473334, 152473336, 152473342, 152473371, 152473373, 152473379, 152473410, 152474697, 152474734, 152475375, 152475383, 152475412, 152475414, 152475420, 152475451, 152475931, 152475939, 152475968, 152475970, 152475976, 152476005, 152476007, 152476013, 152476042, 152476044, 152476050, 152476081, 152476775, 152477453, 152477461, 152477492, 152478009, 152478017, 152478046, 152478048, 152478054, 152478083, 152478085, 152478091, 152478122, 152479409, 152479446, 152480073, 152480081, 152480087, 152480095, 152480112, 152480124, 152480126, 152480132, 152480163, 152481487, 152482165, 152482173, 152482204, 152482721, 152482729, 152482758, 152482760, 152482766, 152482795, 152482797, 152482803, 152482834, 152484799, 152484807, 152484836, 152484838, 152484844, 152484875, 152486199, 152486877, 152486885, 152486916, 152488061, 152488098, 152489511, 152489519, 152489548, 152489550, 152489556, 152489587, 152490139, 152491589, 152491597, 152491628, 152494851, 152496301, 152496309, 152496340, 152520913, 152535523, 152535531, 152535560, 152535562, 152535568, 152535599, 152537601, 152537609, 152537640, 152542313, 152542321, 152542352, 152550965, 152550973, 152551004, 152621857, 152621865, 152621896, 152739847, 152774187, 152774195, 152774226, 152778697, 152778705, 152778734, 152778736, 152778742, 152778771, 152778773, 152778779, 152778808, 152778810, 152778816, 152778847, 152780775, 152780783, 152780812, 152780814, 152780820, 152780849, 152780851, 152780857, 152780888, 152782175, 152782212, 152782853, 152782861, 152782890, 152782892, 152782898, 152782929, 152784253, 152784931, 152784939, 152784970, 152785487, 152785495, 152785524, 152785526, 152785532, 152785561, 152785563, 152785569, 152785600, 152787565, 152787573, 152787602, 152787604, 152787610, 152787641, 152788965, 152789643, 152789651, 152789682, 152792277, 152792285, 152792314, 152792316, 152792322, 152792353, 152794355, 152794363, 152794394, 152797617, 152799067, 152799075, 152799106, 152822703, 152822711, 152822740, 152822742, 152822748, 152822779, 152824781, 152824789, 152824820, 152829493, 152829501, 152829532, 152845079, 152845087, 152845118, 152896776, 152909037, 152909045, 152909076, 152992431, 152992468, 152994509, 152999221, 153007873, 153063323, 153063360, 153065401, 153070113, 153078765, 153088253, 153088261, 153088290, 153088292, 153088298, 153088327, 153088329, 153088335, 153088366, 153090331, 153090339, 153090368, 153090370, 153090376, 153090407, 153091731, 153092409, 153092417, 153092448, 153095043, 153095051, 153095080, 153095082, 153095088, 153095119, 153097121, 153097129, 153097160, 153101833, 153101841, 153101872, 153132259, 153132267, 153132298, 153244217, 153301987, 153350503, 153350540, 153352581, 153357293, 153372879, 153397809, 153397817, 153397846, 153397848, 153397854, 153397885, 153399887, 153399895, 153399926, 153404599, 153404607, 153404638, 153436837, 153555261, 153555269, 153555300, 153559771, 153559779, 153559808, 153559810, 153559816, 153559845, 153559847, 153559853, 153559882, 153559884, 153559890, 153559921, 153561849, 153561857, 153561886, 153561888, 153561894, 153561923, 153561925, 153561931, 153561962, 153563249, 153563286, 153563927, 153563935, 153563964, 153563966, 153563972, 153564003, 153565327, 153566005, 153566013, 153566044, 153566561, 153566569, 153566598, 153566600, 153566606, 153566635, 153566637, 153566643, 153566674, 153568639, 153568647, 153568676, 153568678, 153568684, 153568715, 153570039, 153570717, 153570725, 153570756, 153573351, 153573359, 153573388, 153573390, 153573396, 153573427, 153575429, 153575437, 153575468, 153578691, 153580141, 153580149, 153580180, 153626153, 153626161, 153626192, 153660059, 153697045, 153697053, 153697084, 153707365, 153707373, 153707404, 153811557, 153811565, 153811594, 153811596, 153811602, 153811631, 153811633, 153811639, 153811670, 153813635, 153813643, 153813672, 153813674, 153813680, 153813711, 153815035, 153815713, 153815721, 153815752, 153818347, 153818355, 153818384, 153818386, 153818392, 153818423, 153820425, 153820433, 153820464, 153825137, 153825145, 153825176, 153869327, 153869335, 153869364, 153869366, 153869372, 153869401, 153869403, 153869409, 153869440, 153871405, 153871413, 153871442, 153871444, 153871450, 153871481, 153872805, 153873483, 153873491, 153873522, 153876117, 153876125, 153876154, 153876156, 153876162, 153876193, 153878195, 153878203, 153878234, 153882907, 153882915, 153882946, 153897891, 153897899, 153897928, 153897930, 153897936, 153897967, 153899969, 153899977, 153900008, 153904681, 153904689, 153904720, 153913333, 153913341, 153913372, 153921321, 153921358, 153923399, 153928111, 153984225, 153984233, 153984264, 154083061, 154121113, 154121121, 154121150, 154121152, 154121158, 154121189, 154123191, 154123199, 154123230, 154127903, 154127911, 154127942, 154153953, 154178883, 154178891, 154178920, 154178922, 154178928, 154178959, 154180961, 154180969, 154181000, 154185673, 154185681, 154185712, 154207447, 154207455, 154207486, 154230877, 154339357, 154339394, 154339431, 154341435, 154341472, 154343513, 154346147, 154346184, 154348225, 154352937, 154425691, 154425728, 154427769, 154430669, 154430677, 154430708, 154432481, 154441133, 154488439, 154488447, 154488478, 154512025, 154648913, 154648950, 154650401, 154650409, 154650438, 154650440, 154650446, 154650475, 154650477, 154650483, 154650514, 154650991, 154652479, 154652487, 154652516, 154652518, 154652524, 154652555, 154653879, 154654557, 154654565, 154654596, 154655703, 154657191, 154657199, 154657228, 154657230, 154657236, 154657267, 154659269, 154659277, 154659308, 154663981, 154663989, 154664020, 154735247, 154902187, 154902195, 154902224, 154902226, 154902232, 154902263, 154904265, 154904273, 154904304, 154908977, 154908985, 154909016, 154910175, 154910212, 154910249, 154912253, 154912290, 154914331, 154916965, 154917002, 154919043, 154923755, 154958469, 154959957, 154959965, 154959994, 154959996, 154960002, 154960033, 154962035, 154962043, 154962074, 154966747, 154966755, 154966786, 154973079, 154973087, 154973116, 154973118, 154973124, 154973155, 154975157, 154975165, 154975196, 154979869, 154979877, 154979908, 154988521, 154988529, 154988560, 154993031, 154993039, 154993068, 154993070, 154993076, 154993105, 154993107, 154993113, 154993142, 154993144, 154993150, 154993181, 154995109, 154995117, 154995146, 154995148, 154995154, 154995183, 154995185, 154995191, 154995222, 154996509, 154996546, 154997187, 154997195, 154997224, 154997226, 154997232, 154997263, 154998587, 154999265, 154999273, 154999304, 154999821, 154999829, 154999858, 154999860, 154999866, 154999895, 154999897, 154999903, 154999934, 155001899, 155001907, 155001936, 155001938, 155001944, 155001975, 155003299, 155003977, 155003985, 155004016, 155006611, 155006619, 155006648, 155006650, 155006656, 155006687, 155008689, 155008697, 155008728, 155011951, 155013401, 155013409, 155013440, 155059413, 155059421, 155059452, 155211743, 155211751, 155211782, 155219731, 155219768, 155221809, 155226521, 155269513, 155269521, 155269552, 155282635, 155282643, 155282674, 155302587, 155302595, 155302624, 155302626, 155302632, 155302661, 155302663, 155302669, 155302700, 155304665, 155304673, 155304702, 155304704, 155304710, 155304741, 155306065, 155306743, 155306751, 155306782, 155309377, 155309385, 155309414, 155309416, 155309422, 155309453, 155311455, 155311463, 155311494, 155316167, 155316175, 155316206, 155346593, 155346601, 155346632, 155429987, 155430024, 155432065, 155436777, 155500879, 155500916, 155502957, 155507669, 155516321, 155529287, 155587213, 155612143, 155612151, 155612180, 155612182, 155612188, 155612219, 155614221, 155614229, 155614260, 155618933, 155618941, 155618972, 155681773, 155739543, 155741031, 155741039, 155741068, 155741070, 155741076, 155741107, 155743109, 155743117, 155743148, 155747821, 155747829, 155747860, 155810435, 155874393, 155921699, 155921707, 155921738, 155992817, 155992825, 155992856, 156000805, 156000842, 156002883, 156007595, 156050587, 156050595, 156050626, 156063709, 156063717, 156063748, 156068219, 156068227, 156068256, 156068258, 156068264, 156068293, 156068295, 156068301, 156068330, 156068332, 156068338, 156068369, 156070297, 156070305, 156070334, 156070336, 156070342, 156070371, 156070373, 156070379, 156070410, 156071697, 156071734, 156072375, 156072383, 156072412, 156072414, 156072420, 156072451, 156073775, 156074453, 156074461, 156074492, 156075009, 156075017, 156075046, 156075048, 156075054, 156075083, 156075085, 156075091, 156075122, 156077087, 156077095, 156077124, 156077126, 156077132, 156077163, 156078487, 156079165, 156079173, 156079204, 156081799, 156081807, 156081836, 156081838, 156081844, 156081875, 156083661, 156083669, 156083698, 156083700, 156083706, 156083735, 156083737, 156083743, 156083774, 156083877, 156083885, 156083916, 156085739, 156085747, 156085776, 156085778, 156085784, 156085815, 156087139, 156087817, 156087825, 156087856, 156088589, 156088597, 156088628, 156090451, 156090459, 156090488, 156090490, 156090496, 156090527, 156092529, 156092537, 156092568, 156097241, 156097249, 156097280, 156134601, 156134609, 156134640, 156252591, 156310361, 156335447, 156335455, 156335484, 156335486, 156335492, 156335523, 156337525, 156337533, 156337564, 156342237, 156342245, 156342276, 156377775, 156377783, 156377812, 156377814, 156377820, 156377849, 156377851, 156377857, 156377888, 156379853, 156379861, 156379890, 156379892, 156379898, 156379929, 156381253, 156381931, 156381939, 156381970, 156384565, 156384573, 156384602, 156384604, 156384610, 156384641, 156386643, 156386651, 156386682, 156391355, 156391363, 156391394, 156393217, 156393225, 156393254, 156393256, 156393262, 156393293, 156395295, 156395303, 156395334, 156400007, 156400015, 156400046, 156421781, 156421789, 156421820, 156445211, 156520617, 156591509, 156645003, 156645011, 156645042, 156662401, 156687331, 156687339, 156687368, 156687370, 156687376, 156687407, 156689409, 156689417, 156689448, 156694121, 156694129, 156694160, 156702773, 156702781, 156702812, 156831661, 156831669, 156831700, 156863247, 156863284, 156865325, 156870037, 156949581, 156996887, 156996895, 156996926, 157091435, 157158849, 157158857, 157158886, 157158888, 157158894, 157158923, 157158925, 157158931, 157158962, 157160927, 157160935, 157160964, 157160966, 157160972, 157161003, 157162327, 157163005, 157163013, 157163044, 157165639, 157165647, 157165676, 157165678, 157165684, 157165715, 157167717, 157167725, 157167756, 157172429, 157172437, 157172468, 157172803, 157174291, 157174299, 157174328, 157174330, 157174336, 157174367, 157176369, 157176377, 157176408, 157181081, 157181089, 157181120, 157410635, 157410643, 157410672, 157410674, 157410680, 157410711, 157412713, 157412721, 157412752, 157417425, 157417433, 157417464, 157426077, 157426085, 157426116, 157434065, 157434102, 157436143, 157440855, 157468405, 157468413, 157468442, 157468444, 157468450, 157468481, 157470483, 157470491, 157470522, 157475195, 157475203, 157475234, 157483847, 157483855, 157483886, 157496969, 157496977, 157497008, 157516921, 157516929, 157516958, 157516960, 157516966, 157516995, 157516997, 157517003, 157517034, 157518999, 157519007, 157519036, 157519038, 157519044, 157519075, 157520399, 157521077, 157521085, 157521116, 157523711, 157523719, 157523748, 157523750, 157523756, 157523787, 157525789, 157525797, 157525828, 157530501, 157530509, 157530540, 157720191, 157720199, 157720230, 157743621, 157777961, 157777969, 157778000, 157826477, 157826485, 157826514, 157826516, 157826522, 157826553, 157828555, 157828563, 157828594, 157833267, 157833275, 157833306, 157938435, 157938472, 157940513, 157945225, 157953877, 158024769, 158136033, 158136041, 158136072, 158247991, 158249479, 158249487, 158249516, 158249518, 158249524, 158249555, 158251557, 158251565, 158251596, 158256269, 158256277, 158256308, 158264921, 158264929, 158264960, 158501265, 158501273, 158501304, 158505775, 158505783, 158505812, 158505814, 158505820, 158505849, 158505851, 158505857, 158505886, 158505888, 158505894, 158505925, 158507853, 158507861, 158507890, 158507892, 158507898, 158507927, 158507929, 158507935, 158507966, 158509253, 158509290, 158509931, 158509939, 158509968, 158509970, 158509976, 158510007, 158511331, 158512009, 158512017, 158512048, 158512565, 158512573, 158512602, 158512604, 158512610, 158512639, 158512641, 158512647, 158512678, 158514643, 158514651, 158514680, 158514682, 158514688, 158514719, 158516043, 158516721, 158516729, 158516760, 158519355, 158519363, 158519392, 158519394, 158519400, 158519431, 158521433, 158521441, 158521472, 158524695, 158526145, 158526153, 158526184, 158559035, 158559043, 158559074, 158572157, 158572165, 158572196, 158592109, 158592117, 158592146, 158592148, 158592154, 158592183, 158592185, 158592191, 158592222, 158594187, 158594195, 158594224, 158594226, 158594232, 158594263, 158595587, 158596265, 158596273, 158596304, 158598899, 158598907, 158598936, 158598938, 158598944, 158598975, 158600977, 158600985, 158601016, 158605689, 158605697, 158605728, 158607551, 158607559, 158607588, 158607590, 158607596, 158607627, 158609629, 158609637, 158609668, 158614341, 158614349, 158614380, 158815331, 158815339, 158815368, 158815370, 158815376, 158815405, 158815407, 158815413, 158815444, 158817409, 158817417, 158817446, 158817448, 158817454, 158817485, 158818809, 158819487, 158819495, 158819526, 158822121, 158822129, 158822158, 158822160, 158822166, 158822197, 158824199, 158824207, 158824238, 158828911, 158828919, 158828950, 158859337, 158859345, 158859376, 158901665, 158901673, 158901702, 158901704, 158901710, 158901741, 158903743, 158903751, 158903782, 158908455, 158908463, 158908494, 158917107, 158917115, 158917146, 159029065, 159033575, 159033612, 159033649, 159033686, 159035653, 159035690, 159035727, 159037731, 159037768, 159039809, 159040365, 159040402, 159040439, 159042443, 159042480, 159044521, 159047155, 159047192, 159049233, 159053945, 159099957, 159124887, 159124895, 159124924, 159124926, 159124932, 159124963, 159126965, 159126973, 159127004, 159131677, 159131685, 159131716, 159211221, 159211229, 159211260, 159340109, 159340117, 159340148, 159343131, 159343168, 159343205, 159345209, 159345246, 159347287, 159349921, 159349958, 159351999, 159356711, 159387137, 159434443, 159434451, 159434482, 159580390, 159580427, 159580464, 159580501, 159580538, 159582468, 159582505, 159582542, 159582579, 159584546, 159584583, 159584620, 159586624, 159586661, 159587180, 159587217, 159587254, 159587291, 159588702, 159589258, 159589295, 159589332, 159591336, 159591373, 159593414, 159593970, 159594007, 159594044, 159596048, 159596085, 159596405, 159596413, 159596442, 159596444, 159596450, 159596479, 159596481, 159596487, 159596518, 159598126, 159598483, 159598491, 159598520, 159598522, 159598528, 159598559, 159599883, 159600561, 159600569, 159600600, 159600760, 159600797, 159602838, 159603195, 159603203, 159603232, 159603234, 159603240, 159603271, 159605273, 159605281, 159605312, 159607550, 159609985, 159609993, 159610024, 159652687, 159652724, 159654765, 159659477, 159667297, 159667305, 159667334, 159667336, 159667342, 159667371, 159667373, 159667379, 159667410, 159669375, 159669383, 159669412, 159669414, 159669420, 159669451, 159670775, 159671453, 159671461, 159671492, 159674087, 159674095, 159674124, 159674126, 159674132, 159674163, 159676165, 159676173, 159676204, 159680877, 159680885, 159680916, 159682739, 159682747, 159682776, 159682778, 159682784, 159682815, 159684817, 159684825, 159684856, 159689529, 159689537, 159689568, 159698181, 159698189, 159698220, 159848191, 159848199, 159848228, 159848230, 159848236, 159848267, 159850269, 159850277, 159850308, 159854981, 159854989, 159855020, 159889946, 159889983, 159890020, 159890057, 159892024, 159892061, 159892098, 159894102, 159894139, 159896180, 159896736, 159896773, 159896810, 159898814, 159898851, 159900892, 159903526, 159903563, 159905604, 159905961, 159905969, 159905998, 159906000, 159906006, 159906037, 159908039, 159908047, 159908078, 159910316, 159912751, 159912759, 159912790, 159934525, 159934533, 159934564, 159957955, 159962243, 159976853, 159976861, 159976890, 159976892, 159976898, 159976929, 159978931, 159978939, 159978970, 159983643, 159983651, 159983682, 159992295, 159992303, 159992334, 160040811, 160040819, 160040848, 160040850, 160040856, 160040887, 160042889, 160042897, 160042928, 160047601, 160047609, 160047640, 160124205, 160124242, 160124279, 160126283, 160126320, 160128361, 160130995, 160131032, 160133073, 160137785, 160157747, 160157755, 160157786, 160199502, 160199539, 160199576, 160201580, 160201617, 160203658, 160206292, 160206329, 160208370, 160213082, 160215517, 160215525, 160215556, 160286409, 160286417, 160286448, 160350367, 160350375, 160350406, 160375991, 160376028, 160378069, 160382781, 160433761, 160433798, 160435839, 160440551, 160462325, 160509058, 160509095, 160511136, 160515848, 160671020, 160671057, 160671094, 160671131, 160673098, 160673135, 160673172, 160675176, 160675213, 160677254, 160677810, 160677847, 160677884, 160679888, 160679925, 160681966, 160684600, 160684637, 160685547, 160686678, 160687035, 160687043, 160687072, 160687074, 160687080, 160687111, 160689113, 160689121, 160689152, 160691390, 160693825, 160693833, 160693864, 160743317, 160757927, 160757935, 160757964, 160757966, 160757972, 160758003, 160760005, 160760013, 160760044, 160764717, 160764725, 160764756, 160773369, 160773377, 160773408, 160818614, 160938821, 160938829, 160938860, 160946809, 160946846, 160948887, 160953599, 160980576, 160980613, 160980650, 160982654, 160982691, 160984732, 160987366, 160987403, 160989444, 160994156, 160996591, 160996599, 160996630, 161009713, 161009721, 161009752, 161029665, 161029673, 161029702, 161029704, 161029710, 161029739, 161029741, 161029747, 161029778, 161031743, 161031751, 161031780, 161031782, 161031788, 161031819, 161033143, 161033821, 161033829, 161033860, 161036455, 161036463, 161036492, 161036494, 161036500, 161036531, 161038533, 161038541, 161038572, 161043245, 161043253, 161043284, 161067483, 161067491, 161067522, 161115999, 161116007, 161116036, 161116038, 161116044, 161116075, 161118077, 161118085, 161118116, 161122789, 161122797, 161122828, 161131441, 161131449, 161131480, 161214835, 161214872, 161216913, 161221625, 161256365, 161290132, 161290169, 161292210, 161296922, 161339221, 161339229, 161339258, 161339260, 161339266, 161339297, 161341299, 161341307, 161341338, 161346011, 161346019, 161346050, 161425555, 161425563, 161425594, 161466621, 161524391, 161537513, 161557465, 161557502, 161557539, 161559543, 161559580, 161561621, 161564255, 161564292, 161566333, 161571045, 161599688, 161648777, 161648785, 161648816, 161761650, 161761687, 161761724, 161763728, 161763765, 161765806, 161768440, 161768477, 161770518, 161775230, 161777665, 161777673, 161777704, 161848557, 161848565, 161848596, 161867021, 161867058, 161869099, 161873811, 162037439, 162071206, 162071243, 162073284, 162077996, 162104280, 162104317, 162104354, 162104391, 162104853, 162104861, 162104890, 162104892, 162104898, 162104927, 162104929, 162104935, 162104966, 162106358, 162106395, 162106432, 162106931, 162106939, 162106968, 162106970, 162106976, 162107007, 162108331, 162108436, 162108473, 162109009, 162109017, 162109048, 162110514, 162111070, 162111107, 162111144, 162111643, 162111651, 162111680, 162111682, 162111688, 162111719, 162113148, 162113185, 162113721, 162113729, 162113760, 162115226, 162117860, 162117897, 162118433, 162118441, 162118472, 162119938, 162120295, 162120303, 162120332, 162120334, 162120340, 162120371, 162122373, 162122381, 162122412, 162124650, 162127085, 162127093, 162127124, 162176577, 162191187, 162191195, 162191224, 162191226, 162191232, 162191263, 162193265, 162193273, 162193304, 162197977, 162197985, 162198016, 162206629, 162206637, 162206668, 162305465, 162372081, 162372089, 162372120, 162380762, 162413836, 162413873, 162413910, 162414409, 162414417, 162414446, 162414448, 162414454, 162414485, 162415914, 162415951, 162416487, 162416495, 162416526, 162417992, 162420626, 162420663, 162421199, 162421207, 162421238, 162422704, 162427416, 162429851, 162429859, 162429890, 162500743, 162500751, 162500782, 162564701, 162564709, 162564740, 162632653, 162632690, 162632727, 162634731, 162634768, 162636809, 162639443, 162639480, 162641521, 162646233, 162648095, 162648132, 162650173, 162654885, 162723392, 162723429, 162723965, 162723973, 162724004, 162725470, 162730182, 162852280, 162852317, 162854358, 162859070, 162899881, 162942209, 162942246, 162944287, 162948999, 162957651, 163032948, 163161836, 163179468, 163179505, 163179542, 163179579, 163181546, 163181583, 163181620, 163183624, 163183661, 163185702, 163186258, 163186295, 163186332, 163188336, 163188373, 163190414, 163193048, 163193085, 163194910, 163194947, 163194984, 163195126, 163195483, 163195491, 163195520, 163195522, 163195528, 163195559, 163196988, 163197025, 163197561, 163197569, 163197600, 163199066, 163199838, 163201700, 163201737, 163202273, 163202281, 163202312, 163203778, 163208490, 163210925, 163210933, 163210964, 163251765, 163266375, 163266383, 163266412, 163266414, 163266420, 163266451, 163268453, 163268461, 163268492, 163273165, 163273173, 163273204, 163281817, 163281825, 163281856, 163447269, 163447277, 163447308, 163470699, 163489024, 163489061, 163489098, 163491102, 163491139, 163493180, 163495814, 163495851, 163497892, 163502604, 163504466, 163504503, 163505039, 163505047, 163505078, 163506544, 163511256, 163553555, 163553563, 163553592, 163553594, 163553600, 163553631, 163555633, 163555641, 163555672, 163560345, 163560353, 163560384, 163575931, 163575939, 163575970, 163627628, 163639889, 163639897, 163639928, 163723283, 163723320, 163725361, 163730073, 163738725, 163798580, 163798617, 163800658, 163805370, 163814022, 163863111, 163863119, 163863150, 163942910, 163975069, 164032839, 164081355, 164081392, 164083433, 164088145, 164108136, 164270098, 164270135, 164270172, 164272176, 164272213, 164274254, 164276888, 164276925, 164278966, 164283678, 164285540, 164285577, 164286113, 164286121, 164286152, 164287618, 164292330, 164357005, 164357013, 164357044, 164390911, 164542409, 164542417, 164542446, 164542448, 164542454, 164542483, 164542485, 164542491, 164542522, 164544487, 164544495, 164544524, 164544526, 164544532, 164544563, 164545887, 164546565, 164546573, 164546604, 164549199, 164549207, 164549236, 164549238, 164549244, 164549275, 164551277, 164551285, 164551316, 164555989, 164555997, 164556028, 164579654, 164579691, 164581732, 164586444, 164595096, 164628170, 164628207, 164628244, 164628743, 164628751, 164628780, 164628782, 164628788, 164628819, 164630248, 164630285, 164630821, 164630829, 164630860, 164632326, 164634960, 164634997, 164635533, 164635541, 164635572, 164637038, 164641750, 164644185, 164644193, 164644224, 164715077, 164715085, 164715116, 164813913, 164851965, 164851973, 164852002, 164852004, 164852010, 164852041, 164854043, 164854051, 164854082, 164858755, 164858763, 164858794, 164889210, 164937726, 164937763, 164938299, 164938307, 164938338, 164939804, 164944516, 165070209, 165070246, 165070283, 165072287, 165072324, 165074365, 165076999, 165077036, 165079077, 165083789, 165156543, 165156580, 165158621, 165161521, 165161529, 165161560, 165163333, 165171985, 165247282, 165360728, 165360765, 165362806, 165367518, 165376170, 165379765, 165379802, 165381843, 165386555, 165466099, 165617024, 165617061, 165617098, 165617135, 165619102, 165619139, 165619176, 165621180, 165621217, 165623258, 165623814, 165623851, 165623888, 165625892, 165625929, 165627970, 165630604, 165630641, 165632682, 165633039, 165633047, 165633076, 165633078, 165633084, 165633115, 165635117, 165635125, 165635156, 165637394, 165639829, 165639837, 165639868, 165641027, 165641064, 165641101, 165643105, 165643142, 165645183, 165647817, 165647854, 165649895, 165654607, 165670284, 165689321, 165703358, 165703395, 165703432, 165703931, 165703939, 165703968, 165703970, 165703976, 165704007, 165705436, 165705473, 165706009, 165706017, 165706048, 165707514, 165710148, 165710185, 165710721, 165710729, 165710760, 165712226, 165716938, 165718800, 165718837, 165719373, 165719381, 165719412, 165720878, 165725590, 165790265, 165790273, 165790304, 165884825, 165884833, 165884864, 165926580, 165926617, 165926654, 165928658, 165928695, 165930736, 165933370, 165933407, 165935448, 165940160, 165942595, 165942603, 165942634, 165950583, 165950620, 165952661, 165957373, 166012914, 166012951, 166013487, 166013495, 166013526, 166014992, 166019704, 166028356, 166077445, 166077453, 166077484, 166160839, 166160876, 166162917, 166167629, 166231731, 166231768, 166233809, 166236136, 166236173, 166238214, 166238521, 166242926, 166247173, 166260139, 166322470, 166412625, 166451358, 166470395, 166541287, 166545692, 166605245, 166707654, 166707691, 166707728, 166709732, 166709769, 166711810, 166714444, 166714481, 166716522, 166721234, 166723669, 166723677, 166723708, 166731657, 166731694, 166733735, 166738447, 166778546, 166778583, 166778620, 166780624, 166780661, 166782702, 166785336, 166785373, 166787414, 166792126, 166793988, 166794025, 166794561, 166794569, 166794600, 166796066, 166800778, 166809430, 166865453, 166865461, 166865492, 166983443, 167017210, 167017247, 167019288, 167024000, 167041213, 167066299, 167066307, 167066336, 167066338, 167066344, 167066375, 167068377, 167068385, 167068416, 167073089, 167073097, 167073128, 167088102, 167088139, 167090180, 167094892, 167103544, 167152060, 167152097, 167152633, 167152641, 167152672, 167154138, 167158850, 167251469, 167322361, 167326766, 167375855, 167375863, 167375894, 167397658, 167461616, 167594099, 167594136, 167596177, 167600889, 167680433, 167798284, 167798321, 167800362, 167805074, 167822287, 167869176, 167869213, 167871254, 167875966, 167884618, 167903655, 168107840, 168140914, 168140951, 168140988, 168141487, 168141495, 168141524, 168141526, 168141532, 168141563, 168142992, 168143029, 168143565, 168143573, 168143604, 168145070, 168147704, 168147741, 168148277, 168148285, 168148316, 168149782, 168154494, 168156929, 168156937, 168156968, 168164917, 168164954, 168166995, 168171707, 168178732, 168227248, 168227285, 168227821, 168227829, 168227860, 168229326, 168234038, 168242690, 168450470, 168450507, 168451043, 168451051, 168451082, 168452548, 168457260, 168474473, 168536804, 168669287, 168669324, 168671365, 168676077, 168684729, 168755621, 168760026, 168888914, 168959806, 168978843, 169216102, 169216139, 169216176, 169218180, 169218217, 169220258, 169222892, 169222929, 169224970, 169229682, 169231544, 169231581, 169232117, 169232125, 169232156, 169233622, 169236627, 169236635, 169236664, 169236666, 169236672, 169236701, 169236703, 169236709, 169236738, 169236740, 169236746, 169236777, 169238334, 169238705, 169238713, 169238742, 169238744, 169238750, 169238779, 169238781, 169238787, 169238818, 169240105, 169240142, 169240783, 169240791, 169240820, 169240822, 169240828, 169240859, 169242183, 169242861, 169242869, 169242900, 169243417, 169243425, 169243454, 169243456, 169243462, 169243491, 169243493, 169243499, 169243530, 169245495, 169245503, 169245532, 169245534, 169245540, 169245571, 169246895, 169247573, 169247581, 169247612, 169250207, 169250215, 169250244, 169250246, 169250252, 169250283, 169252285, 169252293, 169252324, 169255547, 169256997, 169257005, 169257036, 169302436, 169302473, 169303009, 169303017, 169303048, 169304514, 169309226, 169317878, 169525658, 169525695, 169527736, 169532448, 169541100, 169546183, 169546191, 169546220, 169546222, 169546228, 169546257, 169546259, 169546265, 169546296, 169548261, 169548269, 169548298, 169548300, 169548306, 169548337, 169549661, 169550339, 169550347, 169550378, 169552973, 169552981, 169553010, 169553012, 169553018, 169553049, 169555051, 169555059, 169555090, 169559763, 169559771, 169559802, 169590189, 169590197, 169590228, 169611992, 169675950, 169759917, 169830809, 169835214, 169855739, 169855747, 169855776, 169855778, 169855784, 169855815, 169857817, 169857825, 169857856, 169862529, 169862537, 169862568, 170117989, 170165295, 170165303, 170165334, 170306732, 170306769, 170308810, 170313522, 170322174, 170327257, 170327265, 170327294, 170327296, 170327302, 170327331, 170327333, 170327339, 170327370, 170329335, 170329343, 170329372, 170329374, 170329380, 170329411, 170330735, 170331413, 170331421, 170331452, 170334047, 170334055, 170334084, 170334086, 170334092, 170334123, 170336125, 170336133, 170336164, 170340837, 170340845, 170340876, 170377624, 170377661, 170379702, 170384414, 170393066, 170579043, 170579051, 170579080, 170579082, 170579088, 170579119, 170581121, 170581129, 170581160, 170585833, 170585841, 170585872, 170616288, 170636813, 170636821, 170636850, 170636852, 170636858, 170636889, 170638891, 170638899, 170638930, 170643603, 170643611, 170643642, 170664804, 170664841, 170665377, 170665385, 170665416, 170666882, 170671594, 170687180, 170688807, 170751138, 170888599, 170888607, 170888638, 170946369, 170946377, 170946408, 170974360, 171106843, 171106880, 171108921, 171113633, 171193177, 171397362, 171416399, 171417887, 171417895, 171417924, 171417926, 171417932, 171417963, 171419965, 171419973, 171420004, 171424677, 171424685, 171424716, 171468254, 171653658, 171653695, 171653732, 171655736, 171655773, 171657814, 171660448, 171660485, 171662526, 171667238, 171669673, 171669681, 171669712, 171677661, 171677698, 171679739, 171684451, 171727443, 171727451, 171727482, 171739992, 171740029, 171740565, 171740573, 171740604, 171742070, 171746782, 171755434, 171760517, 171760525, 171760554, 171760556, 171760562, 171760591, 171760593, 171760599, 171760630, 171762595, 171762603, 171762632, 171762634, 171762640, 171762671, 171763995, 171764673, 171764681, 171764712, 171767307, 171767315, 171767344, 171767346, 171767352, 171767383, 171769385, 171769393, 171769424, 171774097, 171774105, 171774136, 171826326, 171963214, 171963251, 171965292, 171970004, 171987217, 172049548, 172070073, 172070081, 172070110, 172070112, 172070118, 172070149, 172072151, 172072159, 172072190, 172076863, 172076871, 172076902, 172197473, 172268365, 172272770, 172379629, 172379637, 172379668, 172508517, 172508525, 172508556, 172744288, 172744325, 172746366, 172751078, 172768291, 172815180, 172815217, 172817258, 172821970, 172830622, 172835705, 172835713, 172835742, 172835744, 172835750, 172835779, 172835781, 172835787, 172835818, 172837783, 172837791, 172837820, 172837822, 172837828, 172837859, 172839183, 172839861, 172839869, 172839900, 172842495, 172842503, 172842532, 172842534, 172842540, 172842571, 172844573, 172844581, 172844612, 172849285, 172849293, 172849324, 172851147, 172851155, 172851184, 172851186, 172851192, 172851223, 172853225, 172853233, 172853264, 172857937, 172857945, 172857976, 172901514, 173053844, 173102933, 173102941, 173102972, 173124736, 173145261, 173145269, 173145298, 173145300, 173145306, 173145337, 173147339, 173147347, 173147378, 173152051, 173152059, 173152090, 173160703, 173160711, 173160742, 173188694, 173454817, 173454825, 173454856, 173630733, 173834918, 173905810, 173926335, 173926343, 173926372, 173926374, 173926380, 173926411, 173928413, 173928421, 173928452, 173933125, 173933133, 173933164, 173941777, 173941785, 173941816, 173976702, 174177548, 174177585, 174178121, 174178129, 174178160, 174179626, 174184338, 174201551, 174235891, 174235899, 174235930, 174263882, 174284407, 174284415, 174284444, 174284446, 174284452, 174284483, 174286485, 174286493, 174286524, 174291197, 174291205, 174291236, 174487104, 174593963, 174593971, 174594002, 174705921, 175016965, 175016973, 175017004, 175252736, 175252773, 175254814, 175259526, 175268178, 175273261, 175273269, 175273298, 175273300, 175273306, 175273335, 175273337, 175273343, 175273374, 175275339, 175275347, 175275376, 175275378, 175275384, 175275415, 175276739, 175277417, 175277425, 175277456, 175280051, 175280059, 175280088, 175280090, 175280096, 175280127, 175282129, 175282137, 175282168, 175286841, 175286849, 175286880, 175339070, 175359595, 175359603, 175359632, 175359634, 175359640, 175359671, 175361673, 175361681, 175361712, 175366385, 175366393, 175366424, 175375037, 175375045, 175375076, 175562292, 175582817, 175582825, 175582854, 175582856, 175582862, 175582893, 175584895, 175584903, 175584934, 175589607, 175589615, 175589646, 175669151, 175669159, 175669190, 175801061, 175801098, 175801135, 175803139, 175803176, 175805217, 175807851, 175807888, 175809929, 175814641, 175892373, 175892381, 175892412, 176110617, 176110654, 176112695, 176117407, 176343366, 176347876, 176347913, 176347950, 176347987, 176349954, 176349991, 176350028, 176352032, 176352069, 176354110, 176354666, 176354703, 176354740, 176356744, 176356781, 176358822, 176361456, 176361493, 176363534, 176363891, 176363899, 176363928, 176363930, 176363936, 176363967, 176365969, 176365977, 176366008, 176368246, 176370681, 176370689, 176370720, 176414258, 176420173, 176434783, 176434791, 176434820, 176434822, 176434828, 176434859, 176436861, 176436869, 176436900, 176441573, 176441581, 176441612, 176450225, 176450233, 176450264, 176615677, 176615685, 176615716, 176657432, 176657469, 176657506, 176659510, 176659547, 176661588, 176664222, 176664259, 176666300, 176671012, 176673447, 176673455, 176673486, 176701438, 176744339, 176744347, 176744378, 176808297, 176808305, 176808336, 176891691, 176891728, 176893769, 176898481, 176966988, 176967025, 176969066, 176973778, 177143477, 177201247, 177276544, 177438506, 177438543, 177438580, 177440584, 177440621, 177442662, 177445296, 177445333, 177447374, 177452086, 177454521, 177454529, 177454560, 177525413, 177525421, 177525452, 177690292, 177690329, 177692370, 177697082, 177714295, 177748062, 177748099, 177750140, 177754852, 177776626, 177797151, 177797159, 177797188, 177797190, 177797196, 177797227, 177799229, 177799237, 177799268, 177803941, 177803949, 177803980, 177883485, 177883493, 177883524, 177982321, 177999848, 178057618, 178106707, 178106715, 178106746, 178324951, 178324988, 178327029, 178331741, 178529136, 178529173, 178531214, 178535926, 178634507, 178780922, 178838692, 178851814, 178871766, 178871803, 178871840, 178872339, 178872347, 178872376, 178872378, 178872384, 178872415, 178873844, 178873881, 178874417, 178874425, 178874456, 178875922, 178878556, 178878593, 178879129, 178879137, 178879168, 178880634, 178885346, 178887781, 178887789, 178887820, 178958673, 178958681, 178958712, 179181322, 179181359, 179181895, 179181903, 179181934, 179183400, 179188112, 179400139, 179400176, 179402217, 179406929, 179415581, 179490878, 179619766, 179709695, 179946954, 179946991, 179947028, 179949032, 179949069, 179951110, 179953744, 179953781, 179955822, 179960534, 179962396, 179962433, 179962969, 179962977, 179963008, 179964474, 179969186, 180033861, 180033869, 180033900, 180214182, 180256510, 180256547, 180258588, 180263300, 180271952, 180321041, 180321049, 180321080, 180490769, 180566066, 180848841, 181037584, 181037621, 181039662, 181044374, 181053026, 181289370, 181309895, 181309903, 181309932, 181309934, 181309940, 181309971, 181311973, 181311981, 181312012, 181316685, 181316693, 181316724, 181347140, 181395656, 181395693, 181396229, 181396237, 181396268, 181397734, 181402446, 181619451, 181619459, 181619490, 181705212, 181837695, 181837732, 181839773, 181844485, 181924029, 182128214, 182147251, 182384510, 182384547, 182384584, 182386588, 182386625, 182388666, 182391300, 182391337, 182393378, 182398090, 182400525, 182400533, 182400564, 182408513, 182408550, 182410591, 182415303, 182470844, 182470881, 182471417, 182471425, 182471456, 182472922, 182477634, 182486286, 182694066, 182694103, 182696144, 182700856, 182718069, 182780400, 182928325, 182999217, 183003622, 183475140, 183475177, 183477218, 183481930, 183499143, 183546032, 183546069, 183548110, 183552822, 183561474, 183726926, 183784696, 183833785, 183833793, 183833824, 183855588, 183919546, 184361585, 184565770, 184636662, 184908400, 184908437, 184908973, 184908981, 184909012, 184910478, 184915190, 184932403, 184994734, 185217956, 185436773, 185983588, 185983625, 185985666, 185990378, 185999030, 186004113, 186004121, 186004150, 186004152, 186004158, 186004187, 186004189, 186004195, 186004226, 186006191, 186006199, 186006228, 186006230, 186006236, 186006267, 186007591, 186008269, 186008277, 186008308, 186010903, 186010911, 186010940, 186010942, 186010948, 186010979, 186012981, 186012989, 186013020, 186017693, 186017701, 186017732, 186069922, 186293144, 186313669, 186313677, 186313706, 186313708, 186313714, 186313745, 186315747, 186315755, 186315786, 186320459, 186320467, 186320498, 186623225, 186623233, 186623264, 187074218, 187094743, 187094751, 187094780, 187094782, 187094788, 187094819, 187096821, 187096829, 187096860, 187101533, 187101541, 187101572, 187145110, 187346529, 187346537, 187346568, 187404299, 187404307, 187404338, 187432290, 187874329, 188185373, 188185381, 188185412, 188421144, 188421181, 188423222, 188427934, 188445147, 188507478, 188528003, 188528011, 188528040, 188528042, 188528048, 188528079, 188530081, 188530089, 188530120, 188534793, 188534801, 188534832, 188730700, 188837559, 188837567, 188837598, 189511774, 189582666, 189603191, 189603199, 189603228, 189603230, 189603236, 189603267, 189605269, 189605277, 189605308, 189609981, 189609989, 189610020, 189618633, 189618641, 189618672, 189912747, 189912755, 189912786, 190693821, 190693829, 190693860, 190945034, 191051893, 191051901, 191051932, 192020222, 192040747, 192040755, 192040784, 192040786, 192040792, 192040823, 192042825, 192042833, 192042864, 192047537, 192047545, 192047576, 192127081, 192127089, 192127120, 192350303, 192350311, 192350342, 192568547, 192568584, 192570625, 192575337, 192878103, 193115362, 193115399, 193115436, 193117440, 193117477, 193119518, 193122152, 193122189, 193124230, 193128942, 193131377, 193131385, 193131416, 193202269, 193202277, 193202308, 193424918, 193424955, 193426996, 193431708, 193659177, 193734474, 194205992, 194206029, 194208070, 194212782, 194457778, 194515548, 194564637, 194564645, 194564676, 195092437, 195128761, 195128798, 195128835, 195128872, 195128909, 195130839, 195130876, 195130913, 195130950, 195132917, 195132954, 195132991, 195134995, 195135032, 195135551, 195135588, 195135625, 195135662, 195137073, 195137629, 195137666, 195137703, 195139707, 195139744, 195141785, 195142341, 195142378, 195142415, 195144419, 195144456, 195146497, 195149131, 195149168, 195151209, 195155921, 195296622, 195438317, 195438354, 195438391, 195438428, 195440395, 195440432, 195440469, 195442473, 195442510, 195444551, 195445107, 195445144, 195445181, 195447185, 195447222, 195449263, 195451897, 195451934, 195453975, 195458687, 195639252, 195639289, 195639825, 195639833, 195639864, 195641330, 195646042, 195747873, 195747910, 195747947, 195749951, 195749988, 195752029, 195754663, 195754700, 195756741, 195761453, 195948808, 196057429, 196057466, 196059507, 196064219, 196167625, 196219391, 196219428, 196219465, 196219502, 196221469, 196221506, 196221543, 196223547, 196223584, 196225625, 196226181, 196226218, 196226255, 196228259, 196228296, 196230337, 196232971, 196233008, 196235049, 196239761, 196366985, 196528947, 196528984, 196529021, 196531025, 196531062, 196533103, 196535737, 196535774, 196537815, 196542527, 196714440, 196714477, 196716518, 196721230, 196729882, 196838503, 196838540, 196840581, 196845293, 197023996, 197148059, 197310021, 197310058, 197310095, 197312099, 197312136, 197314177, 197316811, 197316848, 197318889, 197323601, 197619577, 197619614, 197621655, 197626367, 197652651, 197652688, 197652725, 197652762, 197654729, 197654766, 197654803, 197656807, 197656844, 197658885, 197659441, 197659478, 197659515, 197661519, 197661556, 197663597, 197666231, 197666268, 197668309, 197673021, 197805070, 197929133, 197962207, 197962244, 197962281, 197964285, 197964322, 197966363, 197968997, 197969034, 197971075, 197975787, 198077381, 198077389, 198077420, 198163142, 198271763, 198271800, 198273841, 198278553, 198400651, 198400688, 198402729, 198407441, 198581319, 198605181, 198710207, 198727839, 198727876, 198727913, 198727950, 198729917, 198729954, 198729991, 198731995, 198732032, 198734073, 198734629, 198734666, 198734703, 198736707, 198736744, 198738785, 198741419, 198741456, 198743281, 198743318, 198743355, 198743497, 198745359, 198745396, 198747437, 198748209, 198750071, 198750108, 198752149, 198756861, 199037395, 199037432, 199037469, 199039473, 199039510, 199041551, 199044185, 199044222, 199046263, 199050975, 199052837, 199052874, 199054915, 199059627, 199151996, 199152033, 199154074, 199158786, 199175999, 199238330, 199346951, 199346988, 199349029, 199353741, 199362393, 199461552, 199491281, 199656507, 199818469, 199818506, 199818543, 199820547, 199820584, 199822625, 199825259, 199825296, 199827337, 199832049, 199833911, 199833948, 199835989, 199840701, 200128025, 200128062, 200130103, 200134815, 200143467, 200176541, 200176578, 200176615, 200178619, 200178656, 200180697, 200183331, 200183368, 200185409, 200190121, 200242626, 200313518, 200437581, 200486097, 200486134, 200488175, 200492887, 200795653, 200909099, 200909136, 200911177, 200915889, 200924541, 201165395, 201165432, 201165469, 201165506, 201167473, 201167510, 201167547, 201169551, 201169588, 201171629, 201172185, 201172222, 201172259, 201174263, 201174300, 201176341, 201178975, 201179012, 201181053, 201185765, 201218655, 201251729, 201251766, 201251803, 201253807, 201253844, 201255885, 201258519, 201258556, 201260597, 201265309, 201267171, 201267208, 201269249, 201273961, 201474951, 201474988, 201475025, 201477029, 201477066, 201479107, 201481741, 201481778, 201483819, 201488531, 201561285, 201561322, 201563363, 201568075, 201576727, 201675886, 201784507, 201784544, 201786585, 201791297, 201870841, 201999729, 202094063, 202256025, 202256062, 202256099, 202258103, 202258140, 202260181, 202262815, 202262852, 202264893, 202269605, 202326917, 202326954, 202326991, 202328995, 202329032, 202331073, 202333707, 202333744, 202335785, 202340497, 202342359, 202342396, 202344437, 202349149, 202357801, 202565581, 202565618, 202567659, 202572371, 202636473, 202636510, 202638551, 202643263, 202651915, 202700431, 202700468, 202702509, 202707221, 202751074, 202771599, 202771607, 202771636, 202771638, 202771644, 202771675, 202773677, 202773685, 202773716, 202778389, 202778397, 202778428, 202875137, 202946029, 203009987, 203081155, 203081163, 203081194, 203346655, 203346692, 203348733, 203353445, 203417547, 203417584, 203419625, 203424337, 203432989, 203656211, 203689285, 203689322, 203689359, 203691363, 203691400, 203693441, 203696075, 203696112, 203698153, 203702865, 203727103, 203775619, 203775656, 203777697, 203782409, 203791061, 203862229, 203862237, 203862268, 203998841, 203998878, 204000919, 204005631, 204085175, 204308397, 204437285, 204508177, 204764473, 204764510, 204764547, 204766551, 204766588, 204768629, 204771263, 204771300, 204773341, 204778053, 204779915, 204779952, 204781993, 204786705, 204850807, 204850844, 204852885, 204857597, 204866249, 205074029, 205074066, 205076107, 205080819, 205089471, 205160363, 205188630, 205224321, 205288795, 205288832, 205288869, 205288906, 205288943, 205290873, 205290910, 205290947, 205290984, 205292951, 205292988, 205293025, 205295029, 205295066, 205295489, 205295497, 205295528, 205295585, 205295622, 205295659, 205295696, 205297107, 205297663, 205297700, 205297737, 205299741, 205299778, 205301819, 205302375, 205302412, 205302449, 205304453, 205304490, 205306531, 205309165, 205309202, 205311243, 205315955, 205383585, 205598351, 205598388, 205598425, 205598462, 205600429, 205600466, 205600503, 205602507, 205602544, 205604585, 205605141, 205605178, 205605215, 205607219, 205607256, 205609297, 205611931, 205611968, 205614009, 205618721, 205855103, 205855140, 205857181, 205861893, 205870545, 205907907, 205907944, 205907981, 205909985, 205910022, 205912063, 205914697, 205914734, 205916775, 205921487, 205925995, 205926032, 205928073, 205932785, 205941437, 206164659, 206213175, 206213212, 206215253, 206217463, 206217500, 206219541, 206219965, 206224253, 206235551, 206299509, 206370677, 206370685, 206370716, 206379425, 206379462, 206379499, 206379536, 206381503, 206381540, 206381577, 206383581, 206383618, 206385659, 206386215, 206386252, 206386289, 206388293, 206388330, 206390371, 206393005, 206393042, 206395083, 206399795, 206522731, 206527019, 206688981, 206689018, 206689055, 206691059, 206691096, 206693137, 206695771, 206695808, 206697849, 206702561, 206945733, 206998537, 206998574, 207000615, 207005327, 207016625, 207202029, 207202066, 207202103, 207204107, 207204144, 207206185, 207208819, 207208856, 207210897, 207215609, 207288363, 207288400, 207290441, 207295153, 207303805, 207308093, 207374697, 207470055, 207470092, 207470129, 207472133, 207472170, 207474211, 207476845, 207476882, 207478923, 207483635, 207511585, 207511622, 207513663, 207518375, 207597919, 207779611, 207779648, 207781689, 207786401, 207812685, 207812722, 207812759, 207812796, 207814763, 207814800, 207814837, 207816841, 207816878, 207818919, 207819475, 207819512, 207819549, 207821141, 207821553, 207821590, 207823631, 207826265, 207826302, 207828343, 207833055, 208089167, 208122241, 208122278, 208122315, 208124319, 208124356, 208126397, 208129031, 208129068, 208131109, 208135821, 208292659, 208292696, 208294737, 208299449, 208363551, 208363588, 208365629, 208370341, 208378993, 208431797, 208431834, 208433875, 208438587, 208449885, 208560685, 208560722, 208562763, 208567475, 208602215, 208673107, 208737065, 208741353, 208808233, 208808241, 208808272, 208870241, 208887873, 208887910, 208887947, 208887984, 208889951, 208889988, 208890025, 208892029, 208892066, 208894107, 208894663, 208894700, 208894737, 208896741, 208896778, 208898819, 208901453, 208901490, 208903315, 208903352, 208903389, 208903531, 208905393, 208905430, 208907471, 208908243, 208910105, 208910142, 208912183, 208916895, 209197429, 209197466, 209197503, 209199507, 209199544, 209201585, 209204219, 209204256, 209206297, 209211009, 209212871, 209212908, 209214949, 209219661, 209336033, 209383289, 209454181, 209506985, 209507022, 209509063, 209513775, 209522427, 209525073, 209651315, 209725919, 209725956, 209727997, 209732709, 209812253, 209816541, 209882848, 209882885, 209884926, 209889638, 209978503, 209978540, 209978577, 209980581, 209980618, 209982659, 209985293, 209985330, 209987371, 209992083, 209993945, 209993982, 209996023, 210000735, 210035475, 210192404, 210288059, 210288096, 210290137, 210294849, 210303501, 210336575, 210336612, 210336649, 210338653, 210338690, 210340731, 210343365, 210343402, 210345443, 210350155, 210597615, 210646131, 210646168, 210648209, 210652921, 210801107, 210801144, 210803185, 210807897, 210816549, 210887441, 210955687, 210973478, 211069133, 211069170, 211071211, 211075923, 211084575, 211110663, 211325429, 211325466, 211325503, 211325540, 211327507, 211327544, 211327581, 211329585, 211329622, 211331663, 211332219, 211332256, 211332293, 211334297, 211334334, 211336375, 211339009, 211339046, 211341087, 211345799, 211378689, 211411763, 211411800, 211411837, 211413841, 211413878, 211415919, 211418553, 211418590, 211420631, 211425343, 211427205, 211427242, 211429283, 211433995, 211634985, 211635022, 211635059, 211637063, 211637100, 211639141, 211641775, 211641812, 211643853, 211648565, 211721319, 211721356, 211723397, 211728109, 211736761, 211891737, 211896247, 211896284, 211896321, 211896358, 211898325, 211898362, 211898399, 211900403, 211900440, 211902481, 211903037, 211903074, 211903111, 211905115, 211905152, 211907193, 211909827, 211909864, 211911905, 211916617, 211944541, 211944578, 211946619, 211951331, 211962629, 212030875, 212159763, 212205803, 212205840, 212205877, 212207881, 212207918, 212209959, 212212593, 212212630, 212214671, 212219383, 212249809, 212254097, 212406738, 212416059, 212416096, 212416133, 212418137, 212418174, 212420215, 212422849, 212422886, 212424927, 212429639, 212486951, 212486988, 212487025, 212489029, 212489066, 212491107, 212493741, 212493778, 212495819, 212500531, 212502393, 212502430, 212504471, 212509183, 212515359, 212515396, 212517437, 212517835, 212522149, 212725615, 212725652, 212727693, 212732405, 212796507, 212796544, 212798585, 212803297, 212811949, 212824915, 212860465, 212860502, 212862543, 212867255, 212986877, 212986914, 212986951, 212988955, 212988992, 212991033, 212993667, 212993704, 212995745, 213000457, 213035171, 213106063, 213170021, 213238663, 213238700, 213240741, 213245453, 213296433, 213296470, 213298511, 213303223, 213324997, 213481926, 213506689, 213506726, 213508767, 213513479, 213548219, 213577581, 213577618, 213579659, 213584371, 213593023, 213605989, 213816245, 213849319, 213849356, 213849393, 213851397, 213851434, 213853475, 213856109, 213856146, 213858187, 213862899, 213887137, 213935653, 213935690, 213937731, 213942443, 213951095, 214077507, 214077544, 214079585, 214084297, 214158875, 214158912, 214160953, 214165665, 214245209, 214329293, 214387063, 214400185, 214420137, 214420174, 214420211, 214422215, 214422252, 214424293, 214426927, 214426964, 214429005, 214433717, 214468431, 214597319, 214668211, 214729693, 214729730, 214731771, 214736483, 214924507, 214924544, 214924581, 214926585, 214926622, 214928663, 214931297, 214931334, 214933375, 214938087, 214939949, 214939986, 214942027, 214946739, 215010841, 215010878, 215012919, 215017631, 215026283, 215039249, 215168137, 215234063, 215234100, 215236141, 215240853, 215249505, 215320397, 215384355, 215491847, 215491855, 215491884, 215491886, 215491892, 215491921, 215491923, 215491929, 215491958, 215491960, 215491966, 215491995, 215491997, 215492003, 215492034, 215493925, 215493933, 215493962, 215493964, 215493970, 215493999, 215494001, 215494007, 215494036, 215494038, 215494044, 215494075, 215495325, 215495362, 215495399, 215496003, 215496011, 215496040, 215496042, 215496048, 215496077, 215496079, 215496085, 215496116, 215497403, 215497440, 215498081, 215498089, 215498118, 215498120, 215498126, 215498157, 215498637, 215498645, 215498674, 215498676, 215498682, 215498711, 215498713, 215498719, 215498748, 215498750, 215498756, 215498787, 215499481, 215500159, 215500167, 215500198, 215500715, 215500723, 215500752, 215500754, 215500760, 215500789, 215500791, 215500797, 215500828, 215502115, 215502152, 215502793, 215502801, 215502830, 215502832, 215502838, 215502869, 215504193, 215504871, 215504879, 215504910, 215505427, 215505435, 215505464, 215505466, 215505472, 215505501, 215505503, 215505509, 215505540, 215507505, 215507513, 215507542, 215507544, 215507550, 215507581, 215508905, 215509583, 215509591, 215509622, 215510767, 215510804, 215512217, 215512225, 215512254, 215512256, 215512262, 215512293, 215512845, 215514295, 215514303, 215514334, 215517557, 215519007, 215519015, 215519046, 215543619, 215762553, 215801403, 215801411, 215801440, 215801442, 215801448, 215801477, 215801479, 215801485, 215801514, 215801516, 215801522, 215801553, 215803481, 215803489, 215803518, 215803520, 215803526, 215803555, 215803557, 215803563, 215803594, 215804881, 215804918, 215805559, 215805567, 215805596, 215805598, 215805604, 215805635, 215806959, 215807637, 215807645, 215807676, 215808193, 215808201, 215808230, 215808232, 215808238, 215808267, 215808269, 215808275, 215808306, 215810271, 215810279, 215810308, 215810310, 215810316, 215810347, 215811671, 215812349, 215812357, 215812388, 215814983, 215814991, 215815020, 215815022, 215815028, 215815059, 215817061, 215817069, 215817100, 215820323, 215821773, 215821781, 215821812, 215919482, 216015137, 216015174, 216017215, 216021927, 216030579, 216086029, 216086066, 216088107, 216092819, 216101471, 216110959, 216110967, 216110996, 216110998, 216111004, 216111033, 216111035, 216111041, 216111072, 216113037, 216113045, 216113074, 216113076, 216113082, 216113113, 216114437, 216115115, 216115123, 216115154, 216117749, 216117757, 216117786, 216117788, 216117794, 216117825, 216119827, 216119835, 216119866, 216124539, 216124547, 216124578, 216324693, 216373209, 216373246, 216375287, 216379999, 216395585, 216420515, 216420523, 216420552, 216420554, 216420560, 216420591, 216422593, 216422601, 216422632, 216427305, 216427313, 216427344, 216459543, 216582477, 216582485, 216582514, 216582516, 216582522, 216582551, 216582553, 216582559, 216582588, 216582590, 216582596, 216582627, 216584555, 216584563, 216584592, 216584594, 216584600, 216584629, 216584631, 216584637, 216584668, 216585955, 216585992, 216586633, 216586641, 216586670, 216586672, 216586678, 216586709, 216588033, 216588711, 216588719, 216588750, 216589267, 216589275, 216589304, 216589306, 216589312, 216589341, 216589343, 216589349, 216589380, 216591345, 216591353, 216591382, 216591384, 216591390, 216591421, 216592745, 216593423, 216593431, 216593462, 216596057, 216596065, 216596094, 216596096, 216596102, 216596133, 216598135, 216598143, 216598174, 216601397, 216602847, 216602855, 216602886, 216682765, 216730071, 216730079, 216730110, 216837741, 216892033, 216892041, 216892070, 216892072, 216892078, 216892107, 216892109, 216892115, 216892146, 216894111, 216894119, 216894148, 216894150, 216894156, 216894187, 216895511, 216896189, 216896197, 216896228, 216898823, 216898831, 216898860, 216898862, 216898868, 216898899, 216900901, 216900909, 216900940, 216905613, 216905621, 216905652, 216944027, 216944064, 216946105, 216950817, 217105767, 217176659, 217201589, 217201597, 217201626, 217201628, 217201634, 217201665, 217203667, 217203675, 217203706, 217208379, 217208387, 217208418, 217253583, 217362063, 217362100, 217362137, 217364141, 217364178, 217366219, 217368853, 217368890, 217370931, 217375643, 217448397, 217448434, 217450475, 217455187, 217463839, 217511145, 217511153, 217511184, 217534731, 217671619, 217671656, 217673107, 217673115, 217673144, 217673146, 217673152, 217673181, 217673183, 217673189, 217673220, 217673697, 217675185, 217675193, 217675222, 217675224, 217675230, 217675261, 217676585, 217677263, 217677271, 217677302, 217678409, 217679897, 217679905, 217679934, 217679936, 217679942, 217679973, 217681975, 217681983, 217682014, 217686687, 217686695, 217686726, 217757953, 217932881, 217932918, 217932955, 217934959, 217934996, 217937037, 217939671, 217939708, 217941749, 217946461, 217981175, 217982663, 217982671, 217982700, 217982702, 217982708, 217982739, 217984741, 217984749, 217984780, 217989453, 217989461, 217989492, 218015737, 218015745, 218015774, 218015776, 218015782, 218015811, 218015813, 218015819, 218015848, 218015850, 218015856, 218015887, 218017815, 218017823, 218017852, 218017854, 218017860, 218017889, 218017891, 218017897, 218017928, 218019215, 218019252, 218019893, 218019901, 218019930, 218019932, 218019938, 218019969, 218021293, 218021971, 218021979, 218022010, 218022527, 218022535, 218022564, 218022566, 218022572, 218022601, 218022603, 218022609, 218022640, 218024605, 218024613, 218024642, 218024644, 218024650, 218024681, 218026005, 218026683, 218026691, 218026722, 218029317, 218029325, 218029354, 218029356, 218029362, 218029393, 218031395, 218031403, 218031434, 218034657, 218036107, 218036115, 218036146, 218242437, 218242474, 218244515, 218249227, 218292219, 218292227, 218292258, 218325293, 218325301, 218325330, 218325332, 218325338, 218325367, 218325369, 218325375, 218325406, 218327371, 218327379, 218327408, 218327410, 218327416, 218327447, 218328771, 218329449, 218329457, 218329488, 218332083, 218332091, 218332120, 218332122, 218332128, 218332159, 218334161, 218334169, 218334200, 218338873, 218338881, 218338912, 218452693, 218452730, 218454771, 218459483, 218523585, 218523622, 218525663, 218530375, 218539027, 218551993, 218609919, 218634849, 218634857, 218634886, 218634888, 218634894, 218634925, 218636927, 218636935, 218636966, 218641639, 218641647, 218641678, 218762249, 218763737, 218763745, 218763774, 218763776, 218763782, 218763813, 218765815, 218765823, 218765854, 218770527, 218770535, 218770566, 218833141, 218897099, 218944405, 218944413, 218944444, 219023511, 219023548, 219025589, 219030301, 219073293, 219073301, 219073332, 219090925, 219090933, 219090962, 219090964, 219090970, 219090999, 219091001, 219091007, 219091036, 219091038, 219091044, 219091075, 219093003, 219093011, 219093040, 219093042, 219093048, 219093077, 219093079, 219093085, 219093116, 219094403, 219094440, 219095081, 219095089, 219095118, 219095120, 219095126, 219095157, 219096481, 219097159, 219097167, 219097198, 219097715, 219097723, 219097752, 219097754, 219097760, 219097789, 219097791, 219097797, 219097828, 219099793, 219099801, 219099830, 219099832, 219099838, 219099869, 219101193, 219101871, 219101879, 219101910, 219104505, 219104513, 219104542, 219104544, 219104550, 219104581, 219106367, 219106375, 219106404, 219106406, 219106412, 219106441, 219106443, 219106449, 219106480, 219106583, 219106591, 219106622, 219108445, 219108453, 219108482, 219108484, 219108490, 219108521, 219109845, 219110523, 219110531, 219110562, 219111295, 219111303, 219111334, 219113157, 219113165, 219113194, 219113196, 219113202, 219113233, 219115235, 219115243, 219115274, 219119947, 219119955, 219119986, 219275297, 219333067, 219400481, 219400489, 219400518, 219400520, 219400526, 219400555, 219400557, 219400563, 219400594, 219402559, 219402567, 219402596, 219402598, 219402604, 219402635, 219403959, 219404637, 219404645, 219404676, 219407271, 219407279, 219407308, 219407310, 219407316, 219407347, 219409349, 219409357, 219409388, 219414061, 219414069, 219414100, 219415923, 219415931, 219415960, 219415962, 219415968, 219415999, 219418001, 219418009, 219418040, 219422713, 219422721, 219422752, 219467917, 219539085, 219539093, 219539124, 219543323, 219614215, 219685107, 219710037, 219710045, 219710074, 219710076, 219710082, 219710113, 219712115, 219712123, 219712154, 219716827, 219716835, 219716866, 219725479, 219725487, 219725518, 219854367, 219854375, 219854406, 219885953, 219885990, 219888031, 219892743, 219972287, 220019593, 220019601, 220019632, 220114141, 220181555, 220181563, 220181592, 220181594, 220181600, 220181629, 220181631, 220181637, 220181668, 220183633, 220183641, 220183670, 220183672, 220183678, 220183709, 220185033, 220185711, 220185719, 220185750, 220188345, 220188353, 220188382, 220188384, 220188390, 220188421, 220190423, 220190431, 220190462, 220195135, 220195143, 220195174, 220195509, 220196997, 220197005, 220197034, 220197036, 220197042, 220197073, 220199075, 220199083, 220199114, 220203787, 220203795, 220203826, 220456771, 220456808, 220458849, 220463561, 220491111, 220491119, 220491148, 220491150, 220491156, 220491187, 220493189, 220493197, 220493228, 220497901, 220497909, 220497940, 220506553, 220506561, 220506592, 220539627, 220539635, 220539664, 220539666, 220539672, 220539701, 220539703, 220539709, 220539740, 220541705, 220541713, 220541742, 220541744, 220541750, 220541781, 220543105, 220543783, 220543791, 220543822, 220546417, 220546425, 220546454, 220546456, 220546462, 220546493, 220548495, 220548503, 220548534, 220553207, 220553215, 220553246, 220766327, 220800667, 220800675, 220800706, 220849183, 220849191, 220849220, 220849222, 220849228, 220849259, 220851261, 220851269, 220851300, 220855973, 220855981, 220856012, 220961141, 220961178, 220963219, 220967931, 220976583, 221047475, 221158739, 221158747, 221158778, 221270697, 221272185, 221272193, 221272222, 221272224, 221272230, 221272261, 221274263, 221274271, 221274302, 221278975, 221278983, 221279014, 221287627, 221287635, 221287666, 221528481, 221528489, 221528518, 221528520, 221528526, 221528555, 221528557, 221528563, 221528592, 221528594, 221528600, 221528631, 221530559, 221530567, 221530596, 221530598, 221530604, 221530633, 221530635, 221530641, 221530672, 221531959, 221531996, 221532637, 221532645, 221532674, 221532676, 221532682, 221532713, 221534037, 221534715, 221534723, 221534754, 221535271, 221535279, 221535308, 221535310, 221535316, 221535345, 221535347, 221535353, 221535384, 221537349, 221537357, 221537386, 221537388, 221537394, 221537425, 221538749, 221539427, 221539435, 221539466, 221542061, 221542069, 221542098, 221542100, 221542106, 221542137, 221544139, 221544147, 221544178, 221547401, 221548851, 221548859, 221548890, 221581741, 221581749, 221581780, 221614815, 221614823, 221614852, 221614854, 221614860, 221614889, 221614891, 221614897, 221614928, 221616893, 221616901, 221616930, 221616932, 221616938, 221616969, 221618293, 221618971, 221618979, 221619010, 221621605, 221621613, 221621642, 221621644, 221621650, 221621681, 221623683, 221623691, 221623722, 221628395, 221628403, 221628434, 221630257, 221630265, 221630294, 221630296, 221630302, 221630333, 221632335, 221632343, 221632374, 221637047, 221637055, 221637086, 221838037, 221838045, 221838074, 221838076, 221838082, 221838111, 221838113, 221838119, 221838150, 221840115, 221840123, 221840152, 221840154, 221840160, 221840191, 221841515, 221842193, 221842201, 221842232, 221844827, 221844835, 221844864, 221844866, 221844872, 221844903, 221846905, 221846913, 221846944, 221851617, 221851625, 221851656, 221924371, 221924379, 221924408, 221924410, 221924416, 221924447, 221926449, 221926457, 221926488, 221931161, 221931169, 221931200, 221939813, 221939821, 221939852, 222051771, 222056281, 222056318, 222056355, 222056392, 222058359, 222058396, 222058433, 222060437, 222060474, 222062515, 222063071, 222063108, 222063145, 222065149, 222065186, 222067227, 222069861, 222069898, 222071939, 222076651, 222122663, 222147593, 222147601, 222147630, 222147632, 222147638, 222147669, 222149671, 222149679, 222149710, 222154383, 222154391, 222154422, 222233927, 222233935, 222233966, 222362815, 222362823, 222362854, 222365837, 222365874, 222365911, 222367915, 222367952, 222369993, 222372627, 222372664, 222374705, 222379417, 222409843, 222457149, 222457157, 222457188, 222619111, 222619119, 222619148, 222619150, 222619156, 222619185, 222619187, 222619193, 222619224, 222621189, 222621197, 222621226, 222621228, 222621234, 222621265, 222622589, 222623267, 222623275, 222623306, 222625901, 222625909, 222625938, 222625940, 222625946, 222625977, 222627979, 222627987, 222628018, 222632691, 222632699, 222632730, 222675393, 222675430, 222677471, 222682183, 222690003, 222690011, 222690040, 222690042, 222690048, 222690077, 222690079, 222690085, 222690116, 222692081, 222692089, 222692118, 222692120, 222692126, 222692157, 222693481, 222694159, 222694167, 222694198, 222696793, 222696801, 222696830, 222696832, 222696838, 222696869, 222698871, 222698879, 222698910, 222703583, 222703591, 222703622, 222705445, 222705453, 222705482, 222705484, 222705490, 222705521, 222707523, 222707531, 222707562, 222712235, 222712243, 222712274, 222720887, 222720895, 222720926, 222928667, 222928675, 222928704, 222928706, 222928712, 222928743, 222930745, 222930753, 222930784, 222935457, 222935465, 222935496, 222980661, 222984949, 222999559, 222999567, 222999596, 222999598, 222999604, 222999635, 223001637, 223001645, 223001676, 223006349, 223006357, 223006388, 223015001, 223015009, 223015040, 223063517, 223063525, 223063554, 223063556, 223063562, 223063593, 223065595, 223065603, 223065634, 223070307, 223070315, 223070346, 223146911, 223146948, 223146985, 223148989, 223149026, 223151067, 223153701, 223153738, 223155779, 223160491, 223238223, 223238231, 223238262, 223309115, 223309123, 223309154, 223373073, 223373081, 223373112, 223398697, 223398734, 223400775, 223405487, 223456467, 223456504, 223458545, 223463257, 223485031, 223708253, 223709741, 223709749, 223709778, 223709780, 223709786, 223709817, 223711819, 223711827, 223711858, 223716531, 223716539, 223716570, 223766023, 223780633, 223780641, 223780670, 223780672, 223780678, 223780709, 223782711, 223782719, 223782750, 223787423, 223787431, 223787462, 223796075, 223796083, 223796114, 223969515, 223969552, 223971593, 223976305, 224019297, 224019305, 224019336, 224052371, 224052379, 224052408, 224052410, 224052416, 224052445, 224052447, 224052453, 224052484, 224054449, 224054457, 224054486, 224054488, 224054494, 224054525, 224055849, 224056527, 224056535, 224056566, 224059161, 224059169, 224059198, 224059200, 224059206, 224059237, 224061239, 224061247, 224061278, 224065951, 224065959, 224065990, 224090189, 224090197, 224090228, 224138705, 224138713, 224138742, 224138744, 224138750, 224138781, 224140783, 224140791, 224140822, 224145495, 224145503, 224145534, 224154147, 224154155, 224154186, 224237541, 224237578, 224239619, 224244331, 224279071, 224361927, 224361935, 224361964, 224361966, 224361972, 224362003, 224364005, 224364013, 224364044, 224368717, 224368725, 224368756, 224448261, 224448269, 224448300, 224489327, 224547097, 224560219, 224580171, 224580208, 224580245, 224582249, 224582286, 224584327, 224586961, 224586998, 224589039, 224593751, 224671483, 224671491, 224671522, 224800371, 224800379, 224800410, 224871263, 224871271, 224871302, 224889727, 224889764, 224891805, 224896517, 225060145, 225127559, 225127567, 225127596, 225127598, 225127604, 225127633, 225127635, 225127641, 225127672, 225129637, 225129645, 225129674, 225129676, 225129682, 225129713, 225131037, 225131715, 225131723, 225131754, 225134349, 225134357, 225134386, 225134388, 225134394, 225134425, 225136427, 225136435, 225136466, 225141139, 225141147, 225141178, 225143001, 225143009, 225143038, 225143040, 225143046, 225143077, 225145079, 225145087, 225145118, 225149791, 225149799, 225149830, 225199283, 225213893, 225213901, 225213930, 225213932, 225213938, 225213969, 225215971, 225215979, 225216010, 225220683, 225220691, 225220722, 225229335, 225229343, 225229374, 225328171, 225437115, 225437123, 225437152, 225437154, 225437160, 225437191, 225439193, 225439201, 225439232, 225443905, 225443913, 225443944, 225452557, 225452565, 225452596, 225523449, 225523457, 225523488, 225587407, 225587415, 225587446, 225655359, 225655396, 225655433, 225657437, 225657474, 225659515, 225662149, 225662186, 225664227, 225668939, 225670801, 225670838, 225672879, 225677591, 225746671, 225746679, 225746710, 225922587, 225964915, 225964952, 225966993, 225971705, 225980357, 226218189, 226218197, 226218226, 226218228, 226218234, 226218265, 226220267, 226220275, 226220306, 226224979, 226224987, 226225018, 226233631, 226233639, 226233670, 226274471, 226289081, 226289089, 226289118, 226289120, 226289126, 226289157, 226291159, 226291167, 226291198, 226295871, 226295879, 226295910, 226304523, 226304531, 226304562, 226493405, 226527745, 226527753, 226527784, 226576261, 226576269, 226576298, 226576300, 226576306, 226576337, 226578339, 226578347, 226578378, 226583051, 226583059, 226583090, 226598637, 226598645, 226598676, 226650334, 226662595, 226662603, 226662634, 226745989, 226746026, 226748067, 226752779, 226761431, 226885817, 226885825, 226885856, 226997775, 227055545, 227104061, 227104098, 227106139, 227110851, 227308819, 227308827, 227308858, 227379711, 227379719, 227379750, 227413617, 227565115, 227565123, 227565152, 227565154, 227565160, 227565189, 227565191, 227565197, 227565228, 227567193, 227567201, 227567230, 227567232, 227567238, 227567269, 227568593, 227569271, 227569279, 227569310, 227571905, 227571913, 227571942, 227571944, 227571950, 227571981, 227573983, 227573991, 227574022, 227578695, 227578703, 227578734, 227651449, 227651457, 227651486, 227651488, 227651494, 227651525, 227653527, 227653535, 227653566, 227658239, 227658247, 227658278, 227666891, 227666899, 227666930, 227737783, 227737791, 227737822, 227836619, 227874671, 227874679, 227874708, 227874710, 227874716, 227874747, 227876749, 227876757, 227876788, 227881461, 227881469, 227881500, 227961005, 227961013, 227961044, 228092915, 228092952, 228092989, 228094993, 228095030, 228097071, 228099705, 228099742, 228101783, 228106495, 228179249, 228179286, 228181327, 228184227, 228184235, 228184266, 228186039, 228194691, 228402471, 228402508, 228404549, 228409261, 228488805, 228655745, 228655753, 228655782, 228655784, 228655790, 228655821, 228657823, 228657831, 228657862, 228662535, 228662543, 228662574, 228663733, 228663770, 228663807, 228665811, 228665848, 228667889, 228670523, 228670560, 228672601, 228677313, 228712027, 228726637, 228726645, 228726674, 228726676, 228726682, 228726713, 228728715, 228728723, 228728754, 228733427, 228733435, 228733466, 228742079, 228742087, 228742118, 228812971, 228812979, 228813010, 228965301, 228965309, 228965340, 228973289, 228973326, 228975367, 228980079, 229036193, 229036201, 229036232, 229100151, 229100159, 229100190, 229183545, 229183582, 229185623, 229190335, 229254437, 229254474, 229256515, 229261227, 229269879, 229282845, 229435331, 229493101, 229563993, 229627951, 229746375, 229746383, 229746414, 229754363, 229754400, 229756441, 229761153, 229817267, 229817275, 229817306, 229888159, 229888167, 229888198, 230006149, 230063919, 230089005, 230089013, 230089042, 230089044, 230089050, 230089081, 230091083, 230091091, 230091122, 230095795, 230095803, 230095834, 230175339, 230175347, 230175378, 230274175, 230345067, 230398561, 230398569, 230398600, 230616805, 230616842, 230618883, 230623595, 230677132, 230677169, 230677206, 230677243, 230677280, 230679210, 230679247, 230679284, 230679321, 230681288, 230681325, 230681362, 230683366, 230683403, 230683922, 230683959, 230683996, 230684033, 230685444, 230686000, 230686037, 230686074, 230688078, 230688115, 230690156, 230690712, 230690749, 230690786, 230692790, 230692827, 230694868, 230697502, 230697539, 230699580, 230703139, 230704292, 230844993, 230926361, 230986688, 230986725, 230986762, 230986799, 230988766, 230988803, 230988840, 230990844, 230990881, 230992922, 230993478, 230993515, 230993552, 230995556, 230995593, 230997634, 231000268, 231000305, 231002346, 231007058, 231164193, 231164201, 231164230, 231164232, 231164238, 231164269, 231166271, 231166279, 231166310, 231170983, 231170991, 231171022, 231179635, 231179643, 231179674, 231187623, 231187660, 231189701, 231194413, 231250527, 231250535, 231250566, 231296244, 231296281, 231296318, 231298322, 231298359, 231300400, 231303034, 231303071, 231305112, 231309824, 231473749, 231473757, 231473788, 231497179, 231605800, 231605837, 231607878, 231612590, 231691993, 231692030, 231694071, 231698783, 231707435, 231767762, 231767799, 231767836, 231767873, 231769840, 231769877, 231769914, 231771918, 231771955, 231773996, 231774552, 231774589, 231774626, 231776630, 231776667, 231778327, 231778708, 231781342, 231781379, 231783420, 231788132, 231915356, 232001549, 232077318, 232077355, 232077392, 232079396, 232079433, 232081474, 232084108, 232084145, 232086186, 232090898, 232254823, 232254831, 232254862, 232259333, 232259341, 232259370, 232259372, 232259378, 232259407, 232259409, 232259415, 232259444, 232259446, 232259452, 232259483, 232261411, 232261419, 232261448, 232261450, 232261456, 232261485, 232261487, 232261493, 232261524, 232262811, 232262848, 232263489, 232263497, 232263526, 232263528, 232263534, 232263565, 232264889, 232265567, 232265575, 232265606, 232266123, 232266131, 232266160, 232266162, 232266168, 232266197, 232266199, 232266205, 232266236, 232268201, 232268209, 232268238, 232268240, 232268246, 232268277, 232269601, 232270279, 232270287, 232270318, 232272913, 232272921, 232272950, 232272952, 232272958, 232272989, 232274991, 232274999, 232275030, 232278253, 232279703, 232279711, 232279742, 232325715, 232325723, 232325754, 232386874, 232386911, 232388952, 232393664, 232568889, 232568897, 232568926, 232568928, 232568934, 232568963, 232568965, 232568971, 232569002, 232570967, 232570975, 232571004, 232571006, 232571012, 232571043, 232572367, 232573045, 232573053, 232573084, 232575679, 232575687, 232575716, 232575718, 232575724, 232575755, 232577757, 232577765, 232577796, 232582469, 232582477, 232582508, 232612895, 232612903, 232612934, 232696430, 232782623, 232853515, 232858392, 232858429, 232858466, 232860470, 232860507, 232862548, 232865182, 232865219, 232867260, 232871972, 232878445, 232878453, 232878482, 232878484, 232878490, 232878521, 232880523, 232880531, 232880562, 232885235, 232885243, 232885274, 233140695, 233167948, 233167985, 233170026, 233174738, 233188001, 233188009, 233188040, 233201022, 233201059, 233201096, 233201133, 233203100, 233203137, 233203174, 233205178, 233205215, 233207256, 233207812, 233207849, 233207886, 233209890, 233209927, 233211968, 233214602, 233214639, 233216680, 233221392, 233349963, 233349971, 233350000, 233350002, 233350008, 233350037, 233350039, 233350045, 233350076, 233352041, 233352049, 233352078, 233352080, 233352086, 233352117, 233353441, 233354119, 233354127, 233354158, 233356753, 233356761, 233356790, 233356792, 233356798, 233356829, 233358831, 233358839, 233358870, 233363543, 233363551, 233363582, 233477504, 233510578, 233510615, 233510652, 233512656, 233512693, 233514734, 233517368, 233517405, 233519446, 233524158, 233601749, 233601757, 233601786, 233601788, 233601794, 233601825, 233603827, 233603835, 233603866, 233608539, 233608547, 233608578, 233659519, 233659527, 233659556, 233659558, 233659564, 233659595, 233661597, 233661605, 233661636, 233666309, 233666317, 233666348, 233688083, 233688091, 233688122, 233711513, 233820134, 233820171, 233822212, 233826924, 233911305, 233911313, 233911344, 233949022, 233949059, 233951100, 233955812, 233969075, 233969083, 233969114, 234129549, 234129586, 234129690, 234131627, 234136339, 234215883, 234258578, 234276210, 234276247, 234276284, 234276321, 234278288, 234278325, 234278362, 234280366, 234280403, 234282444, 234283000, 234283037, 234283074, 234285078, 234285115, 234287156, 234289790, 234289827, 234291652, 234291689, 234291726, 234291868, 234293730, 234293767, 234295808, 234296580, 234298442, 234298479, 234300520, 234305232, 234439105, 234440593, 234440601, 234440630, 234440632, 234440638, 234440669, 234442671, 234442679, 234442710, 234447383, 234447391, 234447422, 234585766, 234585803, 234585840, 234587844, 234587881, 234589922, 234592556, 234592593, 234594634, 234599346, 234601208, 234601245, 234603286, 234607998, 234692379, 234692387, 234692418, 234700367, 234700404, 234702445, 234707157, 234750149, 234750157, 234750188, 234763271, 234763279, 234763310, 234783223, 234783231, 234783260, 234783262, 234783268, 234783297, 234783299, 234783305, 234783336, 234785301, 234785309, 234785338, 234785340, 234785346, 234785377, 234786701, 234787379, 234787387, 234787418, 234790013, 234790021, 234790050, 234790052, 234790058, 234790089, 234792091, 234792099, 234792130, 234796803, 234796811, 234796842, 234895322, 234895359, 234897400, 234902112, 234910764, 235009923, 235039652, 235092779, 235092787, 235092816, 235092818, 235092824, 235092855, 235094857, 235094865, 235094896, 235099569, 235099577, 235099608, 235204878, 235220179, 235291071, 235366840, 235366877, 235366914, 235368918, 235368955, 235370996, 235373630, 235373667, 235375708, 235380420, 235382282, 235382319, 235384360, 235389072, 235402335, 235402343, 235402374, 235531223, 235531231, 235531262, 235676396, 235676433, 235678474, 235683186, 235691838, 235724912, 235724949, 235724986, 235726990, 235727027, 235729068, 235731702, 235731739, 235733780, 235738492, 235790997, 235858411, 235858419, 235858448, 235858450, 235858456, 235858485, 235858487, 235858493, 235858524, 235860489, 235860497, 235860526, 235860528, 235860534, 235860565, 235861889, 235862567, 235862575, 235862606, 235865201, 235865209, 235865238, 235865240, 235865246, 235865277, 235867279, 235867287, 235867318, 235871991, 235871999, 235872030, 235873853, 235873861, 235873890, 235873892, 235873898, 235873929, 235875931, 235875939, 235875970, 235880643, 235880651, 235880682, 235985952, 236034468, 236034505, 236036546, 236041258, 236125639, 236125647, 236125678, 236167967, 236167975, 236168004, 236168006, 236168012, 236168043, 236170045, 236170053, 236170084, 236174757, 236174765, 236174796, 236183409, 236183417, 236183448, 236344024, 236457470, 236457507, 236459548, 236464260, 236472912, 236477523, 236477531, 236477562, 236653439, 236713766, 236713803, 236713840, 236713877, 236715844, 236715881, 236715918, 236717922, 236717959, 236720000, 236720556, 236720593, 236720630, 236722634, 236722671, 236724712, 236727346, 236727383, 236729424, 236734136, 236767026, 236800100, 236800137, 236800174, 236802178, 236802215, 236804256, 236806890, 236806927, 236808968, 236813680, 236815542, 236815579, 236817620, 236822332, 236949041, 236949049, 236949078, 236949080, 236949086, 236949117, 236951119, 236951127, 236951158, 236955831, 236955839, 236955870, 236964483, 236964491, 236964522, 237023322, 237023359, 237023396, 237025400, 237025437, 237027478, 237030112, 237030149, 237032190, 237036902, 237109656, 237109693, 237111734, 237116446, 237125098, 237200827, 237200835, 237200866, 237224257, 237258597, 237258605, 237258636, 237307113, 237307121, 237307150, 237307152, 237307158, 237307189, 237309191, 237309199, 237309230, 237313903, 237313911, 237313942, 237332878, 237332915, 237334956, 237339668, 237419212, 237548100, 237616669, 237616677, 237616708, 237642434, 237728627, 237804396, 237804433, 237804470, 237806474, 237806511, 237808552, 237811186, 237811223, 237813264, 237817976, 237875288, 237875325, 237875362, 237877366, 237877403, 237879444, 237882078, 237882115, 237884156, 237888868, 237890730, 237890767, 237892808, 237897520, 237906172, 238039671, 238039679, 238039710, 238113952, 238113989, 238116030, 238120742, 238184844, 238184881, 238186922, 238191634, 238200286, 238248802, 238248839, 238250880, 238255592, 238295967, 238295975, 238296004, 238296006, 238296012, 238296041, 238296043, 238296049, 238296080, 238298045, 238298053, 238298082, 238298084, 238298090, 238298121, 238299445, 238300123, 238300131, 238300162, 238302757, 238302765, 238302794, 238302796, 238302802, 238302833, 238304835, 238304843, 238304874, 238309547, 238309555, 238309586, 238382301, 238382309, 238382338, 238382340, 238382346, 238382377, 238384379, 238384387, 238384418, 238389091, 238389099, 238389130, 238397743, 238397751, 238397782, 238423508, 238494400, 238558358, 238605523, 238605531, 238605560, 238605562, 238605568, 238605599, 238607601, 238607609, 238607640, 238612313, 238612321, 238612352, 238691857, 238691865, 238691896, 238823767, 238823804, 238823841, 238825845, 238825882, 238827923, 238830557, 238830594, 238832635, 238837347, 238895026, 238895063, 238897104, 238901816, 238915079, 238915087, 238915118, 238965918, 238965955, 238967996, 238972708, 238981360, 239133323, 239133360, 239135401, 239140113, 239204582, 239237656, 239237693, 239237730, 239239734, 239239771, 239241812, 239244446, 239244483, 239246524, 239251236, 239275474, 239323990, 239324027, 239326068, 239330780, 239339432, 239370582, 239370619, 239370656, 239370693, 239372660, 239372697, 239372734, 239374738, 239374775, 239376816, 239377372, 239377409, 239377446, 239379450, 239379487, 239381528, 239384162, 239384199, 239386240, 239386597, 239386605, 239386634, 239386636, 239386642, 239386673, 239388675, 239388683, 239388714, 239390952, 239393387, 239393395, 239393426, 239442879, 239457489, 239457497, 239457526, 239457528, 239457534, 239457565, 239459567, 239459575, 239459606, 239464279, 239464287, 239464318, 239472931, 239472939, 239472970, 239547212, 239547249, 239549290, 239554002, 239633546, 239638383, 239638391, 239638422, 239680138, 239680175, 239680212, 239682216, 239682253, 239684294, 239686928, 239686965, 239689006, 239693718, 239696153, 239696161, 239696192, 239767045, 239767053, 239767084, 239831003, 239831011, 239831042, 239856768, 239914397, 239914434, 239916475, 239921187, 239985656, 239989694, 239989731, 239991772, 239996484, 240056548, 240166183, 240223953, 240299250, 240312844, 240312881, 240312918, 240314922, 240314959, 240317000, 240319634, 240319671, 240321712, 240326424, 240328286, 240328323, 240330364, 240335076, 240399178, 240399215, 240401256, 240405968, 240414620, 240461212, 240461249, 240461286, 240463290, 240463327, 240465368, 240468002, 240468039, 240470080, 240474792, 240477227, 240477235, 240477266, 240548119, 240548127, 240548158, 240622400, 240622437, 240624478, 240629190, 240637842, 240708734, 240737001, 240770768, 240770805, 240772692, 240772846, 240777558, 240819857, 240819865, 240819894, 240819896, 240819902, 240819933, 240821935, 240821943, 240821974, 240826647, 240826655, 240826686, 240837166, 240837203, 240837240, 240837277, 240837314, 240839244, 240839281, 240839318, 240839355, 240841322, 240841359, 240841396, 240843400, 240843437, 240843956, 240843993, 240844030, 240844067, 240845478, 240846034, 240846071, 240846108, 240848112, 240848149, 240850190, 240850746, 240850783, 240850820, 240852824, 240852861, 240854902, 240857536, 240857573, 240859614, 240864326, 240906191, 240906199, 240906230, 240931956, 241005027, 241080324, 241129413, 241129421, 241129452, 241146722, 241146759, 241146796, 241146833, 241148800, 241148837, 241148874, 241150878, 241150915, 241152956, 241153512, 241153549, 241153586, 241155590, 241155627, 241157668, 241160302, 241160339, 241162380, 241167092, 241347657, 241347694, 241349735, 241354447, 241403474, 241403511, 241405552, 241410264, 241418916, 241456278, 241456315, 241456352, 241458356, 241458393, 241460434, 241463068, 241463105, 241465146, 241469858, 241474366, 241474403, 241476444, 241481156, 241489808, 241551842, 241551879, 241553920, 241558632, 241657213, 241713030, 241761546, 241761583, 241763624, 241765834, 241765871, 241767912, 241768336, 241772624, 241783922, 241847880, 241861398, 241894472, 241894509, 241894546, 241895045, 241895053, 241895082, 241895084, 241895090, 241895121, 241896550, 241896587, 241897123, 241897131, 241897162, 241898628, 241901262, 241901299, 241901835, 241901843, 241901874, 241903340, 241908052, 241910487, 241910495, 241910526, 241927796, 241927833, 241927870, 241927907, 241929874, 241929911, 241929948, 241931952, 241931989, 241934030, 241934586, 241934623, 241934660, 241936664, 241936701, 241938742, 241941376, 241941413, 241943454, 241948166, 241981379, 241981387, 241981418, 242071102, 242075390, 242204028, 242204065, 242204601, 242204609, 242204640, 242206106, 242210818, 242237352, 242237389, 242237426, 242239430, 242239467, 242241508, 242244142, 242244179, 242246220, 242250932, 242422845, 242422882, 242424923, 242429635, 242438287, 242494104, 242513584, 242546908, 242546945, 242548986, 242553698, 242564996, 242642472, 242732401, 242750400, 242750437, 242750474, 242752478, 242752515, 242754556, 242757190, 242757227, 242759268, 242763980, 242836734, 242836771, 242838812, 242843524, 242852176, 242856464, 242923068, 242969660, 242969697, 242969734, 242971738, 242971775, 242973816, 242976450, 242976487, 242978528, 242983240, 242985102, 242985139, 242985675, 242985683, 242985714, 242987180, 242991892, 243018426, 243018463, 243018500, 243020504, 243020541, 243022582, 243025216, 243025253, 243027294, 243032006, 243056567, 243056575, 243056606, 243059956, 243059993, 243062034, 243066746, 243146290, 243279216, 243279253, 243281294, 243286006, 243294658, 243327982, 243328019, 243330060, 243334772, 243343747, 243343755, 243343786, 243361056, 243361093, 243361130, 243361167, 243363134, 243363171, 243363208, 243365212, 243365249, 243367290, 243367846, 243367883, 243367920, 243369512, 243369924, 243369961, 243372002, 243374636, 243374673, 243376714, 243381426, 243513475, 243588772, 243637538, 243670612, 243670649, 243670686, 243672690, 243672727, 243674768, 243677402, 243677439, 243679480, 243684192, 243841030, 243841067, 243843108, 243847820, 243871547, 243911922, 243911959, 243914000, 243918712, 243927364, 243980168, 243980205, 243982246, 243986958, 243998256, 244060290, 244060327, 244062368, 244067080, 244075732, 244109056, 244109093, 244111134, 244115846, 244150586, 244221478, 244285436, 244289724, 244332601, 244332609, 244332638, 244332640, 244332646, 244332677, 244334679, 244334687, 244334718, 244339391, 244339399, 244339430, 244369846, 244418362, 244418399, 244418612, 244418935, 244418943, 244418974, 244420440, 244425152, 244436244, 244436281, 244436318, 244436355, 244438322, 244438359, 244438396, 244440400, 244440437, 244442478, 244443034, 244443071, 244443108, 244445112, 244445149, 244447190, 244449824, 244449861, 244451686, 244451723, 244451760, 244451902, 244453764, 244453801, 244455842, 244456614, 244458476, 244458513, 244460554, 244465266, 244642157, 244642165, 244642196, 244727918, 244745800, 244745837, 244745874, 244747878, 244747915, 244749956, 244752590, 244752627, 244754668, 244759380, 244761242, 244761279, 244763320, 244768032, 244860401, 244860438, 244862479, 244867191, 244931660, 244946735, 245002552, 245055356, 245055393, 245057434, 245062146, 245070798, 245073444, 245150920, 245169957, 245199686, 245274290, 245274327, 245276368, 245281080, 245360624, 245364912, 245407216, 245407253, 245407290, 245409294, 245409331, 245411372, 245414006, 245414043, 245416084, 245420796, 245423231, 245423239, 245423270, 245431219, 245431256, 245433297, 245438009, 245493550, 245493587, 245494123, 245494131, 245494162, 245495628, 245500340, 245508992, 245526874, 245526911, 245526948, 245528952, 245528989, 245531030, 245533664, 245533701, 245535742, 245540454, 245542316, 245542353, 245544394, 245549106, 245583846, 245716772, 245716809, 245718850, 245723562, 245740775, 245803106, 245836430, 245836467, 245838508, 245843220, 245851872, 245884946, 245884983, 245885020, 245887024, 245887061, 245889102, 245891736, 245891773, 245893814, 245898526, 245951031, 246021923, 246026328, 246145986, 246194502, 246194539, 246196580, 246201292, 246349478, 246349515, 246351556, 246356268, 246364920, 246435812, 246497846, 246497883, 246499924, 246504058, 246504636, 246521849, 246568738, 246568775, 246570816, 246575528, 246584180, 246617504, 246617541, 246619582, 246624294, 246632946, 246659034, 246807402, 246856491, 246856499, 246856530, 246873800, 246873837, 246873874, 246873911, 246875878, 246875915, 246875952, 246877956, 246877993, 246878294, 246880034, 246880590, 246880627, 246880664, 246882668, 246882705, 246884746, 246887380, 246887417, 246889458, 246894170, 246927060, 246942252, 246960134, 246960171, 246960208, 246962212, 246962249, 246964290, 246966924, 246966961, 246969002, 246973714, 246975576, 246975613, 246977654, 246982366, 247183356, 247183393, 247183430, 247185434, 247185471, 247187512, 247190146, 247190183, 247192224, 247196936, 247269690, 247269727, 247271768, 247276480, 247285132, 247384291, 247440108, 247444618, 247444655, 247444692, 247444729, 247446696, 247446733, 247446770, 247448774, 247448811, 247450852, 247451408, 247451445, 247451482, 247453486, 247453523, 247455564, 247458198, 247458235, 247460276, 247464988, 247492912, 247492949, 247494990, 247499702, 247511000, 247579246, 247588476, 247659368, 247708134, 247754174, 247754211, 247754248, 247756252, 247756289, 247758330, 247760964, 247761001, 247763042, 247767754, 247798180, 247802468, 247931106, 247931143, 247931679, 247931687, 247931718, 247933184, 247937896, 247955109, 247964430, 247964467, 247964504, 247966508, 247966545, 247968586, 247971220, 247971257, 247973298, 247978010, 248017440, 248035322, 248035359, 248035396, 248037400, 248037437, 248039478, 248042112, 248042149, 248044190, 248048902, 248050764, 248050801, 248052842, 248057554, 248063730, 248063767, 248065808, 248066206, 248070520, 248240662, 248273986, 248274023, 248276064, 248280776, 248344878, 248344915, 248346956, 248351668, 248360320, 248373286, 248408836, 248408873, 248410914, 248415626, 248459479, 248535248, 248535285, 248535322, 248537326, 248537363, 248539404, 248542038, 248542075, 248544116, 248548828, 248583542, 248654434, 248718392, 248787034, 248787071, 248789112, 248793824, 248844804, 248844841, 248846882, 248851594, 248873368, 249006294, 249006331, 249008372, 249013084, 249021736, 249026819, 249026827, 249026856, 249026858, 249026864, 249026893, 249026895, 249026901, 249026932, 249028897, 249028905, 249028934, 249028936, 249028942, 249028973, 249030297, 249030975, 249030983, 249031014, 249033609, 249033617, 249033646, 249033648, 249033654, 249033685, 249035687, 249035695, 249035726, 249040399, 249040407, 249040438, 249055060, 249055097, 249057138, 249061850, 249092628, 249096590, 249125952, 249125989, 249128030, 249132742, 249141394, 249154360, 249315850, 249336375, 249336383, 249336412, 249336414, 249336420, 249336451, 249338453, 249338461, 249338492, 249343165, 249343173, 249343204, 249364616, 249397690, 249397727, 249397764, 249399768, 249399805, 249401846, 249404480, 249404517, 249406558, 249411270, 249435508, 249484024, 249484061, 249486102, 249490814, 249499466, 249625878, 249625915, 249627956, 249632668, 249645931, 249645939, 249645970, 249707246, 249707283, 249709324, 249714036, 249793580, 249877664, 249935434, 249948556, 249968508, 249968545, 249968582, 249970586, 249970623, 249972664, 249975298, 249975335, 249977376, 249982088, 250016802, 250096924, 250117449, 250117457, 250117486, 250117488, 250117494, 250117525, 250119527, 250119535, 250119566, 250124239, 250124247, 250124278, 250145690, 250167816, 250216582, 250278064, 250278101, 250280142, 250284854, 250369235, 250369243, 250369274, 250427005, 250427013, 250427044, 250454996, 250472878, 250472915, 250472952, 250474956, 250474993, 250477034, 250479668, 250479705, 250481746, 250486458, 250488320, 250488357, 250490398, 250495110, 250559212, 250559249, 250561290, 250566002, 250574654, 250587620, 250716508, 250782434, 250782471, 250784512, 250789224, 250797876, 250868768, 250897035, 250932726, 250997200, 250997237, 250997274, 250997311, 250997348, 250999278, 250999315, 250999352, 250999389, 251001356, 251001393, 251001430, 251003434, 251003471, 251003990, 251004027, 251004064, 251004101, 251005512, 251006068, 251006105, 251006142, 251008146, 251008183, 251010224, 251010780, 251010817, 251010854, 251012858, 251012895, 251014936, 251017570, 251017607, 251019648, 251024360, 251040218, 251040226, 251040255, 251040257, 251040263, 251040292, 251040294, 251040300, 251040329, 251040331, 251040337, 251040366, 251040368, 251040374, 251040405, 251042296, 251042304, 251042333, 251042335, 251042341, 251042370, 251042372, 251042378, 251042407, 251042409, 251042415, 251042446, 251043696, 251043733, 251043770, 251044374, 251044382, 251044411, 251044413, 251044419, 251044448, 251044450, 251044456, 251044487, 251045774, 251045811, 251046452, 251046460, 251046489, 251046491, 251046497, 251046528, 251047008, 251047016, 251047045, 251047047, 251047053, 251047082, 251047084, 251047090, 251047119, 251047121, 251047127, 251047158, 251047852, 251048530, 251048538, 251048569, 251049086, 251049094, 251049123, 251049125, 251049131, 251049160, 251049162, 251049168, 251049199, 251050486, 251050523, 251051164, 251051172, 251051201, 251051203, 251051209, 251051240, 251052564, 251053242, 251053250, 251053281, 251053798, 251053806, 251053835, 251053837, 251053843, 251053872, 251053874, 251053880, 251053911, 251055876, 251055884, 251055913, 251055915, 251055921, 251055952, 251057276, 251057954, 251057962, 251057993, 251059138, 251059175, 251060588, 251060596, 251060625, 251060627, 251060633, 251060664, 251061216, 251062666, 251062674, 251062705, 251065928, 251067378, 251067386, 251067417, 251091990, 251208079, 251208087, 251208118, 251306756, 251306793, 251306830, 251306867, 251308834, 251308871, 251308908, 251310912, 251310924, 251310949, 251312990, 251313546, 251313583, 251313620, 251315624, 251315661, 251317702, 251320336, 251320373, 251322414, 251327126, 251349774, 251349782, 251349811, 251349813, 251349819, 251349848, 251349850, 251349856, 251349885, 251349887, 251349893, 251349924, 251351852, 251351860, 251351889, 251351891, 251351897, 251351926, 251351928, 251351934, 251351965, 251353252, 251353289, 251353930, 251353938, 251353967, 251353969, 251353975, 251354006, 251355330, 251356008, 251356016, 251356047, 251356564, 251356572, 251356601, 251356603, 251356609, 251356638, 251356640, 251356646, 251356677, 251358642, 251358650, 251358679, 251358681, 251358687, 251358718, 251360042, 251360720, 251360728, 251360759, 251363354, 251363362, 251363391, 251363393, 251363399, 251363430, 251365432, 251365440, 251365471, 251368694, 251370144, 251370152, 251370183, 251443850, 251443887, 251445928, 251450640, 251467853, 251530184, 251550709, 251550717, 251550746, 251550748, 251550754, 251550785, 251552787, 251552795, 251552826, 251557499, 251557507, 251557538, 251563508, 251563545, 251565586, 251570298, 251578950, 251616312, 251616349, 251616386, 251618390, 251618427, 251620468, 251623102, 251623139, 251625180, 251629892, 251634400, 251634437, 251636478, 251641190, 251649842, 251659330, 251659338, 251659367, 251659369, 251659375, 251659404, 251659406, 251659412, 251659443, 251661408, 251661416, 251661445, 251661447, 251661453, 251661484, 251662808, 251663486, 251663494, 251663525, 251666120, 251666128, 251666157, 251666159, 251666165, 251666196, 251668198, 251668206, 251668237, 251672910, 251672918, 251672949, 251753406, 251860265, 251860273, 251860304, 251873064, 251921580, 251921617, 251923658, 251925868, 251925905, 251927946, 251928370, 251932658, 251943956, 251968886, 251968894, 251968923, 251968925, 251968931, 251968962, 251970964, 251970972, 251971003, 251975676, 251975684, 251975715, 252007914, 252087830, 252087867, 252087904, 252087941, 252089908, 252089945, 252089982, 252091986, 252092023, 252094064, 252094620, 252094657, 252094694, 252096698, 252096735, 252098776, 252101410, 252101447, 252103488, 252108200, 252130848, 252130856, 252130885, 252130887, 252130893, 252130922, 252130924, 252130930, 252130959, 252130961, 252130967, 252130998, 252132926, 252132934, 252132963, 252132965, 252132971, 252133000, 252133002, 252133008, 252133039, 252134326, 252134363, 252135004, 252135012, 252135041, 252135043, 252135049, 252135080, 252136404, 252137082, 252137090, 252137121, 252137638, 252137646, 252137675, 252137677, 252137683, 252137712, 252137714, 252137720, 252137751, 252139716, 252139724, 252139753, 252139755, 252139761, 252139792, 252141116, 252141794, 252141802, 252141833, 252144428, 252144436, 252144465, 252144467, 252144473, 252144504, 252146506, 252146514, 252146545, 252149768, 252151218, 252151226, 252151257, 252231136, 252235424, 252278442, 252278450, 252278481, 252386112, 252397386, 252397423, 252397460, 252399464, 252399501, 252401542, 252404176, 252404213, 252406254, 252410966, 252440404, 252440412, 252440441, 252440443, 252440449, 252440478, 252440480, 252440486, 252440517, 252442482, 252442490, 252442519, 252442521, 252442527, 252442558, 252443882, 252444560, 252444568, 252444599, 252447194, 252447202, 252447231, 252447233, 252447239, 252447270, 252449272, 252449280, 252449311, 252453984, 252453992, 252454023, 252492398, 252492435, 252494476, 252499188, 252534480, 252605372, 252625897, 252625905, 252625934, 252625936, 252625942, 252625973, 252627975, 252627983, 252628014, 252632687, 252632695, 252632726, 252641339, 252641347, 252641378, 252654138, 252706942, 252706979, 252709020, 252713732, 252725030, 252749960, 252749968, 252749997, 252749999, 252750005, 252750036, 252752038, 252752046, 252752077, 252756750, 252756758, 252756789, 252801954, 252910434, 252910471, 252910508, 252912512, 252912549, 252914590, 252917224, 252917261, 252919302, 252924014, 252935453, 252935461, 252935492, 252996768, 252996805, 252998846, 253003558, 253012210, 253016498, 253059516, 253059524, 253059555, 253083102, 253178460, 253178497, 253178534, 253180538, 253180575, 253182616, 253185250, 253185287, 253187328, 253192040, 253219990, 253220027, 253221478, 253221486, 253221515, 253221517, 253221523, 253221552, 253221554, 253221560, 253221591, 253222068, 253223556, 253223564, 253223593, 253223595, 253223601, 253223632, 253224956, 253225634, 253225642, 253225673, 253226780, 253228268, 253228276, 253228305, 253228307, 253228313, 253228344, 253230346, 253230354, 253230385, 253235058, 253235066, 253235097, 253306324, 253481252, 253481289, 253481326, 253483330, 253483367, 253485408, 253488016, 253488042, 253488053, 253488079, 253490094, 253490120, 253494806, 253494832, 253521090, 253521127, 253521164, 253521201, 253523168, 253523205, 253523242, 253525246, 253525283, 253527324, 253527880, 253527917, 253527954, 253529546, 253529958, 253529995, 253531034, 253531042, 253531071, 253531073, 253531079, 253531110, 253532036, 253533112, 253533120, 253533151, 253534670, 253534707, 253536748, 253537824, 253537832, 253537863, 253541460, 253564108, 253564116, 253564145, 253564147, 253564153, 253564182, 253564184, 253564190, 253564219, 253564221, 253564227, 253564258, 253566186, 253566194, 253566223, 253566225, 253566231, 253566260, 253566262, 253566268, 253566299, 253567586, 253567623, 253568264, 253568272, 253568301, 253568303, 253568309, 253568340, 253569664, 253570342, 253570350, 253570381, 253570898, 253570906, 253570935, 253570937, 253570943, 253570972, 253570974, 253570980, 253571011, 253572976, 253572984, 253573013, 253573015, 253573021, 253573052, 253574376, 253575054, 253575062, 253575093, 253577688, 253577696, 253577725, 253577727, 253577733, 253577764, 253579766, 253579774, 253579805, 253583028, 253584478, 253584486, 253584517, 253716527, 253716535, 253716566, 253790808, 253790845, 253792886, 253797572, 253797598, 253830646, 253830683, 253830720, 253832724, 253832761, 253834802, 253837436, 253837473, 253839514, 253840590, 253840598, 253840629, 253844226, 253873664, 253873672, 253873701, 253873703, 253873709, 253873738, 253873740, 253873746, 253873777, 253875742, 253875750, 253875779, 253875781, 253875787, 253875818, 253877142, 253877820, 253877828, 253877859, 253880454, 253880462, 253880491, 253880493, 253880499, 253880530, 253882532, 253882540, 253882571, 253887244, 253887252, 253887283, 253967740, 254001064, 254001101, 254003142, 254007854, 254071956, 254071993, 254074034, 254074599, 254074607, 254074638, 254078746, 254087398, 254100364, 254140202, 254140239, 254142280, 254146992, 254158290, 254183220, 254183228, 254183257, 254183259, 254183265, 254183296, 254185298, 254185306, 254185337, 254190010, 254190018, 254190049, 254269090, 254269127, 254271168, 254275880, 254310620, 254312108, 254312116, 254312145, 254312147, 254312153, 254312184, 254314186, 254314194, 254314225, 254318898, 254318906, 254318937, 254381512, 254445470, 254449758, 254492776, 254492784, 254492815, 254571882, 254571919, 254573960, 254578646, 254578672, 254596278, 254596315, 254596352, 254596389, 254598356, 254598393, 254598430, 254600434, 254600471, 254602512, 254603068, 254603105, 254603142, 254605146, 254605183, 254607224, 254609858, 254609895, 254611720, 254611757, 254611794, 254611936, 254613798, 254613835, 254615876, 254616648, 254618510, 254618547, 254620588, 254621664, 254621672, 254621703, 254625300, 254639296, 254639304, 254639333, 254639335, 254639341, 254639370, 254639372, 254639378, 254639407, 254639409, 254639415, 254639446, 254641374, 254641382, 254641411, 254641413, 254641419, 254641448, 254641450, 254641456, 254641487, 254642774, 254642811, 254643452, 254643460, 254643489, 254643491, 254643497, 254643528, 254644852, 254645530, 254645538, 254645569, 254646086, 254646094, 254646123, 254646125, 254646131, 254646160, 254646162, 254646168, 254646199, 254648164, 254648172, 254648201, 254648203, 254648209, 254648240, 254649564, 254650242, 254650250, 254650281, 254652876, 254652884, 254652913, 254652915, 254652921, 254652952, 254654738, 254654746, 254654775, 254654777, 254654783, 254654812, 254654814, 254654820, 254654851, 254654954, 254654962, 254654993, 254656816, 254656824, 254656853, 254656855, 254656861, 254656892, 254658216, 254658894, 254658902, 254658933, 254659666, 254659674, 254659705, 254661528, 254661536, 254661565, 254661567, 254661573, 254661604, 254663606, 254663614, 254663645, 254668318, 254668326, 254668357, 254823668, 254881438, 254905834, 254905871, 254905908, 254907912, 254907949, 254909990, 254912624, 254912661, 254914702, 254919414, 254921276, 254921313, 254923354, 254928066, 254948852, 254948860, 254948889, 254948891, 254948897, 254948926, 254948928, 254948934, 254948965, 254950930, 254950938, 254950967, 254950969, 254950975, 254951006, 254952330, 254953008, 254953016, 254953047, 254955642, 254955650, 254955679, 254955681, 254955687, 254955718, 254957720, 254957728, 254957759, 254962432, 254962440, 254962471, 254964294, 254964302, 254964331, 254964333, 254964339, 254964370, 254966372, 254966380, 254966411, 254971084, 254971092, 254971123, 255016288, 255042928, 255063453, 255063461, 255063490, 255063492, 255063498, 255063529, 255065531, 255065539, 255065570, 255070243, 255070251, 255070282, 255091694, 255149787, 255149795, 255149826, 255162586, 255215390, 255215427, 255217468, 255222180, 255230832, 255233478, 255258408, 255258416, 255258445, 255258447, 255258453, 255258484, 255260486, 255260494, 255260525, 255265198, 255265206, 255265237, 255273850, 255273858, 255273889, 255359720, 255373009, 255373017, 255373048, 255402738, 255402746, 255402777, 255434324, 255434361, 255436402, 255441114, 255520658, 255524946, 255567964, 255567972, 255568003, 255591253, 255591290, 255593331, 255598043, 255662512, 255686908, 255686945, 255686982, 255688986, 255689023, 255691064, 255693698, 255693735, 255695776, 255700488, 255702350, 255702387, 255704428, 255709140, 255729926, 255729934, 255729963, 255729965, 255729971, 255730000, 255730002, 255730008, 255730039, 255732004, 255732012, 255732041, 255732043, 255732049, 255732080, 255733404, 255734082, 255734090, 255734121, 255736716, 255736724, 255736753, 255736755, 255736761, 255736792, 255738794, 255738802, 255738833, 255743506, 255743514, 255743545, 255743880, 255745368, 255745376, 255745405, 255745407, 255745413, 255745444, 255747446, 255747454, 255747485, 255752158, 255752166, 255752197, 255900809, 255996464, 255996501, 255998542, 256003254, 256005142, 256005179, 256007220, 256011906, 256011932, 256039482, 256039490, 256039519, 256039521, 256039527, 256039558, 256041560, 256041568, 256041599, 256044980, 256045017, 256045054, 256046272, 256046280, 256046311, 256047058, 256047095, 256049136, 256051770, 256051807, 256053848, 256054924, 256054932, 256054963, 256058560, 256087998, 256088006, 256088035, 256088037, 256088043, 256088072, 256088074, 256088080, 256088111, 256090076, 256090084, 256090113, 256090115, 256090121, 256090152, 256091476, 256092154, 256092162, 256092193, 256094788, 256094796, 256094825, 256094827, 256094833, 256094864, 256096866, 256096874, 256096905, 256101578, 256101586, 256101617, 256138068, 256138105, 256138142, 256140146, 256140183, 256142224, 256144858, 256144895, 256146936, 256151648, 256154083, 256154091, 256154122, 256224975, 256224983, 256225014, 256306020, 256314698, 256349038, 256349046, 256349077, 256354536, 256354573, 256356614, 256361326, 256397554, 256397562, 256397591, 256397593, 256397599, 256397630, 256399632, 256399640, 256399671, 256404344, 256404352, 256404383, 256447624, 256447661, 256449702, 256454414, 256509512, 256509549, 256511590, 256516302, 256524954, 256595846, 256664092, 256681883, 256707110, 256707118, 256707149, 256757180, 256777538, 256777575, 256779616, 256784328, 256792980, 256819068, 256820556, 256820564, 256820593, 256820595, 256820601, 256820632, 256822634, 256822642, 256822673, 256827346, 256827354, 256827385, 256835998, 256836006, 256836037, 257033834, 257033871, 257033908, 257033945, 257035912, 257035949, 257035986, 257037990, 257038027, 257040068, 257040624, 257040661, 257040698, 257042702, 257042739, 257044780, 257047414, 257047451, 257049492, 257054204, 257076852, 257076860, 257076889, 257076891, 257076897, 257076926, 257076928, 257076934, 257076963, 257076965, 257076971, 257077002, 257078930, 257078938, 257078967, 257078969, 257078975, 257079004, 257079006, 257079012, 257079043, 257080330, 257080367, 257081008, 257081016, 257081045, 257081047, 257081053, 257081084, 257082408, 257083086, 257083094, 257083125, 257083642, 257083650, 257083679, 257083681, 257083687, 257083716, 257083718, 257083724, 257083755, 257085720, 257085728, 257085757, 257085759, 257085765, 257085796, 257087094, 257087120, 257087798, 257087806, 257087837, 257090432, 257090440, 257090469, 257090471, 257090477, 257090508, 257092510, 257092518, 257092549, 257095772, 257097222, 257097230, 257097261, 257120168, 257120205, 257120242, 257122246, 257122283, 257124324, 257126958, 257126995, 257129036, 257130112, 257130120, 257130151, 257133748, 257135610, 257135647, 257137688, 257142400, 257163186, 257163194, 257163223, 257163225, 257163231, 257163260, 257163262, 257163268, 257163299, 257165264, 257165272, 257165301, 257165303, 257165309, 257165340, 257166664, 257167342, 257167350, 257167381, 257169976, 257169984, 257170013, 257170015, 257170021, 257170052, 257172054, 257172062, 257172093, 257176766, 257176774, 257176805, 257178628, 257178636, 257178665, 257178667, 257178673, 257178704, 257180706, 257180714, 257180745, 257185418, 257185426, 257185457, 257228698, 257228735, 257230776, 257235488, 257343390, 257343427, 257343464, 257345468, 257345505, 257347546, 257350180, 257350217, 257352258, 257356970, 257386408, 257386416, 257386445, 257386447, 257386453, 257386482, 257386484, 257386490, 257386521, 257388486, 257388494, 257388523, 257388525, 257388531, 257388562, 257389886, 257390564, 257390572, 257390603, 257393198, 257393206, 257393235, 257393237, 257393243, 257393274, 257395276, 257395284, 257395315, 257399988, 257399996, 257400027, 257429724, 257429761, 257431802, 257436514, 257445166, 257472742, 257472750, 257472779, 257472781, 257472787, 257472818, 257474820, 257474828, 257474859, 257479532, 257479540, 257479571, 257480484, 257488184, 257488192, 257488223, 257538254, 257587343, 257587351, 257587382, 257600142, 257604652, 257604689, 257604726, 257604763, 257606730, 257606767, 257606804, 257608808, 257608845, 257610886, 257611442, 257611479, 257611516, 257613520, 257613557, 257615598, 257618232, 257618269, 257620310, 257625022, 257652946, 257652983, 257655024, 257659736, 257671034, 257695964, 257695972, 257696001, 257696003, 257696009, 257696040, 257698042, 257698050, 257698081, 257702754, 257702762, 257702793, 257739280, 257782298, 257782306, 257782337, 257868168, 257911186, 257911194, 257911225, 257914208, 257914245, 257914282, 257916286, 257916323, 257918364, 257920998, 257921035, 257923076, 257927788, 257958214, 257962502, 258005520, 258005528, 258005559, 258115143, 258124464, 258124501, 258124538, 258126542, 258126579, 258128620, 258131254, 258131291, 258133332, 258138044, 258167482, 258167490, 258167519, 258167521, 258167527, 258167556, 258167558, 258167564, 258167595, 258169560, 258169568, 258169597, 258169599, 258169605, 258169636, 258170960, 258171638, 258171646, 258171677, 258174272, 258174280, 258174309, 258174311, 258174317, 258174348, 258176350, 258176358, 258176389, 258181062, 258181070, 258181101, 258195356, 258195393, 258195430, 258197434, 258197471, 258199512, 258202146, 258202183, 258204224, 258208936, 258210798, 258210835, 258212876, 258217588, 258223764, 258223801, 258225842, 258226240, 258230554, 258238374, 258238382, 258238411, 258238413, 258238419, 258238448, 258238450, 258238456, 258238487, 258240452, 258240460, 258240489, 258240491, 258240497, 258240528, 258241852, 258242530, 258242538, 258242569, 258245164, 258245172, 258245201, 258245203, 258245209, 258245240, 258247242, 258247250, 258247281, 258251954, 258251962, 258251993, 258253816, 258253824, 258253853, 258253855, 258253861, 258253892, 258255894, 258255902, 258255933, 258260606, 258260614, 258260645, 258269258, 258269266, 258269297, 258319328, 258434020, 258434057, 258436098, 258440810, 258477038, 258477046, 258477075, 258477077, 258477083, 258477114, 258479116, 258479124, 258479155, 258483828, 258483836, 258483867, 258504912, 258504949, 258506990, 258511702, 258520354, 258529032, 258533320, 258547930, 258547938, 258547967, 258547969, 258547975, 258548006, 258550008, 258550016, 258550047, 258554720, 258554728, 258554759, 258563372, 258563380, 258563411, 258568870, 258568907, 258570948, 258575660, 258611888, 258611896, 258611925, 258611927, 258611933, 258611964, 258613966, 258613974, 258614005, 258618678, 258618686, 258618717, 258661958, 258661995, 258662531, 258662539, 258662570, 258664036, 258668748, 258695282, 258695319, 258695356, 258697360, 258697397, 258699438, 258702072, 258702109, 258704150, 258708862, 258743576, 258786594, 258786602, 258786633, 258814468, 258857486, 258857494, 258857525, 258878426, 258921444, 258921452, 258921483, 258947068, 258947105, 258949146, 258953858, 258971514, 259004838, 259004875, 259006916, 259011628, 259033402, 259190331, 259215094, 259215131, 259217172, 259221884, 259256624, 259258112, 259258120, 259258149, 259258151, 259258157, 259258188, 259260190, 259260198, 259260229, 259264902, 259264910, 259264941, 259285986, 259286023, 259288064, 259292776, 259301428, 259314394, 259329004, 259329012, 259329041, 259329043, 259329049, 259329080, 259331082, 259331090, 259331121, 259335794, 259335802, 259335833, 259344446, 259344454, 259344485, 259517886, 259517923, 259519964, 259524650, 259524676, 259557724, 259557761, 259557798, 259559802, 259559839, 259561880, 259564514, 259564551, 259566592, 259567668, 259567676, 259567707, 259571304, 259595542, 259600742, 259600750, 259600779, 259600781, 259600787, 259600816, 259600818, 259600824, 259600855, 259602820, 259602828, 259602857, 259602859, 259602865, 259602896, 259604220, 259604898, 259604906, 259604937, 259607532, 259607540, 259607569, 259607571, 259607577, 259607608, 259609610, 259609618, 259609649, 259614322, 259614330, 259614361, 259638560, 259638568, 259638599, 259644058, 259644095, 259646136, 259650848, 259659500, 259687076, 259687084, 259687113, 259687115, 259687121, 259687152, 259689154, 259689162, 259689193, 259693866, 259693874, 259693905, 259702518, 259702526, 259702557, 259737146, 259737183, 259739224, 259743936, 259752588, 259785912, 259785949, 259787990, 259792702, 259827442, 259867280, 259867317, 259869358, 259874070, 259910298, 259910306, 259910335, 259910337, 259910343, 259910374, 259912376, 259912384, 259912415, 259917088, 259917096, 259917127, 259953614, 259996632, 259996640, 259996671, 260037698, 260046702, 260095468, 260108590, 260128542, 260128579, 260128616, 260130620, 260130657, 260132698, 260135332, 260135369, 260137410, 260142122, 260176836, 260219854, 260219862, 260219893, 260305724, 260348742, 260348750, 260348781, 260376616, 260419634, 260419642, 260419673, 260438098, 260438135, 260440176, 260444888, 260608516, 260632912, 260632949, 260632986, 260634990, 260635027, 260637068, 260639702, 260639739, 260641780, 260646492, 260648354, 260648391, 260650432, 260655144, 260675930, 260675938, 260675967, 260675969, 260675975, 260676004, 260676006, 260676012, 260676043, 260678008, 260678016, 260678045, 260678047, 260678053, 260678084, 260679408, 260680086, 260680094, 260680125, 260682720, 260682728, 260682757, 260682759, 260682765, 260682796, 260684798, 260684806, 260684837, 260689510, 260689518, 260689549, 260691372, 260691380, 260691409, 260691411, 260691417, 260691448, 260693450, 260693458, 260693489, 260698162, 260698170, 260698201, 260719246, 260719283, 260721324, 260726036, 260734688, 260747654, 260762264, 260762272, 260762301, 260762303, 260762309, 260762340, 260764342, 260764350, 260764381, 260769054, 260769062, 260769093, 260777706, 260777714, 260777745, 260827776, 260876542, 260942468, 260942505, 260944546, 260949258, 260957910, 260985486, 260985494, 260985523, 260985525, 260985531, 260985562, 260987564, 260987572, 260987603, 260992276, 260992284, 260992315, 261000928, 261000936, 261000967, 261028802, 261071820, 261071828, 261071859, 261092760, 261100087, 261100095, 261100126, 261135778, 261135786, 261135817, 261185848, 261200252, 261200260, 261200289, 261200291, 261200297, 261200326, 261200328, 261200334, 261200363, 261200365, 261200371, 261200400, 261200402, 261200408, 261200439, 261202330, 261202338, 261202367, 261202369, 261202375, 261202404, 261202406, 261202412, 261202441, 261202443, 261202449, 261202480, 261203730, 261203767, 261203804, 261204408, 261204416, 261204445, 261204447, 261204453, 261204482, 261204484, 261204490, 261204521, 261205808, 261205845, 261206486, 261206494, 261206523, 261206525, 261206531, 261206562, 261207042, 261207050, 261207079, 261207081, 261207087, 261207116, 261207118, 261207124, 261207153, 261207155, 261207161, 261207192, 261207886, 261208564, 261208572, 261208603, 261209120, 261209128, 261209157, 261209159, 261209165, 261209194, 261209196, 261209202, 261209233, 261210520, 261210557, 261211198, 261211206, 261211235, 261211237, 261211243, 261211274, 261212598, 261213276, 261213284, 261213315, 261213832, 261213840, 261213869, 261213871, 261213877, 261213906, 261213908, 261213914, 261213945, 261215910, 261215918, 261215947, 261215949, 261215955, 261215986, 261217310, 261217988, 261217996, 261218027, 261219172, 261219209, 261220622, 261220630, 261220659, 261220661, 261220667, 261220698, 261221250, 261222700, 261222708, 261222739, 261225962, 261227412, 261227420, 261227451, 261252024, 261295042, 261295050, 261295081, 261470958, 261509808, 261509816, 261509845, 261509847, 261509853, 261509882, 261509884, 261509890, 261509919, 261509921, 261509927, 261509958, 261511886, 261511894, 261511923, 261511925, 261511931, 261511960, 261511962, 261511968, 261511999, 261513286, 261513323, 261513964, 261513972, 261514001, 261514003, 261514009, 261514040, 261515364, 261516042, 261516050, 261516081, 261516598, 261516606, 261516635, 261516637, 261516643, 261516672, 261516674, 261516680, 261516711, 261518676, 261518684, 261518713, 261518715, 261518721, 261518752, 261520076, 261520754, 261520762, 261520793, 261523388, 261523396, 261523425, 261523427, 261523433, 261523464, 261525466, 261525474, 261525505, 261528728, 261530178, 261530186, 261530217, 261627887, 261723542, 261723579, 261725620, 261730332, 261738984, 261766560, 261766568, 261766597, 261766599, 261766605, 261766636, 261768638, 261768646, 261768677, 261773350, 261773358, 261773389, 261782002, 261782010, 261782041, 261794434, 261794471, 261796512, 261801224, 261809876, 261819364, 261819372, 261819401, 261819403, 261819409, 261819438, 261819440, 261819446, 261819477, 261821442, 261821450, 261821479, 261821481, 261821487, 261821518, 261822842, 261823520, 261823528, 261823559, 261826154, 261826162, 261826191, 261826193, 261826199, 261826230, 261828232, 261828240, 261828271, 261832944, 261832952, 261832983, 261837452, 261837460, 261837489, 261837491, 261837497, 261837528, 261839530, 261839538, 261839569, 261844242, 261844250, 261844281, 261852894, 261852902, 261852933, 262033098, 262041776, 262076116, 262076124, 262076155, 262081614, 262081651, 262083692, 262088404, 262103990, 262124632, 262124640, 262124669, 262124671, 262124677, 262124708, 262126710, 262126718, 262126749, 262128920, 262128928, 262128957, 262128959, 262128965, 262128996, 262130998, 262131006, 262131037, 262131422, 262131430, 262131461, 262135710, 262135718, 262135749, 262147008, 262147016, 262147047, 262167948, 262174702, 262174739, 262176780, 262181492, 262198705, 262210966, 262210974, 262211005, 262261036, 262290882, 262290890, 262290919, 262290921, 262290927, 262290956, 262290958, 262290964, 262290993, 262290995, 262291001, 262291032, 262292960, 262292968, 262292997, 262292999, 262293005, 262293034, 262293036, 262293042, 262293073, 262294360, 262294397, 262295038, 262295046, 262295075, 262295077, 262295083, 262295114, 262296438, 262297116, 262297124, 262297155, 262297672, 262297680, 262297709, 262297711, 262297717, 262297746, 262297748, 262297754, 262297785, 262299750, 262299758, 262299787, 262299789, 262299795, 262299826, 262301150, 262301828, 262301836, 262301867, 262304462, 262304470, 262304499, 262304501, 262304507, 262304538, 262306540, 262306548, 262306579, 262309802, 262311252, 262311260, 262311291, 262391170, 262434188, 262434196, 262434227, 262438476, 262438484, 262438515, 262484258, 262546146, 262600438, 262600446, 262600475, 262600477, 262600483, 262600512, 262600514, 262600520, 262600551, 262602516, 262602524, 262602553, 262602555, 262602561, 262602592, 262603916, 262604594, 262604602, 262604633, 262607228, 262607236, 262607265, 262607267, 262607273, 262607304, 262609306, 262609314, 262609345, 262614018, 262614026, 262614057, 262652432, 262652469, 262654510, 262659222, 262814172, 262857190, 262857198, 262857229, 262885064, 262909994, 262910002, 262910031, 262910033, 262910039, 262910070, 262912072, 262912080, 262912111, 262916784, 262916792, 262916823, 262928082, 262928090, 262928121, 262961988, 263070468, 263070505, 263070542, 263072546, 263072583, 263074624, 263077258, 263077295, 263079336, 263084048, 263113486, 263113494, 263113523, 263113525, 263113531, 263113560, 263113562, 263113568, 263113599, 263115564, 263115572, 263115601, 263115603, 263115609, 263115640, 263116964, 263117642, 263117650, 263117681, 263120276, 263120284, 263120313, 263120315, 263120321, 263120352, 263122354, 263122362, 263122393, 263127066, 263127074, 263127105, 263156802, 263156839, 263158880, 263163592, 263172244, 263199820, 263199828, 263199857, 263199859, 263199865, 263199896, 263201898, 263201906, 263201937, 263206610, 263206618, 263206649, 263215262, 263215270, 263215301, 263219550, 263219558, 263219589, 263243136, 263265332, 263286154, 263286162, 263286193, 263336224, 263380024, 263380061, 263381512, 263381520, 263381549, 263381551, 263381557, 263381586, 263381588, 263381594, 263381625, 263382102, 263383590, 263383598, 263383627, 263383629, 263383635, 263383666, 263384990, 263385668, 263385676, 263385707, 263386814, 263388302, 263388310, 263388339, 263388341, 263388347, 263388378, 263390380, 263390388, 263390419, 263395092, 263395100, 263395131, 263423042, 263423050, 263423079, 263423081, 263423087, 263423118, 263425120, 263425128, 263425159, 263429832, 263429840, 263429871, 263466358, 263509376, 263509384, 263509415, 263641286, 263641323, 263641360, 263643364, 263643401, 263645442, 263648076, 263648113, 263650154, 263654866, 263689580, 263691068, 263691076, 263691105, 263691107, 263691113, 263691144, 263693146, 263693154, 263693185, 263697858, 263697866, 263697897, 263724142, 263724150, 263724179, 263724181, 263724187, 263724216, 263724218, 263724224, 263724253, 263724255, 263724261, 263724292, 263726220, 263726228, 263726257, 263726259, 263726265, 263726294, 263726296, 263726302, 263726333, 263727620, 263727657, 263728298, 263728306, 263728335, 263728337, 263728343, 263728374, 263729698, 263730376, 263730384, 263730415, 263730932, 263730940, 263730969, 263730971, 263730977, 263731006, 263731008, 263731014, 263731045, 263732598, 263732606, 263732637, 263733010, 263733018, 263733047, 263733049, 263733055, 263733086, 263734410, 263735088, 263735096, 263735127, 263737722, 263737730, 263737759, 263737761, 263737767, 263737798, 263739800, 263739808, 263739839, 263743062, 263744512, 263744520, 263744551, 263950842, 263950879, 263952920, 263957632, 264000624, 264000632, 264000663, 264033698, 264033706, 264033735, 264033737, 264033743, 264033772, 264033774, 264033780, 264033811, 264035776, 264035784, 264035813, 264035815, 264035821, 264035852, 264037176, 264037854, 264037862, 264037893, 264040488, 264040496, 264040525, 264040527, 264040533, 264040564, 264042566, 264042574, 264042605, 264047278, 264047286, 264047317, 264161098, 264161135, 264163176, 264167888, 264204116, 264204124, 264204153, 264204155, 264204161, 264204192, 264206194, 264206202, 264206233, 264210906, 264210914, 264210945, 264212104, 264212141, 264212178, 264214182, 264214219, 264216260, 264218894, 264218931, 264220972, 264225684, 264231990, 264232027, 264234068, 264238780, 264247432, 264260398, 264275008, 264275016, 264275045, 264275047, 264275053, 264275084, 264277086, 264277094, 264277125, 264281798, 264281806, 264281837, 264290450, 264290458, 264290489, 264318324, 264343254, 264343262, 264343291, 264343293, 264343299, 264343330, 264345332, 264345340, 264345371, 264350044, 264350052, 264350083, 264361342, 264361350, 264361381, 264470654, 264472142, 264472150, 264472179, 264472181, 264472187, 264472218, 264474220, 264474228, 264474259, 264478932, 264478940, 264478971, 264513672, 264513680, 264513711, 264521660, 264521697, 264523738, 264528450, 264541546, 264584564, 264584572, 264584603, 264605504, 264648522, 264648530, 264648561, 264652810, 264652818, 264652849, 264698592, 264731916, 264731953, 264733994, 264738706, 264781698, 264781706, 264781737, 264799330, 264799338, 264799367, 264799369, 264799375, 264799404, 264799406, 264799412, 264799441, 264799443, 264799449, 264799480, 264801408, 264801416, 264801445, 264801447, 264801453, 264801482, 264801484, 264801490, 264801521, 264802808, 264802845, 264803486, 264803494, 264803523, 264803525, 264803531, 264803562, 264804886, 264805564, 264805572, 264805603, 264806120, 264806128, 264806157, 264806159, 264806165, 264806194, 264806196, 264806202, 264806233, 264808198, 264808206, 264808235, 264808237, 264808243, 264808274, 264809598, 264810276, 264810284, 264810315, 264812910, 264812918, 264812947, 264812949, 264812955, 264812986, 264814772, 264814780, 264814809, 264814811, 264814817, 264814846, 264814848, 264814854, 264814885, 264814988, 264814996, 264815027, 264816850, 264816858, 264816887, 264816889, 264816895, 264816926, 264818250, 264818928, 264818936, 264818967, 264819700, 264819708, 264819739, 264821562, 264821570, 264821599, 264821601, 264821607, 264821638, 264823640, 264823648, 264823679, 264828352, 264828360, 264828391, 264831216, 264983702, 265041472, 265108886, 265108894, 265108923, 265108925, 265108931, 265108960, 265108962, 265108968, 265108999, 265110964, 265110972, 265111001, 265111003, 265111009, 265111040, 265112364, 265113042, 265113050, 265113081, 265115676, 265115684, 265115713, 265115715, 265115721, 265115752, 265117754, 265117762, 265117793, 265122466, 265122474, 265122505, 265124328, 265124336, 265124365, 265124367, 265124373, 265124404, 265126406, 265126414, 265126445, 265131118, 265131126, 265131157, 265176322, 265251728, 265294746, 265294754, 265294785, 265302734, 265302771, 265304812, 265309524, 265322620, 265365638, 265365646, 265365677, 265393512, 265418442, 265418450, 265418479, 265418481, 265418487, 265418518, 265420520, 265420528, 265420559, 265425232, 265425240, 265425271, 265433884, 265433892, 265433923, 265436530, 265436538, 265436569, 265554520, 265562772, 265562780, 265562811, 265594358, 265594395, 265596436, 265601148, 265612290, 265637376, 265637384, 265637413, 265637415, 265637421, 265637452, 265639454, 265639462, 265639493, 265644166, 265644174, 265644205, 265680692, 265723710, 265723718, 265723749, 265727998, 265728006, 265728037, 265773780, 265794305, 265794313, 265794342, 265794344, 265794350, 265794381, 265796383, 265796391, 265796422, 265801095, 265801103, 265801134, 265822546, 265889960, 265889968, 265889997, 265889999, 265890005, 265890034, 265890036, 265890042, 265890073, 265892038, 265892046, 265892075, 265892077, 265892083, 265892114, 265893438, 265894116, 265894124, 265894155, 265896750, 265896758, 265896787, 265896789, 265896795, 265896826, 265898828, 265898836, 265898867, 265903540, 265903548, 265903579, 265903914, 265905402, 265905410, 265905439, 265905441, 265905447, 265905478, 265907480, 265907488, 265907519, 265912192, 265912200, 265912231, 265946932, 265946940, 265946971, 266103861, 266103869, 266103900, 266165176, 266165213, 266167254, 266171966, 266199516, 266199524, 266199553, 266199555, 266199561, 266199592, 266201594, 266201602, 266201633, 266206306, 266206314, 266206345, 266214958, 266214966, 266214997, 266248032, 266248040, 266248069, 266248071, 266248077, 266248106, 266248108, 266248114, 266248145, 266250110, 266250118, 266250147, 266250149, 266250155, 266250186, 266251510, 266252188, 266252196, 266252227, 266254822, 266254830, 266254859, 266254861, 266254867, 266254898, 266256900, 266256908, 266256939, 266261612, 266261620, 266261651, 266393364, 266474732, 266509072, 266509080, 266509111, 266557588, 266557596, 266557625, 266557627, 266557633, 266557664, 266559666, 266559674, 266559705, 266564378, 266564386, 266564417, 266669546, 266669583, 266671624, 266676336, 266684988, 266712564, 266712572, 266712601, 266712603, 266712609, 266712640, 266714642, 266714650, 266714681, 266719354, 266719362, 266719393, 266728006, 266728014, 266728045, 266735994, 266736031, 266738072, 266742784, 266755880, 266798898, 266798906, 266798937, 266867144, 266867152, 266867183, 266884935, 266884943, 266884974, 266979102, 266980590, 266980598, 266980627, 266980629, 266980635, 266980666, 266982668, 266982676, 266982707, 266987380, 266987388, 266987419, 266996032, 266996040, 266996071, 267022120, 267022128, 267022159, 267045550, 267236886, 267236894, 267236923, 267236925, 267236931, 267236960, 267236962, 267236968, 267236997, 267236999, 267237005, 267237036, 267238964, 267238972, 267239001, 267239003, 267239009, 267239038, 267239040, 267239046, 267239077, 267240364, 267240401, 267241042, 267241050, 267241079, 267241081, 267241087, 267241118, 267242442, 267243120, 267243128, 267243159, 267243676, 267243684, 267243713, 267243715, 267243721, 267243750, 267243752, 267243758, 267243789, 267245754, 267245762, 267245791, 267245793, 267245799, 267245830, 267247154, 267247832, 267247840, 267247871, 267250466, 267250474, 267250503, 267250505, 267250511, 267250542, 267252544, 267252552, 267252583, 267255806, 267257256, 267257264, 267257295, 267290146, 267290154, 267290185, 267323220, 267323228, 267323257, 267323259, 267323265, 267323294, 267323296, 267323302, 267323333, 267325298, 267325306, 267325335, 267325337, 267325343, 267325374, 267326698, 267327376, 267327384, 267327415, 267330010, 267330018, 267330047, 267330049, 267330055, 267330086, 267332088, 267332096, 267332127, 267336800, 267336808, 267336839, 267338662, 267338670, 267338699, 267338701, 267338707, 267338738, 267340740, 267340748, 267340779, 267345452, 267345460, 267345491, 267546442, 267546450, 267546479, 267546481, 267546487, 267546516, 267546518, 267546524, 267546555, 267548520, 267548528, 267548557, 267548559, 267548565, 267548596, 267549920, 267550598, 267550606, 267550637, 267553232, 267553240, 267553269, 267553271, 267553277, 267553308, 267555310, 267555318, 267555349, 267560022, 267560030, 267560061, 267632776, 267632784, 267632813, 267632815, 267632821, 267632852, 267634854, 267634862, 267634893, 267639566, 267639574, 267639605, 267648218, 267648226, 267648257, 267760176, 267764686, 267764723, 267764760, 267764797, 267766764, 267766801, 267766838, 267768842, 267768879, 267770920, 267771476, 267771513, 267771550, 267773554, 267773591, 267775632, 267778266, 267778303, 267780344, 267785056, 267803194, 267803202, 267803233, 267807704, 267807712, 267807741, 267807743, 267807749, 267807778, 267807780, 267807786, 267807815, 267807817, 267807823, 267807854, 267809782, 267809790, 267809819, 267809821, 267809827, 267809856, 267809858, 267809864, 267809895, 267811182, 267811219, 267811860, 267811868, 267811897, 267811899, 267811905, 267811936, 267813260, 267813938, 267813946, 267813977, 267814494, 267814502, 267814531, 267814533, 267814539, 267814568, 267814570, 267814576, 267814607, 267816572, 267816580, 267816609, 267816611, 267816617, 267816648, 267817972, 267818650, 267818658, 267818689, 267821284, 267821292, 267821321, 267821323, 267821329, 267821360, 267823362, 267823370, 267823401, 267826624, 267828074, 267828082, 267828113, 267831068, 267855998, 267856006, 267856035, 267856037, 267856043, 267856074, 267858076, 267858084, 267858115, 267862788, 267862796, 267862827, 267874086, 267874094, 267874125, 267942332, 267942340, 267942371, 268071220, 268071228, 268071259, 268074242, 268074279, 268074316, 268076320, 268076357, 268078398, 268081032, 268081069, 268083110, 268087822, 268117260, 268117268, 268117297, 268117299, 268117305, 268117334, 268117336, 268117342, 268117373, 268118248, 268119338, 268119346, 268119375, 268119377, 268119383, 268119414, 268120738, 268121416, 268121424, 268121455, 268124050, 268124058, 268124087, 268124089, 268124095, 268124126, 268126128, 268126136, 268126167, 268130840, 268130848, 268130879, 268161266, 268161274, 268161305, 268165554, 268165562, 268165593, 268211336, 268318195, 268318203, 268318234, 268327516, 268327524, 268327553, 268327555, 268327561, 268327590, 268327592, 268327598, 268327629, 268329594, 268329602, 268329631, 268329633, 268329639, 268329670, 268330994, 268331672, 268331680, 268331711, 268334306, 268334314, 268334343, 268334345, 268334351, 268334382, 268336384, 268336392, 268336423, 268341096, 268341104, 268341135, 268383798, 268383835, 268385876, 268390588, 268398408, 268398416, 268398445, 268398447, 268398453, 268398482, 268398484, 268398490, 268398521, 268400486, 268400494, 268400523, 268400525, 268400531, 268400562, 268401886, 268402564, 268402572, 268402603, 268405198, 268405206, 268405235, 268405237, 268405243, 268405274, 268407276, 268407284, 268407315, 268411988, 268411996, 268412027, 268413850, 268413858, 268413887, 268413889, 268413895, 268413926, 268415928, 268415936, 268415967, 268420640, 268420648, 268420679, 268426816, 268426824, 268426853, 268426855, 268426861, 268426892, 268428894, 268428902, 268428933, 268429292, 268429300, 268429331, 268433606, 268433614, 268433645, 268637072, 268637080, 268637109, 268637111, 268637117, 268637148, 268639150, 268639158, 268639189, 268643862, 268643870, 268643901, 268689066, 268693354, 268707964, 268707972, 268708001, 268708003, 268708009, 268708040, 268710042, 268710050, 268710081, 268714754, 268714762, 268714793, 268723406, 268723414, 268723445, 268736372, 268736380, 268736411, 268771922, 268771930, 268771959, 268771961, 268771967, 268771998, 268774000, 268774008, 268774039, 268778712, 268778720, 268778751, 268855316, 268855353, 268855390, 268857394, 268857431, 268859472, 268862106, 268862143, 268864184, 268868896, 268898334, 268898342, 268898371, 268898373, 268898379, 268898408, 268898410, 268898416, 268898447, 268900412, 268900420, 268900449, 268900451, 268900457, 268900488, 268901812, 268902490, 268902498, 268902529, 268905124, 268905132, 268905161, 268905163, 268905169, 268905200, 268907202, 268907210, 268907241, 268911914, 268911922, 268911953, 268946628, 268946636, 268946667, 269017520, 269017528, 269017559, 269081478, 269081486, 269081517, 269107102, 269107139, 269109180, 269113892, 269150120, 269150128, 269150157, 269150159, 269150165, 269150196, 269152198, 269152206, 269152237, 269156910, 269156918, 269156949, 269164872, 269164909, 269166950, 269171662, 269193436, 269207890, 269207898, 269207927, 269207929, 269207935, 269207966, 269209968, 269209976, 269210007, 269214680, 269214688, 269214719, 269236454, 269236462, 269236493, 269259884, 269393383, 269393391, 269393422, 269416658, 269418146, 269418154, 269418183, 269418185, 269418191, 269418222, 269420224, 269420232, 269420263, 269424936, 269424944, 269424975, 269459676, 269459684, 269459715, 269474428, 269489038, 269489046, 269489075, 269489077, 269489083, 269489114, 269491116, 269491124, 269491155, 269495828, 269495836, 269495867, 269504480, 269504488, 269504519, 269517446, 269517454, 269517485, 269677920, 269677957, 269679998, 269684710, 269727702, 269727710, 269727741, 269760776, 269760784, 269760813, 269760815, 269760821, 269760850, 269760852, 269760858, 269760889, 269762854, 269762862, 269762891, 269762893, 269762899, 269762930, 269764254, 269764932, 269764940, 269764971, 269767566, 269767574, 269767603, 269767605, 269767611, 269767642, 269769644, 269769652, 269769683, 269774356, 269774364, 269774395, 269798594, 269798602, 269798633, 269847110, 269847118, 269847147, 269847149, 269847155, 269847186, 269849188, 269849196, 269849227, 269853900, 269853908, 269853939, 269862552, 269862560, 269862591, 269945946, 269945983, 269948024, 269952736, 269987476, 269988964, 269988972, 269989001, 269989003, 269989009, 269989040, 269991042, 269991050, 269991081, 269995754, 269995762, 269995793, 270070332, 270070340, 270070369, 270070371, 270070377, 270070408, 270072410, 270072418, 270072449, 270077122, 270077130, 270077161, 270156666, 270156674, 270156705, 270197732, 270240750, 270240758, 270240789, 270248738, 270248775, 270250816, 270255502, 270255528, 270268624, 270288576, 270288613, 270288650, 270290654, 270290691, 270292732, 270295366, 270295403, 270297444, 270298520, 270298528, 270298559, 270302156, 270311642, 270311650, 270311681, 270331594, 270331602, 270331631, 270331633, 270331639, 270331668, 270331670, 270331676, 270331707, 270333672, 270333680, 270333709, 270333711, 270333717, 270333748, 270335072, 270335750, 270335758, 270335789, 270338384, 270338392, 270338421, 270338423, 270338429, 270338460, 270340462, 270340470, 270340501, 270345174, 270345182, 270345213, 270379888, 270379896, 270379927, 270508776, 270508784, 270508815, 270558294, 270579668, 270579676, 270579707, 270598132, 270598169, 270600210, 270604922, 270641150, 270641158, 270641187, 270641189, 270641195, 270641226, 270643228, 270643236, 270643267, 270647940, 270647948, 270647979, 270768550, 270835964, 270835972, 270836001, 270836003, 270836009, 270836038, 270836040, 270836046, 270836077, 270838042, 270838050, 270838079, 270838081, 270838087, 270838118, 270839442, 270840120, 270840128, 270840159, 270842754, 270842762, 270842791, 270842793, 270842799, 270842830, 270844832, 270844840, 270844871, 270849544, 270849552, 270849583, 270851406, 270851414, 270851443, 270851445, 270851451, 270851482, 270853484, 270853492, 270853523, 270858196, 270858204, 270858235, 270907688, 270922298, 270922306, 270922335, 270922337, 270922343, 270922374, 270924376, 270924384, 270924415, 270929088, 270929096, 270929127, 270937740, 270937748, 270937779, 270950706, 270950714, 270950745, 271036576, 271079594, 271079602, 271079633, 271145520, 271145528, 271145557, 271145559, 271145565, 271145596, 271147598, 271147606, 271147637, 271152310, 271152318, 271152349, 271160962, 271160970, 271161001, 271231854, 271231862, 271231893, 271295812, 271295820, 271295851, 271339368, 271363764, 271363801, 271363838, 271365842, 271365879, 271367920, 271370554, 271370591, 271372632, 271377344, 271379206, 271379243, 271381284, 271385996, 271403304, 271403312, 271403320, 271403341, 271403343, 271403349, 271403351, 271403357, 271403378, 271403380, 271403382, 271403386, 271403388, 271403394, 271403415, 271403417, 271403419, 271403423, 271403425, 271403431, 271403452, 271403454, 271403456, 271403460, 271403462, 271403468, 271403491, 271403493, 271403499, 271403530, 271405382, 271405390, 271405398, 271405419, 271405421, 271405427, 271405429, 271405435, 271405456, 271405458, 271405460, 271405464, 271405466, 271405472, 271405493, 271405495, 271405497, 271405501, 271405503, 271405509, 271405532, 271405534, 271405540, 271405571, 271406782, 271406790, 271406819, 271406821, 271406827, 271406856, 271406858, 271406864, 271406895, 271407460, 271407468, 271407476, 271407497, 271407499, 271407505, 271407507, 271407513, 271407534, 271407536, 271407538, 271407542, 271407544, 271407550, 271407573, 271407575, 271407581, 271407612, 271408860, 271408868, 271408897, 271408899, 271408905, 271408936, 271409538, 271409546, 271409554, 271409575, 271409577, 271409583, 271409585, 271409591, 271409614, 271409616, 271409622, 271409653, 271410094, 271410102, 271410110, 271410131, 271410133, 271410139, 271410141, 271410147, 271410168, 271410170, 271410172, 271410176, 271410178, 271410184, 271410205, 271410207, 271410209, 271410213, 271410215, 271410221, 271410244, 271410246, 271410252, 271410260, 271410283, 271410938, 271410946, 271410977, 271411616, 271411624, 271411632, 271411655, 271411663, 271411694, 271412172, 271412180, 271412188, 271412209, 271412211, 271412217, 271412219, 271412225, 271412246, 271412248, 271412250, 271412254, 271412256, 271412262, 271412285, 271412287, 271412293, 271412324, 271413572, 271413580, 271413609, 271413611, 271413617, 271413648, 271414250, 271414258, 271414266, 271414287, 271414289, 271414295, 271414297, 271414303, 271414326, 271414328, 271414334, 271414365, 271415650, 271415658, 271415689, 271416328, 271416336, 271416344, 271416367, 271416375, 271416406, 271416884, 271416892, 271416900, 271416921, 271416923, 271416929, 271416931, 271416937, 271416958, 271416960, 271416962, 271416966, 271416968, 271416974, 271416997, 271416999, 271417005, 271417036, 271418962, 271418970, 271418978, 271418999, 271419001, 271419007, 271419009, 271419015, 271419038, 271419040, 271419046, 271419077, 271420362, 271420370, 271420401, 271421040, 271421048, 271421056, 271421079, 271421087, 271421118, 271422224, 271422232, 271422261, 271422263, 271422269, 271422300, 271423674, 271423682, 271423690, 271423711, 271423713, 271423719, 271423721, 271423727, 271423750, 271423752, 271423758, 271423789, 271424302, 271424310, 271424341, 271425752, 271425760, 271425768, 271425791, 271425799, 271425830, 271429014, 271429022, 271429053, 271430464, 271430472, 271430480, 271430503, 271430511, 271430542, 271455076, 271455084, 271455115, 271630992, 271673320, 271673357, 271674010, 271674018, 271674049, 271675398, 271680110, 271688762, 271712860, 271712868, 271712876, 271712897, 271712899, 271712905, 271712907, 271712913, 271712934, 271712936, 271712938, 271712942, 271712944, 271712950, 271712971, 271712973, 271712975, 271712979, 271712981, 271712987, 271713010, 271713012, 271713018, 271713049, 271714938, 271714946, 271714954, 271714975, 271714977, 271714983, 271714985, 271714991, 271715012, 271715014, 271715016, 271715020, 271715022, 271715028, 271715051, 271715053, 271715059, 271715090, 271716338, 271716346, 271716375, 271716377, 271716383, 271716414, 271717016, 271717024, 271717032, 271717053, 271717055, 271717061, 271717063, 271717069, 271717092, 271717094, 271717100, 271717131, 271718416, 271718424, 271718455, 271719094, 271719102, 271719110, 271719133, 271719141, 271719172, 271719650, 271719658, 271719666, 271719687, 271719689, 271719695, 271719697, 271719703, 271719724, 271719726, 271719728, 271719732, 271719734, 271719740, 271719763, 271719765, 271719771, 271719802, 271721728, 271721736, 271721744, 271721765, 271721767, 271721773, 271721775, 271721781, 271721804, 271721806, 271721812, 271721843, 271723128, 271723136, 271723167, 271723806, 271723814, 271723822, 271723845, 271723853, 271723884, 271726440, 271726448, 271726456, 271726477, 271726479, 271726485, 271726487, 271726493, 271726516, 271726518, 271726524, 271726555, 271728518, 271728526, 271728534, 271728557, 271728565, 271728596, 271731780, 271731788, 271731819, 271733230, 271733238, 271733246, 271733269, 271733277, 271733308, 271830939, 271830947, 271830978, 271926594, 271926602, 271926631, 271926633, 271926639, 271926670, 271928672, 271928680, 271928711, 271933384, 271933392, 271933423, 271942036, 271942044, 271942075, 271982876, 271997486, 271997494, 271997523, 271997525, 271997531, 271997562, 271999564, 271999572, 271999603, 272004276, 272004284, 272004315, 272012928, 272012936, 272012967, 272022416, 272022424, 272022432, 272022453, 272022455, 272022461, 272022463, 272022469, 272022490, 272022492, 272022494, 272022498, 272022500, 272022506, 272022529, 272022531, 272022537, 272022568, 272024494, 272024502, 272024510, 272024531, 272024533, 272024539, 272024541, 272024547, 272024570, 272024572, 272024578, 272024609, 272025894, 272025902, 272025933, 272026572, 272026580, 272026588, 272026611, 272026619, 272026650, 272029206, 272029214, 272029222, 272029243, 272029245, 272029251, 272029253, 272029259, 272029282, 272029284, 272029290, 272029321, 272031284, 272031292, 272031300, 272031323, 272031331, 272031362, 272035996, 272036004, 272036012, 272036035, 272036043, 272036074, 272201810, 272236150, 272236158, 272236189, 272284666, 272284674, 272284703, 272284705, 272284711, 272284742, 272286744, 272286752, 272286783, 272291456, 272291464, 272291495, 272307042, 272307050, 272307081, 272331972, 272331980, 272331988, 272332009, 272332011, 272332017, 272332019, 272332025, 272332048, 272332050, 272332056, 272332087, 272334050, 272334058, 272334066, 272334089, 272334097, 272334128, 272338762, 272338770, 272338778, 272338801, 272338809, 272338840, 272358739, 272371000, 272371008, 272371039, 272454394, 272454431, 272456472, 272461184, 272469836, 272493934, 272493942, 272493950, 272493971, 272493973, 272493979, 272493981, 272493987, 272494008, 272494010, 272494012, 272494016, 272494018, 272494024, 272494045, 272494047, 272494049, 272494053, 272494055, 272494061, 272494084, 272494086, 272494092, 272494123, 272496012, 272496020, 272496028, 272496049, 272496051, 272496057, 272496059, 272496065, 272496086, 272496088, 272496090, 272496094, 272496096, 272496102, 272496125, 272496127, 272496133, 272496164, 272497412, 272497420, 272497449, 272497451, 272497457, 272497488, 272498090, 272498098, 272498106, 272498127, 272498129, 272498135, 272498137, 272498143, 272498166, 272498168, 272498174, 272498205, 272499490, 272499498, 272499529, 272500168, 272500176, 272500184, 272500207, 272500215, 272500246, 272500724, 272500732, 272500740, 272500761, 272500763, 272500769, 272500771, 272500777, 272500798, 272500800, 272500802, 272500806, 272500808, 272500814, 272500837, 272500839, 272500845, 272500876, 272502802, 272502810, 272502818, 272502839, 272502841, 272502847, 272502849, 272502855, 272502878, 272502880, 272502886, 272502917, 272504202, 272504210, 272504241, 272504880, 272504888, 272504896, 272504919, 272504927, 272504958, 272507514, 272507522, 272507530, 272507551, 272507553, 272507559, 272507561, 272507567, 272507590, 272507592, 272507598, 272507629, 272509592, 272509600, 272509608, 272509631, 272509639, 272509670, 272512854, 272512862, 272512893, 272514304, 272514312, 272514320, 272514343, 272514351, 272514382, 272594222, 272594230, 272594261, 272641528, 272641536, 272641544, 272641567, 272641575, 272641606, 272706180, 272749198, 272749206, 272749237, 272763950, 272772628, 272803490, 272803498, 272803506, 272803527, 272803529, 272803535, 272803537, 272803543, 272803564, 272803566, 272803568, 272803572, 272803574, 272803580, 272803603, 272803605, 272803611, 272803642, 272805568, 272805576, 272805584, 272805605, 272805607, 272805613, 272805615, 272805621, 272805644, 272805646, 272805652, 272805683, 272806968, 272806976, 272807007, 272807646, 272807654, 272807662, 272807685, 272807693, 272807724, 272810280, 272810288, 272810296, 272810317, 272810319, 272810325, 272810327, 272810333, 272810356, 272810358, 272810364, 272810395, 272812358, 272812366, 272812374, 272812397, 272812405, 272812436, 272812466, 272812503, 272814544, 272817070, 272817078, 272817086, 272817109, 272817117, 272817148, 272819256, 272855484, 272855492, 272855521, 272855523, 272855529, 272855560, 272857562, 272857570, 272857601, 272862274, 272862282, 272862313, 272905554, 272905591, 272907632, 272912344, 273017224, 273017232, 273017263, 273088116, 273088124, 273088155, 273113046, 273113054, 273113062, 273113083, 273113085, 273113091, 273113093, 273113099, 273113122, 273113124, 273113130, 273113161, 273115124, 273115132, 273115140, 273115163, 273115171, 273115202, 273119836, 273119844, 273119852, 273119875, 273119883, 273119914, 273122022, 273165040, 273165048, 273165079, 273215110, 273273520, 273273528, 273273557, 273273559, 273273565, 273273594, 273273596, 273273602, 273273633, 273275598, 273275606, 273275635, 273275637, 273275643, 273275674, 273276998, 273277676, 273277684, 273277715, 273280310, 273280318, 273280347, 273280349, 273280355, 273280386, 273282388, 273282396, 273282427, 273287100, 273287108, 273287139, 273359854, 273359862, 273359891, 273359893, 273359899, 273359930, 273361932, 273361940, 273361971, 273366644, 273366652, 273366683, 273375296, 273375304, 273375335, 273422602, 273422610, 273422618, 273422641, 273422649, 273422680, 273446188, 273446196, 273446227, 273545024, 273583076, 273583084, 273583113, 273583115, 273583121, 273583152, 273584564, 273584572, 273584580, 273584601, 273584603, 273584609, 273584611, 273584617, 273584638, 273584640, 273584642, 273584646, 273584648, 273584654, 273584677, 273584679, 273584685, 273584716, 273585154, 273585162, 273585193, 273586642, 273586650, 273586658, 273586679, 273586681, 273586687, 273586689, 273586695, 273586718, 273586720, 273586726, 273586757, 273588042, 273588050, 273588081, 273588720, 273588728, 273588736, 273588759, 273588767, 273588798, 273589866, 273589874, 273589905, 273591354, 273591362, 273591370, 273591391, 273591393, 273591399, 273591401, 273591407, 273591430, 273591432, 273591438, 273591469, 273593432, 273593440, 273593448, 273593471, 273593479, 273593510, 273598144, 273598152, 273598160, 273598183, 273598191, 273598222, 273669410, 273669418, 273669449, 273801320, 273801357, 273801394, 273803398, 273803435, 273805476, 273808110, 273808147, 273810188, 273814900, 273844338, 273844346, 273844375, 273844377, 273844383, 273844412, 273844414, 273844420, 273844451, 273846416, 273846424, 273846453, 273846455, 273846461, 273846492, 273847816, 273848494, 273848502, 273848533, 273851128, 273851136, 273851165, 273851167, 273851173, 273851204, 273853206, 273853214, 273853245, 273857918, 273857926, 273857957, 273887654, 273887691, 273889732, 273892632, 273892640, 273892671, 273894120, 273894128, 273894136, 273894157, 273894159, 273894165, 273894167, 273894173, 273894196, 273894198, 273894204, 273894235, 273894444, 273896198, 273896206, 273896214, 273896237, 273896245, 273896276, 273900910, 273900918, 273900926, 273900949, 273900957, 273900988, 273903096, 273927194, 273927202, 273927210, 273927231, 273927233, 273927239, 273927241, 273927247, 273927268, 273927270, 273927272, 273927276, 273927278, 273927284, 273927305, 273927307, 273927309, 273927313, 273927315, 273927321, 273927344, 273927346, 273927352, 273927383, 273929272, 273929280, 273929288, 273929309, 273929311, 273929317, 273929319, 273929325, 273929346, 273929348, 273929350, 273929354, 273929356, 273929362, 273929385, 273929387, 273929393, 273929424, 273930672, 273930680, 273930709, 273930711, 273930717, 273930748, 273931350, 273931358, 273931366, 273931387, 273931389, 273931395, 273931397, 273931403, 273931426, 273931428, 273931434, 273931465, 273932750, 273932758, 273932789, 273933428, 273933436, 273933444, 273933467, 273933475, 273933506, 273933984, 273933992, 273934000, 273934021, 273934023, 273934029, 273934031, 273934037, 273934058, 273934060, 273934062, 273934066, 273934068, 273934074, 273934097, 273934099, 273934105, 273934136, 273936062, 273936070, 273936078, 273936099, 273936101, 273936107, 273936109, 273936115, 273936138, 273936140, 273936146, 273936177, 273937462, 273937470, 273937501, 273938140, 273938148, 273938156, 273938179, 273938187, 273938218, 273940774, 273940782, 273940790, 273940811, 273940813, 273940819, 273940821, 273940827, 273940850, 273940852, 273940858, 273940889, 273942852, 273942860, 273942868, 273942891, 273942899, 273942930, 273946114, 273946122, 273946153, 273947564, 273947572, 273947580, 273947603, 273947611, 273947642, 273996184, 274110876, 274110913, 274112954, 274117666, 274153894, 274153902, 274153931, 274153933, 274153939, 274153970, 274155972, 274155980, 274156011, 274160684, 274160692, 274160723, 274197210, 274203676, 274203684, 274203692, 274203715, 274203723, 274203754, 274236750, 274236758, 274236766, 274236787, 274236789, 274236795, 274236797, 274236803, 274236824, 274236826, 274236828, 274236832, 274236834, 274236840, 274236863, 274236865, 274236871, 274236902, 274238828, 274238836, 274238844, 274238865, 274238867, 274238873, 274238875, 274238881, 274238904, 274238906, 274238912, 274238943, 274240228, 274240236, 274240267, 274240906, 274240914, 274240922, 274240945, 274240953, 274240984, 274243540, 274243548, 274243556, 274243577, 274243579, 274243585, 274243587, 274243593, 274243616, 274243618, 274243624, 274243655, 274245618, 274245626, 274245634, 274245657, 274245665, 274245696, 274250330, 274250338, 274250346, 274250369, 274250377, 274250408, 274364150, 274364158, 274364187, 274364189, 274364195, 274364226, 274366228, 274366236, 274366267, 274370940, 274370948, 274370979, 274372138, 274372175, 274372212, 274374216, 274374253, 274376294, 274378928, 274378965, 274381006, 274385718, 274420432, 274435042, 274435050, 274435079, 274435081, 274435087, 274435118, 274437120, 274437128, 274437159, 274441832, 274441840, 274441871, 274450484, 274450492, 274450523, 274463450, 274463458, 274463489, 274521376, 274521384, 274521415, 274546306, 274546314, 274546322, 274546343, 274546345, 274546351, 274546353, 274546359, 274546382, 274546384, 274546390, 274546421, 274548384, 274548392, 274548400, 274548423, 274548431, 274548462, 274553096, 274553104, 274553112, 274553135, 274553143, 274553174, 274673706, 274673714, 274673745, 274675194, 274675202, 274675210, 274675231, 274675233, 274675239, 274675241, 274675247, 274675270, 274675272, 274675278, 274675309, 274677272, 274677280, 274677288, 274677311, 274677319, 274677350, 274681694, 274681731, 274681984, 274681992, 274682000, 274682023, 274682031, 274682062, 274683772, 274688484, 274744598, 274744606, 274744637, 274808556, 274808564, 274808595, 274855862, 274855870, 274855878, 274855901, 274855909, 274855940, 274891950, 274891987, 274894028, 274898740, 274918953, 274918990, 274919027, 274919064, 274921031, 274921068, 274921105, 274923109, 274923146, 274925187, 274925743, 274925780, 274925817, 274927821, 274927858, 274929899, 274932533, 274932570, 274934611, 274934968, 274934976, 274935005, 274935007, 274935013, 274935044, 274937046, 274937054, 274937085, 274939323, 274941758, 274941766, 274941797, 274962842, 274962879, 274964920, 274969632, 274978284, 274984750, 274984758, 274984766, 274984789, 274984797, 274984828, 274991250, 275002382, 275002390, 275002398, 275002419, 275002421, 275002427, 275002429, 275002435, 275002456, 275002458, 275002460, 275002464, 275002466, 275002472, 275002493, 275002495, 275002497, 275002501, 275002503, 275002509, 275002532, 275002534, 275002540, 275002571, 275004460, 275004468, 275004476, 275004497, 275004499, 275004505, 275004507, 275004513, 275004534, 275004536, 275004538, 275004542, 275004544, 275004550, 275004573, 275004575, 275004581, 275004612, 275005860, 275005868, 275005897, 275005899, 275005905, 275005936, 275006538, 275006546, 275006554, 275006575, 275006577, 275006583, 275006585, 275006591, 275006614, 275006616, 275006622, 275006653, 275007938, 275007946, 275007977, 275008616, 275008624, 275008632, 275008655, 275008663, 275008694, 275009172, 275009180, 275009188, 275009209, 275009211, 275009217, 275009219, 275009225, 275009246, 275009248, 275009250, 275009254, 275009256, 275009262, 275009285, 275009287, 275009293, 275009324, 275011250, 275011258, 275011266, 275011287, 275011289, 275011295, 275011297, 275011303, 275011326, 275011328, 275011334, 275011365, 275012650, 275012658, 275012689, 275013328, 275013336, 275013344, 275013367, 275013375, 275013406, 275015962, 275015970, 275015978, 275015999, 275016001, 275016007, 275016009, 275016015, 275016038, 275016040, 275016046, 275016077, 275017824, 275017832, 275017840, 275017861, 275017863, 275017869, 275017871, 275017877, 275017898, 275017900, 275017902, 275017906, 275017908, 275017914, 275017937, 275017939, 275017945, 275017976, 275018040, 275018048, 275018056, 275018079, 275018087, 275018118, 275019902, 275019910, 275019918, 275019939, 275019941, 275019947, 275019949, 275019955, 275019978, 275019980, 275019986, 275020017, 275021302, 275021310, 275021341, 275021980, 275021988, 275021996, 275022019, 275022027, 275022058, 275022752, 275022760, 275022768, 275022791, 275022799, 275022830, 275024614, 275024622, 275024630, 275024651, 275024653, 275024659, 275024661, 275024667, 275024690, 275024692, 275024698, 275024729, 275026692, 275026700, 275026708, 275026731, 275026739, 275026770, 275031404, 275031412, 275031420, 275031443, 275031451, 275031482, 275143736, 275186754, 275186762, 275186793, 275201506, 275228509, 275228546, 275228583, 275230587, 275230624, 275232665, 275235299, 275235336, 275237377, 275242089, 275244524, 275244532, 275244563, 275272398, 275311938, 275311946, 275311954, 275311975, 275311977, 275311983, 275311985, 275311991, 275312012, 275312014, 275312016, 275312020, 275312022, 275312028, 275312051, 275312053, 275312059, 275312090, 275314016, 275314024, 275314032, 275314053, 275314055, 275314061, 275314063, 275314069, 275314092, 275314094, 275314100, 275314131, 275315416, 275315424, 275315455, 275316094, 275316102, 275316110, 275316133, 275316141, 275316172, 275318728, 275318736, 275318744, 275318765, 275318767, 275318773, 275318775, 275318781, 275318804, 275318806, 275318812, 275318843, 275320806, 275320814, 275320822, 275320845, 275320853, 275320884, 275325518, 275325526, 275325534, 275325557, 275325565, 275325596, 275327380, 275327388, 275327396, 275327417, 275327419, 275327425, 275327427, 275327433, 275327456, 275327458, 275327464, 275327495, 275329458, 275329466, 275329474, 275329497, 275329505, 275329536, 275334170, 275334178, 275334186, 275334209, 275334217, 275334248, 275336356, 275379374, 275379382, 275379413, 275429444, 275454780, 275454788, 275454819, 275462768, 275462805, 275464846, 275469558, 275525672, 275525680, 275525711, 275538065, 275538102, 275540143, 275544855, 275596564, 275596572, 275596603, 275621494, 275621502, 275621510, 275621531, 275621533, 275621539, 275621541, 275621547, 275621570, 275621572, 275621578, 275621609, 275623572, 275623580, 275623588, 275623611, 275623619, 275623650, 275628284, 275628292, 275628300, 275628323, 275628331, 275628362, 275636936, 275636944, 275636952, 275636975, 275636983, 275637014, 275714554, 275765824, 275765832, 275765840, 275765863, 275765871, 275765902, 275772324, 275797410, 275797418, 275797447, 275797449, 275797455, 275797486, 275799488, 275799496, 275799527, 275804200, 275804208, 275804239, 275847621, 275883744, 275883752, 275883783, 275931050, 275931058, 275931066, 275931089, 275931097, 275931128, 275982580, 276009583, 276009620, 276009657, 276011661, 276011698, 276013739, 276016373, 276016410, 276018451, 276023163, 276025598, 276025606, 276025637, 276053472, 276093012, 276093020, 276093028, 276093049, 276093051, 276093057, 276093059, 276093065, 276093086, 276093088, 276093090, 276093094, 276093096, 276093102, 276093125, 276093127, 276093133, 276093164, 276095090, 276095098, 276095106, 276095127, 276095129, 276095135, 276095137, 276095143, 276095166, 276095168, 276095174, 276095205, 276096490, 276096498, 276096529, 276097168, 276097176, 276097184, 276097207, 276097215, 276097246, 276099802, 276099810, 276099818, 276099839, 276099841, 276099847, 276099849, 276099855, 276099878, 276099880, 276099886, 276099917, 276101880, 276101888, 276101896, 276101919, 276101927, 276101958, 276106592, 276106600, 276106608, 276106631, 276106639, 276106670, 276106966, 276106974, 276107005, 276108454, 276108462, 276108470, 276108491, 276108493, 276108499, 276108501, 276108507, 276108530, 276108532, 276108538, 276108569, 276110532, 276110540, 276110548, 276110571, 276110579, 276110610, 276115244, 276115252, 276115260, 276115283, 276115291, 276115322, 276285372, 276319139, 276319176, 276321217, 276325210, 276325247, 276325929, 276327288, 276332000, 276368228, 276368236, 276368265, 276368267, 276368273, 276368304, 276370306, 276370314, 276370345, 276375018, 276375026, 276375057, 276402568, 276402576, 276402584, 276402605, 276402607, 276402613, 276402615, 276402621, 276402644, 276402646, 276402652, 276402683, 276404646, 276404654, 276404662, 276404685, 276404693, 276404724, 276409358, 276409366, 276409374, 276409397, 276409405, 276409436, 276411544, 276418010, 276418018, 276418026, 276418049, 276418057, 276418088, 276451084, 276451092, 276451100, 276451121, 276451123, 276451129, 276451131, 276451137, 276451158, 276451160, 276451162, 276451166, 276451168, 276451174, 276451197, 276451199, 276451205, 276451236, 276453162, 276453170, 276453178, 276453199, 276453201, 276453207, 276453209, 276453215, 276453238, 276453240, 276453246, 276453277, 276454562, 276454570, 276454601, 276455240, 276455248, 276455256, 276455279, 276455287, 276455318, 276457874, 276457882, 276457890, 276457911, 276457913, 276457919, 276457921, 276457927, 276457950, 276457952, 276457958, 276457989, 276459952, 276459960, 276459968, 276459991, 276459999, 276460030, 276464664, 276464672, 276464680, 276464703, 276464711, 276464742, 276504632, 276553398, 276628695, 276634766, 276677784, 276677792, 276677823, 276712124, 276712132, 276712140, 276712163, 276712171, 276712202, 276760640, 276760648, 276760656, 276760677, 276760679, 276760685, 276760687, 276760693, 276760716, 276760718, 276760724, 276760755, 276762718, 276762726, 276762734, 276762757, 276762765, 276762796, 276767430, 276767438, 276767446, 276767469, 276767477, 276767508, 276872598, 276872606, 276872635, 276872637, 276872643, 276872674, 276874676, 276874684, 276874715, 276879388, 276879396, 276879427, 276888040, 276888048, 276888079, 276896028, 276896065, 276898106, 276902818, 276958932, 276958940, 276958971, 277070196, 277070204, 277070212, 277070235, 277070243, 277070274, 277100213, 277100250, 277102291, 277107003, 277182154, 277182162, 277182193, 277183642, 277183650, 277183658, 277183679, 277183681, 277183687, 277183689, 277183695, 277183718, 277183720, 277183726, 277183757, 277185720, 277185728, 277185736, 277185759, 277185767, 277185798, 277190432, 277190440, 277190448, 277190471, 277190479, 277190510, 277199084, 277199092, 277199100, 277199123, 277199131, 277199162, 277205584, 277400398, 277400435, 277402476, 277407188, 277409769, 277415840, 277439938, 277439946, 277439954, 277439975, 277439977, 277439983, 277439985, 277439991, 277440012, 277440014, 277440016, 277440020, 277440022, 277440028, 277440049, 277440051, 277440053, 277440057, 277440059, 277440065, 277440088, 277440090, 277440096, 277440127, 277442016, 277442024, 277442032, 277442053, 277442055, 277442061, 277442063, 277442069, 277442090, 277442092, 277442094, 277442098, 277442100, 277442106, 277442129, 277442131, 277442137, 277442168, 277442843, 277442880, 277442917, 277443416, 277443424, 277443453, 277443455, 277443461, 277443492, 277444094, 277444102, 277444110, 277444131, 277444133, 277444139, 277444141, 277444147, 277444170, 277444172, 277444178, 277444209, 277444921, 277444958, 277445494, 277445502, 277445533, 277446172, 277446180, 277446188, 277446211, 277446219, 277446250, 277446728, 277446736, 277446744, 277446765, 277446767, 277446773, 277446775, 277446781, 277446802, 277446804, 277446806, 277446810, 277446812, 277446818, 277446841, 277446843, 277446849, 277446880, 277446999, 277448806, 277448814, 277448822, 277448843, 277448845, 277448851, 277448853, 277448859, 277448882, 277448884, 277448890, 277448921, 277449633, 277449670, 277450206, 277450214, 277450245, 277450884, 277450892, 277450900, 277450923, 277450931, 277450962, 277451711, 277453518, 277453526, 277453534, 277453555, 277453557, 277453563, 277453565, 277453571, 277453594, 277453596, 277453602, 277453633, 277455596, 277455604, 277455612, 277455635, 277455643, 277455674, 277456423, 277458858, 277458866, 277458897, 277460308, 277460316, 277460324, 277460347, 277460355, 277460386, 277486732, 277493198, 277493206, 277493214, 277493237, 277493245, 277493276, 277526272, 277526280, 277526288, 277526309, 277526311, 277526317, 277526319, 277526325, 277526346, 277526348, 277526350, 277526354, 277526356, 277526362, 277526385, 277526387, 277526393, 277526424, 277528350, 277528358, 277528366, 277528387, 277528389, 277528395, 277528397, 277528403, 277528426, 277528428, 277528434, 277528465, 277529750, 277529758, 277529789, 277530428, 277530436, 277530444, 277530467, 277530475, 277530506, 277533062, 277533070, 277533078, 277533099, 277533101, 277533107, 277533109, 277533115, 277533138, 277533140, 277533146, 277533177, 277535140, 277535148, 277535156, 277535179, 277535187, 277535218, 277539852, 277539860, 277539868, 277539891, 277539899, 277539930, 277541714, 277541722, 277541730, 277541751, 277541753, 277541759, 277541761, 277541767, 277541790, 277541792, 277541798, 277541829, 277543792, 277543800, 277543808, 277543831, 277543839, 277543870, 277548504, 277548512, 277548520, 277548543, 277548551, 277548582, 277709954, 277749494, 277749502, 277749510, 277749531, 277749533, 277749539, 277749541, 277749547, 277749568, 277749570, 277749572, 277749576, 277749578, 277749584, 277749607, 277749609, 277749615, 277749646, 277751572, 277751580, 277751588, 277751609, 277751611, 277751617, 277751619, 277751625, 277751648, 277751650, 277751656, 277751687, 277752399, 277752436, 277752972, 277752980, 277753011, 277753650, 277753658, 277753666, 277753689, 277753697, 277753728, 277754477, 277756284, 277756292, 277756300, 277756321, 277756323, 277756329, 277756331, 277756337, 277756360, 277756362, 277756368, 277756399, 277758362, 277758370, 277758378, 277758401, 277758409, 277758440, 277759189, 277763074, 277763082, 277763090, 277763113, 277763121, 277763152, 277835828, 277835836, 277835844, 277835865, 277835867, 277835873, 277835875, 277835881, 277835904, 277835906, 277835912, 277835943, 277837906, 277837914, 277837922, 277837945, 277837953, 277837984, 277842618, 277842626, 277842634, 277842657, 277842665, 277842696, 277851270, 277851278, 277851286, 277851309, 277851317, 277851348, 277963228, 277963236, 277963267, 277967738, 277967746, 277967775, 277967777, 277967783, 277967812, 277967814, 277967820, 277967849, 277967851, 277967857, 277967888, 277969816, 277969824, 277969853, 277969855, 277969861, 277969890, 277969892, 277969898, 277969929, 277971216, 277971253, 277971894, 277971902, 277971931, 277971933, 277971939, 277971970, 277973294, 277973972, 277973980, 277974011, 277974528, 277974536, 277974565, 277974567, 277974573, 277974602, 277974604, 277974610, 277974641, 277976606, 277976614, 277976643, 277976645, 277976651, 277976682, 277978006, 277978684, 277978692, 277978723, 277981318, 277981326, 277981355, 277981357, 277981363, 277981394, 277983396, 277983404, 277983435, 277986658, 277988108, 277988116, 277988147, 278034120, 278034128, 278034159, 278059050, 278059058, 278059066, 278059087, 278059089, 278059095, 278059097, 278059103, 278059126, 278059128, 278059134, 278059165, 278061128, 278061136, 278061144, 278061167, 278061175, 278061206, 278061955, 278065840, 278065848, 278065856, 278065879, 278065887, 278065918, 278145384, 278145392, 278145400, 278145423, 278145431, 278145462, 278190843, 278274272, 278274280, 278274288, 278274311, 278274319, 278274350, 278277294, 278277302, 278277331, 278277333, 278277339, 278277368, 278277370, 278277376, 278277407, 278279372, 278279380, 278279409, 278279411, 278279417, 278279448, 278280772, 278281450, 278281458, 278281489, 278284084, 278284092, 278284121, 278284123, 278284129, 278284160, 278286162, 278286170, 278286201, 278290874, 278290882, 278290913, 278321300, 278321308, 278321339, 278368606, 278368614, 278368622, 278368645, 278368653, 278368684, 278491028, 278518031, 278518068, 278518105, 278520109, 278520146, 278522187, 278524821, 278524858, 278526899, 278530568, 278530576, 278530584, 278530605, 278530607, 278530613, 278530615, 278530621, 278530642, 278530644, 278530646, 278530650, 278530652, 278530658, 278530681, 278530683, 278530689, 278530720, 278531611, 278532646, 278532654, 278532662, 278532683, 278532685, 278532691, 278532693, 278532699, 278532722, 278532724, 278532730, 278532761, 278533473, 278533510, 278534046, 278534054, 278534085, 278534724, 278534732, 278534740, 278534763, 278534771, 278534802, 278535551, 278537358, 278537366, 278537374, 278537395, 278537397, 278537403, 278537405, 278537411, 278537434, 278537436, 278537442, 278537473, 278539436, 278539444, 278539452, 278539475, 278539483, 278539514, 278540263, 278544148, 278544156, 278544164, 278544187, 278544195, 278544226, 278561920, 278586850, 278586858, 278586887, 278586889, 278586895, 278586926, 278588928, 278588936, 278588967, 278593640, 278593648, 278593679, 278601460, 278601468, 278601476, 278601497, 278601499, 278601505, 278601507, 278601513, 278601534, 278601536, 278601538, 278601542, 278601544, 278601550, 278601573, 278601575, 278601581, 278601612, 278603538, 278603546, 278603554, 278603575, 278603577, 278603583, 278603585, 278603591, 278603614, 278603616, 278603622, 278603653, 278604938, 278604946, 278604977, 278605616, 278605624, 278605632, 278605655, 278605663, 278605694, 278608250, 278608258, 278608266, 278608287, 278608289, 278608295, 278608297, 278608303, 278608326, 278608328, 278608334, 278608365, 278610328, 278610336, 278610344, 278610367, 278610375, 278610406, 278615040, 278615048, 278615056, 278615079, 278615087, 278615118, 278616902, 278616910, 278616918, 278616939, 278616941, 278616947, 278616949, 278616955, 278616978, 278616980, 278616986, 278617017, 278618980, 278618988, 278618996, 278619019, 278619027, 278619058, 278623692, 278623700, 278623708, 278623731, 278623739, 278623770, 278632344, 278632352, 278632360, 278632383, 278632391, 278632422, 278827587, 278827624, 278829665, 278834377, 278840124, 278840132, 278840140, 278840161, 278840163, 278840169, 278840171, 278840177, 278840200, 278840202, 278840208, 278840239, 278842202, 278842210, 278842218, 278842241, 278842249, 278842280, 278843029, 278846914, 278846922, 278846930, 278846953, 278846961, 278846992, 278849100, 278892118, 278892126, 278892157, 278896406, 278896414, 278896445, 278911016, 278911024, 278911032, 278911053, 278911055, 278911061, 278911063, 278911069, 278911092, 278911094, 278911100, 278911131, 278913094, 278913102, 278913110, 278913133, 278913141, 278913172, 278917806, 278917814, 278917822, 278917845, 278917853, 278917884, 278926458, 278926466, 278926474, 278926497, 278926505, 278926536, 278942188, 278974974, 278974982, 278974990, 278975011, 278975013, 278975019, 278975021, 278975027, 278975050, 278975052, 278975058, 278975089, 278977052, 278977060, 278977068, 278977091, 278977099, 278977130, 278981764, 278981772, 278981780, 278981803, 278981811, 278981842, 279058368, 279058376, 279058405, 279058407, 279058413, 279058442, 279058444, 279058450, 279058481, 279060446, 279060454, 279060483, 279060485, 279060491, 279060522, 279061846, 279062524, 279062532, 279062563, 279065158, 279065166, 279065195, 279065197, 279065203, 279065234, 279067236, 279067244, 279067275, 279071948, 279071956, 279071987, 279137143, 279149680, 279149688, 279149696, 279149719, 279149727, 279149758, 279220572, 279220580, 279220588, 279220611, 279220619, 279220650, 279284530, 279284538, 279284546, 279284569, 279284577, 279284608, 279310154, 279310162, 279310191, 279310193, 279310199, 279310230, 279312232, 279312240, 279312271, 279316944, 279316952, 279316983, 279367924, 279367932, 279367961, 279367963, 279367969, 279368000, 279370002, 279370010, 279370041, 279374714, 279374722, 279374753, 279396488, 279396496, 279396527, 279419918, 279608661, 279608698, 279610739, 279615451, 279619710, 279619718, 279619749, 279621198, 279621206, 279621214, 279621235, 279621237, 279621243, 279621245, 279621251, 279621274, 279621276, 279621282, 279621313, 279623276, 279623284, 279623292, 279623315, 279623323, 279623354, 279624103, 279627988, 279627996, 279628004, 279628027, 279628035, 279628066, 279677480, 279677488, 279677519, 279692090, 279692098, 279692106, 279692127, 279692129, 279692135, 279692137, 279692143, 279692166, 279692168, 279692174, 279692205, 279694168, 279694176, 279694184, 279694207, 279694215, 279694246, 279698880, 279698888, 279698896, 279698919, 279698927, 279698958, 279707532, 279707540, 279707548, 279707571, 279707579, 279707610, 279837954, 279837991, 279840032, 279844744, 279880972, 279880980, 279881009, 279881011, 279881017, 279881048, 279883050, 279883058, 279883089, 279887762, 279887770, 279887801, 279918217, 279924288, 279930754, 279930762, 279930770, 279930793, 279930801, 279930832, 279963828, 279963836, 279963844, 279963865, 279963867, 279963873, 279963875, 279963881, 279963902, 279963904, 279963906, 279963910, 279963912, 279963918, 279963941, 279963943, 279963949, 279963980, 279965906, 279965914, 279965922, 279965943, 279965945, 279965951, 279965953, 279965959, 279965982, 279965984, 279965990, 279966021, 279966733, 279966770, 279967306, 279967314, 279967345, 279967984, 279967992, 279968000, 279968023, 279968031, 279968062, 279968811, 279970618, 279970626, 279970634, 279970655, 279970657, 279970663, 279970665, 279970671, 279970694, 279970696, 279970702, 279970733, 279972696, 279972704, 279972712, 279972735, 279972743, 279972774, 279973523, 279977408, 279977416, 279977424, 279977447, 279977455, 279977486, 280001646, 280001654, 280001662, 280001685, 280001693, 280001724, 280050162, 280050170, 280050178, 280050199, 280050201, 280050207, 280050209, 280050215, 280050238, 280050240, 280050246, 280050277, 280052240, 280052248, 280052256, 280052279, 280052287, 280052318, 280056952, 280056960, 280056968, 280056991, 280056999, 280057030, 280065604, 280065612, 280065620, 280065643, 280065651, 280065682, 280147510, 280148998, 280149006, 280149035, 280149037, 280149043, 280149074, 280151076, 280151084, 280151115, 280155788, 280155796, 280155827, 280190528, 280190536, 280190567, 280273384, 280273392, 280273400, 280273421, 280273423, 280273429, 280273431, 280273437, 280273460, 280273462, 280273468, 280273499, 280275462, 280275470, 280275478, 280275501, 280275509, 280275540, 280276289, 280280174, 280280182, 280280190, 280280213, 280280221, 280280252, 280359718, 280359726, 280359734, 280359757, 280359765, 280359796, 280400784, 280400792, 280400823, 280408772, 280408809, 280410850, 280415562, 280458554, 280458562, 280458593, 280471676, 280471684, 280471715, 280491628, 280491636, 280491665, 280491667, 280491673, 280491702, 280491704, 280491710, 280491741, 280493706, 280493714, 280493743, 280493745, 280493751, 280493782, 280495106, 280495784, 280495792, 280495823, 280498418, 280498426, 280498455, 280498457, 280498463, 280498494, 280500496, 280500504, 280500535, 280505208, 280505216, 280505247, 280582940, 280582948, 280582956, 280582979, 280582987, 280583018, 280699291, 280711828, 280711836, 280711844, 280711867, 280711875, 280711906, 280718328, 280782720, 280782728, 280782736, 280782759, 280782767, 280782798, 280801184, 280801192, 280801221, 280801223, 280801229, 280801260, 280803262, 280803270, 280803301, 280807974, 280807982, 280808013, 280928584, 280955587, 280955624, 280955661, 280957665, 280957702, 280959743, 280962377, 280962414, 280964455, 280969167, 280971602, 280971610, 280971641, 280979590, 280979627, 280981668, 280986380, 280999476, 281039016, 281039024, 281039032, 281039053, 281039055, 281039061, 281039063, 281039069, 281039090, 281039092, 281039094, 281039098, 281039100, 281039106, 281039129, 281039131, 281039137, 281039168, 281041094, 281041102, 281041110, 281041131, 281041133, 281041139, 281041141, 281041147, 281041170, 281041172, 281041178, 281041209, 281041921, 281041958, 281042494, 281042502, 281042533, 281043172, 281043180, 281043188, 281043211, 281043219, 281043250, 281043999, 281045806, 281045814, 281045822, 281045843, 281045845, 281045851, 281045853, 281045859, 281045882, 281045884, 281045890, 281045921, 281047884, 281047892, 281047900, 281047923, 281047931, 281047962, 281048711, 281052596, 281052604, 281052612, 281052635, 281052643, 281052674, 281054458, 281054466, 281054474, 281054495, 281054497, 281054503, 281054505, 281054511, 281054534, 281054536, 281054542, 281054573, 281056536, 281056544, 281056552, 281056575, 281056583, 281056614, 281057363, 281061248, 281061256, 281061264, 281061287, 281061295, 281061326, 281110740, 281110748, 281110779, 281125350, 281125358, 281125366, 281125387, 281125389, 281125395, 281125397, 281125403, 281125426, 281125428, 281125434, 281125465, 281127428, 281127436, 281127444, 281127467, 281127475, 281127506, 281132140, 281132148, 281132156, 281132179, 281132187, 281132218, 281140792, 281140800, 281140808, 281140831, 281140839, 281140870, 281239628, 281239636, 281239667, 281265143, 281265180, 281267221, 281271933, 281289146, 281348572, 281348580, 281348588, 281348609, 281348611, 281348617, 281348619, 281348625, 281348648, 281348650, 281348656, 281348687, 281350650, 281350658, 281350666, 281350689, 281350697, 281350728, 281351477, 281355362, 281355370, 281355378, 281355401, 281355409, 281355440, 281364014, 281364022, 281364030, 281364053, 281364061, 281364092, 281434906, 281434914, 281434922, 281434945, 281434953, 281434984, 281498864, 281498872, 281498880, 281498903, 281498911, 281498942, 281499402, 281566816, 281566824, 281566853, 281566855, 281566861, 281566890, 281566892, 281566898, 281566929, 281568894, 281568902, 281568931, 281568933, 281568939, 281568970, 281570294, 281570972, 281570980, 281571011, 281573606, 281573614, 281573643, 281573645, 281573651, 281573682, 281574699, 281575684, 281575692, 281575723, 281580396, 281580404, 281580435, 281582258, 281582266, 281582295, 281582297, 281582303, 281582334, 281584336, 281584344, 281584375, 281589048, 281589056, 281589087, 281658128, 281658136, 281658144, 281658167, 281658175, 281658206, 281834044, 281834052, 281834083, 281876372, 281876380, 281876409, 281876411, 281876417, 281876448, 281878450, 281878458, 281878489, 281883162, 281883170, 281883201, 281891814, 281891822, 281891853, 282046217, 282046254, 282048295, 282053007, 282070220, 282117109, 282117146, 282119187, 282123899, 282129646, 282129654, 282129662, 282129683, 282129685, 282129691, 282129693, 282129699, 282129722, 282129724, 282129730, 282129761, 282131724, 282131732, 282131740, 282131763, 282131771, 282131802, 282132551, 282136436, 282136444, 282136452, 282136475, 282136483, 282136514, 282145088, 282145096, 282145104, 282145127, 282145135, 282145166, 282185928, 282185936, 282185967, 282200538, 282200546, 282200554, 282200575, 282200577, 282200583, 282200585, 282200591, 282200614, 282200616, 282200622, 282200653, 282202616, 282202624, 282202632, 282202655, 282202663, 282202694, 282207328, 282207336, 282207344, 282207367, 282207375, 282207406, 282215980, 282215988, 282215996, 282216019, 282216027, 282216058, 282355773, 282361844, 282404862, 282404870, 282404901, 282426665, 282439202, 282439210, 282439218, 282439241, 282439249, 282439280, 282487718, 282487726, 282487734, 282487755, 282487757, 282487763, 282487765, 282487771, 282487794, 282487796, 282487802, 282487833, 282489796, 282489804, 282489812, 282489835, 282489843, 282489874, 282490623, 282494508, 282494516, 282494524, 282494547, 282494555, 282494586, 282510094, 282510102, 282510110, 282510133, 282510141, 282510172, 282561791, 282561799, 282561830, 282574052, 282574060, 282574068, 282574091, 282574099, 282574130, 282657446, 282657454, 282657483, 282657485, 282657491, 282657522, 282659524, 282659532, 282659563, 282664236, 282664244, 282664275, 282672888, 282672896, 282672927, 282797274, 282797282, 282797290, 282797313, 282797321, 282797352, 282909232, 282909240, 282909271, 282932662, 282967002, 282967010, 282967041, 283015518, 283015526, 283015555, 283015557, 283015563, 283015594, 283017596, 283017604, 283017635, 283022308, 283022316, 283022347, 283136847, 283207739, 283220276, 283220284, 283220292, 283220315, 283220323, 283220354, 283291168, 283291176, 283291184, 283291207, 283291215, 283291246, 283325074, 283325082, 283325113, 283437032, 283476572, 283476580, 283476588, 283476609, 283476611, 283476617, 283476619, 283476625, 283476646, 283476648, 283476650, 283476654, 283476656, 283476662, 283476685, 283476687, 283476693, 283476724, 283478650, 283478658, 283478666, 283478687, 283478689, 283478695, 283478697, 283478703, 283478726, 283478728, 283478734, 283478765, 283479477, 283479514, 283480050, 283480058, 283480089, 283480728, 283480736, 283480744, 283480767, 283480775, 283480806, 283481555, 283483362, 283483370, 283483378, 283483399, 283483401, 283483407, 283483409, 283483415, 283483438, 283483440, 283483446, 283483477, 283485440, 283485448, 283485456, 283485479, 283485487, 283485518, 283486267, 283490152, 283490160, 283490168, 283490191, 283490199, 283490230, 283503480, 283562906, 283562914, 283562922, 283562943, 283562945, 283562951, 283562953, 283562959, 283562982, 283562984, 283562990, 283563021, 283564984, 283564992, 283565000, 283565023, 283565031, 283565062, 283565811, 283569696, 283569704, 283569712, 283569735, 283569743, 283569774, 283578348, 283578356, 283578364, 283578387, 283578395, 283578426, 283649240, 283649248, 283649256, 283649279, 283649287, 283649318, 283748076, 283748084, 283748115, 283786128, 283786136, 283786144, 283786165, 283786167, 283786173, 283786175, 283786181, 283786204, 283786206, 283786212, 283786243, 283788206, 283788214, 283788222, 283788245, 283788253, 283788284, 283789033, 283792918, 283792926, 283792934, 283792957, 283792965, 283792996, 283872462, 283872470, 283872478, 283872501, 283872509, 283872540, 284004372, 284004380, 284004409, 284004411, 284004417, 284004446, 284004448, 284004454, 284004485, 284006450, 284006458, 284006487, 284006489, 284006495, 284006526, 284007850, 284008528, 284008536, 284008567, 284011162, 284011170, 284011199, 284011201, 284011207, 284011238, 284013240, 284013248, 284013279, 284017952, 284017960, 284017991, 284090706, 284090714, 284090743, 284090745, 284090751, 284090782, 284092784, 284092792, 284092823, 284095684, 284095692, 284095700, 284095723, 284095731, 284095762, 284097496, 284097504, 284097535, 284106148, 284106156, 284106187, 284313928, 284313936, 284313965, 284313967, 284313973, 284314004, 284316006, 284316014, 284316045, 284320718, 284320726, 284320757, 284400262, 284400270, 284400301, 284532172, 284532209, 284532246, 284534250, 284534287, 284536328, 284538962, 284538999, 284541040, 284545752, 284554665, 284554702, 284556743, 284561455, 284567202, 284567210, 284567218, 284567239, 284567241, 284567247, 284567249, 284567255, 284567278, 284567280, 284567286, 284567317, 284569280, 284569288, 284569296, 284569319, 284569327, 284569358, 284570107, 284573992, 284574000, 284574008, 284574031, 284574039, 284574070, 284575190, 284575198, 284575227, 284575229, 284575235, 284575264, 284575266, 284575272, 284575303, 284577268, 284577276, 284577305, 284577307, 284577313, 284577344, 284578668, 284579346, 284579354, 284579385, 284581980, 284581988, 284582017, 284582019, 284582025, 284582056, 284584058, 284584066, 284584097, 284588770, 284588778, 284588809, 284623484, 284623492, 284623523, 284638094, 284638102, 284638110, 284638131, 284638133, 284638139, 284638141, 284638147, 284638170, 284638172, 284638178, 284638209, 284640172, 284640180, 284640188, 284640211, 284640219, 284640250, 284640999, 284644884, 284644892, 284644900, 284644923, 284644931, 284644962, 284653536, 284653544, 284653552, 284653575, 284653583, 284653614, 284724428, 284724436, 284724444, 284724467, 284724475, 284724506, 284841728, 284841765, 284843806, 284848518, 284864221, 284876758, 284876766, 284876774, 284876797, 284876805, 284876836, 284884746, 284884754, 284884783, 284884785, 284884791, 284884822, 284886824, 284886832, 284886863, 284891536, 284891544, 284891575, 284947650, 284947658, 284947666, 284947689, 284947697, 284947728, 285011608, 285011616, 285011624, 285011647, 285011655, 285011686, 285078987, 285079024, 285079061, 285079098, 285081065, 285081102, 285081139, 285083143, 285083180, 285085221, 285085777, 285085814, 285085851, 285087855, 285087892, 285089933, 285092567, 285092604, 285094645, 285095002, 285095010, 285095039, 285095041, 285095047, 285095078, 285097080, 285097088, 285097119, 285099357, 285101792, 285101800, 285101831, 285151284, 285165894, 285165902, 285165931, 285165933, 285165939, 285165970, 285167972, 285167980, 285168011, 285172684, 285172692, 285172723, 285181336, 285181344, 285181375, 285194302, 285194310, 285194341, 285346788, 285346796, 285346827, 285388543, 285388580, 285388617, 285390621, 285390658, 285392699, 285395333, 285395370, 285397411, 285402123, 285404558, 285404566, 285404597, 285475450, 285475458, 285475489, 285539408, 285539416, 285539447, 285622802, 285622839, 285624880, 285629592, 285645295, 285657832, 285657840, 285657848, 285657871, 285657879, 285657910, 285665820, 285665828, 285665857, 285665859, 285665865, 285665896, 285667898, 285667906, 285667937, 285672610, 285672618, 285672649, 285698099, 285698136, 285700177, 285704889, 285716187, 285728724, 285728732, 285728740, 285728763, 285728771, 285728802, 285799616, 285799624, 285799632, 285799655, 285799663, 285799694, 285874588, 285917606, 285917614, 285917645, 285932358, 285975376, 285975384, 285975415, 286000462, 286000470, 286000478, 286000499, 286000501, 286000507, 286000509, 286000515, 286000538, 286000540, 286000546, 286000577, 286002540, 286002548, 286002556, 286002579, 286002587, 286002618, 286003367, 286007252, 286007260, 286007268, 286007291, 286007299, 286007330, 286007655, 286086796, 286086804, 286086812, 286086835, 286086843, 286086874, 286169617, 286169654, 286169691, 286171695, 286171732, 286173773, 286176407, 286176444, 286178485, 286183197, 286185632, 286185640, 286185671, 286256524, 286256532, 286256563, 286310018, 286310026, 286310034, 286310057, 286310065, 286310096, 286445406, 286479173, 286479210, 286481251, 286485963, 286528262, 286528270, 286528299, 286528301, 286528307, 286528338, 286530340, 286530348, 286530379, 286535052, 286535060, 286535091, 286614596, 286614604, 286614635, 286713432, 286756450, 286756458, 286756489, 286788729, 286837818, 286837826, 286837857, 286992221, 286992258, 286994299, 286999011, 287016224, 287056062, 287056099, 287058140, 287062852, 287075650, 287075658, 287075666, 287075687, 287075689, 287075695, 287075697, 287075703, 287075726, 287075728, 287075734, 287075765, 287077728, 287077736, 287077744, 287077767, 287077775, 287077806, 287078555, 287082440, 287082448, 287082456, 287082479, 287082487, 287082518, 287091092, 287091100, 287091108, 287091131, 287091139, 287091170, 287099080, 287099088, 287099117, 287099119, 287099125, 287099156, 287101158, 287101166, 287101197, 287105870, 287105878, 287105909, 287161984, 287161992, 287162000, 287162023, 287162031, 287162062, 287260247, 287260284, 287262325, 287267037, 287301777, 287365618, 287385206, 287385214, 287385222, 287385245, 287385253, 287385284, 287408636, 287408644, 287408675, 287569803, 287602877, 287602914, 287602951, 287603450, 287603458, 287603487, 287603489, 287603495, 287603526, 287604955, 287604992, 287605528, 287605536, 287605567, 287607033, 287609667, 287609704, 287610240, 287610248, 287610279, 287611745, 287616457, 287618892, 287618900, 287618931, 287689784, 287689792, 287689823, 287912433, 287912470, 287913006, 287913014, 287913045, 287914511, 287919223, 288082851, 288131250, 288131287, 288133328, 288138040, 288146692, 288153743, 288166280, 288166288, 288166296, 288166319, 288166327, 288166358, 288170790, 288170798, 288170806, 288170827, 288170829, 288170835, 288170837, 288170843, 288170864, 288170866, 288170868, 288170872, 288170874, 288170880, 288170901, 288170903, 288170905, 288170909, 288170911, 288170917, 288170940, 288170942, 288170948, 288170979, 288172868, 288172876, 288172884, 288172905, 288172907, 288172913, 288172915, 288172921, 288172942, 288172944, 288172946, 288172950, 288172952, 288172958, 288172981, 288172983, 288172989, 288173020, 288174268, 288174276, 288174305, 288174307, 288174313, 288174344, 288174946, 288174954, 288174962, 288174983, 288174985, 288174991, 288174993, 288174999, 288175022, 288175024, 288175030, 288175061, 288176346, 288176354, 288176385, 288177024, 288177032, 288177040, 288177063, 288177071, 288177102, 288177580, 288177588, 288177596, 288177617, 288177619, 288177625, 288177627, 288177633, 288177654, 288177656, 288177658, 288177662, 288177664, 288177670, 288177693, 288177695, 288177701, 288177732, 288179658, 288179666, 288179674, 288179695, 288179697, 288179703, 288179705, 288179711, 288179734, 288179736, 288179742, 288179773, 288181058, 288181066, 288181097, 288181736, 288181744, 288181752, 288181775, 288181783, 288181814, 288184370, 288184378, 288184386, 288184407, 288184409, 288184415, 288184417, 288184423, 288184446, 288184448, 288184454, 288184485, 288186448, 288186456, 288186464, 288186487, 288186495, 288186526, 288189710, 288189718, 288189749, 288191160, 288191168, 288191176, 288191199, 288191207, 288191238, 288221989, 288237172, 288237180, 288237188, 288237211, 288237219, 288237250, 288350877, 288440806, 288480346, 288480354, 288480362, 288480383, 288480385, 288480391, 288480393, 288480399, 288480420, 288480422, 288480424, 288480428, 288480430, 288480436, 288480459, 288480461, 288480467, 288480498, 288482424, 288482432, 288482440, 288482461, 288482463, 288482469, 288482471, 288482477, 288482500, 288482502, 288482508, 288482539, 288483824, 288483832, 288483863, 288484502, 288484510, 288484518, 288484541, 288484549, 288484580, 288487136, 288487144, 288487152, 288487173, 288487175, 288487181, 288487183, 288487189, 288487212, 288487214, 288487220, 288487251, 288489214, 288489222, 288489230, 288489253, 288489261, 288489292, 288493926, 288493934, 288493942, 288493965, 288493973, 288494004, 288524352, 288524360, 288524368, 288524391, 288524399, 288524430, 288678065, 288678102, 288678139, 288680143, 288680180, 288682221, 288684855, 288684892, 288686933, 288691645, 288693507, 288693544, 288694080, 288694088, 288694119, 288695585, 288700297, 288764972, 288764980, 288765011, 288789902, 288789910, 288789918, 288789939, 288789941, 288789947, 288789949, 288789955, 288789978, 288789980, 288789986, 288790017, 288791980, 288791988, 288791996, 288792019, 288792027, 288792058, 288796692, 288796700, 288796708, 288796731, 288796739, 288796770, 288987621, 288987658, 288989699, 288994411, 289003063, 289052152, 289052160, 289052191, 289099458, 289099466, 289099474, 289099497, 289099505, 289099536, 289221880, 289261420, 289261428, 289261436, 289261457, 289261459, 289261465, 289261467, 289261473, 289261494, 289261496, 289261498, 289261502, 289261504, 289261510, 289261533, 289261535, 289261541, 289261572, 289263498, 289263506, 289263514, 289263535, 289263537, 289263543, 289263545, 289263551, 289263574, 289263576, 289263582, 289263613, 289264898, 289264906, 289264937, 289265576, 289265584, 289265592, 289265615, 289265623, 289265654, 289268210, 289268218, 289268226, 289268247, 289268249, 289268255, 289268257, 289268263, 289268286, 289268288, 289268294, 289268325, 289270288, 289270296, 289270304, 289270327, 289270335, 289270366, 289275000, 289275008, 289275016, 289275039, 289275047, 289275078, 289297177, 289513206, 289513214, 289513222, 289513243, 289513245, 289513251, 289513253, 289513259, 289513282, 289513284, 289513290, 289513321, 289515284, 289515292, 289515300, 289515323, 289515331, 289515362, 289516111, 289519996, 289520004, 289520012, 289520035, 289520043, 289520074, 289570976, 289570984, 289570992, 289571013, 289571015, 289571021, 289571023, 289571029, 289571052, 289571054, 289571060, 289571091, 289573054, 289573062, 289573070, 289573093, 289573101, 289573132, 289577766, 289577774, 289577782, 289577805, 289577813, 289577844, 289579952, 289599540, 289599548, 289599556, 289599579, 289599587, 289599618, 289622970, 289622978, 289623009, 289673040, 289768695, 289768732, 289770773, 289775485, 289784137, 289822762, 289822770, 289822778, 289822801, 289822809, 289822840, 289880532, 289880540, 289880548, 289880571, 289880579, 289880610, 290041006, 290041014, 290041043, 290041045, 290041051, 290041082, 290043084, 290043092, 290043123, 290047796, 290047804, 290047835, 290078251, 290126767, 290126804, 290127340, 290127348, 290127379, 290128845, 290133557, 290350562, 290350570, 290350601, 290352050, 290352058, 290352066, 290352087, 290352089, 290352095, 290352097, 290352103, 290352126, 290352128, 290352134, 290352165, 290354128, 290354136, 290354144, 290354167, 290354175, 290354206, 290358840, 290358848, 290358856, 290358879, 290358887, 290358918, 290436323, 290568806, 290568843, 290570884, 290575596, 290591299, 290603836, 290603844, 290603852, 290603875, 290603883, 290603914, 290611824, 290611832, 290611861, 290611863, 290611869, 290611900, 290613902, 290613910, 290613941, 290618614, 290618622, 290618653, 290655140, 290661606, 290661614, 290661622, 290661645, 290661653, 290661684, 290674728, 290674736, 290674744, 290674767, 290674775, 290674806, 290694680, 290694688, 290694696, 290694717, 290694719, 290694725, 290694727, 290694733, 290694754, 290694756, 290694758, 290694762, 290694764, 290694770, 290694793, 290694795, 290694801, 290694832, 290696758, 290696766, 290696774, 290696795, 290696797, 290696803, 290696805, 290696811, 290696834, 290696836, 290696842, 290696873, 290698158, 290698166, 290698197, 290698836, 290698844, 290698852, 290698875, 290698883, 290698914, 290701470, 290701478, 290701486, 290701507, 290701509, 290701515, 290701517, 290701523, 290701546, 290701548, 290701554, 290701585, 290703548, 290703556, 290703564, 290703587, 290703595, 290703626, 290708260, 290708268, 290708276, 290708299, 290708307, 290708338, 290859325, 290878362, 290921380, 290921388, 290921419, 291004236, 291004244, 291004252, 291004273, 291004275, 291004281, 291004283, 291004289, 291004312, 291004314, 291004320, 291004351, 291006314, 291006322, 291006330, 291006353, 291006361, 291006392, 291011026, 291011034, 291011042, 291011065, 291011073, 291011104, 291115621, 291115658, 291115695, 291117699, 291117736, 291119777, 291122411, 291122448, 291124489, 291129201, 291131636, 291131644, 291131675, 291139624, 291139661, 291141702, 291146414, 291201955, 291201992, 291202528, 291202536, 291202567, 291204033, 291208745, 291217397, 291313792, 291313800, 291313808, 291313831, 291313839, 291313870, 291425177, 291425214, 291427255, 291431967, 291442680, 291442688, 291442696, 291442719, 291442727, 291442758, 291449180, 291511511, 291659436, 291686439, 291686476, 291686513, 291688517, 291688554, 291690595, 291693229, 291693266, 291695307, 291700019, 291702454, 291702462, 291702493, 291730328, 291734733, 291769868, 291769876, 291769884, 291769905, 291769907, 291769913, 291769915, 291769921, 291769942, 291769944, 291769946, 291769950, 291769952, 291769958, 291769981, 291769983, 291769989, 291770020, 291771946, 291771954, 291771962, 291771983, 291771985, 291771991, 291771993, 291771999, 291772022, 291772024, 291772030, 291772061, 291773346, 291773354, 291773385, 291774024, 291774032, 291774040, 291774063, 291774071, 291774102, 291776658, 291776666, 291776674, 291776695, 291776697, 291776703, 291776705, 291776711, 291776734, 291776736, 291776742, 291776773, 291778736, 291778744, 291778752, 291778775, 291778783, 291778814, 291783448, 291783456, 291783464, 291783487, 291783495, 291783526, 291785310, 291785318, 291785326, 291785347, 291785349, 291785355, 291785357, 291785363, 291785386, 291785388, 291785394, 291785425, 291787388, 291787396, 291787404, 291787427, 291787435, 291787466, 291792100, 291792108, 291792116, 291792139, 291792147, 291792178, 291995995, 291996032, 291998073, 292002785, 292037096, 292037104, 292037112, 292037135, 292037143, 292037174, 292079424, 292079432, 292079440, 292079461, 292079463, 292079469, 292079471, 292079477, 292079500, 292079502, 292079508, 292079539, 292081502, 292081510, 292081518, 292081541, 292081549, 292081580, 292086214, 292086222, 292086230, 292086253, 292086261, 292086292, 292094866, 292094874, 292094882, 292094905, 292094913, 292094944, 292206251, 292206288, 292208329, 292213041, 292230254, 292277143, 292277180, 292279221, 292283933, 292292585, 292305551, 292388980, 292388988, 292388996, 292389019, 292389027, 292389058, 292515807, 292564896, 292564904, 292564935, 292586699, 292650657, 292777069, 292777106, 292779147, 292783859, 292860498, 292860506, 292860514, 292860535, 292860537, 292860543, 292860545, 292860551, 292860574, 292860576, 292860582, 292860613, 292862576, 292862584, 292862592, 292862615, 292862623, 292862654, 292867288, 292867296, 292867304, 292867327, 292867335, 292867366, 292875940, 292875948, 292875956, 292875979, 292875987, 292876018, 293028855, 293086625, 293092696, 293112284, 293112292, 293112300, 293112323, 293112331, 293112362, 293135714, 293135722, 293135753, 293170054, 293170062, 293170070, 293170093, 293170101, 293170132, 293218570, 293218578, 293218586, 293218607, 293218609, 293218615, 293218617, 293218623, 293218646, 293218648, 293218654, 293218685, 293220648, 293220656, 293220664, 293220687, 293220695, 293220726, 293225360, 293225368, 293225376, 293225399, 293225407, 293225438, 293296881, 293367773, 293528126, 293528134, 293528142, 293528165, 293528173, 293528204, 293639511, 293639548, 293640084, 293640092, 293640123, 293641589, 293646301, 293663514, 293725845, 293867699, 293949067, 293951128, 293951136, 293951144, 293951167, 293951175, 293951206, 294167884, 294207424, 294207432, 294207440, 294207461, 294207463, 294207469, 294207471, 294207477, 294207498, 294207500, 294207502, 294207506, 294207508, 294207514, 294207537, 294207539, 294207545, 294207576, 294209502, 294209510, 294209518, 294209539, 294209541, 294209547, 294209549, 294209555, 294209578, 294209580, 294209586, 294209617, 294210329, 294210366, 294210902, 294210910, 294210941, 294211580, 294211588, 294211596, 294211619, 294211627, 294211658, 294212407, 294214214, 294214222, 294214230, 294214251, 294214253, 294214259, 294214261, 294214267, 294214290, 294214292, 294214298, 294214329, 294216292, 294216300, 294216308, 294216331, 294216339, 294216370, 294217119, 294221004, 294221012, 294221020, 294221043, 294221051, 294221082, 294293758, 294293766, 294293774, 294293795, 294293797, 294293803, 294293805, 294293811, 294293834, 294293836, 294293842, 294293873, 294295836, 294295844, 294295852, 294295875, 294295883, 294295914, 294300548, 294300556, 294300564, 294300587, 294300595, 294300626, 294309200, 294309208, 294309216, 294309239, 294309247, 294309278, 294516980, 294516988, 294516996, 294517017, 294517019, 294517025, 294517027, 294517033, 294517056, 294517058, 294517064, 294517095, 294519058, 294519066, 294519074, 294519097, 294519105, 294519136, 294519885, 294523770, 294523778, 294523786, 294523809, 294523817, 294523848, 294603314, 294603322, 294603330, 294603353, 294603361, 294603392, 294714699, 294714736, 294716777, 294721489, 294730141, 294735224, 294735232, 294735261, 294735263, 294735269, 294735298, 294735300, 294735306, 294735337, 294737302, 294737310, 294737339, 294737341, 294737347, 294737378, 294738702, 294739380, 294739388, 294739419, 294742014, 294742022, 294742051, 294742053, 294742059, 294742090, 294744092, 294744100, 294744131, 294748804, 294748812, 294748843, 294801033, 294826536, 294826544, 294826552, 294826575, 294826583, 294826614, 295024255, 295044780, 295044788, 295044817, 295044819, 295044825, 295044856, 295046858, 295046866, 295046897, 295051570, 295051578, 295051609, 295282039, 295282047, 295282076, 295282078, 295282084, 295282113, 295282115, 295282121, 295282150, 295282152, 295282158, 295282189, 295284117, 295284125, 295284154, 295284156, 295284162, 295284191, 295284193, 295284199, 295284230, 295285517, 295285554, 295286195, 295286203, 295286232, 295286234, 295286240, 295286271, 295287595, 295288273, 295288281, 295288312, 295288829, 295288837, 295288866, 295288868, 295288874, 295288903, 295288905, 295288911, 295288942, 295290907, 295290915, 295290944, 295290946, 295290952, 295290983, 295292307, 295292985, 295292993, 295293024, 295295619, 295295627, 295295656, 295295658, 295295664, 295295695, 295297697, 295297705, 295297736, 295298054, 295298062, 295298070, 295298091, 295298093, 295298099, 295298101, 295298107, 295298130, 295298132, 295298138, 295298169, 295300132, 295300140, 295300148, 295300171, 295300179, 295300210, 295300959, 295302409, 295302417, 295302448, 295304844, 295304852, 295304860, 295304883, 295304891, 295304922, 295354336, 295354344, 295354375, 295368946, 295368954, 295368962, 295368983, 295368985, 295368991, 295368993, 295368999, 295369022, 295369024, 295369030, 295369061, 295371024, 295371032, 295371040, 295371063, 295371071, 295371102, 295375736, 295375744, 295375752, 295375775, 295375783, 295375814, 295384388, 295384396, 295384404, 295384427, 295384435, 295384466, 295549840, 295549848, 295549856, 295549879, 295549887, 295549918, 295591595, 295591603, 295591632, 295591634, 295591640, 295591669, 295591671, 295591677, 295591708, 295593673, 295593681, 295593710, 295593712, 295593718, 295593749, 295595073, 295595751, 295595759, 295595790, 295598385, 295598393, 295598422, 295598424, 295598430, 295598461, 295600463, 295600471, 295600502, 295605175, 295605183, 295605214, 295607610, 295607618, 295607626, 295607649, 295607657, 295607688, 295678502, 295678510, 295678518, 295678541, 295678549, 295678580, 295742460, 295742468, 295742476, 295742499, 295742507, 295742538, 295805329, 295825854, 295825862, 295825891, 295825893, 295825899, 295825930, 295827932, 295827940, 295827971, 295832644, 295832652, 295832683, 295876221, 295901151, 295901159, 295901188, 295901190, 295901196, 295901227, 295903229, 295903237, 295903268, 295907941, 295907949, 295907980, 296077640, 296077648, 296077679, 296135410, 296135418, 296135449, 296163401, 296210707, 296210715, 296210746, 296372669, 296372677, 296372706, 296372708, 296372714, 296372743, 296372745, 296372751, 296372782, 296374747, 296374755, 296374784, 296374786, 296374792, 296374823, 296376147, 296376825, 296376833, 296376864, 296379459, 296379467, 296379496, 296379498, 296379504, 296379535, 296381537, 296381545, 296381576, 296386249, 296386257, 296386288, 296388684, 296388692, 296388700, 296388723, 296388731, 296388762, 296459576, 296459584, 296459592, 296459615, 296459623, 296459654, 296605440, 296648458, 296648466, 296648497, 296682225, 296682233, 296682262, 296682264, 296682270, 296682301, 296684303, 296684311, 296684342, 296689015, 296689023, 296689054, 296731314, 296731322, 296731330, 296731351, 296731353, 296731359, 296731361, 296731367, 296731390, 296731392, 296731398, 296731429, 296733392, 296733400, 296733408, 296733431, 296733439, 296733470, 296734219, 296738104, 296738112, 296738120, 296738143, 296738151, 296738182, 296817648, 296817656, 296817664, 296817687, 296817695, 296817726, 296916484, 296916492, 296916523, 296991781, 296991789, 296991820, 297040870, 297040878, 297040886, 297040909, 297040917, 297040948, 297152255, 297152292, 297154333, 297159045, 297176258, 297238589, 297259114, 297259122, 297259151, 297259153, 297259159, 297259190, 297261192, 297261200, 297261231, 297265904, 297265912, 297265943, 297461811, 297463299, 297463307, 297463336, 297463338, 297463344, 297463375, 297465377, 297465385, 297465416, 297470089, 297470097, 297470128, 297568670, 297568678, 297568709, 297723073, 297723110, 297725151, 297729863, 297747076, 297772855, 297772863, 297772894, 297805929, 297805937, 297805966, 297805968, 297805974, 297806003, 297806005, 297806011, 297806042, 297806502, 297806510, 297806518, 297806539, 297806541, 297806547, 297806549, 297806555, 297806578, 297806580, 297806586, 297806617, 297808007, 297808015, 297808044, 297808046, 297808052, 297808083, 297808580, 297808588, 297808596, 297808619, 297808627, 297808658, 297809407, 297810085, 297810093, 297810124, 297812719, 297812727, 297812756, 297812758, 297812764, 297812795, 297813292, 297813300, 297813308, 297813331, 297813339, 297813370, 297814797, 297814805, 297814836, 297819509, 297819517, 297819548, 297821944, 297821952, 297821960, 297821983, 297821991, 297822022, 297892836, 297892844, 297892852, 297892875, 297892883, 297892914, 298032629, 298115485, 298115493, 298115522, 298115524, 298115530, 298115561, 298116058, 298116066, 298116074, 298116097, 298116105, 298116136, 298117563, 298117571, 298117602, 298122275, 298122283, 298122314, 298242885, 298313777, 298334302, 298334310, 298334339, 298334341, 298334347, 298334378, 298336380, 298336388, 298336419, 298341092, 298341100, 298341131, 298349744, 298349752, 298349783, 298425041, 298425049, 298425080, 298553929, 298553937, 298553968, 298643858, 298643866, 298643897, 298813703, 298881117, 298881125, 298881154, 298881156, 298881162, 298881191, 298881193, 298881199, 298881230, 298883195, 298883203, 298883232, 298883234, 298883240, 298883271, 298884595, 298885273, 298885281, 298885312, 298887907, 298887915, 298887944, 298887946, 298887952, 298887983, 298889985, 298889993, 298890024, 298894697, 298894705, 298894736, 298896559, 298896567, 298896596, 298896598, 298896604, 298896635, 298897132, 298897140, 298897148, 298897171, 298897179, 298897210, 298898637, 298898645, 298898676, 298903349, 298903357, 298903388, 298968024, 298968032, 298968040, 298968063, 298968071, 298968102, 299190673, 299190681, 299190710, 299190712, 299190718, 299190749, 299192751, 299192759, 299192790, 299197463, 299197471, 299197502, 299206115, 299206123, 299206154, 299255204, 299255212, 299255220, 299255243, 299255251, 299255282, 299424932, 299424940, 299424971, 299500229, 299500237, 299500268, 299676145, 299783004, 299783012, 299783043, 299971747, 299971755, 299971784, 299971786, 299971792, 299971823, 299973825, 299973833, 299973864, 299978537, 299978545, 299978576, 299987189, 299987197, 299987228, 300244058, 300244066, 300244074, 300244095, 300244097, 300244103, 300244105, 300244111, 300244134, 300244136, 300244142, 300244173, 300246136, 300246144, 300246152, 300246175, 300246183, 300246214, 300246963, 300250848, 300250856, 300250864, 300250887, 300250895, 300250926, 300281303, 300281311, 300281342, 300329819, 300329827, 300329856, 300329858, 300329864, 300329895, 300330392, 300330400, 300330408, 300330431, 300330439, 300330470, 300331897, 300331905, 300331936, 300336609, 300336617, 300336648, 300553614, 300553622, 300553630, 300553653, 300553661, 300553692, 300639375, 300639383, 300639414, 300751333, 300771858, 300771866, 300771895, 300771897, 300771903, 300771934, 300773936, 300773944, 300773975, 300778648, 300778656, 300778687, 300858192, 300858200, 300858231, 301062377, 301062385, 301062416, 301081414, 301081422, 301081453, 301299658, 301299695, 301301736, 301306448, 301318673, 301318681, 301318710, 301318712, 301318718, 301318747, 301318749, 301318755, 301318786, 301320751, 301320759, 301320788, 301320790, 301320796, 301320827, 301322151, 301322829, 301322837, 301322868, 301325463, 301325471, 301325500, 301325502, 301325508, 301325539, 301327541, 301327549, 301327580, 301332253, 301332261, 301332292, 301334688, 301334696, 301334704, 301334727, 301334735, 301334766, 301342676, 301342684, 301342713, 301342715, 301342721, 301342752, 301344754, 301344762, 301344793, 301349466, 301349474, 301349505, 301405007, 301405015, 301405044, 301405046, 301405052, 301405083, 301405580, 301405588, 301405596, 301405619, 301405627, 301405658, 301407085, 301407093, 301407124, 301411797, 301411805, 301411836, 301420449, 301420457, 301420488, 301609214, 301628229, 301628237, 301628266, 301628268, 301628274, 301628305, 301630307, 301630315, 301630346, 301635019, 301635027, 301635058, 301652232, 301652240, 301652271, 301714563, 301714571, 301714602, 301846473, 301846510, 301846547, 301848551, 301848588, 301850629, 301853263, 301853300, 301855341, 301860053, 301862488, 301862496, 301862527, 301933380, 301933388, 301933419, 301937785, 301937793, 301937824, 302156029, 302156066, 302158107, 302162819, 302390288, 302409303, 302409311, 302409340, 302409342, 302409348, 302409379, 302411381, 302411389, 302411420, 302416093, 302416101, 302416132, 302433306, 302433314, 302433345, 302465585, 302480195, 302480203, 302480232, 302480234, 302480240, 302480271, 302482273, 302482281, 302482312, 302486985, 302486993, 302487024, 302495637, 302495645, 302495676, 302718859, 302718867, 302718898, 302767948, 302767956, 302767964, 302767987, 302767995, 302768026, 302789751, 302789759, 302789790, 302853709, 302853717, 302853748, 302937103, 302937140, 302939181, 302943893, 303188889, 303246659, 303295748, 303295756, 303295787, 303499933, 303499941, 303499972, 303570825, 303570833, 303570864, 303759707, 303823548, 303842563, 303842571, 303842600, 303842602, 303842608, 303842639, 303843136, 303843144, 303843152, 303843175, 303843183, 303843214, 303844641, 303844649, 303844680, 303849353, 303849361, 303849392, 303866566, 303866574, 303866605, 303928897, 303928905, 303928936, 304027733, 304152119, 304152127, 304152158, 304370363, 304370400, 304370936, 304370944, 304370975, 304372441, 304377153, 304679919, 304898736, 304917751, 304917759, 304917788, 304917790, 304917796, 304917827, 304919829, 304919837, 304919868, 304924541, 304924549, 304924580, 304933193, 304933201, 304933232, 304938276, 304938284, 304938292, 304938313, 304938315, 304938321, 304938323, 304938329, 304938350, 304938352, 304938354, 304938358, 304938360, 304938366, 304938389, 304938391, 304938397, 304938428, 304940354, 304940362, 304940370, 304940391, 304940393, 304940399, 304940401, 304940407, 304940430, 304940432, 304940438, 304940469, 304941754, 304941762, 304941793, 304942432, 304942440, 304942448, 304942471, 304942479, 304942510, 304945066, 304945074, 304945082, 304945103, 304945105, 304945111, 304945113, 304945119, 304945142, 304945144, 304945150, 304945181, 304947144, 304947152, 304947160, 304947183, 304947191, 304947222, 304951856, 304951864, 304951872, 304951895, 304951903, 304951934, 305004085, 305004093, 305004124, 305227307, 305227315, 305227346, 305247832, 305247840, 305247848, 305247869, 305247871, 305247877, 305247879, 305247885, 305247908, 305247910, 305247916, 305247947, 305249910, 305249918, 305249926, 305249949, 305249957, 305249988, 305254622, 305254630, 305254638, 305254661, 305254669, 305254700, 305445551, 305445588, 305447629, 305452341, 305460993, 305557388, 305557396, 305557404, 305557427, 305557435, 305557466, 305755107, 306008381, 306008389, 306008420, 306028906, 306028914, 306028922, 306028943, 306028945, 306028951, 306028953, 306028959, 306028982, 306028984, 306028990, 306029021, 306030984, 306030992, 306031000, 306031023, 306031031, 306031062, 306035696, 306035704, 306035712, 306035735, 306035743, 306035774, 306079273, 306079281, 306079312, 306280692, 306280700, 306280708, 306280731, 306280739, 306280770, 306338462, 306338470, 306338478, 306338501, 306338509, 306338540, 306366453, 306366461, 306366492, 306536181, 306808492, 306808500, 306808531, 306894253, 307119536, 307119544, 307119552, 307119575, 307119583, 307119614, 307336292, 307355307, 307355315, 307355344, 307355346, 307355352, 307355383, 307357385, 307357393, 307357424, 307362097, 307362105, 307362136, 307379310, 307379318, 307379349, 307441641, 307441649, 307441680, 307462166, 307462174, 307462182, 307462203, 307462205, 307462211, 307462213, 307462219, 307462242, 307462244, 307462250, 307462281, 307464244, 307464252, 307464260, 307464283, 307464291, 307464322, 307468956, 307468964, 307468972, 307468995, 307469003, 307469034, 307664863, 307664871, 307664902, 307771722, 307771730, 307771738, 307771761, 307771769, 307771800, 307883107, 307883144, 307885185, 307889897, 307907110, 307969441, 308192663, 308445937, 308445945, 308445976, 308453925, 308453962, 308456003, 308460715, 308516829, 308516837, 308516868, 308537354, 308537362, 308537370, 308537391, 308537393, 308537399, 308537401, 308537407, 308537430, 308537432, 308537438, 308537469, 308539432, 308539440, 308539448, 308539471, 308539479, 308539510, 308544144, 308544152, 308544160, 308544183, 308544191, 308544222, 308552796, 308552804, 308552812, 308552835, 308552843, 308552874, 308763481, 308846910, 308846918, 308846926, 308846949, 308846957, 308846988, 308973737, 309044629, 309544555, 309627984, 309627992, 309628000, 309628023, 309628031, 309628062, 309879197, 309879205, 309879236, 309986056, 309986064, 309986072, 309986095, 309986103, 309986134, 310406997, 310467324, 310467361, 310467398, 310467435, 310469402, 310469439, 310469476, 310471480, 310471517, 310473558, 310474114, 310474151, 310474188, 310476192, 310476229, 310478270, 310480904, 310480941, 310482982, 310487694, 310776880, 310776917, 310776954, 310778958, 310778995, 310781036, 310783670, 310783707, 310785748, 310790460, 310954385, 310954393, 310954424, 310974910, 310974918, 310974926, 310974947, 310974949, 310974955, 310974957, 310974963, 310974986, 310974988, 310974994, 310975025, 310976988, 310976996, 310977004, 310977027, 310977035, 310977066, 310977815, 310981700, 310981708, 310981716, 310981739, 310981747, 310981778, 311061244, 311061252, 311061260, 311061283, 311061291, 311061322, 311086436, 311086473, 311088514, 311093226, 311284466, 311284474, 311284482, 311284505, 311284513, 311284544, 311395992, 311482185, 311502710, 311502718, 311502747, 311502749, 311502755, 311502786, 311504788, 311504796, 311504827, 311509500, 311509508, 311509539, 311557954, 311557991, 311558028, 311560032, 311560069, 311562110, 311564744, 311564781, 311566822, 311571534, 311812266, 311812274, 311812305, 311867510, 311867547, 311869588, 311874300, 312049525, 312049533, 312049562, 312049564, 312049570, 312049599, 312049601, 312049607, 312049638, 312051603, 312051611, 312051640, 312051642, 312051648, 312051679, 312053003, 312053681, 312053689, 312053720, 312056315, 312056323, 312056352, 312056354, 312056360, 312056391, 312058393, 312058401, 312058432, 312063105, 312063113, 312063144, 312065540, 312065548, 312065556, 312065579, 312065587, 312065618, 312136432, 312136440, 312136448, 312136471, 312136479, 312136510, 312177066, 312359081, 312359089, 312359118, 312359120, 312359126, 312359157, 312361159, 312361167, 312361198, 312365871, 312365879, 312365910, 312593340, 312593348, 312593379, 312648584, 312648621, 312650662, 312655374, 312668637, 312668645, 312668676, 312958140, 312991214, 312991251, 312991288, 312993292, 312993329, 312995370, 312998004, 312998041, 313000082, 313004794, 313140155, 313140163, 313140192, 313140194, 313140200, 313140231, 313142233, 313142241, 313142272, 313146945, 313146953, 313146984, 313300770, 313300807, 313302848, 313307560, 313391941, 313391949, 313391980, 313449711, 313449719, 313449750, 313498800, 313498808, 313498816, 313498839, 313498847, 313498878, 313610326, 313739214, 313919741, 314026600, 314026608, 314026639, 314066402, 314066439, 314066476, 314068480, 314068517, 314070558, 314073192, 314073229, 314075270, 314079982, 314081844, 314081881, 314083922, 314088634, 314230785, 314230793, 314230824, 314375958, 314375995, 314378036, 314382748, 314391400, 314490559, 314573415, 314573423, 314573452, 314573454, 314573460, 314573491, 314573988, 314573996, 314574004, 314574027, 314574035, 314574066, 314575493, 314575501, 314575532, 314580205, 314580213, 314580244, 314685514, 314882971, 314882979, 314883010, 315101788, 315101796, 315101827, 315157032, 315157069, 315159110, 315163822, 315172474, 315466588, 315515104, 315515141, 315517182, 315521894, 315648603, 315648611, 315648640, 315648642, 315648648, 315648679, 315650681, 315650689, 315650720, 315655393, 315655401, 315655432, 315664045, 315664053, 315664084, 315824660, 315958159, 315958167, 315958198, 316247662, 316503958, 316503995, 316504032, 316506036, 316506073, 316508114, 316510748, 316510785, 316512826, 316517538, 316590292, 316590329, 316592370, 316597082, 316605734, 316739233, 316739241, 316739272, 316813514, 316813551, 316815592, 316820304, 316899848, 317011544, 317011552, 317011560, 317011583, 317011591, 317011622, 317097305, 317097313, 317097344, 317123070, 317539344, 317539352, 317539383, 317594588, 317594625, 317596666, 317601378, 317665480, 317665517, 317667558, 317672270, 317680922, 317904144, 317975036, 318038994, 318067144, 318086159, 318086167, 318086196, 318086198, 318086204, 318086235, 318088237, 318088245, 318088276, 318092949, 318092957, 318092988, 318110162, 318110170, 318110201, 318172493, 318172501, 318172532, 318395715, 318395723, 318395754, 318613959, 318613996, 318616037, 318620749, 318685218, 318756110, 318923515, 319027848, 319027885, 319029926, 319034638, 319114182, 319160774, 319160811, 319160848, 319162852, 319162889, 319164930, 319167564, 319167601, 319169642, 319174354, 319176789, 319176797, 319176828, 319247681, 319247689, 319247720, 319337404, 319470330, 319470367, 319472408, 319477120, 319704589, 319779886, 320103036, 320103073, 320105114, 320109826, 320118478, 320189370, 320251404, 320251441, 320253482, 320258194, 320412592, 320560960, 320610049, 320610057, 320610088, 320627358, 320627395, 320627432, 320627469, 320629436, 320629473, 320629510, 320631514, 320631551, 320633592, 320634148, 320634185, 320634222, 320636226, 320636263, 320638304, 320640938, 320640975, 320643016, 320647728, 320936914, 320936951, 320936988, 320938992, 320939029, 320941070, 320943704, 320943741, 320945782, 320950494, 321137849, 321193666, 321246470, 321246507, 321248548, 321253260, 321264558, 321342034, 321551738, 321556026, 321684664, 321684701, 321685237, 321685245, 321685276, 321686742, 321691454, 321705762, 321705770, 321705778, 321705799, 321705801, 321705807, 321705809, 321705815, 321705838, 321705840, 321705846, 321705877, 321707840, 321707848, 321707856, 321707879, 321707887, 321707918, 321712552, 321712560, 321712568, 321712591, 321712599, 321712630, 321717988, 321718025, 321718062, 321720066, 321720103, 321722144, 321724778, 321724815, 321726856, 321731568, 321994220, 322015318, 322015326, 322015334, 322015357, 322015365, 322015396, 322027544, 322027581, 322029622, 322034334, 322213037, 322337100, 322540592, 322540629, 322542670, 322547382, 322626926, 322759852, 322759889, 322761930, 322766642, 322775294, 322796392, 322796400, 322796408, 322796431, 322796439, 322796470, 322808618, 322808655, 322810696, 322815408, 322850148, 323069408, 323118174, 323151248, 323151285, 323151322, 323153326, 323153363, 323155404, 323158038, 323158075, 323160116, 323164828, 323460804, 323460841, 323462882, 323467594, 323631222, 323702114, 323770360, 323850482, 323899248, 324122793, 324122801, 324122832, 324208554, 324226436, 324226473, 324226510, 324228514, 324228551, 324229652, 324229660, 324229668, 324229691, 324229699, 324229730, 324230592, 324233226, 324233263, 324235304, 324240016, 324241878, 324241915, 324243956, 324248668, 324535992, 324536029, 324538070, 324542782, 324551434, 324650593, 324845548, 325064482, 325197408, 325197445, 325199486, 325204198, 325221411, 325283742, 325304840, 325304848, 325304856, 325304879, 325304887, 325304918, 325317066, 325317103, 325319144, 325323856, 325332508, 325506964, 325626622, 325675138, 325675175, 325677216, 325681928, 325984694, 326139670, 326288038, 326358930, 326407696, 326663992, 326664029, 326664066, 326666070, 326666107, 326668148, 326670782, 326670819, 326672860, 326677572, 326750326, 326750363, 326752404, 326757116, 326765768, 326973548, 326973585, 326975626, 326980338, 327059882, 327234810, 327234847, 327234884, 327236888, 327236925, 327238966, 327241600, 327241637, 327243678, 327248390, 327283104, 327544366, 327544403, 327546444, 327551156, 327721298, 327742396, 327742404, 327742412, 327742435, 327742443, 327742474, 327754622, 327754659, 327756700, 327761412, 327825514, 327825551, 327827592, 327832304, 327840956, 327853922, 328064178, 328135070, 328199028, 328270196, 328270204, 328270235, 328325440, 328325477, 328327518, 328332230, 328577226, 328634996, 328796486, 328817011, 328817019, 328817048, 328817050, 328817056, 328817087, 328819089, 328819097, 328819128, 328823801, 328823809, 328823840, 328845252, 328916144, 329126567, 329126575, 329126606, 329187882, 329187919, 329189960, 329194672, 329274216, 329416070, 329497438, 329758700, 329758737, 329760778, 329765490, 329907641, 329907649, 329907680, 330068256, 330263070, 330263107, 330265148, 330269860, 330278512, 330349404, 330572626, 330787392, 330787429, 330787466, 330787503, 330789470, 330789507, 330789544, 330791548, 330791585, 330793626, 330794182, 330794219, 330794256, 330796260, 330796297, 330798338, 330800972, 330801009, 330803050, 330807762, 330830410, 330830418, 330830447, 330830449, 330830455, 330830484, 330830486, 330830492, 330830521, 330830523, 330830529, 330830560, 330832488, 330832496, 330832525, 330832527, 330832533, 330832562, 330832564, 330832570, 330832601, 330833888, 330833925, 330834566, 330834574, 330834603, 330834605, 330834611, 330834642, 330835966, 330836644, 330836652, 330836683, 330837200, 330837208, 330837237, 330837239, 330837245, 330837274, 330837276, 330837282, 330837313, 330839278, 330839286, 330839315, 330839317, 330839323, 330839354, 330840678, 330841356, 330841364, 330841395, 330843990, 330843998, 330844027, 330844029, 330844035, 330844066, 330846068, 330846076, 330846107, 330849330, 330850780, 330850788, 330850819, 331096948, 331096985, 331097022, 331099026, 331099063, 331101104, 331103738, 331103775, 331105816, 331110528, 331139966, 331139974, 331140003, 331140005, 331140011, 331140040, 331140042, 331140048, 331140079, 331142044, 331142052, 331142081, 331142083, 331142089, 331142120, 331143444, 331144122, 331144130, 331144161, 331146756, 331146764, 331146793, 331146795, 331146801, 331146832, 331148834, 331148842, 331148873, 331153546, 331153554, 331153585, 331234042, 331340901, 331340909, 331340940, 331353700, 331406504, 331406541, 331408582, 331413294, 331424592, 331449522, 331449530, 331449559, 331449561, 331449567, 331449598, 331451600, 331451608, 331451639, 331456312, 331456320, 331456351, 331711772, 331716060, 331759078, 331759086, 331759117, 331878022, 331878059, 331878096, 331880100, 331880137, 331882178, 331884812, 331884849, 331886890, 331891602, 331921040, 331921048, 331921077, 331921079, 331921085, 331921114, 331921116, 331921122, 331921153, 331923118, 331923126, 331923155, 331923157, 331923163, 331923194, 331924518, 331925196, 331925204, 331925235, 331927830, 331927838, 331927867, 331927869, 331927875, 331927906, 331929908, 331929916, 331929947, 331934620, 331934628, 331934659, 332187578, 332187615, 332189656, 332194368, 332230596, 332230604, 332230633, 332230635, 332230641, 332230672, 332232674, 332232682, 332232713, 332237386, 332237394, 332237425, 332282590, 332416089, 332416097, 332416128, 332497134, 332540152, 332540160, 332540191, 332700626, 332700663, 332702704, 332707416, 332786960, 332968652, 332968689, 332970730, 332975442, 333010182, 333011670, 333011678, 333011707, 333011709, 333011715, 333011746, 333013748, 333013756, 333013787, 333018460, 333018468, 333018499, 333271444, 333271481, 333273522, 333278208, 333278234, 333311282, 333311319, 333311356, 333313360, 333313397, 333315438, 333318072, 333318109, 333320150, 333321226, 333321234, 333321265, 333324862, 333354300, 333354308, 333354337, 333354339, 333354345, 333354374, 333354376, 333354382, 333354413, 333356378, 333356386, 333356415, 333356417, 333356423, 333356454, 333357778, 333358456, 333358464, 333358495, 333361090, 333361098, 333361127, 333361129, 333361135, 333361166, 333363168, 333363176, 333363207, 333367880, 333367888, 333367919, 333581000, 333620838, 333620875, 333622916, 333627628, 333663856, 333663864, 333663893, 333663895, 333663901, 333663932, 333665934, 333665942, 333665973, 333670646, 333670654, 333670685, 333791256, 333862148, 333930394, 333973412, 333973420, 333973451, 334059282, 334102300, 334102308, 334102339, 334362074, 334386470, 334386507, 334386544, 334388548, 334388585, 334390626, 334393260, 334393297, 334395338, 334400050, 334401912, 334401949, 334403990, 334408702, 334429488, 334429496, 334429525, 334429527, 334429533, 334429562, 334429564, 334429570, 334429601, 334431566, 334431574, 334431603, 334431605, 334431611, 334431642, 334432966, 334433644, 334433652, 334433683, 334436278, 334436286, 334436315, 334436317, 334436323, 334436354, 334438356, 334438364, 334438395, 334443068, 334443076, 334443107, 334444930, 334444938, 334444967, 334444969, 334444975, 334445006, 334447008, 334447016, 334447047, 334451720, 334451728, 334451759, 334696026, 334696063, 334698104, 334702816, 334711468, 334739044, 334739052, 334739081, 334739083, 334739089, 334739120, 334741122, 334741130, 334741161, 334745834, 334745842, 334745873, 334754486, 334754494, 334754525, 334853645, 334853653, 334853684, 335005582, 335048600, 335048608, 335048639, 335224516, 335381445, 335477100, 335477137, 335479178, 335483890, 335492542, 335520118, 335520126, 335520155, 335520157, 335520163, 335520194, 335522196, 335522204, 335522235, 335526908, 335526916, 335526947, 335535560, 335535568, 335535599, 335786656, 335795334, 335829674, 335829682, 335829713, 335835172, 335835209, 335837250, 335841962, 335878190, 335878198, 335878227, 335878229, 335878235, 335878266, 335880268, 335880276, 335880307, 335884980, 335884988, 335885019, 335928260, 335928297, 335930338, 335935050, 336144728, 336187746, 336187754, 336187785, 336237816, 336299704, 336567730, 336610748, 336610756, 336610787, 336824026, 336824063, 336824100, 336826104, 336826141, 336828182, 336830816, 336830853, 336832894, 336837606, 336867044, 336867052, 336867081, 336867083, 336867089, 336867118, 336867120, 336867126, 336867157, 336869122, 336869130, 336869159, 336869161, 336869167, 336869198, 336870522, 336871200, 336871208, 336871239, 336873834, 336873842, 336873871, 336873873, 336873879, 336873910, 336875912, 336875920, 336875951, 336880624, 336880632, 336880663, 336910360, 336910397, 336912438, 336917150, 336925802, 336953378, 336953386, 336953415, 336953417, 336953423, 336953454, 336955456, 336955464, 336955495, 336960168, 336960176, 336960207, 336968820, 336968828, 336968859, 337018890, 337133582, 337133619, 337135660, 337140372, 337176600, 337176608, 337176637, 337176639, 337176645, 337176676, 337178678, 337178686, 337178717, 337183390, 337183398, 337183429, 337219916, 337262934, 337262942, 337262973, 337394844, 337394881, 337394918, 337396922, 337396959, 337399000, 337401634, 337401671, 337403712, 337408424, 337443138, 337486156, 337486164, 337486195, 337704400, 337704437, 337706478, 337711190, 337914656, 337914693, 337916734, 337921446, 337957674, 337957682, 337957711, 337957713, 337957719, 337957750, 337959752, 337959760, 337959791, 337964464, 337964472, 337964503, 337985548, 337985585, 337987626, 337992338, 338000990, 338013956, 338028566, 338028574, 338028603, 338028605, 338028611, 338028642, 338030644, 338030652, 338030683, 338035356, 338035364, 338035395, 338044008, 338044016, 338044047, 338224212, 338267230, 338267238, 338267269, 338295104, 338338122, 338338130, 338338161, 338359062, 338402080, 338402088, 338402119, 338452150, 338473248, 338473256, 338473264, 338473287, 338473295, 338473326, 338485474, 338485511, 338487552, 338492264, 338737260, 338795030, 339005286, 339048304, 339048312, 339048343, 339076178, 339119196, 339119204, 339119235, 339308078, 339347916, 339347953, 339349994, 339354706, 339390934, 339390942, 339390971, 339390973, 339390979, 339391010, 339393012, 339393020, 339393051, 339397724, 339397732, 339397763, 339434250, 339477268, 339477276, 339477307, 339527338, 339576104, 339657472, 339700490, 339700498, 339700529, 339918734, 339918771, 339920812, 339925524, 340228290, 340423104, 340423141, 340425182, 340429894, 340438546, 340466122, 340466130, 340466159, 340466161, 340466167, 340466198, 340468200, 340468208, 340468239, 340472912, 340472920, 340472951, 340481564, 340481572, 340481603, 340509438, 340552456, 340552464, 340552495, 340732660, 340775678, 340775686, 340775717, 340990444, 340990452, 340990481, 340990483, 340990489, 340990518, 340990520, 340990526, 340990555, 340990557, 340990563, 340990594, 340992522, 340992530, 340992559, 340992561, 340992567, 340992596, 340992598, 340992604, 340992635, 340993922, 340993959, 340994600, 340994608, 340994637, 340994639, 340994645, 340994676, 340996000, 340996678, 340996686, 340996717, 340997234, 340997242, 340997271, 340997273, 340997279, 340997308, 340997310, 340997316, 340997347, 340999312, 340999320, 340999349, 340999351, 340999357, 340999388, 341000712, 341001390, 341001398, 341001429, 341004024, 341004032, 341004061, 341004063, 341004069, 341004100, 341006102, 341006110, 341006141, 341009364, 341010814, 341010822, 341010853, 341300000, 341300008, 341300037, 341300039, 341300045, 341300074, 341300076, 341300082, 341300113, 341302078, 341302086, 341302115, 341302117, 341302123, 341302154, 341303478, 341304156, 341304164, 341304195, 341306790, 341306798, 341306827, 341306829, 341306835, 341306866, 341308868, 341308876, 341308907, 341313580, 341313588, 341313619, 341513734, 341556752, 341556760, 341556791, 341584626, 341609556, 341609564, 341609593, 341609595, 341609601, 341609632, 341611634, 341611642, 341611673, 341616346, 341616354, 341616385, 341627644, 341627652, 341627683, 341871806, 341914824, 341914832, 341914863, 341919112, 341919120, 341919151, 341964894, 342081074, 342081082, 342081111, 342081113, 342081119, 342081148, 342081150, 342081156, 342081187, 342083152, 342083160, 342083189, 342083191, 342083197, 342083228, 342084552, 342085230, 342085238, 342085269, 342087864, 342087872, 342087901, 342087903, 342087909, 342087940, 342089942, 342089950, 342089981, 342094654, 342094662, 342094693, 342390630, 342390638, 342390667, 342390669, 342390675, 342390706, 342392708, 342392716, 342392747, 342397420, 342397428, 342397459, 342442624, 342700186, 342700194, 342700225, 342860660, 342860697, 342862738, 342867450, 342903678, 342903686, 342903715, 342903717, 342903723, 342903754, 342905756, 342905764, 342905795, 342910468, 342910476, 342910507, 342946994, 342990012, 342990020, 342990051, 343170216, 343171704, 343171712, 343171741, 343171743, 343171749, 343171780, 343173782, 343173790, 343173821, 343178494, 343178502, 343178533, 343213234, 343213242, 343213273, 343431478, 343431515, 343433556, 343438268, 343481260, 343481268, 343481299, 343514334, 343514342, 343514371, 343514373, 343514379, 343514408, 343514410, 343514416, 343514447, 343516412, 343516420, 343516449, 343516451, 343516457, 343516488, 343517812, 343518490, 343518498, 343518529, 343521124, 343521132, 343521161, 343521163, 343521169, 343521200, 343523202, 343523210, 343523241, 343527914, 343527922, 343527953, 343741034, 343823890, 343823898, 343823927, 343823929, 343823935, 343823966, 343825968, 343825976, 343826007, 343830680, 343830688, 343830719, 343951290, 343994308, 343994316, 343994347, 344002296, 344002333, 344004374, 344009086, 344022182, 344065200, 344065208, 344065239, 344133446, 344133454, 344133485, 344262334, 344262342, 344262373, 344311852, 344522108, 344589522, 344589530, 344589559, 344589561, 344589567, 344589596, 344589598, 344589604, 344589635, 344591600, 344591608, 344591637, 344591639, 344591645, 344591676, 344593000, 344593678, 344593686, 344593717, 344596312, 344596320, 344596349, 344596351, 344596357, 344596388, 344598390, 344598398, 344598429, 344603102, 344603110, 344603141, 344604964, 344604972, 344605001, 344605003, 344605009, 344605040, 344607042, 344607050, 344607081, 344611754, 344611762, 344611793, 344899078, 344899086, 344899115, 344899117, 344899123, 344899154, 344901156, 344901164, 344901195, 344905868, 344905876, 344905907, 344914520, 344914528, 344914559, 345092926, 345208634, 345208642, 345208673, 345384550, 345427568, 345427576, 345427607, 345584497, 345584505, 345584536, 345680152, 345680160, 345680189, 345680191, 345680197, 345680228, 345682230, 345682238, 345682269, 345686942, 345686950, 345686981, 345695594, 345695602, 345695633, 345955368, 345989708, 345989716, 345989747, 346015695, 346015732, 346015769, 346015806, 346017773, 346017810, 346017847, 346019851, 346019888, 346021929, 346022485, 346022522, 346022559, 346024563, 346024600, 346026641, 346029275, 346029312, 346031353, 346036065, 346038224, 346038232, 346038261, 346038263, 346038269, 346038300, 346040302, 346040310, 346040341, 346045014, 346045022, 346045053, 346325251, 346325288, 346325325, 346327329, 346327366, 346329407, 346332041, 346332078, 346334119, 346338831, 346347780, 346347788, 346347819, 346459738, 346502756, 346502764, 346502795, 346526186, 346634807, 346634844, 346636885, 346641597, 346770782, 346770790, 346770821, 346944363, 347027078, 347027086, 347027115, 347027117, 347027123, 347027152, 347027154, 347027160, 347027191, 347029156, 347029164, 347029193, 347029195, 347029201, 347029232, 347030556, 347031234, 347031242, 347031273, 347033868, 347033876, 347033905, 347033907, 347033913, 347033944, 347035946, 347035954, 347035985, 347040658, 347040666, 347040697, 347106325, 347106362, 347106399, 347108403, 347108440, 347110481, 347113115, 347113152, 347113412, 347113420, 347113449, 347113451, 347113457, 347113488, 347115193, 347115490, 347115498, 347115529, 347119905, 347120202, 347120210, 347120241, 347128854, 347128862, 347128893, 347336634, 347336642, 347336671, 347336673, 347336679, 347336710, 347338712, 347338720, 347338751, 347343424, 347343432, 347343463, 347415881, 347415918, 347417959, 347422671, 347422968, 347422976, 347423007, 347554878, 347554915, 347554952, 347556956, 347556993, 347559034, 347561668, 347561705, 347563746, 347568458, 347597896, 347597904, 347597933, 347597935, 347597941, 347597970, 347597972, 347597978, 347598009, 347599974, 347599982, 347600011, 347600013, 347600019, 347600050, 347601374, 347602052, 347602060, 347602091, 347604686, 347604694, 347604723, 347604725, 347604731, 347604762, 347606764, 347606772, 347606803, 347611476, 347611484, 347611515, 347646190, 347646198, 347646229, 347725437, 347864434, 347864471, 347866512, 347871224, 347907452, 347907460, 347907489, 347907491, 347907497, 347907528, 347909530, 347909538, 347909569, 347914242, 347914250, 347914281, 348117708, 348117716, 348117745, 348117747, 348117753, 348117784, 348119786, 348119794, 348119825, 348124498, 348124506, 348124537, 348173990, 348188600, 348188608, 348188637, 348188639, 348188645, 348188676, 348190678, 348190686, 348190717, 348195390, 348195398, 348195429, 348196955, 348196992, 348199033, 348203745, 348204042, 348204050, 348204081, 348217008, 348217016, 348217047, 348427264, 348427272, 348427303, 348498156, 348498164, 348498195, 348506511, 348539585, 348539622, 348539659, 348541663, 348541700, 348543741, 348546375, 348546412, 348548453, 348553165, 348562114, 348562122, 348562153, 348645508, 348645545, 348647586, 348652298, 348688526, 348688534, 348688563, 348688565, 348688571, 348688602, 348690604, 348690612, 348690643, 348695316, 348695324, 348695355, 348849141, 348849178, 348851219, 348855931, 348897294, 348940312, 348940320, 348940351, 348955064, 348998082, 348998090, 348998121, 349158697, 349208338, 349208346, 349208377, 349279230, 349279238, 349279269, 349287585, 349468112, 349550968, 349550976, 349551005, 349551007, 349551013, 349551044, 349553046, 349553054, 349553085, 349557758, 349557766, 349557797, 349614773, 349614810, 349614847, 349616851, 349616888, 349618929, 349621563, 349621600, 349623641, 349628353, 349630215, 349630252, 349632293, 349637005, 349637302, 349637310, 349637341, 349736138, 349779156, 349779164, 349779195, 349860524, 349860532, 349860563, 349924329, 349924366, 349926407, 349931119, 349939771, 350038930, 350078768, 350078805, 350080846, 350085558, 350121786, 350121794, 350121823, 350121825, 350121831, 350121862, 350123864, 350123872, 350123903, 350128576, 350128584, 350128615, 350233885, 350388324, 350431342, 350431350, 350431381, 350626156, 350626164, 350626193, 350626195, 350626201, 350626232, 350628234, 350628242, 350628273, 350632946, 350632954, 350632985, 350641598, 350641606, 350641637, 350705403, 350705440, 350707481, 350712193, 350712490, 350712498, 350712529, 350720845, 350935712, 350935720, 350935751, 351014959, 351063475, 351063512, 351065553, 351070265, 351153956, 351153993, 351156034, 351160746, 351169398, 351193496, 351193504, 351193512, 351193533, 351193535, 351193541, 351193543, 351193549, 351193570, 351193572, 351193574, 351193578, 351193580, 351193586, 351193607, 351193609, 351193611, 351193615, 351193617, 351193623, 351193646, 351193648, 351193654, 351193685, 351195574, 351195582, 351195590, 351195611, 351195613, 351195619, 351195621, 351195627, 351195648, 351195650, 351195652, 351195656, 351195658, 351195664, 351195687, 351195689, 351195695, 351195726, 351196974, 351196982, 351197011, 351197013, 351197019, 351197050, 351197652, 351197660, 351197668, 351197689, 351197691, 351197697, 351197699, 351197705, 351197728, 351197730, 351197736, 351197767, 351199052, 351199060, 351199091, 351199730, 351199738, 351199746, 351199769, 351199777, 351199808, 351200286, 351200294, 351200302, 351200323, 351200325, 351200331, 351200333, 351200339, 351200360, 351200362, 351200364, 351200368, 351200370, 351200376, 351200399, 351200401, 351200407, 351200438, 351202364, 351202372, 351202380, 351202401, 351202403, 351202409, 351202411, 351202417, 351202440, 351202442, 351202448, 351202479, 351203764, 351203772, 351203803, 351204442, 351204450, 351204458, 351204481, 351204489, 351204520, 351207076, 351207084, 351207092, 351207113, 351207115, 351207121, 351207123, 351207129, 351207152, 351207154, 351207160, 351207191, 351209154, 351209162, 351209170, 351209193, 351209201, 351209232, 351212416, 351212424, 351212455, 351213866, 351213874, 351213882, 351213905, 351213913, 351213944, 351373031, 351463512, 351503052, 351503060, 351503068, 351503089, 351503091, 351503097, 351503099, 351503105, 351503126, 351503128, 351503130, 351503134, 351503136, 351503142, 351503165, 351503167, 351503173, 351503204, 351505130, 351505138, 351505146, 351505167, 351505169, 351505175, 351505177, 351505183, 351505206, 351505208, 351505214, 351505245, 351506530, 351506538, 351506569, 351507208, 351507216, 351507224, 351507247, 351507255, 351507286, 351509842, 351509850, 351509858, 351509879, 351509881, 351509887, 351509889, 351509895, 351509918, 351509920, 351509926, 351509957, 351511920, 351511928, 351511936, 351511959, 351511967, 351511998, 351516632, 351516640, 351516648, 351516671, 351516679, 351516710, 351716786, 351716794, 351716825, 351787678, 351787686, 351787717, 351796033, 351812608, 351812616, 351812624, 351812645, 351812647, 351812653, 351812655, 351812661, 351812684, 351812686, 351812692, 351812723, 351814686, 351814694, 351814702, 351814725, 351814733, 351814764, 351819398, 351819406, 351819414, 351819437, 351819445, 351819476, 352052329, 352052366, 352052403, 352054407, 352054444, 352056485, 352059119, 352059156, 352061197, 352065909, 352074858, 352074866, 352074897, 352122164, 352122172, 352122180, 352122203, 352122211, 352122242, 352138663, 352138700, 352140741, 352145453, 352154105, 352244586, 352284126, 352284134, 352284142, 352284163, 352284165, 352284171, 352284173, 352284179, 352284200, 352284202, 352284204, 352284208, 352284210, 352284216, 352284239, 352284241, 352284247, 352284278, 352286204, 352286212, 352286220, 352286241, 352286243, 352286249, 352286251, 352286257, 352286280, 352286282, 352286288, 352286319, 352287604, 352287612, 352287643, 352288282, 352288290, 352288298, 352288321, 352288329, 352288360, 352290916, 352290924, 352290932, 352290953, 352290955, 352290961, 352290963, 352290969, 352290992, 352290994, 352291000, 352291031, 352292994, 352293002, 352293010, 352293033, 352293041, 352293072, 352297706, 352297714, 352297722, 352297745, 352297753, 352297784, 352361885, 352361922, 352363963, 352368675, 352448219, 352593682, 352593690, 352593698, 352593719, 352593721, 352593727, 352593729, 352593735, 352593758, 352593760, 352593766, 352593797, 352595760, 352595768, 352595776, 352595799, 352595807, 352595838, 352600472, 352600480, 352600488, 352600511, 352600519, 352600550, 352602658, 352645676, 352645684, 352645715, 352671441, 352695746, 352903238, 352903246, 352903254, 352903277, 352903285, 352903316, 353063712, 353063720, 353063749, 353063751, 353063757, 353063788, 353065790, 353065798, 353065829, 353070502, 353070510, 353070541, 353142959, 353142996, 353145037, 353149749, 353150046, 353150054, 353150085, 353213851, 353213888, 353215929, 353220641, 353229293, 353373268, 353373276, 353373307, 353374756, 353374764, 353374772, 353374793, 353374795, 353374801, 353374803, 353374809, 353374832, 353374834, 353374840, 353374871, 353376834, 353376842, 353376850, 353376873, 353376881, 353376912, 353381546, 353381554, 353381562, 353381585, 353381593, 353381624, 353452515, 353523407, 353587365, 353591512, 353591549, 353593590, 353598302, 353634530, 353634538, 353634567, 353634569, 353634575, 353634606, 353636608, 353636616, 353636647, 353641320, 353641328, 353641359, 353677846, 353684312, 353684320, 353684328, 353684351, 353684359, 353684390, 353717386, 353717394, 353717402, 353717423, 353717425, 353717431, 353717433, 353717439, 353717460, 353717462, 353717464, 353717468, 353717470, 353717476, 353717499, 353717501, 353717507, 353717538, 353719464, 353719472, 353719480, 353719501, 353719503, 353719509, 353719511, 353719517, 353719540, 353719542, 353719548, 353719579, 353720864, 353720872, 353720903, 353721542, 353721550, 353721558, 353721581, 353721589, 353721620, 353724176, 353724184, 353724192, 353724213, 353724215, 353724221, 353724223, 353724229, 353724252, 353724254, 353724260, 353724291, 353726254, 353726262, 353726270, 353726293, 353726301, 353726332, 353730966, 353730974, 353730982, 353731005, 353731013, 353731044, 353901068, 353944086, 353944094, 353944125, 354026942, 354026950, 354026958, 354026979, 354026981, 354026987, 354026989, 354026995, 354027018, 354027020, 354027026, 354027057, 354029020, 354029028, 354029036, 354029059, 354029067, 354029098, 354033732, 354033740, 354033748, 354033771, 354033779, 354033810, 354154342, 354154350, 354154381, 354162330, 354162367, 354164408, 354169120, 354225234, 354225242, 354225273, 354233589, 354304481, 354336498, 354336506, 354336514, 354336537, 354336545, 354336576, 354465386, 354465394, 354465402, 354465425, 354465433, 354465464, 354471886, 354576219, 354576256, 354578297, 354583009, 354662553, 354682142, 354709145, 354709182, 354709219, 354711223, 354711260, 354713301, 354715935, 354715972, 354718013, 354722725, 354725160, 354725168, 354725199, 354753034, 354792574, 354792582, 354792590, 354792611, 354792613, 354792619, 354792621, 354792627, 354792648, 354792650, 354792652, 354792656, 354792658, 354792664, 354792687, 354792689, 354792695, 354792726, 354794652, 354794660, 354794668, 354794689, 354794691, 354794697, 354794699, 354794705, 354794728, 354794730, 354794736, 354794767, 354796052, 354796060, 354796091, 354796730, 354796738, 354796746, 354796769, 354796777, 354796808, 354799364, 354799372, 354799380, 354799401, 354799403, 354799409, 354799411, 354799417, 354799440, 354799442, 354799448, 354799479, 354801442, 354801450, 354801458, 354801481, 354801489, 354801520, 354806154, 354806162, 354806170, 354806193, 354806201, 354806232, 354808016, 354808024, 354808032, 354808053, 354808055, 354808061, 354808063, 354808069, 354808092, 354808094, 354808100, 354808131, 354810094, 354810102, 354810110, 354810133, 354810141, 354810172, 354814806, 354814814, 354814822, 354814845, 354814853, 354814884, 354885775, 355018701, 355018738, 355020779, 355025491, 355102130, 355102138, 355102146, 355102167, 355102169, 355102175, 355102177, 355102183, 355102206, 355102208, 355102214, 355102245, 355104208, 355104216, 355104224, 355104247, 355104255, 355104286, 355108920, 355108928, 355108936, 355108959, 355108967, 355108998, 355117572, 355117580, 355117588, 355117611, 355117619, 355117650, 355252960, 355328257, 355411686, 355411694, 355411702, 355411725, 355411733, 355411764, 355587602, 355587610, 355587641, 355651407, 355651444, 355653485, 355658197, 355666849, 355737741, 355799775, 355799812, 355801853, 355806565, 355883204, 355883212, 355883220, 355883241, 355883243, 355883249, 355883251, 355883257, 355883280, 355883282, 355883288, 355883319, 355885282, 355885290, 355885298, 355885321, 355885329, 355885360, 355889994, 355890002, 355890010, 355890033, 355890041, 355890072, 355898646, 355898654, 355898662, 355898685, 355898693, 355898724, 355960963, 356109331, 356115402, 356158420, 356158428, 356158459, 356175729, 356175766, 356175803, 356175840, 356177807, 356177844, 356177881, 356179885, 356179922, 356181963, 356182519, 356182556, 356182593, 356184597, 356184634, 356186675, 356189309, 356189346, 356191387, 356192760, 356192768, 356192776, 356192799, 356192807, 356192838, 356196099, 356241276, 356241284, 356241292, 356241313, 356241315, 356241321, 356241323, 356241329, 356241352, 356241354, 356241360, 356241391, 356243354, 356243362, 356243370, 356243393, 356243401, 356243432, 356248066, 356248074, 356248082, 356248105, 356248113, 356248144, 356485285, 356485322, 356485359, 356487363, 356487400, 356489441, 356492075, 356492112, 356494153, 356498865, 356550832, 356550840, 356550848, 356550871, 356550879, 356550910, 356662790, 356662798, 356662829, 356686220, 356742037, 356794841, 356794878, 356796919, 356801631, 356812929, 356890405, 356973834, 356973842, 356973850, 356973873, 356973881, 356973912, 357100109, 357104397, 357190590, 357230130, 357230138, 357230146, 357230167, 357230169, 357230175, 357230177, 357230183, 357230204, 357230206, 357230208, 357230212, 357230214, 357230220, 357230243, 357230245, 357230251, 357230282, 357232208, 357232216, 357232224, 357232245, 357232247, 357232253, 357232255, 357232261, 357232284, 357232286, 357232292, 357232323, 357233035, 357233072, 357233608, 357233616, 357233647, 357234286, 357234294, 357234302, 357234325, 357234333, 357234364, 357235113, 357236920, 357236928, 357236936, 357236957, 357236959, 357236965, 357236967, 357236973, 357236996, 357236998, 357237004, 357237035, 357238998, 357239006, 357239014, 357239037, 357239045, 357239076, 357239825, 357243710, 357243718, 357243726, 357243749, 357243757, 357243788, 357266359, 357266396, 357266433, 357268437, 357268474, 357270515, 357273149, 357273186, 357275227, 357279939, 357316464, 357316472, 357316480, 357316501, 357316503, 357316509, 357316511, 357316517, 357316540, 357316542, 357316548, 357316579, 357318542, 357318550, 357318558, 357318581, 357318589, 357318620, 357323254, 357323262, 357323270, 357323293, 357323301, 357323332, 357331906, 357331914, 357331922, 357331945, 357331953, 357331984, 357539686, 357539694, 357539702, 357539723, 357539725, 357539731, 357539733, 357539739, 357539762, 357539764, 357539770, 357539801, 357541764, 357541772, 357541780, 357541803, 357541811, 357541842, 357542591, 357546476, 357546484, 357546492, 357546515, 357546523, 357546554, 357575915, 357575952, 357577993, 357582705, 357626020, 357626028, 357626036, 357626059, 357626067, 357626098, 357757930, 357757938, 357757967, 357757969, 357757975, 357758004, 357758006, 357758012, 357758043, 357760008, 357760016, 357760045, 357760047, 357760053, 357760084, 357761408, 357762086, 357762094, 357762125, 357764720, 357764728, 357764757, 357764759, 357764765, 357764796, 357766798, 357766806, 357766837, 357771510, 357771518, 357771549, 357849242, 357849250, 357849258, 357849281, 357849289, 357849320, 357885471, 358067486, 358067494, 358067523, 358067525, 358067531, 358067562, 358069564, 358069572, 358069603, 358074276, 358074284, 358074315, 358088963, 358089000, 358091041, 358095753, 358175297, 358308223, 358308260, 358310301, 358315013, 358320760, 358320768, 358320776, 358320797, 358320799, 358320805, 358320807, 358320813, 358320836, 358320838, 358320844, 358320875, 358322838, 358322846, 358322854, 358322877, 358322885, 358322916, 358323665, 358327550, 358327558, 358327566, 358327589, 358327597, 358327628, 358356989, 358357026, 358359067, 358363779, 358377042, 358377050, 358377081, 358391652, 358391660, 358391668, 358391689, 358391691, 358391697, 358391699, 358391705, 358391728, 358391730, 358391736, 358391767, 358393730, 358393738, 358393746, 358393769, 358393777, 358393808, 358398442, 358398450, 358398458, 358398481, 358398489, 358398519, 358398520, 358407094, 358407102, 358407110, 358407133, 358407141, 358407172, 358617779, 358630316, 358630324, 358630332, 358630355, 358630363, 358630394, 358666545, 358699619, 358699656, 358699693, 358701208, 358701216, 358701224, 358701247, 358701255, 358701286, 358701697, 358701734, 358703775, 358706409, 358706446, 358708487, 358713199, 358765166, 358765174, 358765182, 358765205, 358765213, 358765244, 358848560, 358848568, 358848597, 358848599, 358848605, 358848636, 358850638, 358850646, 358850677, 358855350, 358855358, 358855389, 359009175, 359009212, 359011253, 359015965, 359100346, 359100354, 359100385, 359158116, 359158124, 359158155, 359179593, 359250485, 359318731, 359398853, 359411390, 359411398, 359411406, 359411429, 359411437, 359411468, 359447619, 359482282, 359482290, 359482298, 359482321, 359482329, 359482360, 359628146, 359671164, 359671172, 359671203, 359754020, 359754028, 359754036, 359754057, 359754059, 359754065, 359754067, 359754073, 359754096, 359754098, 359754104, 359754135, 359756098, 359756106, 359756114, 359756137, 359756145, 359756176, 359756925, 359760810, 359760818, 359760826, 359760849, 359760857, 359760888, 359774807, 359774844, 359774881, 359776885, 359776922, 359778963, 359781597, 359781634, 359783675, 359788387, 359790249, 359790286, 359792327, 359797039, 359840354, 359840362, 359840370, 359840393, 359840401, 359840432, 359939190, 359939198, 359939229, 360063576, 360063584, 360063592, 360063615, 360063623, 360063654, 360084363, 360084400, 360086441, 360091153, 360099805, 360198964, 360281820, 360281828, 360281857, 360281859, 360281865, 360281896, 360283898, 360283906, 360283937, 360288610, 360288618, 360288649, 360393919, 360591376, 360591384, 360591415, 360612853, 360745779, 360745816, 360747857, 360752569, 360769782, 360829208, 360829216, 360829224, 360829245, 360829247, 360829253, 360829255, 360829261, 360829284, 360829286, 360829292, 360829323, 360831286, 360831294, 360831302, 360831325, 360831333, 360831364, 360832113, 360835998, 360836006, 360836014, 360836037, 360836045, 360836076, 360844650, 360844658, 360844666, 360844689, 360844697, 360844728, 360865437, 360865474, 360867515, 360872227, 360880879, 360915542, 360915550, 360915558, 360915581, 360915589, 360915620, 361055335, 361138764, 361138772, 361138780, 361138803, 361138811, 361138842, 361174993, 361223509, 361223546, 361225587, 361230299, 361357008, 361357016, 361357045, 361357047, 361357053, 361357084, 361359086, 361359094, 361359125, 361363798, 361363806, 361363837, 361372450, 361372458, 361372489, 361533065, 361666564, 361666572, 361666603, 361688041, 361836409, 361907301, 361919838, 361919846, 361919854, 361919877, 361919885, 361919916, 361956067, 361990730, 361990738, 361990746, 361990769, 361990777, 361990808, 362212363, 362212400, 362212437, 362214441, 362214478, 362216519, 362219153, 362219190, 362221231, 362225943, 362277910, 362277918, 362277926, 362277949, 362277957, 362277988, 362298697, 362298734, 362300775, 362305487, 362314139, 362447638, 362447646, 362447677, 362521919, 362521956, 362523997, 362528709, 362608253, 362783181, 362783218, 362783255, 362785259, 362785296, 362787337, 362789971, 362790008, 362792049, 362796761, 362805710, 362805718, 362805749, 362831475, 363092737, 363092774, 363094815, 363099527, 363266764, 363266772, 363266780, 363266801, 363266803, 363266809, 363266811, 363266817, 363266840, 363266842, 363266848, 363266879, 363268842, 363268850, 363268858, 363268881, 363268889, 363268920, 363269669, 363273554, 363273562, 363273570, 363273593, 363273601, 363273632, 363302993, 363303030, 363305071, 363309783, 363353098, 363353106, 363353114, 363353137, 363353145, 363353176, 363373885, 363373922, 363375963, 363380675, 363389327, 363402293, 363576320, 363576328, 363576336, 363576359, 363576367, 363576398, 363612549, 363683441, 363747399, 363794564, 363794572, 363794601, 363794603, 363794609, 363794640, 363796642, 363796650, 363796681, 363801354, 363801362, 363801393, 363873811, 363873848, 363875889, 363880601, 363880898, 363880906, 363880937, 364104120, 364104128, 364104159, 364125597, 364183367, 364322364, 364322401, 364324442, 364329154, 364344857, 364357394, 364357402, 364357410, 364357433, 364357441, 364357472, 364365382, 364365390, 364365419, 364365421, 364365427, 364365458, 364367460, 364367468, 364367499, 364372172, 364372180, 364372211, 364393623, 364428286, 364428294, 364428302, 364428325, 364428333, 364428364, 364464515, 364631920, 364674938, 364674946, 364674977, 364736253, 364736290, 364738331, 364743043, 364822587, 364869179, 364869216, 364869253, 364871257, 364871294, 364873335, 364875969, 364876006, 364878047, 364882759, 364885194, 364885202, 364885233, 364956086, 364956094, 364956125, 364964441, 365045809, 365178735, 365178772, 365180813, 365185525, 365307071, 365307108, 365309149, 365313861, 365412994, 365456012, 365456020, 365456051, 365488291, 365616627, 365790654, 365790662, 365790670, 365790693, 365790701, 365790732, 365811441, 365811478, 365813519, 365818231, 365826883, 365897775, 365959809, 365959846, 365961887, 365966599, 366120997, 366269365, 366318454, 366318462, 366318493, 366335763, 366335800, 366335837, 366335874, 366337841, 366337878, 366337915, 366339919, 366339956, 366341997, 366342553, 366342590, 366342627, 366344631, 366344668, 366346709, 366349343, 366349380, 366351421, 366356133, 366378781, 366378789, 366378818, 366378820, 366378826, 366378855, 366378857, 366378863, 366378892, 366378894, 366378900, 366378931, 366380859, 366380867, 366380896, 366380898, 366380904, 366380933, 366380935, 366380941, 366380972, 366382259, 366382296, 366382937, 366382945, 366382974, 366382976, 366382982, 366383013, 366384337, 366385015, 366385023, 366385054, 366385571, 366385579, 366385608, 366385610, 366385616, 366385645, 366385647, 366385653, 366385684, 366387649, 366387657, 366387686, 366387688, 366387694, 366387725, 366389049, 366389727, 366389735, 366389766, 366392361, 366392369, 366392398, 366392400, 366392406, 366392437, 366394439, 366394447, 366394478, 366397701, 366399151, 366399159, 366399190, 366645319, 366645356, 366645393, 366647397, 366647434, 366649475, 366652109, 366652146, 366654187, 366658899, 366688337, 366688345, 366688374, 366688376, 366688382, 366688411, 366688413, 366688419, 366688450, 366690415, 366690423, 366690452, 366690454, 366690460, 366690491, 366691815, 366692493, 366692501, 366692532, 366695127, 366695135, 366695164, 366695166, 366695172, 366695203, 366697205, 366697213, 366697244, 366701917, 366701925, 366701956, 366782413, 366846254, 366865842, 366865850, 366865858, 366865881, 366865889, 366865920, 366889272, 366889280, 366889311, 366902071, 366954875, 366954912, 366956953, 366961665, 366972963, 366997893, 366997901, 366997930, 366997932, 366997938, 366997969, 366999971, 366999979, 367000010, 367004683, 367004691, 367004722, 367050439, 367260143, 367264431, 367307449, 367307457, 367307488, 367393069, 367393106, 367393642, 367393650, 367393681, 367395147, 367399859, 367426393, 367426430, 367426467, 367428471, 367428508, 367430549, 367433183, 367433220, 367435261, 367439973, 367469411, 367469419, 367469448, 367469450, 367469456, 367469485, 367469487, 367469493, 367469524, 367471489, 367471497, 367471526, 367471528, 367471534, 367471565, 367472889, 367473567, 367473575, 367473606, 367476201, 367476209, 367476238, 367476240, 367476246, 367476277, 367478279, 367478287, 367478318, 367482991, 367482999, 367483030, 367702625, 367735949, 367735986, 367738027, 367742739, 367778967, 367778975, 367779004, 367779006, 367779012, 367779043, 367781045, 367781053, 367781084, 367785757, 367785765, 367785796, 367830961, 367921442, 367960982, 367960990, 367960998, 367961019, 367961021, 367961027, 367961029, 367961035, 367961056, 367961058, 367961060, 367961064, 367961066, 367961072, 367961095, 367961097, 367961103, 367961134, 367963060, 367963068, 367963076, 367963097, 367963099, 367963105, 367963107, 367963113, 367963136, 367963138, 367963144, 367963175, 367964460, 367964468, 367964499, 367965138, 367965146, 367965154, 367965177, 367965185, 367965216, 367967772, 367967780, 367967788, 367967809, 367967811, 367967817, 367967819, 367967825, 367967848, 367967850, 367967856, 367967887, 367969850, 367969858, 367969866, 367969889, 367969897, 367969928, 367974562, 367974570, 367974578, 367974601, 367974609, 367974640, 368045505, 368088523, 368088531, 368088562, 368248997, 368249034, 368251075, 368255787, 368270538, 368270546, 368270554, 368270575, 368270577, 368270583, 368270585, 368270591, 368270614, 368270616, 368270622, 368270653, 368272616, 368272624, 368272632, 368272655, 368272663, 368272694, 368277328, 368277336, 368277344, 368277367, 368277375, 368277406, 368335331, 368468257, 368468294, 368470335, 368475047, 368483699, 368517023, 368517060, 368519101, 368523813, 368558553, 368560041, 368560049, 368560078, 368560080, 368560086, 368560117, 368562119, 368562127, 368562158, 368566831, 368566839, 368566870, 368580094, 368580102, 368580110, 368580133, 368580141, 368580172, 368777813, 368819815, 368819852, 368821893, 368826579, 368826605, 368859653, 368859690, 368859727, 368861731, 368861768, 368863809, 368866443, 368866480, 368868521, 368869597, 368869605, 368869636, 368873233, 368902671, 368902679, 368902708, 368902710, 368902716, 368902745, 368902747, 368902753, 368902784, 368904749, 368904757, 368904786, 368904788, 368904794, 368904825, 368906149, 368906827, 368906835, 368906866, 368909461, 368909469, 368909498, 368909500, 368909506, 368909537, 368911539, 368911547, 368911578, 368916251, 368916259, 368916290, 369051612, 369051620, 369051628, 369051649, 369051651, 369051657, 369051659, 369051665, 369051688, 369051690, 369051696, 369051727, 369053690, 369053698, 369053706, 369053729, 369053737, 369053768, 369058402, 369058410, 369058418, 369058441, 369058449, 369058480, 369129371, 369169209, 369169246, 369171287, 369175999, 369212227, 369212235, 369212264, 369212266, 369212272, 369212303, 369214305, 369214313, 369214344, 369219017, 369219025, 369219056, 369303398, 369303406, 369303414, 369303437, 369303445, 369303476, 369339627, 369361168, 369361176, 369361184, 369361207, 369361215, 369361246, 369410519, 369478765, 369521783, 369521791, 369521822, 369558887, 369607653, 369650671, 369650679, 369650710, 369831198, 369831206, 369831237, 369910445, 369916959, 369934841, 369934878, 369934915, 369936919, 369936956, 369938997, 369941631, 369941668, 369943709, 369948421, 369950283, 369950320, 369952361, 369957073, 369977859, 369977867, 369977896, 369977898, 369977904, 369977933, 369977935, 369977941, 369977972, 369979937, 369979945, 369979974, 369979976, 369979982, 369980013, 369981337, 369982015, 369982023, 369982054, 369984649, 369984657, 369984686, 369984688, 369984694, 369984725, 369986727, 369986735, 369986766, 369991439, 369991447, 369991478, 369993301, 369993309, 369993338, 369993340, 369993346, 369993377, 369995379, 369995387, 369995418, 370000091, 370000099, 370000130, 370142242, 370142250, 370142258, 370142281, 370142289, 370142320, 370244397, 370244434, 370246475, 370251187, 370259839, 370287415, 370287423, 370287452, 370287454, 370287460, 370287491, 370289493, 370289501, 370289532, 370294205, 370294213, 370294244, 370302857, 370302865, 370302896, 370358998, 370402016, 370402024, 370402055, 370484872, 370484880, 370484888, 370484909, 370484911, 370484917, 370484919, 370484925, 370484948, 370484950, 370484956, 370484987, 370486950, 370486958, 370486966, 370486989, 370486997, 370487028, 370491662, 370491670, 370491678, 370491701, 370491709, 370491740, 370553953, 370596971, 370596979, 370597010, 370772887, 370794428, 370794436, 370794444, 370794467, 370794475, 370794506, 370905813, 370905850, 370907891, 370912603, 370929816, 370992147, 371025471, 371025508, 371027549, 371032261, 371040913, 371068489, 371068497, 371068526, 371068528, 371068534, 371068565, 371070567, 371070575, 371070606, 371075279, 371075287, 371075318, 371083931, 371083939, 371083970, 371215369, 371335027, 371343705, 371378045, 371378053, 371378084, 371383543, 371383580, 371385621, 371390333, 371426561, 371426569, 371426598, 371426600, 371426606, 371426637, 371428639, 371428647, 371428678, 371433351, 371433359, 371433390, 371476631, 371476668, 371478709, 371483421, 371560060, 371560068, 371560076, 371560097, 371560099, 371560105, 371560107, 371560113, 371560136, 371560138, 371560144, 371560175, 371562138, 371562146, 371562154, 371562177, 371562185, 371562216, 371566850, 371566858, 371566866, 371566889, 371566897, 371566928, 371575502, 371575510, 371575518, 371575541, 371575549, 371575580, 371693099, 371736117, 371736125, 371736156, 371786187, 371848075, 371869616, 371869624, 371869632, 371869655, 371869663, 371869694, 371996443, 372067335, 372116101, 372159119, 372159127, 372159158, 372372397, 372372434, 372372471, 372374475, 372374512, 372376553, 372379187, 372379224, 372381265, 372385977, 372415415, 372415423, 372415452, 372415454, 372415460, 372415489, 372415491, 372415497, 372415528, 372417493, 372417501, 372417530, 372417532, 372417538, 372417569, 372418893, 372419571, 372419579, 372419610, 372422205, 372422213, 372422242, 372422244, 372422250, 372422281, 372424283, 372424291, 372424322, 372428995, 372429003, 372429034, 372458731, 372458768, 372460809, 372465521, 372474173, 372501749, 372501757, 372501786, 372501788, 372501794, 372501825, 372503827, 372503835, 372503866, 372508539, 372508547, 372508578, 372517191, 372517199, 372517230, 372567261, 372650690, 372650698, 372650706, 372650729, 372650737, 372650768, 372681953, 372681990, 372684031, 372688743, 372724971, 372724979, 372725008, 372725010, 372725016, 372725047, 372727049, 372727057, 372727088, 372731761, 372731769, 372731800, 372768287, 372811305, 372811313, 372811344, 372943215, 372943252, 372943289, 372945293, 372945330, 372947371, 372950005, 372950042, 372952083, 372956795, 372991509, 373008762, 373008770, 373008778, 373008801, 373008809, 373008840, 373034527, 373034535, 373034566, 373252771, 373252808, 373254849, 373259561, 373429703, 373463027, 373463064, 373465105, 373469817, 373506045, 373506053, 373506082, 373506084, 373506090, 373506121, 373508123, 373508131, 373508162, 373512835, 373512843, 373512874, 373533919, 373533956, 373535997, 373540709, 373549361, 373562327, 373576937, 373576945, 373576974, 373576976, 373576982, 373577013, 373579015, 373579023, 373579054, 373583727, 373583735, 373583766, 373592379, 373592387, 373592418, 373772583, 373815601, 373815609, 373815640, 373843475, 373886493, 373886501, 373886532, 373907433, 373950451, 373950459, 373950490, 373997616, 373997624, 373997632, 373997653, 373997655, 373997661, 373997663, 373997669, 373997692, 373997694, 373997700, 373997731, 373999694, 373999702, 373999710, 373999733, 373999741, 373999772, 374000521, 374004406, 374004414, 374004422, 374004445, 374004453, 374004484, 374033845, 374033882, 374035923, 374040635, 374083950, 374083958, 374083966, 374083989, 374083997, 374084028, 374285631, 374307172, 374307180, 374307188, 374307211, 374307219, 374307250, 374343401, 374504891, 374525416, 374525424, 374525453, 374525455, 374525461, 374525492, 374527494, 374527502, 374527533, 374532206, 374532214, 374532245, 374553657, 374596675, 374596683, 374596714, 374624549, 374667567, 374667575, 374667606, 374834972, 374834980, 374835011, 374856449, 374896287, 374896324, 374898365, 374903077, 374939305, 374939313, 374939342, 374939344, 374939350, 374939381, 374941383, 374941391, 374941422, 374946095, 374946103, 374946134, 374982621, 375025639, 375025647, 375025678, 375072231, 375072239, 375072268, 375072270, 375072276, 375072305, 375072307, 375072313, 375072344, 375074309, 375074317, 375074346, 375074348, 375074354, 375074385, 375075709, 375076387, 375076395, 375076426, 375079021, 375079029, 375079058, 375079060, 375079066, 375079097, 375081099, 375081107, 375081138, 375085811, 375085819, 375085850, 375088246, 375088254, 375088262, 375088285, 375088293, 375088324, 375124475, 375159138, 375159146, 375159154, 375159177, 375159185, 375159216, 375205843, 375248861, 375248869, 375248900, 375381787, 375381795, 375381824, 375381826, 375381832, 375381863, 375383865, 375383873, 375383904, 375388577, 375388585, 375388616, 375467105, 375467142, 375469183, 375473895, 375616046, 375616054, 375616085, 375691343, 375691351, 375691382, 375776661, 375971475, 375971512, 375973553, 375978265, 375986917, 376014493, 376014501, 376014530, 376014532, 376014538, 376014569, 376016571, 376016579, 376016610, 376021283, 376021291, 376021322, 376029935, 376029943, 376029974, 376057809, 376100827, 376100835, 376100866, 376162861, 376162869, 376162898, 376162900, 376162906, 376162937, 376164939, 376164947, 376164978, 376169651, 376169659, 376169690, 376281031, 376324049, 376324057, 376324088, 376472417, 376472425, 376472456, 376495797, 376495834, 376495871, 376495908, 376497875, 376497912, 376497949, 376499953, 376499990, 376502031, 376502587, 376502624, 376502661, 376504665, 376504702, 376506743, 376509377, 376509414, 376511455, 376516167, 376521506, 376521514, 376521522, 376521545, 376521553, 376521584, 376538815, 376538823, 376538852, 376538854, 376538860, 376538889, 376538891, 376538897, 376538926, 376538928, 376538934, 376538965, 376540893, 376540901, 376540930, 376540932, 376540938, 376540967, 376540969, 376540975, 376541006, 376542293, 376542330, 376542971, 376542979, 376543008, 376543010, 376543016, 376543047, 376544371, 376545049, 376545057, 376545088, 376545605, 376545613, 376545642, 376545644, 376545650, 376545679, 376545681, 376545687, 376545718, 376547683, 376547691, 376547720, 376547722, 376547728, 376547759, 376549083, 376549761, 376549769, 376549800, 376552395, 376552403, 376552432, 376552434, 376552440, 376552471, 376554473, 376554481, 376554512, 376557735, 376559185, 376559193, 376559224, 376805353, 376805390, 376805427, 376807431, 376807468, 376809509, 376812143, 376812180, 376814221, 376818933, 376848371, 376848379, 376848408, 376848410, 376848416, 376848445, 376848447, 376848453, 376848484, 376850449, 376850457, 376850486, 376850488, 376850494, 376850525, 376851849, 376852527, 376852535, 376852566, 376855161, 376855169, 376855198, 376855200, 376855206, 376855237, 376857239, 376857247, 376857278, 376861951, 376861959, 376861990, 376942447, 377049306, 377049314, 377049345, 377062105, 377105123, 377105131, 377105162, 377114909, 377114946, 377116987, 377121699, 377132997, 377157927, 377157935, 377157964, 377157966, 377157972, 377158003, 377160005, 377160013, 377160044, 377164717, 377164725, 377164756, 377176015, 377176023, 377176054, 377253491, 377253499, 377253530, 377420177, 377424465, 377463195, 377463203, 377463234, 377467483, 377467491, 377467522, 377513265, 377586427, 377586464, 377586501, 377588505, 377588542, 377590583, 377593217, 377593254, 377595295, 377596121, 377596129, 377596158, 377596160, 377596166, 377596197, 377596694, 377596702, 377596710, 377596733, 377596741, 377596772, 377598199, 377598207, 377598238, 377600007, 377602911, 377602919, 377602950, 377629445, 377629453, 377629482, 377629484, 377629490, 377629519, 377629521, 377629527, 377629558, 377631523, 377631531, 377631560, 377631562, 377631568, 377631599, 377632923, 377633601, 377633609, 377633640, 377636235, 377636243, 377636272, 377636274, 377636280, 377636311, 377638313, 377638321, 377638352, 377643025, 377643033, 377643064, 377895983, 377896020, 377898061, 377902773, 377905677, 377905685, 377905716, 377939001, 377939009, 377939038, 377939040, 377939046, 377939077, 377941079, 377941087, 377941118, 377945791, 377945799, 377945830, 377990995, 378124494, 378124502, 378124533, 378205539, 378248557, 378248565, 378248596, 378409031, 378409068, 378411109, 378415821, 378452049, 378452057, 378452086, 378452088, 378452094, 378452125, 378454127, 378454135, 378454166, 378458839, 378458847, 378458878, 378495365, 378538383, 378538391, 378538422, 378671309, 378671317, 378671346, 378671348, 378671354, 378671385, 378673387, 378673395, 378673426, 378677057, 378677094, 378678099, 378678107, 378678138, 378679135, 378683847, 378686751, 378686759, 378686790, 378718587, 378720075, 378720083, 378720112, 378720114, 378720120, 378720151, 378722153, 378722161, 378722192, 378726865, 378726873, 378726904, 378761605, 378761613, 378761644, 378979849, 378979886, 378980865, 378980873, 378980904, 378981927, 378986613, 378986639, 379019687, 379019724, 379019761, 379021765, 379021802, 379023843, 379026477, 379026514, 379028555, 379029631, 379029639, 379029670, 379033267, 379062705, 379062713, 379062742, 379062744, 379062750, 379062779, 379062781, 379062787, 379062818, 379064783, 379064791, 379064820, 379064822, 379064828, 379064859, 379066183, 379066861, 379066869, 379066900, 379069495, 379069503, 379069532, 379069534, 379069540, 379069571, 379071573, 379071581, 379071612, 379076285, 379076293, 379076324, 379289405, 379329243, 379329280, 379331321, 379336033, 379372261, 379372269, 379372298, 379372300, 379372306, 379372337, 379374339, 379374347, 379374378, 379379051, 379379059, 379379090, 379499661, 379542679, 379542687, 379542718, 379550667, 379550704, 379552745, 379557457, 379570553, 379613571, 379613579, 379613610, 379638799, 379681817, 379681825, 379681856, 379761939, 379761947, 379761978, 379767687, 379810705, 379810713, 379810744, 379860223, 380034250, 380034258, 380034266, 380034289, 380034297, 380034328, 380070479, 380094875, 380094912, 380094949, 380096953, 380096990, 380099031, 380101665, 380101702, 380103743, 380108455, 380110317, 380110354, 380112395, 380117107, 380120011, 380120019, 380120050, 380137893, 380137901, 380137930, 380137932, 380137938, 380137967, 380137969, 380137975, 380138006, 380139971, 380139979, 380140008, 380140010, 380140016, 380140047, 380141371, 380142049, 380142057, 380142088, 380144683, 380144691, 380144720, 380144722, 380144728, 380144759, 380146761, 380146769, 380146800, 380151473, 380151481, 380151512, 380153335, 380153343, 380153372, 380153374, 380153380, 380153411, 380155413, 380155421, 380155452, 380160125, 380160133, 380160164, 380404431, 380404468, 380406509, 380411221, 380419873, 380447449, 380447457, 380447486, 380447488, 380447494, 380447525, 380449527, 380449535, 380449566, 380454239, 380454247, 380454278, 380462891, 380462899, 380462930, 380562050, 380562058, 380562089, 380641297, 380713987, 380757005, 380757013, 380757044, 380932921, 380975939, 380975947, 380975978, 381089850, 381108865, 381108873, 381108902, 381108904, 381108910, 381108941, 381110943, 381110951, 381110982, 381115655, 381115663, 381115694, 381132868, 381132876, 381132907, 381185505, 381185542, 381187583, 381192295, 381195199, 381195207, 381195238, 381200947, 381228523, 381228531, 381228560, 381228562, 381228568, 381228599, 381230601, 381230609, 381230640, 381235313, 381235321, 381235352, 381243965, 381243973, 381244004, 381418421, 381418429, 381418460, 381495061, 381503739, 381538079, 381538087, 381538118, 381543577, 381543614, 381545655, 381550367, 381586595, 381586603, 381586632, 381586634, 381586640, 381586671, 381588673, 381588681, 381588712, 381593385, 381593393, 381593424, 381636665, 381636702, 381638743, 381643455, 381853133, 381896151, 381896159, 381896190, 381946221, 382008109, 382051127, 382051135, 382051166, 382074557, 382199495, 382199503, 382199534, 382270387, 382270395, 382270426, 382276135, 382319153, 382319161, 382319192, 382532431, 382532468, 382532505, 382534509, 382534546, 382536587, 382539221, 382539258, 382541299, 382546011, 382575449, 382575457, 382575486, 382575488, 382575494, 382575523, 382575525, 382575531, 382575562, 382577527, 382577535, 382577564, 382577566, 382577572, 382577603, 382578927, 382579605, 382579613, 382579644, 382582239, 382582247, 382582276, 382582278, 382582284, 382582315, 382584317, 382584325, 382584356, 382589029, 382589037, 382589068, 382618765, 382618802, 382620843, 382625555, 382634207, 382661783, 382661791, 382661820, 382661822, 382661828, 382661859, 382663861, 382663869, 382663900, 382668573, 382668581, 382668612, 382677225, 382677233, 382677264, 382727295, 382841987, 382842024, 382844065, 382848777, 382885005, 382885013, 382885042, 382885044, 382885050, 382885081, 382887083, 382887091, 382887122, 382891795, 382891803, 382891834, 382928321, 382971339, 382971347, 382971378, 383103249, 383103286, 383103323, 383105327, 383105364, 383107405, 383110039, 383110076, 383112117, 383116829, 383146267, 383146275, 383146304, 383146306, 383146312, 383146341, 383146343, 383146349, 383146380, 383148345, 383148353, 383148382, 383148384, 383148390, 383148421, 383149745, 383150423, 383150431, 383150462, 383151543, 383153057, 383153065, 383153094, 383153096, 383153102, 383153133, 383155135, 383155143, 383155174, 383159847, 383159855, 383159886, 383194561, 383194569, 383194600, 383412805, 383412842, 383414883, 383419595, 383455823, 383455831, 383455860, 383455862, 383455868, 383455899, 383457901, 383457909, 383457940, 383462613, 383462621, 383462652, 383623061, 383623098, 383625139, 383629851, 383632755, 383632763, 383632794, 383666079, 383666087, 383666116, 383666118, 383666124, 383666155, 383668157, 383668165, 383668196, 383672869, 383672877, 383672908, 383693953, 383693990, 383696031, 383700743, 383709395, 383722361, 383736971, 383736979, 383737008, 383737010, 383737016, 383737047, 383739049, 383739057, 383739088, 383743761, 383743769, 383743800, 383752413, 383752421, 383752452, 383765379, 383765387, 383765418, 383932617, 383975635, 383975643, 383975674, 384003509, 384046527, 384046535, 384046566, 384067467, 384110485, 384110493, 384110524, 384160555, 384193879, 384193916, 384195957, 384200669, 384236897, 384236905, 384236934, 384236936, 384236942, 384236973, 384238975, 384238983, 384239014, 384243687, 384243695, 384243726, 384445665, 384488683, 384488691, 384488722, 384503435, 384546453, 384546461, 384546492, 384707943, 384707951, 384707982, 384713691, 384728468, 384728476, 384728484, 384728505, 384728507, 384728513, 384728515, 384728521, 384728544, 384728546, 384728552, 384728583, 384730546, 384730554, 384730562, 384730585, 384730593, 384730624, 384735258, 384735266, 384735274, 384735297, 384735305, 384735336, 384756709, 384756717, 384756748, 384784583, 384827601, 384827609, 384827640, 385016483, 385038024, 385038032, 385038040, 385038063, 385038071, 385038102, 385056321, 385056358, 385058399, 385063111, 385099339, 385099347, 385099376, 385099378, 385099384, 385099415, 385101417, 385101425, 385101456, 385106129, 385106137, 385106168, 385142655, 385185673, 385185681, 385185712, 385235743, 385284509, 385327527, 385327535, 385327566, 385365877, 385408895, 385408903, 385408934, 385587301, 385627139, 385627176, 385629217, 385633929, 385670157, 385670165, 385670194, 385670196, 385670202, 385670233, 385672235, 385672243, 385672274, 385676947, 385676955, 385676986, 385819098, 385819106, 385819114, 385819137, 385819145, 385819176, 385936695, 385979713, 385979721, 385979752, 386131509, 386131546, 386133587, 386138299, 386146951, 386174527, 386174535, 386174564, 386174566, 386174572, 386174603, 386176605, 386176613, 386176644, 386181317, 386181325, 386181356, 386189969, 386189977, 386190008, 386217843, 386260861, 386260869, 386260900, 386441065, 386484083, 386484091, 386484122, 386698849, 386698857, 386698886, 386698888, 386698894, 386698923, 386698925, 386698931, 386698960, 386698962, 386698968, 386698999, 386700927, 386700935, 386700964, 386700966, 386700972, 386701001, 386701003, 386701009, 386701040, 386702327, 386702364, 386703005, 386703013, 386703042, 386703044, 386703050, 386703081, 386704405, 386705083, 386705091, 386705122, 386705639, 386705647, 386705676, 386705678, 386705684, 386705713, 386705715, 386705721, 386705752, 386707717, 386707725, 386707754, 386707756, 386707762, 386707793, 386709117, 386709795, 386709803, 386709834, 386712429, 386712437, 386712466, 386712468, 386712474, 386712505, 386714507, 386714515, 386714546, 386717769, 386719219, 386719227, 386719258, 386741867, 386741875, 386741883, 386741904, 386741906, 386741912, 386741914, 386741920, 386741941, 386741943, 386741945, 386741949, 386741951, 386741957, 386741978, 386741980, 386741982, 386741986, 386741988, 386741994, 386742017, 386742019, 386742025, 386742056, 386743945, 386743953, 386743961, 386743982, 386743984, 386743990, 386743992, 386743998, 386744019, 386744021, 386744023, 386744027, 386744029, 386744035, 386744058, 386744060, 386744066, 386744097, 386745345, 386745353, 386745382, 386745384, 386745390, 386745421, 386746023, 386746031, 386746039, 386746060, 386746062, 386746068, 386746070, 386746076, 386746099, 386746101, 386746107, 386746138, 386747423, 386747431, 386747462, 386748101, 386748109, 386748117, 386748140, 386748148, 386748179, 386748657, 386748665, 386748673, 386748694, 386748696, 386748702, 386748704, 386748710, 386748731, 386748733, 386748735, 386748739, 386748741, 386748747, 386748770, 386748772, 386748778, 386748809, 386750735, 386750743, 386750751, 386750772, 386750774, 386750780, 386750782, 386750788, 386750811, 386750813, 386750819, 386750850, 386752135, 386752143, 386752174, 386752813, 386752821, 386752829, 386752852, 386752860, 386752891, 386755447, 386755455, 386755463, 386755484, 386755486, 386755492, 386755494, 386755500, 386755523, 386755525, 386755531, 386755562, 386757525, 386757533, 386757541, 386757564, 386757572, 386757603, 386760787, 386760795, 386760826, 386762237, 386762245, 386762253, 386762276, 386762284, 386762315, 387008405, 387008413, 387008442, 387008444, 387008450, 387008479, 387008481, 387008487, 387008518, 387010483, 387010491, 387010520, 387010522, 387010528, 387010559, 387011883, 387012561, 387012569, 387012600, 387015195, 387015203, 387015232, 387015234, 387015240, 387015271, 387017273, 387017281, 387017312, 387021985, 387021993, 387022024, 387051423, 387051431, 387051439, 387051460, 387051462, 387051468, 387051470, 387051476, 387051497, 387051499, 387051501, 387051505, 387051507, 387051513, 387051536, 387051538, 387051544, 387051575, 387053501, 387053509, 387053517, 387053538, 387053540, 387053546, 387053548, 387053554, 387053577, 387053579, 387053585, 387053616, 387054901, 387054909, 387054940, 387055579, 387055587, 387055595, 387055618, 387055626, 387055657, 387058213, 387058221, 387058229, 387058250, 387058252, 387058258, 387058260, 387058266, 387058289, 387058291, 387058297, 387058328, 387060291, 387060299, 387060307, 387060330, 387060338, 387060369, 387065003, 387065011, 387065019, 387065042, 387065050, 387065081, 387145499, 387145507, 387145538, 387222139, 387252358, 387252366, 387252374, 387252397, 387252405, 387252436, 387265157, 387265165, 387265196, 387293031, 387317961, 387317969, 387317998, 387318000, 387318006, 387318037, 387320039, 387320047, 387320078, 387324751, 387324759, 387324790, 387336049, 387336057, 387336088, 387360979, 387360987, 387360995, 387361016, 387361018, 387361024, 387361026, 387361032, 387361055, 387361057, 387361063, 387361094, 387363057, 387363065, 387363073, 387363096, 387363104, 387363135, 387367769, 387367777, 387367785, 387367808, 387367816, 387367847, 387580211, 387623229, 387623237, 387623268, 387627517, 387627525, 387627556, 387670535, 387670543, 387670551, 387670574, 387670582, 387670613, 387673299, 387789479, 387789487, 387789516, 387789518, 387789524, 387789553, 387789555, 387789561, 387789592, 387791557, 387791565, 387791594, 387791596, 387791602, 387791633, 387792957, 387793635, 387793643, 387793674, 387796269, 387796277, 387796306, 387796308, 387796314, 387796345, 387798347, 387798355, 387798386, 387803059, 387803067, 387803098, 387832497, 387832505, 387832513, 387832534, 387832536, 387832542, 387832544, 387832550, 387832571, 387832573, 387832575, 387832579, 387832581, 387832587, 387832610, 387832612, 387832618, 387832649, 387834575, 387834583, 387834591, 387834612, 387834614, 387834620, 387834622, 387834628, 387834651, 387834653, 387834659, 387834690, 387835975, 387835983, 387836014, 387836653, 387836661, 387836669, 387836692, 387836700, 387836731, 387839287, 387839295, 387839303, 387839324, 387839326, 387839332, 387839334, 387839340, 387839363, 387839365, 387839371, 387839402, 387841365, 387841373, 387841381, 387841404, 387841412, 387841443, 387846077, 387846085, 387846093, 387846116, 387846124, 387846155, 388099035, 388099043, 388099072, 388099074, 388099080, 388099111, 388101113, 388101121, 388101152, 388105825, 388105833, 388105864, 388142053, 388142061, 388142069, 388142090, 388142092, 388142098, 388142100, 388142106, 388142129, 388142131, 388142137, 388142168, 388144131, 388144139, 388144147, 388144170, 388144178, 388144209, 388148843, 388148851, 388148859, 388148882, 388148890, 388148921, 388151029, 388194047, 388194055, 388194086, 388244117, 388327546, 388327554, 388327562, 388327585, 388327593, 388327624, 388408591, 388408599, 388408630, 388451609, 388451617, 388451625, 388451648, 388451656, 388451687, 388569065, 388569102, 388571143, 388575855, 388612083, 388612091, 388612120, 388612122, 388612128, 388612159, 388614161, 388614169, 388614200, 388618873, 388618881, 388618912, 388655399, 388698417, 388698425, 388698456, 388878621, 388880109, 388880117, 388880146, 388880148, 388880154, 388880185, 388882187, 388882195, 388882226, 388886899, 388886907, 388886938, 388921639, 388921647, 388921678, 388923127, 388923135, 388923143, 388923164, 388923166, 388923172, 388923174, 388923180, 388923203, 388923205, 388923211, 388923242, 388925205, 388925213, 388925221, 388925244, 388925252, 388925283, 388929917, 388929925, 388929933, 388929956, 388929964, 388929995, 389139883, 389139920, 389141961, 389146673, 389182901, 389182909, 389182938, 389182940, 389182946, 389182977, 389184979, 389184987, 389185018, 389189665, 389189673, 389189691, 389189699, 389189704, 389189730, 389222739, 389222747, 389222776, 389222778, 389222784, 389222813, 389222815, 389222821, 389222852, 389224817, 389224825, 389224854, 389224856, 389224862, 389224893, 389226217, 389226895, 389226903, 389226934, 389229529, 389229537, 389229566, 389229568, 389229574, 389229605, 389231607, 389231615, 389231646, 389232683, 389232691, 389232699, 389232722, 389232730, 389232761, 389236319, 389236327, 389236358, 389265757, 389265765, 389265773, 389265794, 389265796, 389265802, 389265804, 389265810, 389265831, 389265833, 389265835, 389265839, 389265841, 389265847, 389265870, 389265872, 389265878, 389265909, 389267835, 389267843, 389267851, 389267872, 389267874, 389267880, 389267882, 389267888, 389267911, 389267913, 389267919, 389267950, 389269235, 389269243, 389269274, 389269913, 389269921, 389269929, 389269952, 389269960, 389269991, 389272547, 389272555, 389272563, 389272584, 389272586, 389272592, 389272594, 389272600, 389272623, 389272625, 389272631, 389272662, 389274625, 389274633, 389274641, 389274664, 389274672, 389274703, 389279337, 389279345, 389279353, 389279376, 389279384, 389279415, 389449439, 389492457, 389492465, 389492496, 389532295, 389532303, 389532332, 389532334, 389532340, 389532371, 389534373, 389534381, 389534412, 389539085, 389539093, 389539124, 389575313, 389575321, 389575329, 389575350, 389575352, 389575358, 389575360, 389575366, 389575389, 389575391, 389575397, 389575428, 389577391, 389577399, 389577407, 389577430, 389577438, 389577469, 389582103, 389582111, 389582119, 389582142, 389582150, 389582181, 389659695, 389702713, 389702721, 389702752, 389710701, 389710738, 389712779, 389717491, 389730587, 389773605, 389773613, 389773644, 389841851, 389841859, 389841890, 389884869, 389884877, 389884885, 389884908, 389884916, 389884947, 389970739, 389970747, 389970778, 390013757, 390013765, 390013773, 390013796, 390013804, 390013835, 390020257, 390230513, 390257516, 390257553, 390257590, 390259594, 390259631, 390261672, 390264306, 390264343, 390266384, 390271096, 390273531, 390273539, 390273570, 390297927, 390297935, 390297964, 390297966, 390297972, 390298001, 390298003, 390298009, 390298040, 390300005, 390300013, 390300042, 390300044, 390300050, 390300081, 390301405, 390302083, 390302091, 390302122, 390304717, 390304725, 390304754, 390304756, 390304762, 390304793, 390306795, 390306803, 390306834, 390311507, 390311515, 390311546, 390313369, 390313377, 390313406, 390313408, 390313414, 390313445, 390315447, 390315455, 390315486, 390320159, 390320167, 390320198, 390340945, 390340953, 390340961, 390340982, 390340984, 390340990, 390340992, 390340998, 390341019, 390341021, 390341023, 390341027, 390341029, 390341035, 390341058, 390341060, 390341066, 390341097, 390343023, 390343031, 390343039, 390343060, 390343062, 390343068, 390343070, 390343076, 390343099, 390343101, 390343107, 390343138, 390344423, 390344431, 390344462, 390345101, 390345109, 390345117, 390345140, 390345148, 390345179, 390347735, 390347743, 390347751, 390347772, 390347774, 390347780, 390347782, 390347788, 390347811, 390347813, 390347819, 390347850, 390349813, 390349821, 390349829, 390349852, 390349860, 390349891, 390354525, 390354533, 390354541, 390354564, 390354572, 390354603, 390356387, 390356395, 390356403, 390356424, 390356426, 390356432, 390356434, 390356440, 390356463, 390356465, 390356471, 390356502, 390358465, 390358473, 390358481, 390358504, 390358512, 390358543, 390363177, 390363185, 390363193, 390363216, 390363224, 390363255, 390567072, 390567109, 390569150, 390573862, 390607483, 390607491, 390607520, 390607522, 390607528, 390607559, 390609561, 390609569, 390609600, 390614273, 390614281, 390614312, 390622925, 390622933, 390622964, 390650501, 390650509, 390650517, 390650538, 390650540, 390650546, 390650548, 390650554, 390650577, 390650579, 390650585, 390650616, 390652579, 390652587, 390652595, 390652618, 390652626, 390652657, 390657291, 390657299, 390657307, 390657330, 390657338, 390657369, 390665943, 390665951, 390665959, 390665982, 390665990, 390666021, 390765102, 390765110, 390765118, 390765141, 390765149, 390765180, 390801331, 390876628, 390917039, 390917047, 390917078, 390960057, 390960065, 390960073, 390960096, 390960104, 390960135, 391092955, 391135973, 391135981, 391136012, 391292902, 391292910, 391292941, 391348146, 391348183, 391350224, 391354936, 391388557, 391388565, 391388594, 391388596, 391388602, 391388633, 391390635, 391390643, 391390674, 391395347, 391395355, 391395386, 391403999, 391404007, 391404038, 391431575, 391431583, 391431591, 391431612, 391431614, 391431620, 391431622, 391431628, 391431651, 391431653, 391431659, 391431690, 391433653, 391433661, 391433669, 391433692, 391433700, 391433731, 391438365, 391438373, 391438381, 391438404, 391438412, 391438443, 391447017, 391447025, 391447033, 391447056, 391447064, 391447095, 391657702, 391663773, 391698113, 391698121, 391698152, 391706791, 391706799, 391706830, 391741131, 391741139, 391741147, 391741170, 391741178, 391741209, 391746629, 391746637, 391746666, 391746668, 391746674, 391746705, 391748707, 391748715, 391748746, 391753419, 391753427, 391753458, 391789647, 391789655, 391789663, 391789684, 391789686, 391789692, 391789694, 391789700, 391789723, 391789725, 391789731, 391789762, 391791725, 391791733, 391791741, 391791764, 391791772, 391791803, 391796437, 391796445, 391796453, 391796476, 391796484, 391796515, 391839717, 391839725, 391839754, 391839756, 391839762, 391839793, 391841795, 391841803, 391841834, 391846507, 391846515, 391846546, 392056185, 392056193, 392056224, 392099203, 392099211, 392099219, 392099242, 392099250, 392099281, 392149273, 392149281, 392149312, 392168143, 392211161, 392211169, 392211200, 392234591, 392438776, 392479187, 392479195, 392479226, 392522205, 392522213, 392522221, 392522244, 392522252, 392522283, 392735483, 392735491, 392735520, 392735522, 392735528, 392735557, 392735559, 392735565, 392735596, 392737561, 392737569, 392737598, 392737600, 392737606, 392737637, 392738961, 392739639, 392739647, 392739678, 392742273, 392742281, 392742310, 392742312, 392742318, 392742349, 392744351, 392744359, 392744390, 392749063, 392749071, 392749102, 392778501, 392778509, 392778517, 392778538, 392778540, 392778546, 392778548, 392778554, 392778575, 392778577, 392778579, 392778583, 392778585, 392778591, 392778614, 392778616, 392778622, 392778653, 392780579, 392780587, 392780595, 392780616, 392780618, 392780624, 392780626, 392780632, 392780655, 392780657, 392780663, 392780694, 392781406, 392781443, 392781979, 392781987, 392782018, 392782657, 392782665, 392782673, 392782696, 392782704, 392782735, 392783484, 392785291, 392785299, 392785307, 392785328, 392785330, 392785336, 392785338, 392785344, 392785367, 392785369, 392785375, 392785406, 392787369, 392787377, 392787385, 392787408, 392787416, 392787447, 392788196, 392792081, 392792089, 392792097, 392792120, 392792128, 392792159, 392821817, 392821825, 392821854, 392821856, 392821862, 392821893, 392823895, 392823903, 392823934, 392828607, 392828615, 392828646, 392837259, 392837267, 392837298, 392864835, 392864843, 392864851, 392864872, 392864874, 392864880, 392864882, 392864888, 392864911, 392864913, 392864919, 392864950, 392866913, 392866921, 392866929, 392866952, 392866960, 392866991, 392871625, 392871633, 392871641, 392871664, 392871672, 392871703, 392880277, 392880285, 392880293, 392880316, 392880324, 392880355, 392930347, 392930355, 392930386, 393045039, 393045047, 393045076, 393045078, 393045084, 393045115, 393047117, 393047125, 393047156, 393051829, 393051837, 393051868, 393088057, 393088065, 393088073, 393088094, 393088096, 393088102, 393088104, 393088110, 393088133, 393088135, 393088141, 393088172, 393090135, 393090143, 393090151, 393090174, 393090182, 393090213, 393090962, 393094847, 393094855, 393094863, 393094886, 393094894, 393094925, 393131373, 393131381, 393131412, 393174391, 393174399, 393174407, 393174430, 393174438, 393174469, 393263283, 393263320, 393263357, 393265361, 393265398, 393267439, 393270073, 393270110, 393272151, 393276863, 393306301, 393306309, 393306338, 393306340, 393306346, 393306375, 393306377, 393306383, 393306414, 393308379, 393308387, 393308416, 393308418, 393308424, 393308455, 393309779, 393310457, 393310465, 393310496, 393313091, 393313099, 393313128, 393313130, 393313136, 393313167, 393315169, 393315177, 393315208, 393319881, 393319889, 393319920, 393354595, 393354603, 393354634, 393397613, 393397621, 393397629, 393397652, 393397660, 393397691, 393572839, 393572876, 393574917, 393579629, 393615857, 393615865, 393615894, 393615896, 393615902, 393615933, 393617935, 393617943, 393617974, 393622647, 393622655, 393622686, 393826113, 393826121, 393826150, 393826152, 393826158, 393826189, 393828191, 393828199, 393828230, 393832903, 393832911, 393832942, 393856594, 393856631, 393858672, 393863384, 393869131, 393869139, 393869147, 393869168, 393869170, 393869176, 393869178, 393869184, 393869207, 393869209, 393869215, 393869246, 393871209, 393871217, 393871225, 393871248, 393871256, 393871287, 393872036, 393875921, 393875929, 393875937, 393875960, 393875968, 393875999, 393882395, 393897005, 393897013, 393897042, 393897044, 393897050, 393897081, 393899083, 393899091, 393899122, 393903795, 393903803, 393903834, 393912447, 393912455, 393912486, 393925413, 393925421, 393925452, 393940023, 393940031, 393940039, 393940060, 393940062, 393940068, 393940070, 393940076, 393940099, 393940101, 393940107, 393940138, 393942101, 393942109, 393942117, 393942140, 393942148, 393942179, 393946813, 393946821, 393946829, 393946852, 393946860, 393946891, 393955465, 393955473, 393955481, 393955504, 393955512, 393955543, 394135669, 394135677, 394135708, 394166150, 394178687, 394178695, 394178703, 394178726, 394178734, 394178765, 394206561, 394206569, 394206600, 394249579, 394249587, 394249595, 394249618, 394249626, 394249657, 394270519, 394270527, 394270558, 394313537, 394313545, 394313553, 394313576, 394313584, 394313615, 394353913, 394353950, 394355991, 394360703, 394363607, 394363615, 394363646, 394396931, 394396939, 394396968, 394396970, 394396976, 394397007, 394399009, 394399017, 394399048, 394403721, 394403729, 394403760, 394605699, 394648717, 394648725, 394648756, 394663469, 394706487, 394706495, 394706526, 394916743, 394916751, 394916782, 394947224, 394959761, 394959769, 394959777, 394959800, 394959808, 394959839, 394987635, 394987643, 394987674, 395030653, 395030661, 395030669, 395030692, 395030700, 395030731, 395176517, 395219535, 395219543, 395219574, 395259373, 395259381, 395259410, 395259412, 395259418, 395259449, 395261451, 395261459, 395261490, 395266163, 395266171, 395266202, 395302391, 395302399, 395302407, 395302428, 395302430, 395302436, 395302438, 395302444, 395302467, 395302469, 395302475, 395302506, 395304469, 395304477, 395304485, 395304508, 395304516, 395304547, 395305296, 395309181, 395309189, 395309197, 395309220, 395309228, 395309259, 395345707, 395345715, 395345746, 395388725, 395388733, 395388741, 395388764, 395388772, 395388803, 395438795, 395438803, 395438834, 395444543, 395487561, 395487569, 395487600, 395568929, 395568937, 395568968, 395611947, 395611955, 395611963, 395611986, 395611994, 395612025, 395747335, 395787173, 395787210, 395789251, 395793963, 395830191, 395830199, 395830228, 395830230, 395830236, 395830267, 395832269, 395832277, 395832308, 395836981, 395836989, 395837020, 396096729, 396139747, 396139755, 396139786, 396294150, 396294187, 396296228, 396300940, 396318153, 396334561, 396334569, 396334598, 396334600, 396334606, 396334637, 396336639, 396336647, 396336678, 396341351, 396341359, 396341390, 396350003, 396350011, 396350042, 396377579, 396377587, 396377595, 396377616, 396377618, 396377624, 396377626, 396377632, 396377655, 396377657, 396377663, 396377694, 396379657, 396379665, 396379673, 396379696, 396379704, 396379735, 396380484, 396384369, 396384377, 396384385, 396384408, 396384416, 396384447, 396393021, 396393029, 396393037, 396393060, 396393068, 396393099, 396420895, 396420903, 396420934, 396463913, 396463921, 396463929, 396463952, 396463960, 396463991, 396603706, 396644117, 396644125, 396644156, 396687135, 396687143, 396687151, 396687174, 396687182, 396687213, 396862361, 396862398, 396864439, 396869151, 396877803, 396901901, 396901909, 396901917, 396901938, 396901940, 396901946, 396901948, 396901954, 396901975, 396901977, 396901979, 396901983, 396901985, 396901991, 396902012, 396902014, 396902016, 396902020, 396902022, 396902028, 396902051, 396902053, 396902059, 396902090, 396903979, 396903987, 396903995, 396904016, 396904018, 396904024, 396904026, 396904032, 396904053, 396904055, 396904057, 396904061, 396904063, 396904069, 396904092, 396904094, 396904100, 396904131, 396905379, 396905387, 396905416, 396905418, 396905424, 396905455, 396906057, 396906065, 396906073, 396906094, 396906096, 396906102, 396906104, 396906110, 396906133, 396906135, 396906141, 396906172, 396907457, 396907465, 396907496, 396908135, 396908143, 396908151, 396908174, 396908182, 396908213, 396908691, 396908699, 396908707, 396908728, 396908730, 396908736, 396908738, 396908744, 396908765, 396908767, 396908769, 396908773, 396908775, 396908781, 396908804, 396908806, 396908812, 396908843, 396910769, 396910777, 396910785, 396910806, 396910808, 396910814, 396910816, 396910822, 396910845, 396910847, 396910853, 396910884, 396912169, 396912177, 396912208, 396912847, 396912855, 396912863, 396912886, 396912894, 396912925, 396915481, 396915489, 396915497, 396915518, 396915520, 396915526, 396915528, 396915534, 396915557, 396915559, 396915565, 396915596, 396917559, 396917567, 396917575, 396917598, 396917606, 396917637, 396920821, 396920829, 396920860, 396922271, 396922279, 396922287, 396922310, 396922318, 396922349, 397171917, 397211457, 397211465, 397211473, 397211494, 397211496, 397211502, 397211504, 397211510, 397211531, 397211533, 397211535, 397211539, 397211541, 397211547, 397211570, 397211572, 397211578, 397211609, 397213535, 397213543, 397213551, 397213572, 397213574, 397213580, 397213582, 397213588, 397213611, 397213613, 397213619, 397213650, 397214935, 397214943, 397214974, 397215613, 397215621, 397215629, 397215652, 397215660, 397215691, 397218247, 397218255, 397218263, 397218284, 397218286, 397218292, 397218294, 397218300, 397218323, 397218325, 397218331, 397218362, 397220325, 397220333, 397220341, 397220364, 397220372, 397220403, 397225037, 397225045, 397225053, 397225076, 397225084, 397225115, 397384780, 397425191, 397425199, 397425230, 397455672, 397468209, 397468217, 397468225, 397468248, 397468256, 397468287, 397496083, 397496091, 397496122, 397521013, 397521021, 397521029, 397521050, 397521052, 397521058, 397521060, 397521066, 397521089, 397521091, 397521097, 397521128, 397523091, 397523099, 397523107, 397523130, 397523138, 397523169, 397527803, 397527811, 397527819, 397527842, 397527850, 397527881, 397539101, 397539109, 397539117, 397539140, 397539148, 397539179, 397783263, 397783271, 397783302, 397826281, 397826289, 397826297, 397826320, 397826328, 397826359, 397830569, 397830577, 397830585, 397830608, 397830616, 397830647, 397876351, 397876359, 397876390, 397952991, 397992531, 397992539, 397992547, 397992568, 397992570, 397992576, 397992578, 397992584, 397992605, 397992607, 397992609, 397992613, 397992615, 397992621, 397992644, 397992646, 397992652, 397992683, 397994609, 397994617, 397994625, 397994646, 397994648, 397994654, 397994656, 397994662, 397994685, 397994687, 397994693, 397994724, 397996009, 397996017, 397996048, 397996687, 397996695, 397996703, 397996726, 397996734, 397996765, 397999321, 397999329, 397999337, 397999358, 397999360, 397999366, 397999368, 397999374, 397999397, 397999399, 397999405, 397999436, 398001399, 398001407, 398001415, 398001438, 398001446, 398001477, 398006111, 398006119, 398006127, 398006150, 398006158, 398006189, 398302087, 398302095, 398302103, 398302124, 398302126, 398302132, 398302134, 398302140, 398302163, 398302165, 398302171, 398302202, 398304165, 398304173, 398304181, 398304204, 398304212, 398304243, 398308877, 398308885, 398308893, 398308916, 398308924, 398308955, 398311063, 398354081, 398354089, 398354120, 398404151, 398611643, 398611651, 398611659, 398611682, 398611690, 398611721, 398772117, 398772125, 398772154, 398772156, 398772162, 398772193, 398774195, 398774203, 398774234, 398778907, 398778915, 398778946, 398815135, 398815143, 398815151, 398815172, 398815174, 398815180, 398815182, 398815188, 398815211, 398815213, 398815219, 398815250, 398817213, 398817221, 398817229, 398817252, 398817260, 398817291, 398818040, 398821925, 398821933, 398821941, 398821964, 398821972, 398822003, 398858451, 398858459, 398858490, 398901469, 398901477, 398901485, 398901508, 398901516, 398901547, 398950966, 398951003, 398953044, 398957756, 399081673, 399081681, 399081712, 399083161, 399083169, 399083177, 399083198, 399083200, 399083206, 399083208, 399083214, 399083237, 399083239, 399083245, 399083276, 399085239, 399085247, 399085255, 399085278, 399085286, 399085317, 399089951, 399089959, 399089967, 399089990, 399089998, 399090029, 399124691, 399124699, 399124707, 399124730, 399124738, 399124769, 399260522, 399299917, 399299954, 399301995, 399306707, 399342935, 399342943, 399342972, 399342974, 399342980, 399343011, 399345013, 399345021, 399345052, 399349725, 399349733, 399349764, 399386251, 399392717, 399392725, 399392733, 399392756, 399392764, 399392795, 399425791, 399425799, 399425807, 399425828, 399425830, 399425836, 399425838, 399425844, 399425865, 399425867, 399425869, 399425873, 399425875, 399425881, 399425904, 399425906, 399425912, 399425943, 399427869, 399427877, 399427885, 399427906, 399427908, 399427914, 399427916, 399427922, 399427945, 399427947, 399427953, 399427984, 399429269, 399429277, 399429308, 399429947, 399429955, 399429963, 399429986, 399429994, 399430025, 399432581, 399432589, 399432597, 399432618, 399432620, 399432626, 399432628, 399432634, 399432657, 399432659, 399432665, 399432696, 399434659, 399434667, 399434675, 399434698, 399434706, 399434737, 399439371, 399439379, 399439387, 399439410, 399439418, 399439449, 399609473, 399652491, 399652499, 399652530, 399735347, 399735355, 399735363, 399735384, 399735386, 399735392, 399735394, 399735400, 399735423, 399735425, 399735431, 399735462, 399737425, 399737433, 399737441, 399737464, 399737472, 399737503, 399742137, 399742145, 399742153, 399742176, 399742184, 399742215, 399862747, 399862755, 399862786, 399870735, 399870772, 399872813, 399877525, 399893228, 399905765, 399905773, 399905781, 399905804, 399905812, 399905843, 399913753, 399913761, 399913790, 399913792, 399913798, 399913829, 399915831, 399915839, 399915870, 399920543, 399920551, 399920582, 399933639, 399933647, 399933678, 399976657, 399976665, 399976673, 399976696, 399976704, 399976735, 400041596, 400044903, 400044911, 400044919, 400044942, 400044950, 400044981, 400173791, 400173799, 400173807, 400173830, 400173838, 400173869, 400180291, 400223309, 400223317, 400223348, 400390547, 400417550, 400417587, 400417624, 400419628, 400419665, 400421706, 400424340, 400424377, 400426418, 400431130, 400433565, 400433573, 400433604, 400461439, 400500979, 400500987, 400500995, 400501016, 400501018, 400501024, 400501026, 400501032, 400501053, 400501055, 400501057, 400501061, 400501063, 400501069, 400501092, 400501094, 400501100, 400501131, 400503057, 400503065, 400503073, 400503094, 400503096, 400503102, 400503104, 400503110, 400503133, 400503135, 400503141, 400503172, 400504457, 400504465, 400504496, 400505135, 400505143, 400505151, 400505174, 400505182, 400505213, 400507769, 400507777, 400507785, 400507806, 400507808, 400507814, 400507816, 400507822, 400507845, 400507847, 400507853, 400507884, 400509847, 400509855, 400509863, 400509886, 400509894, 400509925, 400514559, 400514567, 400514575, 400514598, 400514606, 400514637, 400516421, 400516429, 400516437, 400516458, 400516460, 400516466, 400516468, 400516474, 400516497, 400516499, 400516505, 400516536, 400518499, 400518507, 400518515, 400518538, 400518546, 400518577, 400523211, 400523219, 400523227, 400523250, 400523258, 400523289, 400727106, 400727143, 400729184, 400733896, 400810535, 400810543, 400810551, 400810572, 400810574, 400810580, 400810582, 400810588, 400810611, 400810613, 400810619, 400810650, 400812613, 400812621, 400812629, 400812652, 400812660, 400812691, 400817325, 400817333, 400817341, 400817364, 400817372, 400817403, 400825977, 400825985, 400825993, 400826016, 400826024, 400826055, 400961365, 401004383, 401004391, 401004422, 401036662, 401120091, 401120099, 401120107, 401120130, 401120138, 401120169, 401296007, 401296015, 401296046, 401339025, 401339033, 401339041, 401339064, 401339072, 401339103, 401474856, 401495954, 401495962, 401495970, 401495993, 401496001, 401496032, 401508180, 401508217, 401510258, 401514970, 401591609, 401591617, 401591625, 401591646, 401591648, 401591654, 401591656, 401591662, 401591685, 401591687, 401591693, 401591724, 401593687, 401593695, 401593703, 401593726, 401593734, 401593765, 401598399, 401598407, 401598415, 401598438, 401598446, 401598477, 401607051, 401607059, 401607067, 401607090, 401607098, 401607129, 401817736, 401823807, 401866825, 401866833, 401866864, 401901165, 401901173, 401901181, 401901204, 401901212, 401901243, 401949681, 401949689, 401949697, 401949718, 401949720, 401949726, 401949728, 401949734, 401949757, 401949759, 401949765, 401949796, 401951759, 401951767, 401951775, 401951798, 401951806, 401951837, 401956471, 401956479, 401956487, 401956510, 401956518, 401956549, 402259237, 402259245, 402259253, 402259276, 402259284, 402259315, 402330784, 402371195, 402371203, 402371234, 402394625, 402414213, 402414221, 402414229, 402414252, 402414260, 402414291, 402437643, 402437651, 402437682, 402550044, 402598810, 402682239, 402682247, 402682255, 402682278, 402682286, 402682317, 402898995, 402938535, 402938543, 402938551, 402938572, 402938574, 402938580, 402938582, 402938588, 402938609, 402938611, 402938613, 402938617, 402938619, 402938625, 402938648, 402938650, 402938656, 402938687, 402940613, 402940621, 402940629, 402940650, 402940652, 402940658, 402940660, 402940666, 402940689, 402940691, 402940697, 402940728, 402941440, 402941477, 402942013, 402942021, 402942052, 402942691, 402942699, 402942707, 402942730, 402942738, 402942769, 402943518, 402945325, 402945333, 402945341, 402945362, 402945364, 402945370, 402945372, 402945378, 402945401, 402945403, 402945409, 402945440, 402947403, 402947411, 402947419, 402947442, 402947450, 402947481, 402948230, 402952115, 402952123, 402952131, 402952154, 402952162, 402952193, 403024869, 403024877, 403024885, 403024906, 403024908, 403024914, 403024916, 403024922, 403024945, 403024947, 403024953, 403024984, 403026947, 403026955, 403026963, 403026986, 403026994, 403027025, 403031659, 403031667, 403031675, 403031698, 403031706, 403031737, 403040311, 403040319, 403040327, 403040350, 403040358, 403040389, 403248091, 403248099, 403248107, 403248128, 403248130, 403248136, 403248138, 403248144, 403248167, 403248169, 403248175, 403248206, 403250169, 403250177, 403250185, 403250208, 403250216, 403250247, 403250996, 403254881, 403254889, 403254897, 403254920, 403254928, 403254959, 403334425, 403334433, 403334441, 403334464, 403334472, 403334503, 403466335, 403466343, 403466372, 403466374, 403466380, 403466409, 403466411, 403466417, 403466448, 403468413, 403468421, 403468450, 403468452, 403468458, 403468489, 403469813, 403470491, 403470499, 403470530, 403473125, 403473133, 403473162, 403473164, 403473170, 403473201, 403475203, 403475211, 403475242, 403479915, 403479923, 403479954, 403509353, 403509361, 403509369, 403509390, 403509392, 403509398, 403509400, 403509406, 403509427, 403509429, 403509431, 403509435, 403509437, 403509443, 403509466, 403509468, 403509474, 403509505, 403511431, 403511439, 403511447, 403511468, 403511470, 403511476, 403511478, 403511484, 403511507, 403511509, 403511515, 403511546, 403512831, 403512839, 403512870, 403513509, 403513517, 403513525, 403513548, 403513556, 403513587, 403516143, 403516151, 403516159, 403516180, 403516182, 403516188, 403516190, 403516196, 403516219, 403516221, 403516227, 403516258, 403518221, 403518229, 403518237, 403518260, 403518268, 403518299, 403522933, 403522941, 403522949, 403522972, 403522980, 403523011, 403557647, 403557655, 403557663, 403557686, 403557694, 403557725, 403775891, 403775899, 403775928, 403775930, 403775936, 403775967, 403777969, 403777977, 403778008, 403782681, 403782689, 403782720, 403818909, 403818917, 403818925, 403818946, 403818948, 403818954, 403818956, 403818962, 403818985, 403818987, 403818993, 403819024, 403820987, 403820995, 403821003, 403821026, 403821034, 403821065, 403825699, 403825707, 403825715, 403825738, 403825746, 403825777, 404016628, 404016665, 404018706, 404023418, 404029165, 404029173, 404029181, 404029202, 404029204, 404029210, 404029212, 404029218, 404029241, 404029243, 404029249, 404029280, 404031243, 404031251, 404031259, 404031282, 404031290, 404031321, 404032070, 404035955, 404035963, 404035971, 404035994, 404036002, 404036033, 404085447, 404085455, 404085486, 404100057, 404100065, 404100073, 404100094, 404100096, 404100102, 404100104, 404100110, 404100133, 404100135, 404100141, 404100172, 404102135, 404102143, 404102151, 404102174, 404102182, 404102213, 404106847, 404106855, 404106863, 404106886, 404106894, 404106925, 404115499, 404115507, 404115515, 404115538, 404115546, 404115577, 404128465, 404128473, 404128481, 404128504, 404128512, 404128543, 404326184, 404338721, 404338729, 404338737, 404338760, 404338768, 404338799, 404409613, 404409621, 404409629, 404409652, 404409660, 404409691, 404473571, 404473579, 404473587, 404473610, 404473618, 404473649, 404556965, 404556973, 404557002, 404557004, 404557010, 404557041, 404559043, 404559051, 404559082, 404563755, 404563763, 404563794, 404599983, 404599991, 404599999, 404600020, 404600022, 404600028, 404600030, 404600036, 404600059, 404600061, 404600067, 404600098, 404602061, 404602069, 404602077, 404602100, 404602108, 404602139, 404606773, 404606781, 404606789, 404606812, 404606820, 404606851, 404808751, 404808759, 404808790, 404851769, 404851777, 404851785, 404851808, 404851816, 404851847, 404866521, 404866529, 404866560, 404909539, 404909547, 404909555, 404909578, 404909586, 404909617, 404987600, 405107258, 405119795, 405119803, 405119811, 405119834, 405119842, 405119873, 405190687, 405190695, 405190703, 405190726, 405190734, 405190765, 405336551, 405379569, 405379577, 405379608, 405462425, 405462433, 405462441, 405462462, 405462464, 405462470, 405462472, 405462478, 405462501, 405462503, 405462509, 405462540, 405464503, 405464511, 405464519, 405464542, 405464550, 405464581, 405465330, 405469215, 405469223, 405469231, 405469254, 405469262, 405469293, 405548759, 405548767, 405548775, 405548798, 405548806, 405548837, 405647595, 405647603, 405647634, 405690613, 405690621, 405690629, 405690652, 405690660, 405690691, 405771981, 405771989, 405771997, 405772020, 405772028, 405772059, 405907369, 405950387, 405950395, 405950426, 405990225, 405990233, 405990262, 405990264, 405990270, 405990301, 405992303, 405992311, 405992342, 405997015, 405997023, 405997054, 406033243, 406033251, 406033259, 406033280, 406033282, 406033288, 406033290, 406033296, 406033319, 406033321, 406033327, 406033358, 406035321, 406035329, 406035337, 406035360, 406035368, 406035399, 406040033, 406040041, 406040049, 406040072, 406040080, 406040111, 406299781, 406299789, 406299820, 406342799, 406342807, 406342815, 406342838, 406342846, 406342877, 406454184, 406454221, 406456262, 406460974, 406478187, 406537613, 406537621, 406537629, 406537650, 406537652, 406537658, 406537660, 406537666, 406537689, 406537691, 406537697, 406537728, 406539691, 406539699, 406539707, 406539730, 406539738, 406539769, 406540518, 406544403, 406544411, 406544419, 406544442, 406544450, 406544481, 406553055, 406553063, 406553071, 406553094, 406553102, 406553133, 406623947, 406623955, 406623963, 406623986, 406623994, 406624025, 406763740, 406847169, 406847177, 406847185, 406847208, 406847216, 406847247, 407025002, 407025039, 407027080, 407031792, 407065413, 407065421, 407065450, 407065452, 407065458, 407065489, 407067491, 407067499, 407067530, 407072203, 407072211, 407072242, 407080855, 407080863, 407080894, 407104953, 407104961, 407104969, 407104977, 407104990, 407104992, 407104998, 407105000, 407105006, 407105008, 407105014, 407105027, 407105029, 407105031, 407105035, 407105037, 407105039, 407105043, 407105045, 407105051, 407105064, 407105066, 407105068, 407105070, 407105072, 407105074, 407105076, 407105080, 407105082, 407105088, 407105103, 407105105, 407105107, 407105111, 407105113, 407105119, 407105142, 407105144, 407105150, 407105181, 407107031, 407107039, 407107047, 407107055, 407107068, 407107070, 407107076, 407107078, 407107084, 407107086, 407107092, 407107105, 407107107, 407107109, 407107113, 407107115, 407107117, 407107121, 407107123, 407107129, 407107144, 407107146, 407107148, 407107152, 407107154, 407107160, 407107183, 407107185, 407107191, 407107222, 407108431, 407108439, 407108447, 407108468, 407108470, 407108476, 407108478, 407108484, 407108507, 407108509, 407108515, 407108546, 407109109, 407109117, 407109125, 407109133, 407109146, 407109148, 407109154, 407109156, 407109162, 407109164, 407109170, 407109185, 407109187, 407109193, 407109195, 407109201, 407109224, 407109226, 407109232, 407109263, 407110509, 407110517, 407110525, 407110548, 407110556, 407110587, 407111187, 407111195, 407111203, 407111211, 407111226, 407111234, 407111242, 407111265, 407111273, 407111304, 407111743, 407111751, 407111759, 407111767, 407111780, 407111782, 407111788, 407111790, 407111796, 407111798, 407111804, 407111817, 407111819, 407111821, 407111825, 407111827, 407111829, 407111833, 407111835, 407111841, 407111856, 407111858, 407111860, 407111864, 407111866, 407111872, 407111895, 407111897, 407111903, 407111934, 407113821, 407113829, 407113837, 407113845, 407113858, 407113860, 407113866, 407113868, 407113874, 407113876, 407113882, 407113897, 407113899, 407113905, 407113907, 407113913, 407113936, 407113938, 407113944, 407113975, 407115221, 407115229, 407115237, 407115260, 407115268, 407115299, 407115899, 407115907, 407115915, 407115923, 407115938, 407115946, 407115954, 407115977, 407115985, 407116016, 407118533, 407118541, 407118549, 407118557, 407118570, 407118572, 407118578, 407118580, 407118586, 407118588, 407118594, 407118609, 407118611, 407118617, 407118619, 407118625, 407118648, 407118650, 407118656, 407118687, 407120611, 407120619, 407120627, 407120635, 407120650, 407120658, 407120666, 407120689, 407120697, 407120728, 407123873, 407123881, 407123889, 407123912, 407123920, 407123951, 407125323, 407125331, 407125339, 407125347, 407125362, 407125370, 407125378, 407125401, 407125409, 407125440, 407334558, 407374969, 407374977, 407375008, 407414509, 407414517, 407414525, 407414533, 407414546, 407414548, 407414554, 407414556, 407414562, 407414564, 407414570, 407414583, 407414585, 407414587, 407414591, 407414593, 407414595, 407414599, 407414601, 407414607, 407414622, 407414624, 407414626, 407414630, 407414632, 407414638, 407414661, 407414663, 407414669, 407414700, 407416587, 407416595, 407416603, 407416611, 407416624, 407416626, 407416632, 407416634, 407416640, 407416642, 407416648, 407416663, 407416665, 407416671, 407416673, 407416679, 407416702, 407416704, 407416710, 407416741, 407417987, 407417995, 407418003, 407418026, 407418034, 407418065, 407418665, 407418673, 407418681, 407418689, 407418704, 407418712, 407418720, 407418743, 407418751, 407418782, 407421299, 407421307, 407421315, 407421323, 407421336, 407421338, 407421344, 407421346, 407421352, 407421354, 407421360, 407421375, 407421377, 407421383, 407421385, 407421391, 407421414, 407421416, 407421422, 407421453, 407423377, 407423385, 407423393, 407423401, 407423416, 407423424, 407423432, 407423455, 407423463, 407423494, 407428089, 407428097, 407428105, 407428113, 407428128, 407428136, 407428144, 407428167, 407428175, 407428206, 407544814, 407615706, 407628243, 407628251, 407628259, 407628282, 407628290, 407628321, 407699135, 407699143, 407699151, 407699174, 407699182, 407699213, 407724065, 407724073, 407724081, 407724089, 407724102, 407724104, 407724110, 407724112, 407724118, 407724120, 407724126, 407724141, 407724143, 407724149, 407724151, 407724157, 407724180, 407724182, 407724188, 407724219, 407726143, 407726151, 407726159, 407726167, 407726182, 407726190, 407726198, 407726221, 407726229, 407726260, 407730855, 407730863, 407730871, 407730879, 407730894, 407730902, 407730910, 407730933, 407730941, 407730972, 407986315, 407986323, 407986331, 407986354, 407986362, 407986393, 408033621, 408033629, 408033637, 408033645, 408033660, 408033668, 408033676, 408033699, 408033707, 408033738, 408115632, 408156043, 408156051, 408156082, 408195583, 408195591, 408195599, 408195607, 408195620, 408195622, 408195628, 408195630, 408195636, 408195638, 408195644, 408195657, 408195659, 408195661, 408195665, 408195667, 408195669, 408195673, 408195675, 408195681, 408195696, 408195698, 408195700, 408195704, 408195706, 408195712, 408195735, 408195737, 408195743, 408195774, 408197661, 408197669, 408197677, 408197685, 408197698, 408197700, 408197706, 408197708, 408197714, 408197716, 408197722, 408197737, 408197739, 408197745, 408197747, 408197753, 408197776, 408197778, 408197784, 408197815, 408199061, 408199069, 408199077, 408199100, 408199108, 408199139, 408199739, 408199747, 408199755, 408199763, 408199778, 408199786, 408199794, 408199817, 408199825, 408199856, 408202373, 408202381, 408202389, 408202397, 408202410, 408202412, 408202418, 408202420, 408202426, 408202428, 408202434, 408202449, 408202451, 408202457, 408202459, 408202465, 408202488, 408202490, 408202496, 408202527, 408204451, 408204459, 408204467, 408204475, 408204490, 408204498, 408204506, 408204529, 408204537, 408204568, 408209163, 408209171, 408209179, 408209187, 408209202, 408209210, 408209218, 408209241, 408209249, 408209280, 408505139, 408505147, 408505155, 408505163, 408505176, 408505178, 408505184, 408505186, 408505192, 408505194, 408505200, 408505215, 408505217, 408505223, 408505225, 408505231, 408505254, 408505256, 408505262, 408505293, 408507217, 408507225, 408507233, 408507241, 408507256, 408507264, 408507272, 408507295, 408507303, 408507334, 408511929, 408511937, 408511945, 408511953, 408511968, 408511976, 408511984, 408512007, 408512015, 408512046, 408514115, 408514123, 408514154, 408557133, 408557141, 408557149, 408557172, 408557180, 408557211, 408607203, 408607211, 408607242, 408814695, 408814703, 408814711, 408814719, 408814734, 408814742, 408814750, 408814773, 408814781, 408814812, 408975169, 408975177, 408975185, 408975206, 408975208, 408975214, 408975216, 408975222, 408975245, 408975247, 408975253, 408975284, 408977247, 408977255, 408977263, 408977286, 408977294, 408977325, 408978074, 408981959, 408981967, 408981975, 408981998, 408982006, 408982037, 409061503, 409061511, 409061519, 409061542, 409061550, 409061581, 409284725, 409284733, 409284741, 409284764, 409284772, 409284803, 409286213, 409286221, 409286229, 409286237, 409286250, 409286252, 409286258, 409286260, 409286266, 409286268, 409286274, 409286289, 409286291, 409286297, 409286299, 409286305, 409286328, 409286330, 409286336, 409286367, 409288291, 409288299, 409288307, 409288315, 409288330, 409288338, 409288346, 409288369, 409288377, 409288408, 409293003, 409293011, 409293019, 409293027, 409293042, 409293050, 409293058, 409293081, 409293089, 409293120, 409502969, 409502977, 409503006, 409503008, 409503014, 409503045, 409505047, 409505055, 409505086, 409509759, 409509767, 409509798, 409545987, 409545995, 409546003, 409546024, 409546026, 409546032, 409546034, 409546040, 409546063, 409546065, 409546071, 409546102, 409548065, 409548073, 409548081, 409548104, 409548112, 409548143, 409548892, 409552777, 409552785, 409552793, 409552816, 409552824, 409552855, 409589303, 409589311, 409589342, 409595769, 409595777, 409595785, 409595793, 409595808, 409595816, 409595824, 409595847, 409595855, 409595886, 409628843, 409628851, 409628859, 409628867, 409628880, 409628882, 409628888, 409628890, 409628896, 409628898, 409628904, 409628917, 409628919, 409628921, 409628925, 409628927, 409628929, 409628933, 409628935, 409628941, 409628956, 409628958, 409628960, 409628964, 409628966, 409628972, 409628995, 409628997, 409629003, 409629034, 409630921, 409630929, 409630937, 409630945, 409630958, 409630960, 409630966, 409630968, 409630974, 409630976, 409630982, 409630997, 409630999, 409631005, 409631007, 409631013, 409631036, 409631038, 409631044, 409631075, 409632321, 409632329, 409632337, 409632360, 409632368, 409632399, 409632999, 409633007, 409633015, 409633023, 409633038, 409633046, 409633054, 409633077, 409633085, 409633116, 409635633, 409635641, 409635649, 409635657, 409635670, 409635672, 409635678, 409635680, 409635686, 409635688, 409635694, 409635709, 409635711, 409635717, 409635719, 409635725, 409635748, 409635750, 409635756, 409635787, 409637711, 409637719, 409637727, 409637735, 409637750, 409637758, 409637766, 409637789, 409637797, 409637828, 409642423, 409642431, 409642439, 409642447, 409642462, 409642470, 409642478, 409642501, 409642509, 409642540, 409812525, 409812533, 409812564, 409855543, 409855551, 409855559, 409855582, 409855590, 409855621, 409938399, 409938407, 409938415, 409938423, 409938436, 409938438, 409938444, 409938446, 409938452, 409938454, 409938460, 409938475, 409938477, 409938483, 409938485, 409938491, 409938514, 409938516, 409938522, 409938553, 409940477, 409940485, 409940493, 409940501, 409940516, 409940524, 409940532, 409940555, 409940563, 409940594, 409945189, 409945197, 409945205, 409945213, 409945228, 409945236, 409945244, 409945267, 409945275, 409945306, 410030769, 410030806, 410032847, 410037559, 410053262, 410065799, 410065807, 410065815, 410065838, 410065846, 410065877, 410073787, 410073795, 410073824, 410073826, 410073832, 410073863, 410075865, 410075873, 410075904, 410080577, 410080585, 410080616, 410136691, 410136699, 410136707, 410136730, 410136738, 410136769, 410247955, 410247963, 410247971, 410247979, 410247994, 410248002, 410248010, 410248033, 410248041, 410248072, 410340325, 410376843, 410376851, 410376859, 410376867, 410376882, 410376890, 410376898, 410376921, 410376929, 410376960, 410383343, 410383351, 410383382, 410577584, 410577621, 410577658, 410579662, 410579699, 410581740, 410584374, 410584411, 410586452, 410591164, 410593599, 410593607, 410593638, 410620602, 410620610, 410620639, 410620641, 410620647, 410620676, 410620678, 410620684, 410620715, 410622680, 410622688, 410622717, 410622719, 410622725, 410622756, 410624080, 410624758, 410624766, 410624797, 410627392, 410627400, 410627429, 410627431, 410627437, 410627468, 410629470, 410629478, 410629509, 410634182, 410634190, 410634221, 410636617, 410636625, 410636633, 410636656, 410636664, 410636695, 410664491, 410664499, 410664530, 410704031, 410704039, 410704047, 410704055, 410704068, 410704070, 410704076, 410704078, 410704084, 410704086, 410704092, 410704105, 410704107, 410704109, 410704113, 410704115, 410704117, 410704121, 410704123, 410704129, 410704144, 410704146, 410704148, 410704152, 410704154, 410704160, 410704183, 410704185, 410704191, 410704222, 410706109, 410706117, 410706125, 410706133, 410706146, 410706148, 410706154, 410706156, 410706162, 410706164, 410706170, 410706185, 410706187, 410706193, 410706195, 410706201, 410706224, 410706226, 410706232, 410706263, 410707509, 410707517, 410707525, 410707548, 410707556, 410707587, 410708187, 410708195, 410708203, 410708211, 410708226, 410708234, 410708242, 410708265, 410708273, 410708304, 410710821, 410710829, 410710837, 410710845, 410710858, 410710860, 410710866, 410710868, 410710874, 410710876, 410710882, 410710897, 410710899, 410710905, 410710907, 410710913, 410710936, 410710938, 410710944, 410710975, 410712899, 410712907, 410712915, 410712923, 410712938, 410712946, 410712954, 410712977, 410712985, 410713016, 410717611, 410717619, 410717627, 410717635, 410717650, 410717658, 410717666, 410717689, 410717697, 410717728, 410719473, 410719481, 410719489, 410719497, 410719510, 410719512, 410719518, 410719520, 410719526, 410719528, 410719534, 410719549, 410719551, 410719557, 410719559, 410719565, 410719588, 410719590, 410719596, 410719627, 410721551, 410721559, 410721567, 410721575, 410721590, 410721598, 410721606, 410721629, 410721637, 410721668, 410726263, 410726271, 410726279, 410726287, 410726302, 410726310, 410726318, 410726341, 410726349, 410726380, 410887140, 410887177, 410889218, 410893930, 410930158, 410930166, 410930195, 410930197, 410930203, 410930234, 410932236, 410932244, 410932275, 410936948, 410936956, 410936987, 411013587, 411013595, 411013603, 411013611, 411013624, 411013626, 411013632, 411013634, 411013640, 411013642, 411013648, 411013663, 411013665, 411013671, 411013673, 411013679, 411013702, 411013704, 411013710, 411013741, 411015665, 411015673, 411015681, 411015689, 411015704, 411015712, 411015720, 411015743, 411015751, 411015782, 411020377, 411020385, 411020393, 411020401, 411020416, 411020424, 411020432, 411020455, 411020463, 411020494, 411029029, 411029037, 411029045, 411029053, 411029068, 411029076, 411029084, 411029107, 411029115, 411029146, 411121399, 411164417, 411164425, 411164456, 411196696, 411239714, 411239722, 411239753, 411323143, 411323151, 411323159, 411323167, 411323182, 411323190, 411323198, 411323221, 411323229, 411323260, 411499059, 411499067, 411499075, 411499098, 411499106, 411499137, 411668214, 411668251, 411670292, 411675004, 411711232, 411711240, 411711269, 411711271, 411711277, 411711308, 411713310, 411713318, 411713349, 411718022, 411718030, 411718061, 411794661, 411794669, 411794677, 411794685, 411794698, 411794700, 411794706, 411794708, 411794714, 411794716, 411794722, 411794737, 411794739, 411794745, 411794747, 411794753, 411794776, 411794778, 411794784, 411794815, 411796739, 411796747, 411796755, 411796763, 411796778, 411796786, 411796794, 411796817, 411796825, 411796856, 411801451, 411801459, 411801467, 411801475, 411801490, 411801498, 411801506, 411801529, 411801537, 411801568, 411810103, 411810111, 411810119, 411810127, 411810142, 411810150, 411810158, 411810181, 411810189, 411810220, 411977770, 412020788, 412020796, 412020827, 412026859, 412026867, 412026898, 412069877, 412069885, 412069893, 412069916, 412069924, 412069955, 412104217, 412104225, 412104233, 412104241, 412104256, 412104264, 412104272, 412104295, 412104303, 412104334, 412152733, 412152741, 412152749, 412152757, 412152770, 412152772, 412152778, 412152780, 412152786, 412152788, 412152794, 412152809, 412152811, 412152817, 412152819, 412152825, 412152848, 412152850, 412152856, 412152887, 412154811, 412154819, 412154827, 412154835, 412154850, 412154858, 412154866, 412154889, 412154897, 412154928, 412159523, 412159531, 412159539, 412159547, 412159562, 412159570, 412159578, 412159601, 412159609, 412159640, 412462289, 412462297, 412462305, 412462313, 412462328, 412462336, 412462344, 412462367, 412462375, 412462406, 412490818, 412554659, 412574247, 412574255, 412574263, 412574286, 412574294, 412574325, 412597677, 412597685, 412597716, 412758844, 412801862, 412801870, 412801901, 412885291, 412885299, 412885307, 412885315, 412885330, 412885338, 412885346, 412885369, 412885377, 412885408, 413061636, 413101474, 413101511, 413102047, 413102055, 413102086, 413103552, 413108264, 413141587, 413141595, 413141603, 413141611, 413141624, 413141626, 413141632, 413141634, 413141640, 413141642, 413141648, 413141661, 413141663, 413141665, 413141669, 413141671, 413141673, 413141677, 413141679, 413141685, 413141700, 413141702, 413141704, 413141708, 413141710, 413141716, 413141739, 413141741, 413141747, 413141778, 413143665, 413143673, 413143681, 413143689, 413143702, 413143704, 413143710, 413143712, 413143718, 413143720, 413143726, 413143741, 413143743, 413143749, 413143751, 413143757, 413143780, 413143782, 413143788, 413143819, 413144492, 413144500, 413144529, 413144531, 413144537, 413144568, 413145065, 413145073, 413145081, 413145104, 413145112, 413145143, 413145743, 413145751, 413145759, 413145767, 413145782, 413145790, 413145798, 413145821, 413145829, 413145860, 413146570, 413146578, 413146609, 413148377, 413148385, 413148393, 413148401, 413148414, 413148416, 413148422, 413148424, 413148430, 413148432, 413148438, 413148453, 413148455, 413148461, 413148463, 413148469, 413148492, 413148494, 413148500, 413148531, 413150455, 413150463, 413150471, 413150479, 413150494, 413150502, 413150510, 413150533, 413150541, 413150572, 413151282, 413151290, 413151321, 413155167, 413155175, 413155183, 413155191, 413155206, 413155214, 413155222, 413155245, 413155253, 413155284, 413227921, 413227929, 413227937, 413227945, 413227958, 413227960, 413227966, 413227968, 413227974, 413227976, 413227982, 413227997, 413227999, 413228005, 413228007, 413228013, 413228036, 413228038, 413228044, 413228075, 413229999, 413230007, 413230015, 413230023, 413230038, 413230046, 413230054, 413230077, 413230085, 413230116, 413234711, 413234719, 413234727, 413234735, 413234750, 413234758, 413234766, 413234789, 413234797, 413234828, 413243363, 413243371, 413243379, 413243387, 413243402, 413243410, 413243418, 413243441, 413243449, 413243480, 413411030, 413451143, 413451151, 413451159, 413451167, 413451180, 413451182, 413451188, 413451190, 413451196, 413451198, 413451204, 413451219, 413451221, 413451227, 413451229, 413451235, 413451258, 413451260, 413451266, 413451297, 413453221, 413453229, 413453237, 413453245, 413453260, 413453268, 413453276, 413453299, 413453307, 413453338, 413454048, 413454056, 413454087, 413457933, 413457941, 413457949, 413457957, 413457972, 413457980, 413457988, 413458011, 413458019, 413458050, 413537477, 413537485, 413537493, 413537501, 413537516, 413537524, 413537532, 413537555, 413537563, 413537594, 413629847, 413669387, 413669395, 413669403, 413669424, 413669426, 413669432, 413669434, 413669440, 413669461, 413669463, 413669465, 413669469, 413669471, 413669477, 413669500, 413669502, 413669508, 413669539, 413671465, 413671473, 413671481, 413671502, 413671504, 413671510, 413671512, 413671518, 413671541, 413671543, 413671549, 413671580, 413672865, 413672873, 413672904, 413673543, 413673551, 413673559, 413673582, 413673590, 413673621, 413676177, 413676185, 413676193, 413676214, 413676216, 413676222, 413676224, 413676230, 413676253, 413676255, 413676261, 413676292, 413678255, 413678263, 413678271, 413678294, 413678302, 413678333, 413682967, 413682975, 413682983, 413683006, 413683014, 413683045, 413760699, 413760707, 413760715, 413760723, 413760738, 413760746, 413760754, 413760777, 413760785, 413760816, 413978943, 413978951, 413978959, 413978980, 413978982, 413978988, 413978990, 413978996, 413979019, 413979021, 413979027, 413979058, 413981021, 413981029, 413981037, 413981060, 413981068, 413981099, 413985733, 413985741, 413985749, 413985772, 413985780, 413985811, 414176662, 414176699, 414178740, 414183452, 414192104, 414219680, 414219688, 414219717, 414219719, 414219725, 414219756, 414221758, 414221766, 414221797, 414226470, 414226478, 414226509, 414232217, 414232225, 414232233, 414232241, 414232254, 414232256, 414232262, 414232264, 414232270, 414232272, 414232278, 414232293, 414232295, 414232301, 414232303, 414232309, 414232332, 414232334, 414232340, 414232371, 414234295, 414234303, 414234311, 414234319, 414234334, 414234342, 414234350, 414234373, 414234381, 414234412, 414235122, 414235130, 414235161, 414239007, 414239015, 414239023, 414239031, 414239046, 414239054, 414239062, 414239085, 414239093, 414239124, 414288499, 414288507, 414288515, 414288538, 414288546, 414288577, 414303109, 414303117, 414303125, 414303133, 414303146, 414303148, 414303154, 414303156, 414303162, 414303164, 414303170, 414303185, 414303187, 414303193, 414303195, 414303201, 414303224, 414303226, 414303232, 414303263, 414305187, 414305195, 414305203, 414305211, 414305226, 414305234, 414305242, 414305265, 414305273, 414305304, 414309899, 414309907, 414309915, 414309923, 414309938, 414309946, 414309954, 414309977, 414309985, 414310016, 414318551, 414318559, 414318567, 414318575, 414318590, 414318598, 414318606, 414318629, 414318637, 414318668, 414486218, 414529236, 414529244, 414529275, 414541773, 414541781, 414541789, 414541797, 414541812, 414541820, 414541828, 414541851, 414541859, 414541890, 414612665, 414612673, 414612681, 414612689, 414612704, 414612712, 414612720, 414612743, 414612751, 414612782, 414676623, 414676631, 414676639, 414676647, 414676662, 414676670, 414676678, 414676701, 414676709, 414676740, 414760017, 414760025, 414760033, 414760054, 414760056, 414760062, 414760064, 414760070, 414760093, 414760095, 414760101, 414760132, 414762095, 414762103, 414762111, 414762134, 414762142, 414762173, 414766807, 414766815, 414766823, 414766846, 414766854, 414766885, 415011803, 415011811, 415011819, 415011842, 415011850, 415011881, 415069573, 415069581, 415069589, 415069612, 415069620, 415069651, 415267292, 415310310, 415310318, 415310349, 415322847, 415322855, 415322863, 415322871, 415322886, 415322894, 415322902, 415322925, 415322933, 415322964, 415393739, 415393747, 415393755, 415393763, 415393778, 415393786, 415393794, 415393817, 415393825, 415393856, 415539603, 415539611, 415539642, 415582621, 415582629, 415582637, 415582660, 415582668, 415582699, 415625364, 415665477, 415665485, 415665493, 415665501, 415665514, 415665516, 415665522, 415665524, 415665530, 415665532, 415665538, 415665553, 415665555, 415665561, 415665563, 415665569, 415665592, 415665594, 415665600, 415665631, 415667555, 415667563, 415667571, 415667579, 415667594, 415667602, 415667610, 415667633, 415667641, 415667672, 415668382, 415668390, 415668421, 415672267, 415672275, 415672283, 415672291, 415672306, 415672314, 415672322, 415672345, 415672353, 415672384, 415718452, 415751811, 415751819, 415751827, 415751835, 415751850, 415751858, 415751866, 415751889, 415751897, 415751928, 415850647, 415850655, 415850663, 415850686, 415850694, 415850725, 415975033, 415975041, 415975049, 415975057, 415975072, 415975080, 415975088, 415975111, 415975119, 415975150, 416067403, 416110421, 416110429, 416110460, 416193277, 416193285, 416193293, 416193314, 416193316, 416193322, 416193324, 416193330, 416193353, 416193355, 416193361, 416193392, 416195355, 416195363, 416195371, 416195394, 416195402, 416195433, 416200067, 416200075, 416200083, 416200106, 416200114, 416200145, 416502833, 416502841, 416502849, 416502872, 416502880, 416502911, 416614218, 416614255, 416616296, 416621008, 416638221, 416657236, 416657244, 416657273, 416657275, 416657281, 416657312, 416659314, 416659322, 416659353, 416664026, 416664034, 416664065, 416681239, 416681247, 416681278, 416700552, 416740665, 416740673, 416740681, 416740689, 416740702, 416740704, 416740710, 416740712, 416740718, 416740720, 416740726, 416740741, 416740743, 416740749, 416740751, 416740757, 416740780, 416740782, 416740788, 416740819, 416742743, 416742751, 416742759, 416742767, 416742782, 416742790, 416742798, 416742821, 416742829, 416742860, 416743570, 416743578, 416743609, 416747455, 416747463, 416747471, 416747479, 416747494, 416747502, 416747510, 416747533, 416747541, 416747572, 416756107, 416756115, 416756123, 416756131, 416756146, 416756154, 416756162, 416756185, 416756193, 416756224, 416826999, 416827007, 416827015, 416827023, 416827038, 416827046, 416827054, 416827077, 416827085, 416827116, 416923774, 416966792, 416966800, 416966831, 417050221, 417050229, 417050237, 417050245, 417050260, 417050268, 417050276, 417050299, 417050307, 417050338, 417185036, 417185073, 417187114, 417191826, 417268465, 417268473, 417268481, 417268502, 417268504, 417268510, 417268512, 417268518, 417268541, 417268543, 417268549, 417268580, 417270543, 417270551, 417270559, 417270582, 417270590, 417270621, 417275255, 417275263, 417275271, 417275294, 417275302, 417275333, 417283907, 417283915, 417283923, 417283946, 417283954, 417283985, 417494592, 417578021, 417578029, 417578037, 417578060, 417578068, 417578099, 417704848, 417747866, 417747874, 417747905, 417775740, 417818758, 417818766, 417818797, 417831295, 417831303, 417831311, 417831319, 417831334, 417831342, 417831350, 417831373, 417831381, 417831412, 417902187, 417902195, 417902203, 417902211, 417902226, 417902234, 417902242, 417902265, 417902273, 417902304, 418189367, 418189375, 418189383, 418189391, 418189406, 418189414, 418189422, 418189445, 418189453, 418189484, 418275666, 418359095, 418359103, 418359111, 418359134, 418359142, 418359173, 418717167, 418717175, 418717183, 418717206, 418717214, 418717245, 419138108, 419178221, 419178229, 419178237, 419178245, 419178258, 419178260, 419178266, 419178268, 419178274, 419178276, 419178282, 419178297, 419178299, 419178305, 419178307, 419178313, 419178336, 419178338, 419178344, 419178375, 419180299, 419180307, 419180315, 419180323, 419180338, 419180346, 419180354, 419180377, 419180385, 419180416, 419181126, 419181134, 419181165, 419185011, 419185019, 419185027, 419185035, 419185050, 419185058, 419185066, 419185089, 419185097, 419185128, 419264555, 419264563, 419264571, 419264579, 419264594, 419264602, 419264610, 419264633, 419264641, 419264672, 419487777, 419487785, 419487793, 419487801, 419487816, 419487824, 419487832, 419487855, 419487863, 419487894, 419706021, 419706029, 419706037, 419706058, 419706060, 419706066, 419706068, 419706074, 419706097, 419706099, 419706105, 419706136, 419708099, 419708107, 419708115, 419708138, 419708146, 419708177, 419708926, 419712811, 419712819, 419712827, 419712850, 419712858, 419712889, 419792355, 419792363, 419792371, 419792394, 419792402, 419792433, 420015577, 420015585, 420015593, 420015616, 420015624, 420015655, 420213296, 420233821, 420233829, 420233858, 420233860, 420233866, 420233897, 420235899, 420235907, 420235938, 420240611, 420240619, 420240650, 420256314, 420256322, 420256353, 420268851, 420268859, 420268867, 420268875, 420268890, 420268898, 420268906, 420268929, 420268937, 420268968, 420276839, 420276847, 420276855, 420276876, 420276878, 420276884, 420276886, 420276892, 420276915, 420276917, 420276923, 420276954, 420278917, 420278925, 420278933, 420278956, 420278964, 420278995, 420283629, 420283637, 420283645, 420283668, 420283676, 420283707, 420339743, 420339751, 420339759, 420339767, 420339782, 420339790, 420339798, 420339821, 420339829, 420339860, 420543377, 420543385, 420543416, 420586395, 420586403, 420586411, 420586434, 420586442, 420586473, 420780636, 420780644, 420780673, 420780675, 420780681, 420780710, 420780712, 420780718, 420780749, 420782714, 420782722, 420782751, 420782753, 420782759, 420782790, 420784114, 420784792, 420784800, 420784831, 420787426, 420787434, 420787463, 420787465, 420787471, 420787502, 420789504, 420789512, 420789543, 420794216, 420794224, 420794255, 420796651, 420796659, 420796667, 420796690, 420796698, 420796729, 420867543, 420867551, 420867559, 420867582, 420867590, 420867621, 421090192, 421090200, 421090229, 421090231, 421090237, 421090268, 421092270, 421092278, 421092309, 421096982, 421096990, 421097021, 421324451, 421324459, 421324490, 421367469, 421367477, 421367485, 421367508, 421367516, 421367547, 421399748, 421399756, 421399787, 421702111, 421702119, 421702127, 421702135, 421702150, 421702158, 421702166, 421702189, 421702197, 421702228, 421871266, 421871274, 421871303, 421871305, 421871311, 421871342, 421873344, 421873352, 421873383, 421878056, 421878064, 421878095, 422180822, 422180830, 422180861, 422229911, 422229919, 422229927, 422229950, 422229958, 422229989, 422650852, 422693870, 422693878, 422693909, 422757711, 422757719, 422757750, 422777299, 422777307, 422777315, 422777323, 422777338, 422777346, 422777354, 422777377, 422777385, 422777416, 422800729, 422800737, 422800745, 422800768, 422800776, 422800807, 422961896, 422961904, 422961935, 423221670, 423304526, 423304534, 423304563, 423304565, 423304571, 423304602, 423305099, 423305107, 423305115, 423305138, 423305146, 423305177, 423306604, 423306612, 423306643, 423311316, 423311324, 423311355, 423614082, 423614090, 423614121, 423792488, 423832899, 423832907, 423832938, 423872439, 423872447, 423872455, 423872463, 423872476, 423872478, 423872484, 423872486, 423872492, 423872494, 423872500, 423872513, 423872515, 423872517, 423872521, 423872523, 423872525, 423872529, 423872531, 423872537, 423872552, 423872554, 423872556, 423872560, 423872562, 423872568, 423872591, 423872593, 423872599, 423872630, 423874517, 423874525, 423874533, 423874541, 423874554, 423874556, 423874562, 423874564, 423874570, 423874572, 423874578, 423874593, 423874595, 423874601, 423874603, 423874609, 423874632, 423874634, 423874640, 423874671, 423875917, 423875925, 423875933, 423875956, 423875964, 423875995, 423876595, 423876603, 423876611, 423876619, 423876634, 423876642, 423876650, 423876673, 423876681, 423876712, 423879229, 423879237, 423879245, 423879253, 423879266, 423879268, 423879274, 423879276, 423879282, 423879284, 423879290, 423879305, 423879307, 423879313, 423879315, 423879321, 423879344, 423879346, 423879352, 423879383, 423881307, 423881315, 423881323, 423881331, 423881346, 423881354, 423881362, 423881385, 423881393, 423881424, 423886019, 423886027, 423886035, 423886043, 423886058, 423886066, 423886074, 423886097, 423886105, 423886136, 424181995, 424182003, 424182011, 424182019, 424182032, 424182034, 424182040, 424182042, 424182048, 424182050, 424182056, 424182071, 424182073, 424182079, 424182081, 424182087, 424182110, 424182112, 424182118, 424182149, 424184073, 424184081, 424184089, 424184097, 424184112, 424184120, 424184128, 424184151, 424184159, 424184190, 424188785, 424188793, 424188801, 424188809, 424188824, 424188832, 424188840, 424188863, 424188871, 424188902, 424379714, 424379722, 424379751, 424379753, 424379759, 424379790, 424381792, 424381800, 424381831, 424386504, 424386512, 424386543, 424395156, 424395164, 424395195, 424491551, 424491559, 424491567, 424491575, 424491590, 424491598, 424491606, 424491629, 424491637, 424491668, 424689270, 424689278, 424689309, 424963069, 424963077, 424963085, 424963093, 424963106, 424963108, 424963114, 424963116, 424963122, 424963124, 424963130, 424963145, 424963147, 424963153, 424963155, 424963161, 424963184, 424963186, 424963192, 424963223, 424965147, 424965155, 424965163, 424965171, 424965186, 424965194, 424965202, 424965225, 424965233, 424965264, 424969859, 424969867, 424969875, 424969883, 424969898, 424969906, 424969914, 424969937, 424969945, 424969976, 425214855, 425214863, 425214871, 425214879, 425214894, 425214902, 425214910, 425214933, 425214941, 425214972, 425272625, 425272633, 425272641, 425272649, 425272664, 425272672, 425272680, 425272703, 425272711, 425272742, 425470344, 425470352, 425470383, 425742655, 425742663, 425742671, 425742694, 425742702, 425742733, 425805887, 425805924, 425805961, 425807965, 425808002, 425810043, 425812677, 425812714, 425814755, 425819467, 425828416, 425828424, 425828455, 426053699, 426053707, 426053715, 426053723, 426053738, 426053746, 426053754, 426053777, 426053785, 426053816, 426115443, 426115480, 426117521, 426122233, 426270455, 426270463, 426270494, 426313473, 426313481, 426313489, 426313512, 426313520, 426313551, 426396329, 426396337, 426396345, 426396353, 426396366, 426396368, 426396374, 426396376, 426396382, 426396384, 426396390, 426396405, 426396407, 426396413, 426396415, 426396421, 426396444, 426396446, 426396452, 426396483, 426398407, 426398415, 426398423, 426398431, 426398446, 426398454, 426398462, 426398485, 426398493, 426398524, 426403119, 426403127, 426403135, 426403143, 426403158, 426403166, 426403174, 426403197, 426403205, 426403236, 426424999, 426705885, 426705893, 426705901, 426705909, 426705924, 426705932, 426705940, 426705963, 426705971, 426706002, 426798255, 426817270, 426817278, 426817307, 426817309, 426817315, 426817346, 426819348, 426819356, 426819387, 426824060, 426824068, 426824099, 426841273, 426841281, 426841312, 426896517, 426896554, 426898595, 426903307, 426903604, 426903612, 426903643, 427126826, 427126834, 427126865, 427206073, 427345070, 427345107, 427347148, 427351860, 427388088, 427388096, 427388125, 427388127, 427388133, 427388164, 427390166, 427390174, 427390205, 427394878, 427394886, 427394917, 427471517, 427471525, 427471533, 427471541, 427471554, 427471556, 427471562, 427471564, 427471570, 427471572, 427471578, 427471593, 427471595, 427471601, 427471603, 427471609, 427471632, 427471634, 427471640, 427471671, 427473595, 427473603, 427473611, 427473619, 427473634, 427473642, 427473650, 427473673, 427473681, 427473712, 427478307, 427478315, 427478323, 427478331, 427478346, 427478354, 427478362, 427478385, 427478393, 427478424, 427486959, 427486967, 427486975, 427486983, 427486998, 427487006, 427487014, 427487037, 427487045, 427487076, 427654626, 427697644, 427697652, 427697683, 427781073, 427781081, 427781089, 427781097, 427781112, 427781120, 427781128, 427781151, 427781159, 427781190, 427907900, 427907908, 427907939, 427978792, 427978800, 427978831, 427987147, 428329777, 428329814, 428331855, 428336567, 428435700, 428478718, 428478726, 428478757, 428562147, 428562155, 428562163, 428562171, 428562186, 428562194, 428562202, 428562225, 428562233, 428562264, 428639333, 428920219, 428920227, 428920235, 428920243, 428920258, 428920266, 428920274, 428920297, 428920305, 428920336, 429341160, 429341168, 429341199, 429404965, 429405002, 429407043, 429411755, 429420407, 429714521, 429868960, 429909073, 429909081, 429909089, 429909097, 429909110, 429909112, 429909118, 429909120, 429909126, 429909128, 429909134, 429909149, 429909151, 429909157, 429909159, 429909165, 429909188, 429909190, 429909196, 429909227, 429911151, 429911159, 429911167, 429911175, 429911190, 429911198, 429911206, 429911229, 429911237, 429911268, 429911978, 429911986, 429912017, 429915863, 429915871, 429915879, 429915887, 429915902, 429915910, 429915918, 429915941, 429915949, 429915980, 429995407, 429995415, 429995423, 429995431, 429995446, 429995454, 429995462, 429995485, 429995493, 429995524, 430218629, 430218637, 430218645, 430218653, 430218668, 430218676, 430218684, 430218707, 430218715, 430218746, 430416348, 430416356, 430416387, 430436873, 430436881, 430436889, 430436910, 430436912, 430436918, 430436920, 430436926, 430436949, 430436951, 430436957, 430436988, 430438951, 430438959, 430438967, 430438990, 430438998, 430439029, 430443663, 430443671, 430443679, 430443702, 430443710, 430443741, 430495595, 430746429, 430746437, 430746445, 430746468, 430746476, 430746507, 430853667, 430944148, 430983688, 430983696, 430983704, 430983725, 430983727, 430983733, 430983735, 430983741, 430983762, 430983764, 430983766, 430983770, 430983772, 430983778, 430983801, 430983803, 430983809, 430983840, 430985766, 430985774, 430985782, 430985803, 430985805, 430985811, 430985813, 430985819, 430985842, 430985844, 430985850, 430985881, 430987166, 430987174, 430987205, 430987844, 430987852, 430987860, 430987883, 430987891, 430987922, 430990478, 430990486, 430990494, 430990515, 430990517, 430990523, 430990525, 430990531, 430990554, 430990556, 430990562, 430990593, 430992556, 430992564, 430992572, 430992595, 430992603, 430992634, 430997268, 430997276, 430997284, 430997307, 430997315, 430997346, 430999703, 430999711, 430999719, 430999727, 430999742, 430999750, 430999758, 430999781, 430999789, 430999820, 431070595, 431070603, 431070611, 431070619, 431070634, 431070642, 431070650, 431070673, 431070681, 431070712, 431293244, 431293252, 431293260, 431293281, 431293283, 431293289, 431293291, 431293297, 431293320, 431293322, 431293328, 431293359, 431295322, 431295330, 431295338, 431295361, 431295369, 431295400, 431300034, 431300042, 431300050, 431300073, 431300081, 431300112, 431527503, 431527511, 431527519, 431527542, 431527550, 431527581, 431602800, 431602808, 431602816, 431602839, 431602847, 431602878, 431842521, 431842558, 431844599, 431849311, 431928855, 432074318, 432074326, 432074334, 432074355, 432074357, 432074363, 432074365, 432074371, 432074394, 432074396, 432074402, 432074433, 432076396, 432076404, 432076412, 432076435, 432076443, 432076474, 432081108, 432081116, 432081124, 432081147, 432081155, 432081186, 432152077, 432383874, 432383882, 432383890, 432383913, 432383921, 432383952, 432432963, 432432971, 432432979, 432432987, 432433002, 432433010, 432433018, 432433041, 432433049, 432433080, 432853904, 432853912, 432853943, 432933151, 432960763, 432960771, 432960779, 432960802, 432960810, 432960841, 433004043, 433164948, 433164956, 433164964, 433164987, 433164995, 433165026, 433381704, 433424722, 433424730, 433424761, 433507578, 433507586, 433507594, 433507615, 433507617, 433507623, 433507625, 433507631, 433507654, 433507656, 433507662, 433507693, 433508151, 433508159, 433508167, 433508175, 433508190, 433508198, 433508206, 433508229, 433508237, 433508268, 433509656, 433509664, 433509672, 433509695, 433509703, 433509734, 433514368, 433514376, 433514384, 433514407, 433514415, 433514446, 433817134, 433817142, 433817150, 433817173, 433817181, 433817212, 433952522, 434035951, 434035959, 434035967, 434035990, 434035998, 434036029, 434366411, 434499337, 434499374, 434501415, 434506127, 434582766, 434582774, 434582782, 434582803, 434582805, 434582811, 434582813, 434582819, 434582842, 434582844, 434582850, 434582881, 434584844, 434584852, 434584860, 434584883, 434584891, 434584922, 434589556, 434589564, 434589572, 434589595, 434589603, 434589634, 434598208, 434598216, 434598224, 434598247, 434598255, 434598286, 434808893, 434892322, 434892330, 434892338, 434892361, 434892369, 434892400, 435441599, 435589967, 435673396, 435673404, 435673412, 435673435, 435673443, 435673474, 435945707, 435945715, 435945723, 435945731, 435945746, 435945754, 435945762, 435945785, 435945793, 435945824, 435965921, 435965958, 435965995, 435967999, 435968036, 435970077, 435972711, 435972748, 435974789, 435979501, 436031468, 436031476, 436031484, 436031507, 436031515, 436031546, 436275477, 436275514, 436277555, 436282267, 436473507, 436473515, 436473523, 436473546, 436473554, 436473585, 436585033, 437001307, 437001315, 437001346, 437020322, 437020330, 437020338, 437020359, 437020361, 437020367, 437020369, 437020375, 437020398, 437020400, 437020406, 437020437, 437022400, 437022408, 437022416, 437022439, 437022447, 437022478, 437023227, 437027112, 437027120, 437027128, 437027151, 437027159, 437027190, 437044325, 437044333, 437044341, 437044364, 437044372, 437044403, 437056551, 437056588, 437058629, 437063341, 437106656, 437106664, 437106672, 437106695, 437106703, 437106734, 437329878, 437329886, 437329894, 437329917, 437329925, 437329956, 437366107, 437548122, 437548130, 437548159, 437548161, 437548167, 437548198, 437550200, 437550208, 437550239, 437554912, 437554920, 437554951, 437857678, 437857686, 437857717, 437879155, 438098415, 438110952, 438110960, 438110968, 438110991, 438110999, 438111030, 438147181, 438181844, 438181852, 438181860, 438181883, 438181891, 438181922, 438489811, 438489848, 438491889, 438496601, 438638752, 438638760, 438638791, 438799367, 439544212, 439544220, 439544228, 439544251, 439544259, 439544290, 439564999, 439565036, 439567077, 439571789, 439580441, 439874555, 440072012, 440072020, 440072051, 440535971, 440619400, 440619408, 440619416, 440619439, 440619447, 440619478, 440639925, 440639933, 440639941, 440639949, 440639962, 440639964, 440639970, 440639972, 440639978, 440639980, 440639986, 440640001, 440640003, 440640009, 440640011, 440640017, 440640040, 440640042, 440640048, 440640079, 440642003, 440642011, 440642019, 440642027, 440642042, 440642050, 440642058, 440642081, 440642089, 440642120, 440646715, 440646723, 440646731, 440646739, 440646754, 440646762, 440646770, 440646793, 440646801, 440646832, 440655629, 440949481, 440949489, 440949497, 440949505, 440949520, 440949528, 440949536, 440949559, 440949567, 440949598, 441013701, 441147200, 441147208, 441147239, 441730555, 441730563, 441730571, 441730579, 441730594, 441730602, 441730610, 441730633, 441730641, 441730672, 442002555, 442002592, 442004633, 442009345, 442088889, 442312111, 442573373, 442573410, 442575451, 442580163, 442882929, 443056956, 443056964, 443056972, 443056995, 443057003, 443057034, 443093185, 443163815, 443163823, 443163831, 443163839, 443163854, 443163862, 443163870, 443163893, 443163901, 443163932, 443164077, 443584756, 443584764, 443584795, 443664003, 444112556, 444155574, 444155582, 444155613, 444239003, 444239011, 444239019, 444239027, 444239042, 444239050, 444239058, 444239081, 444239089, 444239120, 444526445, 444659371, 444659408, 444661449, 444666161, 444968927, 445097263, 445601633, 445750001, 446125955, 446125992, 446126029, 446128033, 446128070, 446130111, 446132745, 446132782, 446134823, 446139535, 446168973, 446168981, 446169010, 446169012, 446169018, 446169047, 446169049, 446169055, 446169086, 446171051, 446171059, 446171088, 446171090, 446171096, 446171127, 446172451, 446173129, 446173137, 446173168, 446175763, 446175771, 446175800, 446175802, 446175808, 446175839, 446177841, 446177849, 446177880, 446182553, 446182561, 446182592, 446435511, 446435548, 446437589, 446442301, 446478529, 446478537, 446478566, 446478568, 446478574, 446478605, 446480607, 446480615, 446480646, 446485319, 446485327, 446485358, 446676559, 446676567, 446676575, 446676583, 446676598, 446676606, 446676614, 446676637, 446676645, 446676676, 446745067, 446788085, 446788093, 446788124, 447183261, 447204359, 447204367, 447204375, 447204398, 447204406, 447204437, 447216585, 447216622, 447218663, 447223375, 447259603, 447259611, 447259640, 447259642, 447259648, 447259679, 447261681, 447261689, 447261720, 447266393, 447266401, 447266432, 447526141, 447569159, 447569167, 447569198, 447751174, 447751182, 447751190, 447751211, 447751213, 447751219, 447751221, 447751227, 447751250, 447751252, 447751258, 447751289, 447753252, 447753260, 447753268, 447753291, 447753299, 447753330, 447757964, 447757972, 447757980, 447758003, 447758011, 447758042, 448039189, 448060730, 448060738, 448060746, 448060769, 448060777, 448060808, 448258449, 448307215, 448350233, 448350241, 448350272, 448610007, 448649845, 448649882, 448651923, 448656635, 448692863, 448692871, 448692900, 448692902, 448692908, 448692939, 448694941, 448694949, 448694980, 448699653, 448699661, 448699692, 448841804, 448841812, 448841820, 448841843, 448841851, 448841882, 448959401, 449002419, 449002427, 449002458, 449725033, 449725070, 449727111, 449731823, 449740475, 449768051, 449768059, 449768088, 449768090, 449768096, 449768127, 449770129, 449770137, 449770168, 449774841, 449774849, 449774880, 449783493, 449783501, 449783532, 450034589, 450077607, 450077615, 450077646, 450275064, 450275072, 450275080, 450275103, 450275111, 450275142, 450696005, 450815663, 450858681, 450858689, 450858720, 451173735, 451216753, 451216761, 451216792, 451266823, 451350252, 451350260, 451350268, 451350291, 451350299, 451350330, 452162589, 452162626, 452164667, 452169379, 452205607, 452205615, 452205644, 452205646, 452205652, 452205683, 452207685, 452207693, 452207724, 452212397, 452212405, 452212436, 452248923, 452291941, 452291949, 452291980, 452472145, 452515163, 452515171, 452515202, 452733407, 452733444, 452735485, 452740197, 453042963, 453253219, 453296237, 453296245, 453296276, 453324111, 453367129, 453367137, 453367168, 453787808, 453787816, 453787824, 453787847, 453787855, 453787886, 453824037, 454315608, 454315616, 454315647, 454686479, 454729497, 454729505, 454729536, 454862423, 454862431, 454862460, 454862462, 454862468, 454862499, 454864501, 454864509, 454864540, 454869213, 454869221, 454869252, 455171979, 455171987, 455172018, 455257297, 455761667, 455804685, 455804693, 455804724, 455953053, 455953061, 455953092, 456285989, 456286026, 456286063, 456288067, 456288104, 456290145, 456292779, 456292816, 456294857, 456299569, 456329007, 456329015, 456329044, 456329046, 456329052, 456329081, 456329083, 456329089, 456329120, 456331085, 456331093, 456331122, 456331124, 456331130, 456331161, 456332485, 456333163, 456333171, 456333202, 456335797, 456335805, 456335834, 456335836, 456335842, 456335873, 456337875, 456337883, 456337914, 456342587, 456342595, 456342626, 456595545, 456595582, 456597623, 456602335, 456638563, 456638571, 456638600, 456638602, 456638608, 456638639, 456640641, 456640649, 456640680, 456645353, 456645361, 456645392, 456905101, 456948119, 456948127, 456948158, 457376619, 457376656, 457378697, 457383409, 457386313, 457386321, 457386352, 457407411, 457407419, 457407427, 457407435, 457407450, 457407458, 457407466, 457407489, 457407497, 457407528, 457419637, 457419645, 457419674, 457419676, 457419682, 457419713, 457421715, 457421723, 457421754, 457426427, 457426435, 457426466, 457686175, 457729193, 457729201, 457729232, 458199223, 458242241, 458242249, 458242280, 458461501, 458461509, 458461540, 458467249, 458510267, 458510275, 458510306, 458770041, 458809879, 458809916, 458811957, 458816669, 458852897, 458852905, 458852934, 458852936, 458852942, 458852973, 458854975, 458854983, 458855014, 458859687, 458859695, 458859726, 459119435, 459162453, 459162461, 459162492, 459340859, 459885067, 459885104, 459887145, 459891857, 459900509, 459928085, 459928093, 459928122, 459928124, 459928130, 459928161, 459930163, 459930171, 459930202, 459934875, 459934883, 459934914, 459943527, 459943535, 459943566, 460194623, 460237641, 460237649, 460237680, 460899057, 460899065, 460899096, 460975697, 461018715, 461018723, 461018754, 461333769, 461354258, 461354295, 461354332, 461356336, 461356373, 461358414, 461361048, 461361085, 461363126, 461367838, 461376787, 461376795, 461376826, 461426857, 461663814, 461663851, 461665892, 461670604, 461973370, 462322623, 462322660, 462324701, 462329413, 462365641, 462365649, 462365678, 462365680, 462365686, 462365717, 462367719, 462367727, 462367758, 462372431, 462372439, 462372470, 462408957, 462444888, 462444925, 462446966, 462451678, 462451975, 462451983, 462452014, 462632179, 462675197, 462675205, 462675236, 462754444, 462893441, 462893478, 462895519, 462900231, 462936459, 462936467, 462936496, 462936498, 462936504, 462936535, 462938537, 462938545, 462938576, 462943249, 462943257, 462943288, 463202997, 463246015, 463246023, 463246054, 463413253, 463456271, 463456279, 463456310, 463484145, 463527163, 463527171, 463527202, 463535518, 463878148, 463878185, 463880226, 463884938, 463984071, 464027089, 464027097, 464027128, 464187704, 464518660, 464518668, 464518676, 464518699, 464518707, 464518738, 464846513, 464889531, 464889539, 464889570, 464953336, 464953373, 464955414, 464960126, 464968778, 465262892, 465417331, 465460349, 465460357, 465460388, 465921701, 465964719, 465964727, 465964758, 466043966, 466402038, 466489041, 466489049, 466489078, 466489080, 466489086, 466489115, 466489117, 466489123, 466489154, 466491119, 466491127, 466491156, 466491158, 466491164, 466491195, 466492519, 466493197, 466493205, 466493236, 466495831, 466495839, 466495868, 466495870, 466495876, 466495907, 466497909, 466497917, 466497948, 466502621, 466502629, 466502660, 466532059, 466532067, 466532075, 466532096, 466532098, 466532104, 466532106, 466532112, 466532133, 466532135, 466532137, 466532141, 466532143, 466532149, 466532172, 466532174, 466532180, 466532211, 466534137, 466534145, 466534153, 466534174, 466534176, 466534182, 466534184, 466534190, 466534213, 466534215, 466534221, 466534252, 466535537, 466535545, 466535576, 466536215, 466536223, 466536231, 466536254, 466536262, 466536293, 466538849, 466538857, 466538865, 466538886, 466538888, 466538894, 466538896, 466538902, 466538925, 466538927, 466538933, 466538964, 466540927, 466540935, 466540943, 466540966, 466540974, 466541005, 466545639, 466545647, 466545655, 466545678, 466545686, 466545717, 466798597, 466798605, 466798634, 466798636, 466798642, 466798673, 466800675, 466800683, 466800714, 466805387, 466805395, 466805426, 466841615, 466841623, 466841631, 466841652, 466841654, 466841660, 466841662, 466841668, 466841691, 466841693, 466841699, 466841730, 466843693, 466843701, 466843709, 466843732, 466843740, 466843771, 466848405, 466848413, 466848421, 466848444, 466848452, 466848483, 467108153, 467108161, 467108192, 467151171, 467151179, 467151187, 467151210, 467151218, 467151249, 467390892, 467390929, 467392970, 467397682, 467477226, 467579671, 467579679, 467579708, 467579710, 467579716, 467579747, 467581749, 467581757, 467581788, 467586461, 467586469, 467586500, 467622689, 467622697, 467622705, 467622726, 467622728, 467622734, 467622736, 467622742, 467622765, 467622767, 467622773, 467622804, 467624767, 467624775, 467624783, 467624806, 467624814, 467624845, 467629479, 467629487, 467629495, 467629518, 467629526, 467629557, 467700448, 467889227, 467889235, 467889266, 467932245, 467932253, 467932261, 467932284, 467932292, 467932323, 468359257, 468402275, 468402283, 468402314, 468481522, 468552414, 468670301, 468670309, 468670340, 468713319, 468713327, 468713335, 468713358, 468713366, 468713397, 468930075, 468973093, 468973101, 468973132, 469012931, 469012939, 469012968, 469012970, 469012976, 469013007, 469015009, 469015017, 469015048, 469019721, 469019729, 469019760, 469055949, 469055957, 469055965, 469055986, 469055988, 469055994, 469055996, 469056002, 469056025, 469056027, 469056033, 469056064, 469058027, 469058035, 469058043, 469058066, 469058074, 469058105, 469062739, 469062747, 469062755, 469062778, 469062786, 469062817, 469322487, 469322495, 469322526, 469365505, 469365513, 469365521, 469365544, 469365552, 469365583, 469500893, 469914782, 470047708, 470047745, 470049786, 470054498, 470088119, 470088127, 470088156, 470088158, 470088164, 470088195, 470090197, 470090205, 470090236, 470094909, 470094917, 470094948, 470103561, 470103569, 470103600, 470131137, 470131145, 470131153, 470131174, 470131176, 470131182, 470131184, 470131190, 470131213, 470131215, 470131221, 470131252, 470133215, 470133223, 470133231, 470133254, 470133262, 470133293, 470137927, 470137935, 470137943, 470137966, 470137974, 470138005, 470146579, 470146587, 470146595, 470146618, 470146626, 470146657, 470357264, 470397675, 470397683, 470397714, 470440693, 470440701, 470440709, 470440732, 470440740, 470440771, 470989970, 471138338, 471178749, 471178757, 471178788, 471221767, 471221775, 471221783, 471221806, 471221814, 471221845, 471514292, 471514329, 471514366, 471516370, 471516407, 471518448, 471521082, 471521119, 471523160, 471527872, 471536821, 471536829, 471536860, 471579839, 471579847, 471579855, 471579878, 471579886, 471579917, 471629909, 471629917, 471629948, 471823848, 471823885, 471825926, 471830638, 472133404, 472525675, 472525683, 472525712, 472525714, 472525720, 472525751, 472527753, 472527761, 472527792, 472532465, 472532473, 472532504, 472568693, 472568701, 472568709, 472568730, 472568732, 472568738, 472568740, 472568746, 472568769, 472568771, 472568777, 472568808, 472570771, 472570779, 472570787, 472570810, 472570818, 472570849, 472571598, 472575483, 472575491, 472575499, 472575522, 472575530, 472575561, 472604922, 472604959, 472607000, 472611712, 472612009, 472612017, 472612048, 472655027, 472655035, 472655043, 472655066, 472655074, 472655105, 472835231, 472835239, 472835270, 472878249, 472878257, 472878265, 472878288, 472878296, 472878327, 472914478, 473053475, 473053512, 473055553, 473060265, 473096493, 473096501, 473096530, 473096532, 473096538, 473096569, 473098571, 473098579, 473098610, 473103283, 473103291, 473103322, 473363031, 473406049, 473406057, 473406088, 473427526, 473616305, 473616313, 473616344, 473646786, 473659323, 473659331, 473659339, 473659362, 473659370, 473659401, 473687197, 473687205, 473687236, 473695552, 473730215, 473730223, 473730231, 473730254, 473730262, 473730293, 474038182, 474038219, 474040260, 474044972, 474144105, 474187123, 474187131, 474187162, 474347738, 475049565, 475049573, 475049604, 475092583, 475092591, 475092599, 475092622, 475092630, 475092661, 475113370, 475113407, 475115448, 475120160, 475128812, 475422926, 475577365, 475620383, 475620391, 475620422, 476084342, 476124753, 476124761, 476124792, 476167771, 476167779, 476167787, 476167810, 476167818, 476167849, 476204000, 476562072, 476652553, 476692093, 476692101, 476692109, 476692130, 476692132, 476692138, 476692140, 476692146, 476692167, 476692169, 476692171, 476692175, 476692177, 476692183, 476692206, 476692208, 476692214, 476692245, 476694171, 476694179, 476694187, 476694208, 476694210, 476694216, 476694218, 476694224, 476694247, 476694249, 476694255, 476694286, 476695571, 476695579, 476695610, 476696249, 476696257, 476696265, 476696288, 476696296, 476696327, 476698883, 476698891, 476698899, 476698920, 476698922, 476698928, 476698930, 476698936, 476698959, 476698961, 476698967, 476698998, 476700961, 476700969, 476700977, 476701000, 476701008, 476701039, 476705673, 476705681, 476705689, 476705712, 476705720, 476705751, 477001649, 477001657, 477001665, 477001686, 477001688, 477001694, 477001696, 477001702, 477001725, 477001727, 477001733, 477001764, 477003727, 477003735, 477003743, 477003766, 477003774, 477003805, 477008439, 477008447, 477008455, 477008478, 477008486, 477008517, 477311205, 477311213, 477311221, 477311244, 477311252, 477311283, 477550926, 477550963, 477553004, 477557716, 477637260, 477782723, 477782731, 477782739, 477782760, 477782762, 477782768, 477782770, 477782776, 477782799, 477782801, 477782807, 477782838, 477784801, 477784809, 477784817, 477784840, 477784848, 477784879, 477789513, 477789521, 477789529, 477789552, 477789560, 477789591, 477860482, 478092279, 478092287, 478092295, 478092318, 478092326, 478092357, 478121744, 478121781, 478123822, 478128534, 478431300, 478562309, 478562317, 478562348, 478605327, 478605335, 478605343, 478605366, 478605374, 478605405, 478641556, 478712448, 478741158, 478873353, 478873361, 478873369, 478873392, 478873400, 478873431, 479090109, 479133127, 479133135, 479133166, 479212374, 479215983, 479215991, 479215999, 479216020, 479216022, 479216028, 479216030, 479216036, 479216059, 479216061, 479216067, 479216098, 479218061, 479218069, 479218077, 479218100, 479218108, 479218139, 479222773, 479222781, 479222789, 479222812, 479222820, 479222851, 479525539, 479525547, 479525555, 479525578, 479525586, 479525617, 479660927, 479703945, 479703953, 479703984, 480074816, 480207742, 480207779, 480209820, 480214532, 480291171, 480291179, 480291187, 480291208, 480291210, 480291216, 480291218, 480291224, 480291247, 480291249, 480291255, 480291286, 480293249, 480293257, 480293265, 480293288, 480293296, 480293327, 480297961, 480297969, 480297977, 480298000, 480298008, 480298039, 480306613, 480306621, 480306629, 480306652, 480306660, 480306691, 480517298, 480600727, 480600735, 480600743, 480600766, 480600774, 480600805, 480645634, 481150004, 481298372, 481381801, 481381809, 481381817, 481381840, 481381848, 481381879, 481674326, 481674363, 481674400, 481676404, 481676441, 481678482, 481681116, 481681153, 481683194, 481687906, 481717344, 481717352, 481717381, 481717383, 481717389, 481717418, 481717420, 481717426, 481717457, 481719422, 481719430, 481719459, 481719461, 481719467, 481719498, 481720822, 481721500, 481721508, 481721539, 481724134, 481724142, 481724171, 481724173, 481724179, 481724210, 481726212, 481726220, 481726251, 481730924, 481730932, 481730963, 481739873, 481739881, 481739889, 481739912, 481739920, 481739951, 481983882, 481983919, 481985960, 481990672, 482026900, 482026908, 482026937, 482026939, 482026945, 482026976, 482028978, 482028986, 482029017, 482033690, 482033698, 482033729, 482293438, 482336456, 482336464, 482336495, 482728727, 482728735, 482728743, 482728764, 482728766, 482728772, 482728774, 482728780, 482728803, 482728805, 482728811, 482728842, 482730805, 482730813, 482730821, 482730844, 482730852, 482730883, 482731632, 482735517, 482735525, 482735533, 482735556, 482735564, 482735595, 482764956, 482764993, 482767034, 482771746, 482807974, 482807982, 482808011, 482808013, 482808019, 482808050, 482810052, 482810060, 482810091, 482814764, 482814772, 482814803, 482815061, 482815069, 482815077, 482815100, 482815108, 482815139, 483038283, 483038291, 483038299, 483038322, 483038330, 483038361, 483074512, 483117530, 483117538, 483117569, 483256527, 483256535, 483256564, 483256566, 483256572, 483256603, 483258605, 483258613, 483258644, 483263317, 483263325, 483263356, 483299545, 483299553, 483299561, 483299582, 483299584, 483299590, 483299592, 483299598, 483299621, 483299623, 483299629, 483299660, 483301623, 483301631, 483301639, 483301662, 483301670, 483301701, 483306335, 483306343, 483306351, 483306374, 483306382, 483306413, 483566083, 483566091, 483566122, 483587560, 483609101, 483609109, 483609117, 483609140, 483609148, 483609179, 483806820, 483819357, 483819365, 483819373, 483819396, 483819404, 483819435, 483855586, 483890249, 483890257, 483890265, 483890288, 483890296, 483890327, 483898604, 483898612, 483898643, 484158378, 484198216, 484198253, 484200294, 484205006, 484241234, 484241242, 484241271, 484241273, 484241279, 484241310, 484243312, 484243320, 484243351, 484248024, 484248032, 484248063, 484347157, 484347165, 484347196, 484390175, 484390183, 484390191, 484390214, 484390222, 484390253, 484507772, 484550790, 484550798, 484550829, 485252617, 485252625, 485252633, 485252656, 485252664, 485252695, 485273404, 485273441, 485275482, 485280194, 485288846, 485316422, 485316430, 485316459, 485316461, 485316467, 485316498, 485318500, 485318508, 485318539, 485323212, 485323220, 485323251, 485331864, 485331872, 485331903, 485582960, 485625978, 485625986, 485626017, 485780417, 485780425, 485780456, 485823435, 485823443, 485823451, 485823474, 485823482, 485823513, 486244376, 486327805, 486327813, 486327821, 486327844, 486327852, 486327883, 486364034, 486407052, 486407060, 486407091, 486722106, 486765124, 486765132, 486765163, 486815194, 486855605, 486855613, 486855644, 486895145, 486895153, 486895161, 486895169, 486895182, 486895184, 486895190, 486895192, 486895198, 486895200, 486895206, 486895219, 486895221, 486895223, 486895227, 486895229, 486895231, 486895235, 486895237, 486895243, 486895258, 486895260, 486895262, 486895266, 486895268, 486895274, 486895297, 486895299, 486895305, 486895336, 486897223, 486897231, 486897239, 486897247, 486897260, 486897262, 486897268, 486897270, 486897276, 486897278, 486897284, 486897299, 486897301, 486897307, 486897309, 486897315, 486897338, 486897340, 486897346, 486897377, 486898623, 486898631, 486898639, 486898662, 486898670, 486898701, 486899301, 486899309, 486899317, 486899325, 486899340, 486899348, 486899356, 486899379, 486899387, 486899418, 486901935, 486901943, 486901951, 486901959, 486901972, 486901974, 486901980, 486901982, 486901988, 486901990, 486901996, 486902011, 486902013, 486902019, 486902021, 486902027, 486902050, 486902052, 486902058, 486902089, 486904013, 486904021, 486904029, 486904037, 486904052, 486904060, 486904068, 486904091, 486904099, 486904130, 486908725, 486908733, 486908741, 486908749, 486908764, 486908772, 486908780, 486908803, 486908811, 486908842, 487204701, 487204709, 487204717, 487204725, 487204738, 487204740, 487204746, 487204748, 487204754, 487204756, 487204762, 487204777, 487204779, 487204785, 487204787, 487204793, 487204816, 487204818, 487204824, 487204855, 487206779, 487206787, 487206795, 487206803, 487206818, 487206826, 487206834, 487206857, 487206865, 487206896, 487211491, 487211499, 487211507, 487211515, 487211530, 487211538, 487211546, 487211569, 487211577, 487211608, 487514257, 487514265, 487514273, 487514281, 487514296, 487514304, 487514312, 487514335, 487514343, 487514374, 487710960, 487710997, 487713038, 487717750, 487753978, 487753986, 487754015, 487754017, 487754023, 487754054, 487756056, 487756064, 487756095, 487760768, 487760776, 487760807, 487797294, 487840312, 487840320, 487840351, 487985775, 487985783, 487985791, 487985799, 487985812, 487985814, 487985820, 487985822, 487985828, 487985830, 487985836, 487985851, 487985853, 487985859, 487985861, 487985867, 487985890, 487985892, 487985898, 487985929, 487987853, 487987861, 487987869, 487987877, 487987892, 487987900, 487987908, 487987931, 487987939, 487987970, 487992565, 487992573, 487992581, 487992589, 487992604, 487992612, 487992620, 487992643, 487992651, 487992682, 488020516, 488063534, 488063542, 488063573, 488281778, 488281815, 488283856, 488288568, 488295331, 488295339, 488295347, 488295355, 488295370, 488295378, 488295386, 488295409, 488295417, 488295448, 488591334, 488765361, 488765369, 488765377, 488765400, 488765408, 488765439, 488801590, 488844608, 488844616, 488844647, 488872482, 488915500, 488915508, 488915539, 489076405, 489076413, 489076421, 489076429, 489076444, 489076452, 489076460, 489076483, 489076491, 489076522, 489293161, 489293169, 489293200, 489336179, 489336187, 489336195, 489336218, 489336226, 489336257, 489372408, 489419035, 489419043, 489419051, 489419059, 489419072, 489419074, 489419080, 489419082, 489419088, 489419090, 489419096, 489419111, 489419113, 489419119, 489419121, 489419127, 489419150, 489419152, 489419158, 489419189, 489421113, 489421121, 489421129, 489421137, 489421152, 489421160, 489421168, 489421191, 489421199, 489421230, 489425825, 489425833, 489425841, 489425849, 489425864, 489425872, 489425880, 489425903, 489425911, 489425942, 489728591, 489728599, 489728607, 489728615, 489728630, 489728638, 489728646, 489728669, 489728677, 489728708, 489820961, 489863979, 489863987, 489864018, 490234850, 490277868, 490277876, 490277907, 490367776, 490367813, 490369854, 490374566, 490410794, 490410802, 490410831, 490410833, 490410839, 490410870, 490412872, 490412880, 490412911, 490417584, 490417592, 490417623, 490494223, 490494231, 490494239, 490494247, 490494260, 490494262, 490494268, 490494270, 490494276, 490494278, 490494284, 490494299, 490494301, 490494307, 490494309, 490494315, 490494338, 490494340, 490494346, 490494377, 490496301, 490496309, 490496317, 490496325, 490496340, 490496348, 490496356, 490496379, 490496387, 490496418, 490501013, 490501021, 490501029, 490501037, 490501052, 490501060, 490501068, 490501091, 490501099, 490501130, 490509665, 490509673, 490509681, 490509689, 490509704, 490509712, 490509720, 490509743, 490509751, 490509782, 490677332, 490720350, 490720358, 490720389, 490803779, 490803787, 490803795, 490803803, 490803818, 490803826, 490803834, 490803857, 490803865, 490803896, 490805668, 491310038, 491353056, 491353064, 491353095, 491458406, 491501424, 491501432, 491501463, 491584853, 491584861, 491584869, 491584877, 491584892, 491584900, 491584908, 491584931, 491584939, 491584970, 491834360, 491834397, 491834434, 491836438, 491836475, 491838516, 491841150, 491841187, 491843228, 491847940, 491877378, 491877386, 491877415, 491877417, 491877423, 491877452, 491877454, 491877460, 491877491, 491879456, 491879464, 491879493, 491879495, 491879501, 491879532, 491880856, 491881534, 491881542, 491881573, 491884168, 491884176, 491884205, 491884207, 491884213, 491884244, 491886246, 491886254, 491886285, 491890958, 491890966, 491890997, 491942925, 491942933, 491942941, 491942949, 491942964, 491942972, 491942980, 491943003, 491943011, 491943042, 492143916, 492143953, 492145994, 492150706, 492186934, 492186942, 492186971, 492186973, 492186979, 492187010, 492189012, 492189020, 492189051, 492193724, 492193732, 492193763, 492453472, 492496490, 492496498, 492496529, 492891666, 492924990, 492925027, 492927068, 492931779, 492931780, 492931787, 492931795, 492931803, 492931816, 492931818, 492931824, 492931826, 492931832, 492931834, 492931840, 492931855, 492931857, 492931863, 492931865, 492931871, 492931894, 492931896, 492931902, 492931933, 492933857, 492933865, 492933873, 492933881, 492933896, 492933904, 492933912, 492933935, 492933943, 492933974, 492934684, 492934692, 492934723, 492938569, 492938577, 492938585, 492938593, 492938608, 492938616, 492938624, 492938647, 492938655, 492938686, 492968008, 492968016, 492968045, 492968047, 492968053, 492968084, 492970086, 492970094, 492970125, 492974798, 492974806, 492974837, 493018113, 493018121, 493018129, 493018137, 493018152, 493018160, 493018168, 493018191, 493018199, 493018230, 493234546, 493241335, 493241343, 493241351, 493241359, 493241374, 493241382, 493241390, 493241413, 493241421, 493241452, 493277564, 493277572, 493277603, 493459579, 493459587, 493459595, 493459616, 493459618, 493459624, 493459626, 493459632, 493459655, 493459657, 493459663, 493459694, 493461657, 493461665, 493461673, 493461696, 493461704, 493461735, 493466369, 493466377, 493466385, 493466408, 493466416, 493466447, 493747594, 493769135, 493769143, 493769151, 493769174, 493769182, 493769213, 493790612, 493790620, 493790651, 493966854, 494009872, 494009880, 494009911, 494015620, 494022409, 494022417, 494022425, 494022433, 494022448, 494022456, 494022464, 494022487, 494022495, 494022526, 494058638, 494058646, 494058677, 494093301, 494093309, 494093317, 494093325, 494093340, 494093348, 494093356, 494093379, 494093387, 494093418, 494318412, 494358250, 494358287, 494360328, 494365040, 494401268, 494401276, 494401305, 494401307, 494401313, 494401344, 494403346, 494403354, 494403385, 494408058, 494408066, 494408097, 494550209, 494550217, 494550225, 494550248, 494550256, 494550287, 494667806, 494710824, 494710832, 494710863, 494889230, 495433438, 495433475, 495435516, 495440228, 495448880, 495455669, 495455677, 495455685, 495455693, 495455708, 495455716, 495455724, 495455747, 495455755, 495455786, 495476456, 495476464, 495476493, 495476495, 495476501, 495476532, 495478534, 495478542, 495478573, 495483246, 495483254, 495483285, 495491898, 495491906, 495491937, 495742994, 495786012, 495786020, 495786051, 495983469, 495983477, 495983485, 495983508, 495983516, 495983547, 496404410, 496447428, 496447436, 496447467, 496524068, 496530857, 496530865, 496530873, 496530881, 496530896, 496530904, 496530912, 496530935, 496530943, 496530974, 496567086, 496567094, 496567125, 496882140, 496925158, 496925166, 496925197, 496975228, 497058657, 497058665, 497058673, 497058696, 497058704, 497058735, 497870994, 497871031, 497873072, 497877784, 497914012, 497914020, 497914049, 497914051, 497914057, 497914088, 497916090, 497916098, 497916129, 497920802, 497920810, 497920841, 497957328, 498000346, 498000354, 498000385, 498180550, 498223568, 498223576, 498223607, 498441812, 498441849, 498443890, 498448602, 498484830, 498484838, 498484867, 498484869, 498484875, 498484906, 498486908, 498486916, 498486947, 498491620, 498491628, 498491659, 498751368, 498794386, 498794394, 498794425, 498961624, 498968413, 498968421, 498968429, 498968437, 498968452, 498968460, 498968468, 498968491, 498968499, 498968530, 499004642, 499004650, 499004681, 499032516, 499075534, 499075542, 499075573, 499496213, 499496221, 499496229, 499496252, 499496260, 499496291, 499532442, 499575460, 499575468, 499575499, 500024013, 500024021, 500024052, 500067031, 500067039, 500067047, 500067070, 500067078, 500067109, 500394884, 500437902, 500437910, 500437941, 500570828, 500570836, 500570865, 500570867, 500570873, 500570904, 500572906, 500572914, 500572945, 500577618, 500577626, 500577657, 500880384, 500880392, 500880423, 500965702, 501008720, 501008728, 501008759, 501470072, 501513090, 501513098, 501513129, 501661458, 501661466, 501661497, 501994394, 501994431, 501994468, 501996472, 501996509, 501998550, 502001184, 502001221, 502003262, 502007974, 502037412, 502037420, 502037449, 502037451, 502037457, 502037486, 502037488, 502037494, 502037525, 502039490, 502039498, 502039527, 502039529, 502039535, 502039566, 502040890, 502041568, 502041576, 502041607, 502044202, 502044210, 502044239, 502044241, 502044247, 502044278, 502046280, 502046288, 502046319, 502050992, 502051000, 502051031, 502080430, 502080438, 502080446, 502080467, 502080469, 502080475, 502080477, 502080483, 502080504, 502080506, 502080508, 502080512, 502080514, 502080520, 502080543, 502080545, 502080551, 502080582, 502082508, 502082516, 502082524, 502082545, 502082547, 502082553, 502082555, 502082561, 502082584, 502082586, 502082592, 502082623, 502083908, 502083916, 502083947, 502084586, 502084594, 502084602, 502084625, 502084633, 502084664, 502087220, 502087228, 502087236, 502087257, 502087259, 502087265, 502087267, 502087273, 502087296, 502087298, 502087304, 502087335, 502089298, 502089306, 502089314, 502089337, 502089345, 502089376, 502094010, 502094018, 502094026, 502094049, 502094057, 502094088, 502303950, 502303987, 502306028, 502310740, 502346968, 502346976, 502347005, 502347007, 502347013, 502347044, 502349046, 502349054, 502349085, 502353758, 502353766, 502353797, 502389986, 502389994, 502390002, 502390023, 502390025, 502390031, 502390033, 502390039, 502390062, 502390064, 502390070, 502390101, 502392064, 502392072, 502392080, 502392103, 502392111, 502392142, 502396776, 502396784, 502396792, 502396815, 502396823, 502396854, 502613506, 502656524, 502656532, 502656563, 502699542, 502699550, 502699558, 502699581, 502699589, 502699620, 503085024, 503085061, 503087102, 503091814, 503094718, 503094726, 503094757, 503128042, 503128050, 503128079, 503128081, 503128087, 503128118, 503130120, 503130128, 503130159, 503134832, 503134840, 503134871, 503171060, 503171068, 503171076, 503171097, 503171099, 503171105, 503171107, 503171113, 503171136, 503171138, 503171144, 503171175, 503173138, 503173146, 503173154, 503173177, 503173185, 503173216, 503177850, 503177858, 503177866, 503177889, 503177897, 503177928, 503394580, 503437598, 503437606, 503437637, 503480616, 503480624, 503480632, 503480655, 503480663, 503480694, 503662631, 503662639, 503662647, 503662655, 503662668, 503662670, 503662676, 503662678, 503662684, 503662686, 503662692, 503662707, 503662709, 503662715, 503662717, 503662723, 503662746, 503662748, 503662754, 503662785, 503664709, 503664717, 503664725, 503664733, 503664748, 503664756, 503664764, 503664787, 503664795, 503664826, 503669421, 503669429, 503669437, 503669445, 503669460, 503669468, 503669476, 503669499, 503669507, 503669538, 503907628, 503950646, 503950654, 503950685, 503972187, 503972195, 503972203, 503972211, 503972226, 503972234, 503972242, 503972265, 503972273, 503972304, 504169906, 504169914, 504169945, 504175654, 504218672, 504218680, 504218711, 504261690, 504261698, 504261706, 504261729, 504261737, 504261768, 504478446, 504518284, 504518321, 504520362, 504521464, 504521472, 504521503, 504525074, 504561302, 504561310, 504561339, 504561341, 504561347, 504561378, 504563380, 504563388, 504563419, 504568092, 504568100, 504568131, 504604320, 504604328, 504604336, 504604357, 504604359, 504604365, 504604367, 504604373, 504604396, 504604398, 504604404, 504604435, 504606398, 504606406, 504606414, 504606437, 504606445, 504606476, 504611110, 504611118, 504611126, 504611149, 504611157, 504611188, 504753261, 504753269, 504753277, 504753285, 504753300, 504753308, 504753316, 504753339, 504753347, 504753378, 504827840, 504870858, 504870866, 504870897, 504913876, 504913884, 504913892, 504913915, 504913923, 504913954, 505049264, 505593472, 505593509, 505595550, 505596079, 505596116, 505598157, 505600262, 505602869, 505608914, 505636490, 505636498, 505636527, 505636529, 505636535, 505636566, 505638568, 505638576, 505638607, 505643280, 505643288, 505643319, 505651932, 505651940, 505651971, 505679508, 505679516, 505679524, 505679545, 505679547, 505679553, 505679555, 505679561, 505679584, 505679586, 505679592, 505679623, 505681586, 505681594, 505681602, 505681625, 505681633, 505681664, 505686298, 505686306, 505686314, 505686337, 505686345, 505686376, 505694950, 505694958, 505694966, 505694989, 505694997, 505695028, 505903028, 505905635, 505946046, 505946054, 505946085, 505989064, 505989072, 505989080, 505989103, 505989111, 505989142, 506186521, 506186529, 506186537, 506186545, 506186560, 506186568, 506186576, 506186599, 506186607, 506186638, 506607462, 506607470, 506607501, 506684102, 506686709, 506727120, 506727128, 506727159, 506770138, 506770146, 506770154, 506770177, 506770185, 506770216, 507042174, 507085192, 507085200, 507085231, 507128210, 507128218, 507128226, 507128249, 507128257, 507128288, 507135262, 507178280, 507178288, 507178319, 507261709, 507261717, 507261725, 507261733, 507261748, 507261756, 507261764, 507261787, 507261795, 507261826, 508031028, 508031065, 508033106, 508037818, 508074046, 508074054, 508074083, 508074085, 508074091, 508074122, 508076124, 508076132, 508076163, 508080836, 508080844, 508080875, 508117064, 508117072, 508117080, 508117101, 508117103, 508117109, 508117111, 508117117, 508117140, 508117142, 508117148, 508117179, 508117362, 508119142, 508119150, 508119158, 508119181, 508119189, 508119220, 508119969, 508123854, 508123862, 508123870, 508123893, 508123901, 508123932, 508160380, 508160388, 508160419, 508203398, 508203406, 508203414, 508203437, 508203445, 508203476, 508340584, 508383602, 508383610, 508383641, 508426620, 508426628, 508426636, 508426659, 508426667, 508426698, 508601846, 508601883, 508603924, 508608636, 508644864, 508644872, 508644901, 508644903, 508644909, 508644940, 508646942, 508646950, 508646981, 508651654, 508651662, 508651693, 508911402, 508954420, 508954428, 508954459, 509121658, 509164676, 509164684, 509164715, 509192550, 509195157, 509207694, 509207702, 509207710, 509207733, 509207741, 509207772, 509235568, 509235576, 509235607, 509278586, 509278594, 509278602, 509278625, 509278633, 509278664, 509692476, 509699265, 509699273, 509699281, 509699289, 509699304, 509699312, 509699320, 509699343, 509699351, 509699382, 509735494, 509735502, 509735533, 510227065, 510227073, 510227081, 510227104, 510227112, 510227143, 510554918, 510597936, 510597944, 510597975, 510640954, 510640962, 510640970, 510640993, 510641001, 510641032, 510773880, 510773888, 510773896, 510773917, 510773919, 510773925, 510773927, 510773933, 510773956, 510773958, 510773964, 510773995, 510775958, 510775966, 510775974, 510775997, 510776005, 510776036, 510780670, 510780678, 510780686, 510780709, 510780717, 510780748, 511083436, 511083444, 511083452, 511083475, 511083483, 511083514, 511125736, 511168754, 511168762, 511168793, 511630106, 511632713, 511673124, 511673132, 511673163, 511716142, 511716150, 511716158, 511716181, 511716189, 511716220, 511864510, 511864518, 511864526, 511864549, 511864557, 511864588, 512197446, 512197454, 512197483, 512197485, 512197491, 512197520, 512197522, 512197528, 512197559, 512199524, 512199532, 512199561, 512199563, 512199569, 512199600, 512200924, 512201602, 512201610, 512201641, 512204236, 512204244, 512204273, 512204275, 512204281, 512204312, 512206314, 512206322, 512206353, 512211026, 512211034, 512211065, 512240464, 512240472, 512240480, 512240501, 512240503, 512240509, 512240511, 512240517, 512240538, 512240540, 512240542, 512240546, 512240548, 512240554, 512240577, 512240579, 512240585, 512240616, 512242542, 512242550, 512242558, 512242579, 512242581, 512242587, 512242589, 512242595, 512242618, 512242620, 512242626, 512242657, 512243942, 512243950, 512243981, 512244620, 512244628, 512244636, 512244659, 512244667, 512244698, 512247254, 512247262, 512247270, 512247291, 512247293, 512247299, 512247301, 512247307, 512247330, 512247332, 512247338, 512247369, 512249332, 512249340, 512249348, 512249371, 512249379, 512249410, 512254044, 512254052, 512254060, 512254083, 512254091, 512254122, 512507002, 512507010, 512507039, 512507041, 512507047, 512507078, 512509080, 512509088, 512509119, 512513792, 512513800, 512513831, 512550020, 512550028, 512550036, 512550057, 512550059, 512550065, 512550067, 512550073, 512550096, 512550098, 512550104, 512550135, 512552098, 512552106, 512552114, 512552137, 512552145, 512552176, 512556810, 512556818, 512556826, 512556849, 512556857, 512556888, 512816558, 512816566, 512816597, 512859576, 512859584, 512859592, 512859615, 512859623, 512859654, 513288076, 513288084, 513288113, 513288115, 513288121, 513288152, 513290154, 513290162, 513290193, 513294866, 513294874, 513294905, 513297770, 513297778, 513297786, 513297809, 513297817, 513297848, 513331094, 513331102, 513331110, 513331131, 513331133, 513331139, 513331141, 513331147, 513331170, 513331172, 513331178, 513331209, 513333172, 513333180, 513333188, 513333211, 513333219, 513333250, 513337884, 513337892, 513337900, 513337923, 513337931, 513337962, 513597632, 513597640, 513597671, 513640650, 513640658, 513640666, 513640689, 513640697, 513640728, 514067662, 514110680, 514110688, 514110719, 514153698, 514153706, 514153714, 514153737, 514153745, 514153776, 514289529, 514372958, 514372966, 514372974, 514372997, 514373005, 514373036, 514378706, 514378714, 514378745, 514421724, 514421732, 514421740, 514421763, 514421771, 514421802, 514638480, 514681498, 514681506, 514681537, 514721336, 514721344, 514721373, 514721375, 514721381, 514721412, 514723414, 514723422, 514723453, 514728126, 514728134, 514728165, 514764354, 514764362, 514764370, 514764391, 514764393, 514764399, 514764401, 514764407, 514764430, 514764432, 514764438, 514764469, 514766432, 514766440, 514766448, 514766471, 514766479, 514766510, 514771144, 514771152, 514771160, 514771183, 514771191, 514771222, 515030892, 515030900, 515030931, 515073910, 515073918, 515073926, 515073949, 515073957, 515073988, 515209298, 515252316, 515252324, 515252355, 515756113, 515756150, 515758191, 515762903, 515796524, 515796532, 515796561, 515796563, 515796569, 515796600, 515798602, 515798610, 515798641, 515803314, 515803322, 515803353, 515811966, 515811974, 515812005, 515839542, 515839550, 515839558, 515839579, 515839581, 515839587, 515839589, 515839595, 515839618, 515839620, 515839626, 515839657, 515841620, 515841628, 515841636, 515841659, 515841667, 515841698, 515846332, 515846340, 515846348, 515846371, 515846379, 515846410, 515854984, 515854992, 515855000, 515855023, 515855031, 515855062, 516065669, 516106080, 516106088, 516106119, 516149098, 516149106, 516149114, 516149137, 516149145, 516149176, 516810514, 516810522, 516810530, 516810553, 516810561, 516810592, 516846743, 516887154, 516887162, 516887193, 516930172, 516930180, 516930188, 516930211, 516930219, 516930250, 517245226, 517245234, 517245265, 517288244, 517288252, 517288260, 517288283, 517288291, 517288322, 517338314, 517338322, 517338353, 518234080, 518234088, 518234117, 518234119, 518234125, 518234156, 518236158, 518236166, 518236197, 518240870, 518240878, 518240909, 518277098, 518277106, 518277114, 518277135, 518277137, 518277143, 518277145, 518277151, 518277174, 518277176, 518277182, 518277213, 518279176, 518279184, 518279192, 518279215, 518279223, 518279254, 518280003, 518283888, 518283896, 518283904, 518283927, 518283935, 518283966, 518320414, 518320422, 518320453, 518363432, 518363440, 518363448, 518363471, 518363479, 518363510, 518543636, 518543644, 518543675, 518586654, 518586662, 518586670, 518586693, 518586701, 518586732, 518761880, 518761917, 518763958, 518768670, 518804898, 518804906, 518804935, 518804937, 518804943, 518804974, 518806976, 518806984, 518807015, 518811688, 518811696, 518811727, 518847916, 518847924, 518847932, 518847953, 518847955, 518847961, 518847963, 518847969, 518847992, 518847994, 518848000, 518848031, 518849994, 518850002, 518850010, 518850033, 518850041, 518850072, 518854706, 518854714, 518854722, 518854745, 518854753, 518854784, 519071436, 519114454, 519114462, 519114493, 519157472, 519157480, 519157488, 519157511, 519157519, 519157550, 519324710, 519324718, 519324749, 519355191, 519367728, 519367736, 519367744, 519367767, 519367775, 519367806, 519395602, 519395610, 519395641, 519438620, 519438628, 519438636, 519438659, 519438667, 519438698, 519852510, 519895528, 519895536, 519895567, 519938546, 519938554, 519938562, 519938585, 519938593, 519938624, 520430117, 520430125, 520430133, 520430141, 520430156, 520430164, 520430172, 520430195, 520430203, 520430234, 520757970, 520757978, 520758009, 520800988, 520800996, 520801004, 520801027, 520801035, 520801066, 521285770, 521328788, 521328796, 521328827, 521371806, 521371814, 521371822, 521371845, 521371853, 521371884, 521792747, 521833158, 521833166, 521833197, 521876176, 521876184, 521876192, 521876215, 521876223, 521876254, 522360958, 522363565, 522400498, 522400506, 522400514, 522400535, 522400537, 522400543, 522400545, 522400551, 522400572, 522400574, 522400576, 522400580, 522400582, 522400588, 522400611, 522400613, 522400619, 522400650, 522402576, 522402584, 522402592, 522402613, 522402615, 522402621, 522402623, 522402629, 522402652, 522402654, 522402660, 522402691, 522403976, 522403984, 522404015, 522404654, 522404662, 522404670, 522404693, 522404701, 522404732, 522407288, 522407296, 522407304, 522407325, 522407327, 522407333, 522407335, 522407341, 522407364, 522407366, 522407372, 522407403, 522409366, 522409374, 522409382, 522409405, 522409413, 522409444, 522414078, 522414086, 522414094, 522414117, 522414125, 522414156, 522443516, 522443524, 522443532, 522443540, 522443553, 522443555, 522443561, 522443563, 522443569, 522443571, 522443577, 522443590, 522443592, 522443594, 522443598, 522443600, 522443602, 522443606, 522443608, 522443614, 522443629, 522443631, 522443633, 522443637, 522443639, 522443645, 522443668, 522443670, 522443676, 522443707, 522445594, 522445602, 522445610, 522445618, 522445631, 522445633, 522445639, 522445641, 522445647, 522445649, 522445655, 522445670, 522445672, 522445678, 522445680, 522445686, 522445709, 522445711, 522445717, 522445748, 522446994, 522447002, 522447010, 522447033, 522447041, 522447072, 522447672, 522447680, 522447688, 522447696, 522447711, 522447719, 522447727, 522447750, 522447758, 522447789, 522450306, 522450314, 522450322, 522450330, 522450343, 522450345, 522450351, 522450353, 522450359, 522450361, 522450367, 522450382, 522450384, 522450390, 522450392, 522450398, 522450421, 522450423, 522450429, 522450460, 522452384, 522452392, 522452400, 522452408, 522452423, 522452431, 522452439, 522452462, 522452470, 522452501, 522457096, 522457104, 522457112, 522457120, 522457135, 522457143, 522457151, 522457174, 522457182, 522457213, 522710054, 522710062, 522710070, 522710091, 522710093, 522710099, 522710101, 522710107, 522710130, 522710132, 522710138, 522710169, 522712132, 522712140, 522712148, 522712171, 522712179, 522712210, 522716844, 522716852, 522716860, 522716883, 522716891, 522716922, 522753072, 522753080, 522753088, 522753096, 522753109, 522753111, 522753117, 522753119, 522753125, 522753127, 522753133, 522753148, 522753150, 522753156, 522753158, 522753164, 522753187, 522753189, 522753195, 522753226, 522755150, 522755158, 522755166, 522755174, 522755189, 522755197, 522755205, 522755228, 522755236, 522755267, 522759862, 522759870, 522759878, 522759886, 522759901, 522759909, 522759917, 522759940, 522759948, 522759979, 523019610, 523019618, 523019626, 523019649, 523019657, 523019688, 523062628, 523062636, 523062644, 523062652, 523062667, 523062675, 523062683, 523062706, 523062714, 523062745, 523491128, 523491136, 523491144, 523491165, 523491167, 523491173, 523491175, 523491181, 523491204, 523491206, 523491212, 523491243, 523493206, 523493214, 523493222, 523493245, 523493253, 523493284, 523497918, 523497926, 523497934, 523497957, 523497965, 523497996, 523534146, 523534154, 523534162, 523534170, 523534183, 523534185, 523534191, 523534193, 523534199, 523534201, 523534207, 523534222, 523534224, 523534230, 523534232, 523534238, 523534261, 523534263, 523534269, 523534300, 523536224, 523536232, 523536240, 523536248, 523536263, 523536271, 523536279, 523536302, 523536310, 523536341, 523540936, 523540944, 523540952, 523540960, 523540975, 523540983, 523540991, 523541014, 523541022, 523541053, 523800684, 523800692, 523800700, 523800723, 523800731, 523800762, 523843702, 523843710, 523843718, 523843726, 523843741, 523843749, 523843757, 523843780, 523843788, 523843819, 524270714, 524270722, 524270753, 524313732, 524313740, 524313748, 524313771, 524313779, 524313810, 524449563, 524581758, 524581766, 524581774, 524581797, 524581805, 524581836, 524624776, 524624784, 524624792, 524624800, 524624815, 524624823, 524624831, 524624854, 524624862, 524624893, 524798514, 524841532, 524841540, 524841571, 524884550, 524884558, 524884566, 524884589, 524884597, 524884628, 524924388, 524924396, 524924404, 524924425, 524924427, 524924433, 524924435, 524924441, 524924464, 524924466, 524924472, 524924503, 524926466, 524926474, 524926482, 524926505, 524926513, 524926544, 524931178, 524931186, 524931194, 524931217, 524931225, 524931256, 524967406, 524967414, 524967422, 524967430, 524967443, 524967445, 524967451, 524967453, 524967459, 524967461, 524967467, 524967482, 524967484, 524967490, 524967492, 524967498, 524967521, 524967523, 524967529, 524967560, 524969484, 524969492, 524969500, 524969508, 524969523, 524969531, 524969539, 524969562, 524969570, 524969601, 524974196, 524974204, 524974212, 524974220, 524974235, 524974243, 524974251, 524974274, 524974282, 524974313, 525233944, 525233952, 525233960, 525233983, 525233991, 525234022, 525276962, 525276970, 525276978, 525276986, 525277001, 525277009, 525277017, 525277040, 525277048, 525277079, 525369332, 525412350, 525412358, 525412389, 525916147, 525916184, 525918225, 525922937, 525959165, 525959173, 525959202, 525959204, 525959210, 525959241, 525961243, 525961251, 525961282, 525965955, 525965963, 525965994, 525999576, 525999584, 525999592, 525999613, 525999615, 525999621, 525999623, 525999629, 525999652, 525999654, 525999660, 525999691, 526001654, 526001662, 526001670, 526001693, 526001701, 526001732, 526006366, 526006374, 526006382, 526006405, 526006413, 526006444, 526015018, 526015026, 526015034, 526015057, 526015065, 526015096, 526042594, 526042602, 526042610, 526042618, 526042631, 526042633, 526042639, 526042641, 526042647, 526042649, 526042655, 526042670, 526042672, 526042678, 526042680, 526042686, 526042709, 526042711, 526042717, 526042748, 526044672, 526044680, 526044688, 526044696, 526044711, 526044719, 526044727, 526044750, 526044758, 526044789, 526049384, 526049392, 526049400, 526049408, 526049423, 526049431, 526049439, 526049462, 526049470, 526049501, 526058036, 526058044, 526058052, 526058060, 526058075, 526058083, 526058091, 526058114, 526058122, 526058153, 526225703, 526268721, 526268729, 526268760, 526309132, 526309140, 526309148, 526309171, 526309179, 526309210, 526352150, 526352158, 526352166, 526352174, 526352189, 526352197, 526352205, 526352228, 526352236, 526352267, 527006777, 527049795, 527049803, 527049834, 527090206, 527090214, 527090222, 527090245, 527090253, 527090284, 527133224, 527133232, 527133240, 527133248, 527133263, 527133271, 527133279, 527133302, 527133310, 527133341, 527448278, 527448286, 527448294, 527448317, 527448325, 527448356, 527491296, 527491304, 527491312, 527491320, 527491335, 527491343, 527491351, 527491374, 527491382, 527491413, 527541366, 527541374, 527541382, 527541405, 527541413, 527541444, 528437132, 528437140, 528437148, 528437169, 528437171, 528437177, 528437179, 528437185, 528437208, 528437210, 528437216, 528437247, 528439210, 528439218, 528439226, 528439249, 528439257, 528439288, 528440037, 528443922, 528443930, 528443938, 528443961, 528443969, 528444000, 528480150, 528480158, 528480166, 528480174, 528480187, 528480189, 528480195, 528480197, 528480203, 528480205, 528480211, 528480226, 528480228, 528480234, 528480236, 528480242, 528480265, 528480267, 528480273, 528480304, 528482228, 528482236, 528482244, 528482252, 528482267, 528482275, 528482283, 528482306, 528482314, 528482345, 528483055, 528483063, 528483094, 528486940, 528486948, 528486956, 528486964, 528486979, 528486987, 528486995, 528487018, 528487026, 528487057, 528523466, 528523474, 528523482, 528523505, 528523513, 528523544, 528566484, 528566492, 528566500, 528566508, 528566523, 528566531, 528566539, 528566562, 528566570, 528566601, 528746688, 528746696, 528746704, 528746727, 528746735, 528746766, 528789706, 528789714, 528789722, 528789730, 528789745, 528789753, 528789761, 528789784, 528789792, 528789823, 528964932, 528964940, 528964969, 528964971, 528964977, 528965008, 528967010, 528967018, 528967049, 528971722, 528971730, 528971761, 529007950, 529007958, 529007966, 529007987, 529007989, 529007995, 529007997, 529008003, 529008026, 529008028, 529008034, 529008065, 529010028, 529010036, 529010044, 529010067, 529010075, 529010106, 529014740, 529014748, 529014756, 529014779, 529014787, 529014818, 529274488, 529274496, 529274527, 529317506, 529317514, 529317522, 529317545, 529317553, 529317584, 529515225, 529527762, 529527770, 529527778, 529527801, 529527809, 529527840, 529558243, 529558251, 529558282, 529570780, 529570788, 529570796, 529570804, 529570819, 529570827, 529570835, 529570858, 529570866, 529570897, 529598654, 529598662, 529598670, 529598693, 529598701, 529598732, 529641672, 529641680, 529641688, 529641696, 529641711, 529641719, 529641727, 529641750, 529641758, 529641789, 530055562, 530055570, 530055601, 530098580, 530098588, 530098596, 530098619, 530098627, 530098658, 530961022, 530961030, 530961038, 530961061, 530961069, 530961100, 531004040, 531004048, 531004056, 531004064, 531004079, 531004087, 531004095, 531004118, 531004126, 531004157, 531488822, 531488830, 531488861, 531531840, 531531848, 531531856, 531531879, 531531887, 531531918, 531952781, 531995799, 531995807, 531995838, 532036210, 532036218, 532036226, 532036249, 532036257, 532036288, 532079228, 532079236, 532079244, 532079252, 532079267, 532079275, 532079283, 532079306, 532079314, 532079345, 532523599, 532564010, 532564018, 532564049, 532603550, 532603558, 532603566, 532603574, 532603587, 532603589, 532603595, 532603597, 532603603, 532603605, 532603611, 532603624, 532603626, 532603628, 532603632, 532603634, 532603636, 532603640, 532603642, 532603648, 532603663, 532603665, 532603667, 532603671, 532603673, 532603679, 532603702, 532603704, 532603710, 532603741, 532605628, 532605636, 532605644, 532605652, 532605665, 532605667, 532605673, 532605675, 532605681, 532605683, 532605689, 532605704, 532605706, 532605712, 532605714, 532605720, 532605743, 532605745, 532605751, 532605782, 532607028, 532607036, 532607044, 532607067, 532607075, 532607106, 532607706, 532607714, 532607722, 532607730, 532607745, 532607753, 532607761, 532607784, 532607792, 532607823, 532610340, 532610348, 532610356, 532610364, 532610377, 532610379, 532610385, 532610387, 532610393, 532610395, 532610401, 532610416, 532610418, 532610424, 532610426, 532610432, 532610455, 532610457, 532610463, 532610494, 532612418, 532612426, 532612434, 532612442, 532612457, 532612465, 532612473, 532612496, 532612504, 532612535, 532617130, 532617138, 532617146, 532617154, 532617169, 532617177, 532617185, 532617208, 532617216, 532617247, 532913106, 532913114, 532913122, 532913130, 532913143, 532913145, 532913151, 532913153, 532913159, 532913161, 532913167, 532913182, 532913184, 532913190, 532913192, 532913198, 532913221, 532913223, 532913229, 532913260, 532915184, 532915192, 532915200, 532915208, 532915223, 532915231, 532915239, 532915262, 532915270, 532915301, 532919896, 532919904, 532919912, 532919920, 532919935, 532919943, 532919951, 532919974, 532919982, 532920013, 533222662, 533222670, 533222678, 533222686, 533222701, 533222709, 533222717, 533222740, 533222748, 533222779, 533694180, 533694188, 533694196, 533694204, 533694217, 533694219, 533694225, 533694227, 533694233, 533694235, 533694241, 533694256, 533694258, 533694264, 533694266, 533694272, 533694295, 533694297, 533694303, 533694334, 533696258, 533696266, 533696274, 533696282, 533696297, 533696305, 533696313, 533696336, 533696344, 533696375, 533700970, 533700978, 533700986, 533700994, 533701009, 533701017, 533701025, 533701048, 533701056, 533701087, 534003736, 534003744, 534003752, 534003760, 534003775, 534003783, 534003791, 534003814, 534003822, 534003853, 534473766, 534473774, 534473782, 534473805, 534473813, 534473844, 534516784, 534516792, 534516800, 534516808, 534516823, 534516831, 534516839, 534516862, 534516870, 534516901, 534652615, 534652623, 534652654, 534784810, 534784818, 534784826, 534784834, 534784849, 534784857, 534784865, 534784888, 534784896, 534784927, 535001566, 535001574, 535001605, 535044584, 535044592, 535044600, 535044623, 535044631, 535044662, 535127440, 535127448, 535127456, 535127464, 535127477, 535127479, 535127485, 535127487, 535127493, 535127495, 535127501, 535127516, 535127518, 535127524, 535127526, 535127532, 535127555, 535127557, 535127563, 535127594, 535129518, 535129526, 535129534, 535129542, 535129557, 535129565, 535129573, 535129596, 535129604, 535129635, 535134230, 535134238, 535134246, 535134254, 535134269, 535134277, 535134285, 535134308, 535134316, 535134347, 535436996, 535437004, 535437012, 535437020, 535437035, 535437043, 535437051, 535437074, 535437082, 535437113, 535529366, 535572384, 535572392, 535572423, 535615402, 535615410, 535615418, 535615441, 535615449, 535615480, 536076181, 536076218, 536078259, 536082971, 536119199, 536119207, 536119236, 536119238, 536119244, 536119275, 536121277, 536121285, 536121316, 536125989, 536125997, 536126028, 536202628, 536202636, 536202644, 536202652, 536202665, 536202667, 536202673, 536202675, 536202681, 536202683, 536202689, 536202704, 536202706, 536202712, 536202714, 536202720, 536202743, 536202745, 536202751, 536202782, 536204706, 536204714, 536204722, 536204730, 536204745, 536204753, 536204761, 536204784, 536204792, 536204823, 536209418, 536209426, 536209434, 536209442, 536209457, 536209465, 536209473, 536209496, 536209504, 536209535, 536218070, 536218078, 536218086, 536218094, 536218109, 536218117, 536218125, 536218148, 536218156, 536218187, 536385737, 536428755, 536428763, 536428794, 536512184, 536512192, 536512200, 536512208, 536512223, 536512231, 536512239, 536512262, 536512270, 536512301, 537166811, 537209829, 537209837, 537209868, 537293258, 537293266, 537293274, 537293282, 537293297, 537293305, 537293313, 537293336, 537293344, 537293375, 537651330, 537651338, 537651346, 537651354, 537651369, 537651377, 537651385, 537651408, 537651416, 537651447, 538600071, 538640184, 538640192, 538640200, 538640208, 538640221, 538640223, 538640229, 538640231, 538640237, 538640239, 538640245, 538640260, 538640262, 538640268, 538640270, 538640276, 538640299, 538640301, 538640307, 538640338, 538642262, 538642270, 538642278, 538642286, 538642301, 538642309, 538642317, 538642340, 538642348, 538642379, 538643089, 538643097, 538643128, 538646974, 538646982, 538646990, 538646998, 538647013, 538647021, 538647029, 538647052, 538647060, 538647091, 538726518, 538726526, 538726534, 538726542, 538726557, 538726565, 538726573, 538726596, 538726604, 538726635, 538949740, 538949748, 538949756, 538949764, 538949779, 538949787, 538949795, 538949818, 538949826, 538949857, 539167984, 539167992, 539168000, 539168021, 539168023, 539168029, 539168031, 539168037, 539168060, 539168062, 539168068, 539168099, 539170062, 539170070, 539170078, 539170101, 539170109, 539170140, 539174774, 539174782, 539174790, 539174813, 539174821, 539174852, 539211002, 539211010, 539211018, 539211026, 539211039, 539211041, 539211047, 539211049, 539211055, 539211057, 539211063, 539211078, 539211080, 539211086, 539211088, 539211094, 539211117, 539211119, 539211125, 539211156, 539213080, 539213088, 539213096, 539213104, 539213119, 539213127, 539213135, 539213158, 539213166, 539213197, 539217792, 539217800, 539217808, 539217816, 539217831, 539217839, 539217847, 539217870, 539217878, 539217909, 539477540, 539477548, 539477556, 539477579, 539477587, 539477618, 539520558, 539520566, 539520574, 539520582, 539520597, 539520605, 539520613, 539520636, 539520644, 539520675, 539675259, 539718277, 539718285, 539718316, 539730814, 539730822, 539730830, 539730838, 539730853, 539730861, 539730869, 539730892, 539730900, 539730931, 539801706, 539801714, 539801722, 539801730, 539801745, 539801753, 539801761, 539801784, 539801792, 539801823, 540258614, 540258622, 540258630, 540258653, 540258661, 540258692, 540301632, 540301640, 540301648, 540301656, 540301671, 540301679, 540301687, 540301710, 540301718, 540301749, 541144450, 541144487, 541146528, 541151240, 541164074, 541164082, 541164090, 541164098, 541164113, 541164121, 541164129, 541164152, 541164160, 541164191, 541454006, 541691874, 541691882, 541691890, 541691913, 541691921, 541691952, 541734892, 541734900, 541734908, 541734916, 541734931, 541734939, 541734947, 541734970, 541734978, 541735009, 542112815, 542155833, 542155841, 542155872, 542235080, 542239262, 542239270, 542239278, 542239286, 542239301, 542239309, 542239317, 542239340, 542239348, 542239379, 542683633, 542726651, 542726659, 542726690, 542767062, 542767070, 542767078, 542767101, 542767109, 542767140, 542806602, 542806610, 542806618, 542806626, 542806634, 542806639, 542806641, 542806647, 542806649, 542806655, 542806657, 542806663, 542806665, 542806671, 542806676, 542806678, 542806680, 542806684, 542806686, 542806688, 542806692, 542806694, 542806696, 542806700, 542806702, 542806708, 542806715, 542806717, 542806719, 542806723, 542806725, 542806727, 542806731, 542806733, 542806739, 542806754, 542806756, 542806758, 542806762, 542806764, 542806770, 542806793, 542806795, 542806801, 542806832, 542808680, 542808688, 542808696, 542808704, 542808712, 542808717, 542808719, 542808725, 542808727, 542808733, 542808735, 542808741, 542808743, 542808749, 542808756, 542808758, 542808764, 542808766, 542808772, 542808774, 542808780, 542808795, 542808797, 542808803, 542808805, 542808811, 542808834, 542808836, 542808842, 542808873, 542810080, 542810088, 542810096, 542810104, 542810119, 542810127, 542810135, 542810158, 542810166, 542810197, 542810758, 542810766, 542810774, 542810782, 542810790, 542810797, 542810805, 542810813, 542810821, 542810836, 542810844, 542810852, 542810875, 542810883, 542810914, 542813392, 542813400, 542813408, 542813416, 542813424, 542813429, 542813431, 542813437, 542813439, 542813445, 542813447, 542813453, 542813455, 542813461, 542813468, 542813470, 542813476, 542813478, 542813484, 542813486, 542813492, 542813507, 542813509, 542813515, 542813517, 542813523, 542813546, 542813548, 542813554, 542813585, 542815470, 542815478, 542815486, 542815494, 542815502, 542815509, 542815517, 542815525, 542815533, 542815548, 542815556, 542815564, 542815587, 542815595, 542815626, 542820182, 542820190, 542820198, 542820206, 542820214, 542820221, 542820229, 542820237, 542820245, 542820260, 542820268, 542820276, 542820299, 542820307, 542820338, 543116158, 543116166, 543116174, 543116182, 543116190, 543116195, 543116197, 543116203, 543116205, 543116211, 543116213, 543116219, 543116221, 543116227, 543116234, 543116236, 543116242, 543116244, 543116250, 543116252, 543116258, 543116273, 543116275, 543116281, 543116283, 543116289, 543116312, 543116314, 543116320, 543116351, 543118236, 543118244, 543118252, 543118260, 543118268, 543118275, 543118283, 543118291, 543118299, 543118314, 543118322, 543118330, 543118353, 543118361, 543118392, 543122948, 543122956, 543122964, 543122972, 543122980, 543122987, 543122995, 543123003, 543123011, 543123026, 543123034, 543123042, 543123065, 543123073, 543123104, 543425714, 543425722, 543425730, 543425738, 543425746, 543425753, 543425761, 543425769, 543425777, 543425792, 543425800, 543425808, 543425831, 543425839, 543425870, 543668340, 543897232, 543897240, 543897248, 543897256, 543897264, 543897269, 543897271, 543897277, 543897279, 543897285, 543897287, 543897293, 543897295, 543897301, 543897308, 543897310, 543897316, 543897318, 543897324, 543897326, 543897332, 543897347, 543897349, 543897355, 543897357, 543897363, 543897386, 543897388, 543897394, 543897425, 543899310, 543899318, 543899326, 543899334, 543899342, 543899349, 543899357, 543899365, 543899373, 543899388, 543899396, 543899404, 543899427, 543899435, 543899466, 543904022, 543904030, 543904038, 543904046, 543904054, 543904061, 543904069, 543904077, 543904085, 543904100, 543904108, 543904116, 543904139, 543904147, 543904178, 544206788, 544206796, 544206804, 544206812, 544206820, 544206827, 544206835, 544206843, 544206851, 544206866, 544206874, 544206882, 544206905, 544206913, 544206944, 544676818, 544676826, 544676834, 544676842, 544676857, 544676865, 544676873, 544676896, 544676904, 544676935, 544743528, 544987862, 544987870, 544987878, 544987886, 544987894, 544987901, 544987909, 544987917, 544987925, 544987940, 544987948, 544987956, 544987979, 544987987, 544988018, 545204618, 545204626, 545204634, 545204657, 545204665, 545204696, 545247636, 545247644, 545247652, 545247660, 545247675, 545247683, 545247691, 545247714, 545247722, 545247753, 545330492, 545330500, 545330508, 545330516, 545330524, 545330529, 545330531, 545330537, 545330539, 545330545, 545330547, 545330553, 545330555, 545330561, 545330568, 545330570, 545330576, 545330578, 545330584, 545330586, 545330592, 545330607, 545330609, 545330615, 545330617, 545330623, 545330646, 545330648, 545330654, 545330685, 545332570, 545332578, 545332586, 545332594, 545332602, 545332609, 545332617, 545332625, 545332633, 545332648, 545332656, 545332664, 545332687, 545332695, 545332726, 545337282, 545337290, 545337298, 545337306, 545337314, 545337321, 545337329, 545337337, 545337345, 545337360, 545337368, 545337376, 545337399, 545337407, 545337438, 545640048, 545640056, 545640064, 545640072, 545640080, 545640087, 545640095, 545640103, 545640111, 545640126, 545640134, 545640142, 545640165, 545640173, 545640204, 545732418, 545732426, 545732457, 545775436, 545775444, 545775452, 545775475, 545775483, 545775514, 546279233, 546279241, 546279270, 546279272, 546279278, 546279309, 546281311, 546281319, 546281350, 546286023, 546286031, 546286062, 546322251, 546322259, 546322267, 546322288, 546322290, 546322296, 546322298, 546322304, 546322327, 546322329, 546322335, 546322366, 546324329, 546324337, 546324345, 546324368, 546324376, 546324407, 546329041, 546329049, 546329057, 546329080, 546329088, 546329119, 546405680, 546405688, 546405696, 546405704, 546405712, 546405717, 546405719, 546405725, 546405727, 546405733, 546405735, 546405741, 546405743, 546405749, 546405756, 546405758, 546405764, 546405766, 546405772, 546405774, 546405780, 546405795, 546405797, 546405803, 546405805, 546405811, 546405834, 546405836, 546405842, 546405873, 546407758, 546407766, 546407774, 546407782, 546407790, 546407797, 546407805, 546407813, 546407821, 546407836, 546407844, 546407852, 546407875, 546407883, 546407914, 546412470, 546412478, 546412486, 546412494, 546412502, 546412509, 546412517, 546412525, 546412533, 546412548, 546412556, 546412564, 546412587, 546412595, 546412626, 546421122, 546421130, 546421138, 546421146, 546421154, 546421161, 546421169, 546421177, 546421185, 546421200, 546421208, 546421216, 546421239, 546421247, 546421278, 546588789, 546588797, 546588828, 546631807, 546631815, 546631823, 546631846, 546631854, 546631885, 546715236, 546715244, 546715252, 546715260, 546715268, 546715275, 546715283, 546715291, 546715299, 546715314, 546715322, 546715330, 546715353, 546715361, 546715392, 547181084, 547369863, 547369871, 547369902, 547412881, 547412889, 547412897, 547412920, 547412928, 547412959, 547496310, 547496318, 547496326, 547496334, 547496342, 547496349, 547496357, 547496365, 547496373, 547496388, 547496396, 547496404, 547496427, 547496435, 547496466, 547854382, 547854390, 547854398, 547854406, 547854414, 547854421, 547854429, 547854437, 547854445, 547854460, 547854468, 547854476, 547854499, 547854507, 547854538, 548803123, 548803131, 548803162, 548843236, 548843244, 548843252, 548843260, 548843268, 548843273, 548843275, 548843281, 548843283, 548843289, 548843291, 548843297, 548843299, 548843305, 548843312, 548843314, 548843320, 548843322, 548843328, 548843330, 548843336, 548843351, 548843353, 548843359, 548843361, 548843367, 548843390, 548843392, 548843398, 548843429, 548845314, 548845322, 548845330, 548845338, 548845346, 548845353, 548845361, 548845369, 548845377, 548845392, 548845400, 548845408, 548845431, 548845439, 548845470, 548846141, 548846149, 548846157, 548846180, 548846188, 548846219, 548850026, 548850034, 548850042, 548850050, 548850058, 548850065, 548850073, 548850081, 548850089, 548850104, 548850112, 548850120, 548850143, 548850151, 548850182, 548929570, 548929578, 548929586, 548929594, 548929602, 548929609, 548929617, 548929625, 548929633, 548929648, 548929656, 548929664, 548929687, 548929695, 548929726, 549152792, 549152800, 549152808, 549152816, 549152824, 549152831, 549152839, 549152847, 549152855, 549152870, 549152878, 549152886, 549152909, 549152917, 549152948, 549371036, 549371044, 549371052, 549371060, 549371073, 549371075, 549371081, 549371083, 549371089, 549371091, 549371097, 549371112, 549371114, 549371120, 549371122, 549371128, 549371151, 549371153, 549371159, 549371190, 549373114, 549373122, 549373130, 549373138, 549373153, 549373161, 549373169, 549373192, 549373200, 549373231, 549377826, 549377834, 549377842, 549377850, 549377865, 549377873, 549377881, 549377904, 549377912, 549377943, 549680592, 549680600, 549680608, 549680616, 549680631, 549680639, 549680647, 549680670, 549680678, 549680709, 549837900, 549878311, 549878319, 549878350, 549921329, 549921337, 549921345, 549921368, 549921376, 549921407, 549933866, 549933874, 549933882, 549933890, 549933898, 549933905, 549933913, 549933921, 549933929, 549933944, 549933952, 549933960, 549933983, 549933991, 549934022, 550004758, 550004766, 550004774, 550004782, 550004790, 550004797, 550004805, 550004813, 550004821, 550004836, 550004844, 550004852, 550004875, 550004883, 550004914, 550461666, 550461674, 550461682, 550461690, 550461705, 550461713, 550461721, 550461744, 550461752, 550461783, 551304484, 551304521, 551306562, 551311274, 551367126, 551367134, 551367142, 551367150, 551367158, 551367165, 551367173, 551367181, 551367189, 551367204, 551367212, 551367220, 551367243, 551367251, 551367282, 551614040, 551894926, 551894934, 551894942, 551894950, 551894965, 551894973, 551894981, 551895004, 551895012, 551895043, 552315867, 552315875, 552315906, 552358885, 552358893, 552358901, 552358924, 552358932, 552358963, 552395114, 552442314, 552442322, 552442330, 552442338, 552442346, 552442353, 552442361, 552442369, 552442377, 552442392, 552442400, 552442408, 552442431, 552442439, 552442470, 552843667, 552886685, 552886693, 552886724, 552970114, 552970122, 552970130, 552970138, 552970153, 552970161, 552970169, 552970192, 552970200, 552970231, 553828374, 554879870, 554879878, 554879886, 554879894, 554879902, 554879909, 554879917, 554879925, 554879933, 554879948, 554879956, 554879964, 554879987, 554879995, 554880026, 554903562, 555407670, 555407678, 555407686, 555407694, 555407709, 555407717, 555407725, 555407748, 555407756, 555407787, 555935470, 555935478, 555935486, 555935509, 555935517, 555935548, 555978488, 555978496, 555978504, 555978512, 555978527, 555978535, 555978543, 555978566, 555978574, 555978605, 556482285, 556482293, 556482301, 556482322, 556482324, 556482330, 556482332, 556482338, 556482361, 556482363, 556482369, 556482400, 556484363, 556484371, 556484379, 556484402, 556484410, 556484441, 556489075, 556489083, 556489091, 556489114, 556489122, 556489153, 556791841, 556791849, 556791857, 556791880, 556791888, 556791919, 557341118, 557572915, 557572923, 557572931, 557572954, 557572962, 557572993, 557911936, 559006175, 559006183, 559006191, 559006214, 559006222, 559006253, 559574088, 559574096, 559574104, 559574112, 559574120, 559574125, 559574127, 559574133, 559574135, 559574141, 559574143, 559574149, 559574151, 559574157, 559574164, 559574166, 559574172, 559574174, 559574180, 559574182, 559574188, 559574203, 559574205, 559574211, 559574213, 559574219, 559574242, 559574244, 559574250, 559574281, 559576166, 559576174, 559576182, 559576190, 559576198, 559576205, 559576213, 559576221, 559576229, 559576244, 559576252, 559576260, 559576283, 559576291, 559576322, 559580878, 559580886, 559580894, 559580902, 559580910, 559580917, 559580925, 559580933, 559580941, 559580956, 559580964, 559580972, 559580995, 559581003, 559581034, 559883644, 559883652, 559883660, 559883668, 559883676, 559883683, 559883691, 559883699, 559883707, 559883722, 559883730, 559883738, 559883761, 559883769, 559883800, 559997934, 560081363, 560081371, 560081379, 560081402, 560081410, 560081441, 560664718, 560664726, 560664734, 560664742, 560664750, 560664757, 560664765, 560664773, 560664781, 560664796, 560664804, 560664812, 560664835, 560664843, 560664874, 561464518, 561464555, 561466596, 561471308, 561507536, 561507544, 561507573, 561507575, 561507581, 561507612, 561509614, 561509622, 561509653, 561514326, 561514334, 561514365, 561774074, 561817092, 561817100, 561817131, 562097978, 562097986, 562097994, 562098002, 562098010, 562098017, 562098025, 562098033, 562098041, 562098056, 562098064, 562098072, 562098095, 562098103, 562098134, 562518919, 562518927, 562518935, 562518958, 562518966, 562518997, 562555148, 562598166, 562598174, 562598205, 563046719, 563046727, 563046758, 563089737, 563089745, 563089753, 563089776, 563089784, 563089815, 563173166, 563173174, 563173182, 563173190, 563173198, 563173205, 563173213, 563173221, 563173229, 563173244, 563173252, 563173260, 563173283, 563173291, 563173322, 563988408, 564031426, 564031434, 564031465, 565063596, 565106614, 565106622, 565106653, 565610722, 565610730, 565610738, 565610746, 565610754, 565610761, 565610769, 565610777, 565610785, 565610800, 565610808, 565610816, 565610839, 565610847, 565610878, 566138522, 566138530, 566138538, 566138546, 566138561, 566138569, 566138577, 566138600, 566138608, 566138639, 566685337, 566685345, 566685353, 566685361, 566685374, 566685376, 566685382, 566685384, 566685390, 566685392, 566685398, 566685413, 566685415, 566685421, 566685423, 566685429, 566685452, 566685454, 566685460, 566685491, 566687415, 566687423, 566687431, 566687439, 566687454, 566687462, 566687470, 566687493, 566687501, 566687532, 566692127, 566692135, 566692143, 566692151, 566692166, 566692174, 566692182, 566692205, 566692213, 566692244, 566994893, 566994901, 566994909, 566994917, 566994932, 566994940, 566994948, 566994971, 566994979, 566995010, 567501152, 567544170, 567544178, 567544209, 567775967, 567775975, 567775983, 567775991, 567776006, 567776014, 567776022, 567776045, 567776053, 567776084, 568071970, 569209227, 569209235, 569209243, 569209251, 569209266, 569209274, 569209282, 569209305, 569209313, 569209344, 570157968, 570200986, 570200994, 570201025, 570284415, 570284423, 570284431, 570284439, 570284454, 570284462, 570284470, 570284493, 570284501, 570284532, 571624552, 571624589, 571626630, 571631342, 571667570, 571667578, 571667607, 571667609, 571667615, 571667646, 571669648, 571669656, 571669687, 571674360, 571674368, 571674399, 571934108, 571977126, 571977134, 571977165, 572715182, 572721971, 572721979, 572721987, 572721995, 572722010, 572722018, 572722026, 572722049, 572722057, 572722088, 572758200, 572758208, 572758239, 573249771, 573249779, 573249787, 573249810, 573249818, 573249849, 574148442, 574191460, 574191468, 574191499, 575223630, 575266648, 575266656, 575266687, 576341574, 576341582, 576341590, 576341598, 576341606, 576341613, 576341621, 576341629, 576341637, 576341652, 576341660, 576341668, 576341691, 576341699, 576341730, 576692821, 576692858, 576694899, 576699611, 577002377, 577661186, 577704204, 577704212, 577704243, 577783451, 578232004, 578275022, 578275030, 578275061, 579216711, 580291899, 580361020, 580361028, 580361059, 581784586, 581784623, 581786664, 581791376, 581827604, 581827612, 581827641, 581827643, 581827649, 581827680, 581829682, 581829690, 581829721, 581834394, 581834402, 581834433, 581870622, 581870630, 581870638, 581870659, 581870661, 581870667, 581870669, 581870675, 581870698, 581870700, 581870706, 581870737, 581872700, 581872708, 581872716, 581872739, 581872747, 581872778, 581877412, 581877420, 581877428, 581877451, 581877459, 581877490, 582094142, 582137160, 582137168, 582137199, 582180178, 582180186, 582180194, 582180217, 582180225, 582180256, 582729455, 582875216, 582918234, 582918242, 582918273, 582961252, 582961260, 582961268, 582961291, 582961299, 582961330, 583452823, 583452831, 583452839, 583452847, 583452862, 583452870, 583452878, 583452901, 583452909, 583452940, 584308476, 584351494, 584351502, 584351533, 584394512, 584394520, 584394528, 584394551, 584394559, 584394590, 585383664, 585386271, 585426682, 585426690, 585426721, 585469700, 585469708, 585469716, 585469739, 585469747, 585469778, 586852855, 586852892, 586854933, 586859645, 587162411, 587821220, 587864238, 587864246, 587864277, 587907256, 587907264, 587907272, 587907295, 587907303, 587907334, 587943485, 588392038, 588435056, 588435064, 588435095, 589376745, 590451933, 590564072, 590564080, 590564088, 590564111, 590564119, 590564150, 591987638, 591987646, 591987675, 591987677, 591987683, 591987714, 591989716, 591989724, 591989755, 591994428, 591994436, 591994467, 592030656, 592030664, 592030672, 592030693, 592030695, 592030701, 592030703, 592030709, 592030732, 592030734, 592030740, 592030771, 592032734, 592032742, 592032750, 592032773, 592032781, 592032812, 592037446, 592037454, 592037462, 592037485, 592037493, 592037524, 592297194, 592297202, 592297233, 592340212, 592340220, 592340228, 592340251, 592340259, 592340290, 592889489, 593078268, 593078276, 593078307, 593121286, 593121294, 593121302, 593121325, 593121333, 593121364, 593460307, 594511528, 594511536, 594511567, 594554546, 594554554, 594554562, 594554585, 594554593, 594554624, 595546305, 595586716, 595586724, 595586755, 595629734, 595629742, 595629750, 595629773, 595629781, 595629812, 597012889, 597012926, 597014967, 597019679, 597055907, 597055915, 597055944, 597055946, 597055952, 597055983, 597057985, 597057993, 597058024, 597062697, 597062705, 597062736, 597322445, 597365463, 597365471, 597365502, 598024272, 598024280, 598024311, 598067290, 598067298, 598067306, 598067329, 598067337, 598067368, 598103519, 598146537, 598146545, 598146576, 598552072, 598595090, 598595098, 598595129, 598638108, 598638116, 598638124, 598638147, 598638155, 598638186, 599536779, 599579797, 599579805, 599579836, 600611967, 600654985, 600654993, 600655024, 602190690, 602190698, 602190706, 602190727, 602190729, 602190735, 602190737, 602190743, 602190766, 602190768, 602190774, 602190805, 602192768, 602192776, 602192784, 602192807, 602192815, 602192846, 602197480, 602197488, 602197496, 602197519, 602197527, 602197558, 602233708, 602233716, 602233724, 602233732, 602233745, 602233747, 602233753, 602233755, 602233761, 602233763, 602233769, 602233784, 602233786, 602233792, 602233794, 602233800, 602233823, 602233825, 602233831, 602233862, 602235786, 602235794, 602235802, 602235810, 602235825, 602235833, 602235841, 602235864, 602235872, 602235903, 602240498, 602240506, 602240514, 602240522, 602240537, 602240545, 602240553, 602240576, 602240584, 602240615, 602500246, 602500254, 602500262, 602500285, 602500293, 602500324, 602543264, 602543272, 602543280, 602543288, 602543303, 602543311, 602543319, 602543342, 602543350, 602543381, 603049523, 603092541, 603092549, 603092580, 603281320, 603281328, 603281336, 603281359, 603281367, 603281398, 603324338, 603324346, 603324354, 603324362, 603324377, 603324385, 603324393, 603324416, 603324424, 603324455, 603620341, 604714580, 604714588, 604714596, 604714619, 604714627, 604714658, 604757598, 604757606, 604757614, 604757622, 604757637, 604757645, 604757653, 604757676, 604757684, 604757715, 605706339, 605749357, 605749365, 605749396, 605789768, 605789776, 605789784, 605789807, 605789815, 605789846, 605832786, 605832794, 605832802, 605832810, 605832825, 605832833, 605832841, 605832864, 605832872, 605832903, 607172923, 607172960, 607175001, 607179713, 607215941, 607215949, 607215978, 607215980, 607215986, 607216017, 607218019, 607218027, 607218058, 607222731, 607222739, 607222770, 607482479, 607525497, 607525505, 607525536, 608227324, 608227332, 608227340, 608227363, 608227371, 608227402, 608263553, 608270342, 608270350, 608270358, 608270366, 608270381, 608270389, 608270397, 608270420, 608270428, 608270459, 608306571, 608306579, 608306610, 608755124, 608755132, 608755163, 608798142, 608798150, 608798158, 608798181, 608798189, 608798220, 609696813, 609739831, 609739839, 609739870, 610772001, 610815019, 610815027, 610815058, 612393742, 612393750, 612393758, 612393766, 612393779, 612393781, 612393787, 612393789, 612393795, 612393797, 612393803, 612393818, 612393820, 612393826, 612393828, 612393834, 612393857, 612393859, 612393865, 612393896, 612395820, 612395828, 612395836, 612395844, 612395859, 612395867, 612395875, 612395898, 612395906, 612395937, 612400532, 612400540, 612400548, 612400556, 612400571, 612400579, 612400587, 612400610, 612400618, 612400649, 612703298, 612703306, 612703314, 612703322, 612703337, 612703345, 612703353, 612703376, 612703384, 612703415, 613209557, 613252575, 613252583, 613252614, 613484372, 613484380, 613484388, 613484396, 613484411, 613484419, 613484427, 613484450, 613484458, 613484489, 613780375, 613823393, 613823401, 613823432, 614917632, 614917640, 614917648, 614917656, 614917671, 614917679, 614917687, 614917710, 614917718, 614917749, 615866373, 615909391, 615909399, 615909430, 615992820, 615992828, 615992836, 615992844, 615992859, 615992867, 615992875, 615992898, 615992906, 615992937, 617332957, 617332994, 617335035, 617339747, 617375975, 617375983, 617376012, 617376014, 617376020, 617376051, 617378053, 617378061, 617378092, 617382765, 617382773, 617382804, 617418993, 617419001, 617419009, 617419030, 617419032, 617419038, 617419040, 617419046, 617419069, 617419071, 617419077, 617419108, 617421071, 617421079, 617421087, 617421110, 617421118, 617421149, 617425783, 617425791, 617425799, 617425822, 617425830, 617425861, 617642513, 617685531, 617685539, 617685570, 617728549, 617728557, 617728565, 617728588, 617728596, 617728627, 618423587, 618430376, 618430384, 618430392, 618430400, 618430415, 618430423, 618430431, 618430454, 618430462, 618430493, 618466605, 618466613, 618466644, 618509623, 618509631, 618509639, 618509662, 618509670, 618509701, 618958176, 618958184, 618958192, 618958215, 618958223, 618958254, 619001194, 619001202, 619001210, 619001218, 619001233, 619001241, 619001249, 619001272, 619001280, 619001311, 619856847, 619899865, 619899873, 619899904, 619942883, 619942891, 619942899, 619942922, 619942930, 619942961, 620932035, 620934642, 620975053, 620975061, 620975092, 621018071, 621018079, 621018087, 621018110, 621018118, 621018149, 622596794, 622596802, 622596810, 622596818, 622596826, 622596831, 622596833, 622596839, 622596841, 622596847, 622596849, 622596855, 622596857, 622596863, 622596870, 622596872, 622596878, 622596880, 622596886, 622596888, 622596894, 622596909, 622596911, 622596917, 622596919, 622596925, 622596948, 622596950, 622596956, 622596987, 622598872, 622598880, 622598888, 622598896, 622598904, 622598911, 622598919, 622598927, 622598935, 622598950, 622598958, 622598966, 622598989, 622598997, 622599028, 622603584, 622603592, 622603600, 622603608, 622603616, 622603623, 622603631, 622603639, 622603647, 622603662, 622603670, 622603678, 622603701, 622603709, 622603740, 622906350, 622906358, 622906366, 622906374, 622906382, 622906389, 622906397, 622906405, 622906413, 622906428, 622906436, 622906444, 622906467, 622906475, 622906506, 623369591, 623412609, 623412617, 623412648, 623455627, 623455635, 623455643, 623455666, 623455674, 623455705, 623687424, 623687432, 623687440, 623687448, 623687456, 623687463, 623687471, 623687479, 623687487, 623687502, 623687510, 623687518, 623687541, 623687549, 623687580, 623940409, 623983427, 623983435, 623983466, 625120684, 625120692, 625120700, 625120708, 625120716, 625120723, 625120731, 625120739, 625120747, 625120762, 625120770, 625120778, 625120801, 625120809, 625120840, 626069425, 626069433, 626069464, 626112443, 626112451, 626112459, 626112482, 626112490, 626112521, 626195872, 626195880, 626195888, 626195896, 626195904, 626195911, 626195919, 626195927, 626195935, 626195950, 626195958, 626195966, 626195989, 626195997, 626196028, 627492991, 627493028, 627495069, 627499781, 627536009, 627536017, 627536046, 627536048, 627536054, 627536085, 627538087, 627538095, 627538126, 627542799, 627542807, 627542838, 627579027, 627579035, 627579043, 627579064, 627579066, 627579072, 627579074, 627579080, 627579103, 627579105, 627579111, 627579142, 627581105, 627581113, 627581121, 627581144, 627581152, 627581183, 627585817, 627585825, 627585833, 627585856, 627585864, 627585895, 627802547, 627845565, 627845573, 627845604, 627888583, 627888591, 627888599, 627888622, 627888630, 627888661, 628583621, 628626639, 628626647, 628626678, 628633428, 628633436, 628633444, 628633452, 628633460, 628633467, 628633475, 628633483, 628633491, 628633506, 628633514, 628633522, 628633545, 628633553, 628633584, 628669657, 628669665, 628669673, 628669696, 628669704, 628669735, 629161228, 629161236, 629161244, 629161252, 629161267, 629161275, 629161283, 629161306, 629161314, 629161345, 630016881, 630059899, 630059907, 630059938, 630102917, 630102925, 630102933, 630102956, 630102964, 630102995, 631092069, 631094676, 631135087, 631135095, 631135126, 631178105, 631178113, 631178121, 631178144, 631178152, 631178183, 633529625, 633572643, 633572651, 633572682, 633615661, 633615669, 633615677, 633615700, 633615708, 633615739, 634100443, 634143461, 634143469, 634143500, 634186479, 634186487, 634186495, 634186518, 634186526, 634186557, 636272477, 636272485, 636272493, 636272516, 636272524, 636272555, 637696043, 637696051, 637696080, 637696082, 637696088, 637696119, 637698121, 637698129, 637698160, 637702833, 637702841, 637702872, 637739061, 637739069, 637739077, 637739098, 637739100, 637739106, 637739108, 637739114, 637739137, 637739139, 637739145, 637739176, 637741139, 637741147, 637741155, 637741178, 637741186, 637741217, 637745851, 637745859, 637745867, 637745890, 637745898, 637745929, 637782079, 637782087, 637782095, 637782103, 637782116, 637782118, 637782124, 637782126, 637782132, 637782134, 637782140, 637782155, 637782157, 637782163, 637782165, 637782171, 637782194, 637782196, 637782202, 637782233, 637784157, 637784165, 637784173, 637784181, 637784196, 637784204, 637784212, 637784235, 637784243, 637784274, 637788869, 637788877, 637788885, 637788893, 637788908, 637788916, 637788924, 637788947, 637788955, 637788986, 638005599, 638005607, 638005638, 638048617, 638048625, 638048633, 638048656, 638048664, 638048695, 638091635, 638091643, 638091651, 638091659, 638091674, 638091682, 638091690, 638091713, 638091721, 638091752, 638786673, 638786681, 638786712, 638829691, 638829699, 638829707, 638829730, 638829738, 638829769, 638872709, 638872717, 638872725, 638872733, 638872748, 638872756, 638872764, 638872787, 638872795, 638872826, 639364280, 639364288, 639364296, 639364304, 639364312, 639364319, 639364327, 639364335, 639364343, 639364358, 639364366, 639364374, 639364397, 639364405, 639364436, 640219933, 640219941, 640219972, 640262951, 640262959, 640262967, 640262990, 640262998, 640263029, 640305969, 640305977, 640305985, 640305993, 640306008, 640306016, 640306024, 640306047, 640306055, 640306086, 641254710, 641295121, 641295129, 641295160, 641297728, 641297736, 641297767, 641338139, 641338147, 641338155, 641338178, 641338186, 641338217, 641381157, 641381165, 641381173, 641381181, 641381196, 641381204, 641381212, 641381235, 641381243, 641381274, 643732677, 643732685, 643732716, 643775695, 643775703, 643775711, 643775734, 643775742, 643775773, 643818713, 643818721, 643818729, 643818737, 643818752, 643818760, 643818768, 643818791, 643818799, 643818830, 644260477, 644303495, 644303503, 644303534, 644346513, 644346521, 644346529, 644346552, 644346560, 644346591, 646475529, 646475537, 646475545, 646475553, 646475568, 646475576, 646475584, 646475607, 646475615, 646475646, 647899095, 647899103, 647899111, 647899132, 647899134, 647899140, 647899142, 647899148, 647899171, 647899173, 647899179, 647899210, 647901173, 647901181, 647901189, 647901212, 647901220, 647901251, 647905885, 647905893, 647905901, 647905924, 647905932, 647905963, 647942113, 647942121, 647942129, 647942137, 647942150, 647942152, 647942158, 647942160, 647942166, 647942168, 647942174, 647942189, 647942191, 647942197, 647942199, 647942205, 647942228, 647942230, 647942236, 647942267, 647944191, 647944199, 647944207, 647944215, 647944230, 647944238, 647944246, 647944269, 647944277, 647944308, 647948903, 647948911, 647948919, 647948927, 647948942, 647948950, 647948958, 647948981, 647948989, 647949020, 648208651, 648208659, 648208667, 648208690, 648208698, 648208729, 648251669, 648251677, 648251685, 648251693, 648251708, 648251716, 648251724, 648251747, 648251755, 648251786, 648989725, 648989733, 648989741, 648989764, 648989772, 648989803, 649032743, 649032751, 649032759, 649032767, 649032782, 649032790, 649032798, 649032821, 649032829, 649032860, 650422985, 650422993, 650423001, 650423024, 650423032, 650423063, 650466003, 650466011, 650466019, 650466027, 650466042, 650466050, 650466058, 650466081, 650466089, 650466120, 651414744, 651457762, 651457770, 651457801, 651498173, 651498181, 651498189, 651498212, 651498220, 651498251, 651541191, 651541199, 651541207, 651541215, 651541230, 651541238, 651541246, 651541269, 651541277, 651541308, 653935729, 653935737, 653935745, 653935768, 653935776, 653935807, 653978747, 653978755, 653978763, 653978771, 653978786, 653978794, 653978802, 653978825, 653978833, 653978864, 654463529, 654463537, 654463568, 654506547, 654506555, 654506563, 654506586, 654506594, 654506625, 654549565, 654549573, 654549581, 654549589, 654549604, 654549612, 654549620, 654549643, 654549651, 654549682, 656483013, 658102147, 658102155, 658102163, 658102171, 658102184, 658102186, 658102192, 658102194, 658102200, 658102202, 658102208, 658102223, 658102225, 658102231, 658102233, 658102239, 658102262, 658102264, 658102270, 658102301, 658104225, 658104233, 658104241, 658104249, 658104264, 658104272, 658104280, 658104303, 658104311, 658104342, 658108937, 658108945, 658108953, 658108961, 658108976, 658108984, 658108992, 658109015, 658109023, 658109054, 658145165, 658145173, 658145181, 658145189, 658145197, 658145202, 658145204, 658145210, 658145212, 658145218, 658145220, 658145226, 658145228, 658145234, 658145241, 658145243, 658145249, 658145251, 658145257, 658145259, 658145265, 658145280, 658145282, 658145288, 658145290, 658145296, 658145319, 658145321, 658145327, 658145358, 658147243, 658147251, 658147259, 658147267, 658147275, 658147282, 658147290, 658147298, 658147306, 658147321, 658147329, 658147337, 658147360, 658147368, 658147399, 658151955, 658151963, 658151971, 658151979, 658151987, 658151994, 658152002, 658152010, 658152018, 658152033, 658152041, 658152049, 658152072, 658152080, 658152111, 658411703, 658411711, 658411719, 658411727, 658411742, 658411750, 658411758, 658411781, 658411789, 658411820, 658454721, 658454729, 658454737, 658454745, 658454753, 658454760, 658454768, 658454776, 658454784, 658454799, 658454807, 658454815, 658454838, 658454846, 658454877, 659192777, 659192785, 659192793, 659192801, 659192816, 659192824, 659192832, 659192855, 659192863, 659192894, 659235795, 659235803, 659235811, 659235819, 659235827, 659235834, 659235842, 659235850, 659235858, 659235873, 659235881, 659235889, 659235912, 659235920, 659235951, 660626037, 660626045, 660626053, 660626061, 660626076, 660626084, 660626092, 660626115, 660626123, 660626154, 660669055, 660669063, 660669071, 660669079, 660669087, 660669094, 660669102, 660669110, 660669118, 660669133, 660669141, 660669149, 660669172, 660669180, 660669211, 661574778, 661617796, 661617804, 661617835, 661660814, 661660822, 661660830, 661660853, 661660861, 661660892, 661701225, 661701233, 661701241, 661701249, 661701264, 661701272, 661701280, 661701303, 661701311, 661701342, 661744243, 661744251, 661744259, 661744267, 661744275, 661744282, 661744290, 661744298, 661744306, 661744321, 661744329, 661744337, 661744360, 661744368, 661744399, 664138781, 664138789, 664138797, 664138805, 664138820, 664138828, 664138836, 664138859, 664138867, 664138898, 664181799, 664181807, 664181815, 664181823, 664181831, 664181838, 664181846, 664181854, 664181862, 664181877, 664181885, 664181893, 664181916, 664181924, 664181955, 664666581, 664666589, 664666597, 664666620, 664666628, 664666659, 664709599, 664709607, 664709615, 664709623, 664709638, 664709646, 664709654, 664709677, 664709685, 664709716, 666643047, 668305199, 668305207, 668305215, 668305223, 668305231, 668305236, 668305238, 668305244, 668305246, 668305252, 668305254, 668305260, 668305262, 668305268, 668305275, 668305277, 668305283, 668305285, 668305291, 668305293, 668305299, 668305314, 668305316, 668305322, 668305324, 668305330, 668305353, 668305355, 668305361, 668305392, 668307277, 668307285, 668307293, 668307301, 668307309, 668307316, 668307324, 668307332, 668307340, 668307355, 668307363, 668307371, 668307394, 668307402, 668307433, 668311989, 668311997, 668312005, 668312013, 668312021, 668312028, 668312036, 668312044, 668312052, 668312067, 668312075, 668312083, 668312106, 668312114, 668312145, 668614755, 668614763, 668614771, 668614779, 668614787, 668614794, 668614802, 668614810, 668614818, 668614833, 668614841, 668614849, 668614872, 668614880, 668614911, 669395829, 669395837, 669395845, 669395853, 669395861, 669395868, 669395876, 669395884, 669395892, 669395907, 669395915, 669395923, 669395946, 669395954, 669395985, 670829089, 670829097, 670829105, 670829113, 670829121, 670829128, 670829136, 670829144, 670829152, 670829167, 670829175, 670829183, 670829206, 670829214, 670829245, 671777830, 671777838, 671777869, 671820848, 671820856, 671820864, 671820887, 671820895, 671820926, 671904277, 671904285, 671904293, 671904301, 671904309, 671904316, 671904324, 671904332, 671904340, 671904355, 671904363, 671904371, 671904394, 671904402, 671904433, 674341833, 674341841, 674341849, 674341857, 674341865, 674341872, 674341880, 674341888, 674341896, 674341911, 674341919, 674341927, 674341950, 674341958, 674341989, 674869633, 674869641, 674869649, 674869657, 674869672, 674869680, 674869688, 674869711, 674869719, 674869750, 674912651, 674912659, 674912667, 674912675, 674912683, 674912690, 674912698, 674912706, 674912714, 674912729, 674912737, 674912745, 674912768, 674912776, 674912807, 676803081, 676846099, 676846107, 676846138, 678508251, 678508259, 678508267, 678508275, 678508283, 678508288, 678508290, 678508291, 678508296, 678508298, 678508304, 678508306, 678508312, 678508314, 678508320, 678508322, 678508327, 678508328, 678508329, 678508335, 678508337, 678508343, 678508345, 678508351, 678508353, 678508359, 678508366, 678508368, 678508374, 678508376, 678508382, 678508384, 678508390, 678508405, 678508407, 678508413, 678508415, 678508421, 678508444, 678508446, 678508452, 678508483, 678510329, 678510337, 678510345, 678510353, 678510361, 678510368, 678510369, 678510376, 678510384, 678510392, 678510400, 678510407, 678510415, 678510423, 678510431, 678510446, 678510454, 678510462, 678510485, 678510493, 678510524, 678515041, 678515049, 678515057, 678515065, 678515073, 678515080, 678515081, 678515088, 678515096, 678515104, 678515112, 678515119, 678515127, 678515135, 678515143, 678515158, 678515166, 678515174, 678515197, 678515205, 678515236, 678817807, 678817815, 678817823, 678817831, 678817839, 678817846, 678817847, 678817854, 678817862, 678817870, 678817878, 678817885, 678817893, 678817901, 678817909, 678817924, 678817932, 678817940, 678817963, 678817971, 678818002, 679598881, 679598889, 679598897, 679598905, 679598913, 679598920, 679598921, 679598928, 679598936, 679598944, 679598952, 679598959, 679598967, 679598975, 679598983, 679598998, 679599006, 679599014, 679599037, 679599045, 679599076, 681032141, 681032149, 681032157, 681032165, 681032173, 681032180, 681032181, 681032188, 681032196, 681032204, 681032212, 681032219, 681032227, 681032235, 681032243, 681032258, 681032266, 681032274, 681032297, 681032305, 681032336, 681980882, 681980890, 681980898, 681980921, 681980929, 681980960, 682023900, 682023908, 682023916, 682023924, 682023939, 682023947, 682023955, 682023978, 682023986, 682024017, 682107329, 682107337, 682107345, 682107353, 682107361, 682107368, 682107369, 682107376, 682107384, 682107392, 682107400, 682107407, 682107415, 682107423, 682107431, 682107446, 682107454, 682107462, 682107485, 682107493, 682107524, 684544885, 684544893, 684544901, 684544909, 684544917, 684544924, 684544925, 684544932, 684544940, 684544948, 684544956, 684544963, 684544971, 684544979, 684544987, 684545002, 684545010, 684545018, 684545041, 684545049, 684545080, 685072685, 685072693, 685072701, 685072709, 685072717, 685072724, 685072732, 685072740, 685072748, 685072763, 685072771, 685072779, 685072802, 685072810, 685072841, 686963115, 687006133, 687006141, 687006172, 692031384, 692183934, 692183942, 692183950, 692183958, 692183973, 692183981, 692183989, 692184012, 692184020, 692184051, 695275737, 695275745, 695275753, 695275761, 695275769, 695275776, 695275777, 695275784, 695275792, 695275800, 695275808, 695275815, 695275823, 695275831, 695275839, 695275854, 695275862, 695275870, 695275893, 695275901, 695275932, 697123149, 697166167, 697166175, 697166206, 697209185, 697209193, 697209201, 697209224, 697209232, 697209263, 702191418, 702386986, 702386994, 702387002, 702387010, 702387018, 702387025, 702387033, 702387041, 702387049, 702387064, 702387072, 702387080, 702387103, 702387111, 702387142, 707283183, 707326201, 707326209, 707326240, 707369219, 707369227, 707369235, 707369258, 707369266, 707369297, 712351452, 712394470, 712394478, 712394509, 717486235, 717486243, 717486274, 717529253, 717529261, 717529269, 717529292, 717529300, 717529331, 717572271, 717572279, 717572287, 717572295, 717572310, 717572318, 717572326, 717572349, 717572357, 717572388, 722511486, 722554504, 722554512, 722554543, 727689287, 727689295, 727689303, 727689326, 727689334, 727689365, 727732305, 727732313, 727732321, 727732329, 727732344, 727732352, 727732360, 727732383, 727732391, 727732422, 732671520, 732714538, 732714546, 732714577, 732757556, 732757564, 732757572, 732757595, 732757603, 732757634, 737892339, 737892347, 737892355, 737892363, 737892378, 737892386, 737892394, 737892417, 737892425, 737892456, 737935357, 737935365, 737935373, 737935381, 737935389, 737935396, 737935404, 737935412, 737935420, 737935435, 737935443, 737935451, 737935474, 737935482, 737935513, 742831554, 742874572, 742874580, 742874611, 742917590, 742917598, 742917606, 742917629, 742917637, 742917668, 748095391, 748095399, 748095407, 748095415, 748095423, 748095430, 748095438, 748095446, 748095454, 748095469, 748095477, 748095485, 748095508, 748095516, 748095547, 752991588, 753034606, 753034614, 753034645, 753077624, 753077632, 753077640, 753077663, 753077671, 753077702, 753120642, 753120650, 753120658, 753120666, 753120681, 753120689, 753120697, 753120720, 753120728, 753120759, 758298443, 758298451, 758298459, 758298467, 758298475, 758298482, 758298483, 758298490, 758298498, 758298506, 758298514, 758298521, 758298529, 758298537, 758298545, 758298560, 758298568, 758298576, 758298599, 758298607, 758298638, 763194640, 763194648, 763194679, 763237658, 763237666, 763237674, 763237697, 763237705, 763237736, 763280676, 763280684, 763280692, 763280700, 763280715, 763280723, 763280731, 763280754, 763280762, 763280793, 773397692, 773397700, 773397708, 773397731, 773397739, 773397770, 773440710, 773440718, 773440726, 773440734, 773440749, 773440757, 773440765, 773440788, 773440796, 773440827, 773483728, 773483736, 773483744, 773483752, 773483760, 773483767, 773483775, 773483783, 773483791, 773483806, 773483814, 773483822, 773483845, 773483853, 773483884, 783600744, 783600752, 783600760, 783600768, 783600783, 783600791, 783600799, 783600822, 783600830, 783600861, 783643762, 783643770, 783643778, 783643786, 783643794, 783643801, 783643809, 783643817, 783643825, 783643840, 783643848, 783643856, 783643879, 783643887, 783643918, 793803796, 793803804, 793803812, 793803820, 793803828, 793803835, 793803843, 793803851, 793803859, 793803874, 793803882, 793803890, 793803913, 793803921, 793803952, 793846814, 793846822, 793846830, 793846838, 793846846, 793846853, 793846854, 793846861, 793846869, 793846877, 793846885, 793846892, 793846900, 793846908, 793846916, 793846931, 793846939, 793846947, 793846970, 793846978, 793847009, 804006848, 804006856, 804006864, 804006872, 804006880, 804006887, 804006888, 804006895, 804006903, 804006911, 804006919, 804006926, 804006934, 804006942, 804006950, 804006965, 804006973, 804006981, 804007004, 804007012, 804007043, 814209900, 814209908, 814209916, 814209924, 814209932, 814209939, 814209940, 814209947, 814209948, 814209955, 814209963, 814209971, 814209978, 814209979, 814209986, 814209994, 814210002, 814210010, 814210017, 814210025, 814210033, 814210041, 814210056, 814210064, 814210072, 814210095, 814210103, 814210134]

In [ ]:
#@title Verification code
import itertools
import numpy as np

def compute_lower_bound(u: list[int]) -> float:
  """Returns the lower bound obtained from the input set u, which must satisfy min(u) == 0."""
  if min(u) != 0:
    raise AssertionError("Set U must be nonnegative and must contain 0; got minimum value ", min(u))

  max_u = max(u)
  u_minus_u = np.zeros(2 * max_u + 1, dtype=bool)  # Store the set u - u as an array of booleans.
  u_plus_u = np.zeros(2 * max_u + 1, dtype=bool)  # Store the set u + u as an array of booleans.
  u_np = np.array(u)

  for i in u:
    u_minus_u[i - u_np + max_u] = True
    u_plus_u[i + u_np] = True

  u_minus_u_size = np.sum(u_minus_u)
  u_plus_u_size = np.sum(u_plus_u)

  if u_minus_u_size > 2 * max_u + 1:
    raise AssertionError(
        "The constraint |U - U| <= 2 max (U) + 1 is not satisfied. Got: "
        f"lhs={u_minus_u_size} but rhs={2 * max_u + 1}."
    )

  return np.log(u_minus_u_size / u_plus_u_size) / np.log(2 * max_u + 1) + 1.0

In [ ]:
#@title Verification (solution with 2003 integers)

print(f"Size of the construction: {len(solution_1)}.")
print(f"This construction proves the lower bound {compute_lower_bound(solution_1)} <= C6.")

In [ ]:
#@title Verification (solution with 54265 integers)

print(f"Size of the construction: {len(solution_2)}.")
print(f"This construction proves the lower bound {compute_lower_bound(solution_2)} <= C_6.")

# B.7. Packing unit regular hexagons inside a regular hexagon

Consider the problem of packing $n$ disjoint regular hexagons with unit side length into a larger regular hexagon, minimizing the side length of the outer
hexagon. The state of the art can be found on
[Erich Friedman's homepage](https://erich-friedman.github.io/packing/hexinhex/).

For $n=11$ and $n=12$, the best known constructions use outer hexagons of side
lengths $3.943$ and $4.0$, respectively. AlphaEvolve found packing arrangements
that improve these bounds to $3.931$ and $3.942$, respectively. These arrangements are shown below.

In [ ]:
#@title Verification code

import numpy as np
import math
import matplotlib.pyplot as plt


def hexagon_vertices(
    center_x: float,
    center_y: float,
    side_length: float,
    angle_degrees: float,
) -> list[tuple[float, float]]:
  """Generates the vertices of a regular hexagon.

  Args:
    center_x: x-coordinate of the center.
    center_y: y-coordinate of the center.
    side_length: Length of each side.
    angle_degrees: Rotation angle in degrees (clockwise from horizontal).

  Returns:
    A list of tuples, where each tuple (x, y) represents the vertex location.
  """
  vertices = []
  angle_radians = math.radians(angle_degrees)
  for i in range(6):
    angle = angle_radians + 2 * math.pi * i / 6
    x = center_x + side_length * math.cos(angle)
    y = center_y + side_length * math.sin(angle)
    vertices.append((x, y))
  return vertices


def normalize_vector(v: tuple[float, float]) -> tuple[float, float]:
  """Normalizes a 2D vector."""
  magnitude = math.sqrt(v[0]**2 + v[1]**2)
  return (v[0] / magnitude, v[1] / magnitude) if magnitude != 0 else (0., 0.)


def get_normals(
    vertices: list[tuple[float, float]]
) -> list[tuple[float, float]]:
  """Gets the outward normals of a polygon's edges."""
  normals = []
  for i in range(len(vertices)):
    p1 = vertices[i]
    p2 = vertices[(i + 1) % len(vertices)]  # Wrap around to the first vertex.
    edge = (p2[0] - p1[0], p2[1] - p1[1])
    normal = normalize_vector((-edge[1], edge[0]))  # Rotate edge by 90 degrees.
    normals.append(normal)
  return normals


def project_polygon(
    vertices: list[tuple[float, float]],
    axis: tuple[float, float],
) -> tuple[float, float]:
  """Projects a polygon onto an axis and returns the min/max values."""
  min_proj = float('inf')
  max_proj = float('-inf')
  for vertex in vertices:
    projection = vertex[0] * axis[0] + vertex[1] * axis[1]  # Dot product.
    min_proj = min(min_proj, projection)
    max_proj = max(max_proj, projection)
  return min_proj, max_proj


def overlap_1d(min1: float, max1: float, min2: float, max2: float) -> bool:
  """Determines whether two 1D intervals overlap."""
  return max1 >= min2 and max2 >= min1


def polygons_intersect(
    vertices1: list[tuple[float, float]],
    vertices2: list[tuple[float, float]],
) -> bool:
  """Determines if two polygons intersect using the Separating Axis Theorem."""
  normals1 = get_normals(vertices1)
  normals2 = get_normals(vertices2)
  axes = normals1 + normals2

  for axis in axes:
    min1, max1 = project_polygon(vertices1, axis)
    min2, max2 = project_polygon(vertices2, axis)
    if not overlap_1d(min1, max1, min2, max2):
      return False  # Separating axis found, polygons are disjoint.
  return True  # No separating axis found, polygons intersect.


def hexagons_are_disjoint(
    hex1_params: tuple[float, float, float, float],
    hex2_params: tuple[float, float, float, float],
) -> bool:
  """Determines if two hexagons are disjoint given their parameters."""
  hex1_vertices = hexagon_vertices(*hex1_params)
  hex2_vertices = hexagon_vertices(*hex2_params)
  return not polygons_intersect(hex1_vertices, hex2_vertices)


def is_inside_hexagon(
    point: tuple[float, float],
    hex_params: tuple[float, float, float, float],
) -> bool:
  """Checks if a point is inside a hexagon (given its parameters)."""
  hex_vertices = hexagon_vertices(*hex_params)
  for i in range(len(hex_vertices)):
    p1 = hex_vertices[i]
    p2 = hex_vertices[(i + 1) % len(hex_vertices)]
    edge_vector = (p2[0] - p1[0], p2[1] - p1[1])
    point_vector = (point[0] - p1[0], point[1] - p1[1])
    cross_product = (
        edge_vector[0] * point_vector[1] - edge_vector[1] * point_vector[0]
    )
    if cross_product < 0:
      return False
  return True


def all_hexagons_contained(
    inner_hex_params_list: list[tuple[float, float, float, float]],
    outer_hex_params: tuple[float, float, float, float],
) -> bool:
  """Checks if all inner hexagons are contained within the outer hexagon."""
  for inner_hex_params in inner_hex_params_list:
    inner_hex_vertices = hexagon_vertices(*inner_hex_params)
    for vertex in inner_hex_vertices:
      if not is_inside_hexagon(vertex, outer_hex_params):
        return False
  return True


def verify_construction(
    inner_hex_data: tuple[float, float, float],
    outer_hex_center: tuple[float, float],
    outer_hex_side_length: float,
    outer_hex_angle_degrees: float,
):
  """Verifies the hexagon packing construction with a rotated outer hexagon.

  Args:
    inner_hex_data: List of (x, y, angle_degrees) tuples for inner hexagons.
    outer_hex_center: (x, y) tuple for the outer hexagon center.
    outer_hex_side_length: Side length of the outer hexagon.
    outer_hex_angle_degrees: Rotation angle of the outer hexagon in degrees.

  Raises:
    AssertionError if the construction is not valid.
  """

  inner_hex_params_list = [
      (x, y, 1, angle) for x, y, angle in inner_hex_data
  ]  # Sets the side length to 1.
  outer_hex_params = (
      outer_hex_center[0], outer_hex_center[1],
      outer_hex_side_length, outer_hex_angle_degrees
  )

  # Disjointness check.
  for i in range(len(inner_hex_params_list)):
    for j in range(i + 1, len(inner_hex_params_list)):
      if not hexagons_are_disjoint(
          inner_hex_params_list[i], inner_hex_params_list[j]
      ):
        raise AssertionError(f"Hexagons {i+1} and {j+1} intersect!")

  # Containment check.
  if not all_hexagons_contained(inner_hex_params_list, outer_hex_params):
    raise AssertionError(
        "Not all inner hexagons are contained in the outer hexagon!"
    )

  print("Construction is valid.")

In [ ]:
#@title Plotting code

import matplotlib.pyplot as plt


def plot_construction(
    inner_hex_data: list[tuple[float, float, float]],
    outer_hex_center: tuple[float, float],
    outer_hex_side_length: float,
    outer_hex_angle_degrees: float,
):
  """Plots the hexagon packing construction using matplotlib.

  Args:
    inner_hex_data: List of (x, y, angle_degrees) tuples for inner hexagons.
    outer_hex_center: (x, y) tuple for the outer hexagon center.
    outer_hex_side_length: Side length of the outer hexagon.
    outer_hex_angle_degrees: Rotation angle of the outer hexagon in degrees.
  """

  inner_hex_params_list = [(x, y, 1, angle) for x, y, angle in inner_hex_data]
  outer_hex_params = (
      outer_hex_center[0], outer_hex_center[1],
      outer_hex_side_length, outer_hex_angle_degrees
  )

  _, ax = plt.subplots(figsize=(6, 6))
  ax.set_aspect('equal')

  # Plot outer hexagon (in red).
  outer_hex_vertices = hexagon_vertices(*outer_hex_params)
  outer_hex_x, outer_hex_y = zip(*outer_hex_vertices)
  ax.plot(
      outer_hex_x + (outer_hex_x[0],),
      outer_hex_y + (outer_hex_y[0],),
      'r-',
      label='Outer Hexagon',
  )

  # Plot inner hexagons (in blue).
  for i, inner_hex_params in enumerate(inner_hex_params_list):
    inner_hex_vertices = hexagon_vertices(*inner_hex_params)
    inner_hex_x, inner_hex_y = zip(*inner_hex_vertices)
    ax.fill(
        inner_hex_x + (inner_hex_x[0],),
        inner_hex_y + (inner_hex_y[0],),
        alpha=0.7,
        color='blue',
        label='Inner Hexagons' if i == 0 else None,  # Label only once.
    )

  ax.set_title('Hexagon Packing Construction')
  ax.legend()
  ax.grid(False)
  plt.show()

In [ ]:
#@title Construction 1 (11 hexagons): Data

inner_hex_data = [
    [7.227468518248726, 1.406189555916512, 64.96835531696001],
    [9.108934177226912, 6.282338181661271, 82.7839419155727],
    [9.953071339809288, 4.757920599499527, 22.75943751294596],
    [9.119104027817858, 1.3635244317245632, 64.98416780884727],
    [8.202877059101247, 4.72907294863739, 22.702381798755646],
    [7.272127593730819, 3.260296487997907, 202.68420631619662],
    [9.055201371937533, 3.2081226414835964, 82.70304387527644],
    [11.605104986630607, 3.9103231530709373, 120.97244798016398],
    [10.63107376404371, 2.224090366073967, 244.99786687183936],
    [6.427221005557855, 4.784445729508918, 262.6855070070916],
    [7.357815645136251, 6.25309652428302, 22.7012724369423],
]

print("Number of inner hexagons:", len(inner_hex_data))
outer_hex_center = [
    np.float64(8.675544349693055), np.float64(3.861025611329817)
]
outer_hex_side_length = 3.930092
outer_hex_angle_degrees = np.float64(60.96370590710646)

In [ ]:
#@title Construction 1 (11 hexagons): Verification and plotting

verify_construction(
    inner_hex_data, outer_hex_center,
    outer_hex_side_length, outer_hex_angle_degrees
)
print(f"Outer hexagon side length: {outer_hex_side_length}.")
plot_construction(
    inner_hex_data, outer_hex_center,
    outer_hex_side_length, outer_hex_angle_degrees
)

In [ ]:
#@title Construction 2 (12 hexagons): Data

inner_hex_data = [
    [-4.189650776376798, 1.8191996226926388, 27.083656548467786],
    [-4.1762891342159225, 7.064090887028092, 267.1521094125118],
    [-2.742942219672671, 3.9055564132509595, 27.098852933212644],
    [-6.194948523724643, 4.243623380441444, 87.15308054130165],
    [-2.4735773559547236, 6.43056712077689, 27.144807633484653],
    [-3.8735033091694633, 5.272827562035259, 207.12012595575163],
    [-1.040517842157661, 3.2724339122261963, 147.11193087313353],
    [-1.3431542020437193, 5.063486444235776, 27.124004797633685],
    [-5.89197412464008, 2.452466762507662, 207.10748153590382],
    [-4.492206360794807, 3.610020660157929, 327.09918513988094],
    [-2.4402287984241076, 2.1146830723057635, 147.1073847274004],
    [-5.576327284388157, 5.906127413424367, 207.16788311720808],
]

print("Number of inner hexagons:", len(inner_hex_data))
outer_hex_center = [
    np.float64(-3.7029107170331157), np.float64(4.262832492467158)
]
outer_hex_side_length = 3.9419123
outer_hex_angle_degrees = np.float64(219.59113156095745)

In [ ]:
#@title Construction 2 (12 hexagons): Verification and plotting

verify_construction(
    inner_hex_data, outer_hex_center,
    outer_hex_side_length, outer_hex_angle_degrees
)
print(f"Outer hexagon side length: {outer_hex_side_length}.")
plot_construction(
    inner_hex_data, outer_hex_center,
    outer_hex_side_length, outer_hex_angle_degrees
)

# B.8. Minimizing the ratio of maximum to minimum distance

For any $n$ and $d$, the goal of this problem is to find $n$ points in the $d$-dimensional space so as to minimize the ratio between the maximum and minimum pairwise distances. AlphaEvolve found two new constructions improving the best known bounds:

* In 2 dimensions, AlphaEvolve found 16 points with ratio $\approx \sqrt{12.889266112}$, improving the best known bound of $\sqrt{12.89}$ ([reference](https://erich-friedman.github.io/packing/maxmin/)).

* In 3 dimensions, AlphaEvolve found 14 points with ratio $\approx \sqrt{4.165849767}$, improving the best known bound of $\sqrt{4.168}$ ([reference](https://erich-friedman.github.io/packing/maxmin3/)).

The found constructions are shown below. (In the above reference, instead of the ratio itself, the square of the ratio is reported, and we use the same convention.)

In [ ]:
#@title Construction 1: Data
import numpy as np

construction_1 = np.array([(0.8470278707092158, 0.2918282514726773), (-0.23745459443195124, -0.8571978965568918), (0.3557900209463371, 0.279097289488927), (-0.8607342601440215, -0.15089717398730187), (-0.4726876231254993, 0.7174253632783496), (-0.3788383952523786, -0.05470430230958746), (-0.03757671392994971, -0.4082819539247519), (0.4800129548135765, 0.7545395596986599), (-0.000979763279861421, 0.8551512034827423), (-0.7797718797834859, 0.33379016991681987), (-0.6541792267650747, -0.5967802443605281), (0.6682256918291543, -0.6282247082504422), (0.9028949024748079, -0.1963885801084583), (0.41165705912895084, -0.2091194383890215), (0.25280396897954216, -0.8907156599623591), (-0.12757658935294616, 0.3676038779372552)])

In [ ]:
#@title Construction 1: verification
import scipy as sp

print(f'Construction 1 has {len(construction_1)} points in {construction_1.shape[1]} dimensions.')
pairwise_distances = sp.spatial.distance.pdist(construction_1)
min_distance = np.min(pairwise_distances)
max_distance = np.max(pairwise_distances)

ratio_squared = (max_distance / min_distance)**2
print(f"Ratio of max distance to min distance: sqrt({ratio_squared})")

In [ ]:
#@title Construction 1: Visualization
import matplotlib.pyplot as plt

def plot_point_configuration(points: np.ndarray):
  """Plots a 2D point configuration, highlighting closest and furthest pairs."""
  condensed_distances = sp.spatial.distance.pdist(points)
  min_distance = np.min(condensed_distances)
  max_distance = np.max(condensed_distances)
  ratio = (max_distance / min_distance)**2
  distance_matrix = sp.spatial.distance.squareform(condensed_distances)

  max_distance_pairs_indices = np.where(np.isclose(distance_matrix, max_distance))
  min_distance_pairs_indices = np.where(np.isclose(distance_matrix, min_distance))

  plt.figure(figsize=(6, 6))
  plt.scatter(points[:, 0], points[:, 1], color='black', s=100)

  # Highlight max distance pairs (red).
  for i_index, j_index in zip(*max_distance_pairs_indices):
    plt.plot([points[i_index, 0], points[j_index, 0]], [points[i_index, 1], points[j_index, 1]], color='red', linewidth=2, label='maximum distance' if 'maximum distance' not in plt.gca().get_legend_handles_labels()[1] else "")

  # Highlight min distance pairs (blue).
  for i_index, j_index in zip(*min_distance_pairs_indices):
    plt.plot([points[i_index, 0], points[j_index, 0]], [points[i_index, 1], points[j_index, 1]], color='blue', linewidth=2, label='minimum distance' if 'minimum distance' not in plt.gca().get_legend_handles_labels()[1] else "")

  # Customize plot appearance.
  plt.title(f"{len(points)} points on the 2D plane with ratio between maximum distance and minimum distance ~ sqrt({ratio:.9f})")
  plt.axis('off')
  plt.gca().set_aspect('equal', adjustable='box')
  plt.legend()
  plt.show()

plot_point_configuration(construction_1)

In [ ]:
#@title Construction 2: Data
import numpy as np

construction_2 = np.array([(-5.596123673583122, -5.695471111926107, 4.959047320055149), (5.116926601383272, -2.9088174983840958, -2.8844213258624003), (-6.606159121469469, 1.1976559300318508, 4.205115060194249), (0.33263279200499823, -9.151690403732108, 3.5421854577228027), (-0.43984530846509556, 0.2175383405880955, 7.387161767180721), (-1.0482338026105975, 5.032254234659363, 2.3318033382908223), (0.33760218404299963, -8.010249719722893, -3.371583323549962), (4.745850590864617, -3.9828624854138783, 5.248008874626355), (-2.144088866983676, -2.3007882626834237, -6.588159606967483), (-0.8091020398380586, -1.7732214204952712, 0.6786858286172662), (5.334738572382582, 2.1672115465961803, 1.9415133400809574), (-6.83950764442355, -4.338724821681177, -1.8023709245292203), (-5.346496436622179, 2.4644877336392232, -2.570708726021154), (1.5164799724062628, 3.0213278117227174, -3.8718307541919557)])

In [ ]:
#@title Construction 2: Verification
import scipy as sp

print(f'Construction 2 has {len(construction_2)} points in {construction_2.shape[1]} dimensions.')
pairwise_distances = sp.spatial.distance.pdist(construction_2)
min_distance = np.min(pairwise_distances)
max_distance = np.max(pairwise_distances)

ratio_squared = (max_distance / min_distance)**2
print(f"Ratio of max distance to min distance: sqrt({ratio_squared})")

In [ ]:
#@title Construction 2: Visualization
import matplotlib.pyplot as plt

def plot_point_configuration(points: np.ndarray):
  """Plots a 3D point configuration, highlighting closest and furthest pairs."""
  condensed_distances = sp.spatial.distance.pdist(points)
  min_distance = np.min(condensed_distances)
  max_distance = np.max(condensed_distances)
  ratio = (max_distance / min_distance)**2
  distance_matrix = sp.spatial.distance.squareform(condensed_distances)

  max_distance_pairs_indices = np.where(np.isclose(distance_matrix, max_distance))
  min_distance_pairs_indices = np.where(np.isclose(distance_matrix, min_distance))

  # Create the 3D plot.
  fig = plt.figure(figsize=(8, 8))
  ax = fig.add_subplot(111, projection='3d')

  ax.scatter(points[:, 0], points[:, 1], points[:, 2], color='black', s=100)

  # Highlight max distance pairs (red).
  for i_index, j_index in zip(*max_distance_pairs_indices):
    ax.plot([points[i_index, 0], points[j_index, 0]], [points[i_index, 1], points[j_index, 1]], [points[i_index, 2], points[j_index, 2]], color='red', linewidth=2, label='maximum distance' if 'maximum distance' not in ax.get_legend_handles_labels()[1] else "")

  # Highlight min distance pairs (blue lines).
  for i_index, j_index in zip(*min_distance_pairs_indices):
    ax.plot([points[i_index, 0], points[j_index, 0]], [points[i_index, 1], points[j_index, 1]], [points[i_index, 2], points[j_index, 2]], color='blue', linewidth=2, label='minimum distance' if 'minimum distance' not in ax.get_legend_handles_labels()[1] else "")

  # Customize plot appearance.
  plt.title(f"{len(points)} points in the 3D space with ratio between maximum distance and minimum distance ~ sqrt({ratio:.9f})")
  plt.gca().set_aspect('equal', adjustable='box')
  plt.legend()
  plt.show()

plot_point_configuration(construction_2)

# B.9. The Heilbronn problem for triangles

The goal of this problem is to find $n$ points on or inside a triangle with unit area so that the area of the smallest triangle formed by these points is maximized. The state of the art can be found on [Erich Friedman's homepage](https://erich-friedman.github.io/packing/heiltri/).

For $n=11$, the SOTA was $0.036$, and AlphaEvolve found a construction with minimum area larger than $0.0365$, which is shown below.

In [ ]:
#@title Data
import numpy as np
found_points = np.array([[0.855969059106645, 0.0], [0.2956511458813343, 0.0], [0.5084386802534074, 0.7384882411813929], [0.4328408482561001, 0.32744948893270326], [0.6757703386424172, 0.2918847665632379], [0.13872648895775305, 0.2402813272304711], [0.11466976286752831, 0.05646046982765845], [0.647825572940666, 0.609984000793226], [0.3612735110422483, 0.6257440765539699], [0.5851055464997592, 0.13484874011447245], [0.9279845295533241, 0.12473445374461754]])

# Vertices of an equilateral triangle that contains the points.
a = np.array([0, 0])
b = np.array([1, 0])
c = np.array([0.5, np.sqrt(3)/2])

In [ ]:
#@title Verification
import itertools

def check_inside_triangle(points: np.ndarray):
  """Checks that all points are inside the triangle with vertices (0,0), (1,0), (0.5, sqrt(3)/2)."""
  for (x, y) in points:
    assert (y >= 0) and (np.sqrt(3) * x <= np.sqrt(3) - y) and (y <= np.sqrt(3) * x), f'Point ({x}, {y}) is outside the equilateral triangle.'

def triangle_area(a: np.array, b: np.array, c: np.array) -> float:
  return np.abs(a[0]*(b[1] - c[1]) + b[0]*(c[1] - a[1]) + c[0]*(a[1] - b[1])) / 2

check_inside_triangle(found_points)
print(f"Number of points is {len(found_points)} and they lie on or inside the equilateral triangle.")

min_triangle_area = min([triangle_area(p1,p2,p3) for p1, p2, p3 in itertools.combinations(found_points, 3)])
# Normalize the minimum triangle area (since the equilateral triangle is not unit).
min_area_normalized = min_triangle_area / triangle_area(a, b, c)
print(f"Minimum triangle area: {min_area_normalized}.")

In [ ]:
#@title Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def plot_points_in_triangle(points: np.ndarray, triangle_vertices: np.ndarray):
  """Plots the points and the outer equilateral triangle."""
  _, ax = plt.subplots(1, figsize=(5, 5))
  ax.set_aspect('equal')

  # Plot equilateral triangle.
  triangle_patch = patches.Polygon(triangle_vertices, closed=True, edgecolor='black', facecolor='none')
  ax.add_patch(triangle_patch)

  # Plot points inside the triangle.
  ax.scatter(points[:, 0], points[:, 1], color='blue')

  ax.set_xlim(-0.1, 1.1)
  ax.set_ylim(-0.1, 1.0)
  ax.set_title("Points inside a unit-area triangle that maximize the area of the smallest formed triangle")
  ax.set_axis_off()

  plt.grid(False)
  plt.show()

plot_points_in_triangle(found_points, np.array([a,b,c]))

# B.10. The Heilbronn problem for convex regions
The goal of this problem is to find $n$ points on or inside a convex region with unit area so that the area of the smallest triangle formed by these points is maximized. The state of the art can be found on [Erich Friedman's homepage](https://erich-friedman.github.io/packing/heilconvex/).


* For $n=13$, the SOTA was $0.0306$, and AlphaEvolve improved it to $0.0309$ (Construction 1).
* For $n=14$, the SOTA was $0.0277$, and AlphaEvolve improved it to $0.0278$ (Construction 2).

The constructions found by AlphaEvolve are shown below.

In [ ]:
#@title Verification function
import numpy as np
def triangle_area(p1: np.ndarray, p2: np.ndarray, p3: np.ndarray) -> float:
  """Calculates the area of a triangle given its vertices p1, p2, and p3."""
  return abs(
      p1[0] * (p2[1] - p3[1])
      + p2[0] * (p3[1] - p1[1])
      + p3[0] * (p1[1] - p2[1])
  ) / 2

In [ ]:
#@title Visualization function
import matplotlib.pyplot as plt

def plot_heilbronn_construction(points: np.ndarray, min_triangle_area: float):
  """Plots a set of points and their convex hull."""
  plt.figure(figsize=(8, 8))
  plt.gca().set_aspect('equal', adjustable='box')

  # Plot the points.
  plt.plot(points[:, 0], points[:, 1], 'bo')

  # Plot the convex hull.
  for simplex in ConvexHull(points).simplices:
    plt.plot(points[simplex, 0], points[simplex, 1], 'k-', linewidth=2)

  plt.title(f'{len(points)} points on or inside a unit-area convex polygon with smallest triangle having area {min_triangle_area}')
  plt.axis('off')
  plt.show()

In [ ]:
#@title Construction 1: Data
import numpy as np

construction_1 = np.array([[0.7262503272554176, 0.4388890568372681],[0.5700515606090156, 0.3079437775114338],[0.6327871120895331, 0.5328479759933831],[0.6416207987804122, 0.31270211238697715],[0.5148490471164644, 0.3395137443428024],[0.5679223148906077, 0.5390628289790125],[0.495131275787839, 0.41235659338159497],[0.5800095646202962, 0.46771551597629857],[0.6499329141672348, 0.3910833999601802],[0.5584150294032443, 0.38652266849736683],[0.7052096091448784, 0.48702642736130247],[0.7055856016358004, 0.3662981535556451],[0.5096822331932992, 0.49886360333134927]])

In [ ]:
#@title Construction 1: Verification
import itertools
from scipy.spatial import ConvexHull

min_triangle_area = min([triangle_area(p1,p2,p3) for p1, p2, p3 in itertools.combinations(construction_1, 3)])
convex_hull_area = ConvexHull(construction_1).volume
construction_1_score = min_triangle_area / convex_hull_area
print(f'Construction 1 has {len(construction_1)} points.')
print(f'After normalizing so the points lie in a unit convex region, the minimum triangle has area {construction_1_score}.')

In [ ]:
#@title Construction 1: Visualization
plot_heilbronn_construction(construction_1, construction_1_score)

In [ ]:
#@title Construction 2: Data
construction_2 = np.array([[0.20379865808395928, 0.38641175027464647],[0.7322582251226939, 0.8023825499098627],[0.9235020285628059, 0.5636539878831605],[0.5406929358339732, 0.43236480927414933],[0.5107352242483283, 0.23351702480518888],[0.33090123099682506, 0.5368686340109353],[0.6996892191561037, 0.5984909224208574],[0.05974508135403103, 0.8227363439100202],[0.01267557937837638, 0.5684070460592283],[0.7656257377750475, 0.19333896865308486],[0.18465416566003395, 0.9316973270780137],[0.4813032999198722, 0.9273920472734931],[0.3521502387277767, 0.7716110101695554],[0.9232590363682629, 0.39434874250593155]])

In [ ]:
#@title Construction 2: Verification
import itertools
from scipy.spatial import ConvexHull

min_triangle_area = min([triangle_area(p1,p2,p3) for p1, p2, p3 in itertools.combinations(construction_2, 3)])
convex_hull_area = ConvexHull(construction_2).volume
construction_2_score = min_triangle_area / convex_hull_area
print(f'Construction 2 has {len(construction_2)} points.')
print(f'After normalizing so the points lie in a unit convex region, the minimum triangle has area {construction_2_score}.')

In [ ]:
#@title Construction 2: Visualization
plot_heilbronn_construction(construction_2, construction_2_score)

# B.11 Kissing number in dimension 11
The kissing problem asks how many spheres can be arranged tangent to a given sphere, if they all have the same size and their interiors do not overlap. The maximum such number in $d$ dimensions is called the $d$-dimensional kissing number.
For $d=11$, the best known lower bound was 592 ([Ganzhinov (2022)](https://www.arxiv.org/abs/2207.08266)), and AlphaEvolve improved this to 593.

This section contains the description of a sphere packing in dimension 11, discovered by AlphaEvolve, which shows that the kissing number in dimension 11 is at least 593. That is, it shows that one can arrange 593 nonoverlapping unit spheres in dimension 11 so that they all touch a common unit sphere. This follows from the following lemma, which is proved in our paper
[Novikov et al. (2025)](https://storage.googleapis.com/deepmind-media/DeepMind.com/Blog/alphaevolve-a-gemini-powered-coding-agent-for-designing-advanced-algorithms/AlphaEvolve.pdf).


**Lemma.**
Let $C \subset \mathbb{R}^d$ be a set of points satisfying
$0 \notin C$ and
\begin{equation*}
    \min \left\{ \|x-y\| : x \neq y \in C \right\} \ge \max \left\{ \|x\|: x \in C \right\}.
\end{equation*}
Then unit spheres centred at $\left\{\frac{2x}{\|x\|}: {x\in C}\right\}$ form a valid kissing configuration in dimension $d$. In particular, the kissing number in dimension $d$ is at least $|C|$.

In [ ]:
#@title Data
import numpy as np
import itertools

sphere_centers = np.array([[ -126140549599, -4345934944399,   470267207045,   263582420739,
   -214242641164,  6323834592662, -5619609064852, -1309426459793,
    402713598698,  2669019955225,  -434830723701],
 [ 4995872033121, -3686427151476, -2854162550811,  1374657172375,
  -1039798804579,  2439226626247, -3321352498532, -4613529709005,
   1873895977070, -2351434851488, -1740061033492],
 [ -148878709727,  6009198311530,  3423705484214, -3908379065200,
  -2039242115428,   740038679089, -1903248798804, -3229808619747,
    -67448542119,  2608302674396,  3361875535061],
 [-3613653483807,  -421965187481,  4386947861356,  4668066279141,
   2659643185631,  -793481959702,  -782545296485,  3044001988734,
   -210044264644,   732930704465,  5250176454220],
 [ 2145601851578,  2286907266338, -6008630565540,  -951491851263,
  -1128202168949, -2351035905822,   338686846605,     2773377260,
  -3167080097124, -2121084647622,  5631442707990],
 [ 2905871765916, -8266251805063,   212038997927,  -422612434475,
  -2967768531066,   917651742589, -1199305228751,  -428118279913,
   1046645354339,   629591995938,  3199722319491],
 [ 4101374281925,   140190714949,  7791359990956,  2617604236962,
   2117466506344,  1449617600436,   733498008403,  1078801552552,
  -1108251520223, -2353519449875,  -740114970023],
 [-4114734847133,  -711568804977, -1096919699416, -2179625341981,
  -3277074074174,  3846872693149, -1343845317573, -4029410886259,
   -760938481950,  4323960137424,  3708537789894],
 [ 3749853727043,  -920834513083, -2932082478667,   453628989161,
  -3091441942608,  6398413670681,  2921263471507,   652306845488,
   3309571127773, -1260515535742, -2071139339072],
 [-5502577676278, -1227448640831, -3806962231330,  -700239558766,
   -634751583528,   218595111082,    36644542589, -4043060252879,
  -4497661105875, -2852762755592,  2840093077865],
 [ 6431155805256,  2472183772617,  1852141756766, -3447375436498,
  -2150831199996, -1035995564180,  -582868441525,    -7570292152,
  -2212911701699,  4854953687339, -1645298702536],
 [ 5457305326303,  1209487773311,  3838331900010,   672841580707,
    640009753701,  -223039211201,   -29010721761,  4023147410765,
   4562777370943,  2817042290572, -2858123518221],
 [  947994596354,  -493063270445,  -891551370805,  1943900881453,
   4808222709486,  4576541247433,   121556039265, -2928496070412,
   2867556918429,  5676016954336, -1090838534345],
 [ 2487751367449, -4435069885581,  1203527162971, -7039059911283,
  -2789489469568, -1507124796768,   408730098019,  2199308898207,
  -2690789476039,  -886790999066,   246907513281],
 [ 1258393365327,  1558236082994, -2901279533502, -4588804764838,
    608788356464,    99979762711, -2072852177408, -5237300731547,
   2206084299760,  5428740521977,  -262884833863],
 [-2167736272270, -3517473061426, -2581444334945, -4875068237304,
   1316469763812, -5045504207465,  1421790102552, -1042617987235,
  -2433031020177,  1273290801609,  3828642503948],
 [-1748341091839,    84738171505,   491018476370,  4552281419561,
   -872666527835,  3684504527423, -4225299894261, -2607643958170,
   1747681580888,   -38267500797,  5824587161476],
 [-1190649965834, -4307368301932,  -590143473885,  1188487865738,
   1457925043503,  5074991372962,  -835020531624,  4328426802452,
   1938681290116,   658478446157,  5173244484302],
 [ 1157198248631, -3276093261966,  4564492907322,    71866472319,
   6569376668375,  2635857316126, -2535627189471,   666339223157,
   -370257470349,  2316182402253,  2146785163984],
 [-3008255020828,  3938057143766, -5277192922587,   593373194319,
    313114162483,  4123287795940,  1308453155442, -2544545673675,
   1482541632927,  -605248563516,  4403486139737],
 [ 2953917709599,  -440143619596,  -971489179779,  2328335269090,
   2051703342736, -1981036966235,  1096402607228, -8032142516776,
   1008884833142,  2051765017859, -2374176269185],
 [ -260449853094, -1557229191143, -2449904390021,  7358451024369,
   -307525874663, -2110837486887, -1514682853981, -3642026943603,
   -341788837820, -3181174130215, -2648178657093],
 [-4525545210765, -4561293261761, -2207465574022, -3656150761762,
  -1355527319115, -1325108959607, -2606830665241,  4286163870684,
  -2272423058824,  2270296581664, -1181276273370],
 [-4989184462869, -2211501213310, -3272216512438,  2932576526381,
  -4053121684822,  -741725216911, -1173137805746,  2181007677136,
  -2973820944500, -3480396984635, -2615745041603],
 [ 3602451243212,  1363170942462, -5335236817537, -5099315765146,
   1671801957653,  -733157544910, -1557322490939,  3489905697601,
  -2568635080802,  2482181047248,     9540974612],
 [ -117106507206, -1557986758662, -1613271333904,  3599924179366,
  -4726770521248,  -122491678154,  3003684160051, -2037648443680,
  -1689648678287, -6017572885613,  2720160888071],
 [  656948042076,  5303758392177,  1223385282487,  4208826988200,
  -2322770619544, -1931938705746,   194867179198,  -149785416225,
   -975234488353, -6149937849011, -2065621641353],
 [-2822884624434,  1200686951368,  -740362280165,  2549091748238,
   6473313457564,   198867059897, -5879821576926,  -497127882332,
   -583143250935, -1042480487756,  2313887084841],
 [ 2071972627918,  1689375919671,  2561773181285,  -215304283110,
   3678351753570,  -481570249929, -4708236211267,  -713440165548,
  -2024413488114, -6510658512889, -1821622848642],
 [ 1540576902788,   553319845306,  5192033407329,  3790472513109,
   -989994836245, -6751185485237,    63271511119,  1311050643347,
   -364367110835, -1981107885558,  1912214169188],
 [ 1043725858252,  1320396911651, -3526919864724, -4284357877757,
   1470366412833,  4775346083201,  1368745214981,  3701799302088,
   2580249413700, -3202483086523,  2985971881393],
 [ 2328981318412,   782277770357,  1613319529120,  6708838015807,
   2409644663632,  -164697695288, -2876391291223,  1405607425611,
   2563177561828, -4390790783825,  2102566367890],
 [-2898694569620, -1993213074975,  4767991564340,  -549183545871,
  -1716073975618, -1376827268341,  3962365401068, -1804381043737,
    364314410734, -2366368669369, -5921165923758],
 [-2821980692939, -2907092870119,   526484763216,  -283219412226,
   1816656338961,  4630761564997,  5094816643786,   822612609763,
  -1628794974160,  4497378787802, -2995063685088],
 [ 3981893553028,  6726656929267,  4509428321078, -1718973949677,
   1235696414272, -3105774701931,  -561577844505,   806709024917,
    670699852230, -1703554165369,  -340483077176],
 [     958740882,   492343935025, -4877156942860,  3610017499034,
   1303862660779,  2282799096236, -4071872206188, -4319287760002,
  -3382516964626,  3040863514676,   320653772456],
 [ 2668842066670, -3328224345221,  1245442050272,   807268381271,
   6338371398067, -1028063433204,  2718246800023,  1487663933151,
    641493721090,  2991619189786, -4404795224958],
 [  491355192733, -1231352019317,  2214609261146, -4246171762475,
   3060427833343, -6012660473699,   369697486180, -1926711613647,
   1222190747138, -4895424389279,  -694078468931],
 [-3936231732764,  3646035008845,  3917837393650, -2302689585385,
   -631606199075, -1190907291088, -1462454097742, -1026315700574,
  -3403121373084,  4358068529006, -3870064655165],
 [ -891693194815, -3159218172959,   -36815057657, -5392159737721,
   2695159539664,  4846777170940, -2774317263822,  2475655478805,
  -3507372987730, -1040335371092,  1477389122043],
 [-1909886271559, -3025194791002,   273889648930,    22368016637,
   1769343232414, -6711034204699,  1807271442431,  5498553429397,
  -2113841429687,  -965335059539,  -236521072887],
 [ 1421740442238, -4445189941322, -6335315158518,  -671536406806,
   2646092406700,  -638731456569,  -171872290986, -2000918638949,
   2251948982557,  2831806435076, -3619165775254],
 [-3348523808774, -3469324536046,   -76708109001,  1992777744163,
  -5049711606642,   581253431088, -3927998298964,  2296310725868,
   2561198403600,  3752242898555,  2365461437834],
 [  779042434547, -1416461625219,  1452341221883,  1439971242011,
   1285662467259,  2679379627727,  1926099800655, -1269351586896,
   2663459313927, -8084575368127, -2568350771324],
 [-3980831743069,  3637325407505, -1572468831509, -2438291232144,
  -3064062866153, -2476525715560,  4274880397145, -2716987458463,
  -1175083650793,  1660991539240,  4145711223394],
 [ 6967450268144,  2625304133625, -1401474346935,  1995863610145,
    763569629212, -3804173165507,   997773279985,  1860188439504,
   1442735815840,  1895879894694,  3664470120161],
 [ 3015840597743,  1234709422899, -6952463247052, -1298276880914,
  -1224429458570, -1308306783583,   972871140414,  2450262921520,
   5290015394161,  -560255643005,  -947955712831],
 [ -936946977342,  1116679407669,  -396124541576, -5766839382952,
   2927744211058, -1495168010655,  2861812090255,  3711225318549,
  -3673521240929, -3839839100081,  1858038850084],
 [ 1308510697687, -4987098459834, -5143597325210, -1869372147752,
   -978771635962,  3179840980566, -3422701861744,  3615532701633,
   1765018082471, -1744159035660, -1204896731728],
 [-1646730796381,  4421637104002,    37254831639,  4273667291949,
   -655020690827, -2640710658714,  1397627767521, -1309174476974,
   1380616410107, -2725730878362,  6250309303605],
 [ -627474373499,  -451185628535,  1125092239854,  1228838206359,
     26093010133, -2939555208566,  3370434196692,   667250715880,
  -7209365367712, -1252881093849,  4757673344364],
 [ -797420603404, -1227566289334,  3086021511548,  -505080489488,
   5478728185869,  4126710775002,   391076993732,   139481050671,
  -3647775845204, -2041750080177, -4835799186849],
 [ 4092334232882,   729220292791, -1168955935796, -3085250972755,
     82524549053,  5275269956420,  3666568911398,  2396726507160,
  -4862220845505,   358540800263, -1021153633823],
 [ 1756033032922, -1110377613430, -1249242600393,  2221908240877,
   -400217238665, -3426298784081,  6127013124122,  4593495905370,
   3265502690632,  1973356712550, -2021902581270],
 [-3725619678320, -2329459894461, -5070699197459,  1534301307867,
  -1037135676352, -3205158454204,  6159879240390,   559487604350,
  -1262339301196,  -862647461628,   826929767550],
 [-2274391587253,  -270713035752, -1330852865067,  -377690887956,
  -2008376261801,  4177033325257, -2292376198981,   547802074232,
  -7458187593121,  2184009598005, -2325794304528],
 [-1452869174536, -1495810745460,  3812294435170,  4056541456396,
  -1452578046306, -4834849869320, -1310486979894, -3907635677827,
  -2007229588715,  2874984727132, -3145687074383],
 [ 1794443008446,  -307543586686,  3579316950366,  4062541899933,
    930653960952,  1957509312467, -2003233153037,   999980394766,
  -3454594428395,  5958959150112, -3196146826361],
 [-1460561567534,  -916399636812,  4669810481354,  -882217985126,
   2825371943033,  4288562595538,  4110001703233,  3676245445344,
  -2776098864617,  -863483729868,  3034437958314],
 [-3278999167478, -1273384556037,   681284367484,  4877315854280,
   3219855412980, -1975019416928,   259552774384, -5776746289350,
  -1601605704779,   635789280058,  3563508544421],
 [ 3932398091349, -3647552677504, -3915156254426,  2300359279437,
    632049048201,  1190512933335,  1463079709812,  1024625838059,
   3408695534850, -4361112761007,  3868556093990],
 [ 3405094292821,  1309788195129,  2321854096188, -1882365030199,
   4076617425181,  6149102664930,  -142554192789, -1909405757679,
   -914754807934, -3485849481473,  2582854034473],
 [-2463493552091, -6182166095073,   698027052685,  5496165743777,
  -2223299965981, -3647879802948,   628799524443,   494699815054,
  -1002913242476,  -295445251725,  2243571445095],
 [ -729861928381,   580654590161,   736639591977, -1815111899007,
  -4839349528509, -4562434137639,  -151406490905,  3024687926959,
  -3192175323619, -5507105846599,  1169574501828],
 [ -944932369434, -3799932528535,  1269141877026, -1606029533014,
  -5042197520192,  1753508262775, -5757405933996,  1676387264954,
  -2659952371999, -2907761426523,   700425645446],
 [ -313412211989, -1472067839495,  2014918908248, -2994085643890,
  -3631601113505,   798675905998, -5151770012594, -4504700612925,
   4645803566421,   250361628870,  1550709051811],
 [ 2806524625245, -4911847282592,  2447813070402,  -867370776942,
   2711443083233, -1173006619677,  4693314097340,  1244183268233,
  -3834905781184, -3651786699409,  -952863639178],
 [ 5102102592725,    26765691116,  -331028209281, -2142605004855,
    884206303205, -6425311667055,  -950525299835,   262446895670,
  -4980233069174, -1053283462981,   562620518775],
 [ 1098954479782, -1109038396398,  1216752926622,  2017980461121,
  -7349092644345,  3487054908779,  1652822263220, -2099842431090,
   2295686494056,  1022158793060,  3519196027836],
 [-4358075667015, -3089508657759,  4802679891308, -4810354941838,
   4228949294611, -1392241901612,  1450152791677,   688600723787,
   -547780846332,   728656385898, -1423515444254],
 [-3368253979501, -1295226691162, -2347476680864,  1904689227772,
  -4081008654975, -6145377037947,   136414911035,  1925567171135,
    861357305081,  3514943751022, -2568129787920],
 [ 2231100299994, -4273492332869, -4596691742304,  3642873481075,
  -5001877658405, -1406691322979,  2412453357834, -1290390423599,
   2512433823420,   586628501749, -1104670360982],
 [-2575376667766,  -960065509919,  -340750042435, -3074326806221,
   5178574195494, -1775696830323, -2226386336847,  5177679668223,
    209007473419,  1808719719559,  4222211433121],
 [-3578650785329,  -760578973239, -3477963098591, -2048910481829,
   3686410787928, -3078459201831,  3150278587757,  2610639587537,
    384617180163,  5179605695847, -1880968201088],
 [-1453680890271,  -673688713741, -4079118060194,  6145074907773,
   2493898205845,  1386249452566,   358139860104,  1448264243842,
  -3561849396434, -1272570622967,  4284699431295],
 [ 3328068080720, -2292691401219,  1230258674419, -4178966504561,
   1952019534393, -5264768742287,  1386218014402,  2997466555571,
   2074496226701,  4120699917727,   985876677105],
 [ 2539767610843,    94765789165,  3854189084765,  1251736924810,
    288435856169, -2041205772562, -2845981392459, -4127782598606,
   4941323231817, -2767998454739, -3956333511561],
 [ 1067212670401,   -37369907419,  1058370258904,  -923482582767,
  -1672287471130,  1249103492221, -4785063288729, -5636554728192,
  -1540216098658,  2012650649787, -5607074089976],
 [ 2693775044582,  2069745791644,  2364479048946,  -861476754182,
   -369013705992, -6661486740994, -1991744606252, -3971515305028,
   1993645640090,  3364267727602,  1606968451704],
 [-1060152337718, -1462090647628,  4542812980901,  -128973585724,
   4479127796943,  2331049535271,  4969221611534, -4599291027477,
   1873218159741, -1057368339060,  -338512596613],
 [ 1828308410223, -5297151224782,  4618727669960,   -42450901494,
   1921798353915, -4281708414571,   722583002058, -2788707550640,
  -1892464744087,  1898148718778,  3123025242617],
 [-2974029255356,   -91718543918, -2789616442708,  -417822031141,
  -4472352298733,  4615977419360,  5600790978158, -2574878657168,
  -1894939235615,  -308842022774,  -442698710154],
 [ 1645018045050, -4422289912096,   -36103964862, -4274703871227,
    655235768532,  2640591254105, -1397378950407,  1308418198678,
  -1378166081991,  2724363162782, -6250975307286],
 [  433691886277,  2523890861790,  2284499873225, -3703627868154,
   -518512695555, -4938318927311,   257841134127,  1840194784536,
    184975564588,  -457909300815,  6793639833180],
 [ 3325316992112, -2486276617276,  1123273178383,  4294677510387,
  -1936032844161, -2776149438784,  -572327617931, -4803422897516,
   2993538631796, -1489999107747,  4125247597048],
 [ 2393744590379,  1049137728331,  5122417881385, -1088629195634,
   5105537095120, -2435589424350, -1707333453883, -3641898816074,
  -2375980295032,  1693223337093, -3009270903611],
 [   28402196897,  1416027777891,  3440429273442,   203961451796,
  -3940869930791,  2347105807829,  3199355476507,   922594830306,
  -4951120644270,  4761531968667,  2608391131483],
 [ -813227623899, -5357941337067, -1112669738057, -4304097048545,
   2345298447370,  1911013266658,  -174960144340,    85301205749,
   1204772867285,  6020219044345,  1995755808449],
 [  996433361992,  3887894764559,   -61480148325,  3439374904618,
   4912048980888, -5693090961926,  -928200494105, -3021190348807,
    299266571934, -2320681651943,  -233525862680],
 [ 2453568242579, -1987414549672, -4478808100402, -1731895787954,
   5278273305930, -3861728068050,  4222749196955,  1356061448728,
    966666015199, -1824003862323,   515695447649],
 [-2664724735182, -1495474258595,  2944595283119, -2919860019644,
  -2308099678557,  -818004793386,  2276432949055,  4387646832659,
   2527039579293,  4206496484419, -4353847210838],
 [ 2103427320648,  3064498166470, -2140607330478,  -214240562173,
   2185849739279,  3694612769131, -3992827456340,  1545977029970,
  -1036921145639,  4214246024451,  5095140041841],
 [-4722758966585,  1206959806325, -2218480002577, -2968990772048,
   4375690183492,  2762753454305,  -688043457918, -1384779569811,
    297008499206, -5649005618160, -1154239157355],
 [-2609408155367,   294141147806,  2298399803607, -1460949423253,
  -6206872086624,  3044769045395,   637152553136,  1635378523205,
   2401015430092, -4948485353184, -2135126898687],
 [ 3867944404697,  2385613461166,  4971200333675, -1447973537461,
   1019789091334,  3219226368043, -6182937104131,  -497220049095,
   1055477569740,   975358261409,  -770526134964],
 [  768762538555,  -861169573712, -3921515618745,   -22343685772,
  -3124204955684,   646496969665,  2154751469802,  5008107566907,
  -2780174801716,  4768474481990, -3594186429160],
 [-5829157725768, -1845011523627,  5847631650732,  -217123128185,
  -2762905325685, -2720689317598, -2609714245707,  2013419676738,
   1074499522941,  -866115882344,  -754946967281],
 [-1311298215779,  -486042628530, -2763893228432,  1526263490041,
   2632583689192, -3621344081841,  1368629186733,    20242687756,
  -3847994704863, -7116704442527,  -838416785108],
 [-4079307778929, -2848117033354,   180600291286,  3916227562361,
   1526038123223, -1150669460319,  3432721011725, -2556067021564,
   4840526733645,  2294106198479, -3035542058866],
 [  129849177296,  -895608414736,   739412179265, -3313217288439,
   1347265236537, -1350474981128,  8576800884753,   753529921447,
  -1070437463712,  2225825733472,  1946412591514],
 [ 1022976878190,   -45968183236, -4432158106829, -1058813047828,
  -4104820558761,   -36539537494,   952233064667, -7327066689318,
    687280832481,  -684194666759,  2408791472363],
 [  726100104866, -3850927701095, -5622396324275, -2466174786399,
  -2197077396011, -5197407057482,  -329370102200,  1038406981314,
   -989600011579, -2648022596253, -2435463239897],
 [ -165613374687,   552413143883, -1161665499778, -3926523097504,
   7762169581077,  3071383417728,  3006518773230,   704634129572,
   -541373108803,  1699918236493,   705136590918],
 [-2463707289161,     7334104944, -5254000589214, -1875851987073,
    575650872497,   193540586190, -4724021792447,    88382347075,
  -1404280131318,  -733098432327, -6132233566685],
 [-1068227955381,    36991913966, -1057698833338,   922844025654,
   1672425887977, -1249208449583,  4785238023229,  5636085487623,
   1541691946288, -2013439331427,  5606684229854],
 [ 2220517143322,   198705222749,   247604570500,  1479799498957,
   2489049282415,   814044846502,  3030144160090,  4590197712109,
   -561532095582,  7075362333048,  2300632700445],
 [ -156780111086,  2736931326151,   965004659863, -1831034245257,
  -3726675002694,  5210673474784,  1453882908988,  -361014855400,
   -136594164620,  3121012600730, -5929417932372],
 [-2949947628793,  2988361859580,  -212814868864,   903455432497,
  -2046005627366, -1876329775279,  6503528569315,  -277099644684,
  -1478109831366,  3366507572296, -4231829163727],
 [ 2045601175700, -1564278540916,  2774622143877,  -964561825269,
  -1253678239635,   552184872386,  3180838585545, -2911804386954,
  -6180323347514,  2204084547490, -4605681208142],
 [   55250421914, -7350188953636,  -641569884634, -1083937595197,
  -1091999530200, -2998210223345,  3830530928145,   490910348930,
  -1415454125947,  3413215490274, -2374004857697],
 [-5017343821737,  3677836937740,  2869185859152, -1387660752676,
   1042242648295, -2441379816081,  3324919305925,  4604118406050,
  -1842614982590,  2334378964167,  1731413185941],
 [ 4334119519474, -2595497670494, -1222943617868,  2697227087236,
  -2416592134269, -1744364781539, -4568500959292,  2410005633708,
   5468207643593,    52150443924,   492042756756],
 [ 2658920838129,  5641550619343,   442931548469, -1551669065028,
  -5542562030946,   579266633195, -3639887265300, -1186440616563,
   1501365270434, -1381312597597, -2936851176766],
 [ 1437696918439, -3901230133907,  5939123390411,   696495681099,
  -5307342745969,  -526543773849,  1855636404266,    60757669874,
  -3092578936240, -1971580173004,  1269547930945],
 [-1279198726336, -4494764829706,  1092293047395,  2850892770464,
  -3897572176262,  1684173370526, -2252708806981, -6118712302944,
  -2711864726485,  -639474370511,   732162729451],
 [ 2571548192351,  1062970499840,  5931954641540, -4827888529490,
    682286003349,  -442106562559,  2804703382020, -2023617203399,
  -3776836195923, -1111938053742,  2374317197047],
 [ 1987507400483,  2047084168016, -3002059435990,  1244072815805,
  -2542649156248,   176634825883,  7225040063273, -1155681023105,
   3321068587346,  -876174069295,  3077163956589],
 [-1461334920223, -5683558990067,  1458287405745, -3140510195587,
   -547138900823,  2658674539715,  3374598110250, -2852546789839,
  -4372452460752,   550693577616,  2694030473882],
 [ 2789684933477,  2418905805336,  3920543907820,  -531513340616,
   2693654810086,  1969467153943, -1396197508824,  5134579301295,
   2611529296474, -1045763809032,  4832372637798],
 [ 3650744567980, -4644629683627, -3699694543658,  2612553277032,
    519883653911, -1881417480910,   943481947691, -3790800920758,
  -4416251706114,  -408269127764,  2417850592918],
 [ 1436248473698, -6015909485541, -1585609778223,  -728279221654,
  -4061556287510,  2506355530637,  1623620972154,  -769589298542,
  -2935741997700, -3367508117025, -3568797013361],
 [ -644923126395, -2161877991026, -2767262539037, -5203279765784,
  -2774283868680,   612706970955, -3061605161094,  2123312292904,
    925203316429,   593138457327,  6084138308239],
 [  252405123934,  2206139084474, -4595639419590,    85111351867,
  -4782930422763,  -721735987789, -5272745523900,  4265015464793,
  -2085605756537,   408606626432,  -189359683156],
 [ 7053112995937,  -721686034826,  1665017815644,  3883332184132,
   -338594544788,  4332759688617,  -659180497339,   353063806376,
    -22371930753,  2334890792379,  2642228581106],
 [-2358290547732, -5112615059945,  1377813744578, -1209079881217,
   -964930564226, -3896741748046, -1290056495803, -1441252411451,
  -6180287803578, -2142260186258, -1515589161384],
 [ 2501097637297,  4123491123748,  4271158567686,  3389529547678,
  -3347701183653,  -578176305504, -2597493200365,    60111207331,
   4259394584395,  1251306377062,  3001572968103],
 [-3905847290960, -1488771800312, -2053247525676,   -90486946243,
  -1054900276341,  1382249202756, -2234835133169,  -955881342876,
    812023295000,  7153556583528, -4188021476324],
 [ 3894547097085,  2814755872739,  3839420830742,  1947626151590,
   1950087300099, -1442382122810, -2803223032330, -1748644554391,
  -3530905638874,   850911700296,  5327737153655],
 [ 5123151777017,  -404226532300,  3075206595856, -5134854073886,
   4638569941373,   465818284017,  -707657296426,  3205407676666,
   -665340777495, -1290658656591, -1774565213611],
 [ 1916117580566, -4093266494671, -5821590032850,  1263077964827,
   -497586117275, -2100172023666,  -682742738160,  3330924624014,
   -388213188429,  3880245331412,  3558210516981],
 [ 3401202788258,   810363437763, -5274150637157,  -928677899866,
  -2090316636540, -4612443348259,  2952211444677, -1956336686949,
  -1824801816226,  4161313173728,  -512573096978],
 [-6969575388595, -1819483463926, -1381333443046, -4002918516902,
    798912659548,  2344670318129,  3174898396419,  2144891020890,
   1607026711522,  -356366181421,  2580164864728],
 [  276997024511,  1029528219459,  -412748310462, -1893341701947,
   -764769161531, -2085125184027,  -779580331768, -5726504265913,
  -6450058000710,  3162432669128,  2273938482379],
 [-2006373209666, -2054493934083,  3015218165908, -1255518043090,
   2544926372296,  -178539759170, -7221911766451,  1147400549023,
  -3293605634553,   861176827029, -3084659981606],
 [ 1304027205357,  3324730053896,  -243321857757,  5642880845393,
  -2736638803105, -4801593338556,  2713255478063, -2296124924557,
   2924603069862,  1363744529287, -1323946982056],
 [ 1313958374220,  1081499628340,  4072809931350,  -172775867819,
   6779271255119, -3685975109542,  3079865241934,  1989147097123,
   -817455291683,  -329359888877,  2592743966517],
 [-1567783876789, -7384170965839,  3311780178305,  1327667060418,
    389718199022,  2636863689822,  -630307270725,  3399475322545,
  -3073903523076,   297542749995, -1299777063762],
 [-1504145081860,  -538826415777, -5217302414788, -3768713739576,
    985738688357,  6755155777720,   -69482645562, -1295245807684,
    311515406667,  2010148660475, -1897294536706],
 [-2082074589211, -4576770136906,  3060705179466, -1792361754528,
  -1929011305802, -1520940025193,  2120807677184,  4188855567239,
   -838713657044,  2987248595378,  4942862485457],
 [-1118160218577, -3932752314296,   145244832515, -3513492678208,
  -4897566395071,  5677719152947,   949385945492,  2966833768506,
   -121064747752,  2223158760683,   185123993948],
 [-2885810971379,  3921536918930,  1089916425343, -3072576745427,
  -1665297542424,  7397430565361,    96402644487,   723506423342,
  -1998710348122,  -790422177651,  1737077257039],
 [ 4824082055808,  3789854052554,   424544297045, -3281115028812,
  -2102394783795,  1626782367763, -3185969944244,  2411113323944,
  -3913025740612, -2319500673832,  2774360181845],
 [ -947768904213,   842278199349,  5370428414965,  -548373867687,
   3148686234987,  -737691635251, -2420660381267, -3985317011510,
   2700872971740, -3297024906174,  4345599604186],
 [-2483741090108, -4116493995975, -4283394863320, -3379040828527,
   3345662921050,   580094565410,  2594282927079,   -52488891633,
  -4284802307248, -1237329360849, -2994423995937],
 [-1098108373298,  3299541851152, -4605910635694,   -36129840727,
  -6576311788859, -2629800174936,  2525791880294,  -640513138117,
    284228178609, -2269218166260, -2123139014045],
 [-2219471355897, -6471651795388,   353742166342,   171015677393,
  -1471849306206, -1991285927739, -3589311739944, -1466374989604,
   3344329836540,   137984938737, -4546287909462],
 [ 2073182936613,   466328932539, -2791012405298, -3313588062417,
   3530568343429,  2216032040681,  2265865618174, -4346765336882,
   5293052453049, -2639133395520,  -573740069659],
 [  214459878412,    19398313189,  6305390579996, -3608487383784,
  -1989842234019,  3278350146195, -1223967877264,  3305477784465,
  -3617311292070,  -114403255569, -2635362871908],
 [  426321805749,  1137571596648,  -934133552682,  6233342815683,
    -48487549937, -4002542435362, -4889499143122,   703316459258,
   -975659268269,  3514709109664,  2251081605192],
 [ 2468565829596,  5155658424646, -1453270930266,  1276928271948,
    949302333786,  3903659962438,  1275142291747,  1489697404928,
   6018387529292,  2226900271067,  1554937693002],
 [ 2715506054844,  3482379220419, -1729557792903, -3606698334999,
   5937100067475, -1554453997960, -2096963359890,   -92230435877,
   1000574089023, -2751065007208,  3722722845190],
 [ 2634743781118,   983127432399,   299209752327,  3110492334599,
  -5185766104877,  1781713739917,  2216061204070, -5151489900582,
   -295402636687, -1761199754849, -4197899465723],
 [ 2077555732930,   283559312757, -2760538621384,   417300252388,
   5927503430910, -3533754014174, -1001230518933, -1223879516797,
  -2575219126684,  4882369112087,  2686317858147],
 [ 1168466204065,  3822971229690,  5029915887937, -1996991926131,
   3051301848435,  1525811987757,  4531837892615,  1633064178647,
   -970556221271,  2800621530552, -3332129692759],
 [-4333405640177,  2595803503469,  1222444806936, -2696811891176,
   2416496360838,  1744487793328,  4568361186596, -2409674071826,
  -5469243574926,   -51579532841,  -491734613348],
 [-2467630147606,  1981816676657,  4488642911129,  1723347036133,
  -5276606227799,  3860287962711, -4220417788713, -1362249656124,
   -946152098270,  1812822630005,  -521332300988],
 [-2241001781198,   -40972216213, -4450445816436, -1830900109084,
    583181392280,  2373038873262,  2677962053627,  2142841574856,
  -4906715376677,  3515275831386,  4201176544176],
 [ 3566116295130,   755631566161,  3486726151340,  2041359806509,
  -3684933736093,  3077109175398, -3148124103181, -2616165387334,
   -366438653115, -5189581246319,  1875913477081],
 [ 1169756207380,  4450182319364, -1017069826463, -2917632011119,
   3909587453197, -1692287000160,  2268879223154,  6069084923479,
   2871909406721,   554215792900,  -771496783680],
 [ 4545643710456, -2839300538053,  3196872591342,    48408958543,
  -5617612880939,  2851958971972, -2444934555215,   415011797637,
   1397334760407,   814873213931, -3549001395164],
 [ 1369104443879,   508626624505,  2723357463617, -1490777202574,
  -2639761015160,  3626693760144, -1377786536700,     5248435782,
   3763602126491,  7162511645981,   861345608385],
 [ 2440131171132,  -819355214094, -1455810769125,  -601774225686,
    415079064241,  -101645112079, -2020387519010,  5515392925687,
  -6019543383458, -2866131618202, -3429160170996],
 [ 3404935097258, -1475777975123, -1624555070751, -3064900974353,
    510268804990, -3238776936905, -1707968738275,  6124575621962,
   1783373494878, -3781535254565,  2353218057669],
 [  480568627596,  2119861335709, -2131094079761, -6433151784280,
  -4224803047866, -4467323734356, -2216351640087, -2233676451907,
   -916228259636,  -116451429973,   887544630281],
 [ 1347340652764, -2425982744085,  1305009012156,  4911066205519,
   4558713915155, -3083628705040, -1527150020658,  1092432828781,
  -5648998324766,  -375301281712,  -780636116878],
 [ -645955447182,  -573412844198,   291621049695, -2851751604036,
  -6178291223918, -2042697299143, -1653484381312,  1144334097639,
  -3019079742109,   208920449350, -5957391471904],
 [  643872500447, -3889012601565,  2040770000899, -5164439015432,
   3090436298776,   367803434308,  2592146963002,  -244364271528,
   3210036084153, -1870466177752,  4832892053998],
 [-3489708056613, -3565689589022,   421784280972,  4382563289631,
    822777100831,  1565898196956,  4032550397657, -2172439184083,
  -4750428726450, -2287378729032, -1953049895133],
 [ 1510363990797, -5503286576671,  -695235405965,  2412916532682,
   -599696475475,  2886241647929,   526584803957,  3231823250970,
   3842006991659,  4548131061588, -2503718301923],
 [ 1226958249894, -2773201525784,    91343677483,   909406635811,
   2053979594022, -3961278384597, -6239413826246, -5274237794401,
  -1407879540787, -1106055587451,   323498175969],
 [ -754260506420,  4933069749976,   -59774358666,  -581839150403,
   5435911970421,  1679010859911,  -379642495186, -6244963647764,
   -688048364149,   979863489122,  1344747317080],
 [-4466010900252,  1515606562093,   561844871854,  3992079077555,
   1161311755868,  1990294207776,  6492035972605,  -485351519506,
   -252078871908,  1773771427134,  3256301020790],
 [ 1800158889991, -4861798591592, -2635881641748,  3375896166871,
  -2368461074210, -1053116459665, -4421962342629,   680808497648,
  -3685268727265,  2081875272447, -3182211796350],
 [ -700568222425, -2059326140183,   969918550260, -2331659486757,
   6358449782368,  -702075227607,  -515434118465, -3327861842950,
  -4776211464527, -2744252145422,  2506941608789],
 [-2707039197917,  2328954056365,  4390223123745,   -73968588581,
   2565737297374, -5510058522172,   906559073621,   430873253591,
   3999558820412,  1152761661004, -3562554875280],
 [-1024528772430,    45347036460,  4433248776900,  1057871180179,
   4104998210599,    36381024375,  -951957791240,  7326387516491,
   -684996202517,   682937228904, -2409432229150],
 [ 4067741710999, -2566271397519, -3256492925837, -4386848618314,
   1886947461174,   944956097375,  -461004655030,  -898479697604,
  -1812451258699, -6184293797316,   121525174369],
 [ 1315366752583,   800997282555,  4566987593264, -6895175556057,
   -699226297501, -1451542326893,  -328352244017, -1928137156462,
   3132392662035,  1114118165728, -3428239828999],
 [ 2144150939231,  1554529906139,  1257893659262,  -353150017642,
   2410536308459, -3423149484282,  1256725264007,   506784869501,
   7845923503507, -2077267572141,  2457775205201],
 [ 6273929664447,  1986709087805, -4968720868214,  -284932371925,
   2539623792742,  2613082443799,  1482794570863, -3206489923520,
  -1170089623014,  1375141403761,  1703863316471],
 [ 5113286992062,  -262519979148, -1205611478137,  4656492845107,
   1575614313201,  2901229953062,  1651661934635,    53279269656,
  -4041644034623, -4390196272182, -1187317734356],
 [ 3978464984049, -3638245937238,  1574138884919,  2436835214069,
   3064334755163,  2476356395527, -4274505026665,  2715967838611,
   1178536371096, -1662873006476, -4146618641126],
 [ 3350770810383, -6123869174151, -1791629481518,  -101508661756,
   3779184489497,  5473608843151,  1686970086472,  -501827889720,
   -750568283023,  -129282778282,  -360452279019],
 [  474331919901,  1699069111081,  1367225341153, -3383752095864,
   4667453902113,   151303632031, -3058063988967,  2202965005450,
   1165308647261,  6289559138375, -2576773702522],
 [-2005176305822,  3377760252313,  2836300312586,  4011574435132,
   4826243835769,  1579668182124,  1692136264531, -1004400577170,
  -2904950274070, -4442988350488,  1610486381879],
 [-1169046415570, -2309917566597,  5637115874876,  3572460874826,
    816413649445, -2106133162533,  5137698753257,  2181218129160,
  -2153312905457,  2606837452574, -1034268872376],
 [ 2474067050690, -1851274671465,  2426267545725,  1659327951792,
   -906277412481,  2756227575435,  3811770018044,  4421784098451,
   -842976716678,  -721246831441, -6170974772918],
 [ -750066121913, -1354049898449, -3732247107281,  1991239201577,
   4538863103650,  1845719188469, -3121715318647,  4478824024069,
  -3273309473431,  3661450392158, -1332495781935],
 [ 3433238320095, -3537063600267, -2635906680144,   261302237613,
  -2759443456705,  3386536019652,   949511068238, -2950096444044,
  -2047497480251,  5963423843188,  -493262884399],
 [-4105722097165,  -141913655347, -7788314434300, -2620215156652,
  -2116962346128, -1450024151373,  -732778649936, -1080692149409,
   1114563921787,  2350123655098,   738406232725],
 [ 2200620897542,   705226451779,  2723763190353,  3782000913440,
  -1482406284152,  3009948542426,  5720928376417, -1025025527459,
   2998989283179,  3937668088876, -1844267429759],
 [-4001613157751,  3378934508215,   409158451737,  5494366397472,
  -3327412136028,  1079984219353, -2631470696356,  4020893842667,
   1536948449196, -1726419747723,  1241540389589],
 [  888673029462,  2258870927984,  2595865553564,  5350662068447,
   2747261226114,  -588630358854,  3020228413670, -2016224262166,
  -1280527406098,  -400562929971, -5988204141971],
 [ -784448296601,  -864748064806,   559798999754,  1688253545921,
    240653265587,  1648793052913,   -17280348977,  5756292949770,
   6805727075975, -3089022346053, -1923969062188],
 [-2732454644794,  1183497496595,   127337376432, -1243692020947,
   2061785440351,  2186341094142, -1357024113404,  1082754121094,
  -1857273260280, -3914021204478,  7665252583820],
 [-1236793825863, -2333318951451,   211305234637,  2283356120529,
  -2629243609853, -5271440586761,  1723316268126,  -415823216033,
   4601979636555, -5151650335502, -1485309781336],
 [ 2821642382102,  2906951566716,  -526257070986,   283003410036,
  -1816610494370, -4630780065571, -5094770360351,  -822804971694,
   1629306854930, -4497641960600,  2994931491381],
 [ 5757045691793,   187120190092,  3814983805307,   225282311117,
    632342828617, -4181668994943,  5188218310499, -1815462636634,
    503375258259,   788144965033, -1800630830432],
 [ 2678559131102,  -555673908285,  1769279437351,  2948585676564,
   4394538007670,  2199184553610,  4013515708219, -2604227000232,
  -4707767382466,  2690870493540,  2061431279089],
 [  379190190073, -2714425600161,  5334870609655, -1772639083263,
   1735877144851, -1931477068771, -2679193262334,  3669975714046,
  -3492163017355, -3229146038935,  3297234694602],
 [ 1518980188259,  4863668733161,  2236438537843, -1739700465146,
  -2429011831799, -2695460666052,  4238729018848,  1324838215596,
   3326618888075,  4511503228801,  1304426815049],
 [-2033226249345,  -405083959238,   463453849128,  2230853372887,
   1182847258739, -6433452357703,  3685783962573,   116931247361,
   1510246758153,  3814439635060,  4158424407346],
 [ -951517934441,  1675539718712, -1336285940935, -1645421904301,
  -2336910100273, -2747786607563, -2184017666043,  1806679730034,
  -2597175549173,  7708279758434,  2141425780852],
 [-1083879915157, -4640600124494,  4167562887098, -2758177942872,
  -3592361677113,  -279859805331, -2652509510016, -1477927943039,
  -2277043972392,  4944375002945,  -691862540020],
 [-3898439760039,  -257945608795,   323292417029,   151022835822,
   4501929287046,  -634192149356,  5746049858676,  2593260480066,
   2269947530578, -3311852390405, -2841763833478],
 [  141864048396, -2742792494647,  -954616381445,  1821875393772,
   3728473011512, -5212023826256, -1451554866067,   354522574070,
    158471783785, -3132772093270,  5923484286679],
 [ 1071769248680,   334438752973, -3760862096385,  3089607262596,
   3359666382977,  6606247254319, -2968708276988,   215987412493,
   2515190151398, -2201877893616,   292432061488],
 [-1230530900302,  2771727322867,   -88812425037,  -911548894030,
  -2053580890639,  3960865394137,  6240083048958,  5272634654412,
   1413139988991,  1103161460414,  -324993353976],
 [  -86998330244,  1058762300965, -5641212497529, -3083499428291,
   3245614970178, -3524730716672,  -698745775891, -5232054699530,
  -1592394393518, -1715033802990, -1114791669436],
 [ 4809659318974,  -149233818971, -5314475619759,  2436109542017,
   3216650845950, -1668718122847, -4689437464696,   970733506968,
  -1702018773990, -1901992922741,   295923281742],
 [  569079240735,  -615142865604, -1688765527873, -3981623417149,
  -2356414755070, -2577678633195,  5070931344481, -3371149123615,
  -4372302060906, -3469182077811,  -405136618554],
 [ -900259136377,   812233153653, -2247324243777,  5222932711560,
  -2659657768943,  5571312149424,  -298641624508,  2116878044993,
  -1279522203441,  4522091249610,  1195898864850],
 [  902141610581,  3298918161640, -3342757931425,  -682634882119,
   3097205669747,  1059949340361,  1339024616970, -1293379261324,
  -4136728035009,  5057105421452, -4449093393518],
 [  -73353143388,  -815510154657,  2852365403161, -1111929579292,
   1441350269881, -2025644047242,  3093622515804, -3125572943450,
    285987548666,  7905010645558, -1365933247070],
 [ 2465951952348,  6328200956618,    18797023916,   194486333831,
   1417719448242,  1679612858144,  4017966457568,  1466430593215,
  -2758941377335,   454029325736,  4784493105961],
 [-4109098251814,  4344566738087,  1241576641643, -1336655497978,
  -2058584846899,   801204721188, -4497140463018,  1970940831292,
  -3318281163024, -3749067380961, -2618551918968],
 [ 1648454925486,  4747015799246, -3471514447645, -3561631541054,
    201564745507, -2919312907498, -1240784771082,  1963570428022,
   -488321627252, -5111461208275, -3112577709569],
 [-2551031962307,  5068904330152,  2119147294801, -3762883681579,
   2615881186145,  1034576539254,  4104081891625,  -547162595789,
   2990336877677, -2732468275682,  2771968680612],
 [-3245165420869,  -170031800418,  2008342982470,  3956781345125,
  -1036193413253, -2028762948150, -4275902223348, -1111504962239,
  -3408998867350, -5018672581355,  2869446580626],
 [ 2180933012478,  -546194667748,  3575643651836,  2455368469823,
    389037085827, -2570081736166,  1314204101992, -5721618825168,
  -3927732407186, -4401729105455,  -328809010366],
 [ 3945590192550, -1756704785784,  -137454169152, -1789453902934,
  -2979714527466,  1252082126175,  2555339462242,  4431903196124,
   4945122565474,  1738074490083,  3745956922482],
 [ 2259619080308,  4590487947210, -2251005923542, -1947039143183,
  -2494199878457,  3514421935768,  2391274822796, -2570700740949,
   -561771487022, -5792662736345,   440604120366],
 [  682425135818,   587302795151,  -317869972891,  2871329834968,
   6175415094364,  2049437638613,  1643654972210, -1128116887017,
   2969019410103,  -180019819214,  5973785736564],
 [-4671784204114, -3636066015725, -4541700480367,  2150041272614,
   3184135409938, -4505677577062, -2292223611054,  -700095384547,
  -1667730528470,   331259751111,  -792720660805],
 [  931616503018, -1118731315337,   399845528585,  5763565676285,
  -2927098487698,  1494653556681, -2860994615124, -3713538466516,
   3681274686970,  3835698208950, -1860120472308],
 [-2660324687047,  -191977827658,  4155438805119,  1558555138949,
  -4256397452858, -2912318234252, -1266917535702, -2756313453107,
  -1000147214125,  2275249532232,  5586692736924],
 [-3399778343912,  6159145363346, -3063878260556, -4689453897494,
    165191722795,  -292410107528, -1867435094055,  2612690604473,
   -333003423324,    77320108261,  2930013664243],
 [  531147775146, -2170328967690, -2293201254125,  5367543080577,
    121326811130,  4429621544785,  3170478944287, -5098314400427,
   1619930635276,  -576670726531,  1516748305541],
 [  327608737716,  3811429821160, -3932622782162,  -826291283558,
   4066076816263,  3047328830449, -1581534415805,   320899075464,
  -5970199045313, -2078284885189,   911827996371],
 [ 4518785347160,  -388700493896, -2490485817584,  5359924247762,
   2449270871360,   498109529865,  1367907884520,  -424892833282,
   5044293942255,  -116702307384, -3277333634716],
 [-6525162079463, -1181354204005,   624956783856,   358842171217,
   3469986050917,   255099458968,  -915621384173,  3284176993775,
  -5185525820581, -2168287046212,   431412415300],
 [-2941652726062,  3110504473894,   821394292308, -3588079192813,
   2753639156092, -2618552523411, -5756456571727, -2919392392152,
   2519754256098, -1237747428259, -2035256728565],
 [ 5048423056420, -2313715010096,  -761025759937, -1804248447970,
   3211761556085,   726679387348,  1117003513562,  2661551201487,
  -3278751911642,   339342731636,  5940141443264],
 [-4042579749446, -6750360135928, -4466751606843,  1681946903260,
  -1228916050969,  3098704888002,   571813546750,  -833218627166,
   -581856657591,  1654660302331,   315409511224],
 [-1035333893018, -1442118298059, -5617578658270,  1864867197505,
   3156317086217,  3338066033401,  4743005829326,  2175921776371,
   3365096684350,  1379639723178,   500414999183],
 [  -11427126637,  7367594985755,   610962666431,  1110263111105,
   1087256905897,  3002689697382, -3838425077956,  -471264988323,
   1351394896230, -3377921610308,  2392062008724],
 [ 1358649557230,  -462924873050,  2962306138089,  4756468859486,
   3530257458269,  3657347279216, -3369408194109, -5111547699335,
   -689873076083, -1041547577939, -1288422971606],
 [-1702441183899,  2379753603282, -1977514619740, -6150487283618,
  -3941633834821,  2272934930821,  1398178305322, -1214292023464,
   5041711426171,   290921398220,   252589314494],
 [-1789898852424, -6045978389703, -2407782197450, -1215663486795,
   -669694319880,  -197516091123,   875316813228,  3264967464244,
  -2738960314402, -4523497111234,  3617945509846],
 [ -690529249041,  -551543126829, -1669071497316, -3584852110174,
   -472677388588, -4932348321410,  4822503098792, -3153350238000,
   4686045472618,  1009143113924, -1689075334590],
 [-3403775108391,  1476258452330,  1623721323956,  3065601595098,
   -510393100108,  3238892394793,  1707768905743, -6124079953293,
  -1785076817159,  3782462175439, -2352725566758],
 [ 2396927824505, -1801977200565,  1370352050487, -4811131694918,
  -3379559646078, -1225803264230,  2281600137297, -3168189949940,
    886080762421,  3919090990916,  4655234166250],
 [-1503549869037,  1212735318063,  1076597747714, -2066580829533,
    368020916081,  3447110116613, -6157411151982, -4483552814227,
  -3629149843989, -1778234244327,  2114705819539],
 [ 4462028326936, -1517183397319,  -559064623985, -3994470582582,
  -1160814586133, -1990740040881, -6491343119880,   483609499542,
    257883507284, -1776945142174, -3257932052113],
 [ 3614730897172,   422382984848, -4387680174406, -4667356932295,
  -2659814430804,   793535112010,   782431041005, -3043534264706,
    208539181470,  -732100135520, -5249789136291],
 [-1647666307448, -1710410004391,  3156752345801,  4341228612939,
   -578114935938,  -138079452183,  2126032864910,  5033459758869,
  -1675582532975, -5703048434848,    83595080804],
 [  -71611541693, -2208735633139, -3149735813939,   174588134464,
   3916295713013, -1950257452486, -3625851960195,  -222009405516,
   5048747575851, -4908852220951, -1800386877248],
 [ -757479774973,  1842556725253, -5335399910162,  2409191958230,
   1896290788129, -4407192826846,  2176298076868,   267030096684,
    748547596232,  -790242619193, -5722661824073],
 [-1708067341610,  3778225389386, -5739863819085,  -849033045991,
   5333610382016,   503226217254, -1805456153799,  -183182963432,
   3486590152969,  1757208882586, -1379371915984],
 [-1372830112462,   -26521209511, -2124080741896,  1688948421360,
  -2531925342951, -7620134073784,   133402939606, -3907749557550,
  -2114497263376, -1692611313949,  1910221036734],
 [ 6964785946583,  2532233513343,  -655135734296,  -587153760045,
    750596564141,  1991363355268, -2056984275819,  1289181637869,
   3749221382394, -4405034503841,  -650008468001],
 [-1729882284524, -1344800787370, -1317512698711, -2649263530170,
  -2705350082060, -1070400469548, -2711924816032, -4646510085482,
    934003050063, -6279053663051, -2955746292466],
 [-1274233595323,  5412938535530, -5424784225187,  2834538558180,
    -96685163611, -1338972520537,  2243850168311,  2928084938293,
   -273640652930,  3877623040267,  1044965244792],
 [ 2898149547620, -2990432543296,   247745768225,  -937802201036,
   2045083409460,  1891270956134, -6501832935269,   252749135059,
   1534982709221, -3375257889106,  4227489575329],
 [-2269865736199, -4594524252363,  2258191837167,  1940818929425,
   2495428789378, -3515406037013, -2389572202205,  2566217428711,
    576709992011,  5784526702722,  -444671457975],
 [-2245342351779,  3384856863440, -3568161851038, -1754563009282,
  -1186980873045, -3900929284580,   629702048159,  1640587456022,
  -6769181311722,   545232058360, -1362935439985],
 [-1437312260121, -1847697111240, -3049619556769, -7835377675664,
   2880335165688, -1331949009768,  -945504108148,   875356400851,
   2987731743090,  -466280584155, -1718203090590],
 [ -278532810318,  2179880920650, -2011458323430,  1781485798113,
  -2859562978183,  1726766616329, -1351503498302, -2376402753467,
   7188032423483, -4181862900095,   403013457950],
 [-4066954318074,  2566602669850,  3255939520713,  4387344278805,
  -1887028511272,  -944891597795,   460897739197,   898848728882,
   1811289058759,  6184899102792,  -121259486508],
 [ 2345548484243,  1039005251056,  -365378882558, -1226402450103,
   2673509261365, -3721614510411,  4030589000797, -5334143996077,
   -142721473165, -1006686008224,  5005068766952],
 [ 4341459031588,  4484224767691,  2332866018647,  3544769452216,
   1375523081600,  1309149831450,  2636685662476, -4361177622051,
   2528756086563, -2417713797695,  1103603852692],
 [  750664436098, -2148943522252,  4008663719182,  1762059704738,
  -1869060518421,  7708557739294,  1945574678840, -1790633954168,
  -2004669972305,  -687062970437, -1116542776830],
 [ 4329647159964,  1262405360551,  2440979596453,  -879866742125,
  -1927676754522, -2131337489055,  3443513668473,  6450458962922,
  -3211827601687,   305451932838,   891100160463],
 [ 2193404209828, -3317090496399,   122821783843,  -978804771565,
   2462904558619,  2600348067812, -1585390611262, -6654730960632,
    297410577503,  1642402629124,  4560722388450],
 [-2409481474165, -1064732547599,   410769153834,  1188086537465,
  -2665858946672,  3714006215367, -4019149086204,  5305475113973,
    235643584566,   955830409373, -5030810211646],
 [ -845161422135,  3925130330303, -2077708073922,  5179976208925,
  -3200810750409,  -723860328055, -1555830229190,  -337924643751,
  -3427840044559,  2255422092479, -4752793863350],
 [-1168114860623, -3822826448574, -5030151779254,  1997211921609,
  -3051362506056, -1525745276917, -4531922239602, -1632910473648,
    970029768864, -2800300569351,  3332289694638],
 [ -581794081007, -2595079464845, -3123555538763,    -6773249377,
  -6874918744212, -1391281133731,   532315418523,  -112614262325,
  -5048355315029,  1671291716108,  2322738285131],
 [ 2338012133440, -1336683953743, -3115196655662,   613370202792,
  -2374543281464,  5833833429157, -1156933975158, -1302463235103,
  -4390871575893, -1174256566617,  4393255672731],
 [ 3891039367281,  2404273366108,  -564822641231, -1541134362753,
  -1423497284225,  1936471871239,  -454771626662, -2158090210442,
   3181298476237, -1668814410250,  7269052382347],
 [-2922513445504,  8259732626733,  -200487366516,   412388800769,
   2969837437489,  -919097665817,  1201968335381,   420914703816,
  -1022507411409,  -642740879182, -3206284945675],
 [  413661737289,  1618152381840,  2340126452014, -7269061056510,
    290262527334,  2130166416306,  1486826857866,  3708222574137,
    117242906970,  3307994981739,  2712013942179],
 [-3403108019382,  5241633649204, -3092714393079, -2995093677834,
  -2318344914351,  1801341565116,  -992301959469, -4822779014794,
  -2128535474010,   262292422313, -2223912831128],
 [-3171278517329,  1036972010098,  -163496287217, -3097558909873,
   3999139351372,   601001380339,  1711936961218, -3465559911694,
   1956544678212,  1185749576185, -6535813520357],
 [ 2579929796494,  1485575743088,  4112675668364,  1154406856819,
  -2756282810637,  -860769984091, -4468232066595,  5422364409564,
    699865213083, -3564103950269, -1411548383478],
 [ 1316062228161, -3831074514174,  5731057985101,  3994159989126,
  -1191850856492,  -436276944900, -5541018219297,   -41243974686,
    181432992600,   610688088654,  1439173634386],
 [-2667525179584,  4965987709141, -2544676139347,   951835957644,
  -2728676268728,  1186067930727, -4715622352279, -1182564942954,
   3632600866661,  3760780591228,  1007304374680],
 [ 2453277201468,  -491631561492,   786969316878,  3033260699249,
  -4049219737779,  -289916809971, -1342406267171,  3788364791102,
  -2045268192298,  -519415590365,  6843451545319],
 [  366765670903, -1200692417917, -1429467588701, -1496779860285,
   1391348379768,  4134240253425,  1275738234114, -7138858643012,
  -3162845062064, -2649152656940, -2345184615911],
 [  914341258819, -1781328446830,  5225313203447, -2312903454284,
  -1915917711758,  4421371639718, -2200482167067,  -197760517690,
   -979348654132,   914118035842,  5782922630427],
 [ -782180416810,  2052558781591,  2470844921564, -5517471215525,
    -84460969577, -4465076441973, -3115934462584,  4968069001646,
  -1239746927806,   356061150341, -1621444586483],
 [ 2355142884752, -1017634208111, -1736346495382, -3010885033044,
   6053769566543,  2670424431904, -4806248354162, -2995681693375,
   -210057713591,      889905356, -2332347547099],
 [-2645384386648, -3900273217887,  1163765540114, -3009688646456,
    666557069730,  -688711131228, -1335498699546,  3861250470782,
    304172533591, -5656942428963, -4203841385314],
 [-2615974363572,  1031046264966,   468394941374, -4180199374433,
   3651122727237,  4505931176975, -3351586434352, -1101201563112,
   4177012293650,  1099521763330,  3108680079658],
 [ -136897195983, -1081886913472,  -657609981089,  6686299501168,
  -1254690698723,   245381970475,  1924363243205,  1922466896425,
   5223373108159,  1821950711408,  3746342618508],
 [ 2424156496536, -2198726064092, -2884756557450,   903276598819,
    975991121091,   758060057008,  7373144182348, -2427240019001,
   -338080780553,  -425095511148, -4251396499489],
 [-6325490988486,  2598656058563, -1207708054039, -1211334362887,
  -5737679409125,  1245158976971,   244137249672,  2617541998517,
  -1033672732075,  2668395449981,  -859029787814],
 [-2772603497675, -5685522069233,  -361553801111,  1482882840208,
   5553003984629,  -593738427560,  3659694568985,  1136926900543,
  -1335602128256,  1289653471776,  2887293051407],
 [-4095417183648,  4839098221725,  -465435241876, -2479348325824,
   1395032968339,  2992221080202,  2556976049731,  -799439188010,
   1550778200364,  5741502567849,    19751812624],
 [-6908934403500,  1531241071359, -4706719133239,  2128676614342,
   1734681290776,  2186163589365,  1764336556458,  -387799201584,
  -1686620354607,  1057206905840, -2867643679802],
 [-2140503593039,   561984352407, -3604049075333, -2430617584736,
   -394188466297,  2573789311727, -1320334788844,  5739390159704,
   3868688028118,  4433547836917,   344374165714],
 [-1893666539574,  1659730122758,   946544030408,   -76189385339,
     20559221468,   571107687498,  2129920519145, -5678538610919,
   6180191631216,  2276775563167,  3502877120755],
 [-2794745255581,   372415077265,  2972006867864, -2348892576132,
   3501669213866,   161804686635,  1734915599637, -2037765678905,
  -1587118565352,  3999728050819,  6303453485896],
 [ -968718573149,  5689743280001,  1952593569916, -5300349284827,
  -3695445383604,  -448512674898,  3189704448263,  1223541944931,
  -2084765359924, -1293321077701, -1798445430447],
 [-1525591131074, -4866300238791, -2231803949435,  1735707161763,
   2429785224139,  2694759754916, -4237612395193, -1327733563264,
  -3316991137642, -4516772902518, -1307125169001],
 [-4466940599372, -1647027909491,  2531558090634, -3028270226670,
    910724778055,   192877798160,   344148556492, -1765984879127,
   6830022701532, -3059479270272, -1281072994700],
 [ 1541635846422,  1362353363053,  -198072058243,  -316239710940,
  -4095584293555,  5659216425601, -4483688773694,  3175096487836,
   3329121932947,  -665912022636,  2261940867051],
 [ -437105715614, -2149437760686, -3236107456867,  6005716132078,
   1287626975401, -1604415904986,  2739810393095,  -592828819392,
  -2493162444723,  5324874624305, -1409804529466],
 [ 1397647866505,  7312735332159, -3192838943251, -1429753386275,
   -370962974316, -2656027893092,   659706429203, -3466751520431,
   3313054721054,  -432560781839,  1234078745832],
 [ -293708146266,  4440039470869,   432358239719, -5887353939842,
   3500817164191, -4164343477674,  1982663104286, -1491944365746,
  -1311915602379,  2801491320417,   167661571787],
 [-6163626891441,  -136261226569, -2176276951445,   622796697888,
  -2011615445323, -3964961823186, -1210594725027,  2980786895674,
   -904406545254,  -448590894926,  5072449632079],
 [  385073785918,  2021446299101,   936796987436,  3667782702405,
  -1063153037472,  5814427729415,  2760137975099,  1761773698065,
   1632899051403, -3975044593693,  4160269612796],
 [  109714879158,  -120550740434,  6360099365214, -3047956367250,
   -457719420703, -1620627096654,  3283373657054,  4116387541021,
   2998192254453, -3266540573839,  -109342959566],
 [ 2816298045683,  5843722832103,  2492182163736,  1654309583123,
   -639788473056,  3284375995947,  -433160286357, -2696718584479,
  -5261469995038,  -372036857907, -1579178515081],
 [ 4664308006134,  -395601340473, -1392181786846,    91212707612,
  -4217859713708,  -609179267588, -5649307507006, -2511802374320,
  -1501802234682,  2648283927022,  3238387930209],
 [ 4735606214913,     6260817852,  1358663626156,    84369215894,
   3699432657352,  2648486372163,  6050758041547,  2492903514265,
   2945068298062, -1838695677162,   374376326382],
 [ 5417322970567, -2065248616302,  -519393174767, -5889580242966,
   -750460081537,  4570124985149, -1835866028015,  -901827283530,
   1979879462268,   962684788193,   975100689454],
 [ -911512194716,   565281354632,   -70196043250,  1921790595333,
   -415630650321, -6556423890214, -4662785919001,  1404684530327,
  -1785704456781, -2705015278984, -4215613080425],
 [-1418676255514, -2529630289464,  2732865434791,    42265025956,
  -3126937889025,  -229710682698, -1326022638311,  1808071533522,
   4989560834424, -3938532880436,  5371430314699],
 [-2154770173062, -1655489775893,  6405363148531,   546322166172,
   1272231190292,  2843001042335, -1745559831129, -1136676524727,
   3206356279165,  2677931970476, -4449370082130],
 [ 3167144720180,  3400465286298,   209022404118, -2100076134849,
   5075564382316,  -609283725349,  3961560836596, -2381212947343,
  -2292497261332, -3903117524233, -2442422997698],
 [ -288840861068,  1594920895700, -1527469002679,  2656358099910,
   3564908985902,     4911908294,  5131175474934,  4519124677249,
  -5082505477121,   280208763309, -1616468280120],
 [ 2158636040022,  3513858812998,  2587785249793,  4869552631870,
  -1315394109919,  5044557560708, -1420213024157,  1038640418726,
   2446280530658, -1280515133208, -3832271169069],
 [ -161207628592,  1424471926574, -2257894915604, -7117922473962,
   1060663107915,  2350965116796,  1047783051663, -3719425115065,
  -1954167589074,  -747088199132,  4027537035630],
 [ 2970021068420,    90202386471,  2792402654131,   415340747262,
   4472871111723, -4616263033670, -5600231136442,  2573077535170,
   1900778557469,   305707229798,   441162857580],
 [  175691387717,    12983192677,   811036497227, -3740551373609,
  -4422972150623,  1993335857013,  4512377635188,  1617341950259,
  -1397734151434, -2806884824984,  5382729663013],
 [ 2260116487747,   -31441253112,  4462528514067,  5599611951951,
   3850480954428, -2189852968882,   216461989754, -1718012042775,
   2842688854909,  3517794520944,   738397699658],
 [ 3510207103853, -6112307016110,  3003367419328,  4758336321777,
   -190958098368,   297496767962,  1871066810970, -2557954314180,
    169590084297,     4470543314, -2908332380072],
 [-4056853878728,  2088231267532,  2306312239637,  1295837021287,
  -5357441368825, -1313994440561,  1542310809410, -3065882661537,
  -5088082843024, -1662287496516, -1150674485511],
 [-4196431697300, -1517019629998,  -164153174035,  1261121010131,
   4237229602749,  6496602069360,     5994676524, -2510946868028,
  -2367512895552,   453987039040,  2489010883646],
 [-3812995858287,  -446150972528,  1029791281779,  3279436620691,
    277503140652, -3743425449490, -4126554994269, -1774179768184,
   6042491720568,   293725272166,  1606998591499],
 [-3777648277536,   909754842129,  2951619154550,  -470607509255,
   3094849665213, -6401365658946, -2916636724929,  -664522333345,
  -3269020243665,  1238465293599,  2060088943158],
 [-4168834440227,  3619474011711, -2374851631283,  1294784688238,
   1360399529916, -4471080017925,  -235437613868, -4339691111753,
    240612553421,  4458017410701, -1241947169126],
 [-3956823318298, -4744591949417,   -90763831885, -3724676022218,
  -5814213810699, -2038890118243,  1502989889968, -1565786109659,
   1831121126423, -1007317820938,   954166256105],
 [  762150529743,    -4708941881, -1237543599039, -2548647433955,
    145343016379,  3056651884393, -3645833955480,  -334735077049,
   6467857952495,   653714211045, -5134497014594],
 [ -750814928953,  2148867259446, -4008573099659, -1762178348707,
   1869101610402, -7708544182978, -1945580238424,  1790570978275,
   2004905471401,   686933180014,  1116499587732],
 [ 4083148129093, -4439835127532,  5235858794965, -2414215105043,
     82444671866,  2444225410125,  3331102481012,  1208773168150,
     63373379130,  3437159794961,  -128913033634],
 [ -773814964513,  -222898812318,  3568293909779, -2909424648814,
  -3401226932222, -6584682394845,  2936928259888,   -88014711735,
  -2948957039760,  2421586839552,  -184232961144],
 [ -815764090125,  1944969591284, -4012422151591,  1422208542377,
   1299999338763,  -259239012726, -3562650436673,  3226679983667,
   4740167343852, -1370950528653,  5312210159669],
 [ 1604476820817, -4429392743063, -5529218278565, -4407828402920,
  -1777715771558,  1355318695197,  4339320904962,  -380491261787,
    843969667101,    30503134227,  1766372319428],
 [ 1900004359421,  3193119945072,   879428022823,  2140011849071,
   1518589953933, -5905485332377,   915396365000,  2769300395353,
  -2479123920671,  4561357908670, -2863744936299],
 [-1774755740880,  -548200206493, -2969017914696, -3529848401167,
   1437887855970, -2989713695603, -5746601802979,  1192653194931,
  -3584817931168, -3609419357469,  1986008658016],
 [-1625213889932,  4421119860839,  5543654927871,  4395226412743,
   1780196769319, -1357444062802, -4335770064592,   371441570966,
   -813723749688,   -46935696425, -1774620718433],
 [ 3715263790834, -2507441356067,   916933214642, -1140143779240,
  -1613057939667,  1637675316378, -4053977038224,  5891778174886,
  -2113162615126,  4002114501913,   924535278642],
 [ 3559719361406,  3593611015819,  -470681462291, -4340318713557,
   -831229915233, -1558565867977, -4044317435861,  2202917506989,
   4648560925856,  2343087643899,  1981376332508],
 [-5149326266331,   425278277647, -2358074079089, -4157768090984,
   4060365081707,   363375547590, -3004681046676, -1898453697484,
  -3150592017271,  3351575609492,   214239081516],
 [-1885574942447, -3414768961929, -2237633145328,  1009180982198,
  -1258984424464,    83569138486,  2748852970600, -2104550426355,
    845115067889,   278675248156,  8024293216243],
 [-2616397571692,  -557163011524,  3825473785314,  -264303622221,
   3632263036396,  1472196243522,  3117970572795,  2829316578089,
   5699698566962,  3100518803696,  1718335559607],
 [-2001171217114,  3769109590606,  4843242734316, -2207726731685,
     20425655823,  3295100129708,  -666418356220, -4366181203628,
    719822543059, -3684959001579, -2999207897268],
 [ -416440675793,  3966595907281,  5404744038843,  2659058883616,
   2152299922905,  5211934991105,   283496230637,  -898514893236,
    536756726137,  2884298612590,  2553271449435],
 [  335206399612,  2429619549291, -1339859634853, -2402137072018,
   3268570292751,  2164671951558, -6122786239087,  5227011286623,
   1193309401770, -1771237023356, -1295498554102],
 [-4535646281527,   381955225200,  2502373372548, -5370195363497,
  -2447289461544,  -499927932217, -1365080082561,   417493881829,
  -5019672401578,   103242561889,  3270547937599],
 [ 1320294581199,  4358886860887,   498976946644, -1109624493511,
  -1473532016269, -5062368760463,   813784390254, -4271521122302,
  -2127754988860,  -555742065341, -5121863011168],
 [  319954682508,  1644930456827, -4048011124793, -3586693983952,
   2262601583584, -1140899394730,  2693872741624,   452698386592,
   3558061465140,  3516425128220,  5388238523679],
 [  576511139734,  1064716969756,  3503843991704, -3344260267626,
  -7471489976141,  -941661781581, -1519077763886,  3002302559918,
    663039771013,  1605726864235,  2005922589557],
 [ 4250970101161, -2011115256118, -2442314255295, -1179417975855,
   5333733062342,  1334393134560, -1574038371390,  3150621287536,
   4805338266734,  1816225427635,  1226974967377],
 [ 4983683345524,  1852620815355, -3743000131140,  1791614456828,
   -567655115089,  2415036182526, -3770153436093,  1353588916043,
    865467369408,  4540416849173, -3308391327027],
 [  653875695174, -2088800981864,  -396296213420,   574777457479,
   6775229964712,  1746981249329,  -217765778774,  4178116225117,
   -436521096935, -5077695128414,  1512824196198],
 [ 6338261765354,  1103907876255,  -495193503598,  -471410004876,
  -3451108817434,  -276797009173,   953344819002, -3358985667948,
   5441258081614,  2023964355435,  -508173452306],
 [ 2536643074779, -3692952713363, -1574451231015,  3508714524187,
  -2094648436830,  2337704889084,  5790495109123,  2579128237328,
  -2852311810587,   988057334451,  2447749092585],
 [ 1107940979252,  4585036193841,  2271843849403,  -965363534190,
   1620650767048,  3533508575939,  1463699849379,  4695963528445,
  -1064452003406, -5208530782449, -2021923639670],
 [-5180192026516,  -682001819054,  3365130146399, -1146825897072,
    833223008031,  3879974708325, -2289723829805,  3278672359678,
  -1386497537237,  4975083926998,  1283100178528],
 [-1173309723898,  1972452553536,  2451361482700,   640154543412,
  -1461649408553,  3335640582544,  2487877767457, -5842578928918,
  -2412034499213, -1801630687007,  5065751601085],
 [-1861323243252,  7134713267597,  1043143150418,  2407895374026,
  -4374893776181,  2563292203874,  2861480832555, -1695338029534,
   1327685478187,   348924414534,   297206271745],
 [-3160682237136, -1293519066942,  7051223326360,  1212818181618,
   1240982054689,  1292819826873,  -946419879752, -2511751426302,
  -5074675901170,   439922048758,   885621582510],
 [ 2188964509271,  3776040312346,   599471719195, -3057340945717,
  -1053173182133,  1489062314445,  5484454041137, -5695084326043,
   -279665896719,  2114489660823,  -924268607476],
 [-2476893720297,    34883886820, -3998421890342, -3476509194583,
   -303750020300, -2556222197138,  1420669576580, -1107807790381,
   3628549044514, -5341968597365,  3770306885879],
 [ 5580243107323,   870088036249, -3653724378671,  1377611388532,
   -854444995382, -3805450560520,  2204562554778, -3085267538823,
    792883755047, -4628634029265, -1101316943758],
 [ -443073828773, -2527575553027, -2277870089923,  3697915945056,
    519660245092,  4937386511160,  -256261999794, -1844351342821,
   -171254223366,   450416274550, -6797384473024],
 [-1614063693797, -5687794257250,   631118897913,   614539461954,
   3872879641616,   667496606968, -1141353611169, -4459858887340,
  -3008967932029,  3376122946463, -2679170226183],
 [-1077503001626, -3179364363585, -6355780313745, -1760217878449,
   4672660384934,  1558429221160, -1551256510307,  -821218393081,
     54578924800,  -901856944419,  4132401386776],
 [  474012047316,  -418492364892,   935216300467,  4101362436092,
  -7784530923283, -3021692423649, -3072488345516,  -563620609805,
     92653544193, -1445327407673,  -565170540846],
 [ -727291774990, -1199846378894,  1522043036747, -6424513445156,
    690286592565,  4094735859791,  4355456852779,    48083592644,
    762082613769, -3107376393562, -2826339611100],
 [-4785539510058,  -195122238138,   -87015655469,  1355600434426,
  -1894721462168,  6343721346916,   895517818241,    69205432358,
   5249358217569,  1388620904496, -1044879626466],
 [ 1403217816097, -2759955368282,   901952695799, -2830822998223,
  -2369047932369, -1967888992644, -1727162378428, -3656626225376,
  -2806465725575, -3912460635556,  5706484915651],
 [  900933303598,  3782303821132, -1238220147270,  1579435167099,
   5047188700148, -1757508394950,  5764458100935, -1695557742093,
   2723593858156,  2873081511427,  -717836154020],
 [-5591360915532, -2299262997255,  1023283552056,  6123785470364,
    542599258027,  1749957055294, -3776886321330,  -428384580942,
  -1507417469899,  1669069049272, -1427924670054],
 [-4320802718315, -4408013786734,  -951706198440,  -404213484961,
   5292587337125,  3116007011938, -1327154063206,  1881795726692,
   3074349217745,   540156817422, -2839732225352],
 [ -833007262294,  4275286650201,  1266984179126,  3194602883065,
  -2303239298218, -1413718556553, -3341802901801, -7042332430762,
    194373106103,  -911851377636,   531440032197],
 [ -503498478826,   641644035566,  1642707177561,  4021124148244,
   2349026710612,  2585350178490, -5082390229479,  3399573781690,
   4277067791291,  3520956838531,   432036574956],
 [-1075519744251,  -657866402426,  2279633816118, -1583392676961,
   -872446027515,  3055025066170,   613936841094,  -472056376972,
  -5999177194784, -6602001415793,   660273253248],
 [ 5458222620096,   726821298710,  2895790786049, -2423330058281,
  -2830370552335,  -594589717906,  1054714318464,    54724324427,
  -2323222848422, -5038937807997, -3892710328751],
 [ 4925348223428, -4238808160264,  1883829333124,  -696988168671,
   1612785621524,  -904493180526, -3575203377554, -2438749496283,
   2613419908066,  4676413794434, -1700373938607],
 [  789135193341,   876771811315,  2994340714412,  8164545731350,
  -2280921572686,   485848730350,  1031053925182, -1120268957357,
  -3550513617260,   344133324190,  1581566277484],
 [ 1148805594811, -1982179045540, -2434235799568,  -655088635527,
   1464554557163, -3338144399259, -2483848910744,  5831855229633,
   2447698493683,  1782260254513, -5075501792497],
 [  759758233706, -5381479911596,  5463015225044, -3259655955371,
    419547755608,  1674286216232, -1590599768081, -3251143232941,
    385436596648, -3472422706879, -1306550293429],
 [ 5850570351917, -2844347723006,  1152180391276, -1028321395205,
  -1803891869981,  2328543877781,  4261580125936, -4060654542183,
    -83855060725, -2791238368926,  2042396090035],
 [ 1021382773756,  -522249914981,    -6452390609, -1854192881588,
    401939677286,  6565397550965,  4646202339613, -1356365726256,
   1625089296953,  2790761783288,  4257201916257],
 [ -916864521284,  4882119279119, -3037114480034, -3598840689545,
  -1856465954116, -2498825285240,  -490199796195,  1425503727696,
   1881741783977,  3800615982163, -4816634486056],
 [ 3228333574343, -3775735676247, -1337034225973,  3272909243213,
   1640070794750, -7339972984229,  -162276523455,  -567635823245,
   1489156798299,  1078987798315, -1589561147745],
 [ 1009310519958,  6374344250331,  1321053132808,   278696672762,
  -1448952220730,  2205414665864, -1469892187714,  5270798397805,
  -2122826126532,  3721495116100, -1124741123393],
 [ 3884090917285,   989485866732, -2756010325512,  2997744873691,
    613459920307,  3384135229556,   581056758324,  7411009364290,
    161905946025,  -432413260354,   223835611333],
 [   -6416220723,    62298005769,  3509315492983, -4613741636480,
   2127034776469,  5178045787303,  -446050563912, -3615014478561,
  -2191147944382,  4018469496647,  -917179569092],
 [ -367232182889, -2442285948819,  1361271870123,  2381968867543,
  -3265462546848, -2167879350239,  6127632329518, -5240087527930,
  -1146828021226,  1745666144638,  1281263630456],
 [-1251017490073, -4642095748751, -2171086712114,   879411328420,
  -1608892765072, -3552415305994, -1436676767663, -4762639888399,
   1269985074694,  5097038000814,  1962471083642],
 [ -864633438705,  2007639737117,   549361299094,  -700145530508,
  -6756070553071, -1777328588218,   244120353697, -4279423278929,
    741244752520,  4912354536451, -1603955937975],
 [-2047245747673,  4353208118734,  4466611782840, -3548383600830,
   4999839198597,  1454755980125, -2458960259203,  1357907326478,
  -2777115559538,  -417104527389,  1193844659660],
 [ 3675941328561, -3486332771489,  -160820464138, -5684858841984,
   3339235986597, -1121182615107,  2683706915249, -4143352587015,
  -1057443892780,  1469492407342, -1390920364707],
 [  993837045276,  -438636370991, -8717966456784,  1766684229717,
  -1069629155048,  2133201807113,  1194040007098,   219210991929,
  -2630650088855, -1554823751875, -1785927066321],
 [-1367498723704,  6042596120959,  1537745030550,   770188839683,
   4053118337687, -2500307022642, -1634141626027,   799918225857,
   2835543828405,  3421800261642,  3594511289418],
 [  982823525219,  1200708454647,  1945809394940,  4024567441855,
    792309730526,  4512350191374, -4653124964118,  3383451539119,
  -4439864514732, -1275738318566,  1476739467597],
 [ 1106754610890, -2394320131487, -4897587067212,   292142796785,
  -1555079481674,  3451029272877, -1532062569004, -1802822589670,
   5499495710060,  2971643738652,  3157568531073],
 [-4345262074852,  1492498015734,  2537782846725,  2481763074580,
   6679536299018,   -19914908484,   942943074417,  -290013121900,
  -1700796117075,  3988954204029, -1377440865793],
 [-5432043423830, -2493218984646,  -841654449445,  3599258880566,
   -348924243872,   484952891758,  2333307556399,  6303940442131,
    332828845772,  2211691559452,  -265415436677],
 [-6529936089701, -2957302376195,  1140672433647, -3261543281125,
  -1437066055571,  3684338528247, -1442955524331, -2187329993338,
   -974580392896,  -619358671550, -3582742616297],
 [-1116902797656, -6743508315541, -1817602116827,  -434253387542,
   2035342670101, -1329262058964,  2502140641844, -4715189368996,
   1732358794722, -3514538145381,  -170437457561],
 [ 1029206877767,  1439657260208,  5621866832294, -1868589302411,
  -3155615871098, -3338652844717, -4742001464113, -2178623887785,
  -3356167984795, -1384513098671,  -502864970606],
 [  320429667982, -3946305475847,   930598799560, -1497434135835,
  -2953541775817, -6590623479233, -4481515560774,  1560827004200,
   -104091485319,   698423516255,  2457062236060],
 [-2978034195832,   430650162067,   988431484766, -2343115725466,
  -2048729141708,  1978716107729, -1092585951018,  8021485138991,
   -973519475706, -2070889786771,  2364691006950],
 [-2712219526408,  3310764168929, -1214934419161,  -833709350239,
  -6332709171084,  1023517680586, -2711222605694, -1506671847696,
   -578139731924, -3026126981594,  4387288104757],
 [  -48903615025, -4144457369607,  3793232874556, -4122724679103,
    752987362722,   452285299083, -3863667028717,  3829371623270,
   4513875875097,   831222628119,    77709434942],
 [ -666721920985,  1088966220636,  1634692132718,  1319565594151,
  -1376175422408, -4184119051681, -1246328486604,  7021352075837,
   3601519529610,  2399648537017,  2230129144183],
 [ -450541913644,  2941073593038,  2689672157268,   794082250564,
   6644727806254,  2166885399331, -1002974015615,    45546992483,
   4739837281813, -2258073274905, -2421035200007],
 [ 5497457887295, -3331913189351,  2120338954097,  -812017651650,
   1035748752659,  1874777695388, -3730526907220, -1404709821964,
  -5727686966612,  -471728256119,  -120698876988],
 [ 4191754506567,  1514866414373,   167435490699, -1263952486962,
  -4236564175885, -6497242769183,    -5043047408,  2508947347210,
   2374411993924,  -457576869117, -2490797587303],
 [-2398785273054,  1801277758244, -1369097835142,  4809980281419,
   3379854663624,  1225743869128, -2281363772418,  3167390354183,
   -883402227426, -3920501243994, -4655889932572],
 [ 1980737205664,  3053404339992,  -323943331253,    20867510076,
  -1777393188178,  6718044530180, -1819063817558, -5467050226464,
   2011280082124,  1021475674295,   265493871800],
 [ 1599869272571,  3377719330174,  6001091289038,  2081212663557,
  -4732197244452, -1506882132131,  1470853105028,  1047753312878,
   -814439480096,  1323626136882, -3917398298387],
 [ 4083313123547, -3653278918306,  2434503600644, -1346087693035,
  -1349610752951,  4461562678186,   249213063476,  4301172841954,
   -116976792089, -4525602358660,  1207228751899],
 [ 3505121213808,   -54964681561,  6330466537123,   936869135621,
  -2245033441714,  1052802142894,   -56606474689, -5736285575286,
    -98364524252,  2725251134633,   514937353695],
 [-3826788354731, -1549339011126, -2816648506289,   823919117818,
   1815495823546,  2433189957751, -3034549349066, -6366145289961,
   3649625483909,  -303992348639, -1406405494359],
 [-2879258515750,    18437974896, -1788210766892, -2916196539595,
  -3765367125024, -2883992614280, -4247599018791,  2690967326177,
   5052300992980, -2295362186083, -1197661048381],
 [   85474414588, -1059316670792,  5642243025391,  3082520650536,
  -3245404137150,  3524664313803,   698971571389,  5231397282827,
   1594622910311,  1713869112672,  1114247507789],
 [-2571844755038, -1063082595609, -5931752930656,  4827706638804,
   -682261633973,   442064640260, -2804656597975,  2023493164644,
   3777253088939,  1111726471262, -2374431254449],
 [-1451865255754,  1434879333664,  -222934803148, -6032820997610,
  -2754518011731,  2875699145837, -5660480943146,   761341028648,
   -493538755737,  2870569005255, -1557925806967],
 [-3597540747759, -2287267736214,   358447275322,  1718727051754,
   1388467521087, -1907068375136,   404805979464,  2287631596342,
  -3608626971544,  1902172164019, -7151312756131],
 [-3056848776557,  3910743207505,   275583471771,   670911991649,
   2757070913723,  5404126904843, -4416514500153,  -892044492083,
   -608187213609,  2020639000540, -3643548954192],
 [-2751876334569, -1550898849626, -3990090800205, -1259031734331,
   2776101594701,   846161726660,  4491773842344, -5498344852050,
   -446646521187,  3420148409171,  1337324771812],
 [ 2126034257521,  -566761087013, -3202512097426,  -955255048591,
  -1957779120038,   -98981002933,  5343663038295,  4232773194344,
   -325280990750, -5643641570750, -1311436791707],
 [ 2020061716733,   399958811246,  -454228609744, -2238824832238,
  -1181283168353,  6432062026078, -3683641723228,  -122700509248,
  -1491086193600, -3824909829059, -4163691453763],
 [ 2144395714720,  1048885248121, -2961713568381,   862342851146,
  -3262508813018,  -962393920531, -3358586808125, -2840852389626,
  -6185303554465, -3784145993324, -1134711364446],
 [-7141283903264,   551834985988, -1228956085633, -3427703495950,
    -79846573493, -4673140541172,  1013599254678,  -369132877622,
   -117029324101, -2203279605677, -2749847384473],
 [ 2870850102456,  -342530910644, -3025183865207,  2395044282745,
  -3510929228914,  -154488423091, -1747033003302,  2071335827742,
   1476515852691, -3939638776374, -6273676579891],
 [ 5455606189908,  5161643859152, -1963267172191,  1962918350939,
  -1506569440660,   152020201372,  4080574760052,  1488678826955,
   -919845174390,   710634322540, -3657197615865],
 [ 1666038769592, -1144981454845,  2413758433311, -6344060479149,
   2341156973304, -1828752695926, -4756556500031, -1554846357901,
  -1327644699779,  2221633325014,  3045320137186],
 [  392718162477,  4676739216630,    77425302885,  1417772730177,
   2597757214163, -3343608510592, -2224972537263,  5445498862369,
  -3671191664282, -1706044551920,  2652838335825],
 [ 3724160656674, -4061502039499,  1023701547041, -1786358074463,
  -4415514817647, -4179494048051,  -316437645212, -4851662235880,
   -425996373440,  -300935959329, -2126041476570],
 [-5364912915398,  3342606599752, -1736109834737,   750035218692,
  -1193393702016, -2098009179891,  2967467968446,  1770139858878,
   6204110394335,    -6003298785,   460376157721],
 [-2393626640652, -1049091665942, -5122478167056,  1088697622782,
  -5105574070497,  2435637978174,  1707308916357,  3641923304831,
   2375835061606, -1693161318275,  3009285224425],
 [-5137934506729,   398416258386, -3064932278532,  5125884313080,
  -4636778408632,  -467235353649,   710025386347, -3211870763345,
    686833201600,  1279034186343,  1768743224370],
 [-8499479993195,  2896057304196,  2300168240403,   381215387805,
   1211236786556,   912532188414, -1081041844269, -2481562130878,
   1138235680116,  -391226534827,  1691254723136],
 [ 4033905237261, -4871081723045,   527943332897,  2438280532272,
  -1392948478908, -2994883443748, -2550199728912,   785654231548,
  -1459817677244, -5797650556225,   -44856538943],
 [ 1798605426441,  1782407567411,  -127633268326,  2585887676284,
  -6406453074812,  3248170520160,  1039266846782,  3233776473994,
  -4249459430586, -2128554974155, -1079650933168],
 [-1524879561131, -1348790227653,   185509040260,   329713680789,
   4083642895560, -5661792069994,  4485155389945, -3159021193549,
  -3361256296350,   675473040821, -2264953045065],
 [  415419510304, -3420403757556,  4999987567914,    53920267208,
  -2835870013600, -2987658367312,  1388260272541, -1118843304886,
   6096363081975,  2392770253011,   295213822969],
 [ 5427952107234,  2491529961799,   844488585689, -3601761663589,
    349389474711,  -485436373845, -2332590907306, -6305750705375,
   -326824207595, -2214997798918,   263710502677],
 [ 5410465593212,  2284182716040, -1830665274215, -2386366134810,
   3881010473360, -3743818152518,  -734591050674, -1191235830145,
   2912744793533,  1133676778387, -3956880585831],
 [-1519966253416,  5499495095313,   701947719793, -2418777804335,
    600867197418, -2887137450951,  -525054769308, -3236045504713,
  -3827991322506, -4555732971445,  2499908413479],
 [ -220262711735,   -21686679736, -6301349010170,  3604978483523,
   1990534831616, -3278942072131,  1224944290631, -3308018241872,
   3625732944572,   109840321507,  2633054746403],
 [-4610510376437,  2813617129952, -3151596993439,   -88494734667,
   5624819370040, -2857888666106,  2454279412895,  -443157633996,
  -1303047844300,  -865935395631,  3523149627136],
 [ 1591898023250,  2469941412807, -5931303111509, -3304855274361,
   -876943572352,  2133776999267, -5194459233467, -1995517495373,
   1538603604609, -2276570139335,  1194245096370],
 [-5854130447786,  -227585494107, -3740308754695,  -288690684276,
   -617533036803,  4168516334893, -5171221429765,  1767360391392,
   -361225444154,  -867392702819,  1760194993347],
 [-2073846400455, -4864443057694,  3768191015592,  3279233077955,
   -127179189053,  2890636750283,  1306208403755, -2158360568070,
   1156806226339,  4731226850683,  2936725638233],
 [ 4380402244915,  1718336099159, -2490203993633,  2962792650483,
   -609296656817,   -88183771411,  -528514858921,  1835636669616,
  -6875078311611,  3073438996418,  1462704076511],
 [ 1808548016756, -7156596836882, -1005793147626, -2439553382253,
   4381084164538, -2569424600789, -2852359039920,  1672380191489,
  -1251321929623,  -391277073385,  -318876263817],
 [ 3050479025203, -1112913585673,   583195931072, -2411256428011,
  -6500725040245,  -177287830264,  5844127903559,   597134495908,
    253318857497,  1220454761246, -2224137621907],
 [ 2081026457771,  4576326717134, -3059971157905,  1791758352530,
   1929136083966,  1520812361717, -2120604198642, -4189323624596,
    840252242315, -2988107401845, -4943288443379],
 [  374633385085,  4064005181314, -3643593868621,  3916456117646,
    731274504682, -1390998890607,  3891975595409, -3931218738608,
  -4577567048717,  -775483008911,   319402267704],
 [-2835830561561,  6330276368832,  1469030071465,   402575127112,
  -3894140626988, -5490744757063, -1778186499571,   725501790480,
     51664708241,   528979064893,   531180438358],
 [ -407377873713, -1054311025402, -2688240679126,  -400274214617,
   3049297157362,  2934281209053,  6032558300772, -1387844054570,
  -2051260454929, -4417782970588,  3374173701078],
 [  684982537657, -4959087143757,   106581022743,   541496847341,
  -5426086784485, -1687822946375,   394145178486,  6213307814871,
    787828294802, -1036369422923, -1373765538818],
 [ 2436940108262,   -17959317412,  5272745875180,  1859736836422,
   -572591553586,  -196355684278,  4728622193259,  -100172273328,
   1443302561770,   711844762922,  6121630990387],
 [  223238508073, -5979661064274, -3475906742063,  3953573669553,
   2030325741149,  -732468271069,  1890864460635,  3262533163758,
    -41044222721, -2549195724778, -3332092252171],
 [-1774327400291, -5768496654472,  2193622125140,   685714364485,
  -1578315854585,  2649364870676,  5560588530162,  1445106797385,
   3397613217391, -2045535048810,   217585806122],
 [ 4658041545519,  3630655850203,  4551294374768, -2158382540553,
  -3182453315019,  4504288573550,  2294451726266,   694082709894,
   1687704540905,  -342007246067,   787331961924],
 [-2465951597827, -1864199930418, -2270856798900,   -22595841598,
  -3664322575097,   410068708466,  4800776928729,   516836974097,
   2660325854959,  6212676802611,  1643732824380],
 [  359846782248,  1044409315702, -2799990553119,   262881376730,
  -5481381738941, -4220241657587,  -294707117620,  -304138737129,
   4283597060107,  1718359722968,  4640405561466],
 [  276745595187, -4446743546668,  -420469848217,  5877043034202,
  -3498803237174,  4162570629251, -1979809669506,  1484541890021,
   1336631902838, -2815013300598,  -174470552559],
 [ 5102172976455,  4259278196381,   646116293517,  2524356648706,
   6102964143435,  1254045003892,  -957282256921,  1873342983288,
  -1573270140866,  1489934935322, -1047305543583],
 [-3026034433384,  4955504388403,  4333613435828, -2343097902290,
  -1131529879827,  1679795382350, -2094050038152,  2926412538950,
   4863771372296,   454595688732, -1049686222394],
 [ 1413569263672,  5664984198517, -1424796239863,  3110895550026,
    552999270359, -2663403492361, -3366801478812,  2831708540909,
   4442817380487,  -589019091899, -2713492019071],
 [-1196383610997, -2317590034549,   488155267722, -1612569113807,
   6813504685085, -2793562988534,  -651338217079, -3533045594143,
   4362308711116,  1960841909849,   591300922110],
 [-6225964858091,  2664169319390,  -865309055360,   819974520373,
   1830992948396, -2381820387657, -4175153119521,  3871551417818,
    656947569754,  2466614045870, -2180885828971],
 [ 1769560500047,  6037989935725,  2421935205712,  1203216048338,
    672174795176,   195496479510,  -872012665847, -3273813092151,
   2768564775195,  4507389782533, -3626020480703],
 [-5047746784092,  2453874805493,   605654139215,  1972416669127,
  -2665071988780,  -422111933646, -2198438897961, -1931046200821,
   3511718736225,  -872604552173, -5941804355173],
 [-3328181454099,  2292646681385, -1230196268189,  4178873463660,
  -1952023152818,  5264778689679, -1386241486348, -2997510223232,
  -2074354162481, -4120755939342,  -985908910127],
 [-2128722363465,  3342442538371,  -168399550678,  1018145277613,
  -2470638175877, -2594108650010,  1574952019260,  6683203855831,
   -391972896290, -1591148785755, -4535096826577],
 [-5201779311507, -5745047858643,  1148085852099, -1972658788736,
   1626021341995,  -443420182872, -3687980730355, -1722445336866,
   1126867930447,  -728342002724,  3676619806419],
 [ 1025623808061,  2255781933926,  3434448250170,  1588728653730,
  -1672468315028,  1399827775510,  7540246372905,  -415840499655,
  -2622968673028, -3238462954919,  -622524241645],
 [-2330888881958,  -783041291864, -1611954800518, -6710003615179,
  -2409429846717,   164483025058,  2876722005477, -1406424960130,
  -2560402079163,  4389258928773, -2103339319444],
 [  981353914797,  4600570118021, -4095848035320,  2695778349891,
   3604971925019,   268230099620,  2669684981719,  1433109276176,
   2427347451707, -5026339548167,   650854878249],
 [ -577346357670, -1065056354349, -3503260666052,  3343754775146,
   7471599453113,   941566908039,  1519206252295, -3002675888721,
   -661863774295, -1606353643907, -2006230896461],
 [-1530270045251, -1165038214408, -3910191264386,    42757199935,
  -6749578664506,  3655208406975, -3037805775025, -2087407439549,
   1129782270412,   146803346119, -2684898890488],
 [ 1124648695462,   619841252926, -4343167328282,  5126149833326,
  -2012444152893, -4492681408815,   228998673850,  4372547039269,
    461299896200, -2790084369521,  1344378474112],
 [-4934004008631,  4235433403434, -1877806736947,   691697509341,
  -1611705070234,   903752864656,  3576555598625,  2434967123936,
  -2600892774175, -4683201034835,  1697030426423],
 [ 2586198910446,   161208459786, -4102174412712, -1602983963062,
   4264078758861,  2903065212760,  1279685247477,  2722901928486,
   1107788946521, -2335008002528, -5616653031119],
 [ 3230590122770,   164261854428, -1998144233601, -3965645048161,
   1037949781618,  2027214434103,  4278313498655,  1105158967135,
   3430220607132,  5007148562462, -2875265811886],
 [ 6761580377743,  1346954619468,  1557368388076,  4516065361338,
  -1017022327120, -2592367674560, -2820017724509, -1495826289031,
  -1863906044044,   102998318679, -2866539059950],
 [-1377509159657,   455485182438, -2949168797229, -4767997985688,
  -3528005486674, -3659205157792,  3372542508170,  5103307954216,
    717304332569,  1026608060109,  1280917880731],
 [ 4576680007827,  3360652753004,  1053724557420,  1053718472281,
  -5382930598816, -2808844103951,  2350474900212, -2196771652022,
  -3356114285095,  -478194092956,  2613636825032],
 [ 2852163474361, -1135949180783,  -211725018832,  1315891010135,
  -2075288507285, -2173478161662,  1336495634981, -1030302943854,
   1682346975780,  4009979323683, -7616402455926],
 [ 3857915221829, -4463500003492, -1062016014312,  1187751209208,
   2073567876700,  -835202429467,  4556863313623, -2101419111944,
   3696351907824,  3543026342009,  2502151362963],
 [  401382018489,  1051931974710,  2692405464188,   396635193722,
  -3048558168232, -2934833001129, -6031609308015,  1385264991156,
   2059965050099,  4413057494559, -3376567915437],
 [ -169167788834,   880019797476,  -711861100149,  3289188658280,
  -1342563601245,  1346475859727, -8570110050068,  -770689718251,
   1127827874954, -2257027985665, -1961898300042],
 [ 3930207101923, -4788272198497,   656309023670, -2770544407001,
    557594079672, -1221527037189,  1949795656742,   754170867605,
   5202682587302, -2373954082924, -3826510589359],
 [-1070179288209,  2415714774160,  4871960243032,  -273380831179,
   1555236788003, -3439007596191,  1518887854371,  1816423812368,
  -5553714292496, -2942765660316, -3138907782101],
 [ 3940018399641, -2506993712381,  1927018684527,  3369813266746,
  -4298165936640, -3300625203781,   924147333853,  1210577413806,
   -875664104610,  5450673764923,   978600957183],
 [-4957499786340,  -560490955659, -3196530206184,  2729154604869,
   2729502871664,   637868740008, -1108714879636,   155258764813,
   1587348802021,  5408907770018,  4068977713458],
 [-2489799141264,  4434234188279, -1202064872044,  7037820836620,
   2789740578487,  1506913373001,  -408374381403, -2200198212345,
   2693826864962,   885099416630,  -247788096976],
 [ -389022934133,  2710548948948, -5327964571766,  1766636537686,
  -1734693563450,  1930537485456,  2680796992755, -3674273444797,
   3506538711694,  3221342484833, -3301123849731],
 [-1316371471221,  4984144857377,  5149034340701,  1864552720515,
    979887311061, -3180597467149,  3423896005233, -3618977271654,
  -1753560603942,  1737977565955,  1201821181321],
 [-3962236789207,  1750259608810,   148960340831,  1779306059384,
   2981987855892, -1253539672826, -2552909536048, -4439189885805,
  -4920945901236, -1751087652709, -3752301685105],
 [-1246240693862,    23064000757, -1867078026750,  4661938680281,
   6095716390748, -3242714454691,   273216436183,  4017729577537,
   2942885259625,   791597255920,   223038496552],
 [  -30649670188, -3871015798898,   704807144470, -2320378138230,
  -2357264920905,  3134715677961,  2576542245250, -5342097663914,
   3860660551080,  1647128407834, -3309975816110],
 [ 6894364675615, -1536991055335,  4716871019965, -2137498887887,
  -1732935394892, -2187576148777, -1761947809958,   381412063761,
   1707817215756, -1068696752044,  2861883710692],
 [  112529695580,  1072157541057,   674722309129, -6700999288825,
   1257537007593,  -247891340853, -1920166825819, -1933231865155,
  -5187788661630, -1841296457101, -3756111585811],
 [-3458907956722, -2289333716918,  1197674347308, -2043672756747,
  -3032049510746,    79839440230,  4925347625983,  3887792418169,
  -4954357866607,  -935659668029, -1785801254471],
 [-1895276345846,  5269942247678, -4571502851511,      573694574,
  -1913509641237,  4275131330747,  -712911549218,  2758847108459,
   1991204978412, -1950884204095, -3147761728208],
 [  911004404572, -4884438497405,  3041146246306,  3595341898145,
   1857144844875,  2498224328604,   491146027412, -1428032983801,
  -1873303763754, -3805218707120,  4814323097766],
 [ 2004940182490, -3377861273990, -2836125064246, -4011742049221,
  -4826198407407, -1579685127785, -1692090129551,  1004315098601,
   2905310241165,  4442787437574, -1610585800723],
 [ 6321039821476, -2600451384479,  1210817283665,  1208651410377,
   5738198705036, -1245592945410,  -243411859957, -2619475786472,
   1040117842899, -2671921119669,   857236814176],
 [ 1464911252042, -1429635132299,   214089203797,  6040223039937,
   2753642908473, -2874026574776,  5657954751029,  -756132119662,
    475762436227, -2860233689147,  1563968488668],
 [ 2024222593676,  3280398978682,  -776485984108,  3547678058283,
  -1037485022957,   101765635860,  1222606531210, -3699551759045,
   -414345348428,  6291285314975,  3991543685779],
 [  418325636565,  -976628268689,  1960130308613, -3131328225267,
   3158424345724, -1238270377957,   224716563958,  3089029508549,
  -6964846779600,  3950714977405,   -71241800321],
 [ 1489527748759, -1672027828684,     6450993624,  3553007823045,
  -3447768798791, -4787063826997,  3771831286954,   782420859434,
  -3615334610695, -1957132225043, -3977276642890],
 [-1828623208758,  -278174491750,  2607271135651,  2827484184789,
  -3362214664270, -2695356375558, -2098354025067,  4762092301817,
  -5709012503524,  1853552845219,  -328174437859],
 [  406777144877,  1082675837846,  8160639820723, -1302165462571,
    570799717723, -2118686180845, -1205705957770,  1864618075055,
    220710737538,  4169787838341,  1785960103011],
 [ 5431184070137,  -309737848981,  2160845782374,  4325354834855,
  -4093168843177,  -335438513349,  2957437208333,  2026067676495,
   2741298099837, -3128261655508,   -98395198705],
 [-3801575303236, -1326300030816,  2902747383118, -3164052910919,
    102088557005, -2691942486225,   232441481562, -7596192477934,
    157609214931,   281536406512,  -484284223556],
 [  870030828371,  4241678915053,   808293549586, -1616247604007,
   -787092901633,  -132753496525,  3552033927118,   233893752987,
   5370834839860, -2915968611203, -5227626775815],
 [ -506870497010, -2130343626318,  2149499315013,  6417195180367,
   4227919514470,  4464540212850,  2220798395012,  2222111890082,
    954554218823,    95595711536,  -898061942099],
 [ -324237004813,  3944786027795,  -927906648754,  1495096796314,
   2953976428372,  6590277965583,  4482138600796, -1562494879528,
    109672701189,  -701466848923, -2458541732536],
 [ 1245292199359,   -23444566233,  1867756989833, -4662601885381,
  -6095537007032,  3242683991266,  -273108949196, -4018152292284,
  -2941412956581,  -792344336969,  -223379123929],
 [   71316212551,   814716464961, -2850928767408,  1110667068218,
  -1441092269598,  2025468463968, -3093331235645,  3124664682535,
   -283034304041, -7906608771207,  1365176650977],
 [-1428258126828,  4442614282252,  6339864332681,   667625196526,
  -2645336875445,   638054729150,   172956580553,  1998043476530,
  -2242537274146, -2836925815891,  3616561855795],
 [ 1862005254671,  3405517571734,  2253927404268, -1023599065883,
   1261979031615,   -85706852298, -2745053539391,  2094330190451,
   -810787164646,  -297173455525, -8033696546314],
 [-1405446876079,  2759059261876,  -900391877185,  2829467448571,
   2369321616397,  1967666102388,  1727535445022,  3655643684313,
   2809717869209,  3910704397889, -5707371709276],
 [-5674064524733, -2501010713368, -2027147460036,  4003059107952,
   1467189200434,  1306497371464,   747398537831,   -26249408325,
   3599012165758, -4470903015833,  2013800548899],
 [-4061251660559,  3970924364989,  -826536677181,  1630672788067,
   4420910313958,  4159312858739,   360275634236,  4772058270784,
    840163189121,    56015589920,  1984957418361],
 [ 3100369373946,  3618829690233, -3943659757447, -2085962908171,
  -5284398817946,  2817370743566,  1310074672458,  1099853485048,
   -660564537771,  3373159711192,  2604825104656],
 [-1599835715474,  1171222746699, -2460125766453,  6384414730274,
  -2349076021091,  1835123130434,  4745784133927,  1583862847820,
   1231102912196, -2169672507598, -3019234328815],
 [ 4315279050541, -1504471071267, -2516854702418, -2499608888206,
  -6675867671970,    16443473889,  -937797981903,   277048238808,
   1744117134227, -4012814526763,  1365286042378],
 [-2576691553872, -4168355303390,  4537168270309,  3289883489257,
   4545811286758, -2343742404114,  -883746419592,  -423757223990,
   1797956997202, -3357840962479, -1717670949614],
 [-2770757495943, -2100276262903, -2310355957864,   815198757292,
    378008579111,  6654075747375,  2004799154313,  3938478545291,
  -1882585389522, -3425197886948, -1637953937110],
 [-1957929560953,   632058228489,  3085463734040,  1058548099344,
   1935515723895,   113529956234, -5368507010752, -4158778252136,
     79583267059,  5775251335043,  1375182887763],
 [ 1043608214197,   645438379821, -2257458783399,  1563856913321,
    876487024162, -3057822807242,  -609133909223,   458157961937,
   6045544209448,  6576656903033,  -672338544894],
 [-4142674083739,   191452447235,  5844843854843, -3328434054873,
  -2950087353136,  2629530078902,  3529773720027, -1643439769921,
   1641203888894,  1727359214850,  1051648120926],
 [-2536425192805, -3412028873887,  1603269936707,  3714143636044,
  -5958326431609,  1573982825063,  2066214946180,   169698750068,
  -1260855487109,  2893560055930, -3649617067653],
 [-3370910924887, -2664833577526, -4212224537043, -1664159625471,
  -2346009074864,   941991345314,  3116568323270,  1337408376752,
   3417509333245, -1300488029900, -5455268464385],
 [  658394595659, -5845490182112, -1734937401395,  5132881559044,
   3731320560325,   406316780591, -3110371669968, -1368398831925,
   2549752914186,  1048224195587,  1674960469699],
 [-4069018215699,  2876255152390,  1999107398479,   288371782958,
   2154913562127, -3129067806294, -1567216762796,  2815472413652,
   2748385177040, -6162870836304,   859237607542],
 [-4081397478017,  4440571166733, -5237098386667,  2415216733177,
    -82617245215, -2443974120286, -3331468168048, -1207989621760,
    -65932252178, -3435715154676,   129657560479],
 [ 4540975362633, -2962280943473, -3727641214052, -3334039256515,
    570980206559, -3755688295128, -2042533319394, -2636445564667,
   3169252319979,  -272285070269,  3149763104314],
 [-1970685170517, -3212396105451, -1122959696365, -2522166126002,
  -1796870295135,  5719246575631,   184448827929, -3166927469451,
   2390181077127, -4324669286196,  2785111405287],
 [  200644095783, -1408319683626,  2230072881087,  7141211846352,
  -1065713117235, -2345629089454, -1055615018478,  3737240815498,
   1897700894272,   777942571718, -4010083583079],
 [ 3207211747233, -3860205876030,  5140569835651,  -471207339362,
   -337229668079, -4107979119479, -1338074993769,  2629126458727,
  -1774101172768,   760813963139, -4329765470951],
 [ 2143894337056,  6169806192886, -1781372515887,  -719475094542,
   1035253435287, -2386252505629, -5559323574118, -1325703604772,
  -3012161951007,  2175050982007,  -642487683129],
 [-1100041253761, -4027322736527,   229361570027,  1637503933953,
    608768884422,   512867167535, -3736704991334,    67174202189,
  -5291870130711,  3555218083799,  4959030247620],
 [ 1420617715787,   900534716736, -4641612714007,   858580710316,
  -2821052711788, -4292847420429, -4102875100962, -3693473208841,
   2833909355485,   831540662000, -3050318865731],
 [-4069141454652,  2397030807707, -1277633907502,  1139048765540,
    -81027369382,  -511257043713,  4392208537723, -5619448373269,
   2158427361261, -4069786076345, -1551431016185],
 [-2187452393712,  2289066386737,  2720374946090,  -754365626582,
  -1007844680787,  -738792081057, -7404298713659,  2534366005211,
    -13257462581,   611993086866,  4338524326161],
 [-2261267990905,    31011633178, -4461713731260, -5600361739838,
  -3850335661845,  2189742342684,  -216324100412,  1717505044705,
  -2840927392525, -3518702564105,  -738885782573],
 [ 3835162625076,  4018145639189, -3969032215791,  5489480336209,
  -3320287131143,  1771420196090, -1544616085553, -1452735700983,
     93447194857, -1212172345514,  1766865015618],
 [-3082861861208,  3058375781651,  1165336696419,  -124017674997,
  -2636642417196,  2258840895000,   782669515714,  2647187849760,
   2623806578654,  2724227790305,  6766735682713],
 [-3924384276199, -1257058873969,  4552529697420,   387469317643,
   1545295213894,  5125273879835, -3071156583424,  1850357772371,
   1429128901788, -4262910977786,   645241204936],
 [ 3701699616746,  2383614266719, -1367424358250,  2190400968495,
   3001798884658,   -56795233975, -4965441320966, -3781381548309,
   4601491412457,  1125849679522,  1881052978621],
 [-2189333732830, -3776207341475,  -599211629311,  3057114021789,
   1053223858244, -1489095677808, -5484378375890,  5694926621678,
    280207726420, -2114773711882,   924126154637],
 [ 2665403576355, -2944060824958,  3157438546205,  1078691117758,
   1382923782302,  4459945621588,  -505584133266, -1501254246342,
   6777564021800, -1030807481720,  1335749241587],
 [ 4212558911551,   750153481199,  1028258998901,  2239063169609,
   3265243301827, -3837339068027,  1328070175434,  4072573976198,
    618748560534, -4246823513919, -3669594372664],
 [  -93602350411, -1556291309329,  3891657186953,  3723295327173,
  -2291127866238,  1160236616592, -2729402661166,  -356981559810,
  -3888216948351, -3338076730838, -5303564299718],
 [-1401779354450,  2278330163184, -1845651571694,  -265759043783,
  -1100547800936,  1597321845900,  2809607617862,  -311530978355,
  -4080685213673, -3942194199680, -6739487572218],
 [-2719919362899, -2391373190823, -3969228213732,   574141537015,
  -2702607686033, -1963447676327,  1386211653292, -5103770747637,
  -2713538834489,  1100638362345, -4805724744400],
 [ 2523998535100,  6205802308153,  -740529115356, -5459372639947,
   2215638830763,  3653337752122,  -638121391235,  -468164566807,
    914489609231,   343023507309, -2219921307491],
 [-2350524571905,  1019407106351,  1732790704046,  3013675873791,
  -6054456149167, -2669548888846,  4804552750018,  2998031686008,
    203759526543,     2877308034,  2335521273375],
 [ 5028263042894,  2227314771702,  3244450087185, -2909303881378,
   4048893324798,   746603634549,  1165671661409, -2163688951411,
   2917552225784,  3511807906586,  2632286844261],
 [-4714905505924,  2895973072505,  3843863522651,  3233205036684,
   -552014538054,  3737708661071,  2071463682670,  2568124973805,
  -2931446631079,   135373795872, -3221953642295],
 [ -399808288793, -2027286498904,  -926551441885, -3676835599757,
   1065045172846, -5815841324681, -2757928855920, -1768235050712,
  -1611389885744,  3963506395496, -4165996467535],
 [  637654470917,  1884384072534,  3454373034475, -2817649664288,
  -4340794990017, -2094754538411,  2347442561387, -4183501013377,
   3191236618832, -4027802664085,  1876379562685],
 [ 1169509591568,   888130497109, -2780123094578,  2233387019770,
   2466835615100,   826095556511, -1869080414836, -5231726772561,
   -402940143583, -5836576954281,  3642284968454],
 [ 1920808507726,   -17069658569,  -611385986731, -4447880127311,
    851659797175, -3668042836712,  4197058334701,  2683661104136,
  -1998312720929,   174016363969, -5756043708775],
 [   26883657634, -4931237083796, -3796335053355, -4723730642380,
  -1415711385171,   720768095098, -3722199717928, -3871322798266,
  -2669188419925,   653348669329,  -198240881117],
 [ 1376090713691,    52794369058,  2105420292898, -1682550534643,
   2542698783425,  7624383483088,  -148779774205,  3913076935775,
   2098374493774,  1721371826116, -1882198869441],
 [  681296782274,  2051665305196,  -956475448024,  2319926209191,
  -6356153701200,   700058790681,   518621453633,  3319478656765,
   4804216979985,  2728945451941, -2514612942808],
 [ 1618110226052, -2193439357575,  1695044091886,   397656341641,
   1073388847499, -1576671686419, -2843336252998,   407189794653,
   3766298781170,  4112633385347,  6822906738995],
 [ 6164789949614,   136739143580,  2175454310590,  -622114272459,
   2011485522508,  3965112522977,  1210393364110, -2980277210958,
    902750629627,   449521686336, -5071953818337],
 [ 3881218273012,  1479027175218,  2070483335377,    75560330693,
   1057802184062, -1384772852312,  2238953203018,   945086030287,
   -776171777608, -7173099477397,  4178226133483],
 [ 3235814803974,  1256294864562,  -650900085810, -4903438216654,
  -3214890352777,  1970650069612,  -252075430182,  5758156457942,
   1664750015810,  -670281194160, -3581101717788],
 [ 3378074137115, -5251527924600,  3110186772230,  2979998202959,
   2321062449968, -1804075580466,   996582370415,  4811856309508,
   2164952889186,  -282367400925,  2213709332985],
 [  574905043173, -4386090861270, -1092429566069, -3352980169208,
   2327865872888,  1391131336277,  3388203163344,  6943674361107,
    166333751023,   709426251370,  -645073660217],
 [-5093483115816,  -172520412035, -1119927279528,  -298914389540,
  -3684232686653, -2715581429324, -5996134939421, -2654472304753,
  -2448900299536,  1566386998418,  -533468096138],
 [ -694829797925,  1077657175865,  2426418664382, -6898838565028,
   -630728354927,   891813183909, -3004655039870,   712908865280,
   1510580937205, -5343119532124,  1820512127979],
 [-2431689551338, -5427049186581, -2338557330124,  -815424187913,
    981962290892, -3062993059108,  1128316606362,  2535891852174,
   6084947796540,   617493934047,  1744232021160],
 [-5435703869515, -2294163621518,  1848370199428,  2371054182793,
  -3877945120584,  3741103307158,   738838875423,  1180167173884,
  -2875974256951, -1153626673599,  3946706754350],
 [ 8219585512141, -3454414521062, -2804725743538,  -219356185092,
  -1174314716502,  -448706854993,   982240147012,  2239687163655,
  -1333597836815,   384385903109, -1761603754385],
 [ 1591055287346,  5678683783494,  -615012178719,  -628567142182,
  -3870188383064,  -669880486908,  1145281971929,  4449831881766,
   3042481569158, -3394493236261,  2669963658827],
 [-3754248407042,  5294073804022,  -296736245623,  2912063924725,
   -791691392173,   751000806695, -1982989567750,  -407567361095,
  -5153438369097,  2080369983896,  3624255814498],
 [-1109208301621,  1105002101950, -1209577854326, -2024207507329,
   7350334972180, -3488128656973, -1651140239034,  2095339824418,
  -2280768401335, -1030270576505, -3523292766568],
 [-5069180086335, -1838030708503,  3563978996722,  -302815316047,
    440743640453, -3187355375955,  4347216998557, -1328131210369,
    353915357918, -3762804268885,  3582142309006],
 [-3661582264231,   -11651104094, -6216800774575, -1027515731357,
   2255363625886, -1074441912663,    87463515947,  5662135623408,
    329269044501, -2851689332257,  -582416329253],
 [   56248471539,  4946765000660,  3758738313061,  4784112795759,
   1394864083549,  -728062773091,  3727632364237,  3905942835080,
   2549768126638,  -577510027537,   219804846640],
 [ 5587427890418,  2297669997514, -1020505798020, -6126153847699,
   -542119172009, -1750346906402,  3777573935242,   426662882168,
   1513176033026, -1672209304069,  1426348449094],
 [  259175505863,  4398571312812,  -563569394904,  -182986310141,
    198425934183, -6310433112917,  5597938858309,  1367486286215,
   -595778885575, -2563939288918,   487534331752],
 [-4802134395185,   159246504059,  1550301624727, -4853626304168,
   -500450446506, -3445176305248, -1086016460941,  -198494483359,
   3935712232757,  4174986491713,  2166600367239],
 [-2955758367798,   197182840185,  5575429750308,  4821829786482,
  -2151221731203,  1377147185183,  1369265209861, -2672300951780,
   3787808035786, -2629332593910,   296485252008],
 [ -907637263048, -2208932588521, -3516691104412, -1517457288318,
   1659108833526, -1386977275547, -7560769135554,   467600743208,
   2452516555656,  3332072477470,   670257831686],
 [-5343273458688,  2094742859491,   467642153205,  5934014125908,
    741852808966, -4562356494190,  1823039549615,   934097259328,
  -2087710423337,  -903735244984,  -945273930686],
 [-7240969135778, -2639902878880,   849460636225,   414183625708,
   -726198001149, -2021521159959,  2097041348792, -1405395811621,
  -3323825049514,  4169543904047,   535356746516],
 [-2450513019016,  1863099990948, -2437818958899, -1649975566093,
    913002479189, -2735144788138, -3826123267412, -4408375983513,
    804673891768,   752627625256,  6185218403530],
 [-1335399479000,  3823343667766, -5717460499457, -4005933034396,
   1194120926048,   434347561966,  5544143288165,    32763093488,
   -153238809945,  -626002138389, -1446843381290],
 [ 3384149124219, -3775174772430,  -505726264120,  -482507757887,
  -2778435182795, -5352559874474,  4355591684268,  1038732361168,
    127898700843, -1750769851358,  3782290455385],
 [ 3090835195085, -3054792832893, -1170619376721,   128325298949,
   2636523183051, -2257212225518,  -784444777154, -2644511696348,
  -2635072545182, -2717234652626, -6763588745282],
 [-3729763407548,  2337035244907,  -846339999856, -4541567007554,
   1974270756041,  2750510194087,   636060278562,  4678194935433,
  -2434295636669,  1169387614202, -4270669080080],
 [ -182352232618,   -15639990365,  -806361709654,  3736501768658,
   4423778632034, -1993974011012, -4511280703919, -1620258902598,
   1407442349625,  2801596965688, -5385383608572]], dtype=np.int64)

In [ ]:
#@title Verification
def compute_squared_norm(point: list[int]) -> int:
  """Returns the squared norm of an integer vector using exact computation."""
  return sum(pow(int(x), 2) for x in point)


def verify_sphere_packing(sphere_centers: np.ndarray):
  """Checks that after normalizing, the points correspond to a valid sphere packing for kissing numbers.

  Args:
    sphere_centers: the list of sphere centers, of shape [num_spheres, dimension].

  Raises:
    AssertionError: if the sphere packing is not a valid kissing configuration.
  """
  # Rounding to integers to guarantee exact computation throughout.
  sphere_centers = np.around(sphere_centers).astype(np.int64)
  squared_norms = [compute_squared_norm(list(center)) for center in sphere_centers]

  # Checks that the set doesn't contain 0.
  min_squared_norm = min(squared_norms)
  assert min_squared_norm > 1e-6, f'Verification failed because the set contains 0.'

  # Checks that the minimum pairwise distance between centers >= the maximum norm of the centers.
  max_squared_norm = max(squared_norms)
  min_squared_distance = min(compute_squared_norm(list(a - b)) for a, b in itertools.combinations(sphere_centers, 2))
  assert min_squared_distance >= max_squared_norm, f'Verification failed because the minimum squared distance = {min_squared_distance} < {max_squared_norm} = maximum squared norm.'


num_spheres = sphere_centers.shape[0]
dimension = sphere_centers.shape[1]

verify_sphere_packing(sphere_centers)
print(f'Verified the sphere packing showing kissing number in dimension {dimension} is at least {num_spheres}.')

# B.12. Packing circles inside a unit square to maximize sum of radii

Given a positive integer $n$, the problem is to pack $n$ disjoint circles inside a unit square so as to maximize the sum of their radii. The state of the art can be found on [Erich Friedman's homepage](https://erich-friedman.github.io/packing/cirRsqu/).

* For $n=26$, the SOTA was $2.634$, and AlphaEvolve improved it to $2.635$ (Construction 1).
* For $n=32$, the SOTA was $2.936$, and AlphaEvolve improved it to $2.937$ (Construction 2).

The constructions found by AlphaEvolve are shown below.

In [ ]:
#@title Verification function
import numpy as np
import itertools

def verify_circles(circles: np.ndarray):
  """Checks that the circles are disjoint and lie inside a unit square.

    Args:
      circles: A numpy array of shape (num_circles, 3), where each row is
        of the form (x, y, radius), specifying a circle.

    Raises:
      AssertionError if the circles are not disjoint or do not lie inside the
      unit square.
  """
  # Check pairwise disjointness.
  for circle1, circle2 in itertools.combinations(circles, 2):
    center_distance = np.sqrt((circle1[0] - circle2[0])**2 + (circle1[1] - circle2[1])**2)
    radii_sum = circle1[2] + circle2[2]
    assert center_distance >= radii_sum, f"Circles are NOT disjoint: {circle1} and {circle2}."

  # Check all circles lie inside the unit square [0,1]x[0,1].
  for circle in circles:
    assert (0 <= min(circle[0], circle[1]) - circle[2] and max(circle[0],circle[1]) + circle[2] <= 1), f"Circle {circle} is NOT fully inside the unit square."

In [ ]:
#@title Visualization function
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def plot_circles(circles: np.ndarray):
  """Plots the circles."""
  _, ax = plt.subplots(1, figsize=(7, 7))
  ax.set_xlim(0, 1)
  ax.set_ylim(0, 1)
  ax.set_aspect('equal')  # Make axes scaled equally.

  # Draw unit square boundary.
  rect = patches.Rectangle((0, 0), 1, 1, linewidth=1, edgecolor='black', facecolor='none')
  ax.add_patch(rect)

  # Draw the circles.
  for circle in circles:
    circ = patches.Circle((circle[0], circle[1]), circle[2], edgecolor='blue', facecolor='skyblue', alpha=0.5)
    ax.add_patch(circ)

  plt.title(f'A collection of {len(circles)} disjoint circles packed inside a unit square to maximize the sum of radii')
  plt.show()

In [ ]:
#@title Construction 1: Data

construction_1 = np.array([[0.68180423, 0.90401948, 0.09598051040194801], [0.90598057, 0.49972596, 0.094019420598057], [0.08464495, 0.08464502, 0.084644941535505], [0.4818405, 0.1034156, 0.10341558965844], [0.48234279, 0.89652324, 0.10347674965232401], [0.88928295, 0.11071646, 0.11071644892835401], [0.07852351, 0.50033803, 0.07852350214764901], [0.27330428, 0.1051224, 0.10512238948776001], [0.38153556, 0.70211016, 0.115517618448237], [0.13252625, 0.70386646, 0.132047406795258], [0.59610341, 0.72720176, 0.10051153994884501], [0.59576502, 0.2725969, 0.10054788994521], [0.88902242, 0.88902318, 0.11097680890231801], [0.68087256, 0.09573004, 0.095730030426996], [0.53098174, 0.49996661, 0.13587084641291403], [0.90742967, 0.68631006, 0.09257029074297], [0.74197859, 0.40386139, 0.095943250405674], [0.08471273, 0.91528735, 0.08471263152873601], [0.76300349, 0.7598366, 0.069494603050539], [0.27358888, 0.89472069, 0.105279299472069], [0.13242993, 0.29639184, 0.132426596757339], [0.90745313, 0.31316575, 0.092546580745341], [0.76284098, 0.23948486, 0.06975189302481001], [0.38095848, 0.29797266, 0.115741008425898], [0.74207757, 0.59573774, 0.09593309040669], [0.27141024, 0.50032568, 0.114361818563817]])

In [ ]:
#@title Construction 1: Verification
print(f"Construction 1 has {len(construction_1)} circles.")
verify_circles(construction_1)
print(f"The circles are disjoint and lie inside the unit square.")
sum_radii = np.sum(construction_1[:, 2])
print(f"Construction 1 sum of radii: {sum_radii}")

In [ ]:
#@title Construction 1: Visualization
plot_circles(construction_1)

In [ ]:
#@title Construction 2: Data

construction_2 = np.array([[0.09076163, 0.40381803, 0.090761620923837], [0.07310993, 0.92689178, 0.07310821268917801], [0.08745017, 0.22570576, 0.087381421261857], [0.24855246, 0.30880277, 0.093428060657193], [0.4079865, 0.06300614, 0.063006133699386], [0.47646318, 0.90136179, 0.09863820013617901], [0.89604966, 0.10309934, 0.10309932969006601], [0.9066386, 0.68096117, 0.09336139066386], [0.08962002, 0.76509474, 0.0895289910471], [0.06973669, 0.06965159, 0.06965158303484101], [0.40979823, 0.21756451, 0.09156283084371601], [0.25742466, 0.88393887, 0.11606111839388701], [0.09064689, 0.58506214, 0.090482500951749], [0.90294698, 0.30231577, 0.09623644037635501], [0.57265603, 0.10585396, 0.105853949414604], [0.74007588, 0.40129314, 0.09435083056491601], [0.57539962, 0.71183255, 0.115160168483982], [0.7367635, 0.21592191, 0.09104997089500201], [0.41096972, 0.40263617, 0.093512520648747], [0.88664452, 0.88667032, 0.113317128668286], [0.57582722, 0.49961748, 0.09705531029446801], [0.24962585, 0.49417195, 0.09194421080557799], [0.90546338, 0.49309632, 0.094507120549287], [0.67381348, 0.90149423, 0.09850576014942301], [0.24310147, 0.1077195, 0.10771948922805], [0.40815297, 0.5886157, 0.09248833075116601], [0.24737889, 0.6771266, 0.090994980900501], [0.75801377, 0.7532924, 0.07192969280703], [0.73526642, 0.06243992, 0.062439303756069], [0.57415412, 0.30715219, 0.095403150459684], [0.39239379, 0.75259664, 0.07223814277618501], [0.7439361, 0.58879735, 0.093166630683336]])

In [ ]:
#@title Construction 2: Verification
print(f"Construction 2 has {len(construction_2)} circles.")
verify_circles(construction_2)
print(f"The circles are disjoint and lie inside the unit square.")
sum_radii = np.sum(construction_2[:, 2])
print(f"Construction 2 sum of radii: {sum_radii}")

In [ ]:
#@title Construction 2: Visualization
plot_circles(construction_2)

# B.13. Packing circles inside a rectangle of perimeter 4 to maximize sum of radii

Given a positive integer $n$, the problem is to pack $n$ disjoint circles inside a rectangle of perimeter 4 so as to maximize the sum of their radii. The state of the art can be found on [Erich Friedman's homepage](https://erich-friedman.github.io/packing/cirRrec/).

For $n=21$, the SOTA was $2.364$, and AlphaEvolve improved it to $2.365$. The construction found by AlphaEvolve is shown below.

In [ ]:
#@title Data
import numpy as np

circles = np.array([[0.12396161, 0.12396161, 0.12396159760383901], [0.0747415, 0.31647214, 0.07474149252585001], [0.12740019, 0.51163445, 0.127400177259981], [0.0747415, 0.70679675, 0.07474149252585001], [0.12396161, 0.89930729, 0.12396159760383901], [0.36284141, 0.11508312, 0.115083108491688], [0.25916864, 0.31452627, 0.10969588903041], [0.37244262, 0.51163445, 0.117642228235776], [0.25916864, 0.70874263, 0.10969588903041], [0.3628414, 0.90818577, 0.11508311849168701], [0.59083599, 0.11292169, 0.11292167870783101], [0.48083152, 0.30914434, 0.112032298796769], [0.62891008, 0.51163445, 0.138825206117478], [0.48083152, 0.71412455, 0.112032298796769], [0.59083598, 0.9103472, 0.11292168870783001], [0.67852787, 0.2919303, 0.086412051358794], [0.85766825, 0.39257159, 0.11906284809371401], [0.85766825, 0.6306973, 0.11906284809371401], [0.67852787, 0.73133859, 0.086412051358794], [0.83965981, 0.8861976, 0.13707128629287002], [0.83965981, 0.1370713, 0.13707128629287002]])

In [ ]:
#@title Verification
import itertools

def minimum_circumscribing_rectangle(circles: np.ndarray) -> tuple[float, float]:
  """Returns the width and height of the minimum circumscribing rectangle.

  Args:
    circles: A numpy array of shape (num_circles, 3), where each row is of the
      form (x, y, radius), specifying a circle.

  Returns:
    A tuple (width, height) of the minimum circumscribing rectangle.
  """
  min_x = np.min(circles[:, 0] - circles[:, 2])
  max_x = np.max(circles[:, 0] + circles[:, 2])
  min_y = np.min(circles[:, 1] - circles[:, 2])
  max_y = np.max(circles[:, 1] + circles[:, 2])
  return max_x - min_x, max_y - min_y


def verify_circles_disjoint(circles: np.ndarray):
  """Checks that circles are disjoint.

  Args:
    circles: A numpy array of shape (num_circles, 3), where each row is of the
      form (x, y, radius), specifying a circle.

  Raises:
    AssertionError: if the circles are not disjoint.
  """
  for circle1, circle2 in itertools.combinations(circles, 2):
    center_distance = np.sqrt((circle1[0] - circle2[0])**2 + (circle1[1] - circle2[1])**2)
    radii_sum = circle1[2] + circle2[2]
    assert center_distance >= radii_sum, f"Circles are NOT disjoint: {circle1} and {circle2}."


num_circles = len(circles)
verify_circles_disjoint(circles)
print(f"Construction has {num_circles} disjoint circles.")
width, height = minimum_circumscribing_rectangle(circles)
print(f"Perimeter of minimum circumscribing rectangle: {2 * (width + height):.6f}")
assert (width + height) <= 2, "The circles cannot be contained within a rectangle with perimeter 4."
sum_radii = np.sum(circles[:, 2])
print('Sum of radii:', sum_radii)

In [ ]:
#@title Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def plot_circles_rectangle(circles: np.ndarray):
  width, height = minimum_circumscribing_rectangle(circles)
  _, ax = plt.subplots(1, figsize=(7, 7))
  ax.set_xlim(0, width)
  ax.set_ylim(0, height)
  ax.set_aspect('equal') # Make axes scaled equally.

  # Draw rectangle boundary.
  rect = patches.Rectangle((0, 0), width, height, linewidth=1, edgecolor='black', facecolor='none')
  ax.add_patch(rect)

  # Draw the circles.
  for circle in circles:
    circ = patches.Circle((circle[0], circle[1]), circle[2], edgecolor='blue', facecolor='skyblue', alpha=0.5)
    ax.add_patch(circ)

  plt.title(f"{len(circles)} disjoint circles packed inside a rectangle of perimeter {round(2 * width + 2 * height,6)}")
  plt.show()

plot_circles_rectangle(circles)